{/* cspell:ignore sqrtm cytotoxicity  Nalm Cytotoxicity binarize Utro Filippo Hsin */}

# Enhancing feature classification using projected quantum kernels

*Usage estimate: 80 minutes on a Heron r3 processor (NOTE: This is an estimate only. Your runtime might vary.)*

In this tutorial, we demonstrate how to run a [Projected Quantum Kernel](https://www.nature.com/articles/s41467-021-22539-9) (PQK) with Qiskit on a real-world biological dataset, based on paper "[Enhanced Prediction of CAR T-Cell Cytotoxicity with Quantum-Kernel Methods](#references)
". PQK is a method used in quantum machine learning (QML) to transform classical data by encoding them into a quantum feature space and projecting them back into the classical domain, enabling the use of quantum computers for enhancing feature selection. It involves encoding classical data into quantum states using a quantum circuit, typically through a process called feature mapping, where the data is transformed into a high-dimensional Hilbert space. The "projected" aspect refers to extracting classical information from the quantum states, by measuring specific observables, to construct a kernel matrix that can be used in classical kernel-based algorithms like support vector machines. This approach leverages the computational advantages of quantum systems to potentially achieve better performance on certain tasks compared to classical methods.

This tutorial also assumes general familiarity with QML methods. For further exploration of QML, readers can refer to [this](https://quantum.cloud.ibm.com/learning/en/courses/quantum-machine-learning) course.

### Requirements
Before starting this tutorial, ensure that you have the following installed:

- Qiskit SDK 2.0 or later with visualization support (`pip install 'qiskit[visualization]'`)
- Qiskit Runtime 0.40 or later (`pip install qiskit-ibm-runtime`)
- Category encoders 2.8.1 (`pip install category-encoders`)
- NumPy 2.3.2 (`pip install numpy`)
- Pandas 2.3.2 (`pip install pandas`)
- Scikit-learn 1.7.1 (`pip install scikit-learn`)
- Tqdm 4.67.1 (`pip install tqdm`)

### Setup

In [1]:
import warnings

# Standard libraries
import os
import numpy as np
import pandas as pd

# Machine learning and data processing
import category_encoders as ce
from scipy.linalg import inv, sqrtm
from sklearn.metrics.pairwise import rbf_kernel
from sklearn.model_selection import GridSearchCV, StratifiedKFold
from sklearn.svm import SVC

# Qiskit and Qiskit Runtime
from qiskit import QuantumCircuit
from qiskit.circuit import ParameterVector
from qiskit.circuit.library import UnitaryGate, ZZFeatureMap
from qiskit.quantum_info import SparsePauliOp, random_unitary
from qiskit.transpiler import generate_preset_pass_manager
from qiskit_ibm_runtime import (
    Batch,
    EstimatorOptions,
    EstimatorV2 as Estimator,
    QiskitRuntimeService,
)

# Progress bar
import tqdm

warnings.filterwarnings("ignore")

## Step 1: Map classical inputs to a quantum problem

### Dataset preparation

In this tutorial, we have a real-world biological dataset for a binary classification task, which is generated by Daniels et al. (2022) and can be downloaded from its [supplementary material](#references). The data consists of CAR T-cells, which are genetically engineered T-cells used in immunotherapy to treat certain cancers. T-cells, a type of immune cell, are modified in a lab to express chimeric antigen receptors (CARs) that target specific proteins on cancer cells. These modified T-cells can recognize and destroy cancer cells more effectively. The data features are the CAR T-cell motifs, which refer to the specific structural or functional component of the CAR engineered into T-cells. Based on these motifs, our task is to predict the cytotoxicity of a given CAR T-cell, labeling it as either toxic or non-toxic.

Below, we have the helper functions to preprocess this dataset.

In [2]:
def preprocess_data(dir_root, args):
    """
    Preprocess the training and test data.
    """
    # Read from the csv files
    train_data = pd.read_csv(
        os.path.join(dir_root, args["file_train_data"]),
        encoding="unicode_escape",
        sep=",",
    )
    test_data = pd.read_csv(
        os.path.join(dir_root, args["file_test_data"]),
        encoding="unicode_escape",
        sep=",",
    )

    # Fix the last motif ID
    train_data[train_data == 17] = 14
    train_data.columns = [
        "Cell Number",
        "motif",
        "motif.1",
        "motif.2",
        "motif.3",
        "motif.4",
        "Nalm 6 Cytotoxicity",
    ]
    test_data[test_data == 17] = 14
    test_data.columns = [
        "Cell Number",
        "motif",
        "motif.1",
        "motif.2",
        "motif.3",
        "motif.4",
        "Nalm 6 Cytotoxicity",
    ]

    # Adjust motif at the third position
    if args["filter_for_spacer_motif_third_position"]:
        train_data = train_data[
            (train_data["motif.2"] == 14) | (train_data["motif.2"] == 0)
        ]
        test_data = test_data[
            (test_data["motif.2"] == 14) | (test_data["motif.2"] == 0)
        ]

    train_data = train_data[
        args["motifs_to_use"] + [args["label_name"], "Cell Number"]
    ]
    test_data = test_data[
        args["motifs_to_use"] + [args["label_name"], "Cell Number"]
    ]

    # Adjust motif at the last position
    if not args["allow_spacer_motif_last_position"]:
        last_motif = args["motifs_to_use"][len(args["motifs_to_use"]) - 1]
        train_data = train_data[
            (train_data[last_motif] != 14) & (train_data[last_motif] != 0)
        ]
        test_data = test_data[
            (test_data[last_motif] != 14) & (test_data[last_motif] != 0)
        ]

    # Get the labels
    train_labels = np.array(train_data[args["label_name"]])
    test_labels = np.array(test_data[args["label_name"]])

    # For the classification task use the threshold to binarize labels
    train_labels[train_labels > args["label_binarization_threshold"]] = 1
    train_labels[train_labels < 1] = args["min_label_value"]
    test_labels[test_labels > args["label_binarization_threshold"]] = 1
    test_labels[test_labels < 1] = args["min_label_value"]

    # Reduce data to just the motifs of interest
    train_data = train_data[args["motifs_to_use"]]
    test_data = test_data[args["motifs_to_use"]]

    # Get the class and motif counts
    min_class = np.min(np.unique(np.concatenate([train_data, test_data])))
    max_class = np.max(np.unique(np.concatenate([train_data, test_data])))

    num_class = max_class - min_class + 1
    num_motifs = len(args["motifs_to_use"])
    print(str(max_class) + ":" + str(min_class) + ":" + str(num_class))

    train_data = train_data - min_class
    test_data = test_data - min_class

    return (
        train_data,
        test_data,
        train_labels,
        test_labels,
        num_class,
        num_motifs,
    )


def data_encoder(args, train_data, test_data, num_class, num_motifs):
    """
    Use one-hot or binary encoding for classical data representation.
    """
    if args["encoder"] == "one-hot":
        # Transform to one-hot encoding
        train_data = np.eye(num_class)[train_data]
        test_data = np.eye(num_class)[test_data]

        train_data = train_data.reshape(
            train_data.shape[0], train_data.shape[1] * train_data.shape[2]
        )
        test_data = test_data.reshape(
            test_data.shape[0], test_data.shape[1] * test_data.shape[2]
        )

    elif args["encoder"] == "binary":
        # Transform to binary encoding
        encoder = ce.BinaryEncoder()

        base_array = np.unique(np.concatenate([train_data, test_data]))
        base = pd.DataFrame(base_array).astype("category")
        base.columns = ["motif"]
        for motif_name in args["motifs_to_use"][1:]:
            base[motif_name] = base.loc[:, "motif"]
        encoder.fit(base)

        train_data = encoder.transform(train_data.astype("category"))
        test_data = encoder.transform(test_data.astype("category"))

        train_data = np.reshape(
            train_data.values, (train_data.shape[0], num_motifs, -1)
        )
        test_data = np.reshape(
            test_data.values, (test_data.shape[0], num_motifs, -1)
        )

        train_data = train_data.reshape(
            train_data.shape[0], train_data.shape[1] * train_data.shape[2]
        )
        test_data = test_data.reshape(
            test_data.shape[0], test_data.shape[1] * test_data.shape[2]
        )

    else:
        raise ValueError("Invalid encoding type.")

    return train_data, test_data

Below, we include the training and test dataset. To run this notebook locally, you should copy and paste these text in two files named `train_data.csv` and `test_data.cvs`.

`train_data.csv`

Cell Number,motif,motif.1,motif.2,motif.3,motif.4,Nalm 6 Cytotoxicity
2746,12,9,10,14,0,0.207063911
4208,3,6,12,14,0,0.73986604
928,6,8,11,14,0,0.854397557
2128,12,12,11,14,0,0.63470872
418,1,6,2,14,0,0.975838764
2780,8,6,3,14,0,0.761623248
11301,1,7,10,14,0,0.266907573
4827,7,1,6,14,0,0.845628147
4552,9,1,1,14,0,0.342823448
6717,10,10,9,14,0,0.2915224
7504,10,7,9,14,0,0.188935424
499,3,9,8,14,0,0.847729781
5152,7,1,12,14,0,0.503772358
1574,3,8,10,14,0,0.704743041
1388,10,10,7,10,14,0.379791046
5755,8,11,2,14,0,0.493475861
8209,7,8,9,14,0,0.752974796
4866,8,4,11,14,0,0.487140719
776,11,5,11,14,0,0.949500295
2002,8,12,7,14,0,0.666384433
12561,6,6,11,14,0,0.850300125
8837,6,11,11,14,0,0.476632547
2393,6,9,3,14,0,0.748484253
3820,9,7,2,14,0,0.257351941
5237,3,8,7,14,0,0.582636568
3593,7,1,6,14,0,0.871225752
5353,12,3,2,14,0,0.607145557
1495,11,10,10,14,0,0.855894404
6757,10,7,2,14,0,0.392718366
3136,1,10,2,14,0,0.836541224
321,11,12,11,14,0,1.005654757
4321,11,9,2,14,0,0.447043348
4006,12,6,5,14,0,0.830795748
15065,1,8,7,14,0,0.856952781
2552,11,6,12,14,0,0.443611183
2040,10,10,9,14,0,0.209649375
1568,11,10,6,14,0,0.479338968
2303,8,2,11,7,14,0.593598331
442,2,7,8,14,0,0.866765449
1118,1,2,11,14,0,0.886284945
928,12,12,8,14,0,0.7213898
2227,8,8,3,14,0,0.776243971
9618,10,10,10,14,0,0.661334462
1491,1,13,14,0,0,0.686478477
3554,11,11,11,6,14,0.460378899
6623,10,14,0,0,0,0.37731142
3365,2,8,3,14,0,0.751084836
5371,6,8,8,3,14,0.764375028
5331,10,1,12,14,0,0.439604469
1947,1,1,10,14,0,0.31179788
4760,8,2,5,14,0,0.959781672
1484,10,8,8,14,0,0.802023012
2090,10,9,9,14,0,0.330606752
4278,11,11,9,10,14,0.093197659
2814,12,11,3,14,0,0.565551339
2392,13,6,11,14,0,0.735012625
954,9,9,9,14,0,0.331030103
1429,3,6,6,14,0,0.805107425
4063,13,2,8,14,0,0.519587535
3685,10,6,3,14,0,0.528674458
2647,2,2,8,14,0,0.777408186
1310,8,8,8,14,0,0.891954822
13379,3,8,11,14,0,0.633045556
6243,11,10,7,14,0,0.11206701
2050,11,5,11,14,0,0.876547876
2964,6,1,13,14,0,0.470554438
6955,10,9,2,14,0,0.281770211
8259,6,3,12,14,0,0.59825519
5832,1,3,12,14,0,0.514537565
157,2,11,13,2,14,1.114304722
4439,12,1,2,14,0,0.459003009
6837,5,6,12,14,0,0.750737084
1569,10,8,10,14,0,0.789806317
10112,8,6,1,14,0,0.762636266
585,8,9,3,14,0,0.742511982
1256,6,8,3,14,0,0.797335914
4742,11,13,3,6,14,0.772312856
7224,8,11,11,14,0,0.720754774
1685,5,14,0,0,0,0.808993181
2079,11,10,1,14,0,0.004702218
5713,10,1,7,14,0,0.403287017
4804,9,2,1,14,0,0.275692103
3205,9,1,9,14,0,0.330032205
2237,1,12,8,14,0,0.687763649
816,2,13,11,14,0,0.84546183
2903,1,1,1,14,0,0.485810188
11395,8,6,9,11,14,0.657342869
541,10,7,1,14,0,0.42803792
7528,10,8,6,14,0,0.595337093
4481,6,8,8,14,0,0.542085608
1263,8,6,2,14,0,0.85332406
9603,13,2,6,14,0,0.588669318
6571,9,11,11,14,0,0.394714163
2191,1,2,2,14,0,0.492901314
3205,8,12,8,14,0,0.520676152
3606,6,11,8,14,0,0.481667397
4053,8,3,13,14,0,0.689411694
4326,8,12,7,14,0,0.57057107
1145,7,12,9,14,0,0.84780538
4596,2,9,9,14,0,0.501882399
1595,11,10,9,14,0,0.112308925
546,2,2,8,14,0,0.933896793
4376,11,12,3,9,14,0.826592479
1252,3,11,12,14,0,0.820877243
3204,10,2,10,14,0,0.776138133
2857,12,5,6,14,0,0.831309817
2849,11,6,6,14,0,0.873554181
7043,2,8,11,14,0,0.751946658
1026,11,8,4,14,0,0.683363825
668,12,9,11,14,0,0.858116996
1586,10,7,7,14,0,0.772721087
628,10,6,11,14,0,0.820983081
4325,10,9,1,14,0,0.163594853
15015,9,10,10,14,0,0.443913576
2779,4,8,3,14,0,0.756346482
1197,10,10,1,14,0,0.050363628
9999,12,12,9,14,0,0.479384327
1931,1,1,9,14,0,0.430895538
1585,11,1,7,14,0,0.534510652
2748,8,3,2,14,0,0.708795113
1031,5,3,3,14,0,0.860022075
12473,9,9,7,14,0,0.284461513
1686,4,9,1,14,0,0.00231331
4618,11,8,2,14,0,0.482317543
271,7,11,11,14,0,0.962352016
516,8,8,6,14,0,0.965753942
2194,1,1,10,14,0,0.366198461
3494,12,13,2,14,0,0.481969791
410,10,1,2,14,0,0.288120474
9076,8,3,12,14,0,0.535947021
1510,1,1,1,2,14,0.44559186
1593,7,6,6,14,0,0.987722826
78,1,8,6,14,0,1.001799241
2019,9,2,7,14,0,0.410196707
1191,6,9,11,14,0,0.74030451
150,11,6,2,14,0,1.197447799
4133,10,7,14,0,0,0.314428703
10767,5,6,6,14,0,0.855576891
4001,11,8,6,14,0,0.60286669
1020,12,8,8,14,0,0.77074041
2560,12,3,4,14,0,0.468528402
1884,12,5,8,14,0,0.782851268
2842,8,7,3,14,0,0.55214019
586,9,1,9,14,0,0.867899424
7195,2,2,11,14,0,0.719756876
10006,5,3,6,14,0,0.780326283
11650,13,10,9,14,0,0.30119899
4302,1,10,1,14,0,0.346815041
7382,9,1,7,14,0,0.654485251
2782,12,9,2,14,0,0.303648377
55,10,8,8,14,0,1.072241794
828,1,12,6,14,0,0.984668652
2914,2,8,7,14,0,0.467832897
4814,12,11,2,14,0,0.670920334
2497,2,8,6,14,0,0.710957226
2049,8,8,3,14,0,0.786540468
4303,1,14,0,0,0,0.440541889
5281,11,8,5,14,0,0.866206021
3625,1,7,9,14,0,0.456341947
4078,8,9,2,14,0,0.473548133
1641,1,13,3,14,0,0.600976731
718,9,10,10,14,0,0.410861973
2618,1,5,3,14,0,0.945297026
8242,12,14,0,0,0,0.430759461
8159,1,3,11,14,0,0.6632849
1118,1,12,6,14,0,0.906061477
1561,13,13,14,0,0,0.435491919
6207,3,6,6,14,0,0.817898668
1098,12,10,1,14,0,0.119596607
1099,2,11,6,14,0,0.833895281
1400,12,2,1,14,0,0.308486672
9012,2,3,6,14,0,0.759128502

`test_data.csv`

Cell Number,motif,motif.1,motif.2,motif.3,motif.4,Nalm 6 Cytotoxicity
2738,1,1,8,14,0,0.875202226
6328,12,9,10,14,0,0.269674473
5624,11,8,6,14,0,0.791892832
873,6,9,8,14,0,0.896006895
4446,12,9,14,0,0,0.27120156
3161,6,8,8,3,14,0.744553138
6335,10,1,7,14,0,0.423925369
7326,3,8,12,14,0,0.635978772
3952,8,6,8,14,0,0.749391433
4961,10,10,9,14,0,0.372442885
3949,10,9,9,14,0,0.110842317
4266,10,4,9,14,0,0.40266711
3709,6,11,8,14,0,0.508610653
51,2,8,7,14,0,1.089054869
540,11,6,2,14,0,1.070276236
10156,8,2,12,14,0,0.730522082
2814,8,2,11,14,0,0.691059738
5799,1,9,14,0,0,0.395500386
1862,11,11,1,14,0,0.240236472
8695,1,3,6,14,0,0.808463993
2334,4,1,14,0,0,0.200063503
4037,12,3,12,14,0,0.489620345
768,9,1,14,0,0,0.335762561
836,12,2,14,0,0,0.401215622
648,11,8,3,14,0,0.880252196
14594,5,9,3,14,0,0.576845734
2601,8,6,11,14,0,0.788248991
135,6,11,12,14,0,1.102133386
5903,10,9,7,14,0,0.094951541
3875,2,11,7,14,0,0.821421552
10566,5,1,13,14,0,0.797260315
2070,12,5,8,14,0,0.792739533
519,10,1,1,14,0,0.04842831
4045,5,1,13,14,0,0.794327099
2817,10,10,14,0,0,0.40684014
608,4,1,12,14,0,0.664056003
4348,8,13,2,14,0,0.503288529
2935,5,11,2,14,0,0.50644854
3420,12,4,11,14,0,0.631095118
2341,2,11,2,14,0,0.455434766
3405,13,2,5,14,0,0.910340344
2886,10,6,6,14,0,0.833532409
11758,11,9,10,14,0,0.375088828
2150,2,14,0,0,0,0.551459804
1301,13,2,6,14,0,0.934924931
13385,1,10,2,14,0,0.834303512
510,2,13,12,14,0,1.0129122
111,2,2,8,14,0,1.052707177
2885,4,4,14,0,0,0.350035531
2704,6,2,2,14,0,0.839126688
3280,6,6,11,14,0,0.771420796
1485,1,12,11,14,0,0.970879511
5474,12,10,10,14,0,0.48893996
194,1,13,6,14,0,1.090113246
3645,2,11,13,14,0,0.211100863
2722,1,3,11,14,0,0.693509125
3234,10,2,14,0,0,0.388651174
2508,1,13,11,14,0,0.539076793
6623,11,9,2,14,0,0.353921287
4348,5,6,11,14,0,0.846127096
10679,8,2,12,14,0,0.508081465
3180,9,2,10,14,0,0.254086091
3992,3,1,1,14,0,0.384463025
4900,9,12,14,0,0,0.432604061
3561,12,3,8,14,0,0.684815313
3385,7,7,3,14,0,0.781747532
7785,8,3,11,14,0,0.823508066
1635,11,6,3,1,14,0.762560668
2091,3,1,14,0,0,0.401593613
968,2,3,5,14,0,0.968808116
4262,12,1,14,0,0,0.049547166
3196,12,8,12,14,0,0.517501021
2026,3,6,8,14,0,0.687582213
351,3,8,6,3,14,0.997792528

In [ ]:
args = {
    "file_train_data": "train_data.csv",
    "file_test_data": "test_data.csv",
    "motifs_to_use": ["motif", "motif.1", "motif.2", "motif.3"],
    "label_name": "Nalm 6 Cytotoxicity",
    "label_binarization_threshold": 0.62,
    "filter_for_spacer_motif_third_position": False,
    "allow_spacer_motif_last_position": True,
    "min_label_value": -1,
    "encoder": "one-hot",
}
dir_root = "./"

# Preprocess data
train_data, test_data, train_labels, test_labels, num_class, num_motifs = (
    preprocess_data(dir_root=dir_root, args=args)
)

# Encode the data
train_data, test_data = data_encoder(
    args, train_data, test_data, num_class, num_motifs
)

14:0:15


We also transform the dataset such that $1$ is represented as $\pi/2$ for scaling purposes.

In [4]:
# Change 1 to pi/2
angle = np.pi / 2

tmp = pd.DataFrame(train_data).astype("float64")
tmp[tmp == 1] = angle
train_data = tmp.values

tmp = pd.DataFrame(test_data).astype("float64")
tmp[tmp == 1] = angle
test_data = tmp.values

Let's verify sizes and shapes of the training and test datasets.

In [5]:
print(train_data.shape, train_labels.shape)
print(test_data.shape, test_labels.shape)

(172, 60) (172,)
(74, 60) (74,)


## Step 2: Optimize problem for quantum hardware execution

### Quantum circuit

We now construct the feature map that embeds our classical dataset into a higher dimensional feature space. For this embedding, we use the ``ZZFeatureMap`` from Qiskit.

In [6]:
feature_dimension = train_data.shape[1]
reps = 24
insert_barriers = True
entanglement = "pairwise"

# ZZFeatureMap with linear entanglement and a repetition of 2
embed = ZZFeatureMap(
    feature_dimension=feature_dimension,
    reps=reps,
    entanglement=entanglement,
    insert_barriers=insert_barriers,
    name="ZZFeatureMap",
)
embed.decompose().draw(output="mpl", style="iqp", fold=-1)

<Image src="/docs/images/tutorials/projected-quantum-kernels/extracted-outputs/45956df4-5472-4394-a3e1-5514c456791d-0.avif" alt="Output of the previous code cell" />

Another quantum embedding one can use is the 1D-Heisenberg Hamiltonian evolution ansatz. You can skip running this section if you would like to continue with the `ZZFeatureMap`.

In [7]:
feature_dimension = train_data.shape[1]
num_qubits = feature_dimension + 1
embed2 = QuantumCircuit(num_qubits)
num_trotter_steps = 6
pv_length = feature_dimension * num_trotter_steps
pv = ParameterVector("theta", pv_length)

# Add Haar random single qubit unitary to each qubit as initial state
np.random.seed(42)
seeds_unitary = np.random.randint(0, 100, num_qubits)
for i in range(num_qubits):
    rand_gate = UnitaryGate(random_unitary(2, seed=seeds_unitary[i]))
    embed2.append(rand_gate, [i])


def trotter_circ(feature_dimension, num_trotter_steps):
    num_qubits = feature_dimension + 1
    circ = QuantumCircuit(num_qubits)
    # Even
    for i in range(0, feature_dimension, 2):
        circ.rzz(2 * pv[i] / num_trotter_steps, i, i + 1)
    for i in range(0, feature_dimension, 2):
        circ.rxx(2 * pv[i] / num_trotter_steps, i, i + 1)
    for i in range(0, feature_dimension, 2):
        circ.ryy(2 * pv[i] / num_trotter_steps, i, i + 1)
    # Odd
    for i in range(1, feature_dimension, 2):
        circ.rzz(2 * pv[i] / num_trotter_steps, i, i + 1)
    for i in range(1, feature_dimension, 2):
        circ.rxx(2 * pv[i] / num_trotter_steps, i, i + 1)
    for i in range(1, feature_dimension, 2):
        circ.ryy(2 * pv[i] / num_trotter_steps, i, i + 1)
    return circ


# Hamiltonian evolution ansatz
for step in range(num_trotter_steps):
    circ = trotter_circ(feature_dimension, num_trotter_steps)
    if step % 2 == 0:
        embed2 = embed2.compose(circ)
    else:
        reverse_circ = circ.reverse_ops()
        embed2 = embed2.compose(reverse_circ)


embed2.draw(output="mpl", style="iqp", fold=-1)

<Image src="/docs/images/tutorials/projected-quantum-kernels/extracted-outputs/659dbf23-fd3f-4e01-94b4-33e6d672172c-0.avif" alt="Output of the previous code cell" />

## Step 3: Execute using Qiskit primitives

### Measure 1-RDMs

The main building blocks of projected quantum kernels are the reduced density matrices (RDMs), which are obtained though projective measurements of the quantum feature map. In this step, we obtain all single qubit reduced density matrices, i.e. 1-RDMs, which will later be provided into the classical exponential kernel function.

Let's look at how to compute the 1-RDM given a single data point from the dataset before we run over all data. The 1-RDMs are a collection of single qubit measurements of Pauli ``X``, ``Y`` and ``Z`` operators on all qubits. This is because a single-qubit RDM can be fully expressed as: $$\rho = \frac{1}{2} \big( I + \braket \sigma_x \sigma_x  + \braket \sigma_y \sigma_y + \braket \sigma_z \sigma_z  \big)$$

Now, let's select the backend to use.

In [ ]:
service = QiskitRuntimeService()
backend = service.least_busy(
    operational=True, simulator=False, min_num_qubits=133
)
target = backend.target

Then we can run the quantum circuit and measure the projections. Note that we turn on error mitigation including Zero Noise Extrapolation (ZNE).

In [10]:
# Let's select the ZZFeatureMap embedding for this example
qc = embed
num_qubits = feature_dimension

# Identity operator on all qubits
id = "I" * num_qubits

# Let's select the first training datapoint as an example
parameters = train_data[0]

# Bind parameter to the circuit and simplify it
qc_bound = qc.assign_parameters(parameters)
transpiler = generate_preset_pass_manager(
    optimization_level=3, basis_gates=["u3", "cz"]
)
transpiled_circuit = transpiler.run(qc_bound)

# Transpile for hardware
transpiler = generate_preset_pass_manager(optimization_level=3, target=target)
transpiled_circuit = transpiler.run(transpiled_circuit)

# We group all commuting observables
# These groups are the Pauli X, Y and Z operators on individual qubits
observables_x = [
    SparsePauliOp(id[:i] + "X" + id[(i + 1) :]).apply_layout(
        transpiled_circuit.layout
    )
    for i in range(num_qubits)
]
observables_y = [
    SparsePauliOp(id[:i] + "Y" + id[(i + 1) :]).apply_layout(
        transpiled_circuit.layout
    )
    for i in range(num_qubits)
]
observables_z = [
    SparsePauliOp(id[:i] + "Z" + id[(i + 1) :]).apply_layout(
        transpiled_circuit.layout
    )
    for i in range(num_qubits)
]

# We define the primitive unified blocs (PUBs) consisting of the embedding circuit,
# set of observables and the circuit parameters
pub_x = (transpiled_circuit, observables_x)
pub_y = (transpiled_circuit, observables_y)
pub_z = (transpiled_circuit, observables_z)

# Experiment options for error mitigation
num_randomizations = 300
shots_per_randomization = 100
noise_factors = [1, 3, 5]

experimental_opts = {}
experimental_opts["resilience"] = {
    "measure_mitigation": True,
    "zne_mitigation": True,
    "zne": {
        "noise_factors": noise_factors,
        "amplifier": "gate_folding",
        "return_all_extrapolated": True,
        "return_unextrapolated": True,
        "extrapolated_noise_factors": [0] + noise_factors,
    },
}
experimental_opts["twirling"] = {
    "num_randomizations": num_randomizations,
    "shots_per_randomization": shots_per_randomization,
    "strategy": "active-accum",
}
options = EstimatorOptions(experimental=experimental_opts)

# We define and run the estimator to obtain <X>, <Y> and <Z> on all qubits
estimator = Estimator(mode=backend, options=options)

job = estimator.run([pub_x, pub_y, pub_z])

We can retrieve the results.

In [11]:
job_result_x = job.result()[0].data.evs
job_result_y = job.result()[1].data.evs
job_result_z = job.result()[2].data.evs

In [12]:
print(job_result_x)
print(job_result_y)
print(job_result_z)

[ 3.67865951e-03  1.01158571e-02 -3.95790878e-02  6.33984326e-03
  1.86035759e-02 -2.91533268e-02 -1.06374793e-01  4.48873518e-18
  4.70201764e-02  3.53997968e-02  2.53130819e-02  3.23903401e-02
  6.06327843e-03  1.16313667e-02 -1.12387504e-02 -3.18457725e-02
 -4.16445718e-04 -1.45609602e-03 -4.21737114e-01  2.83705669e-02
  6.91332890e-03 -7.45363001e-02 -1.20139326e-02 -8.85566135e-02
 -3.22648394e-02 -3.24228074e-02  6.20431299e-04  3.04225434e-03
  5.72795792e-03  1.11288428e-02  1.50395861e-01  9.18380197e-02
  1.02553163e-01  2.98312847e-02 -3.30298912e-01 -1.13979648e-01
  4.49159340e-03  8.63861493e-02  3.05666566e-02  2.21463145e-04
  1.45946735e-02  8.54537275e-03 -8.09805979e-02 -2.92608104e-02
 -3.91243644e-02 -3.96632760e-02 -1.41187613e-01 -1.07363243e-01
  1.81089440e-02  2.70778895e-02  1.45139414e-02  2.99480458e-02
  4.99137134e-02  7.08789852e-02  4.30565759e-02  8.71287156e-02
  1.04334798e-01  7.72191962e-02  7.10059720e-02  1.04650403e-01]
[-7.31765102e-05  7.4266

We can print out the circuit size and the two-qubit gate depth.

In [13]:
print(f"qubits: {qc.num_qubits}")
print(
    f"2q-depth: {transpiled_circuit.depth(lambda x: x.operation.num_qubits==2)}"
)
print(
    f"2q-size: {transpiled_circuit.size(lambda x: x.operation.num_qubits==2)}"
)
print(f"Operator counts: {transpiled_circuit.count_ops()}")
transpiled_circuit.draw("mpl", fold=-1, style="clifford", idle_wires=False)

qubits: 60
2q-depth: 64
2q-size: 1888
Operator counts: OrderedDict({'rz': 6016, 'sx': 4576, 'cz': 1888, 'x': 896, 'barrier': 31})


<Image src="/docs/images/tutorials/projected-quantum-kernels/extracted-outputs/4f573436-ec5c-451b-976c-ad718b3c201d-1.avif" alt="Output of the previous code cell" />

We can now loop over the entire training dataset to obtain all 1-RDMs.

We also provide the results from an experiment that we ran on quantum hardware. You can either run the training yourself by setting the flag below to `True` or use the projection results that we provide.

In [ ]:
# Set this to True if you want to run the training on hardware
run_experiment = False

In [ ]:
# Identity operator on all qubits
id = "I" * num_qubits

# projections_train[i][j][k] will be the expectation value of the j-th Pauli operator (0: X, 1: Y, 2: Z)
# of datapoint i on qubit k
projections_train = []
jobs_train = []

# Experiment options for error mitigation
num_randomizations = 300
shots_per_randomization = 100
noise_factors = [1, 3, 5]

experimental_opts = {}
experimental_opts["resilience"] = {
    "measure_mitigation": True,
    "zne_mitigation": True,
    "zne": {
        "noise_factors": noise_factors,
        "amplifier": "gate_folding",
        "return_all_extrapolated": True,
        "return_unextrapolated": True,
        "extrapolated_noise_factors": [0] + noise_factors,
    },
}
experimental_opts["twirling"] = {
    "num_randomizations": num_randomizations,
    "shots_per_randomization": shots_per_randomization,
    "strategy": "active-accum",
}
options = EstimatorOptions(experimental=experimental_opts)

if run_experiment:
    with Batch(backend=backend):
        for i in tqdm.tqdm(
            range(len(train_data)), desc="Training data progress"
        ):
            # Get training sample
            parameters = train_data[i]

            # Bind parameter to the circuit and simplify it
            qc_bound = qc.assign_parameters(parameters)
            transpiler = generate_preset_pass_manager(
                optimization_level=3, basis_gates=["u3", "cz"]
            )
            transpiled_circuit = transpiler.run(qc_bound)

            # Transpile for hardware
            transpiler = generate_preset_pass_manager(
                optimization_level=3, target=target
            )
            transpiled_circuit = transpiler.run(transpiled_circuit)

            # We group all commuting observables
            # These groups are the Pauli X, Y and Z operators on individual qubits
            observables_x = [
                SparsePauliOp(id[:i] + "X" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]
            observables_y = [
                SparsePauliOp(id[:i] + "Y" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]
            observables_z = [
                SparsePauliOp(id[:i] + "Z" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]

            # We define the primitive unified blocs (PUBs) consisting of the embedding circuit,
            # set of observables and the circuit parameters
            pub_x = (transpiled_circuit, observables_x)
            pub_y = (transpiled_circuit, observables_y)
            pub_z = (transpiled_circuit, observables_z)

            # We define and run the estimator to obtain <X>, <Y> and <Z> on all qubits
            estimator = Estimator(options=options)

            job = estimator.run([pub_x, pub_y, pub_z])
            jobs_train.append(job)

Training data progress: 100%|██████████| 172/172 [13:03<00:00,  4.55s/it]


Once the jobs are complete, we can retrieve the results.

In [ ]:
if run_experiment:
    for i in tqdm.tqdm(
        range(len(train_data)), desc="Retrieving training data results"
    ):
        # Completed job
        job = jobs_train[i]

        # Job results
        job_result_x = job.result()[0].data.evs
        job_result_y = job.result()[1].data.evs
        job_result_z = job.result()[2].data.evs

        # Record <X>, <Y> and <Z> on all qubits for the current datapoint
        projections_train.append([job_result_x, job_result_y, job_result_z])

Let's repeat this for the test set.

In [ ]:
# Identity operator on all qubits
id = "I" * num_qubits

# projections_test[i][j][k] will be the expectation value of the j-th Pauli operator (0: X, 1: Y, 2: Z)
# of datapoint i on qubit k
projections_test = []
jobs_test = []

# Experiment options for error mitigation
num_randomizations = 300
shots_per_randomization = 100
noise_factors = [1, 3, 5]

experimental_opts = {}
experimental_opts["resilience"] = {
    "measure_mitigation": True,
    "zne_mitigation": True,
    "zne": {
        "noise_factors": noise_factors,
        "amplifier": "gate_folding",
        "return_all_extrapolated": True,
        "return_unextrapolated": True,
        "extrapolated_noise_factors": [0] + noise_factors,
    },
}
experimental_opts["twirling"] = {
    "num_randomizations": num_randomizations,
    "shots_per_randomization": shots_per_randomization,
    "strategy": "active-accum",
}
options = EstimatorOptions(experimental=experimental_opts)

if run_experiment:
    with Batch(backend=backend):
        for i in tqdm.tqdm(range(len(test_data)), desc="Test data progress"):
            # Get test sample
            parameters = test_data[i]

            # Bind parameter to the circuit and simplify it
            qc_bound = qc.assign_parameters(parameters)
            transpiler = generate_preset_pass_manager(
                optimization_level=3, basis_gates=["u3", "cz"]
            )
            transpiled_circuit = transpiler.run(qc_bound)

            # Transpile for hardware
            transpiler = generate_preset_pass_manager(
                optimization_level=3, target=target
            )
            transpiled_circuit = transpiler.run(transpiled_circuit)

            # We group all commuting observables
            # These groups are the Pauli X, Y and Z operators on individual qubits
            observables_x = [
                SparsePauliOp(id[:i] + "X" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]
            observables_y = [
                SparsePauliOp(id[:i] + "Y" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]
            observables_z = [
                SparsePauliOp(id[:i] + "Z" + id[(i + 1) :]).apply_layout(
                    transpiled_circuit.layout
                )
                for i in range(num_qubits)
            ]

            # We define the primitive unified blocs (PUBs) consisting of the embedding circuit,
            # set of observables and the circuit parameters
            pub_x = (transpiled_circuit, observables_x)
            pub_y = (transpiled_circuit, observables_y)
            pub_z = (transpiled_circuit, observables_z)

            # We define and run the estimator to obtain <X>, <Y> and <Z> on all qubits
            estimator = Estimator(options=options)

            job = estimator.run([pub_x, pub_y, pub_z])
            jobs_test.append(job)

Test data progress: 100%|██████████| 74/74 [00:13<00:00,  5.56it/s]


We can retrieve the results as before.

In [ ]:
if run_experiment:
    for i in tqdm.tqdm(
        range(len(test_data)), desc="Retrieving test data results"
    ):
        # Completed job
        job = jobs_test[i]

        # Job results
        job_result_x = job.result()[0].data.evs
        job_result_y = job.result()[1].data.evs
        job_result_z = job.result()[2].data.evs

        # Record <X>, <Y> and <Z> on all qubits for the current datapoint
        projections_test.append([job_result_x, job_result_y, job_result_z])

## Step 4: Post-process and return result in desired classical format

### Define the projected quantum kernel

The projected quantum kernel is defined with the following kernel function: $$k^{\textrm{PQ}}(x_i, x_j) = \textrm{exp} \Big(-\gamma \sum_k \sum_{P \in \{ X,Y,Z \}} (\textrm{Tr}[P \rho_k(x_i)] - \textrm{Tr}[P \rho_k(x_j)])^2 \Big) $$
In the above equation, $\gamma>0$ is a tunable hyperparameter. The $K^{\textrm{PQ}}_{ij} = k^{\textrm{PQ}}(x_i, x_j)$ are the entries of the kernel matrix $K^{\textrm{PQ}}$.

Using the definition of 1-RDMs, we can see that the individual terms within the kernel function can be evaluated as $\textrm{Tr}[P \rho_k (x_i)] = \braket P$, where $P \in \{ X,Y,Z \}$. These expectation values are precisely what we measured above.

By using ``scikit-learn``, we can in fact compute the kernel even more easily. This is due to the readily available radial basis function (``'rbf'``) kernel: $ \textrm{exp} (-\gamma \lVert x - x' \rVert^2)$. First, we simply need to reshape the our new projected training and test datasets into 2-dimensional arrays.

Note that going over the entire dataset can take about 80 minutes on the QPU. To make sure that the rest of the tutorial is easily executable, we additionally provide projections from a previously run experiment. If you performed the training yourself, you can continue the tutorial with your own results.

To execute this notebook locally using the previously run projections, make sure to copy and paste the cells below to two files named `projections_train.txt` and `projections_test.txt`.

`projections_train.txt`

1.403929591348540562e-01 -7.277816299492935459e-02 9.653661158433118894e-02 1.921305499255184651e-02 1.004025792877845727e-02 1.569281405547618899e-02 7.972552503466631690e-02 5.774463365809491718e-03 2.808418701438102766e-02 2.785662254715886438e-02 4.109813205247288964e-02 1.060266502897170074e-02 1.293138503678901985e-02 1.946037913400459549e-02 1.704866835589005994e-02 4.635164067913108421e-02 5.433354477022537404e-02 7.203889952674757437e-03 -8.453949130668270790e-04 -4.590568825906439498e-02 -1.304903002144001262e-02 5.811933105920491158e-03 2.918123889593431108e-02 2.420109900569243459e-03 1.144615968772471183e-04 2.822057236594389257e-02 1.828759553446966390e-01 6.174311549254437106e-03 1.222859431128524646e-02 1.246812893506650749e-03 3.172507262105119918e-03 3.510188229278836392e-05 5.272692272876878042e-03 2.146726983037508529e-02 -1.152697630583070093e-01 -8.947737424922846960e-02 -4.903065168212548258e-02 2.555148082820116184e-02 2.634436398553383421e-02 1.649949947690113397e-03 4.326327677674947011e-03 2.019156574039181543e-02 2.162374444108188312e-02 1.369469689507485551e-02 2.304492036493604665e-02 2.266780081915786837e-02 -3.127145960808257186e-02 -6.423860634908289124e-03 -5.604985451393971509e-04 7.609437030669569113e-02 5.238968501872995343e-02 4.601512229811214640e-03 2.943098636771194002e-02 2.717562058690902588e-02 5.047402773117908720e-02 4.241409952619404344e-02 4.711557345004536296e-02 3.281860577450070732e-02 7.002922056704255493e-02 5.368629735823259724e-02 -9.899451989043835110e-02 1.674834274730679223e-03 2.831602715347042790e-02 -1.177427585435506241e-01 2.391122534055619211e-02 -9.473155297697780086e-03 -1.532828697887606288e-04 -2.629277626187437582e-03 7.618836610399482012e-02 3.132694340320044005e-05 1.605077529483659718e-03 3.582951953494436376e-02 2.108793730551048601e-02 -9.771405114153454419e-03 2.707659065771169521e-02 1.122885831420637574e-02 -2.410244167185156120e-18 -8.786366669916660654e-02 -5.211847724697814266e-02 8.123158554249835378e-03 -6.978352329926236497e-02 -8.225224307961400902e-03 7.437376241243973465e-03 -9.184609267461158952e-03 5.907494204047761439e-03 1.318605840864590234e-03 1.180826372754294543e-02 -1.861140458855902596e-02 4.413156241189925000e-02 -3.198408277828728660e-18 -3.357148516275348243e-02 3.482646802115204471e-02 1.052797155805928683e-02 -2.349710068974367183e-02 1.618034944746333864e-03 5.981291193762646961e-17 4.191725514502340350e-03 5.687290592212839585e-02 1.411740528282936146e-03 -2.312027950442217469e-03 9.138300484704098281e-03 9.672105766589550457e-02 -2.206210525348260149e-02 1.580874835018944435e-02 3.104256049609925572e-02 2.406073284809702190e-03 4.286107006406115333e-02 -6.424565120755527936e-02 5.565943247117278309e-02 1.646639874258835207e-02 -2.815902847637373127e-02 -1.155359985450560636e-02 5.094914748940457261e-03 -4.400718579432476990e-02 2.421667136185114144e-02 1.029744820373743909e-02 2.712373321604625784e-02 -2.528370836437930039e-03 -4.654456591557695988e-05 -8.005316146321697635e-02 6.070606795696502633e-01 4.572427282870890619e-01 2.711544996429621560e-02 1.425368190386011764e-02 1.941759158162933291e-02 9.149338376205240486e-03 4.517876404720171796e-02 1.134961319405397573e-02 2.426195671051325706e-02 1.840920331340773625e-02 3.769998742702759115e-02 7.112638738189725612e-03 3.310389572507962258e-02 7.709991632876000978e-03 2.655467381507664593e-02 2.326016839837773131e-02 5.818427513471260970e-02 6.290878529948337565e-03 4.237470555922052684e-01 7.002172659557580614e-01 5.824668092522079510e-01 1.314317126787915846e-03 3.950995297749009677e-02 3.121645345171774125e-02 2.280672477335801696e-02 4.919908038820833968e-02 6.700575308615475123e-02 1.971500721262033073e-02 1.789592439482286246e-02 1.225320230970400586e-03 8.787341064107723199e-03 5.991376087073673438e-03 1.038443744705792871e-02 1.308576156716387577e-03 3.504855966309550763e-01 1.448756070177854349e+00 5.708419851832898795e-01 4.958119224300195499e-02 1.296447290999708409e-02 2.038310298832562290e-02 5.540411216726532501e-02 4.400089156721286032e-02 3.522498495615011133e-02 9.271842815366427779e-03 2.966489212778438550e-02 2.543920055056524479e-02 5.579485326467098494e-01 9.342889273837434727e-01 4.852944401565320587e-01 1.857238774902697986e-03 1.803675150480804623e-02 -1.490116119452305915e-08 1.221439066427085739e-02 3.707187848159572419e-02 6.175154684778239728e-02 4.622241305050732441e-02 1.162835651985844598e-01 6.600022214097454876e-02 7.601569822704663804e-02 7.388083645749017603e-02
2.552547016897047105e-01 2.086851816337888937e-02 9.514939824061841370e-03 3.124262006518753573e-04 2.717693475521702908e-05 1.974857072159632315e-05 1.712373234396939465e-03 2.797309811709904363e-04 3.941215035474968206e-05 1.089341691068978699e-02 2.555641835052486696e-02 5.041587134261555150e-03 1.246680558106101969e-02 2.370775254547389035e-02 3.113451284640965031e-02 2.565410166420270099e-03 -3.387938050075045299e-03 5.225957759391844371e-03 -2.220991335906515561e-03 5.237367140349268035e-02 7.063142039712766440e-02 2.907038849821974927e-02 3.754843459940217689e-17 2.488222414062643204e-02 6.908876876459005278e-02 2.929603834951484062e-02 3.497940456862300151e-02 5.387188564758704085e-02 5.056490466569792813e-02 4.145359565281548503e-02 9.961492473920273427e-02 8.629657704937139517e-02 1.157232160944046823e-01 6.800465700509701616e-02 7.008422438685575628e-02 2.654803067754224477e-02 1.031126269972357234e-02 6.781719285826184884e-02 -2.888655336173459945e-02 5.742822588785103863e-04 1.811102895296785517e-02 1.214277764045897968e-02 2.268465509844318218e-02 2.025756300105325169e-04 1.766836185650700122e-02 2.069830646133370006e-02 3.654508559273882129e-02 7.159594052334514354e-03 3.038627352401806689e-02 2.466160733908387068e-02 2.664138469959912228e-02 3.970827970284051254e-04 1.297857806437985101e-03 3.821244578130998377e-05 1.826988263256206640e-02 1.370082999511382954e-04 -3.909772215347544905e-02 -4.859764923858679586e-02 4.667480330006607558e-01 1.067247610925311369e-01 -2.352295910934593670e-01 -5.542382130229270287e-03 -4.852626003551894357e-02 7.090785699591049818e-02 -9.709967819662972816e-04 8.497934755422014829e-04 4.321067153113093001e-03 2.943729035021808507e-02 4.770753148120202708e-03 1.028046068208071824e-02 1.689513876390985603e-02 -1.456382987696843712e-01 5.581121533533295320e-03 3.724215446470191451e-03 2.423790055512623290e-02 -9.697519561343171529e-04 2.969058397715651845e-02 3.516879223585567770e-17 2.042498801365590783e-02 9.142171126358877506e-03 6.580098917085026996e-04 2.639557604700600198e-03 1.243327035589953007e-02 -3.173663817117853420e-02 2.161403600782044535e-02 -6.200447750111502416e-04 -2.465683299032773532e-18 1.467871760836003073e-03 -2.367603500043304748e-03 -1.805539487593241926e-02 2.797457227041089123e-02 1.171943410588548564e-03 -3.483770927540050808e-04 -1.763036768024566284e-02 -6.321545279148204459e-18 3.545837704202020949e-03 7.483851128405174863e-04 1.601288260291884591e-04 4.297768439150997238e-02 3.740523104257566928e-02 -8.571272101384775147e-03 4.925206944637995632e-02 -1.170071403605724321e-02 6.573212494587333046e-03 2.748566841187066040e-02 1.694030572855946288e-03 -1.439239541510668741e-02 -1.347620599867511461e-02 -3.183655388687114257e-02 -4.292492319711513522e-03 -1.116398332229361426e-03 -6.524869852263842814e-03 2.883244961888118936e-04 -2.398170689842493014e-02 2.948043848954121429e-03 1.896536810969471146e-02 6.177411189927217890e-03 2.829464161623648083e-02 1.289828163168698673e-02 1.894795195328071724e-01 5.944760200088252677e-01 3.774573952457764281e-01 3.948800092573060555e-02 1.581780602726424187e-03 8.758684474064280107e-03 4.898583672961052959e-03 1.159490974446227528e-02 2.238764840591212814e-03 1.603589427469691298e-02 1.058842205590385371e-03 4.838099638957159970e-02 1.377198928888980108e-02 4.109416393366625148e-02 1.057831770385630164e-02 1.835692876352987279e-02 8.892569055611830586e-04 4.591814170558210506e-01 1.030097284209412489e+00 4.461336290796529269e-01 1.997668197833755641e-02 2.928524918824532836e-02 1.664444346641456504e-02 1.416315431815884200e-02 3.208703921895998679e-02 4.304556851711326820e-02 1.531484777788580734e-02 6.407604295844139541e-02 3.145351280868428218e-02 2.632401906618300064e-02 8.092731381421741643e-02 4.838970101155310594e-02 4.706641057845411397e-02 6.581111433185860005e-02 4.920347324993823435e-02 3.336987923115796589e-02 2.536084305066633061e-03 9.995188373116756578e-03 6.179371143114854581e-01 7.024908028323626485e-01 6.963443510392294256e-01 1.067296111674707892e-02 2.862976364147910499e-02 2.966901508167704704e-02 1.621742783023539256e-03 3.092054369304157030e-02 3.262262452791946099e-02 3.984371817154179485e-02 1.196235917910523291e-02 1.693543035508508071e-02 1.594551802365941004e-02 2.677919200370127220e-02 -1.248833361625384039e-02 2.143975448635848310e-02 6.154115326984995436e-03 5.040128695091559280e-02 3.850349710602343767e-01 9.727324688085310644e-02 2.757761573399649113e-01 -4.017489049441662075e-01 -7.052685210226903312e-01
1.351485251573215640e-01 -6.157646116391772562e-02 2.294376859263208557e-02 2.705243328873047429e-02 1.462506872051924992e-02 2.061219004077761291e-02 5.430291253432020793e-02 1.708680431284893830e-02 2.053716482568349044e-02 1.889320393409275878e-02 4.420649105442616550e-02 1.220738441717072499e-02 1.930799220765327603e-02 1.735181314385528945e-02 3.048259823700286489e-02 2.058640005687971397e-02 2.921300624356659020e-02 -1.872559475885117800e-02 5.346166189721050094e-02 8.859985257726422578e-04 3.923340708394808078e-04 1.278415278433353390e-02 3.065076474342245602e-02 1.586879065433902461e-02 1.564837916221028039e-02 1.190758619088622822e-02 8.948138790717506674e-02 1.755494375579116570e-02 4.514901705968570045e-03 2.310139014778506226e-03 9.088696764809796064e-04 5.768831948364789270e-14 3.148642016604343442e-03 7.860394531129863763e-03 2.735390740836958837e-02 -7.409836472867786794e-01 -3.848335553807518683e-01 -8.213500470663842590e-02 4.131074229627629268e-02 4.022659772334724604e-02 5.244109977878685241e-02 3.903521591411662772e-02 3.362305927186296645e-02 5.464296692593485598e-03 3.746351238486572699e-02 7.313857774911119991e-02 1.436095762584981006e-01 3.566930487055344073e-02 2.039438849832282685e-02 3.824567982142850625e-02 3.564304274685201057e-02 5.927219913752882234e-04 -9.336252220005643657e-03 -4.008397934370615950e-02 -5.261682940001043834e-02 2.049138006605865370e-01 8.661065850512539299e-02 2.114391261794957566e-01 3.331375116879470299e-01 4.316032043360366610e-01 -8.668621108510721207e-02 9.571555677358909234e-03 -2.762849379494107874e-02 6.607882923604325651e-03 2.627766676834864326e-02 -3.588844538661353778e-02 -2.787799235811544863e-03 4.026344083244549205e-02 4.154549532590672521e-03 6.726327384884721317e-02 -8.756444343098905569e-03 -1.004181672638767199e-07 2.958932854817144072e-05 1.440066662420735888e-02 4.810741701502127543e-03 5.228535150588207309e-02 -1.855705082167529579e-02 5.292525436278795736e-02 9.299456010375327375e-02 6.884896218934548340e-02 1.290616441333975514e-03 1.668605978344837523e-02 5.850187158502558049e-02 -4.199875843112194085e-02 -6.870769849738542212e-04 -1.220182660302182553e-17 5.594442625139990806e-03 1.459998639999186831e-02 -9.061723043724875592e-18 1.956528906831913120e-02 -1.456406110372547059e-02 2.447222171576475015e-03 -5.120870200562434240e-18 6.042450842470626499e-02 -8.663847614368053521e-03 -2.976561880565166320e-03 -1.387001332528346731e-02 -2.578116840974054025e-02 1.852870584914429949e-02 -1.053729229709234917e-02 4.045305788295668233e-03 -8.158915189005388688e-03 7.454602519720764965e-03 -1.286024198650256541e-04 -2.505864774597420935e-02 -5.734707922794178838e-17 -1.041397288586670689e-02 3.664778465400973795e-02 -8.724712378923921152e-02 1.490678938776577971e-02 -2.376750065645280726e-04 1.557360693607468590e-03 -2.465685451403750589e-02 -1.318810283985081579e-02 -1.676634025970448125e-02 4.912959672248207871e-02 5.490294257296001595e-02 8.205823096499306052e-03 4.570375387699884828e-02 1.333343672611113882e-02 6.122040440077254342e-01 3.231915510802249036e-01 6.063076747142821799e-02 3.208436804059896369e-02 2.520218772529667431e-02 5.581690958276888659e-02 6.131425771567799304e-02 1.113232857626321112e-02 2.765437882381899942e-02 1.881729703207203619e-02 3.802699397091041655e-02 2.653657455285329814e-02 2.590953097216540726e-02 5.219426539744670066e-03 2.324461906758107030e-02 2.250394593446784081e-02 6.470586319418562826e-02 4.847356786508508208e-01 4.934690491268505785e-01 5.648437590443797784e-01 5.190057645255671120e-02 4.942864544899086728e-03 3.979778700513309647e-02 4.205303588845982210e-02 4.719992517198438875e-02 4.807281658579937861e-02 9.773039100487691810e-02 6.463707367230414928e-05 1.951210651516283034e-02 5.316776082936936453e-03 1.467656984366511215e-02 1.144960503687036946e-03 1.122970469557816285e-02 1.449256779896841707e-02 2.954182252522105645e-02 9.534615901620815404e-01 8.499570123252673248e-01 7.235418367560223496e-01 4.168010207089320462e-02 1.711647587977465149e-02 5.696985943274906344e-02 4.198272244205943776e-02 4.552617015981170329e-02 8.220928178653351404e-03 3.128629433040272523e-02 2.460921269710435524e-02 4.650479765690879175e-02 1.861030777742330583e-02 1.637761149537378391e-02 5.983395246591351364e-03 2.552946387946536735e-02 6.035559888774313801e-17 7.500655774702934542e-01 3.172097262552610597e-01 2.355217586909485739e-01 2.081353428293638771e-02 1.036492150816435004e-01 1.046442533456463375e-01 3.248045716169711133e-02 1.802004015997000358e-02
1.257579504889927979e-01 -6.712032752501469191e-02 2.693440471723142215e-02 1.702858496183388623e-02 1.866695517675699689e-02 3.531949979831053993e-02 5.291463681810643777e-02 1.279673979701512199e-02 1.831321992702597198e-02 2.553561812721986610e-02 1.036493772019352888e-01 1.911793397069092362e-02 2.111066958038421149e-02 5.388378175526938735e-02 3.940474190981696490e-02 2.297584907377840657e-02 3.497906021273074900e-02 -1.836445548582383083e-02 5.234387652801895247e-02 6.614301812472882756e-03 6.513645313015156701e-02 1.766231617242513402e-02 1.539361302559343579e-02 1.453756042081537012e-02 5.697814397080160770e-02 4.600159067652060535e-02 2.959755463030979694e-01 4.534506677221483012e-02 1.317253819611389348e-02 1.133795167899136550e-02 5.922975383214452993e-02 4.271131737690830876e-02 1.014069373285658299e-01 3.329669669499625562e-02 3.611276895119877161e-03 1.045005330652519580e-03 -4.912468044504566395e-02 1.418475028784561094e-03 1.575890758390937826e-02 4.717230730308909215e-02 2.308677991975357016e-02 2.442516480476676285e-02 2.049586303340860879e-02 1.119713021921220309e-03 1.037221307893897854e-02 2.186117134273739618e-02 -2.209320389854614250e-02 -3.911001206016632460e-03 -1.554418038046479801e-02 1.013145512417477634e-01 3.928660809400882914e-02 1.404638351603698133e-02 3.095807409667130847e-02 2.140947766567860219e-02 5.844270462309023723e-02 4.954883005957633896e-02 6.984343582712781995e-02 4.458569797748378194e-02 6.365264932198012870e-02 6.803879808539413376e-02 -8.530748219763031881e-02 1.374492520134917850e-02 -8.034971986770417240e-03 3.187777482221131997e-18 3.962452793835032072e-02 -3.480297542809366457e-02 -2.841724066539850243e-04 9.128246313729745548e-17 5.296462409166653729e-02 3.033261869777492334e-02 -1.152624830222828376e-14 8.056413961160168932e-04 1.186460655124352281e-02 -4.156415718226051767e-04 5.608095439740242057e-03 7.533290456422192527e-02 -7.618877221359598535e-18 6.441043548932579021e-02 9.992257069407416148e-02 1.092531092149551797e-01 1.303985921686147582e-02 -1.644602979848848803e-02 -1.602776748303540860e-02 -4.404928380942160770e-02 -2.038549110278117978e-03 -1.662495527086641639e-02 -4.690975618389286832e-03 4.870904995258260853e-05 -8.737287426785436944e-18 1.210780193479464063e-02 -9.476729631559535627e-02 5.636711547779087172e-02 4.456081866784224188e-02 -8.303446401074919329e-18 2.220493894468378304e-04 2.577614752002197612e-02 3.536545192449098637e-03 1.009498773873378441e-04 1.260841878718134967e-01 -2.532771400562409192e-17 2.806145510721215486e-02 1.180612102242440580e-03 2.317159877747354199e-02 6.015431246330525672e-02 5.806878677382942833e-03 5.254430500568879614e-02 -2.432241603890218304e-02 -2.956762605191698076e-02 -9.257585113204714732e-04 1.389705378928361192e-02 -1.699031598069609864e-17 -8.901596311330489675e-02 1.549626576884658574e-02 -2.228070856089296377e-17 4.375449384838997480e-03 4.981755865894431715e-03 3.321639373238863269e-02 -6.732859523568759941e-02 4.066132355201569626e-02 -2.767494850299330109e-02 5.963643965825399240e-01 3.904269516777161275e-01 4.494903589818748102e-02 3.562517077438726032e-02 2.652891151764837432e-02 4.222387241897429533e-02 7.874076006793560845e-02 7.889789954747858380e-03 2.089907287248176163e-02 2.597245605714604277e-02 3.762546558367948352e-02 7.168523207427304061e-03 2.457314576256097380e-02 1.938861893639736225e-02 2.687866280065818667e-02 3.311987859504707254e-02 7.390144500830499175e-02 5.616704621196689784e-01 5.028076125951712605e-01 5.290826084311441857e-01 6.753494849896179118e-03 1.602325687392941759e-03 3.071664482141912150e-02 6.947556434258819746e-02 3.261122578697555480e-02 7.539603125756015600e-02 9.185059279381373176e-02 5.526164083416410140e-03 3.560615457430175382e-03 3.122727389831024720e-03 1.083382183298624638e-02 6.582024949133473690e-01 7.523348368959220656e-01 7.551944162689149254e-01 2.784278751179456585e-02 8.529683059415942362e-03 6.257187385328300509e-03 3.284930910333858783e-02 2.708411808937902232e-02 4.076472511744617250e-02 3.768756647483347683e-02 3.002762325763469811e-02 3.378126522061136727e-02 1.010170113098050881e-02 2.051904712595530381e-02 4.360875224995630801e-03 5.335998551246399213e-01 8.501199149362292928e-01 4.242693807659453364e-01 2.617680654947955512e-03 2.007708114125989624e-02 -3.328407518827278689e-03 1.342599938729151926e-02 8.116630345784593345e-03 5.898302142431260248e-02 5.001250424057555183e-02 1.277651287244237766e-01 1.196778004660741107e-01 7.472360166544722571e-02 5.320800014192519911e-02
2.670436097044079915e-01 9.558521781979039192e-03 1.010344256036134654e-02 2.531665586839655538e-04 5.101359568669607686e-05 1.833234405073717616e-03 7.998401317881980973e-04 7.725866461174699844e-04 4.451855072659743028e-03 6.563468621283263804e-03 4.575933922154099692e-02 1.497514247054729936e-02 3.980332534217242979e-02 2.897748270950912045e-02 2.364201272472038137e-02 1.684888751588030711e-02 1.405337504615363636e-02 8.509556028969453290e-03 2.294633797711878587e-02 4.049464156591221775e-02 7.417241529364407282e-02 1.063399793148608885e-02 7.134488621609431057e-04 2.524458979271918674e-02 7.509910439180503272e-02 7.811673079165405015e-03 5.566472025632853565e-02 1.941333101329472677e-01 2.252169007218438393e-02 8.805175562678302370e-02 1.122650362060216012e-01 1.168857276189431832e-01 1.657166343102770278e-01 8.711837608119478438e-02 1.222257527853494452e-01 5.945721931265929988e-02 3.164121542312439000e-02 6.726974695060104759e-02 -3.454644794215059461e-02 6.831716289904867520e-04 9.083158839638135959e-03 1.784291601032662183e-02 1.868190050968879939e-02 1.853277083384969682e-03 1.555801212211316634e-02 1.306911588859598246e-02 9.549799715272491718e-02 1.451343043845765811e-02 3.323080767214737358e-02 3.928391821930433264e-02 2.878348164572294351e-02 2.792153504049397182e-03 1.058137143406903159e-02 2.580308491459251240e-03 2.464038803532215063e-02 3.502188248699380797e-03 4.174875423396574858e-02 2.789281164009593805e-02 9.307388594717731778e-02 1.055465441350423444e-01 -2.038605815124981957e-01 -1.606200827628870721e-03 -3.158498000427215763e-02 2.954298285768176477e-02 -4.156953782201311392e-17 9.625568462213001525e-03 -3.783838664759551964e-03 2.981474499729557665e-03 3.167594115932471227e-02 1.534569016208508357e-02 -3.979267823075312294e-02 -1.161595704324961884e-02 3.547452618596671353e-02 3.325697879661818068e-03 -2.513862533204649044e-16 3.797551013066286472e-03 2.634057807729876452e-03 1.969443141121481750e-02 9.155443224339752939e-02 1.109481011272437159e-02 -2.042357001890662209e-16 2.010835015497559539e-02 -1.903199310829715107e-04 1.237930192121526303e-03 -1.627318665518382381e-05 1.191202792643160896e-18 1.861549731183580539e-02 3.516762322105104061e-02 3.956208083097055399e-03 -1.639077308176756231e-02 7.624329750153997776e-02 -1.208899670884344170e-02 -1.300076751135046507e-16 -3.136058786446072055e-02 4.207365446822920046e-02 6.795238222674143333e-03 -1.443648224407650428e-02 2.411728325788769633e-04 2.856811067766425183e-02 6.498412958491336933e-02 -5.941161344606946809e-02 3.416172609550666844e-02 7.933175873372001231e-04 3.094856582040545270e-02 2.823340218878812174e-02 -1.475718679867458879e-13 -6.030956637704328223e-03 3.246165528962823010e-02 -2.701857695635138251e-02 -1.618921743729025359e-03 -2.536681916921067156e-17 -1.175544274600135320e-03 -1.207700178584983006e-02 3.526473202917463495e-02 7.944086070816061508e-04 2.056785069980435893e-02 3.341056494055288928e-02 1.101692226728074597e-01 1.724138823340917726e-01 3.707524272495964918e-02 5.608078640303962858e-01 3.628641517073657452e-01 4.544439506274552260e-02 -1.097341161014587204e-17 1.104161212307609718e-02 1.106004176004426342e-04 1.657957288610016275e-02 6.894134277048235594e-03 5.334809058822395991e-03 3.802593759869001867e-03 4.421637886653681515e-02 2.227438008883241699e-02 4.650575460011840967e-02 2.768989889780546637e-02 3.757886445253254964e-02 8.943580848042673864e-03 2.207335966794725726e-02 1.294089069879275329e-03 2.812239670922891607e-02 3.409187946645166056e-02 7.397679325356733282e-02 2.395599517901236125e-02 2.243845479260232062e-02 2.188819866291303456e-02 2.008606666905345464e-02 4.167191965587811742e-03 5.391817777447845961e-01 9.466036456340249927e-01 6.488635051997303504e-01 1.327077977181214882e-02 8.565876292371245315e-03 4.892202506738477230e-02 5.310324563322300906e-02 1.008761276331701512e-01 2.347314840971703731e-02 -4.764453672629118119e-02 6.302005969949724118e-03 5.435400019485827627e-01 6.903139539056291074e-01 6.739296453177391344e-01 1.459587260052478026e-02 2.155334463690133928e-02 2.857309076685890514e-02 8.732885316344174073e-03 3.564689538246402362e-02 5.424845783718420383e-02 5.253141714642108051e-02 8.790153612357796617e-03 1.363262939372144981e-02 3.188269077416238523e-02 2.937604134411432588e-02 -7.203842392037924274e-04 9.716313988902519627e-03 1.204394394171670775e-02 3.588306914701540340e-02 2.343468382837183422e-02 1.098269987349844296e-01 4.515115041874826485e-01 5.501244706697131504e-01 8.609437349991455601e-01
1.775548309906236077e-01 -7.158062056335280643e-02 7.049317613793826265e-02 3.613324092991018754e-02 1.881869734127124966e-02 7.566726790011447934e-02 6.595552580299642031e-02 1.499690281238678577e-02 1.735493756886858849e-02 4.731854015567464600e-02 6.727091439562497832e-02 5.237223578346806262e-03 8.388115673169467892e-03 1.426171574823516253e-02 2.217781492096443721e-02 3.067542921930177524e-02 5.126114865027965595e-02 2.245136748448439690e-02 2.933898614728565248e-03 2.744557358129446484e-02 6.243501263856113953e-02 3.220785196622289598e-02 3.443220435974364557e-02 2.070655060524502142e-02 2.889861160148773922e-02 4.750065823327885572e-02 5.285156822093852597e-02 4.101244665846982135e-02 2.984756345986226689e-02 1.839560807420970515e-02 2.413064237135046849e-02 1.613914982271695206e-02 9.625081271280412154e-04 9.647393761467993462e-03 5.699240491990465243e-03 4.217853433399744618e-03 1.029270312051502878e-02 2.428204162808928965e-02 4.944242725548916861e-02 6.500528838303433069e-18 3.401739039628701167e-02 5.475325000270871156e-02 7.552600995287836461e-02 1.533327468300245237e-02 3.249812248503722995e-02 7.606631507060308828e-02 8.788897255344152448e-02 2.440210431796426688e-02 2.558895224635129168e-02 2.791325953117905490e-02 -1.126995937315644278e-02 -4.288813089372717885e-18 -8.362446784792816612e-03 2.303159261560431062e-02 1.146478647177262417e-01 1.426674728117342250e-01 8.561851090550277876e-02 4.092000601125647674e-01 1.312515015397486551e-01 5.645621647662678322e-02 -1.090920764898895279e-01 1.630002790322597137e-02 -1.144347939307519164e-02 2.968335971313302733e-18 1.950562201316426506e-02 -2.302779300214726280e-02 -8.674610757334028739e-04 8.484000252089433802e-17 2.107178788883455958e-02 2.224201007663585694e-02 -9.439551978019791040e-03 1.102659908229680803e-02 7.720715081728034453e-03 3.159145413456986885e-03 -1.264507082034883011e-02 7.404051556527875555e-02 -7.444923310601291207e-03 5.665462176116603521e-03 -9.131096638121265902e-03 -9.060031090474216300e-03 -1.894490002810128517e-04 2.141638925540926761e-02 2.835519737604090193e-02 -1.703961438445653759e-02 2.659414505780681795e-02 4.632304565055929413e-03 3.790131879068335541e-02 -3.418158356151872814e-15 -1.120685698882514195e-16 2.351078303602693181e-02 1.505068787848037816e-04 1.794183901743167872e-02 2.557808562477784098e-02 4.797809089322121329e-02 3.707957819212365225e-02 3.288139634834385400e-02 4.654744624440163708e-02 2.423465573639958098e-03 2.498581507620369069e-02 1.545795878425477478e-02 2.940864834628614810e-02 -3.297863149101384200e-02 1.136405442805775261e-02 -4.263474815410669971e-02 -3.375551041092027249e-02 -1.501464356925366096e-02 3.574053283982345230e-02 2.391617445389003158e-02 -2.755760759753224202e-04 -1.364074690763313752e-02 -1.169956941026021983e-02 -2.732985330506370464e-02 -4.113244484555675373e-03 3.543295256955089276e-03 -2.184179564120456146e-03 2.868852893632480375e-03 9.818837052011274158e-03 3.740580900291808234e-07 5.297939259062206530e-02 -4.201083533868039260e-02 6.001728367947193332e-01 3.947697874344126001e-01 4.357757054678607772e-02 4.126376652947579687e-02 1.528190057774301246e-02 5.387937499093928273e-02 6.349390262284249076e-02 1.148175617170180997e-02 2.478394147528989802e-02 1.639728833030291988e-02 3.808505714164320960e-02 2.094733426831867779e-02 3.345285035890876307e-02 1.432841778099953542e-02 3.423838287210981962e-02 3.423896268077522564e-02 4.359609399616070496e-02 5.598421349879661828e-03 1.671371078635908169e-02 2.213837535075334137e-02 6.187891787629754003e-02 2.100131803641949388e-02 5.517220405036776798e-02 2.464714946327234077e-02 1.956903543404996809e-02 6.983654168702370102e-01 8.637629658026463320e-01 6.400039377128282636e-01 2.302910759651494513e-02 1.196886802627513801e-03 8.243880224997598982e-03 5.303554859635749123e-03 1.859854284970879901e-02 3.643472543146003464e-03 1.699508183846477199e-02 3.102258347213882311e-03 1.569267181047251380e-02 7.143034381500559249e-01 8.004607577122099249e-01 8.361348632204370634e-01 3.412392246343060737e-02 3.202104222449200788e-02 6.155201675617466561e-02 2.326768410816655239e-02 5.977064457105858014e-02 6.240287068505442902e-02 5.249461542353758836e-02 1.320540774785835050e-02 2.252487007628125107e-02 6.480238068957304420e-03 4.279833221810551147e-01 6.599156248717573670e-01 5.030416211217605404e-01 7.482385791118392401e-03 4.390032907991684580e-02 6.283656185088040691e-02 1.209974906390612059e-01 1.480721660822102541e-02 4.167073211228590479e-02 2.766550751511658371e-02
1.715492039638272237e-01 -4.572503422782935728e-02 7.285022812731989372e-02 2.168261940817797773e-02 2.356625823362837671e-02 4.829260980789473051e-02 6.475896316383297768e-02 4.494635211225703891e-02 3.910458613465006561e-02 2.763738948321672190e-02 3.685966375436086628e-02 2.306304460915482857e-02 2.331718607938837104e-02 3.174822216898173788e-02 4.681547290668681971e-02 3.055698874797680917e-02 5.943286822915606199e-02 1.651618447915037535e-02 -6.535052607139604612e-03 -4.346411862359910588e-02 -1.916280329334568364e-02 1.034772588197374436e-02 2.033670489572071693e-02 7.547958691507812394e-03 1.021257015694966748e-02 1.043707209016296253e-02 6.129810498238948135e-02 1.971763249813497654e-03 -9.224615314240831518e-03 1.745364939703418277e-04 2.472134445214597682e-04 -1.407792746092741305e-02 2.040231410417427029e-03 1.404819627406127128e-03 1.472032933095926074e-02 2.857353745788597521e-02 3.536193051005323551e-02 3.824609874657514813e-02 -3.912011692468076764e-03 2.876271850646376638e-02 2.629352099548697488e-02 3.336048501176631931e-02 3.400658619182101577e-02 3.816327851880710071e-03 3.416519396638052070e-02 6.334485115593346816e-02 6.962046238240626739e-02 5.972155206767807561e-03 1.447266135323402141e-02 3.468149540977436690e-02 4.522064750421084245e-02 4.347039767703802897e-03 1.622881708994940858e-02 1.161430440423429078e-02 5.384863912055638802e-02 1.198762692204759055e-02 5.903077713790396558e-02 3.618885423926096554e-04 7.720525979238837544e-02 2.223913947932733975e-02 -9.808752607170299898e-02 4.613651258530189090e-04 3.808993751649984716e-02 -3.311003171941882817e-02 -7.010851176093068965e-04 -5.470904838964157219e-03 3.896476669087892142e-02 -6.959046041909545477e-04 3.345644106925793615e-02 -1.649061804753605892e-15 -8.012402127636491719e-03 1.754525713674652890e-02 2.189612359180319670e-02 -5.121964852332822060e-03 1.842770350741699398e-02 7.128351058318264065e-03 -5.549694582862601915e-03 -4.283847881248181722e-02 -4.735021491523093429e-02 -4.474561900011535254e-02 -6.192661677642569140e-02 -2.250590338601538530e-03 6.173790363846168219e-04 2.772237431457601903e-02 9.372534282438360967e-03 -2.719213790255113269e-02 -5.446874196515543922e-15 -1.191910915957236639e-02 -2.860872434840499634e-17 -1.014234181258759013e-03 -1.120207473762601172e-02 2.043247831365613710e-03 3.883220787681278957e-02 7.052229810943133528e-04 -4.926101217022925459e-04 2.868434564922758487e-02 -9.391133425153444861e-03 -7.725380298716555405e-03 5.869340740395387335e-02 -2.866746349985327055e-02 4.218401503866996832e-03 -2.052538549902695880e-02 5.041879088119820274e-02 -2.344006522398559694e-02 1.193615280923799628e-02 3.847386434340453049e-02 -7.018337247493146996e-03 -9.331765331985448358e-03 -1.357989956446387439e-02 -2.986258700549176059e-04 4.837357412476200186e-02 -1.517834686622736295e-02 1.367412445680962546e-03 -6.670950515083405976e-06 1.462683027955641660e-02 4.812339813087755658e-03 5.209314347075836432e-02 1.394708053390976699e-01 1.918348301890293317e-01 4.371121008973729433e-02 5.728473664114188679e-01 3.666989805873087649e-01 3.295151449467990934e-02 7.609502392362348967e-03 1.829288677596884599e-02 2.521758096033287905e-02 6.630529301960207123e-02 1.240030787745851487e-02 2.041471847763565162e-02 5.102605174614708444e-02 3.161115191778035571e-02 2.504900546094565236e-02 3.521352680960689735e-02 2.100830177349597219e-02 3.440722041999019071e-02 3.466137264308818272e-02 5.605382149050849167e-02 1.932453842047089382e-03 4.154263174502678679e-01 7.372634275542652160e-01 5.544969191330894054e-01 9.487961411922513728e-04 3.753597981404366291e-02 3.484767698005038217e-02 3.389573405591094629e-02 3.647561798257437093e-02 9.044049807286783094e-02 1.013448933331173064e-02 6.452140913821278502e-03 1.953628468528948066e-03 1.013871570876643570e-02 3.987704448883229100e-03 1.539632666365120528e-02 4.026008401927106371e-03 1.853603424811988559e-02 2.521238630660561073e-04 5.571999160978303722e-01 9.376105762658538678e-01 6.162419661014014727e-01 3.278511011787489833e-03 4.491899036341556045e-02 2.948282579059808770e-02 3.817019235430846313e-02 1.148601072909607480e-02 3.266052351310151647e-02 4.341099792813094688e-02 4.311267939853236086e-02 5.322883204773286477e-03 1.783275941745855048e-02 2.201045211410176256e-02 2.707710245660734788e-02 -4.701817150382228000e-03 9.100436476731981958e-03 1.297581662674366851e-02 5.793957022797065204e-02 3.337879850501964535e-02 1.181617096837302844e-01 4.638791643253572516e-01 5.480388459348874974e-01 9.102257302937896144e-01
1.901524155432429697e-01 -5.082087765548466002e-02 6.875054090011226138e-02 1.648098560953767042e-02 2.747078162802663731e-02 1.764088375490823965e-02 5.727705028422572925e-02 8.430684180193317653e-03 1.445826937882584151e-02 4.816231584759706613e-02 5.702936948103731057e-02 1.709173381369566772e-02 1.849773566807893718e-02 3.927879134586521187e-02 2.742056610155339802e-02 2.438870094778848932e-02 7.380293604852901368e-02 5.052548892704448145e-03 1.008630099322907079e-17 1.792288775291097508e-02 7.099058548887864084e-02 1.722595893085141980e-02 7.537600039043688971e-02 3.085148503600074754e-01 5.048048040753407711e-02 5.688375019620618353e-03 1.978900609016509896e-02 1.318360245405556831e-02 7.766235387043768368e-04 5.946748995614458896e-03 1.083538787483091098e-02 1.618591793230604245e-02 7.640576328793520673e-04 4.747292108467132590e-04 9.396394137959288712e-03 6.703461997974072273e-03 3.984745649932078249e-04 8.690058195874992078e-03 1.780129590422591460e-02 7.525946973366025305e-03 1.344551307322568341e-02 1.213434079102468019e-02 -1.503357231238421077e-03 4.043645236684954153e-02 -2.048127024350115261e-02 3.774676847926510675e-02 1.535591446709293206e-01 7.518306469035758444e-02 3.080313702005706145e-02 4.214807966567205716e-02 8.125653457607526567e-03 3.362797503812013644e-02 4.262120146885536864e-02 3.609757427020899440e-02 2.989348276672104676e-02 5.769751631046481088e-02 5.637045777620918602e-02 2.609557751024055294e-02 4.660137416747932304e-02 5.186609723036471797e-05 -8.474708119521583150e-02 9.299643432978138167e-04 -1.659423860044250895e-02 -8.288037018059286540e-03 2.608525957836186487e-02 -4.379380913396963525e-03 -2.073116819640533922e-05 -2.029761992729630857e-02 3.104560500707823079e-02 7.715229203638613448e-03 -3.831952112053116366e-02 1.345856318918135612e-02 1.438936884951015930e-03 4.651095503558052950e-03 4.238549042096359520e-02 1.841849093752674985e-02 -3.002362023295012793e-03 2.289819689351349843e-02 -4.486749813551439009e-02 3.833982147347704018e-03 -2.343772448159539903e-03 6.127007672822397388e-02 7.476169565574668219e-02 -7.626386379947471844e-16 -7.298174203432639932e-15 2.223846992104647202e-02 -8.929243265784880515e-19 3.742951677592543797e-02 4.034364655571408970e-02 5.196088761153722091e-02 7.660650715282607201e-03 2.708305839962215841e-02 2.823741990173007227e-02 1.073806991585534408e-02 4.364632415643187311e-04 1.720524498402141098e-02 -5.791986580350788093e-18 6.228387410101584795e-03 4.951985317251944227e-03 -1.271770967084722911e-02 2.305074636935186549e-02 3.045945386152881795e-02 -4.249971881660771257e-03 -2.276385833098579399e-03 -2.074658079103571227e-15 3.939719011013748107e-01 -7.791750187647325521e-02 6.442752654299579995e-02 8.228459481051558430e-03 -1.739758525148251897e-02 9.966148817048454955e-02 2.565977657158542355e-02 6.941027471028220208e-02 3.266947119572110453e-02 1.933919577411733260e-17 1.511571995528675563e-02 2.880040642456357936e-02 7.530325514624579275e-02 -6.186657533511662720e-03 -5.049518879917915676e-03 5.955826267859302137e-01 3.679199809743647354e-01 3.493990055936201100e-02 1.246884163104177226e-02 2.200521908744044325e-02 3.366934566481998015e-02 5.548418152822015559e-02 2.473868412458948367e-13 2.608404576348058793e-02 2.439468145673382576e-02 3.334003600797737221e-02 1.090375201889282920e-02 2.052158952420161594e-02 1.310725303315361454e-02 3.566193497157250925e-02 3.992988287045511758e-02 7.740187814668694766e-02 1.105226677357896113e-02 2.127720792546924575e-02 1.641126296165790590e-03 3.095646771359228042e-02 1.006125775256717231e-02 5.835251644975365526e-01 8.695383127809510615e-01 7.036368397039145517e-01 2.296611693613980973e-03 5.660265740978306231e-02 1.706934265522175309e-02 1.271217656301425192e-02 3.745802880961582228e-03 1.017589859899339748e-02 4.047071925694776216e-03 5.948837065916005515e-03 1.823556196051758210e-03 6.342634300868624925e-03 7.150934869213852437e-04 3.346138105623770818e-03 4.316571052841303331e-03 1.550615718516095212e-02 1.219363578478503858e-02 3.850108723148937262e-02 1.850740289413616360e-02 4.951142065361470723e-01 7.458974552282244819e-01 4.387463869468390398e-01 5.634034344929899317e-03 4.982615899481210309e-02 1.709406425517773914e-02 2.381083540015177524e-02 7.635921379042606705e-02 1.795305874484594277e-02 5.385919749541034918e-01 6.408215444054159438e-01 2.506158917987457158e-01 9.072090652247856435e-02 1.625478827176696506e-01 1.680593066417357950e-01 1.516287745729577863e-01 1.991064493960936488e-01 3.693403072823265854e-01
1.769573230250146845e-01 -5.397545768724469617e-02 9.047567009466281340e-02 3.059146659617502026e-02 1.810451946981237945e-02 3.918102094347274827e-02 6.238618464450153278e-02 2.576988166968973478e-03 3.353976575888482914e-02 1.869788757993982170e-02 4.852416887302098136e-02 9.766932864653255983e-03 1.179713514940845809e-02 1.383986239933200114e-02 3.411483114996848726e-02 2.869668420911703974e-02 4.729468966164394189e-02 7.799525479588143798e-04 -4.063918096312580655e-04 2.827498946648350844e-02 1.114745302689840178e-01 2.972043332476685326e-02 2.848455107017111229e-02 2.210270368088010015e-02 6.967278238481943754e-03 4.994010326751043699e-02 1.848150931599671087e-01 -1.298102397748514825e-01 -8.170911313701256784e-02 -7.462341956662708886e-02 1.764296525149280448e-03 -1.433500886513930372e-02 8.853881770103120890e-04 7.778546799090841355e-04 1.934628188671109854e-03 6.083570752948500374e-03 -4.658427811846502231e-02 3.867961299166039219e-03 1.328615843361873050e-02 1.555145138965071403e-03 7.461654588982559629e-03 1.249554515189748270e-02 -2.047077246987070564e-03 -1.503934307373405331e-03 -4.722379306773604531e-04 2.195132355525463569e-02 9.089310548705725745e-02 3.251825404641750517e-02 4.543528539253569552e-03 -7.180379785936089618e-04 -3.960963470631724201e-03 1.106770970539718447e-03 2.110601210489106497e-02 2.255175454665905854e-02 7.557380215411929614e-02 8.144081232596804554e-02 1.073888250577609837e-01 9.959853596672860188e-02 1.259032801247449984e-01 5.596996259082868269e-02 -1.050909588376907622e-01 6.588776946947275115e-03 -2.996853721529551054e-02 2.584245922687602309e-04 8.663556294452869413e-03 -1.390311475770339769e-02 2.353712417497628648e-02 -1.889432144148332077e-03 2.881317868227002699e-02 -7.522473944828506238e-04 -4.828329281725624034e-02 4.695379107744297037e-02 4.131129341751966881e-03 2.271437268470226367e-02 6.044416591750768550e-03 1.426313350055403301e-02 -9.280219110274727859e-03 -2.258315131474249345e-03 -9.786527739213116959e-18 7.921963944759248012e-04 -5.704751921837818845e-04 -1.913312571194834326e-18 2.205123349762563509e-02 1.013850068701370108e-02 -7.137317280108254093e-03 1.761734112581848227e-02 -4.631635899106926330e-03 8.613541140528593876e-02 1.133527313621475636e-02 -5.317733286778443536e-02 6.595506885346083016e-03 1.459613962988601521e-01 9.718690497482336890e-03 4.854121115252488412e-03 8.299225947267974135e-02 5.993068062294973353e-02 -5.075605034527906319e-04 2.134802744351926157e-02 3.638502557335986098e-02 3.482510723091825078e-02 8.529199157928874300e-03 1.833430334939666062e-02 -2.540351605965703984e-03 -1.109167127637477506e-03 4.905462217983531242e-02 2.895744006077783858e-02 6.350617392940349770e-02 2.732721659111428947e-01 3.160283451832922308e-02 -3.494127563075433152e-02 3.898652638414352480e-03 1.330532009014798178e-03 2.265467410825896741e-02 -3.689268396956212920e-15 -1.863452017446705702e-02 1.704856782744375212e-03 2.579025182214247924e-02 -2.537407591918863781e-02 -3.213570539419469715e-04 2.852580944310995764e-02 5.893591869758371127e-01 3.773451378317136395e-01 3.499774331682924122e-02 5.201603264761075866e-03 1.715796945997271825e-02 8.083023576922271358e-02 5.480281075010156461e-02 6.762644179094739853e-03 3.269042252545949212e-02 2.122368791474769228e-02 3.319303800608359306e-02 3.961146364954072630e-03 1.704435149477499106e-02 1.177058971494401515e-02 2.650507580832426810e-02 3.977854206998727993e-02 4.807896369987714469e-02 1.317768756088412603e-02 1.061788971519013816e-02 1.069724419242958253e-02 5.731086957122503295e-02 2.382640184939005534e-02 4.253203946389334800e-02 2.204542390227363924e-02 3.163113140072839474e-02 2.466340450733735951e-02 2.037762024395749910e-02 5.952559542307123142e-01 5.300963986931146454e-01 5.222302810504176884e-01 2.372047147877469342e-02 1.064926224778751408e-02 1.454773313146349133e-02 3.378161350008305076e-03 1.859413870858081172e-02 8.668281716663174888e-03 7.984883848430619074e-03 1.237351333218922149e-02 2.396641963022022939e-02 1.625103107972063199e-02 3.656377706973691633e-02 4.514220760852252941e-03 4.267142134017808952e-01 6.886794922246959327e-01 3.609910300460755783e-01 9.690855517390387086e-02 4.640747889731550169e-02 1.533076496048406054e-02 2.998207176327175386e-02 2.430773235266148191e-01 2.077440384459707801e-01 5.440104260786378099e-01 1.508784737220542072e-02 9.373641412630400599e-03 7.134619884516861521e-02 2.690062909192229498e-02 1.106401483293465854e-01 3.687740719955658747e-02 4.591438745001334698e-02 6.502452086011412764e-04
1.881220020617987154e-01 -6.639515505790222205e-02 4.876380614531702906e-02 1.082141623475061862e-02 1.500396989045486330e-02 1.892648479167532071e-03 7.386467234429880402e-02 9.705312089428409783e-03 2.448233013298689587e-02 3.773460730673176139e-02 5.048038086744985331e-02 2.644828208667743957e-02 2.106400191719512080e-02 2.514801868619139305e-02 2.640695887975214040e-02 3.913437389550608603e-02 5.667157656850523723e-02 1.516191881713416928e-02 1.990973898008986664e-02 -1.311294201639660317e-01 -1.685161896369032353e-01 -5.539037153011485554e-02 2.603685016598228028e-02 1.462558947912099075e-02 3.116329138995078832e-04 1.845200510007172917e-02 7.362795951995877719e-02 2.577042421312065427e-16 5.903856647293013321e-03 9.995385783071878902e-03 1.662780798419754560e-03 5.668678394164328692e-03 6.273511377950987192e-03 -1.192522591827159008e-02 -4.906139335955985586e-02 1.498078802784582789e-17 1.721110623435940980e-03 2.533266680978241842e-03 2.029117096836897602e-02 1.300776463457828033e-02 4.178160819376322554e-02 4.915222752961790481e-02 4.096512149041320328e-02 1.377190438921975513e-02 3.349327647612204001e-02 4.739024738472997350e-02 6.587531170105347111e-02 4.121787745521098695e-02 3.662678936181244099e-02 -2.711275071388981922e-05 -4.527579876972374386e-03 8.937502206051399847e-03 1.667858679016851614e-02 2.821016861155720840e-02 4.310187736891404192e-02 3.283302081198399142e-02 5.294263736481397142e-02 1.778477421954622884e-01 4.428280206640226868e-02 2.947075372541406793e-02 -9.286635022191928879e-02 -1.021483727204447732e-17 -8.530022128065707643e-02 1.787504870152505088e-02 2.756575637541412716e-02 -7.056566763537590325e-02 -7.855152069723576533e-03 1.795584357306196158e-02 6.819400271793424340e-02 3.688368203702072079e-02 -1.168679048570351013e-17 1.650010266991935176e-02 4.650554628937277686e-04 3.402333296198772200e-02 -3.847501158010722907e-18 1.223261283086863119e-02 -7.676586008870073875e-19 2.177339699128818043e-02 -6.777882014411111544e-02 9.004432751552566422e-03 1.081674985928906749e-02 -2.987333284112064433e-02 2.990029333633811501e-02 -4.043567529582871994e-02 2.687335555087950328e-03 4.862944578532047910e-03 1.647097771286698542e-02 -9.580620133983964093e-04 -2.732211381593164308e-04 -6.186864862889605507e-03 6.927149916207580611e-07 5.630899581305240376e-02 -5.214296434203409547e-02 -5.866291649259752772e-02 1.366657220343938522e-02 4.866711083670102224e-03 -7.576893559813150172e-03 2.717011118596027175e-03 9.778788193810536535e-19 1.079559393158768983e-02 1.150919545129791381e-02 -4.020901592250816087e-02 1.719358407187927124e-02 -1.550807778510417123e-02 3.143918605600500710e-02 -3.784535723426983946e-18 2.414407063063275100e-02 2.972146947496888403e-02 2.201738030875369581e-02 6.082473580176214756e-02 6.056583552925717123e-02 -5.031845872588424327e-03 1.996261144461584230e-02 -1.763561816420981462e-02 3.404867396536980445e-17 1.983513264925593732e-03 1.855721266811986500e-02 -1.485222606030281618e-02 -9.379659313083764036e-03 -3.801030477943569824e-02 5.728087375203475551e-01 3.869184213516918547e-01 6.362461487087683887e-02 9.248240579359971042e-03 1.832799897012240414e-02 5.696888141122462279e-02 4.866324949788843751e-02 2.738049631255086609e-03 2.097653712307010615e-02 2.329142629637399875e-02 4.054573395497387822e-02 1.118086142603885377e-02 2.896234041192921604e-02 1.598395771630148690e-02 3.400139420931776180e-02 1.727838892368336113e-02 4.338326234462365483e-02 1.117002368880870566e-02 4.888922043950383670e-03 5.973408531751721595e-01 7.530561525502026399e-01 5.993016027194444373e-01 3.669923722358504198e-02 4.079912630574743237e-02 3.408221800531065215e-02 7.398914269344801509e-02 1.237836075903998995e-01 3.891607229049846156e-02 1.714627306897884007e-02 1.263789732740544861e-03 9.481682714180537583e-03 6.188519494788415065e-03 1.251111108780969539e-02 8.174878052660765615e-01 5.340007381084785898e-01 7.727574203118978691e-01 1.309326587612631015e-02 1.377390463948175488e-03 1.705642305633203043e-02 1.435840782858973513e-02 4.093696719817047774e-02 3.275695227805427989e-02 3.814487716594407163e-02 1.825717514429598884e-02 3.120511798139736365e-02 3.176485708890929710e-02 3.877250756679501142e-02 2.410879952519169491e-04 3.087032435054606339e-01 4.574046075042009774e-01 1.784726305212489816e-01 -1.426017376346591821e-02 1.097212297367448214e-02 2.978761279117356914e-02 5.177649221228938087e-02 9.137028980355323127e-02 1.282061342259456227e-01 4.487161435906776574e-02 8.787964779980206309e-02 3.717103640287862387e-02
1.262780137524487922e-01 -5.732069081319659393e-02 1.772332079539539612e-02 1.284379043792831590e-02 5.863466234767827912e-03 3.957789119291784535e-03 3.668222695249465787e-02 3.374981385693637172e-03 1.178296353519916149e-02 9.744683430592390380e-03 4.729890387493763254e-02 6.511213264497692915e-03 3.392482308640316479e-03 1.978994071197110599e-02 2.704349103913040661e-02 2.998440255041414162e-02 5.131914335775172170e-02 1.529704592272550234e-02 2.895138372436013158e-02 -2.134122411277703316e-02 -2.334628070564908620e-01 -1.491770987022722310e-02 2.994618111555037016e-02 3.620754664882411961e-02 3.496476639682989662e-02 5.091908225868586185e-02 4.033422419822112887e-02 6.360827974125617297e-03 -2.994210388916650374e-03 7.620748327574005353e-04 5.444980463675176685e-03 2.245348549165463576e-04 8.266201057317667904e-04 6.188388015435813613e-03 2.225552289245603596e-02 2.533187587217580056e-02 4.360245668927976992e-02 3.982408926049398751e-02 -8.298870960191728069e-03 3.965692610990010497e-02 5.026675464562103957e-02 3.932262101795197529e-02 4.528582096740554203e-02 5.166609630495430996e-03 3.312967430908697608e-02 6.217536738100300914e-02 1.099292319949940755e-01 2.788532936173245685e-02 7.138409581492448364e-02 3.291996407148332221e-02 4.470234045478150425e-02 5.230780791386519663e-02 3.853114695939104800e-02 6.002233276131292705e-02 8.887282495844806007e-02 3.749064266537353862e-02 5.676093691523480950e-02 2.026289500959881085e-01 1.637586289408935747e-01 3.806744127668222621e-02 -1.005586020918775442e-01 3.093116778910454112e-02 -3.524681838080822444e-02 4.423480624075461201e-18 1.653815133464360890e-02 6.909445229693877669e-08 1.600753799441952466e-02 3.008254472674336266e-02 2.943013688279697351e-02 3.370919632882282690e-02 -5.280251086018215069e-19 6.598053894332928104e-02 -5.982892811500986271e-03 2.456641737157590211e-02 6.424314459646009168e-03 1.001015404462070324e-02 -3.817224049631707385e-17 4.605319691440067781e-02 -2.689696798500449196e-02 1.573171102280181838e-02 1.125349695447758723e-02 -2.493402933262739082e-02 7.516393592549251712e-02 -1.085077841904085122e-03 1.237066956922466959e-01 1.040719569739358185e-03 2.499391476600953954e-02 6.095566194444780768e-03 5.757457514895044509e-02 -1.690926960818007064e-05 3.727705688038391771e-04 1.201635544350174905e-03 -2.311392918284953506e-03 -6.338502056077680025e-03 -2.189611834422928484e-03 2.608163057496767565e-02 -2.561167042998495000e-02 2.824036427187329515e-02 2.719375043415175885e-01 2.486080673601328075e-02 3.588371503947389296e-02 -4.421665753374123364e-04 1.520420866460015256e-03 1.990801806989505457e-02 5.166352431081732734e-03 -3.174691186397575106e-06 -2.431764316843817129e-03 -7.728506557777751508e-03 -8.064815112736953154e-04 2.494441607728931434e-02 2.057598523282287706e-02 1.961607024256333573e-04 2.703225579679033319e-02 -2.829013319016047620e-02 3.005281102031916174e-03 1.612602261981399554e-03 1.653576061293984129e-02 -3.401482840634754284e-02 -1.835676159852103749e-15 -5.033604073150938635e-02 6.485138996152168822e-01 3.651386464253719155e-01 4.992753994182418109e-02 1.017061478189732610e-02 2.054580783627748403e-02 2.892269659209132510e-02 4.503589010175287205e-02 5.067823542609053988e-03 3.595551775698541813e-02 3.567257800919090144e-02 4.420891418619566499e-02 1.170602814657585415e-02 4.109733175794631072e-02 1.319420269617084876e-02 2.762147948179906679e-02 6.757650427187038289e-02 6.090210446026526891e-02 1.834810272187510291e-02 1.149677256890155493e-02 5.433273220918857227e-01 7.145093666524567899e-01 6.323761150154494359e-01 2.841810236262846262e-02 4.013203380372774853e-03 1.299137143765506826e-02 3.565939828068753065e-02 3.129247272067781083e-02 5.427253200702463268e-03 9.000024088461820032e-03 7.026496216245458303e-03 1.511859880865951480e-02 5.215399452359128357e-03 1.090313085801011654e-02 1.774050128229026895e-03 2.017582374444307214e-02 2.071229135066945427e-03 5.049162103542136348e-01 9.730264634217482556e-01 5.826182985491240585e-01 1.752935416827456488e-02 3.307829661046358949e-02 2.538904477004224822e-02 2.763125171685617415e-02 3.396103492952043152e-04 2.987788598094160558e-02 4.154181459102014584e-02 7.391511331130500018e-02 5.580280063286403858e-02 2.739380544070879608e-01 3.896892267254714604e-01 2.018549921458984153e-01 -2.409331806010226590e-02 8.393624620036729128e-03 3.488292967160830371e-02 5.922601607663590534e-02 4.176122939207164292e-02 1.160459329494927944e-01 4.324943754264084367e-02 4.521425877090690104e-02 4.153047214848516744e-02
2.399077481461288353e-01 4.395018409700027912e-03 1.471192737329937369e-02 3.427609908982285139e-03 6.954747765661729032e-17 7.323329567691471255e-04 6.303598322162359824e-03 5.604991714106230885e-03 9.375719815331966245e-03 2.155656159467188832e-17 8.765348537089005798e-02 1.133787483414772220e-03 2.000471368319821364e-02 2.576406825121013364e-02 1.048392032671426055e-01 2.862165877763439559e-02 8.729995287808395982e-03 2.197422255722342871e-02 2.631115341421751824e-02 2.398287074073501141e-02 -1.024287529676869146e-01 -6.541573725221498659e-02 -1.631596990353077203e-01 8.945141693477175193e-02 1.727487889512889507e-01 4.874394729998986209e-02 7.111570754350111934e-02 9.510696988701390631e-02 9.448980178847543598e-02 9.152363782751055110e-02 1.199857247141201261e-01 8.957509091190812844e-02 1.433691355265014067e-01 9.135132580236673960e-02 -4.303420225142680355e-02 -1.916487867959415156e-02 -1.774784820102578292e-02 2.202806649643677603e-02 6.571105750954971125e-02 2.023101460411052752e-02 -1.184275023900570045e-01 1.246571758709113034e-02 1.317345137659277181e-02 4.427034144462165151e-03 1.923413389591006883e-02 2.650954293890772639e-02 6.804736256370112757e-02 2.375314706731189440e-02 2.638637511824136472e-02 3.032469342522617697e-02 2.649462079339796189e-02 1.377748086838289417e-03 1.318050856998339605e-02 7.228067906875653911e-03 3.480435917862767514e-02 4.338466286290724658e-03 -2.171031004373100148e-02 -5.025439213344427142e-02 5.279628614689603205e-01 7.753022998421278678e-02 -1.456303343013465312e-01 -3.187206057854635732e-03 5.868867863977107573e-03 9.152484916317716809e-03 2.188059142124795450e-02 -1.384485285570819131e-02 2.193744405410734399e-02 -2.395242587661775317e-04 8.789806818044549469e-03 1.642269659357990796e-02 4.251554703082157868e-03 -5.162883450851966927e-04 -2.035368246667190620e-18 -1.408516962030725676e-02 6.156322141218882604e-02 3.587997151997946582e-02 9.522346182607265355e-03 2.695080043618793345e-03 2.745832101575510290e-02 -5.591774711150397995e-03 -5.272808680228879222e-04 -5.650795558669859109e-02 -1.466406589652933211e-03 9.189397938543895569e-03 8.063467650342298620e-02 2.337929345194391870e-02 1.711327809512250622e-02 -8.999998330084428913e-03 2.796449941350118845e-02 -1.122543837240897423e-02 5.221823462906393515e-02 -3.744651177190589514e-02 -6.404552181765315149e-06 -4.013937162239920789e-09 -4.901402022502134720e-02 -8.698031101546422478e-03 6.069116852802976941e-02 -2.780096386764328159e-02 1.909367156071691690e-02 3.136177749631353095e-16 -2.248721633353988471e-17 3.967491955649651847e-02 -3.759716073235955030e-04 -5.562549662594649252e-03 8.776371352200849907e-03 1.199204737497110845e-02 1.391739627806519441e-02 5.711089208184741937e-03 -8.721467159011887099e-03 4.867406355749503977e-02 -2.529241550494667860e-03 -1.046165017217848226e-17 2.721144963036220207e-03 -6.004948650490578625e-18 2.811341790881558673e-04 5.590121765853113545e-02 2.155986752046067795e-02 6.385555589853834924e-02 4.277812416227561726e-03 2.825360244476021188e-01 5.781608105638810224e-01 3.093450632431073433e-01 2.261893738821750197e-02 1.588021457121772524e-14 5.677904449323494321e-03 3.663668491308932256e-04 1.434690097477214939e-02 4.936117338500006953e-03 1.532229478621695631e-02 1.588130946416867282e-03 4.038824835356383258e-02 1.551170638666771705e-02 3.504181026359905887e-02 1.613536138427608055e-02 3.429291753427751027e-02 8.262809492737922812e-03 1.105032969249306919e-02 6.521248543025972601e-03 2.870928961295908174e-02 1.751317338686067537e-04 4.985131796260740544e-01 8.219158655820212322e-01 4.370864339843828494e-01 9.183492477519389721e-05 3.651979677782413863e-02 2.045537808703656521e-02 4.839813495530050774e-02 1.419868021953600568e-02 1.720041925011163431e-02 7.023953758573757075e-03 6.539196194425200837e-02 8.301611749145115793e-02 4.139657890132102808e-02 2.914877416447443637e-02 5.173209057476491068e-01 7.195942055886627164e-01 4.500688905429793274e-01 2.026569410107815009e-02 4.115097062460928989e-02 -1.259603260590917749e-03 4.738296550534744876e-03 1.704824056296634291e-02 2.795795606902255759e-02 3.044570236063976008e-03 3.834555398252942338e-02 1.867943993683379364e-02 5.435887511950646817e-02 2.435673264835292937e-02 1.446908169389858959e-02 3.283781644394131310e-02 5.782825886186957615e-02 -1.146910625980944280e-03 1.650715938178742848e-02 4.272923531368312718e-03 4.153332400760758464e-02 8.246930454324120641e-01 1.056933224919530262e-01 2.791684582268627035e-01 -3.984357451058171962e-01 -1.057239715026777160e+00
1.425556807273842264e-01 -6.599188043346261578e-02 6.098378435528844371e-02 3.559544799026472667e-02 3.961375453915056549e-02 2.263760932603127440e-02 1.251554944852704410e-01 8.991897821997361234e-03 2.430153155344985313e-02 2.761210990790579148e-02 5.296368168286521433e-02 1.844752583175617022e-02 1.193357565854246891e-02 1.015165361287446305e-02 2.492490900788363328e-02 1.959373190456691408e-02 -1.016973359202123645e-01 -9.189933517091324056e-02 -1.078877712330037208e-01 1.227682287856053309e-02 6.507461001345563045e-02 1.148558608898001855e-02 2.363775832825655565e-02 1.412948199145224598e-02 1.545259186377519883e-02 3.894545573284855650e-02 7.613037679225885535e-02 3.732341175171002679e-18 2.869389248645385243e-17 4.027330182123959007e-04 1.022764085165862001e-03 5.344213087060735792e-03 3.340820929892872543e-04 2.156823156184750016e-03 1.185230658333425759e-02 2.211555519423973049e-03 -2.794705777080444889e-02 -7.843747595875241382e-03 1.202926048733912560e-02 1.443494825329032748e-02 2.625038750640413046e-02 2.019413889480026203e-02 -1.756788470413338720e-03 4.260648750105627131e-02 -4.119393482837421794e-04 4.971833310420771534e-02 1.366281484291431703e-01 4.057737168673164474e-02 3.152569324078694640e-02 3.285873305507113623e-02 1.714664774503722733e-02 6.435720329227914704e-03 5.011039804320665397e-02 3.256687290070529178e-02 4.289731760648053671e-02 1.012848192788756013e-01 4.240744702404907612e-02 -1.246262718906466366e-02 5.793531603511085232e-02 -2.100949921748049981e-02 -1.278283502605331023e-01 -2.863648231831204018e-17 2.285705163416092839e-02 1.855387539277010556e-02 2.825765969822886803e-02 -2.221002822424686385e-02 1.156683607119113333e-02 3.073504930931074061e-02 3.839317703565236356e-02 6.191198006959947563e-03 -1.091295603664804091e-04 7.337232574923900109e-03 3.907822733537332470e-03 -2.276871968149668524e-02 -8.485938821899356027e-04 -2.904920134973191723e-02 -8.422743988351675355e-04 -5.728953449017657601e-02 -5.001612316385532775e-02 3.826515040439586179e-02 -1.287170193209109001e-04 1.040527537704627817e-02 8.042338730860014440e-02 -1.323021169294318327e-03 8.084847736633189383e-03 -3.142903614583290785e-03 -7.906894787222605255e-17 2.762504014733901427e-03 3.382711243558474828e-02 2.908380424876953751e-02 9.471004981601964523e-04 1.013919864212352306e-01 2.105447755994642803e-02 -3.191934355634310633e-02 -8.885902393364780511e-18 5.178688474445791029e-02 3.365334459843680875e-02 -5.135951608383946231e-03 8.358113626422440309e-05 -7.295837536159395856e-03 7.289813507890569033e-03 1.410627619213717515e-02 -2.696221182530290309e-03 -3.837433173745019577e-03 -6.752625966813663986e-07 6.509193451052193435e-02 -1.367785324448935469e-01 4.479543086644009048e-02 6.450572571131095445e-04 3.864152932105314412e-03 8.050299690746866299e-02 3.731544754602047348e-02 7.241526649175780705e-02 3.768651541825670387e-02 5.888640687345124414e-02 2.109182216068017715e-02 1.326045494879156406e-02 1.021803469073255706e-02 -5.478916630020882333e-02 2.025437101371077575e-02 6.362697776242643988e-01 4.107994849330183218e-01 1.023433432551075148e-02 1.360916412086287351e-02 2.722076126786161787e-02 5.754350895104418107e-02 6.030442718776993921e-02 2.200329664666907797e-02 2.692771568888814235e-02 3.751548236772843381e-02 2.580515051246787608e-02 7.655708558614735849e-03 1.484191437671603803e-02 1.496231533432300365e-02 3.082795574029698754e-02 1.718474736745234915e-03 4.749539199905598585e-01 8.216593552742866846e-01 4.680823942336338472e-01 4.988837014612638662e-03 3.788643637924731661e-02 4.277741026188670859e-03 4.938377877754517031e-02 2.799113841716199813e-02 2.738679230413812102e-02 2.349443855954072222e-02 7.666550712606684526e-02 2.573827575634984987e-18 1.243366629701072278e-02 2.045145299332626777e-03 1.829443119544580873e-02 6.331723316312339207e-03 7.604733373105834228e-03 9.023585471760894004e-04 1.401887097153250994e-02 1.237230614736432085e-03 4.345278821647761275e-03 7.737505815414813352e-03 2.051335648818793794e-02 9.281715427953593847e-03 2.524335084505074933e-02 5.985157074416657362e-03 5.465883836944122054e-01 7.842930422156799697e-01 4.674936904626642020e-01 2.455608067510495901e-02 5.271591425080394272e-02 2.740923827503940341e-02 2.075007894193769270e-02 6.311153160352210578e-02 3.729471470809513772e-02 3.225966896519478211e-01 4.168083715237566600e-01 7.761933197309903720e-01 1.263265306726130133e-01 4.382464012186662594e-01 2.094572372320025744e-01 2.027450009868174419e-01 4.523135520428948064e-01 2.158067288996580735e-01
1.431501767042228646e-01 -8.127473065856782863e-02 7.859184558703434520e-02 4.885147029262524448e-02 4.143961014833385387e-02 4.608727165461946884e-02 9.316397038089699323e-02 1.609519512826949536e-02 4.387408473587869201e-02 4.576964757238997233e-02 8.205771293233325714e-02 2.616328960906457585e-02 2.574399578329436555e-02 2.955108256377285761e-02 4.061814099895928731e-02 6.332719756138241451e-02 9.369949626996014469e-02 3.625599619834420778e-02 8.537784935131430275e-03 -6.020412079798159732e-02 -3.537101688242764996e-02 8.963027403839387425e-03 3.311035092871955632e-02 3.706092066583083905e-03 7.299880706198990689e-03 5.836606004141990656e-02 6.859251821966676210e-02 9.727655518827812292e-03 8.411450171586591293e-03 1.082471218321262975e-03 1.386648269421072240e-04 -1.240118775753182995e-02 6.337362545596935607e-05 4.182105417358730239e-03 1.986819437841139818e-02 -2.683029448519307913e-01 -1.956901632305279304e-01 -7.489535221379894914e-02 3.381458017936820465e-02 1.413372039203846485e-02 1.309139408663828955e-02 1.790031012463446161e-02 2.609224381491265579e-02 5.469280491824077099e-03 2.970934112864944895e-02 2.803963976209264061e-02 5.061166084457195680e-02 1.288582268654802840e-02 2.983521471218694016e-02 2.022496122992170978e-02 3.187480942476873108e-02 2.803458566488034858e-03 5.596679824051417618e-03 3.560693538335799740e-03 2.063221665968660579e-02 3.495291530778099620e-03 -2.375372938413324031e-02 -5.817655257424662191e-02 5.369501385145497574e-01 1.265466070751935235e-01 -1.001257483076731375e-01 3.561385805943391263e-03 1.495526162437870732e-01 -2.595978347034838540e-01 1.680543549515636087e-02 -1.309383823774618212e-02 -1.156016765112724857e-03 1.114563419936332594e-02 9.707760237275397941e-02 1.095979332915206439e-17 -9.715260797295649945e-18 7.706814906307565827e-02 5.565719838732680752e-03 -8.667363609659790510e-03 7.434262840123277705e-02 -8.982067865660013986e-03 -1.860120740937748485e-03 -9.775685769129452418e-02 -3.415153044347229300e-02 -5.070246530371563809e-02 -6.977024386756811025e-02 9.131511751896054241e-03 5.241195565547672464e-02 3.015583320778456380e-02 6.758322202556812780e-18 -4.542006086425851193e-03 -4.496915093463959429e-03 1.001535999126925217e-02 -1.014579980775557134e-05 -1.127121448353000992e-04 -1.117627997197376613e-02 -4.003551044658719589e-03 -4.360016749822071165e-03 1.223971004579132472e-02 -4.710820068866697179e-03 -3.520731009830932977e-03 -7.013698792440243857e-01 -1.805995617333599687e-02 -3.052925432361642752e-02 -1.191888370043958255e-03 1.086582976754191417e-02 -2.440599206600888095e-16 -1.874791638635419505e-03 3.638230819532087962e-02 3.853054714244465943e-03 -1.041164153376634809e-02 -7.332170643058290478e-03 -4.734926381627852020e-04 -3.808377141478572203e-02 3.947999101187676940e-02 -2.594211408482081406e-03 -2.913484542677048737e-03 8.459482777026227154e-03 -1.113714758062787488e-08 1.630992437984196339e-02 5.546630197502771392e-02 4.052693466233666698e-03 7.565163627193800810e-02 8.096611823204999045e-03 9.745572794664637828e-02 6.341425550353240181e-01 3.758104235568289209e-01 4.207397950139317444e-02 1.075659854419660881e-02 2.468227795268039537e-02 4.386191686900098308e-02 6.931685470940640581e-02 1.498130099557074164e-02 2.348238101278121367e-02 1.733449549989812821e-02 4.162981379163670920e-02 8.191732671311605443e-03 3.467118886273289069e-02 9.517706871206855698e-03 3.562323795654537967e-02 1.657199938543088211e-02 4.332308917993812208e-02 1.295653632454174878e-03 4.491017085757347260e-01 7.665262819841864861e-01 5.495608880095322890e-01 1.026044935755843299e-03 2.401276697219498213e-02 1.454638477851530678e-02 2.488695454341079139e-02 2.289617628638394853e-02 3.400215718699117762e-02 8.725477130947093592e-04 2.495393783254024056e-02 6.303904888312542379e-03 2.044677239366122362e-02 9.337615591410761791e-04 1.110513327839730488e-02 1.331212786816730193e-02 4.078119073187631144e-02 7.319949535548004871e-01 7.312410447046711059e-01 7.428770501157335726e-01 1.841551160670767615e-02 2.295275804540499939e-02 3.686504259972325781e-02 2.578181299020609191e-02 3.894910504924017219e-02 3.788150730276941460e-03 2.832931514859252445e-02 2.596535158952481201e-02 5.069075923362837549e-02 1.133726681934574265e-02 2.167981658454612784e-02 1.650726952970696035e-02 3.947640259798884665e-02 2.264772731106107044e-03 2.335420928802807786e-02 9.131110274513548619e-03 4.637747032712929290e-02 1.612155736016564755e-01 1.530861689477272247e-01 3.948866936014289175e-01 -4.245632557394160012e-01 -1.275611202938382815e+00
2.721646476979394658e-02 3.309016051628873439e-02 7.862066148489761652e-02 -4.016657935929207612e-02 7.937989343365071948e-03 3.896151854716658060e-04 2.218031533230462038e-02 4.204147769874079510e-03 2.017696632269618331e-02 1.514532191756237309e-02 7.822530550392606252e-02 7.724673856593825862e-03 2.478049846107551260e-02 1.639642582579649982e-02 3.844843570774054975e-02 1.301131597019954816e-02 1.222741296336083035e-02 1.759565199253773984e-02 2.094537996675888483e-02 2.278766434387324499e-03 6.290346746269301174e-03 -2.115530436890459415e-02 7.483774607722867844e-02 -2.945354704996018449e-03 9.035279205248350487e-02 4.951311166580427831e-02 6.701075786315806304e-02 3.992322323798063594e-02 2.259102852468001729e-02 7.038407313159236733e-02 7.017548625952602093e-02 7.880072842414621859e-02 7.296033397997478409e-02 -2.657564594620888039e-07 -3.377221427559527062e-02 2.581693509934362345e-04 1.416064554930859115e-02 1.344899975731140424e-02 8.478429669895165555e-02 1.385202450672720804e-02 -6.940479507101625497e-04 1.834258192397397388e-02 1.763087882327356304e-02 6.152094749102254498e-03 2.766951187651273125e-02 5.641341376083471720e-02 1.010601676345478511e-01 5.870079868894865355e-02 4.412105889538451425e-02 6.001519879473340651e-17 -3.155937671840899905e-03 8.045820553685224807e-03 1.081706526050614688e-02 3.610476622307210587e-02 7.048666862062584471e-02 3.830777698457392411e-02 5.799296474098578069e-02 4.915490747808409083e-02 9.611627343643318333e-02 6.887133258490407695e-02 -7.871425819723897321e-01 2.308860667090224111e-02 2.821966699159236813e-02 4.346702467924692947e-02 -3.410761794744394948e-02 -3.293953747945174704e-02 1.495489440469613684e-01 8.454130257157600520e-05 4.741976801440501111e-18 -2.502372590047513556e-02 4.468584280254532526e-03 8.055709023443173578e-02 2.128629662406150702e-02 -6.827822936915361485e-18 -1.117235573198927488e-02 -3.957148894254969296e-03 2.020856477511950682e-03 4.233785298608884473e-03 1.697831568938076158e-02 -3.331141297324063250e-02 -1.052655929218497717e-02 2.910506772968465133e-02 1.343780937043088930e-01 6.580804420214141082e-02 1.415857141382025584e-02 4.329976130161060366e-02 2.673557341651819352e-02 -2.459340594040172051e-02 6.695632623652828650e-18 -4.325667526335193191e-02 3.227675027164928628e-02 -4.000439550526015131e-02 4.094352333391995669e-02 1.311821435498616573e-01 3.331078651029362875e-02 3.205648008937374283e-02 5.061727063079670819e-16 -1.529250121916785055e-02 2.877834573299129645e-03 -1.704784397146060816e-03 2.455498053500879130e-02 -3.041331325187228429e-02 1.515658972734114200e-03 9.693997205063349318e-03 1.511456237145310938e-02 -7.118755982292927024e-18 1.434603835895645460e-03 2.375634822946726121e-02 4.706890938486987114e-02 2.389855291460851044e-02 2.934364517382979237e-02 1.251990157047477153e-04 1.712210262605745950e-02 -7.659945104297399832e-02 1.874449688356466916e-03 1.055349009645618202e-03 7.481319500335814610e-03 -3.934795688665666119e-02 -2.441755294110451151e-03 -4.405089878828739869e-02 3.240540924490494246e-01 1.108767972163463428e-01 3.411422584826116489e-01 7.150380560613069925e-01 7.979993660385222842e-01 8.356331603852555912e-01 9.294233355650920875e-03 4.085195540422638730e-03 1.517474424457699797e-02 4.758576508517132521e-03 4.069318574056453308e-02 8.328922690992162457e-17 2.247186940636423588e-02 1.313850655651785555e-02 3.742737029509479035e-02 2.193473177791634490e-02 3.927770963896832362e-02 1.791452386079585984e-02 3.334255436636865411e-02 4.164782295465978973e-02 7.210892106978107619e-02 6.892334667216722943e-01 6.769894914562296639e-01 6.025364490354918612e-01 1.456095144708197640e-02 1.655856123838505070e-02 4.674921528708621266e-02 1.554214259375152896e-02 5.540080360665046617e-03 1.522479323223599468e-02 4.792658914095473260e-02 6.280474690462960197e-02 2.419927628187154567e-02 4.661909254528833757e-01 5.429565503922021419e-01 5.639278222260495355e-01 2.375272275634219415e-02 5.391611165856746019e-03 3.776232184744779397e-02 -1.314174540557884246e-03 5.031494381698716108e-03 2.664736132560942619e-02 4.824685609965644051e-02 9.203222143233847144e-03 3.208979566358390045e-02 5.452214956642430221e-02 4.686907420166550736e-02 1.061231474865089542e-04 2.447451848417806219e-01 3.376727821969388033e-01 1.157885107026626059e-01 -1.643433283556511651e-02 8.290080908285346259e-03 4.604364033360783259e-02 6.211039005723854173e-02 2.334759230037182845e-02 1.196732038290607841e-01 1.005003184315756415e-01 7.315891389278333934e-02 6.414291841535681160e-02
1.399838169636652419e-01 -7.984098047527900255e-02 6.930359158997875801e-02 3.152720101462672120e-02 3.532785205340552986e-02 2.705640518418999177e-02 1.328537268279311179e-01 1.191707624948597395e-02 4.657589683920035362e-02 4.191687728044406203e-02 6.735083716594011949e-02 2.217413883980528383e-02 3.200298534473801013e-03 1.824564449138971894e-02 3.784974113370265930e-02 3.417388805075746022e-02 9.804032676097743093e-02 2.943033476057901193e-02 2.982894425075213133e-03 4.336553862840542006e-02 2.545283892896201272e-01 1.186691278502461921e-01 3.454427843243101232e-02 2.959658869043176391e-02 3.248365967170253193e-02 3.272997392777488024e-02 -6.496511221892034560e-02 -6.454572787748606710e-02 4.850764361252742793e-14 -7.899278152449307200e-02 1.701152276899902879e-02 7.288256774589328690e-02 2.887814764904240300e-02 -6.899613863648795808e-02 -3.803108668591319297e-02 1.565928659957473157e-02 4.393054889719973446e-03 4.586307613343611547e-03 2.451277094449785116e-02 2.074430142814470256e-03 3.756815844275354643e-03 1.816423724436571144e-02 1.178259252578836742e-02 1.320957780150755490e-02 2.010474149771315430e-02 2.758200891258116155e-02 2.832128819785021193e-02 1.943162800543714666e-03 1.033396097709754317e-02 1.468158005120644670e-02 -8.936712711708792028e-03 9.427120908864692478e-03 -1.800020156874170387e-03 7.092541025457543835e-02 1.427052080821167857e-01 1.762423204148252931e-01 1.050621006519429634e-01 9.345825261322167343e-02 1.893021723627725250e-01 7.238256760408930124e-02 -8.554099922751474960e-02 -1.101983480977314643e-17 -8.589917599702288561e-03 -3.675595700105401746e-04 -3.770345823912413588e-03 -4.514313188729474373e-04 -5.450648244063368067e-07 1.135315167688381865e-02 3.287845257797612081e-02 1.047853697456777207e-02 -2.622559264012633480e-04 -2.231097523428661486e-05 3.571027239413476512e-03 4.564293034245315434e-04 4.531458734798591258e-03 3.828979578488069663e-03 -3.221629408715948745e-02 1.331881034767171167e-03 -4.796896890234510213e-02 -5.620700759494969001e-02 -6.121384490984821706e-04 9.499837086397303065e-03 -2.843075314362282220e-04 -7.358250856277455182e-04 -9.506325865243472057e-06 -1.438069635746600048e-03 6.263910578993535545e-03 1.558444119851444933e-03 3.711996435881859211e-03 -2.086295970154772653e-02 -3.272201522655553575e-03 -3.020432236992510663e-04 4.058406591920083774e-03 1.185052410213900502e-02 2.164847599691592864e-02 1.777870242683824917e-02 1.293776067822014451e-02 1.185511818689444566e-02 4.851436602517190990e-18 -7.666939367164578439e-18 7.478897279221763741e-03 -1.488669304192537929e-02 5.571544222529308888e-04 5.499964597413037858e-03 4.092827530980993472e-02 2.699874979046944928e-02 -8.533798443156522320e-03 1.048355923853718785e-01 7.316183304909898862e-17 3.072632644204196031e-02 -3.078350519226277435e-02 -2.645898836986397898e-02 -1.766884495891775028e-02 -2.337711254979701544e-03 5.612298102644287703e-05 3.689601758544397794e-04 2.138321159495221006e-04 2.228904648567565233e-02 5.841191114031920456e-02 -1.739335759091601569e-02 6.807713742836403181e-01 3.486577326751786332e-01 4.073755970940060728e-02 4.632769989899852836e-02 2.149380481236978779e-02 5.965667496608315079e-02 5.931062588593241647e-02 8.431281314163869070e-03 1.687192957260537363e-02 4.670372747349689491e-02 4.105056183485525367e-02 4.400344620309423749e-02 2.714449498016855214e-02 1.330914335494761888e-02 3.006154315161015805e-02 1.983783282547502483e-02 5.644866483384311517e-02 8.324896537069449709e-03 1.817523661823172362e-02 2.133076322511898507e-02 5.676474467219257136e-02 1.051689705588645420e-02 5.100288664025649094e-02 1.462462806273396448e-02 2.437116653288567969e-02 5.786861650427145218e-03 6.955944816860591962e-01 3.594760549076600475e-01 5.020716743297014562e-01 5.477763146041776388e-01 3.537923264346421459e-01 4.115460713304087137e-01 4.835227288764253895e-01 9.300837132894357318e-01 3.171175704778222970e-01 7.948003768897864358e-04 7.409860367513491745e-03 5.186058280830031439e-03 1.344431399428614241e-02 2.648825356480703080e-02 4.605116492072004047e-02 3.739196380608209025e-02 3.934869910993348679e-02 3.291961241633984889e-03 4.509335295377174535e-02 5.351127116390630434e-02 6.382456070851169616e-02 2.247431940058470384e-02 1.801052507465060112e-02 1.317889371715367901e-02 1.941651367422635166e-01 4.821939861782680858e-01 3.003559647013430700e-01 5.446801027813539912e-03 3.066750886381194516e-02 3.187524671545060445e-02 8.671020055914679947e-02 4.419766019132002899e-02 2.451427603513557651e-02 3.664782558010031693e-02
1.330709418868140093e-01 -5.226117482566286498e-02 2.188448447476916761e-02 1.858450723465129156e-02 1.024743559147529674e-02 6.810603044188598236e-03 4.536848580635301914e-02 7.927832172446245542e-04 3.614167896655693518e-02 2.447394058565234565e-02 4.437707514880898096e-02 1.013483709881630480e-02 2.096303940854056178e-02 2.365391757932133049e-02 3.202985451520917737e-02 3.674487914216455209e-02 5.733084160673238699e-02 1.889819168565758101e-02 4.120481314372843873e-02 -6.841639086633724554e-02 -4.451716199426143661e-02 -3.419089795201200876e-02 2.061455786579014307e-02 1.148280939353138290e-02 3.663739724966695349e-04 2.103050540607420346e-02 1.165513719747637567e-01 8.465804099330599686e-03 6.283301673699535211e-03 2.500458336434848187e-04 1.041760459465504137e-03 8.077694518811141897e-04 5.274328373421331186e-04 5.842175946641226143e-03 2.960296833417663809e-02 -1.175466901890671045e-01 -1.542762013513896413e-01 -3.452002502618521373e-01 3.194436678509600391e-02 7.544344374270805891e-03 3.344205201860290383e-02 2.992691177596934135e-02 2.884770522011678717e-02 1.780264147231604144e-02 3.469911191160398450e-02 5.281599590386355098e-02 1.030618997216911220e-01 1.755480400068245780e-02 1.934234648167101889e-02 7.299232567595662427e-03 1.249366415335034959e-02 7.930321478741991781e-02 1.925819315699115464e-01 3.542843249427168884e-02 5.299886665715417694e-02 8.448713586398053088e-02 4.726786805043913658e-02 1.515664800305863797e-03 1.219070350954943975e-02 -8.637391189477974449e-04 -8.235002371043624592e-02 -1.396099180946363908e-17 -2.261646595791462142e-02 2.399494044598033879e-02 1.726650338597063641e-02 -8.998809765019662394e-04 -2.227950749773411431e-18 4.837938289424711935e-02 6.792185538825083746e-02 1.970165094444654985e-02 -1.355401370462262778e-02 1.799529406589588446e-02 1.729681422635282600e-02 2.425660307927226048e-02 7.309128668409712801e-03 1.708045704035541373e-02 -3.617944566805669782e-04 6.969250380403770040e-02 -2.795452399048308986e-02 2.209795995822860099e-02 8.946283333524875439e-03 -3.315567423647393375e-02 7.432175130792667561e-02 -9.239975402661889173e-03 8.734223543673242220e-04 -3.414935038749754807e-05 -4.515092430604238402e-12 4.699799718701615919e-02 2.274087774205958007e-02 1.980989914201223071e-02 1.039176231807904201e-17 4.253463239327370110e-02 -1.180198132197223665e-02 -2.139487369991586313e-02 1.208231504846229987e-02 -6.599052045520108732e-03 2.372984128049015692e-03 -3.220365879877047643e-02 9.355391075544050583e-02 -2.649569990953272620e-03 1.620090456604086557e-02 1.271227700316981783e-02 -8.671785450519081673e-03 1.206349261460506334e-02 9.418073009217904981e-03 -6.877582384754917302e-03 -1.743829939509696078e-03 1.086252775979450191e-02 -3.698564570010517191e-03 -8.104404354803894600e-05 2.708460393007524475e-02 1.785660873624551545e-02 4.530832483500436825e-02 3.732568590126120189e-02 7.643112570925339466e-02 1.176880640695972778e-02 1.159502438879704209e-02 2.222387661242671876e-03 -4.907621164464172347e-02 1.568291928916061909e-02 6.434538622709375932e-01 3.859049718001469897e-01 4.689984292394856757e-02 1.349304038216079388e-02 1.373556485968198751e-02 4.265748397997511793e-02 5.642810202250866242e-02 1.819513004731481462e-04 2.460671565408638004e-02 1.572210717310771125e-02 4.378025596025676702e-02 1.245525823933602699e-02 2.746951830254560495e-02 1.789502679292359275e-02 3.782537067904643419e-02 1.624511680918998627e-02 5.299849229863159289e-02 2.480996701770427776e-02 4.494461745900311944e-03 5.648059534614551236e-01 7.312897493408199168e-01 6.517945363598167674e-01 5.253227157815405779e-02 1.440334777294371114e-02 3.832444975651625590e-02 5.199086178069638187e-02 7.709848781687893282e-02 1.113138729797425071e-02 1.928189752619834801e-02 5.986411928271004844e-03 1.660627978690316919e-02 1.209141908709699884e-02 1.684482316132816099e-02 1.118155453073628978e-02 3.054805829450559299e-02 7.135773246798463587e-01 6.958024619044756998e-01 7.641314573838998703e-01 1.825061720901779289e-02 1.953583496930283436e-02 4.801841001976854539e-02 3.445100269786985359e-02 4.870435620206404331e-02 1.644398652321532942e-02 3.544700665921948074e-02 5.103532627532934851e-02 7.282676420863559708e-02 1.433659754451062883e-02 2.262934390232466952e-02 1.682153130018625847e-02 3.379354400168138595e-02 5.537558782868244878e-01 4.541292827510919894e-01 9.312425551803175106e-01 1.215430411694635171e-01 2.996653867354484579e-01 2.191450245066119962e-01 6.909598051712882993e-01 3.887600232344393669e-01 5.452961872450248215e-01
1.185156214746925368e-01 -7.606830586253440774e-02 2.890430060091667142e-02 1.503880696171579733e-02 2.675547345758732751e-02 3.280682405154042303e-02 5.309246835431132711e-02 1.308700462923970478e-02 5.016569235332921789e-02 3.181892358779732366e-02 4.371366167550858051e-02 3.016093347295333710e-02 1.220577714279671294e-02 3.245101820783360957e-02 4.798802075385046745e-02 1.060344100216924559e-02 2.661960133661862155e-02 -6.626661503810740358e-03 8.147773161618795945e-02 -2.791151676385155312e-03 1.805974566368570566e-02 1.011509396030135267e-02 1.263569628085930117e-02 1.716726860568439095e-02 1.109480758710508670e-02 3.069325948971895340e-02 1.019938668752642663e-01 2.253225435820153943e-02 -1.528102391334611133e-03 1.091907479646987526e-03 1.444762296387664888e-02 1.399959035860942570e-03 8.872806230897582313e-04 7.643350658919746167e-03 2.706743524402666860e-03 4.482220246158863559e-03 2.033446577167265321e-03 1.504056536651672245e-03 2.375268187057232028e-02 3.938758531332936519e-02 -9.090869258148340931e-03 3.093098016592228661e-02 2.859512381350146418e-02 2.473501131958683735e-02 3.910546643030859970e-02 8.490781426788501784e-02 1.724449949316874398e-01 4.347146584017933663e-02 3.683234805304230058e-02 3.053644840164649113e-02 -4.223069102260569238e-03 -9.459703545223126925e-03 -3.375832377575495227e-03 7.784792014939584393e-02 1.631756299371807661e-01 1.464627640841693057e-01 1.241541398014256448e-01 4.016941875695240749e-01 2.791415068100995267e-01 8.766501480960740833e-02 -7.477373407210699541e-02 -1.039560381628004426e-16 -1.318003585026405154e-02 5.078739844066058190e-02 2.471566853576272205e-02 -1.089219298209473386e-01 -7.555452913243339492e-17 1.000237085718623520e-02 2.866082102368298565e-02 1.787420925331626234e-02 -7.256125828312027896e-03 9.670907803997968316e-03 1.146471067601760423e-02 7.239611736948136159e-03 1.698684527882795073e-02 1.842453486067758411e-02 -1.218044204571063785e-15 4.116130197604799673e-02 1.058751334054474080e-01 3.326630701438004073e-02 6.136443010515989227e-02 2.136007062562097353e-02 5.899760992901192241e-02 -3.525075406364018300e-03 9.548061579329099064e-04 -6.595260659494167317e-04 2.092561387017368227e-02 1.836440435724034578e-02 -5.321404035094533900e-18 4.695841953513997524e-02 7.193588121663727865e-17 2.064845682559399511e-18 1.359010934988556796e-02 3.982215268530051033e-02 -2.853007170434147116e-04 1.041061498766950554e-02 -4.393766979626681501e-02 -2.100930980136693472e-04 -2.066006547113826211e-02 2.284230306391783760e-02 7.195460721220432732e-02 -2.147479904010462723e-03 2.849626415983417919e-03 3.743637018999196320e-02 1.102468857401172524e-03 -4.691293077971166181e-02 8.138354407984632070e-18 -7.998402532753136687e-03 -3.546179379695954797e-03 2.294380290103290865e-01 -7.015034745970907294e-03 -2.270138412209413842e-02 -2.194502429416677822e-02 -3.912238069274716346e-03 3.137487549788183731e-03 1.679694353395690771e-03 2.938498957097493598e-03 -6.553945917728670224e-03 6.699125993112349520e-02 -1.686173944163541980e-02 6.452370956654067680e-01 3.295779117030295802e-01 6.057189819299577116e-02 4.726608045790544471e-02 2.557784560882793434e-02 5.336734056208378152e-02 7.753284455184847324e-02 9.591344169801188646e-03 2.029597445072866754e-02 3.553447385514390405e-02 3.506817007363719624e-02 1.797221761963179940e-02 1.940059633677320219e-02 6.292897101990720110e-03 3.163935979151793337e-02 2.041922511212715396e-02 3.805426104959492434e-02 7.088504778239802517e-01 6.661415109897391718e-01 5.995847349030570816e-01 1.564407011706433689e-02 5.254200325136237047e-03 4.504873637418963656e-02 3.024005435212435283e-02 3.161617690622321036e-02 1.757002908495794088e-02 6.311348456895177206e-02 9.668160744362669021e-03 9.950472668044815783e-03 7.515357247558966669e-03 9.646134635642778649e-03 7.754612320122001093e-03 1.557890165762262566e-02 1.277407186599064694e-03 2.452556915471667751e-02 4.845617188488434042e-03 6.314144503003822455e-03 1.351325741461664861e-03 1.585300370928745956e-02 7.513505691368412842e-01 7.408950803668955531e-01 5.315220577113860534e-01 3.487704399530824114e-03 -7.734967833661707259e-02 9.646977571264330267e-03 5.873471376140393038e-03 2.164587357566934850e-02 3.074841156685016123e-03 1.360852264841079344e-02 4.277098230455955545e-03 9.738771037662974361e-02 5.536355725304082931e-01 2.772174344109675226e-01 1.097789264688855959e-02 3.722247873450405115e-02 4.796823860307126985e-02 1.002073951875732949e-01 1.392222552631538178e-02 2.796522096365829449e-02 2.742562522315358797e-02
1.247872838006332946e-01 -8.481816940091799206e-02 2.035128814051804721e-02 2.610049692579987160e-02 2.351583347515660838e-02 1.647107722731234666e-02 5.757538214216626621e-02 2.143349730376170603e-02 3.585516247830999870e-02 5.013438866596120480e-02 8.826543627757869881e-02 3.043876675061331361e-02 2.072805501033373604e-02 2.485014738575081622e-02 3.831449294715886383e-02 1.213936667161383785e-02 2.042408436737903552e-02 -2.241215050048906907e-02 6.657086948973386931e-02 9.100217930871184412e-18 3.141155318438675070e-02 6.216374010897742983e-03 2.343491098805407236e-02 1.097780375231661130e-02 2.044209831181242280e-03 1.273611977052009625e-02 1.145617395352525325e-01 1.320425931571489941e-03 6.954333840486039122e-18 1.330868455424014498e-03 4.153085055237119502e-03 6.410682912110530346e-03 5.611818219741381746e-04 1.636121268580312393e-05 5.266781939386719347e-03 3.484567296011215152e-03 -4.003394277489317338e-02 8.599915436350104997e-03 -1.480928104038884904e-02 -1.005018987482579097e-02 -1.137855842319285835e-03 1.456082544126715075e-02 7.439042096726690423e-02 1.015588490796881654e-01 8.002914549365110208e-02 1.036363936601508856e-01 1.656501960207739377e-01 7.657715122108124661e-02 4.627983842450833496e-02 8.159419167702135212e-02 2.910121314501363382e-02 1.944371646307442956e-16 1.022920729974655454e-02 2.029302311634717526e-02 5.813571798258904133e-02 5.993800287659981768e-02 4.770744194399550997e-02 1.157610155871353907e-01 6.836475753207055250e-02 1.115287382549148311e-02 -7.651938332725551073e-02 -9.539624609273529559e-18 -1.382662571426093376e-02 1.826254052815045117e-02 2.584768232134666832e-02 -2.939713077362959837e-02 3.455156407012310166e-02 9.552351595560919458e-17 2.096951806583476421e-02 2.162836568862853970e-02 -1.181386038714872695e-02 3.793265749214470503e-03 5.273797220202935954e-03 1.995357595855073302e-02 1.211833958407579175e-02 2.538756557626387425e-02 -3.426609703878602531e-02 6.344051446186535492e-02 8.831739196430964112e-02 3.095796765890015609e-02 1.684138477675971024e-01 1.089933248234459771e-02 1.144866620160276960e-01 -2.000628729501348215e-16 9.693751612479189345e-03 -1.669880077190872419e-02 5.708195018599082626e-02 4.145874653637636464e-02 -1.223661279381994738e-05 9.047608037357860464e-04 -8.085834797225640615e-04 -9.758867566806998101e-03 1.139222806150684726e-02 9.819633816582144792e-02 -2.445403888566608386e-03 -5.777577015135134351e-03 -3.793057781600612209e-03 -3.131814597494517877e-02 -3.815406068167678799e-02 -3.143121940497779518e-17 -1.274861849253577754e-02 7.658166574290315998e-01 -7.377441322127997227e-03 1.100556824033533387e-01 -2.433618431610206148e-02 -1.859371649441917546e-02 1.216522119920129125e-01 8.932973131256832788e-02 1.345007004043899737e-02 5.972543650440495511e-02 1.079168355257661555e-01 -2.072106578377272412e-02 5.070577759404278334e-14 -7.472383819998225374e-02 2.590184530107514718e-02 3.295173261829367848e-03 4.828707890409602438e-02 9.063072856665341237e-03 2.524009465848004938e-02 1.770078692325518024e-02 6.420431221350917372e-01 3.111677674527908444e-01 4.754602495532506135e-02 4.640663425228876149e-02 3.177651228938058875e-02 4.895375736767672986e-02 5.802910868457089588e-02 1.036243504904948781e-15 2.440122073919561221e-02 3.954511906907427765e-02 3.910176104523385709e-02 1.200480306091215345e-02 2.137025585307361294e-02 1.895123440022544736e-02 2.726010854615460235e-02 1.301470305027679387e-02 3.738709346518484522e-02 6.812684358663066986e-01 6.646711841687171773e-01 5.666380763442152579e-01 1.935428236860740198e-02 1.846815759049212635e-02 3.967847687075701235e-02 3.510764961400791168e-02 1.943878536149872341e-02 2.890178555811538652e-02 8.851112759486326476e-02 1.386048010325234732e-02 4.877297357158436562e-03 1.463145568803049697e-03 9.990411008136202906e-03 1.396693250759568775e-03 8.451274522518027049e-03 4.239150706993093966e-03 2.073487393033620133e-02 6.919286364749164218e-03 5.485603287559363828e-03 5.583787555902781918e-03 7.664335066277465636e-01 1.222705010659097535e+00 6.250225545420672324e-01 9.038636341368129490e-02 7.493989205590051439e-02 -4.411988876264009291e-02 1.820492366539870274e-02 4.911985797198348963e-02 2.353310918111007483e-02 3.784313328763084794e-01 3.881533711855136293e-01 2.789674518787053636e-01 2.822149924290296916e-02 7.986667840262518664e-03 1.506186118722313907e-02 5.922895418045053678e-02 6.710785682927676576e-02 4.859969086707661978e-02 1.070851203980063282e-01 3.886020044357429981e-02 1.033648341048071101e-01 5.617415129938540391e-02
2.022633483533786969e-01 5.168917543463649386e-03 9.319112234387588387e-03 1.075772214237011607e-03 1.691255360223473028e-03 2.493121598730784622e-03 5.177891725107923795e-03 1.719777701934342706e-03 3.899453417111802878e-04 4.335357802893388679e-04 4.010533423481305038e-02 2.633908974171118248e-03 2.081505067916528465e-02 1.471610435648089063e-02 4.047271541882869333e-02 1.481009329275713091e-02 4.239886273664024603e-03 1.808083805751996634e-02 1.878600159563490954e-02 2.367385831296085763e-02 5.044652765164743535e-02 -7.730265699707165525e-03 6.008858040997717975e-02 1.621179146121285927e-02 5.204038809716085012e-01 2.708175982190678677e-01 3.127565546374623251e-01 3.160925235136625711e-01 4.494130605626928676e-01 2.875726953787058893e-01 4.304486694451458151e-01 -1.278202563053218999e-01 -1.483180107175810676e-02 1.858316993979903424e-03 4.879844641176805248e-02 2.453895183845150943e-02 2.399092752504475984e-02 2.999427363165864044e-02 7.698098927521526691e-02 9.134497955487114879e-03 -1.251540092275728877e-01 8.207006918212737812e-03 9.769896230949969354e-03 4.636968246485522090e-03 2.729416289195690445e-02 2.469526677224875844e-02 7.550249021105812086e-02 2.613833775666672687e-02 6.066692429411190898e-03 1.546882279889784211e-02 2.683019024986756616e-03 5.100209921766062776e-02 9.360617747122892962e-03 5.165110990487063042e-02 1.286259814851965777e-01 8.342289373329277169e-02 1.115891684726216665e-01 4.315048100896717309e-01 1.031830396400054278e-01 9.630117876280275269e-02 -1.721601058369426618e-01 -7.542257484942072304e-04 -3.596331850024653276e-02 1.242950656487183531e-01 -1.089091042242876402e-02 1.081220512166482977e-03 2.553864541259432547e-02 1.085490322624605292e-01 3.923308047129574513e-17 4.634442683569749317e-03 2.187876753120459802e-02 2.156453013579982814e-02 2.143129306464996650e-02 6.595692060946005408e-03 1.226916152982391976e-02 5.358239167030838697e-03 2.381294066611772901e-03 4.184180905674015835e-03 2.699061313554228930e-02 1.397716723395896546e-02 3.849656535647191097e-02 3.282370731570141004e-02 7.838212923897236450e-02 8.890851556003348533e-02 4.975116088657320634e-02 -4.627031475432254182e-02 -1.553284788373998507e-03 1.091493359985041753e-02 -3.020806637995667268e-04 -2.015826850385962765e-02 -7.861969359772746913e-02 8.375360318421982975e-02 2.281470545710346035e-02 5.993543240464965088e-02 7.487268386351970950e-02 2.162518955085792130e-02 2.819260365197031118e-03 -1.117133862914877623e-01 1.252998560919872488e-02 -1.561442359801924751e-02 1.104978617766439949e-02 7.748345776982832472e-03 1.421172348767513628e-03 5.776812279333681278e-03 6.688142989719895234e-04 -9.596566179559340437e-03 2.736817496334810830e-17 2.215925761223736315e-02 -1.328223462247277034e-03 -3.202413686535933012e-04 -2.365500198214046123e-04 -1.251138087284077265e-02 -1.322609206658636495e-02 -1.009411654620511445e-17 6.985452331800828306e-17 1.933484506904339673e-05 2.556719273407941518e-03 2.291030301138597888e-02 5.887865903750953006e-02 4.017930126717498239e-18 5.519747957711410580e-01 3.045830997575180166e-01 2.728699031217957441e-02 2.621672621636260067e-03 1.216002605267876673e-02 4.777801174045726539e-03 1.191607890460511313e-02 7.876856103502555273e-03 2.280905224796835051e-02 4.757370458600362895e-03 2.827454445731299865e-02 1.549370528245090119e-02 3.983844159776732397e-02 3.351131193579087036e-02 4.026667975778500108e-02 2.197455426572763118e-02 2.418171988242779138e-02 1.019791607818261794e-02 3.107803632687215767e-02 1.432883286938210765e-02 3.640392507204795458e-02 6.557931798182846661e-01 6.701110756147986569e-01 5.261525239319388980e-01 3.204304570081458869e-03 3.911571033540784531e-02 8.479023945907956938e-02 8.487221421120592912e-02 5.809860941575806931e-02 8.255993027068149470e-02 2.257944715937243441e-02 5.482814933302125882e-01 5.494516082258114054e-01 5.095538591030047693e-01 1.089562529182414898e-02 2.340781233431023754e-03 2.473756660874636831e-02 4.768108566972343275e-03 1.622843012309474822e-02 1.368428513394101022e-02 6.010518680659160395e-04 1.199506070143140264e-02 2.967407225670217613e-02 1.612601485358571701e-02 3.199245070495092269e-02 2.195946945070902448e-02 5.657530414487107334e-02 2.244504534683757282e-02 1.704106194539514035e-02 1.665979453412450018e-02 1.990943905670200242e-01 4.044908329716135342e-01 2.960400770863063080e-01 6.920978431799208741e-03 3.509142424829053697e-02 4.955018788860427598e-02 8.840520606231849765e-02 6.020567310898804897e-02 3.803262900935564655e-02 4.606684305392998169e-02
1.097615953876167755e-01 -7.897057377734055528e-02 3.002802868328021887e-02 3.052460690867632095e-02 3.019779316459772142e-02 2.806373813745190182e-02 5.844947450610731704e-02 2.233291131900078108e-02 3.640223594339515867e-02 2.791151380993046188e-02 9.836267973957210642e-02 3.843341557762984528e-02 1.256085218121521727e-02 2.504347223819197374e-02 4.373261453956388961e-02 1.085441400164526872e-02 2.335658528273623982e-02 -2.953032183271656064e-03 7.110777202477297498e-02 4.522610683155124309e-03 1.889320027569117016e-02 9.857685445418861228e-03 2.413487734503531879e-02 1.396100358215727859e-02 2.643749787361725917e-02 2.458082834152507112e-02 8.966840572105698459e-02 9.676511888443607365e-03 -2.059971537876535161e-17 5.481950569569115153e-03 1.439448602803189165e-03 7.870918749542194989e-03 7.791520160402266252e-05 9.437063797125766961e-04 2.053616871327722943e-02 2.947438234435107753e-03 1.025040459308954391e-02 5.404122640582790377e-02 7.383631131622914179e-02 1.507394832467041576e-02 3.351058164607707140e-02 2.174252177936914637e-02 1.304340469204037442e-02 1.045625865367167014e-02 3.316628238766577280e-02 7.047505082309916957e-02 6.408130252525219561e-02 3.324909746417255979e-02 3.090096953918303885e-02 2.112574303723386768e-02 4.078619531846324614e-02 1.325472897523399633e-02 -4.659763881196667644e-03 -7.132562587022894980e-03 -5.623157927953358753e-02 1.832677554129578945e-01 1.133444032414800545e-01 2.016776331243210485e-01 3.657757016106667480e-01 3.615529541874255570e-01 -8.270417839011941219e-02 -1.589021239393379973e-16 -1.618811392042026481e-02 8.008554929724518920e-03 3.588993848155941357e-02 -1.426966663284441952e-02 2.923541001787707469e-02 -2.799862769974240567e-18 4.056150402355649526e-02 3.666581703221277744e-02 -1.731722828427144020e-02 1.356658082385404744e-02 2.486531180985783462e-02 2.791039281963122488e-03 1.196537591941260539e-02 2.648670354029741825e-02 -1.082321407852893724e-18 3.848762601704350250e-02 1.145034306747508696e-01 7.971096839702956904e-02 5.459290925199593997e-02 4.870572389201539149e-02 2.832650939300373691e-02 -9.068473390992278990e-03 -2.065004643492712733e-03 2.230375285020491649e-02 -6.110848211452521829e-04 3.302266591998866130e-02 1.737892402767920574e-03 1.748934966438782726e-02 -2.042514103497907255e-02 4.515420659360835226e-02 1.264150481976137989e-02 4.170438207471102027e-02 -1.508545291264351551e-02 1.710896517834967212e-02 -1.541374799124747286e-01 -2.218731766779400653e-03 3.253893768337934311e-02 1.894028203088129678e-02 6.252311886829546410e-03 -1.078718082495283212e-02 1.320775973856471103e-03 -3.418042243056200192e-18 8.425833763873959017e-18 -4.483419917970908261e-03 2.838673478683462984e-03 2.453913660628949042e-02 -1.762052480443847312e-03 -3.400789814369791636e-02 2.212678018754730448e-03 1.452798945359594796e-03 -9.080673437672227497e-03 -3.956410559475948691e-02 -7.393746880081444005e-03 5.131965706683794259e-02 6.137394298577560775e-02 1.617307625032628898e-02 3.991349493756071731e-02 2.175606292347745732e-02 6.634101400053831199e-01 3.373786799612476517e-01 5.480289002222513139e-02 5.817422291449456800e-02 3.129220372785786186e-02 8.781009667129494045e-02 7.282362470858255721e-02 5.699224410680280711e-03 1.994006370697767178e-02 4.384207662748663969e-02 3.974511067762190319e-02 2.757875293458036253e-02 1.853405937026168446e-02 1.931912643716146225e-02 3.938309753742356978e-02 1.221668457278333314e-02 4.114726166815148994e-02 6.647894122249526916e-01 6.705076999166955654e-01 5.728894082166802182e-01 2.951712553801799990e-02 7.870945495241186019e-04 3.569845477451634441e-02 1.909007399024484006e-02 1.355406723907682034e-02 2.700936993148430124e-02 5.123214629692354055e-02 5.244151045433449385e-03 1.401812755097910784e-02 3.225890329071359492e-03 1.343328015677582206e-02 5.065067656247748337e-04 1.481776194195303688e-02 2.019980411715791881e-03 2.082608733474381607e-02 3.449230788316520746e-03 7.371208835148780614e-03 7.203619846467557153e-01 7.918647670609665346e-01 8.213762978977880991e-01 2.706976541881000661e-02 1.873674078262151793e-02 4.027372257847312148e-02 1.621340845077119328e-02 3.676873790792555691e-02 3.554119571108227404e-02 6.103713743147609616e-02 2.488941663645086178e-02 2.360449891095392494e-02 9.304372675614997965e-03 2.759549945159036830e-02 -1.332228327375302793e-02 2.414739869000009365e-01 1.131119168437641376e+00 2.256016217554420999e-01 7.621066322131996550e-03 1.027546844391105524e-01 5.458203138198878396e-02 1.003866882229762468e-02 1.356178800878590079e-02
1.413872954968860007e-01 -8.810043115329302399e-02 2.658623067518348787e-02 1.495882350519830810e-02 3.049299489500960358e-02 2.471114231747567613e-02 9.071852556114934352e-02 2.088777171429188895e-02 1.266977839664048047e-02 2.982638273790452765e-02 6.466700534831697211e-02 2.507617339517657223e-02 1.536049285329094648e-02 5.281661177324772222e-02 5.822121877346821128e-02 7.726878227296138900e-03 2.982036337474603185e-02 -2.612637732640013533e-02 7.172745557569951136e-02 1.213062622506673561e-14 6.986888283862462135e-02 5.132009716786493751e-03 1.229170908994850486e-02 2.264639561250148675e-02 3.601459304247493776e-02 5.101099254998564536e-02 2.280745524638335964e-01 2.200800340567294830e-02 7.429339029592285301e-03 5.906735975730574849e-05 1.962093823577117785e-03 1.150963013477703835e-03 -1.027358060849810524e-02 -7.635413735688660782e-03 -2.554446643944034093e-03 1.522723582394965892e-02 1.621021178444735384e-03 8.923153250623929236e-03 8.889211171425005428e-03 1.558348775620458683e-02 2.609134946109012768e-02 1.379060171684448090e-02 2.130381120994273569e-02 2.455972339481244607e-03 2.523450710671936575e-02 1.988924287642205094e-02 5.876789729427738668e-02 1.265924245645360172e-02 1.103786301044741222e-02 1.366145586984720064e-02 1.765416008465585732e-02 8.957027741114455613e-03 -1.143118100731665455e-02 -1.033402847895186633e-02 1.518797440901143063e-05 1.439514503617914665e-01 1.074731035204782442e-01 2.254594999486042028e-01 1.426168629489060202e-01 1.551066663028352610e-01 -8.858473688528654966e-02 7.461293933902414521e-03 -2.440857124655732013e-02 2.988255808844381828e-02 2.998058019622702741e-02 -2.224689191193727697e-02 -7.163165037308906219e-05 3.197782560568129507e-18 2.838247735547996808e-02 2.090870769323678255e-02 -1.385436488402593234e-18 -3.393723035183913300e-02 2.572399057107030863e-03 9.922566288572283455e-02 -2.711180207289105512e-02 2.794229630791623606e-02 -1.117918593151964214e-02 6.012614079843645043e-02 1.506419863123318326e-01 7.695224173919980315e-02 -1.771360583415770374e-03 9.525161086955473422e-02 8.565533791645682515e-02 -4.815375873800276096e-02 1.296007708441167250e-02 -1.515245712094383072e-02 -4.030517668185427428e-17 -2.202742422351751037e-02 -7.381455084612563553e-18 -6.232379588430795453e-02 2.516938994055071405e-02 1.324350379718772057e-02 2.146895469851476963e-03 6.991157002268572063e-03 1.081289079112820496e-02 -2.665853600282000035e-03 4.588717186817679351e-03 2.439634756591104250e-02 6.406264894847207023e-04 -3.076719101576858798e-03 2.410135460065330765e-03 1.847331463157770789e-02 -2.866431619590568101e-03 -3.974343345269271307e-17 5.774116754475961018e-03 -1.874040255586640092e-17 5.268973654296437055e-03 -3.718383473378089350e-03 -6.697588183179264488e-04 -2.128183347127580113e-17 3.501466856600985583e-02 5.999119669750560546e-02 -1.730030630398944613e-02 -1.228583720634100415e-02 -1.286299901840778394e-02 6.203885752357417988e-02 4.411772374029529487e-02 2.160417402442819240e-02 4.057444697844828085e-02 1.139410814858738720e-02 6.736169105951462788e-01 3.875191737138015680e-01 5.867362839278984776e-02 3.790753454350923035e-02 3.245985980325481052e-02 5.587587022954960958e-02 5.903217410512415914e-02 1.602316157425127952e-02 1.947148087225500368e-02 4.053170099164836448e-02 4.098346808041381656e-02 1.896697069611176512e-02 2.913769887280747914e-02 1.689142700657769725e-02 2.822955023791690055e-02 4.774783907525381559e-03 3.087458723720830703e-02 6.615091188430123337e-01 6.775384920793598997e-01 5.508557734364490921e-01 9.503704160651514632e-03 2.180829052419798117e-03 4.776612438588253434e-02 2.520787385560222318e-02 4.305452415433156516e-02 4.802910876285028385e-02 9.418177209171361974e-02 2.378928714577344464e-02 2.680662488634816612e-02 1.368026441439348234e-03 1.971043085572937725e-02 2.782363286657485446e-03 4.794169645100682664e-01 1.145843316440130932e+00 2.975425426847014831e-01 1.542747488732195034e-03 1.075457635602999548e-02 2.057583938230352766e-03 2.095444749759470607e-02 1.348572303937599359e-02 4.221932541521567633e-02 3.155026623564535782e-02 3.450957877810178348e-02 4.451537831300261469e-03 3.805392801712612766e-02 2.650042820970895108e-02 5.398769600114491224e-02 2.064586064809258506e-02 1.637030525445042420e-02 1.148039163306619793e-02 4.265573653525747688e-02 -1.746191800274928332e-02 6.316773257378849316e-01 3.168876055849652595e-01 2.770351478493905395e-01 1.452924886802046955e-02 9.171519537086363083e-02 9.255314564250337328e-02 2.171708044103640156e-02 4.982878403997628020e-02
1.423265940399076612e-01 -7.149555584325706370e-02 7.148272287396555125e-02 3.753339657976691862e-02 4.666676883292073669e-02 3.940799097532518558e-02 8.170293334803281626e-02 1.401440129861034403e-02 2.710140620944048034e-02 3.291411428922649818e-02 5.174175524417006650e-02 2.830675559474126954e-02 7.573599581376825457e-03 1.093656327774343920e-02 2.144550335570469951e-02 5.841304782489604996e-02 9.147890575257415502e-02 4.284678897534037467e-02 2.235992180255069045e-02 5.745759583163650291e-02 8.263556306991504119e-02 2.860633113108417003e-02 3.115075282721186528e-02 4.848213612827293600e-02 2.208410732161667639e-02 1.400442970567719776e-02 5.797988453073129461e-02 7.011420606980293263e-03 3.555433967832841158e-02 7.982379190785178838e-03 1.912577697260072103e-03 -1.595123868705734110e-02 5.359972088640476643e-04 2.145542131620112661e-03 -1.267781837321191873e-01 -1.867366680404635715e-01 -1.373258380494504771e-01 -1.112244278507291614e-02 1.446435039899284306e-02 8.064901986275711462e-03 1.691111801798413430e-02 2.199817826503993462e-02 1.696345514570523835e-02 6.381443332931317595e-03 2.135944647191496265e-02 2.459330791282160478e-02 4.590728345032213387e-02 1.530222216887621757e-02 1.152230370422796697e-02 1.021093127314381607e-02 1.369847316052433782e-02 1.974530594002455678e-03 -3.136296363098380752e-02 -2.802055705193530477e-02 -4.049698771721245427e-02 3.994622839300220885e-01 1.032649962972497920e-01 2.897779694353331825e-01 3.007865795480157423e-01 3.591409681505321472e-01 -8.501277939799835315e-02 -7.412103647513400002e-16 -1.159265078563880176e-02 2.781942986047994696e-04 1.122409841238782707e-02 2.250295461485460769e-03 -8.847415640733011482e-06 5.660599535483774175e-03 2.449731980987945174e-02 -1.852004300640541681e-04 -5.928045758792009142e-17 1.097987330935755630e-02 7.623248940537077911e-03 -8.152306453743272525e-19 6.385504620352254988e-03 1.381689556166024180e-02 -1.344356322799608941e-02 -5.654453613022089077e-03 -7.435809746569348698e-03 -1.587843892307502433e-03 2.263876946096866558e-02 -1.555794728746629982e-03 3.234629282746069329e-03 -6.035631066623208321e-03 1.480556707620090534e-02 9.704823216102768935e-03 1.809911679912967752e-02 5.206028097582610786e-02 -6.409197874496520278e-04 -2.960061414170155752e-02 -2.517117111697435988e-02 1.063832690124200640e-03 1.511239155072678598e-03 1.620383347746547631e-01 -5.266341407790341156e-02 1.242352903145641492e-05 7.949003992535001728e-03 4.175903209832758714e-03 5.636137596578429195e-06 -1.051129552862023996e-02 8.128938238993632659e-03 5.095639964691212748e-02 -4.673051376245760356e-03 -1.525732321242137361e-02 6.181185085304264151e-03 -1.622453631183682146e-02 -9.390907664843890432e-03 -8.452775632435007133e-05 -5.347790543560653788e-03 1.439445832328935075e-02 1.169636983743808922e-02 5.861383994239679693e-03 -3.045273723429098300e-02 -2.451249802971989036e-02 -7.741880458182295519e-03 8.583150932056499149e-02 1.705386823200796820e-01 5.891512882629240319e-03 6.177982682115786689e-02 -1.142121417348849338e-04 6.280961352306506651e-01 3.771513142930650320e-01 3.874066000915813573e-02 3.111681661625968071e-02 2.636356445666630105e-02 4.290617737526421227e-02 5.963292219057331894e-02 4.399635382635240163e-03 2.166834176337640791e-02 1.656533493103099636e-02 2.445070208647099133e-02 8.566839917231172663e-03 1.706506543951094579e-02 7.823755094655835199e-03 2.551057889024815548e-02 7.961140445557049819e-03 6.400582659668031804e-02 2.643711080759237469e-02 1.345623910883119237e-02 2.344651471667378792e-02 6.034622695673386161e-02 2.552203316973933669e-02 5.609156819701850011e-02 1.380468341304325747e-02 2.356101076228948269e-03 6.811888823044640606e-01 9.568926783360899302e-01 7.218072863624755975e-01 3.593899709606204751e-02 2.162883903938215766e-02 1.072705030299330071e-02 1.554863788994145443e-02 3.602686969012759005e-02 5.477641840630784881e-02 3.894788836773575014e-01 1.373930506701452225e+00 4.587089528415073669e-01 1.898091883972551075e-02 2.010999206042667076e-02 1.727249540215499216e-02 4.641561051710407781e-02 3.357935728105164441e-02 3.726443422736320371e-02 2.729653085912209058e-03 3.057930210582553154e-02 3.269490056849234244e-02 3.909407242890940160e-02 1.430285943734271523e-02 2.429199011053774157e-02 1.566401922861523563e-02 3.469685507782634820e-02 -2.610053152893056228e-02 2.594170399868838128e-01 3.121750717534628183e-01 2.515006603052679646e-01 4.014940768443686636e-02 1.025814648394430678e-01 4.312651520324067150e-02 1.186336752341961287e-02 2.044704955576641392e-02
1.216288424770348986e-01 -7.570081774210728198e-02 6.663605980798063821e-02 3.090499253973987698e-02 1.909927116669695932e-02 3.229633511277896218e-02 5.654255428442080816e-02 6.273387236039765051e-03 3.482579178991698776e-02 2.996210329880505396e-02 6.191429622001291821e-02 4.785800354075219441e-03 7.464697887830287759e-03 2.333582164178132912e-02 2.583752213252894434e-02 3.551058810838361723e-02 7.062666881697957089e-02 3.297820043864392592e-02 2.652431210776528062e-02 1.346768387799677363e-02 6.799409699843329824e-02 4.149319324048443558e-02 3.773058403818332213e-02 4.539441563222519627e-02 3.207212455956854474e-02 2.796813266039835030e-02 -8.957806385300630392e-02 -2.472984365246593333e-02 -7.098692789662754132e-02 1.015763134067038326e-03 -3.611707303037603456e-02 1.029180729490868952e-02 2.895517296710285477e-03 8.421221326556788123e-03 3.886439756261809986e-02 5.048074724358893317e-02 8.589684008510768687e-02 2.858131319578255045e-02 3.844399161340004284e-02 1.993014915063521728e-02 3.296280358739760857e-02 4.097517190328996833e-02 3.252265152886255756e-02 1.239890787478833967e-02 4.662872790157134711e-02 5.667739534792758105e-02 1.220896432445740670e-01 5.115312469226836883e-02 3.125208406941534095e-02 -6.884421522362743065e-04 -1.469880466068322188e-03 8.945927763193282960e-06 1.447358064580127682e-02 2.731488989397308845e-02 1.035413116424569596e-01 1.169533367301344995e-01 9.600798226943853264e-02 7.936157602504555209e-02 8.895513347038987162e-02 2.923819879927610360e-02 -9.768848590348522265e-02 -1.034528129484458799e-03 -1.249211862931104769e-02 -3.605368974227243870e-03 1.974740574440712534e-02 -1.863805737844508731e-02 -1.165163195559200863e-05 4.545252828664792660e-03 6.344202922246508503e-02 -2.890302223448224309e-03 -1.301837719148411998e-08 6.274441137849235892e-03 1.450655627273346893e-03 2.017032838086761214e-04 3.563615843656527463e-03 1.843925473547394431e-02 -7.456991622114834094e-03 3.504523309881559488e-03 -6.723294304987046570e-04 -4.850401067369673747e-02 -1.786424478826972689e-06 4.757247923878541417e-03 1.279516421284727055e-02 -3.054564158243452260e-03 9.504032542175349716e-18 -2.612452054907149490e-04 -4.601837979149034363e-02 2.081016401901500130e-03 1.061297803295301045e-02 -1.615255398119733004e-17 2.988168713474746111e-03 3.034862055666100525e-02 1.008304198485299348e-02 1.858135807632111543e-15 -1.044654179695332297e-17 -3.539345412256084122e-02 3.911635738306110111e-02 -2.491653077031540358e-03 6.513734601325760210e-02 -2.378893917359428418e-03 6.316875450177670756e-03 -3.604696705149791998e-17 7.568823187357616444e-02 6.071792121400158490e-04 8.970908933205888960e-03 2.992479945294075627e-02 -7.160220503265585365e-04 -1.134581517915419710e-03 -1.390249269938225764e-02 -2.449714362829357730e-05 -3.480734637188204195e-02 -7.988405768199174917e-03 2.745038706312325633e-02 9.792395787305823579e-03 6.257215804790289748e-04 -3.076240481720337455e-02 1.948607435717869193e-02 -2.042739734477100333e-02 -8.959580204845914511e-03 3.606247053090339688e-02 6.512059117740057745e-01 4.255428401094348834e-01 4.767545117630986318e-02 2.310588660188155063e-02 2.285956693525682815e-02 5.497702434448561992e-02 5.485546594996545955e-02 8.670875097072304818e-03 4.164422418128081033e-02 3.056532075141967994e-02 4.955630764769373975e-02 2.196116768065528579e-02 4.406513554645836300e-02 2.179915689196879405e-02 2.313484842028122948e-02 1.293229453914719954e-02 5.392380272706052274e-02 7.359203026063106459e-03 1.012814875012721036e-02 8.540282929746794913e-03 5.613949428978236983e-02 1.440533425744517856e-02 3.869118766542332943e-02 1.849326816368232484e-02 1.283199796129980835e-02 6.753198146973024216e-03 8.531991618999006377e-01 1.072180194281283194e+00 6.184603212868383659e-01 4.021178380805632835e-01 2.175897886631208422e-01 2.110174224323277392e-01 1.373463687420886925e-01 3.059378262350359035e-01 1.254904220431966588e-01 3.900898861466746137e-01 5.530067858352361032e-01 1.005957734816905313e+00 6.507805788700983252e-01 4.535193847858547123e-03 3.639500730469231310e-02 3.688060694725145416e-02 5.671781356624222559e-02 9.154330404744264599e-03 3.097395192361358102e-02 2.592162357907777659e-02 5.262970160602940817e-02 2.685167112906368305e-03 1.482953922460159002e-02 2.509103487500306540e-01 1.779416862715159087e-01 2.995222499472420807e-01 9.314459911832901479e-03 8.361728662786380595e-04 7.004390783190819758e-02 4.140154237079873151e-02 9.857703712437826393e-02 5.284708383961368022e-02 1.280982168358303641e-02 2.819264636612708440e-03
1.449571566113353471e-01 -5.515735394154738791e-02 3.392324374239197304e-02 2.019173270370068843e-02 2.366085422687579098e-02 1.235413613417388548e-02 7.170306987362567996e-02 2.579677736892214621e-03 2.239018311618817814e-02 2.879530921649915698e-02 6.092438102173369785e-02 4.067482967060078586e-02 2.515475878764817860e-02 2.887660369071239769e-02 4.247592707023962327e-02 5.625199400026286572e-02 7.540801581933727349e-02 2.702359805702490500e-02 2.227809848741003906e-02 2.685761264832835574e-02 2.649050561099675072e-02 7.882091513873958066e-02 -1.434779254235176923e-03 1.363081437067903759e-02 8.807776575725560775e-03 8.366339088417525452e-03 9.943731337394855208e-02 5.042741420017132943e-03 6.090628164188624475e-17 2.582593526067900828e-03 8.467228526729642592e-18 1.294539584760508351e-02 1.452983168589806534e-03 5.579801122445333071e-03 1.701281945596454931e-02 -5.760665976128879207e-01 -8.196121520576886421e-02 -8.005633774475508224e-02 4.229629158506803543e-02 2.383801064139385445e-02 1.911121172732740264e-02 2.258334912386775195e-02 2.285544973666173499e-02 9.611570341397262726e-03 1.920876174758895036e-02 5.434620009970172333e-02 3.520406057199659128e-02 7.955026950691107893e-03 1.422592782966249775e-02 1.613588391039117234e-02 3.507229126493877774e-02 8.728124528936045053e-07 1.072952536746733625e-02 1.451178632736003019e-02 2.589555290880747943e-02 -2.038463212675290306e-02 -2.110563563183158811e-02 -6.529292273810094471e-02 5.796293077911942593e-01 1.258698929045598547e-01 -9.921217923783355597e-02 2.965649064845664318e-03 -1.451293358240086755e-02 -2.721541451974259992e-02 -5.059616132982077026e-03 -3.501968753158048175e-02 4.507453422317802497e-04 2.358838124514497231e-02 1.068790284777360461e-01 -1.122556192576352793e-17 -1.111121367990900183e-17 2.568806351617445791e-02 2.387975553776656296e-02 1.087100235658609686e-02 3.572801914823937885e-02 -8.533299865914824108e-16 -4.778777950951746713e-03 2.489742110917261483e-02 -1.966430409751218980e-02 -2.097574835893130027e-02 -9.558707101157931207e-05 7.720203105238379604e-02 2.381521203248063157e-02 5.751972467867597083e-02 4.136152970129463316e-05 6.571371345006510323e-02 -1.026225869997876225e-03 -9.768347690575640216e-03 2.258710380042308619e-02 2.776135128066871231e-03 -6.078915772186520511e-04 1.383452532994798242e-02 1.267347646209748650e-02 -3.334502846068833348e-02 -7.061644522145143262e-04 -9.463914488836495942e-03 6.674035757741681504e-04 -1.577161908374624319e-02 -8.706869665884064968e-02 -7.372139743772786239e-06 1.935378174644749455e-02 4.878160041004409378e-03 5.415034000902471016e-04 2.970387264897945620e-02 1.018587994555003292e-03 -8.066350437821811700e-03 -4.904782547766888164e-03 -1.266582130842136208e-02 -3.454619560835480406e-03 2.595635178919415351e-02 -1.276932824254888889e-03 -3.348564037329544718e-02 8.490903378554627977e-08 3.163182078987961804e-02 5.124166978652229011e-03 5.910632316779220602e-02 6.949503804634978542e-03 5.195124325467843224e-02 4.564882425504808612e-03 1.977639031761471988e-01 6.455434719832893187e-01 3.653955859741807033e-01 5.201788627870933268e-02 2.660266451389450143e-02 1.419939259761396344e-02 5.127278495352510790e-02 4.883696235739933694e-02 1.858798304026994253e-02 2.648891104729004728e-02 1.517523460618599128e-02 3.595451137630212290e-02 2.519039427240967258e-02 3.365833663251632107e-02 1.781569862795577205e-02 3.173680230272399316e-02 1.672879771836745244e-02 4.607200517538870971e-02 2.843320656280113462e-02 2.065285517187039066e-02 1.498760177745943160e-02 3.256188990468346367e-03 4.874210390284645644e-01 6.633830090455044015e-01 6.591661821339637184e-01 1.124211398222403104e-02 3.587399066436536277e-02 8.172376700336535238e-02 2.153302456506099583e-02 2.622298941946332382e-02 7.783752920266876277e-03 1.251948118556906603e-02 4.854846951192815820e-03 1.018237691734090455e-02 1.629809002967360504e-02 2.912056961191094792e-02 7.354600360250532765e-01 7.074690221854667005e-01 6.843815931990946400e-01 1.520538712608174130e-02 2.112974299394124525e-02 4.958138098809556499e-02 3.770155649712598339e-02 5.031036723132877031e-02 2.116357571121971030e-03 2.637293272403499339e-02 2.106499236775506712e-02 4.637034890272179188e-02 2.538455458467145029e-02 1.602617816415987870e-02 1.735042134067736955e-02 3.888818093776807983e-02 -3.822573295684056783e-03 1.550593235442813099e-02 1.184764241901335377e-02 4.578822202234188871e-02 4.798894401382373109e-01 1.485794960757099714e-01 2.696988989369802359e-01 -3.948056918877346777e-01 -7.327240045604681562e-01
1.402151890126873413e-01 -7.617320835994394190e-02 7.295967788948573562e-02 2.724419319930267147e-02 2.990033068525875873e-02 3.180732989273533928e-02 3.262999394270010844e-02 3.662259844874104638e-06 1.479736286639866776e-02 3.605713047203740196e-02 7.511727418493846509e-02 3.489820555276918618e-02 1.532878757720987362e-02 2.875766165832191468e-02 4.607845808196386200e-02 5.550054948954574047e-02 7.519450483079018199e-02 3.199207644360291952e-02 7.188957962453916203e-03 2.590071010973461416e-02 8.495556106173401956e-02 2.784094616225326127e-02 7.523002651647241190e-02 1.986455438742290303e-01 4.799360191271229842e-02 1.054954029790486358e-02 1.910358313039752995e-02 7.980065373139612686e-03 3.347100717566600977e-17 1.615860469997616993e-03 5.265474057124075458e-03 -1.264580211608314249e-02 8.856916450117748665e-08 2.057346320362898797e-03 3.138413330459145778e-03 1.839375479796064180e-04 -4.111762832771612142e-02 4.733479226501176502e-04 7.699432261688552749e-03 1.205283046708437861e-02 9.387559799471596744e-03 1.286190666842985247e-02 3.018226956247049148e-02 1.940269689213449875e-02 -3.206709832494616507e-03 6.211415340403634439e-02 1.774257735847033779e-01 4.503153301319726826e-02 2.428147221936468106e-02 3.105488705514685560e-02 6.716588468015518600e-03 1.659380665977897712e-02 1.640924652650456173e-01 5.943775834669769165e-02 2.856910067305420617e-02 6.500348519660796576e-02 7.340564212881961481e-02 1.575767258417603377e-06 2.346790873071444988e-02 -4.445184029896168121e-02 -1.029662975163362637e-01 6.047843835596680186e-03 -1.534007455177982711e-02 4.879627277729545187e-03 4.339254264048636456e-02 -9.769940683382452584e-03 -7.880968119723410566e-18 -1.918245467312769750e-02 2.747875566719544527e-02 4.465735358773543183e-03 -4.616452207578622402e-02 1.401530670226604561e-02 4.766354777713484639e-04 -6.517290749882560796e-03 1.395655696564589848e-02 2.241495935372588705e-02 -1.097609464385559778e-03 -9.524234770678687012e-03 -2.208146104260001741e-02 6.767832632202331819e-03 -4.100302061311615620e-03 1.003939135699942649e-02 2.508129450286981227e-02 3.313013105263926984e-03 -1.718429081533155711e-03 2.274985583642058842e-02 1.387068072035500159e-03 6.974723697615879681e-02 8.234096707578932028e-02 3.946145382416584024e-02 4.815537039284987358e-02 3.396347126130212779e-02 1.342197304215730566e-02 5.753780613685658069e-04 -1.523638572157894316e-04 3.562455678930469966e-02 -3.716371623347183763e-03 2.684255586917760539e-18 8.536575494695159208e-03 -2.189356828685280754e-02 4.166242020027840359e-02 2.246205954066384652e-02 5.075018598746745130e-02 -6.004026626436480794e-03 -1.462629088874576754e-07 2.153111395956215646e-01 -2.886690484689274916e-02 1.270354343743723968e-01 2.610280328328193344e-02 -6.829692085577283162e-03 3.505037858547272456e-01 5.598023069974478183e-03 5.542647406841140489e-02 6.128023471220662988e-02 8.555210026448181737e-02 1.421787734908815754e-16 8.642665439180761966e-03 2.704528220178353728e-03 -1.042314846951032936e-01 2.606178458163547346e-02 6.274861038510840539e-01 3.942486080553357919e-01 3.865201416874039508e-02 3.029996027611028472e-02 2.408043122825076141e-02 8.626293452638228443e-02 8.231732114260301159e-02 2.336827805874995903e-02 2.764059194676236203e-02 2.494794364136059678e-02 4.185486854317415489e-02 1.486471054615578526e-02 2.601718194623621819e-02 1.377437742777620766e-02 2.956686372700086801e-02 1.385111299002403720e-02 6.199600268706882689e-02 1.597826927844927733e-02 1.736274221465391315e-02 2.769082387637259932e-02 1.648728725024271868e-02 7.567209810668723211e-03 5.400671833265180410e-01 9.683130296364901302e-01 6.978205793051749373e-01 2.405420848952823532e-03 6.972553079256864350e-02 1.617558208393018310e-02 8.239496003471694086e-03 1.061980825213023260e-03 1.221716759038347054e-02 1.067091629926956997e-02 1.192871467774053729e-02 5.308397961773432018e-03 1.426823078106635376e-02 1.731864941542397276e-03 3.568596295243066374e-03 3.928037807091326605e-03 1.991458368738881213e-02 1.716931001973335397e-02 4.179005561545209857e-02 4.454602500273387167e-03 5.514651290011164386e-01 7.644048334113775489e-01 4.828952939845508885e-01 8.294589415643491628e-03 5.832808623936388409e-02 2.346978120422815728e-02 2.698096897954367981e-02 3.873057271399354268e-02 2.470880431224351664e-02 4.599661684677907592e-01 5.953506635046043050e-01 7.012700098650479497e-01 1.313535995825168001e-01 3.877402866967207262e-01 2.317946310590617842e-01 1.966513260325123136e-01 4.037951434397506478e-01 2.157799734518668355e-01
1.401594460222735372e-01 -6.073018466343929611e-02 5.631707092817829496e-02 3.545884038875157795e-02 3.629122006213462981e-02 2.744705584447618141e-02 3.694331233301617612e-02 1.740477711464872264e-17 1.251183121363099587e-02 2.260174995191456884e-02 5.176890511399891148e-02 1.011748803649752462e-02 3.722870848456190374e-04 1.074553772434490816e-02 2.048793705358363873e-02 1.436636574326522614e-02 2.536897055359916414e-02 1.057244548170131049e-04 2.077122777171867645e-04 3.429826122824953388e-02 6.350814625660851365e-02 3.774723204302091184e-02 2.484576157571824137e-02 1.073900193226231051e-02 1.814181954129890501e-02 1.425284408053681222e-02 -1.720455638570552240e-02 -5.764768858702442461e-02 -8.216652783088125578e-02 1.043377639229239873e-02 1.094019189203710059e-02 1.316599537144412697e-02 1.193883068067223309e-02 1.475013346613087556e-02 3.744944311674155285e-02 1.477674875259605766e-02 7.928740833023017413e-04 1.183348404616039497e-02 2.862400089546312393e-03 5.028743655560941050e-03 -2.368571630058128371e-03 -1.069348658871124846e-02 -1.102459124926882458e-02 -1.803895926203980538e-01 1.180394931630887601e-01 1.846569706499351649e-01 -2.563431385387798800e-02 -2.391074261892556121e-04 -2.644839393835202689e-03 5.617464771614988361e-02 5.618045649556008964e-02 3.330595223026140521e-03 1.254832638556419890e-02 1.194335852203906416e-02 4.348900647654575097e-02 2.436037785434239100e-02 5.334002089089363852e-02 3.772413393596645709e-02 7.488458765754631152e-02 4.966074435415428895e-02 -8.769329214452584176e-02 1.260677587940484152e-05 -1.739033140869205532e-04 -4.258739515649092970e-03 -5.417135218222325415e-04 7.336580565246996133e-18 -1.344831603663763399e-03 1.215601899732947182e-02 6.591176090535708076e-02 -4.832794235070149311e-05 -3.402590625725158903e-04 1.731575739175177445e-02 1.730451468821397193e-02 6.092460618177459805e-03 -1.592164005816493547e-02 9.397421898965068604e-03 -3.681049877214559349e-06 3.230881996779888089e-18 -8.651233165228684394e-03 7.063386088532943186e-04 -4.147578112563589245e-05 1.368120766336844082e-02 -1.153917276321315672e-17 -5.051404725880114270e-04 -6.987554047883011996e-04 -6.307256642661512348e-03 2.463070392634618324e-03 7.785418682486698778e-03 3.077004066326953502e-02 6.040416683820049037e-02 5.603688981275890086e-04 7.682978512646973512e-02 -1.849616554343515535e-18 7.971921032571554697e-03 -5.619746009216207964e-05 6.104894035081714909e-02 2.190591415938104684e-03 1.199829252390567560e-03 2.256472979345661156e-02 4.259930186215599678e-03 -5.496384611716524649e-03 -5.511423215719316038e-02 -3.332132454350242301e-02 3.212708890217682578e-01 1.608517247217766503e-03 -1.464645020306956369e-01 3.118978900287791914e-02 -2.812370265448042861e-02 4.829030890872487713e-02 2.536039394573893005e-03 -3.494032982976076618e-02 -1.390794924028061763e-02 4.327270675627274861e-03 -1.873695315343119894e-02 1.101695967184256400e-02 1.811717240611657566e-03 1.855632878612290329e-02 -1.010923253187937789e-03 1.884185512774666157e-02 -2.302187693907872812e-02 6.350761768643581329e-01 3.637730229953626826e-01 2.903593814188131644e-02 3.069836971720590196e-02 2.070357783865499346e-02 5.100175093356319050e-02 5.574399639783229887e-02 1.166311790748156794e-02 2.199084335051129310e-02 2.672750658133424967e-02 3.361107993671788585e-02 1.903126725589211407e-02 2.977224891466100781e-02 1.537387902468652021e-02 2.259473201021321154e-02 3.391198866698336145e-03 6.554569829813190925e-03 6.297453476167613534e-04 1.033390066472553506e-02 1.002415632902849650e-02 5.083847174239942285e-02 1.052734266752813376e-02 3.212037394918452332e-02 1.698213787368600333e-02 1.132663124953134891e-02 7.173869253079556696e-03 7.099037884953832700e-01 1.374068669837223089e+00 4.865605421384601725e-01 1.193697265640042155e-03 1.380531823285758609e-02 8.418143667030903929e-03 7.693247984069034005e-03 3.796099574298679213e-05 8.077934737549044564e-03 2.210602940708057337e-03 2.570495781431777257e-03 9.482536228277810390e-18 1.495529253654931043e-02 3.605107567323892915e-03 4.971381771041658570e-01 7.410831551107368576e-01 6.703031960370841036e-01 5.141465193138684553e-01 5.410186918815942381e-01 9.188962241721990720e-01 5.307555188443423555e-01 7.074243054022493205e-01 3.867870297363750209e-01 1.418292883347858566e-02 1.524213068819672660e-02 5.395248815288140008e-03 1.508858023691632931e-02 3.468832192702077377e-02 6.749674197855250191e-02 2.260421791914041592e-02 1.180142022476459024e-01 1.112517935955964626e-01 5.873486667686407536e-02 3.534706497816757914e-02
1.561467563354830834e-01 -5.824418486456571509e-02 7.792162733878381375e-02 3.670855342415639544e-02 2.614186895767413310e-02 4.066445470545797247e-02 1.381491341618792668e-01 1.736238063486512592e-02 3.290787550385967930e-02 1.624093070103472397e-02 1.181673009605719027e-01 6.798380263440403914e-02 2.136958070203325011e-02 7.262661142771102349e-03 3.714239967903013029e-02 1.218356529630394529e-02 7.241782245627981229e-02 3.836833429312241917e-02 5.095451547995183926e-04 -3.428583563868398132e-02 -2.475503423066039571e-02 7.807527289361820833e-03 3.526579941860395057e-02 1.729561143419244135e-02 2.010197845953141244e-02 4.667546297345052941e-02 1.108416489448163372e-01 1.032563405799689915e-02 5.340908501958310522e-18 6.271814887038150302e-04 3.643483705269926105e-03 1.611574842167486522e-03 4.324096544168172691e-03 6.065698356433790969e-18 -4.541228560906483691e-02 2.277871616556785422e-17 1.938866870035211784e-02 1.539913462144060045e-02 2.717635319396076443e-02 2.511621525576783823e-02 3.266007257621958426e-02 1.198802181757133156e-02 2.900365802172159835e-02 6.214648820195224806e-03 2.452749081627769723e-02 3.934419917259893845e-02 5.952581958910645177e-02 1.839369610450435413e-02 5.585170351248455040e-02 6.722214352979194278e-03 2.934500753960580036e-02 7.656419791329264134e-05 2.925905963644807525e-02 2.093195310520324509e-02 3.439337756888512926e-02 3.620005940763298957e-02 5.424587819921330989e-02 9.532874280061653938e-02 7.403383424047108030e-02 1.545807874042826394e-02 -1.037303015444722143e-01 -2.134554503020498386e-18 4.208654868047306818e-02 -9.158556708322462825e-02 1.783360825891512624e-02 -5.291471313743255356e-02 -8.370047570636059489e-04 -1.512104551063674952e-03 5.945913162552727421e-02 -1.230690481739501151e-02 -1.126480012188546195e-03 2.279651256321242603e-02 3.257671269883114298e-03 -1.691910372354507933e-02 2.461825044775975413e-02 5.962357972668299796e-03 -8.821935199832001140e-03 -1.489142133640369847e-01 -3.654595639960228626e-02 -4.043993018135192546e-02 -6.750474036857591342e-02 -2.499101192103622693e-02 -7.820809931187195378e-18 2.662457287579303097e-02 1.847214365687055784e-02 -1.040080340790230321e-03 -2.255387115171085858e-04 5.985564998957790261e-03 2.168820582241283088e-02 3.026789145789660421e-02 -3.226685360965316635e-02 8.892913553886695743e-03 -5.742348457983886129e-02 2.011296574374898428e-03 1.390996166752832969e-02 3.871970348390455646e-02 -2.285167710154290682e-02 2.063696081703866175e-02 5.874353555383765579e-02 -3.632625081619880042e-02 1.629392476472944773e-03 -1.339113082417732123e-02 1.120797560354597124e-03 1.056933092608914378e-03 -2.566975516115421047e-02 -1.575308158307876826e-04 1.831349609806882939e-02 2.873736195296614296e-02 -9.976511181045632452e-05 -3.937800043139619353e-04 2.453005981855748471e-03 -2.282693163430266070e-02 3.540484107047612520e-04 -4.232023067960000162e-02 2.129299822914552248e-02 5.178009277574342544e-04 4.816305404510445171e-02 6.537033520827139939e-17 -3.176431201283557056e-03 1.032285231236533691e-17 6.533403557362696956e-01 4.010704742891830765e-01 2.578280347641075457e-02 1.464165518374940626e-02 1.960909513860171344e-02 2.930805121186321263e-02 4.774628067095812445e-02 8.811473031933560185e-03 2.055728943548420715e-02 3.084490534719630875e-02 3.152804691319613384e-02 1.391602290430450639e-02 1.889748452460767239e-02 1.094697545983064173e-02 2.545453874079820281e-02 1.182374699722435821e-02 3.957567381404231838e-02 2.027379959878609916e-03 4.373898359177201822e-01 7.490407023190587266e-01 5.908419701623924825e-01 2.733759919029069528e-03 2.761812829596037647e-02 1.148494949814372890e-02 1.505904022794398410e-02 2.472168957239474774e-02 3.337551802387239003e-02 1.815852103318746011e-02 1.149222777821427843e-02 6.028204258004246988e-03 1.952209343749323686e-02 2.227821242229248433e-03 1.426809961242728186e-02 1.026058332947064500e+00 5.422389572434187510e-01 7.764790220980347657e-01 6.715707423695228655e-03 7.786227363573833965e-03 2.164214662961178384e-02 1.272725792145946826e-02 3.851622864783881689e-02 2.772784956325549666e-02 4.558962884560669587e-02 1.189757284114758076e-02 3.745732756698529758e-02 2.946197209783256443e-02 5.905878357502696019e-02 3.923058129181661302e-01 3.890440136119576731e-01 2.638512167576553824e-01 2.029230760988051316e-02 4.247127702448232162e-18 1.141635006031343699e-02 3.857642884697440322e-02 6.133789232539989256e-02 4.854206677237813544e-02 1.179384124840916481e-01 6.886848657331043344e-02 8.189501126182791024e-02 5.440693878989609489e-02
1.348653337788887907e-01 -7.296324487071254028e-02 5.850667005137095977e-02 3.007954625899164972e-02 2.429843481045963760e-02 1.921123571210098940e-02 7.462086402393831863e-02 2.319437367033170411e-02 4.191929363831482097e-02 2.947056952731553700e-02 5.296456175498843316e-02 1.865215418130415984e-02 6.116388392628341678e-03 2.109979958866167390e-02 2.274765685214182390e-02 6.570784208647736935e-02 7.190951957628392843e-02 2.083949423143058657e-02 1.292807979218757930e-16 4.221899315014973619e-02 5.809069989207078788e-02 2.975440622722265449e-02 3.132038578230953579e-02 3.748261711433091559e-02 1.961925612710472416e-02 3.168169418967389794e-02 -9.020631566491381370e-02 -4.824098829655476539e-02 -9.335412181549930521e-02 9.562032376475908214e-04 1.531045636416650257e-03 2.429401380025042988e-03 9.229608004735320899e-04 7.783334194428500469e-03 4.567807257985197905e-02 3.496613866357231593e-02 5.633389728435689303e-02 4.441856254525288622e-02 -8.316216821622063327e-03 3.558938083333712532e-02 3.115585989775608924e-02 3.157288037282218740e-02 3.171330114558094937e-02 9.855398824888788809e-03 3.892658311405426674e-02 3.122429854256357842e-02 4.819753359004590870e-02 9.994433372732163326e-03 5.514120662067023559e-02 4.938750960516095911e-02 3.031797083684161301e-02 9.569424703627154791e-03 2.588624602510892700e-02 4.029323270185881239e-02 6.467822779009885226e-02 4.407450098641210734e-02 7.992544341661637908e-02 8.089446020617128974e-02 4.942754548265302500e-02 1.170900798564937229e-01 -9.661574010170762006e-02 1.026601654758247870e-02 -2.164964227630665830e-02 5.150372264637025901e-05 1.565457382589671362e-02 -1.324198885501762062e-02 3.719928642593540308e-02 1.063607518854207436e-02 5.999253374834328384e-02 -1.429808918883475304e-18 1.330689042433730318e-02 2.057867147507132168e-04 -7.887178587552477231e-06 4.432489260962321591e-03 -1.116957526142746231e-02 1.154072560428302287e-02 -1.599799908786810748e-02 3.580309818729292964e-03 -1.909195423057418600e-02 -1.882086254569015349e-02 -2.821604294439139645e-04 2.914093172137838944e-02 5.870203905675465406e-02 -9.263202937840288504e-06 -2.858698686711233158e-02 -1.487499592566219296e-02 3.859045891361656110e-03 6.323325840344927624e-03 4.569695343739892807e-03 -9.147084890173528410e-03 2.229483098826868123e-18 7.779490936705062143e-02 2.755932846808836073e-02 1.468073971556776731e-03 -6.044723485835643527e-05 -1.842234128867301438e-02 3.425111791402335799e-18 2.051250989554184723e-02 8.398374461360572085e-02 3.836294844443560031e-02 8.644311163911077428e-03 3.069886261617464987e-02 -7.738394781465764150e-02 6.066447442172371538e-05 -1.672747316553907862e-02 -1.370479412129237606e-03 -2.039787313905947613e-03 -3.791087980611312219e-02 2.563938336554586958e-02 6.306972258450253832e-02 4.824737402394975450e-02 5.428828318782336591e-03 2.330189867213875929e-02 -3.800929197597624487e-02 4.021417612067445559e-03 1.395005213088251035e-04 1.202837018921002095e-02 -2.117354542021239808e-02 -2.114131182740619855e-02 -2.335146971969163585e-02 6.469328644148497620e-01 3.879316563983341015e-01 4.477806327678129916e-02 2.662997656752002715e-02 1.660832871750319087e-02 5.633417806026948249e-02 4.030103767388131669e-02 8.800758326774937509e-03 2.188660783567027493e-02 3.361396548593022920e-02 3.922725033459117244e-02 6.694845506969819907e-03 2.470260906230980005e-02 1.696251770764065775e-02 3.243685220441118455e-02 1.540137066716796427e-02 6.645751764878787826e-02 1.020333211005349482e-02 2.077039610193467598e-02 2.407593228426390883e-02 5.428037038533856801e-02 8.189852240181370521e-03 4.788772086116164367e-02 1.604237038031245538e-02 1.758411053030465784e-02 1.257790827213541886e-02 7.550275098063425894e-01 1.218030169018498698e+00 5.837747643377119156e-01 3.504165698467004431e-01 1.936673807812138992e-01 2.291960534927805215e-01 1.450098443802743664e-01 3.846912362097818505e-01 1.267922581159336981e-01 5.283990384778058935e-01 6.131069700724511851e-01 9.685738188181913788e-01 6.566829668287004207e-01 3.671677141676164130e-02 4.143153723764373547e-02 1.776009606319950418e-02 2.400469458123558053e-02 9.056592933287347389e-04 4.711385154044884366e-02 8.696072942466653855e-02 7.103492422186261090e-02 7.021730424661870584e-02 2.950714738457911324e-01 3.890050475704464050e-01 1.901776765758415588e-01 -3.373782643829975614e-02 1.338549717488776367e-02 4.329144071707651670e-02 6.368091861586769320e-02 7.028304682773667267e-02 1.137461784030017603e-01 7.691083652433598761e-02 7.887067017809698544e-02 1.081274535627052835e-01
1.435294139680170222e-01 -7.343070596520341609e-02 7.957904624433048690e-02 3.752380198307578990e-02 3.526441751003822622e-02 4.617738552495975063e-02 6.268386903135493571e-02 3.865164925126716045e-03 3.501829826880724361e-02 5.885129469304369726e-02 7.900932387110252975e-02 3.322232663897527888e-02 7.555912144210070666e-03 1.581608078434862050e-02 3.337486726975825296e-02 5.513633103851344613e-02 7.681524126126609842e-02 5.636662008473309476e-02 3.025892438574744742e-17 3.641999247034461568e-02 7.488474616714196896e-02 6.939260615072156968e-02 4.846052465047365199e-02 5.859738444159545701e-02 4.053989276725337987e-02 3.788709399070027545e-02 -1.682775248860280126e-01 -1.227765553022169137e-01 3.484729570867158757e-17 3.243790682876807052e-06 6.158769102724087618e-03 7.739576358189532469e-03 2.941222813307732193e-05 1.739196137401159692e-02 -5.398478498516495749e-02 2.926934250641600950e-17 5.767678603391572859e-03 2.364660311844343516e-03 1.364062185602211638e-02 7.155022675431083924e-03 1.005845576649227503e-02 1.094273028528985024e-02 1.453301421895464990e-02 7.817520867932364545e-03 2.779590030381412155e-02 4.173019758551416158e-02 7.347621230611099674e-02 1.870221098073285382e-15 1.180425236808710365e-02 3.729610934696916263e-02 4.559692957040478722e-02 9.979701596400519792e-03 2.327507001418782553e-03 1.591625078347806352e-02 4.991015040888244908e-02 1.734379447361606916e-02 5.187885574013904183e-02 3.856895532081251743e-02 9.000364811841110591e-02 1.415011066599740075e-01 -8.420021562163375328e-02 1.434135111117336610e-02 -1.731661273921157596e-02 -1.652790413166180250e-02 1.966152384125452110e-02 1.034213336577201175e-17 -3.434507375193395109e-18 -1.186491077487935577e-03 5.323181785233185165e-02 4.106487691274329083e-02 -6.574895593838235853e-18 1.307790768940481595e-02 3.038695534347113109e-03 -7.416418674060016618e-18 -4.304144992112792292e-03 1.472100695665046893e-02 -5.212081195286380302e-03 2.412310557609183195e-03 -5.702598226264400140e-03 -2.494480171452156947e-02 -6.173104878428244645e-03 1.991464672415769327e-02 3.516838474460782377e-02 -7.301173575620227042e-03 -7.856289774774235027e-03 -8.958593248582832208e-03 7.941501498592037564e-03 6.622824533014375938e-03 6.697079588911088177e-03 3.405596278489387424e-02 -3.984282484532641888e-02 1.324057092747983611e-02 -9.672620122263231269e-04 1.033070815143973236e-03 1.222619957848854709e-02 2.996673395811099219e-02 -2.338095378612823922e-03 8.543521981525433948e-03 -2.319258205737875431e-02 -4.643153883090116348e-03 1.968033420455682966e-03 -1.571753740162266869e-17 -3.990955049834239189e-03 -9.539841549940142329e-03 3.802663019264258121e-03 -1.019100439208795253e-04 -4.258593883583077118e-03 -5.488218271022967092e-05 2.172636203904821909e-18 1.433595536846988573e-02 -5.652131747919019784e-05 8.851591563973189997e-03 -9.185014311432156998e-04 5.602137602623646212e-02 3.801025111709146684e-03 1.188804412032800576e-03 4.003708036127017617e-02 8.613544196346137327e-02 3.098301710905150275e-01 1.564032839224771196e-03 6.263572077052830256e-01 3.659573428470007106e-01 3.538997586918005145e-02 1.417651473663415589e-02 1.577804796181155739e-02 9.654554926854035102e-02 6.492740556768322735e-02 2.381574749453801987e-02 2.622433902127810010e-02 2.185267053321868561e-02 4.155201016981553003e-02 2.301629973871229667e-02 2.768624690441761488e-02 1.888834266754602087e-02 2.650541478582851482e-02 1.444086166747039594e-02 5.894408540666692953e-02 3.246898257604243290e-03 2.022828887783122490e-02 2.281220079086574218e-02 3.928416138408274277e-02 1.493928634959307104e-02 4.106829272205534537e-02 1.392173849080929546e-02 1.588069251196516998e-02 6.838526791240332955e-03 6.065852462733932748e-01 1.305138968563460899e+00 3.797673756679415535e-01 1.978471349278988854e-03 1.684965067667408556e-02 3.771849546070741004e-03 2.062895145700424301e-02 9.712921624402006682e-01 5.279119506514701188e-01 6.752041187918261800e-01 9.587792723426229874e-03 7.100252749394984908e-03 1.991259751107188336e-02 2.549023796888204990e-02 3.349431591077265541e-02 1.791120800031809762e-02 3.152538407992066388e-02 2.036297937951288533e-03 2.902903797658287766e-02 1.967568940387440929e-02 3.705321151960482701e-02 1.080689406196522206e-02 1.964750930170445359e-02 3.496779298319579316e-02 3.240974892171173682e-02 -1.536603541567267868e-02 1.428642360063782368e-02 3.121999460959100359e-02 6.420887503114067196e-02 2.051271675187302246e-02 1.021817312494741475e-01 4.151835564183803395e-01 5.856819258183786348e-01 8.828573247529538648e-01
1.347833517356392674e-01 -7.909682766274110655e-02 3.011260812719672442e-02 1.746840684824368939e-02 2.098691720675166170e-02 3.172994801596337733e-02 5.752930884916555854e-02 5.012881454610589038e-03 4.018403027706986957e-02 3.057286570189647959e-02 1.008270711627892308e-01 3.338678666609280060e-02 8.576125553324006820e-03 2.481287768256295462e-02 3.170745677142695201e-02 1.064705835157678422e-02 2.879665932855127175e-02 -3.489618919363002975e-02 8.341082725866723335e-02 2.488429487497299785e-17 3.859135822416633749e-02 2.356690978944197060e-02 2.581788416651498250e-03 6.871075971395026842e-03 2.302175025678903503e-02 3.663789329863278987e-02 2.328346864430709540e-01 5.232890859698086788e-02 1.811735899133334882e-02 2.342324162867374243e-02 3.713656566596763853e-02 2.574760084327488638e-02 9.868534871073916803e-02 4.106710962314228525e-02 4.479606296956316867e-03 4.748648980064436136e-03 9.003804177649798318e-04 2.846411673462410485e-03 1.062483432504866127e-02 4.179290057425088557e-03 2.248388268380001678e-02 1.127377859169147352e-02 1.772779028899557421e-02 1.167407206931652883e-03 3.963464726509305716e-03 2.531871775738098965e-03 1.728143191508525744e-03 3.085511871743272516e-02 6.648580562156738982e-02 8.362002175471307946e-03 2.330154676518410642e-02 5.210731733434393644e-03 5.910517079677643504e-03 1.093005017667074072e-02 4.605191073367993770e-02 4.163003884119862580e-02 5.993591273840000178e-02 1.395941182860840624e-01 5.259592610012696345e-02 2.101450069585438993e-02 -8.921419378179468174e-02 -5.326435878910910221e-04 -2.285311994502365815e-02 2.770139046379659423e-02 2.451442759201663199e-02 -2.720679349938649738e-02 -1.257799169136171577e-17 8.513952133337452169e-03 -1.807707554613686854e-17 3.590360194639893865e-02 -3.199249415268240731e-03 -6.002208781737546853e-04 2.115483167116970469e-03 4.879313491455970764e-02 3.501529505184894971e-03 3.288409503685193330e-02 -3.032808711618806621e-02 5.530935125501041522e-02 1.213897847161100746e-01 1.576879011661585783e-01 3.581159790363221129e-02 -1.763518447953558999e-02 -5.971369968587430553e-18 -2.269872082432482230e-02 1.985416772471664676e-02 -1.714166183791420231e-02 -1.321271484861060233e-02 -1.887887090353567235e-02 6.733718133572916265e-02 2.559959560824513758e-02 -2.018708604043093630e-02 4.255962405729976639e-02 7.527821718274299578e-02 -3.517195713307671480e-03 6.289685369783520885e-02 -1.372645939647746186e-02 -2.988747367623690102e-02 7.049200401459204679e-03 4.785573646174866996e-02 -3.611181843695332114e-04 1.637187439852174145e-02 2.058144094114493922e-02 2.850362991084163470e-03 -3.089251486705240127e-04 1.128568393988101191e-03 1.622461195379254831e-02 -7.712530451914710941e-03 3.844096308920694205e-02 -6.373040060539969694e-03 -1.067923380786852056e-03 5.427837214982346448e-02 -1.531902223369650123e-02 -3.423879149342784131e-03 -1.987697948990982053e-02 2.292642945600793097e-02 7.205919647301464899e-04 5.946490542522174516e-02 1.324595888745052318e-02 -3.972459466674759833e-03 8.487441746731612391e-03 6.211697001181005762e-01 3.234381663949623120e-01 5.536769200677851138e-02 3.116293501537734448e-02 2.892560133093362643e-02 6.149773955808338771e-02 6.148442589666615332e-02 1.456390263190322949e-02 1.990336807714486109e-02 4.705696648925550285e-02 5.043047674918010176e-02 2.479201450150336772e-02 1.698369472298855848e-02 1.399354270831945982e-02 2.427871169822508335e-02 3.011808717267621099e-03 2.737250663905575060e-02 6.011965745285731133e-01 6.248207846061575310e-01 5.385811593202308734e-01 5.864363545331157009e-03 2.332331758338876491e-03 3.925800641774051386e-02 5.495358288236105310e-02 4.069958896808090892e-02 6.319640447065434652e-02 4.595500442183906437e-02 8.153654143525061607e-04 2.986481088311543439e-03 4.143347417905077429e-04 5.390470832046718487e-03 6.858154613858641335e-01 7.656704062669014155e-01 7.825351801966278220e-01 2.098770097834496903e-02 1.297794795811619932e-02 1.152489026688019273e-02 3.975900261257484490e-03 2.569281287422383458e-02 1.998670559142560685e-02 3.533956953920742994e-02 1.708149948237367918e-02 3.837881712039913767e-02 1.121730529008899309e-02 5.110377427579285148e-02 9.544972409952451331e-02 1.488768822701372341e-01 3.902540305077388250e-01 4.409357482942613538e-01 2.670630072844127345e-01 2.134036628498350505e-02 8.396647939450682316e-03 1.018332126925989081e-02 4.350948896945087269e-02 8.124609667000738666e-02 4.882030247402677214e-02 1.205501236389486946e-01 4.019440462088033628e-02 7.455124912926705538e-02 1.758028554243831199e-01
1.418259302317791959e-01 -7.326899106879342094e-02 7.825705143255787255e-02 3.924211674638923564e-02 3.952027340271627165e-02 3.331969813703668237e-02 8.213933022619801516e-02 8.288334000668938495e-04 3.268878693596664192e-02 2.903126505839732918e-02 3.698017703253742711e-02 1.018916076356592322e-02 6.954933057971520761e-03 2.061484369230276795e-02 2.735587283673742223e-02 2.948780420119790149e-02 5.171519837772405731e-02 1.014421268171204760e-02 6.608563622532750185e-17 3.804592075062082634e-02 6.051836508292587902e-02 4.727540502500571923e-02 2.780392049737864801e-02 4.401524205029985065e-02 2.832277358288937466e-02 2.187493004119824697e-02 -1.884747811428804515e-02 -2.267493065029082266e-01 -9.809537546508080574e-02 1.585224334280456449e-02 2.364713719412678272e-02 3.555440274973940828e-02 1.251862883488991801e-02 1.303785047862952330e-02 -1.764670727367146363e-01 -1.040960762914934640e-01 -3.163354445582355207e-02 2.662203696023886235e-02 1.077541216703626833e-02 3.027165187472659683e-02 4.015650362359979991e-02 1.823662349947697756e-02 1.700883160097395666e-02 1.702990541405266858e-02 3.419297642873233428e-02 4.407899320750593292e-02 8.814220920266394033e-02 2.789306350652013058e-02 4.769966108566131568e-02 1.861929395208537091e-02 2.180203007022765704e-02 2.945453167643317182e-17 8.624828197901146173e-03 1.433109832148318254e-02 6.383042077938577696e-02 3.231553111596764671e-02 5.068681452823980971e-02 5.229757673395789980e-02 5.672768476453365211e-02 1.626394957508853212e-02 -1.003529413516356245e-01 -4.558935880815440539e-05 -2.211512272151096650e-02 2.999354517780713353e-03 2.153547710214289909e-02 -1.612572754434096626e-03 2.268912584812171444e-03 1.687509553714570606e-02 5.140534757201184468e-02 3.266623273332294741e-02 5.657753614517033354e-03 -1.370813347465892670e-03 7.804492426082673164e-03 -5.057643687053494513e-03 -1.295521108377939261e-02 -1.935758194363855836e-03 -2.161898426857849003e-16 9.695122213473558737e-03 -6.282185725098583633e-03 -5.998330000282564578e-02 -6.704223291847310952e-05 1.292186002041042885e-02 1.311504394514726238e-02 8.210825730060107575e-03 3.370987616416828373e-03 -1.197923275685252274e-02 2.336078727519723397e-03 3.860162267074301358e-03 6.056349234501270766e-03 -2.275082393489635455e-06 1.151907793738697497e-17 -2.691185079017188037e-02 1.317542428502443530e-16 2.456682663096452016e-02 3.154160875495968144e-06 5.175018335009265398e-05 4.755094770066350951e-02 7.600254404261006479e-03 -1.165447837479475102e-03 3.763820401388884990e-02 5.464408113683829303e-03 5.748532251733852461e-02 1.887522256466829182e-02 2.027253209709718382e-02 2.046596115596273704e-02 -1.338526370965027181e-03 1.275297152109135940e-02 9.798547499325431509e-02 -4.656401081708332057e-03 1.277942058797286971e-02 4.681557884709756229e-02 -6.565705359188284129e-02 -1.999975427429314355e-03 -1.699813051365471964e-02 1.691969667512613351e-02 8.376219469914672131e-04 4.581623605544641459e-02 1.447275106444679410e-02 -4.042570489312221135e-05 8.297608565499201005e-06 6.562629596200395543e-01 3.482441383930450218e-01 3.578668368529022009e-02 2.675651377953129281e-02 2.700405968369655463e-02 3.787352422622809744e-02 5.844577744228738464e-02 1.137226651316805626e-15 2.358290486466391861e-02 2.668692261644409897e-02 4.014751262490079409e-02 1.469361983356703288e-02 1.351708106937465209e-02 7.028781868904957972e-03 2.961492083464217079e-02 6.661928128752084635e-03 4.037528608242987177e-02 3.959341307105226568e-03 1.727285746600671495e-02 2.827092617555253262e-02 4.589163066601947838e-02 3.959121789456679190e-03 3.258201418263198534e-02 1.194990233051001875e-02 1.420461329357416633e-02 2.115641841285242561e-03 6.717117655556120726e-01 1.150146938881166259e+00 3.862515349029202461e-01 -8.371849121357177737e-02 2.270457234189791201e-02 7.158587289836358149e-03 7.888464621711376371e-03 -5.421788753674662564e-02 3.293803132397816191e-01 1.210307955938294322e+00 5.903184638810252682e-01 5.915023701629026735e-03 1.670787050493134987e-02 3.063080752159332704e-02 3.434500454242873230e-02 2.179826880199554548e-02 3.543815308242444923e-02 3.162355842780170245e-03 2.615864827149953081e-02 3.058297803338609622e-02 2.966128100278321578e-02 4.095026167265724548e-01 4.300768232711985606e-01 3.033414024954261823e-01 1.957662155451041219e-02 1.267878354590163463e-17 1.676207497584978473e-02 3.811115171373561511e-02 7.174480595255276760e-02 5.688341558387894487e-02 1.053251309228465937e-01 5.378089744981028736e-02 7.257112050948361093e-02 3.930414250121792852e-02
2.768668641296237465e-01 1.960235874421416627e-02 5.354340439947730991e-04 2.992807690454082686e-04 2.158407893858095821e-03 3.205210639131070554e-03 4.928897781935575109e-03 1.168706386608327208e-02 9.042077230378066377e-03 1.366471103366553516e-02 2.672467490592420405e-02 1.256178191787483620e-02 1.854059448837377944e-02 2.261176527025843264e-02 5.459951690899240290e-02 1.927485874557550993e-02 2.754063006449376360e-03 2.046961261972537766e-03 1.014128431706557559e-02 2.856631643317581945e-02 5.660231951368853431e-02 1.963391334826453810e-02 1.700310109769209016e-02 -9.560491789450570321e-02 -7.474857637635194674e-02 -4.708318706246999302e-02 5.284598988497773986e-02 6.082937874082246926e-02 3.223364296839644622e-02 4.218666790160841701e-02 1.184659429948711762e-01 7.609911007611082356e-02 1.247456994459761603e-01 4.029294590784888996e-02 6.271104233196152233e-02 6.918910415626712551e-02 6.027940657898999549e-02 2.959267162058388037e-02 -5.785741991313650412e-02 -3.117975018583171720e-02 -7.031707053693832254e-02 9.578507436060958061e-05 4.617624114657475215e-03 2.176125079089110586e-04 2.581812397215813362e-02 2.561587617121359114e-02 -3.881433740134943022e-02 -9.319207266389455124e-03 -1.947381948692825798e-02 4.252213841908173558e-02 6.133565686968953101e-02 9.721126338496836536e-03 1.636485215128522311e-02 3.433473038208414291e-02 4.759253792723485560e-02 4.430509718680714082e-02 7.377921253651525957e-02 4.257343807145622105e-02 6.168943579275218414e-02 4.478765634775299515e-02 -2.202521267637778668e-01 -3.174713033714438373e-03 -2.643600932318006941e-02 7.883937012037188496e-02 -7.095485354027847534e-18 7.615819099289533427e-03 6.394856196227709190e-04 3.725996150710700339e-02 4.978985681160739384e-18 1.640887904210060547e-02 1.350841282189961003e-02 5.117300319522850166e-02 6.029899506889274501e-03 1.179876639776704893e-02 -4.417337603528414786e-18 2.455436128552411848e-02 4.467128223775545517e-04 3.786506122388359763e-02 2.060131980953792269e-02 9.804163494578715075e-04 -2.033300223121090675e-17 2.041460417598725888e-02 2.704544822352392477e-03 3.771394311556796039e-16 7.065688450374617464e-03 5.197810738711673186e-03 6.047075255732754263e-03 9.118030401039500929e-04 8.818619010103871669e-02 -5.973627366870326583e-02 1.245068915962630196e-01 -1.817600143133283122e-02 -2.654536642532762114e-04 1.177453728688303627e-02 2.929041426744351945e-02 -9.860388634173053793e-04 -2.728491253953618106e-02 -2.855357819143013284e-04 1.240152666214747700e-02 -5.048125862540617931e-02 -1.336360653087362814e-02 4.533526359052546771e-03 2.546790934264447914e-02 -4.885416843913567564e-02 1.216148177741986913e-01 -2.298160672145132047e-02 -1.645486755902647244e-02 -4.329972388912436271e-02 -1.822331681194235952e-03 9.293759855712853862e-18 -1.710040852134349712e-02 -4.748366464381223451e-02 2.143135688748637815e-02 -1.692692584753001037e-02 1.593918883785440388e-02 3.016314095106547286e-04 2.686575127536601493e-02 -2.275110255915964799e-04 -1.074499116294418241e-04 -7.009744143627577584e-02 6.008410694992790679e-01 3.420554645587881026e-01 2.909633796619461379e-02 4.134537090878090052e-05 5.288598311570330557e-03 2.091006157736207468e-04 2.120325539523753786e-02 5.705003437828270324e-03 2.311142426954769860e-02 2.500797438379729894e-04 3.168607270895763828e-02 2.871581557073101844e-03 3.692049351922409989e-02 4.700141089562731578e-02 3.985880152425316791e-02 2.326808828765689063e-02 2.652663828689936706e-02 1.492178579563187106e-02 3.164630617501540399e-02 9.441551154641063259e-03 4.016238586675457967e-02 3.999538959497134921e-03 9.786507464826524338e-03 5.419842088154336324e-01 6.659409939280797053e-01 6.093954506251920700e-01 3.707125652710718811e-02 1.304715204859802254e-02 1.082901150140953839e-02 4.376080721004680513e-02 5.677486433673980398e-02 9.230550896135882033e-02 7.735941031282235480e-02 7.598061104155776069e-02 3.321176396427355515e-02 5.600868638786279247e-03 8.475652949776913553e-03 6.088838694311862199e-01 7.176126144892525094e-01 7.009175758207883966e-01 6.192887967195007115e-02 1.458364871336351350e-02 2.945556737338695871e-02 6.182264145356119317e-04 1.880506094738861739e-02 4.660421132529123621e-03 5.338397233028348010e-01 8.175406184297353551e-01 3.733160706571514442e-01 5.891557228712891899e-03 1.815783707262160970e-02 1.311475930580386501e-03 9.241088141269821962e-03 1.711245670354887352e-02 5.933170633241874325e-02 4.446732723856865188e-02 1.059229113601971861e-01 5.820562755078989570e-02 8.159023685940508530e-02 4.341547869153888906e-02
1.159030108802518166e-01 -6.151770699351986199e-02 7.255803546293604356e-02 5.058785908977461027e-02 2.960045959743749094e-02 1.251440792777811818e-02 7.344166372650975938e-02 7.470808243922480961e-03 5.482883623988873967e-02 3.145250326680226749e-02 7.094990706286415094e-02 3.821220493400247220e-02 1.148410448637668564e-02 2.762563192155007008e-02 3.297301457332166286e-02 4.996393809856965185e-02 6.085992662325342972e-02 2.103859131172037059e-02 1.212472129042014603e-02 3.888318043066966989e-02 5.872319800756801295e-02 5.345235875870067016e-02 3.127045071972941320e-02 1.754114613327592553e-02 8.093945941873967667e-03 2.440200784783849892e-03 4.252766809096533707e-02 2.253066442882895037e-03 2.245885295325790961e-18 1.104399807517016628e-03 1.132807799010441749e-02 9.689655713065155676e-03 1.645629881193602477e-03 1.480733121429416745e-02 2.053572420455825054e-02 -3.576051419775686524e-01 -1.867345129238729695e-01 -8.190187049888812687e-02 4.191585194220542876e-02 2.357345417318165368e-02 2.634613502597742823e-02 1.009505032183674947e-02 3.146530867099662371e-02 6.805813870683565144e-03 2.621517274531323660e-02 3.422712064880516214e-02 8.282061657627418083e-02 1.579309015355951501e-02 1.535038689397369080e-02 2.693972479723302435e-02 4.340544146746630472e-02 1.334752256676964739e-02 2.846281681783019782e-03 2.087095961174840336e-02 5.044170163424125525e-02 2.658456425737770051e-02 4.988752945924933291e-02 5.456052100805303118e-02 2.000308399300327888e-01 5.498222870882937818e-02 -1.032086090010327084e-01 -1.868050530909037128e-17 -3.318755801585215187e-02 -2.891039476328094351e-04 -1.028542081391544717e-02 -1.155091283034068335e-02 3.970245357674955650e-02 1.812262598431738439e-02 2.677070045291022465e-02 -2.685254739857982822e-18 -2.900041000994997954e-17 2.344016585295782482e-02 4.068546524332233155e-02 2.760810495938720646e-02 3.907030505716219820e-02 2.244606828561285239e-02 -1.175606267551970903e-02 3.174662826311935165e-02 -1.912249413520300187e-02 -1.173800104951786341e-02 2.564247556816084614e-02 1.009478972233900373e-01 1.672945626285743739e-02 3.339512983685561515e-02 -2.310033155634498743e-03 9.549880834264305382e-03 -6.890500001968468714e-03 -3.214780884725872406e-03 -5.286979722429767542e-16 7.846587113757174173e-03 4.714108315077388499e-03 1.015896759555123515e-03 3.289070754490904663e-02 -5.204274050305018884e-02 6.570844946455253743e-03 -1.552323938085247573e-02 -8.116183574141304716e-03 -2.013543427585871501e-02 3.239887328318491640e-03 -2.083997947333373426e-18 2.106668189398462534e-03 1.071257577282804831e-03 2.603996076374651136e-02 2.980952117536879095e-02 1.437453861496538815e-03 -3.023575958186678139e-03 -2.652834846057740975e-02 -3.723221490269558974e-03 -4.869293932758792312e-03 2.799403444422261800e-02 4.725860427374802344e-03 -3.472007916017252926e-18 -2.257940250775002794e-02 3.100357298847587068e-02 8.596393800487493048e-03 5.876017479653851449e-04 3.467092993309286997e-02 7.979219995413999733e-02 1.129984573551399696e-01 -3.796996372531281307e-18 6.446265094262758621e-01 3.829199004723716215e-01 3.852098568717700344e-02 2.986310095770144252e-02 1.672949001181361570e-02 2.346780937298822631e-02 3.551258652439229918e-02 6.116425485444650556e-05 2.562667704731310531e-02 3.921791471112282684e-02 3.766538742135935175e-02 1.608514911682194098e-02 1.790079497195133651e-02 1.431984184982784333e-02 2.549604834687388064e-02 7.702580607376029349e-03 3.504741333706554857e-02 3.690113134754489216e-04 1.008489793567365553e-02 2.605162277458496559e-03 6.751324267353389814e-03 4.985881750088480557e-01 6.790373450870155203e-01 6.178631346301064475e-01 1.520950206099568568e-02 6.725290138153114383e-02 7.992992963120092165e-02 1.543722517941607635e-02 2.886796584359995962e-02 6.185581931032679663e-03 1.421735013746416344e-02 2.341510361815401196e-03 2.012434083014698685e-02 1.173209852433152248e-02 2.324197408538803203e-02 6.948365779662536434e-01 8.168901866849415105e-01 6.876154949042365683e-01 1.577133892163644704e-02 1.467246453331040243e-02 3.663734113560063177e-02 2.241837407788875233e-02 3.925177903851725936e-02 2.132602937766002807e-03 2.518905002633475138e-02 5.842368717602911682e-02 4.248095367182707499e-02 2.074263974263029428e-02 1.956677999214222544e-02 3.943188982076552968e-02 2.539088586312764073e-02 -1.232125381717644836e-03 1.526096731979086074e-02 2.207627886197681757e-02 6.075981523379942600e-02 1.057118480459552819e-02 9.451811468076951050e-02 4.774478797327190160e-01 5.732991337788719832e-01 9.453339318637581368e-01
2.601257364166203456e-01 1.435326428943927042e-02 6.693415393620084529e-04 1.008962614017551044e-04 5.164802760485914443e-05 2.046072281805746615e-03 3.925244036039192939e-03 2.163369104056729868e-03 3.595705379728895285e-03 8.843112253111910098e-03 3.629436329553781954e-02 2.076123291553011452e-02 2.580664059956474035e-02 2.657526453013702231e-02 3.357647511867169199e-02 6.043832067243584683e-03 -1.521287330802698025e-03 1.870571474145506577e-02 -4.028080990915460652e-03 6.410220457780488323e-02 8.537307239577299200e-02 2.071702644243074778e-02 4.011471924508724224e-18 3.187190016519829416e-02 6.056860290026434140e-02 3.397428727987264474e-02 3.492990963552213002e-02 2.977397916652022372e-02 3.233736089932479768e-02 4.778397466065605753e-02 1.203612151243508566e-01 8.621091225347048137e-02 1.190720015168406087e-01 6.355580248623370065e-02 6.323687693330205806e-02 2.745555286405575862e-02 5.980698852260644439e-03 1.003612799587641213e-01 -2.075878687138050377e-02 1.435851001526600168e-03 2.340195714207689726e-03 1.593446385765626971e-01 3.056042098244680094e-01 1.625160962694983113e-01 3.265192371176302011e-01 3.476115388945432705e-01 4.227800039653656361e-01 4.676989553200844307e-02 5.315249374541675814e-02 1.031017824327246611e-02 2.620739352387875531e-02 1.867503109603144444e-03 4.829944740674638019e-03 1.120637456637922982e-02 7.573330757022178616e-02 8.520563980160690554e-02 6.777052277826565996e-02 5.008927019819343907e-02 9.627816661279602062e-02 6.164989126807756195e-03 -2.068907642656715451e-01 -2.600550258592247092e-03 4.663695378573360847e-03 6.155262331019543259e-02 5.676788351682929843e-04 -5.028006170300885602e-04 -4.742941239652935110e-18 1.641347641738282945e-02 3.361243653232957613e-03 1.782697648746755376e-02 1.124722293431231791e-02 3.616331866132957429e-02 2.785055235934785420e-02 -1.784070104765290139e-17 -2.686419886551791387e-05 3.246118811718515240e-02 2.757565162323163954e-02 2.675425340267940932e-02 1.579059229216559987e-02 2.195754991089597652e-02 -1.998940755007989553e-03 9.445576175731943552e-04 -2.106251916599106294e-02 -2.461442084931102919e-02 1.408098797737037874e-03 -2.739642649400788818e-03 -2.124421386018881806e-07 -5.911508996357287687e-03 -2.085043354754400734e-03 -2.025206108052737819e-02 -1.320646494085124132e-03 1.736851486106374314e-03 -4.964868501866623665e-04 -3.489024355645808234e-04 -3.177801543039974210e-16 5.020725931603029282e-03 2.503031294552249958e-03 6.062090345364642571e-07 3.911797590932605123e-02 6.839431125735902228e-02 -7.066083399726569292e-02 4.371014753220068411e-02 -5.415377145529347985e-02 2.233313328492832900e-02 -1.154716487783808039e-02 -5.694658603351249876e-02 -2.185383586928429586e-02 1.702766580455119805e-01 2.057494687790822896e-02 2.486782622531235817e-02 8.214691972032439615e-02 -8.210602292999072405e-02 5.962603498995783902e-19 -2.763162357869200503e-02 1.004843879841966180e-02 1.499750059606142719e-03 5.916007526590601195e-02 -9.317049981529025582e-03 6.043023632393471067e-02 -1.065084448316770335e-02 5.962496218730536368e-01 3.302069239752800089e-01 4.163013951136612656e-02 2.123546360512041444e-05 1.967658988946963811e-02 2.143506363878432911e-03 1.613659435825000668e-02 4.467153241294282047e-03 6.149547175492747937e-03 4.939932771048483702e-04 2.989543897598815128e-02 6.618437143703099296e-18 3.185301700551407267e-02 1.500406614422542320e-02 2.409430097398410422e-02 1.224384890303797284e-03 5.892454625836718307e-01 9.418108146418615600e-01 4.414692862243024818e-01 2.235226313607277179e-02 3.419855898226693836e-02 5.081773778952966352e-05 1.495351871124594653e-02 4.913256037467989695e-02 2.465589030879743529e-02 2.863479170434235499e-02 4.655467986298595512e-02 2.777330001082585834e-02 3.053236850083128340e-02 3.211403091366856616e-02 7.924954023181550455e-02 8.316880545196922436e-02 7.479707657263574838e-02 6.913354024393270070e-02 3.609303638658679231e-02 8.786430997019961404e-04 1.190215583496364270e-02 5.839158180513361884e-01 7.557522621736068036e-01 7.699818784546570427e-01 2.639068878538329277e-01 1.395781199524380456e-01 4.732933127689246955e-02 6.191462480377818434e-02 5.480987313884166240e-02 4.390241395902548180e-03 9.730370540322162351e-03 3.304984098222376043e-01 4.318960636166662703e-01 3.029147337554969899e-01 2.770656586688386575e-02 3.615789551076012301e-05 1.435055699733076701e-02 2.329169017846501738e-02 7.625596001697471094e-02 4.637555241069681727e-02 1.205010343377155013e-01 3.580929441913159594e-02 6.588394250040338629e-02 5.475672049859731105e-02
1.282393347716760945e-01 -5.997503408523765372e-02 2.461053238203197296e-02 7.346602134583044182e-03 1.023916555671592135e-02 4.854115420736346796e-03 4.137108304550010662e-02 2.715328972407026788e-03 2.386357422381302565e-02 1.570304576236494429e-02 4.222178775092488845e-02 2.140033497687830397e-02 1.276360314367844424e-02 3.228509051826363579e-02 1.999909849054264457e-02 2.640095989389185452e-02 4.636804023936216129e-02 1.573298661043790250e-02 1.315660687697417681e-02 -4.725088353162489041e-02 -4.394578157106061173e-02 -5.143133661004815893e-02 1.471067468497846281e-02 7.043702317582524290e-03 1.416413328469345694e-02 1.585242261654230128e-02 8.919731623906321238e-02 7.205314195325991482e-03 9.562022441917866766e-03 5.909168097558003575e-03 1.561123071991740371e-02 5.021476237492696710e-04 7.197680396546530558e-03 1.204907293666789957e-03 1.962607993809106015e-05 3.715112371950171740e-17 2.148560431595356122e-03 1.114795314563689875e-02 7.277185010120552217e-03 7.277298671947670869e-03 1.857233647251551462e-02 1.000596292388217408e-02 3.159467209462834408e-02 8.994263040963238615e-03 2.879898264919584713e-02 5.557596418200701216e-02 1.257225774977796251e-01 3.223454165152289663e-02 3.064556250308504010e-02 2.767136349278890434e-02 -8.416820230314013941e-04 1.333365385592491341e-02 2.353877290131902930e-02 4.517050158863031295e-02 6.851953807133719099e-02 5.144078996347760391e-02 6.170369382840601435e-02 9.692146885592374528e-02 4.880650187584324035e-02 1.476355894214956450e-02 -1.045811808943239579e-01 -1.327375110206689572e-16 -1.786222676935607726e-02 2.421862132452720986e-02 1.951540477531285220e-02 1.157994788135961219e-02 -4.224486842435564403e-03 9.756470425742418476e-03 3.394515088444729350e-02 5.123119517048647931e-02 -3.237430894597884135e-02 3.862606471080663739e-02 5.287411016897660028e-03 -4.589211530346153348e-17 -1.401634845333305922e-04 3.925909654990644010e-02 -1.193424660405727210e-16 2.351726490933237976e-02 -1.149323670646498624e-01 2.042810999054047241e-02 1.065819821809841153e-02 -2.347709181617079963e-02 4.008443045652874959e-02 -5.252577639360199446e-02 1.920237361520357733e-02 -1.513069056955473023e-02 5.430146230604214475e-02 2.139181669178887631e-03 7.105346609404324110e-02 -2.305573580435358455e-02 2.353464917012757385e-03 1.109560027459308672e-02 -2.561935455278372548e-02 1.478538629675949997e-03 8.441850704962552618e-03 4.341327364888013934e-02 -6.658961766335355256e-17 3.185482549912305693e-03 -1.299406836167429045e-02 -6.670324926027756745e-05 3.627073034744073693e-02 -4.035077841665647529e-03 9.996117498796933121e-03 -5.616988872229784663e-17 1.741111837642814186e-02 -1.255895251595950048e-17 2.967542641087925204e-02 2.641445997450878741e-02 1.001805477636862236e-02 5.075257669650781733e-02 3.312816752000011244e-02 -1.888564267695284210e-02 1.211680433807108687e-02 -1.131979478641460812e-01 2.819690649197775963e-03 -3.666894765136079204e-02 1.875507665699835402e-02 -8.486619175463570863e-02 -6.930613821671147319e-03 -4.965996550409291360e-02 6.444645825086667834e-01 3.541100593213029613e-01 4.916175682634434801e-02 3.508000587457041353e-02 1.940338427010671746e-02 3.345878127122363377e-02 7.633941243165928703e-02 3.203014141709945722e-02 2.824082275594987437e-02 1.194846859579086484e-02 3.594446371594896272e-02 2.320153778825738075e-02 2.307709372041957807e-02 1.771508217890402012e-02 2.927442978149731936e-02 1.467457936042566337e-02 5.969889284987398015e-02 1.429297214131563208e-02 7.630191288720237532e-03 5.152100417841808389e-01 6.995788759137165913e-01 6.582849457334221110e-01 4.428851646785537249e-02 2.849694943206167175e-02 4.233682743395964648e-02 5.379985592894257623e-02 4.960323999861571825e-02 1.694497582119578599e-02 8.907019387020523662e-03 2.309339471672072590e-03 1.274759905735822242e-02 6.164103855607466591e-03 1.764575778700516739e-02 8.683332655254685228e-01 5.355311535094675612e-01 7.039474490986391553e-01 1.674287671792393586e-02 3.158326281447400470e-03 2.851831711577856171e-02 1.650016517554335482e-02 3.480374180295534026e-02 1.410548901562404603e-02 4.403747823171941245e-02 1.077758047244498606e-02 2.904063565431168337e-02 2.260174622356149812e-02 3.163920317084318451e-02 1.077831790897161010e-04 3.036933257617410176e-01 4.281851621862991508e-01 1.653559150646027598e-01 -2.174046355719946866e-02 7.037249638384552447e-03 4.673762965018796178e-02 6.052334177053095982e-02 2.575190324365075542e-02 1.198010792775392741e-01 6.771988606423653967e-02 4.940640514706021169e-02 7.573617100411503267e-02
1.756123444688217072e-01 2.319463793079110103e-02 4.318237048696268749e-04 2.648289428756938658e-03 1.397033862812549487e-03 6.836111396960876246e-03 3.524310592386945169e-03 5.200703711043173222e-03 6.468605770042687268e-03 5.600563874096126452e-03 4.315078688829544068e-02 3.944211523354397564e-03 2.863358666444984837e-02 1.784737012403826012e-02 5.830329276299137803e-02 1.900609483647935916e-02 8.455205697824650854e-03 5.634511105789593265e-03 1.704255374615512228e-02 1.915123581807758046e-02 4.009482161614754542e-02 2.595240750325535778e-02 -2.571766913311911845e-03 -7.342296578719141598e-02 -9.123912208748027886e-02 5.916500957239027164e-02 1.847452594870856724e-01 9.243111727690346557e-02 1.878061689266436463e-01 1.628444643686789151e-01 1.816074578171298082e-01 7.808951103809903815e-02 1.952066362336022931e-01 -1.067379096537692214e-15 -1.712226053862700159e-01 -1.373557433229280304e-02 9.371804302651281712e-03 2.112309588508166566e-02 4.292851233264032007e-02 1.522555340690633086e-02 -8.627996250835086312e-02 4.697831479834538825e-03 1.668249424406701917e-02 5.086693393241288261e-03 1.261808916793892912e-02 4.777494843577555217e-02 3.755516271072898565e-02 2.269556830045190224e-02 3.740294595439796338e-02 3.578298857034568209e-02 3.314664461950737190e-02 2.184141198995799560e-03 7.396589753073091154e-03 8.013689979713781511e-03 3.815864880719867930e-02 5.842438598619002776e-02 4.300030640405431287e-02 9.048573110426846866e-02 7.773514527380595562e-02 4.009641760345376144e-02 -2.148742208424708477e-01 -4.887943927409732839e-04 -3.693755126708034298e-02 3.040435023266745834e-02 2.113395389602517119e-02 -6.009679660964613312e-03 -3.449991524139772725e-03 -1.422319077805814106e-03 1.208598860818953097e-01 7.783459711666000820e-03 7.856675123157066606e-03 1.266601593292002170e-02 2.553561411316655066e-02 -1.187434034491416032e-02 -2.903967120837608918e-16 1.916357658250906516e-03 6.404776405299038300e-03 1.112348723595091640e-02 2.280092058197381172e-02 -1.123637504192144674e-18 -9.572359982987670196e-18 4.618115390807797649e-02 -5.035516373621452824e-02 -7.398525881450897235e-02 1.966909314474950233e-04 -1.195660460188195358e-03 4.682216234320773357e-02 4.280896645928994326e-02 3.067840137735141632e-17 2.104473429345461027e-02 6.396158365160582726e-02 -4.576863890420729969e-02 -4.250845805080776668e-02 7.953676832750845860e-02 2.476094534743760595e-02 6.041505424530953759e-03 1.051302493085713888e-14 3.502279381436308042e-02 2.815585675759880654e-02 -2.807040167472731745e-02 4.252446913831986648e-03 1.125680210110567916e-02 -6.642397638658615114e-03 -9.132898734549112262e-04 -6.297879725283858643e-02 -4.177193344102383965e-02 2.574156423629277157e-02 4.287585713496889356e-02 1.665599224694994801e-02 -2.496587486870417445e-03 6.596031257123027847e-02 -2.156156225648950958e-02 5.515477841421775285e-18 -5.521789521632188696e-02 3.464731865813316936e-02 2.487132502004140919e-03 3.845090243211909259e-02 3.270708671033397547e-02 2.887056354838573105e-02 -1.196672273802635265e-03 6.724697605781772003e-01 3.501769475374689189e-01 1.973704856864485352e-02 1.051670840941563935e-04 9.160197576581749490e-03 1.594037912956218148e-02 1.919552020930846703e-02 7.524671760935104127e-03 1.592880394093896396e-02 1.671425387112223174e-02 5.256316668530726377e-02 2.343910400241875150e-02 3.045536845422424851e-02 3.012590979359128923e-02 3.611146535894534798e-02 3.767264800249935791e-02 2.029124653588923466e-02 9.897165821663073820e-03 3.033777327869737264e-02 2.244830456120287809e-02 3.067939320531660324e-02 3.799566203568766325e-03 4.976569785257451417e-01 7.187876227892204817e-01 5.631648890594193047e-01 -5.263156239937695491e-02 1.679569465981623974e-02 3.568257124631012134e-02 6.122751488395761915e-02 2.600270812981861704e-02 2.146691822968216715e-02 9.338963291733471186e-03 1.941440439353636638e-02 3.767958655152955916e-01 5.594713853874491161e-01 5.048834554333221369e-01 1.258509628823651846e-02 4.562417456384931617e-03 4.961423467480173544e-02 -4.672244440237477534e-04 4.287410759892626433e-03 7.988628640988733667e-03 3.911305920623373011e-02 1.934546626700252552e-03 6.316996972940706889e-02 3.728990856886373051e-02 4.642390391111891196e-02 3.963395780537910529e-01 4.166053103443906891e-01 3.526499900850654790e-01 2.159903181847030312e-02 -2.097625262952071847e-17 1.202075630475468900e-02 3.509919575606988001e-02 6.110316439271070571e-02 3.821732740861617794e-02 1.202145050090223377e-01 2.223007615611554896e-02 6.692877390735565979e-02 3.412172245137821275e-02
9.431733185356457050e-02 1.248708501521432895e-01 1.228407007364997056e-01 3.680189186734827977e-02 1.392210155661563475e-02 2.056767520091587562e-03 -5.822050954018160099e-03 6.960036154820182890e-03 -9.003163498645270699e-03 4.274628646292166767e-02 1.749266858697255123e-01 2.685833256513502770e-02 2.340968569760465065e-02 1.367577503080271480e-03 1.321610253296393381e-02 1.776825049198623580e-02 8.515415633096830306e-02 -3.026485367772007926e-02 9.744689802582603977e-02 6.039922494660780738e-03 2.326734586457042744e-02 1.760971601395746808e-02 1.891233212106020745e-02 2.676748264270132488e-02 1.252154710863539501e-02 2.341698962499343464e-02 7.455115345199922472e-02 1.851537945113058276e-02 1.286368162895885250e-17 9.110810010301883318e-05 1.272127311257657085e-03 1.907067319835319824e-03 7.321812604116964574e-06 4.911619006475680681e-18 3.870484963328875427e-03 2.429316634011327754e-03 -3.360620081669177944e-02 7.173266470572232413e-03 9.601103141113440098e-03 7.163967537072347784e-03 1.170447171017984997e-02 3.348673782566126411e-02 -4.006931367556247625e-03 -4.791623101511776768e-15 8.777947097987771896e-02 5.714645903798121901e-02 7.354529406652209056e-02 2.779849438871581299e-02 2.687905465391598331e-02 1.103520258103824619e-02 -2.133641222444642924e-02 4.158525822312431064e-02 -6.821124736152620223e-03 4.512602974370009262e-02 1.304384194112542250e-01 1.293831423048087403e-01 1.049620753092525144e-01 2.773034793661827702e-01 2.084044153527738197e-01 8.138709849762242043e-02 5.280748416130482542e-02 6.255480515112887829e-02 -5.482225958732759447e-02 2.835288016856062077e-02 -1.489914945002917690e-18 3.441809278763613422e-02 1.591220410176803768e-02 9.844640737397342828e-03 3.483463889103425443e-02 -3.509015652633754728e-02 -2.066450475251364882e-02 -7.068564599200879289e-03 7.200674123296637447e-03 1.686460794617541670e-01 6.032435983177569100e-02 -1.287347353027191565e-03 -1.201521540039914789e-06 2.426294505988833491e-02 8.013554206153025716e-02 2.466752945748599407e-02 1.605318810340535252e-01 3.908429801702522667e-03 5.615516479825926471e-02 2.522131877593023798e-03 8.014848568915348623e-03 -3.254954326945818516e-04 1.707304364058293694e-02 4.490300425392413503e-02 5.041871855739234881e-02 -1.653476198092112756e-02 3.892781903492625130e-18 8.189261051889808185e-03 1.475682004983733947e-02 7.723637875568091504e-02 -5.317945144818392360e-05 -3.052623892385427241e-03 -2.225676777295611253e-02 -3.133766510597107285e-02 6.209210508021545567e-04 -1.830946875722467682e-17 2.095949489732221518e-03 1.359753165239688776e-02 6.377547391074323870e-03 8.018815155860961608e-02 -1.273435783985065253e-02 1.044010454097867024e-01 -1.284823698316277588e-02 3.290749519052624944e-02 -2.932149233567677565e-03 -7.458854501317776445e-16 2.548393373288958264e-02 -7.769483064611651087e-04 -1.787172208102586661e-03 -6.184712889053211454e-03 6.890276073677211550e-04 5.550164610957579459e-03 3.179706419043396796e-03 -1.638795247541594610e-03 4.936942496585884632e-02 -2.687624364426241877e-02 7.103785392161691969e-02 1.094129009497593480e-01 2.258239687530908257e-01 2.011338992679806426e-01 7.812267745553694298e-02 3.131485638354547518e-01 5.999623979724945100e-01 9.354236787095536743e-01 4.808778304208847221e-01 -5.861039343624091513e-02 8.248994310437260044e-03 -5.593147258542669614e-02 8.952432310478600913e-03 -5.755955304628572466e-02 5.585810769587153633e-03 -5.418455272108266274e-02 7.613339211079419785e-03 5.491459811909720212e-01 6.341936077644831915e-01 5.204437503505405793e-01 2.203637895114651454e-02 1.882062226492632953e-02 5.020921491604364573e-02 1.579772467493709490e-02 3.209256291950999401e-02 1.695759355251360775e-02 4.377765057762034867e-02 1.094678461550555038e-02 2.341579026744326081e-02 1.005880958722620401e-03 1.459358112479556234e-02 1.908287825261478126e-03 2.168965894541536460e-02 3.131453207687767996e-04 1.904657015351420707e-02 7.830744029855342750e-03 5.202330209612580567e-03 5.087617597346990456e-03 1.695091173866810144e-02 1.372149734092173445e-02 2.209720297162219571e-02 6.319976398010342944e-01 7.136331147147964016e-01 6.215332444710722770e-01 5.309962361491554067e-02 2.050923664406802821e-02 7.200663778526766867e-02 5.428308779947836615e-02 2.361894962494155009e-02 2.325112942345264333e-02 1.740647827950198012e-01 3.673537079399808447e-01 2.918532760400416315e-01 2.960049847791485292e-02 5.372280314373210908e-02 5.374688262764629881e-02 1.056308281888166667e-01 2.748179694753812019e-02 4.555635126313841571e-02 5.249450490075016629e-02
2.333811808686521994e-01 1.416512516750044817e-02 6.751477637460232689e-03 2.091768168087225488e-03 7.837761079920284313e-04 2.809632442618862844e-03 6.248223281569000552e-03 2.773655772270800131e-03 9.907609715265946679e-03 3.701958791653259424e-03 5.449927076693516365e-02 2.042680712258189077e-02 2.070953626007218315e-02 2.464528133590366057e-02 6.305507370183061067e-02 2.951633282469605379e-02 9.845321209469348744e-03 1.933475264649974282e-02 1.710838560824003846e-02 3.997018679998416008e-02 -6.894958703483770879e-02 -3.205150900414490550e-02 -1.128944545466926963e-01 3.444601781755650405e-02 1.097926787314574409e-01 5.482544944323018699e-02 6.734372857421602743e-02 6.001178085885471719e-02 4.925321890474486375e-02 9.579474189379821869e-02 1.606707648095940799e-01 1.197167523047541760e-01 1.291864366183440860e-01 8.568032072840996161e-02 8.535410794837390569e-02 5.740705213528062012e-02 -4.089766791199311857e-02 -3.523795661227020759e-02 9.540364087317523278e-01 1.045321562782764211e-17 2.634963408031126245e-17 1.057731703322390727e-02 1.214764391598581288e-02 3.495690878566014835e-04 1.450779613730311668e-02 1.673198652359683333e-02 4.437963547189812535e-02 4.457770568721354520e-03 1.567855899782406859e-02 2.099201384158649875e-02 1.634874209490036132e-02 5.633176929537661408e-03 1.584828394810267743e-03 1.023075908606436990e-02 2.937604604513214049e-02 7.659827380358526921e-03 -1.122053447447214636e-01 -1.608732849991237057e-01 -1.898324425117041137e-01 7.303156847679649621e-02 -1.914601528428301891e-01 -4.159727740962044541e-03 -1.189960398561949392e-02 9.397001748995947173e-03 -5.531143937237427538e-17 3.311561816470680862e-04 1.684084261560408677e-02 -4.492522096668921055e-17 7.131610555758968392e-03 -7.269327626534883072e-05 -5.542081176168369566e-02 2.467783850179072064e-02 -3.573968018049531782e-18 -4.238522405911972907e-02 2.021355030420930038e-02 1.382077333658620830e-02 1.470837725048350711e-02 3.059986720965975505e-03 8.459930472823513903e-02 -5.818814838527953080e-03 -3.733155341285100595e-03 -5.272984283640032299e-02 -8.450378774925323966e-02 3.611091118414895811e-03 -9.692235630138846321e-05 3.336871654625696987e-02 7.513111646689996309e-04 -1.386159673916374270e-02 2.588402433733538732e-02 -3.345770790874761508e-02 4.925653928981475749e-02 -7.064826928414461027e-02 -1.461157737004518216e-03 6.018786235994379868e-02 -9.707550412604926474e-03 1.085302611169142433e-03 -1.740626443599261122e-02 -2.774650449994183962e-02 -5.241298407310100715e-03 1.005156216331989927e-02 -3.326818381078486958e-02 5.760952312099755501e-03 -5.712518864974418433e-18 -1.877516635371421951e-04 7.699274262296824760e-03 5.606552632898335886e-02 -1.410809084175386778e-03 -1.860263534583722687e-04 -2.160658460364283176e-02 -6.853923761785619875e-04 -5.731155628690195319e-04 1.188548482313778534e-02 -1.465799247704109749e-02 -3.801310273502583013e-03 8.487057293885390957e-03 3.393385288326746657e-02 -1.105306104319662974e-01 -4.616343951278246288e-02 -2.770480535494524998e-01 1.966429318946063820e-01 7.607884106650684153e-01 3.473727682266706807e-01 1.764138447100708010e-02 4.377611280229307968e-04 6.550598653747449762e-03 4.364526919633409437e-04 1.675293608656875863e-02 7.168136063350259365e-03 1.423097478350646150e-02 2.379347441000280699e-04 4.214635343298762937e-02 2.302494421382102543e-02 3.223481522730085175e-02 2.569936370800090727e-02 4.035431199134181107e-02 1.694845393562269958e-02 2.256176944664999365e-02 1.084125750561938141e-02 2.093844801867555949e-02 5.541337266989333982e-04 4.834224179725683523e-01 7.214098836171021212e-01 4.013825627891090542e-01 7.844103782345992733e-04 4.183522632923276224e-02 2.162742796155150537e-03 3.981281169422424410e-02 2.459718486945569604e-02 2.443713008030833425e-02 2.960585137114068663e-02 7.202051950905574706e-02 7.138515105619962220e-02 6.466249453961347982e-02 2.733714934310165628e-02 3.376869652106969516e-02 1.508291148460357695e-02 3.853005432881342895e-01 7.805468595397258014e-01 6.298376575805725297e-01 -1.047453099856470865e-01 4.327555548726306003e-03 5.401544491586329245e-03 3.314991293267394168e-02 3.257820770493011220e-03 3.302731683119018713e-02 2.052962446836956514e-02 4.698716787807715595e-02 1.851745232331227994e-02 1.911395209332301254e-02 8.642674734352345789e-03 3.491519879468991255e-02 -2.765397924104412083e-04 1.893278046080723240e-02 1.418070369988755179e-02 5.879507231088509922e-02 4.338540403339761001e-03 1.108322360329944273e-01 4.954643758684839083e-01 3.779959764860489346e-01 8.239458277105693451e-01
1.363204088440948536e-01 -4.673655427631773218e-02 3.437013540779298992e-03 1.092798822227828651e-03 2.095747034398460877e-03 2.255443758072875999e-04 1.506837974024139923e-02 3.048119900127773275e-03 1.273177047127969572e-02 1.076513569673532601e-02 1.808083567151553411e-02 4.603731325887424806e-04 7.935905955355312047e-04 1.046146392179705698e-08 1.575557437082971812e-03 -3.063883403145343035e-02 -5.458531010950509049e-02 2.026716756508035490e-03 1.191607976522244677e-01 1.988048849878059632e-01 1.400980526816922832e-02 5.369133043495137071e-03 2.745526833833039290e-02 2.735792773382357199e-02 1.527150260229652445e-02 2.222229702491368072e-02 6.885928097933374792e-02 8.384329627960287329e-03 6.873779159505084097e-18 3.343977558480859538e-06 4.827013558156191195e-03 1.561999826192166877e-03 7.302035802125702516e-05 4.416976187483288960e-04 1.014288507776671313e-03 3.577309797406548648e-03 2.636024568286708904e-12 2.732392208724174383e-03 2.147021721660825888e-02 1.425525058776234109e-02 1.049808299542800451e-02 3.550003858147893904e-02 -7.593487190214685476e-03 1.881672295767577996e-02 3.077765713792209382e-02 5.350999116197226180e-02 6.423538641616333145e-02 2.240472632183614435e-02 2.716406591546757440e-02 3.282349872959412773e-02 2.640415632876872906e-02 1.511215922569750236e-02 1.251479867702526803e-02 6.504884620287557095e-02 6.697288451100555373e-02 3.309255404599836803e-02 6.833545497094065113e-02 2.997361186552160300e-02 1.163562300950471001e-01 1.565346395659178436e-01 -6.560202629768706328e-02 -6.154836443729358713e-04 -1.899669044109927526e-02 1.832354993817700631e-02 2.569718090773455074e-02 -6.656446210730407320e-02 -1.402504715937023371e-03 1.473598820447189508e-17 4.549641692763223971e-02 6.814901934780227821e-02 1.940890988768585734e-02 7.352254928953753889e-04 1.739179597740547062e-02 2.074720260214146100e-02 -1.983852558453806994e-03 4.826981391458626952e-02 -5.289800884179280890e-03 4.159062881542038154e-02 7.677023780030028943e-02 5.844608716061466119e-02 4.470800027620640965e-02 -5.109054787287868387e-05 5.410624474118434085e-02 1.830316112043012078e-16 1.317779121004053062e-17 3.753856712386884759e-02 5.364463123468053013e-02 4.502243929312414739e-02 2.800792693828918142e-02 9.913287551119094598e-03 -8.969030676091652513e-03 -8.042527876917093463e-03 7.348644415632013538e-02 7.863644359901558745e-02 -4.150328154002170700e-15 2.567165087856187224e-15 5.803891589189330395e-03 2.197192494193620287e-07 1.411635529563425072e-03 -6.524976738605600332e-03 7.715212367175129649e-03 1.149675822795418606e-02 2.407326351005312920e-02 2.456395128910733583e-02 2.021201279207830714e-01 -1.454845040793602421e-18 -4.291142690375308877e-02 1.002669094526043112e-03 -1.385812634633897152e-02 5.685707427445156614e-02 -7.766557431825329296e-18 -8.596658301222276846e-18 1.322154471794961123e-02 3.249445642179780991e-02 1.030297435398319075e-03 2.033237413538338794e-03 2.463361951686383194e-02 3.292084282364642900e-01 1.217007657987198843e-01 -2.478471648631847884e-16 6.303146284997831472e-01 3.742725892102178875e-01 4.325394088363213735e-02 2.368409710441750707e-02 2.319451662097330932e-02 2.716740546837010870e-02 6.251239701318477004e-02 7.770632594677582758e-03 2.007107764673581735e-02 3.608707982352910015e-02 3.887948875146413769e-02 3.693866474747475349e-03 2.897059158063820805e-02 6.949359965953923191e-03 4.247106925478631029e-02 -6.025667225981386808e-17 3.204694981393364089e-02 2.907205277766384266e-01 1.849720698072760405e-01 4.289820687668203791e-01 1.724070813206854402e-02 1.734053183011943777e-02 4.495516156918184791e-02 2.647834263536219188e-02 3.830970876231459821e-02 1.947042890152543559e-02 6.492340623391792964e-02 6.122507182926929803e-03 1.714266242682762134e-02 3.282958180837522791e-03 1.469998916106652655e-02 6.735438180100583594e-03 1.192100549471588646e-02 1.741028927475929541e-03 1.567733250743366669e-02 1.157757776191756238e-02 4.578730734781868636e-03 2.186884730651829454e-03 1.983110400456255390e-02 2.161942248957167412e-02 2.210547143428037362e-02 6.101229877050210471e-01 7.337663925934945208e-01 6.027878525468329807e-01 1.646128117823825870e-02 1.185604206664177436e-02 3.124032576231461703e-02 1.005292132450656337e-02 1.607637810794033950e-02 2.689050083492861704e-02 4.346725245621689548e-02 1.720123006216854600e-02 1.410842912304901588e-02 1.053033548165730610e-02 5.354290817190117402e-02 1.091633276865991431e-02 1.067535730252356097e-01 3.832894444537171541e-01 5.463732608677012159e-01 9.494057514457900915e-01
2.135833130740649732e-01 2.475760314922517344e-02 5.542933772180624885e-03 4.478961349310651096e-04 5.943172292043169200e-03 3.516397512496056543e-03 4.810216138912738922e-03 2.591734638924641978e-03 1.297669595071496498e-02 1.300395187514125951e-02 4.177383845695187281e-02 3.023349298669061533e-02 3.871699263019592191e-02 3.800249679062361885e-02 8.985413169229997110e-02 4.042661476006814691e-02 1.684982441348973486e-02 1.322082219176919267e-02 1.972160834930850384e-02 1.196598852098081263e-02 8.361071815034255839e-02 3.801047367893981704e-02 2.645898580002460296e-02 6.769303053892898603e-02 3.660875286636958159e-01 7.854647253826763098e-02 5.565293155402009639e-02 7.807362906995010676e-02 5.023837001438406086e-02 5.457208115445918301e-02 1.859600961738438196e-01 -1.823240676709273270e-02 -1.248077098626008616e-02 6.523620289409591663e-05 5.077682896112542532e-02 2.514818352110236716e-02 8.091182309822860658e-03 2.563557700107980258e-02 8.671721074433552667e-02 1.256413296144065148e-02 -5.649805534433379600e-02 6.858849310773998528e-04 1.829969970802399273e-02 1.301077943499530983e-03 2.441324080662465520e-02 5.655743204094504373e-02 -3.343300892254458950e-02 -6.492002240966899401e-03 -4.880323243146297146e-03 4.027192913560596194e-02 6.600138716186626509e-02 1.502724504930596540e-02 1.578722392002481975e-02 1.598512430461315748e-02 4.590648962376836506e-02 2.865385551657554167e-02 4.660082932436242914e-02 7.071318319563933696e-02 1.581668238639710178e-01 1.685625846840236794e-02 -2.359239285767733008e-01 -6.598021597784484160e-03 -5.070506699682422735e-02 1.519822297904107611e-01 -7.481285896333170865e-17 -1.196291185417768660e-01 9.870249686546833656e-02 -4.587541008828075046e-18 2.848422712759477175e-04 -4.055483645637260152e-03 5.234783026399641512e-14 2.743644771397920137e-02 3.759137720901024882e-02 -3.121969167277070453e-02 1.430323208372430432e-01 -5.137713848941219755e-03 5.138439681264988720e-03 2.475190394153488349e-02 3.595635014253095035e-02 3.896784075217780974e-02 -1.281108603207627497e-02 5.622045462303183749e-02 3.373078200245267690e-03 3.982721642098317749e-02 -2.513888506073242889e-18 1.049415295385106722e-03 8.628576050818952126e-03 5.799285112613933313e-02 -3.153468661392020561e-02 -2.165258624300902754e-02 -1.040556134212953458e-03 4.245753789573373377e-02 1.817693875417890001e-02 5.898659204709996717e-02 4.136779621083083031e-02 8.599761865424405960e-02 9.080443307694165748e-18 -5.690937734641809270e-03 8.719080890903599390e-02 8.228455351614110628e-04 -8.115205143753045269e-16 -9.274077075546546811e-03 -7.886614206921455789e-03 -3.489172609629292987e-02 3.515188151595718263e-02 -1.147939393742292841e-03 -2.586374426124012776e-02 -4.066017935849886022e-02 -1.705807699493179031e-03 1.590934157977027044e-02 -6.612326227898784881e-03 -2.492609745305511895e-02 3.993916020061018951e-03 7.889622340753663782e-03 5.400152816785127458e-03 7.806668503278072109e-03 1.323877827595782916e-02 1.596363206249839638e-03 1.813878222676337110e-02 -3.517235310445900043e-02 5.766944695482881800e-01 3.437241068525782040e-01 4.063437154989776173e-02 3.849589119813301256e-03 9.005697489376844611e-03 2.538722805181335435e-03 1.802720182107753200e-02 7.591784251207401094e-03 2.104573570837785929e-02 3.774322711342515801e-03 6.224220463935487696e-02 3.358179376590699622e-02 3.641821549163189869e-02 4.377483660827155565e-02 3.630713608047628216e-02 9.740181232672447814e-03 2.288933855009931545e-02 2.306028430721254047e-02 2.316950672495015490e-02 2.418689842228858179e-03 4.354495516905679819e-01 7.866914837735611288e-01 3.768529989178870387e-01 8.010966674011955196e-03 1.221881187930087646e-02 2.364954958738791323e-03 2.142111558620689651e-02 5.902810534375135240e-05 1.755646312969355973e-03 -5.212066960452618053e-02 1.425248854522169007e-03 4.933698161618210731e-01 5.703221447830497670e-01 5.411624464696067616e-01 7.041201307035056638e-03 9.992435039518523902e-04 2.304420821832797034e-02 2.756014343712411344e-02 5.642773575137910447e-02 2.819684771428197789e-02 3.018808269846864900e-03 2.876789094673898751e-03 4.065689028276662370e-02 2.194018691254385339e-02 1.768785041003945527e-02 9.844070455184561821e-03 5.412452432125144375e-01 7.607006248056819020e-01 3.783003578839435055e-01 4.370324087604666660e-03 1.858496804106503711e-02 -1.341439899003951403e-03 1.301198365139103787e-02 1.027124181814255410e-02 5.556458207561374568e-02 5.984246102715568699e-02 1.374144923146215225e-01 2.952984518656806209e-02 7.564483657860340826e-02 4.344781672216388446e-02
1.358473100609116357e-01 -7.073650729623778832e-02 5.770068511734257471e-02 4.643993121924840789e-02 2.553068671182039812e-02 3.714817180276361319e-02 4.576401071948793531e-02 1.128869563002198213e-02 2.179162202056838041e-02 3.023963028180711582e-02 1.005089859789578544e-01 5.021351237018807645e-02 8.799061629911441099e-03 1.931319292311464014e-02 4.200293327087399609e-02 3.778357601481148464e-02 7.184217832515775448e-02 2.054054418401218967e-16 1.044259479656394403e-16 6.188096787723166431e-02 1.805792954435521402e-01 4.451292824283356597e-02 4.845125587736213296e-02 3.627374499274810954e-02 5.282613607832040842e-02 2.380906386996100091e-02 6.996905671480858901e-02 2.445753590695073573e-02 1.692228839411146282e-02 1.262137418712126931e-02 3.475614978408584493e-02 5.548745448004072101e-03 6.912873398841578622e-03 5.703747778462083320e-03 2.569034557419997561e-02 -3.917749389678848448e-01 -2.146899859184353276e-01 -7.432662780950756443e-02 4.311722269485637749e-02 3.817392002548788271e-02 2.874795674154493122e-02 1.935321083499852435e-02 3.754373009423665752e-02 3.050725194336477084e-02 4.685168088089428778e-02 7.531660591490187129e-02 1.013851194590079385e-01 2.721087473806854066e-03 2.233102317388241387e-03 1.318359693414832695e-02 -9.817115566074950054e-05 3.504548009408698511e-02 1.506383617594003223e-02 5.946544279493937851e-02 1.457969303313156328e-01 1.852744821784808871e-01 9.455176259421624319e-02 3.236365728862119484e-01 1.668998339718055501e-01 2.995628215695617103e-02 -8.028232446755126372e-02 -5.338214190687698227e-17 8.781377380207061514e-04 -2.512884605002925747e-03 1.994901959108895742e-02 -1.148144944938932557e-02 2.923459029293129507e-02 1.596120670050031752e-18 2.640652028034154095e-02 4.848701466512637948e-03 -2.347685459561689212e-18 2.376110010991323504e-02 3.125434347890094621e-03 1.293282378852647629e-02 -2.120877602345322618e-17 1.556157506052907244e-02 -8.114767947871470710e-03 -1.471398688031204903e-02 -1.290496764487425755e-03 -6.098318903554146575e-03 -3.557134418665673707e-05 -8.797556842884699316e-04 3.570414225133865943e-02 -2.720730937852329848e-02 4.486372195326680801e-02 3.412068515261403728e-02 3.869506569685014080e-02 3.936381192332514406e-02 -1.656699604567544309e-02 1.117295402747664783e-03 2.776002060192196350e-02 4.497991704417810037e-05 -4.362553532968245842e-03 -1.057859022788960792e-03 -1.453419618638851745e-04 -2.531309277966539359e-02 -1.715227088213528220e-02 -1.041009286332273721e-02 7.545729469099628868e-04 3.462244333364308813e-02 1.039597181008029055e-03 -7.810816774499493124e-03 6.903630459039676939e-03 2.033354055201906305e-02 6.870687851179062451e-03 -1.043910151180449560e-02 -2.273312806435326058e-02 -1.933739911270043396e-02 1.308669229244115335e-02 -3.051398526499106682e-17 6.277035076965189683e-03 -1.662699665904477514e-02 -9.380095105202773328e-03 -9.071986997690814755e-19 1.633838670145396348e-04 9.182869523026986058e-03 6.581664819836598873e-03 2.051934613866982849e-17 3.457266933737208198e-02 -5.829483097999878932e-03 6.535580713498104766e-01 3.892311293179595566e-01 4.761951580190140754e-02 3.192435941573561198e-02 1.590314778796112286e-02 2.990330193646376436e-02 6.194605642715473454e-02 1.286392039610121524e-02 1.907480668086247375e-02 4.143272733929467821e-02 3.173095972259790121e-02 2.228384447865542489e-02 2.601587056322143821e-02 1.112920587306619712e-02 3.093869623996007082e-02 2.003127910183146940e-02 3.580370033001602920e-02 7.721309339805771180e-03 1.241698870437983084e-02 1.493345735582743590e-02 6.461546495792090672e-02 3.910434342328120150e-02 3.952675914116352690e-02 2.319961596465242373e-02 1.333786159231145282e-02 6.979431079748225075e-01 7.893734552052275033e-01 8.724424924399878378e-01 2.778316081204223464e-01 2.606389182651189240e-01 1.298615567517725222e-01 2.163359805591708318e-01 1.614274768209591759e-01 4.889964860302837280e-01 2.526532901188688895e-01 9.132285597660382281e-01 7.813945053195134882e-01 7.650251571749621116e-01 1.999534824759329962e-02 2.054128297257709276e-02 4.301102037565043140e-02 1.190813727878324568e-02 3.129829260818185743e-02 8.215321348309954919e-03 2.963347133755443730e-02 4.552787938328955314e-02 5.663367636901532609e-02 2.207227244098552282e-02 1.146570663189672330e-02 1.875529955907936572e-02 2.028063669538716085e-01 4.457573023989263028e-01 2.877850974413857510e-01 1.063854138936920490e-02 4.231564711933270134e-02 6.449330706941518165e-02 1.019909949588723164e-01 3.867032343391987176e-02 3.317434620265183320e-02 3.534286491188228657e-02
1.370383984165899072e-01 -5.138156044270934808e-02 7.394855053295480707e-02 2.134948622017342548e-02 4.268347612195284857e-02 3.359766351405998192e-02 7.750124553076084832e-02 3.433169913407976837e-02 2.750197979740671636e-02 2.957689918384595607e-02 7.787299315772036601e-02 6.247923143372986071e-02 1.535804714652227315e-02 4.583610704034186967e-02 6.253201866714656887e-02 2.102159331186849098e-02 5.432499172777847246e-02 3.859158543591776486e-02 1.942216377626512823e-03 -5.026356901251330389e-02 -3.216929260366395427e-02 1.983203898032251736e-02 2.850221965358086182e-02 1.730945778935031112e-02 2.178895308841997600e-02 2.796055484096116733e-02 8.700680626154319586e-02 1.693362062844098320e-02 5.975390021626817219e-03 1.078870962369773805e-02 3.712541280862604633e-03 3.828171909275204614e-03 9.621351449305634573e-03 1.011077105358146537e-17 5.128712471555982794e-16 2.035796817600688812e-17 2.036163935430552584e-03 4.939642113164396638e-04 2.771480143257626372e-02 8.732542177314932957e-03 1.670851348772395620e-02 1.600107997806896401e-02 2.151585560406935743e-02 1.830351703385452514e-02 4.366104275389696976e-02 5.164507390226759365e-02 8.369569115783193070e-02 1.049643976007983044e-02 4.134748049995188440e-02 8.795785760844990198e-02 -5.758528128115779875e-04 6.029106025761056115e-17 2.319894129356900661e-02 3.246099931504894259e-02 8.835910725468577864e-02 6.728615113344912912e-02 6.318021622997201103e-02 1.341082007610916449e-01 5.153275596951768789e-02 5.050823369313336975e-02 -7.975532440423611924e-02 -8.930414533083189804e-03 9.483595291635674163e-02 -3.263754357090230263e-02 3.009845805893184012e-02 2.640438457439864468e-04 -5.989206193804420819e-04 4.972434288643479139e-02 6.465645335290449991e-02 -5.267491886800637277e-03 -3.715875793878208391e-03 1.303742919776589883e-02 7.890216301077680391e-03 -4.167527572384711273e-17 2.343956434477161010e-02 1.971022187017511512e-03 2.025492671286842350e-02 -1.131127102464346557e-01 5.955442052611829851e-04 8.528675697928839172e-03 -6.740567348635644052e-02 -2.901854146416986413e-02 2.427256666693183764e-03 1.168089125609811374e-02 3.261589627365768518e-02 -2.358532131568693777e-02 2.516597915242078931e-02 6.805790964733054657e-03 -6.429858607975963683e-18 2.109798253419610872e-02 -2.813175745940924705e-03 2.701546420180773317e-02 -5.624498262911453023e-02 9.873845822043468903e-03 1.651461413087286501e-02 2.016749218578347408e-02 -7.243969362494676913e-03 1.221356256889429208e-17 -6.413085108594987482e-02 -7.531222598612441260e-05 2.250089702265842956e-02 1.280687487309453828e-03 4.844911652477194158e-03 -2.112069101618500581e-15 1.922693735592968453e-02 -2.884898598971324556e-18 3.660864785021262363e-02 7.789235956759424373e-02 8.220148545878856738e-02 4.374265336697565759e-02 1.098594423125859032e-01 5.244019920843109326e-18 3.164323999908404822e-02 -9.132655068687799138e-02 4.194016734596117506e-03 9.135056983860459371e-04 2.088812592996979953e-02 -1.820305665935128756e-02 -2.209145626661292508e-02 -3.170153095863103426e-02 6.533543166182027573e-01 3.817552585778844487e-01 2.892489870772349883e-02 4.059166568946757231e-03 2.095891227130694534e-02 3.853423529963988237e-02 5.295143116886757290e-02 7.506657705415274717e-03 3.668425923965509816e-02 2.519180566970818935e-02 4.621340351665917390e-02 1.775840474357498824e-02 3.008075169329747431e-02 2.021536821569807491e-02 2.471783984499936401e-02 1.092275197665420160e-02 3.248038075653445628e-02 5.626678890965780729e-03 4.247772893179581621e-01 6.965232478092969748e-01 5.422176804201359612e-01 3.857885841240069492e-03 3.046562329770940991e-02 1.971094837398714578e-02 1.599072231845364081e-02 1.550550682375479578e-02 5.059201761469613728e-02 7.013727826890354781e-03 1.591538526339061899e-02 1.110644660643529940e-03 1.842922347093660373e-02 1.781328352373606630e-03 1.184725028836204098e-02 7.374396875420153874e-01 5.724735604896589169e-01 7.836813699871756356e-01 1.825549272103630868e-02 1.153396891790460163e-03 1.417126859669607178e-02 5.029594443054569673e-03 4.209112129294428212e-02 1.749931636147642952e-02 4.121962289986862149e-02 8.430095168721748675e-03 2.923046947815007013e-02 2.476630197235801453e-02 3.579948882900537560e-02 3.887074819799824724e-03 3.322403218779459988e-01 5.289326591976480918e-01 3.040552173058055363e-01 -1.137163630285636592e-02 8.162991954522635929e-03 1.622859990190203203e-02 5.436801886123672489e-02 6.367688105566907120e-02 1.194242637459602963e-01 1.037216171928520697e-01 6.809690669762664961e-02 6.588459713843961008e-02
1.386623545054580138e-02 1.418209985566670946e-02 8.015963921059045261e-02 2.374096931484172973e-02 3.739990672504081570e-02 4.831581772981069500e-02 1.769470090481022051e-01 1.284357290993205086e-02 5.635340593095212319e-02 6.287591938619986609e-02 6.280580733464394272e-02 1.799887157616752303e-02 2.193023926942294644e-02 1.563371234233601470e-02 3.733841957853575432e-03 5.623790638735786523e-02 -7.769249201745126454e-03 1.404802474698580231e-02 7.900161058228181836e-18 3.204661591504390228e-02 9.805398114203574267e-02 1.251768171129074778e-02 1.631562594499638350e-02 1.434158945399040784e-02 2.467097834015931704e-02 2.892321500743822577e-02 1.092009199286045618e-01 3.649993740438570167e-02 4.332082153021766385e-03 9.644870031008273906e-03 2.930649313984506552e-02 7.477099588246007011e-02 4.685891628100004314e-02 1.180583265237171962e-02 3.698490269098007356e-03 6.132111090578453819e-17 1.109785074972140286e-14 5.340031324929012157e-04 9.253655622473284242e-03 8.711248282588261380e-03 2.550853551127826857e-02 3.986794053867311755e-03 1.993439232189832266e-02 9.453664828589562709e-04 1.957312853755368079e-02 3.947800527081704591e-02 5.336586341583268028e-02 1.121879716975052697e-02 1.046818819643587431e-02 3.416888054683583226e-02 4.352570892458121343e-02 1.325137993148458801e-02 2.728735244507686855e-02 1.509281961799264875e-02 5.594868567368383588e-02 3.103150369321679494e-02 4.487792188821394801e-02 1.189187255735038411e-02 1.303028158310785278e-01 1.443801156712445910e-02 -2.382814422659656345e-02 -2.508425967876507930e-05 -3.435752912138541601e-16 -1.285458217039012890e-02 -7.160345461800230252e-04 4.881323652138949416e-04 -1.396616512549731403e-05 -2.335259823233723823e-04 1.623504820765356293e-03 4.298982577249269889e-03 1.050223913122570697e-03 7.972304005441145647e-02 1.802729016557049618e-02 6.473658508353404700e-03 -6.918534775798913447e-03 -1.858130146728399767e-02 -1.817153113533818629e-02 3.157707066092418913e-03 -1.139687800532241391e-01 2.919362065453273009e-04 -7.183509178462179256e-05 -6.914780665291718129e-03 -2.548754318327991980e-17 -1.040172141497698599e-02 1.590821861690678435e-02 -1.268903930988699589e-03 1.147015948136022312e-02 8.884102240819536422e-06 -1.330743462136870147e-17 -1.014644682442861906e-01 6.003374912697156257e-02 4.704690809167037802e-02 3.475146165864777259e-02 7.092391294527157844e-03 4.710417929818776911e-02 -4.662810112517927486e-02 2.160028775641259849e-02 4.204923954809363963e-18 -1.949499365677369658e-02 -7.761674450414577579e-05 6.652451340096550078e-04 -8.064335535359993049e-03 3.558072775281807820e-02 2.764961104959595337e-02 1.493656222747430720e-02 -3.665651708136400978e-08 1.336698864326358455e-02 -4.261100883546417122e-18 6.590065109593526947e-03 1.329182380272157793e-01 -1.648243479370239621e-15 2.292302941598867855e-03 3.267824709137228441e-03 1.174327845967624157e-01 2.941452206718905837e-02 6.595345853853923183e-03 4.536826063392193631e-02 2.785244690576782478e-01 1.073081446410956985e-01 -5.937753769562188972e-04 3.369714210434848756e-02 4.422870665568381238e-02 5.215381337423511537e-02 2.278294649268520003e-02 3.041045528841206796e-02 7.462914140248910755e-02 5.388391772453000189e-02 1.522298277801145856e-03 1.034169999877403330e-02 4.769832655779298811e-03 1.833406197779370236e-02 6.883545163432274686e-04 1.234381587707956106e-02 1.993070697127318491e-01 -1.511212652004553514e-01 -1.947498654882292290e-01 1.040164294042696946e-01 2.253298577712422430e-04 1.303982562379171241e-02 1.756124612203948873e-02 2.860453642119013470e-02 4.698872591362040704e-03 3.797551960493735640e-02 1.071463150307565629e-02 2.088300012498853220e-02 2.923251249031576585e-02 5.268969921241119930e-02 5.723369976528814261e-04 9.300694957203984453e-03 1.191519391761163916e-03 4.593707426647094527e-01 1.000509791295486339e+00 5.122473932212242564e-01 9.029371343225485588e-04 1.204476466325154344e-02 7.085060569856135165e-04 3.960209357745911145e-03 3.085179467647339215e-03 1.832630985370004364e-02 2.038194591478323381e-02 3.834012413499620492e-02 1.087891359279445115e-02 4.699527276870757442e-02 1.198422967652108644e-02 2.448001372098053061e-02 4.977205867590642208e-02 4.121670131060089365e-02 2.511658921409598219e-02 1.830613626560807436e-02 3.346234481739055605e-02 3.368657341231678232e-02 2.252899379011820240e-03 1.583423234647819700e-02 1.323503848981323616e-02 5.780031344935730037e-02 3.738327760588087273e-02 1.080729408062912206e-01 3.316122978839444180e-01 4.686681393168161125e-01 8.731480899367128901e-01
2.602939887255311113e-01 2.041901566047825967e-01 2.708930287610403220e-01 2.320650451444837559e-01 3.005355851170761206e-01 1.272946331688409316e-01 1.890307319055689605e-01 2.684365081782785073e-02 1.210063242765569980e-01 1.659646805006998599e-02 1.802444235692819349e-02 3.410486725880546716e-02 1.480854597433178474e-02 1.013032077869067786e-02 8.279855568275311384e-03 7.105829458211779248e-02 1.264752912187120659e-01 3.366469745955272264e-02 1.046209787798988572e-01 7.479886213543176510e-01 5.552179872683142559e-02 4.323697726051888360e-03 3.576801683368744517e-02 2.091479986213167636e-02 3.121762734639264847e-02 4.687169614464037348e-02 1.332251750096876686e-01 4.628027239891426697e-02 6.069547027608568866e-03 6.528772803455550966e-04 2.778876510153821099e-03 1.104783800851806769e-03 -3.033901069619789059e-02 -3.614783090801413495e-02 -4.775972395495276662e-02 1.348392459806731225e-03 6.921738360167615980e-04 7.665087156964431369e-04 1.004318025284150519e-02 8.344207893576139484e-03 2.137661105262467220e-02 7.999505061459535882e-03 1.900185274568940569e-02 1.280849522619846928e-02 3.510502782479275885e-02 4.715618013186281127e-02 4.761417759910351738e-02 4.181541841327553882e-02 6.475920957267443379e-02 3.021609247675590207e-02 1.520898983790535189e-02 8.882177573824941337e-04 3.163804220381540022e-02 2.137319483237142340e-02 7.607109803290861005e-02 1.290235156423798490e-01 7.428677901398203498e-02 9.655881296922687318e-02 8.558432539698797303e-02 1.944618172815077914e-02 -3.052951931352095027e-02 3.084323540192719301e-02 5.912827083630682601e-02 3.848660893479437933e-02 -6.709229142590607264e-02 -5.940338735807031839e-02 7.224496458456822201e-02 8.047511109300753418e-02 3.171043288627725665e-02 1.739222246938677552e-01 -5.739492388917748905e-02 -5.120021558940342221e-02 1.094713121662926891e-02 4.093000550722808027e-04 1.290850294188792321e-03 -6.555373981430209773e-02 2.270927568653637312e-04 1.504897928318037759e-01 8.376087569480455963e-02 1.985015064688648856e-01 4.881647687626085655e-03 3.653919966220973287e-02 3.885452129606841581e-02 -1.391414084283589328e-02 1.581081543536421927e-02 -1.766481395484735192e-02 -3.792900101515623365e-18 -2.151545042820382503e-02 -9.726080910776234052e-17 -1.782422296818832463e-01 2.473889185158544349e-02 1.063481953797618292e-02 -3.950737185112451755e-02 1.067693106375489434e-02 3.528469716407775514e-02 -2.368325061068399153e-03 1.648756768300252418e-01 -3.756468089374567082e-02 3.974618462258272533e-04 -1.550665403810149168e-01 1.190096272890081575e-02 2.792627056264831552e-03 5.479798745513322911e-02 6.450703917879006648e-03 9.091194894980847430e-02 1.290207668439639077e-01 5.272914155907313521e-02 5.720002806655002242e-02 -1.390919241137723317e-05 -4.045434930101068525e-05 9.889899831016306375e-02 -4.196674622133066135e-02 5.871332368865150603e-04 -1.196970496706474540e-02 9.032218796201967048e-03 1.144617459347820922e-02 3.866263828304990213e-02 2.816611149336190750e-02 6.545002066045343958e-02 -4.153030404340053890e-03 8.250044061409779472e-02 4.312578895281981434e-02 1.017768640828458637e-01 1.817120994322167782e-01 1.045472286333969936e-01 5.799658359884368730e-02 3.629033536417525241e-03 7.919796750919420036e-01 8.555562519288056889e-01 6.101626923842893291e-01 6.477369474896943480e-02 1.076951997616985429e-01 1.050146608517055491e-01 1.045334670572739960e-01 8.864155113312834067e-02 1.221503148419593454e-01 4.471158539437018742e-02 6.468991923171566460e-01 5.852170301713215395e-01 5.586598484005018683e-01 1.308881375468018936e-02 2.464312814243985791e-03 3.869654581061596904e-02 2.724322589634390127e-02 2.958443710088242209e-02 3.155153340339516416e-02 7.876839357564605071e-02 3.885335624249040137e-02 1.640265480789796296e-02 1.059668683799752653e-02 1.301510785236809423e-02 1.006581205276685723e-02 6.101595203439791337e-01 1.086929336643831290e+00 4.148120287370257175e-01 3.616970078964013881e-03 7.371095121734893250e-03 1.127822852275344248e-02 2.117843472724435447e-02 2.633025068287051310e-02 3.221104508596200899e-02 1.576874225812186642e-02 3.269132261873954753e-02 1.585528078936704506e-03 1.913905263603537729e-02 1.903165688125749155e-02 1.364310873433102929e-02 4.284628255617969894e-01 5.602660519024182939e-01 4.816276373682417122e-01 2.399682077514756623e-02 1.668557979616570369e-16 1.420798283165652411e-02 5.583117400784566586e-02 6.818233902848190586e-02 8.322457314958522923e-02 1.154908123218907623e-01 2.918129892525287220e-02 8.451530452233492474e-02 5.622722874928637316e-02
1.199433800739978066e-02 1.721910945919209041e-02 7.883572593208477974e-02 6.213260483308814808e-02 5.427979473238576935e-02 4.852019557472822542e-02 7.862791957627916772e-02 4.413687774768024902e-02 5.139743012778560938e-02 6.977355316556949072e-02 1.305297280294026030e-01 8.918236239572975566e-02 1.906256553295394493e-02 1.587670699130392105e-01 4.151301344798905696e-02 3.692346791687762652e-03 2.457026322246138803e-03 2.155860693422089955e-02 4.804304064155399435e-03 1.443331630046759684e-02 7.262524252277165193e-02 1.872726285941198748e-02 3.312929172744252110e-02 8.472465275019566353e-03 1.717309672045539901e-02 1.206269704876364660e-02 1.751376740546374544e-02 6.549446832020069595e-03 -4.667887401260142205e-02 9.268019836993168087e-03 5.087486434186288924e-03 -2.869191876438842090e-02 1.663997338560432947e-06 4.954773110601122868e-03 2.066749360798922634e-04 2.366485020894802367e-04 7.942670107388592751e-06 1.490116119557015069e-08 7.247810468698364338e-03 2.679527379100380986e-03 1.120621238821167820e-02 4.129587674449623422e-03 1.479046086206778007e-02 1.589458551513053515e-03 1.569075844127939753e-02 4.306668125993146773e-02 1.027616767919252300e-01 3.901194259198465398e-03 3.994795887355494235e-02 5.589609332857974067e-02 5.424827796373619254e-02 3.844110737322823518e-02 1.160132027063506127e-02 5.640879860697233655e-02 8.818169195171300656e-02 7.972299370079626024e-02 6.452065285910386916e-02 1.277046109576768229e-01 5.327440643378890395e-02 3.816532174851015174e-02 -6.425795434853973087e-02 -2.246239721310911688e-04 3.225321920218908456e-02 -4.150676374749345116e-03 9.870099940440557260e-03 1.657103649995423636e-03 -9.466871790595940677e-04 3.598775919582242883e-02 1.017066618534643273e-02 1.982216981683855919e-02 -1.394527142475309248e-02 1.034339774308420390e-01 -4.493334123048877164e-04 1.814270170658390663e-02 1.551658255889059868e-01 8.136832696828276423e-02 -1.263692969729737180e-05 -8.893871458789960041e-04 2.230951256308152443e-04 -3.192274758633324716e-02 -6.070598986895447274e-04 3.389404318091025076e-02 1.765484359813344895e-02 2.050928805926732768e-02 7.508638072084591886e-02 4.650403175888443569e-02 5.050692871814303070e-02 -1.814544825537528494e-03 1.808058564308854393e-02 4.409202756121026467e-02 2.991388680361591057e-02 1.319229729414096892e-02 -5.968236962186665440e-03 -5.785777705366348250e-02 -1.167369239188749072e-04 -1.862310279308234187e-02 2.507978736006393969e-02 1.858731869895502196e-18 1.113521338702716258e-04 -1.559511790117615315e-02 2.052039711521952067e-02 2.236264018957409387e-03 1.199339167549006054e-02 5.053219663177899862e-02 1.111312366267203783e-02 -3.605403144648199540e-17 2.161008490049048589e-02 5.922561776554086110e-02 2.551232720227854592e-02 6.311551545978505429e-02 6.896087795237756946e-02 2.527020919592961289e-03 3.047284717899836903e-02 -3.341792054001533480e-02 1.063108212610423299e-03 2.897013616655292299e-03 2.351242836868391861e-02 1.597700997177082082e-15 -1.769868005183050305e-02 -4.092115979799375997e-02 6.202286683983989496e-02 7.316489723748087481e-02 7.485829094040256926e-02 5.454893199197145132e-02 2.442141235488907644e-02 2.899360586543107271e-02 6.652812090541426748e-02 1.037794572363149280e-02 2.089501876800285013e-02 1.560194518538980321e-02 1.736405916859147872e-02 1.931265695708602424e-03 6.853429964393961682e-03 6.094333469774991796e-01 8.858721047678888572e-02 8.466636723879324145e-02 7.454179937155760993e-03 3.347448339706495836e-03 3.722262981470848137e-02 3.089399332632865661e-02 3.739455168768941740e-02 1.254964373483018203e-02 4.820373560272976643e-02 2.985478428901669132e-02 2.017849028419755816e-02 1.360063416240142771e-02 3.502283378588180035e-02 2.205076435587568877e-03 1.472524118861615605e-01 -2.980494295066748456e-01 -8.642796681222679467e-02 1.469889364247333186e-01 2.259735189321455257e-02 1.860683604084919715e-02 1.794124059183275360e-02 5.174182963336317227e-03 6.471613694655318702e-03 2.511425364475494346e-02 1.621038735554431245e-02 1.317904017155938079e-02 3.291761728380388635e-02 1.324578014920057591e-02 4.413622073675373819e-02 3.669055123392136412e-03 2.971944259365747604e-02 1.880229008643828290e-02 1.850020486692062829e-02 4.842238732898966028e-04 4.045461624322428951e-01 6.245256266378348409e-01 3.812970660804896883e-01 2.034336468135451714e-03 1.142889843414480154e-02 1.072679134480136222e-02 4.001142674258977711e-02 3.395080369434397977e-02 1.188883094343446550e-01 6.017113975301251194e-02 7.347982036740459411e-02 4.631886478661070145e-02
1.284612151957288673e-01 -5.741458338402652406e-02 6.244169887897677018e-02 2.046673267653348685e-02 2.099023798285273856e-02 7.083925740499687207e-02 4.534204611645881872e-02 4.882588251038784584e-03 1.432041391338558650e-02 3.839374298135293795e-02 7.859362736618209777e-02 2.637896981460399401e-02 1.221138886043973519e-02 2.234288283757668997e-02 1.301784123050381096e-02 2.166964635983594303e-02 4.490722597581877501e-02 1.280410077557104023e-03 1.908033368230976103e-17 4.887387988254485865e-02 1.104274312809819109e-01 3.312103417694137880e-02 2.609906636115540135e-02 2.178814263838145179e-02 3.358552604218346255e-02 4.809726315981522476e-02 3.198001376422292957e-02 4.721878401509114337e-02 1.902504805020020220e-03 1.242284837632156441e-02 3.164463815039778721e-02 1.014776103997413505e-02 3.476680488054329811e-04 1.820564625429997238e-02 3.039498933664196850e-02 -3.274486682537540716e-01 -2.812024813680865654e-01 -8.249111195792999118e-02 3.105261214894522448e-02 1.578799376361311041e-02 2.222405384308494616e-02 1.795498231337191303e-02 2.577389418761647996e-02 3.913975772180239981e-03 1.891414638432121695e-02 4.340687783385385051e-02 6.688054714213685636e-02 4.974016920127399995e-03 2.722262909040387008e-02 2.200159063073496568e-02 4.649838440361882957e-02 5.429421148810683641e-03 5.382937114090628403e-03 5.265887291553310486e-03 2.806681693849318385e-02 1.452411032074862884e-02 -8.634897179714538795e-02 -1.175711888333180050e-01 -1.740067168555960164e-01 6.827072522509805563e-02 -9.704449952292103221e-02 -3.365938322900397135e-03 -8.669627304653591479e-03 3.771255153822658003e-03 2.378611792906026173e-02 3.244567231538627448e-18 -1.381232872947702843e-06 -3.943218276572462919e-03 2.956235274728261433e-02 1.882410790242299709e-02 -3.937267855294366216e-18 4.482836054069756077e-02 -4.718155668983544853e-18 1.554395798646365052e-03 7.668289990702636255e-03 2.796183863081602369e-02 -9.662725852183708724e-18 1.108920339199544728e-04 -9.664782817430427400e-03 4.392223080849487482e-03 1.618219955114252873e-02 1.291534884667653281e-02 3.774279514694720700e-02 -2.243926914076565041e-02 4.688756848214372525e-02 4.104593985222169603e-03 1.151483066716824273e-02 5.504663879417685118e-02 -4.976187361766089279e-03 -1.140128514199610817e-02 2.440807326980117681e-02 2.802547933090078144e-03 -2.312826309461469936e-03 -6.866359457616112369e-03 -2.877299630475017942e-03 -1.471568905517549666e-02 -1.321361567008309465e-02 -1.723858704506610676e-02 2.993365822503578268e-03 -3.580080806018016367e-18 2.012137537801750309e-02 -1.943876290547473171e-02 5.188093608538835166e-04 -3.271789214637875064e-04 1.225855073693898839e-16 -3.065912564893959798e-03 -2.641532188771961859e-03 -8.373420116919828111e-04 -7.004769382689840507e-02 -6.091696713966527135e-03 -8.620031813725841270e-04 1.814308599257241639e-02 7.935032377289342846e-05 -1.375497102182842518e-17 1.185868293368944927e-02 8.002270373069157727e-03 -1.388820557393044275e-01 -5.158243486784453574e-02 -3.476728378941634778e-01 1.969451363045821579e-01 6.408061452441325967e-01 3.486730858967874713e-01 4.203920763016966156e-02 3.455846995293336554e-02 1.586075482419680230e-02 3.801689643409238151e-02 6.393280275721291428e-02 1.790270568172942778e-02 3.014355062222920031e-02 3.992229681790834123e-02 3.825181080840796277e-02 1.626451129746800395e-02 2.797748049523886282e-02 1.388509296610403350e-02 2.767638114179436434e-02 1.020837438578247637e-02 3.963413852466358966e-02 1.121109217811421324e-02 1.304116926606808946e-02 2.547312357396629839e-02 4.420647218561853109e-02 3.707028815632763125e-02 4.139409724513561789e-02 1.924044731711862383e-02 2.075200169295190675e-02 6.698133489899698034e-01 6.800850144224519500e-01 7.316866430613772065e-01 2.875460430509556020e-01 2.190837262016634424e-01 1.407809626095769817e-01 2.111219652363907073e-01 1.419194539566420443e-01 3.564123692479233552e-01 2.448130888683743844e-01 8.251874433085749638e-01 7.672744929509270895e-01 7.392390568978559617e-01 2.398966400637998417e-02 2.128256990013472488e-02 4.775638083648873350e-02 2.304535434264369520e-02 3.501696476937548458e-02 9.619843630534067117e-03 2.696096808042282994e-02 3.737973390692263947e-02 4.783757103652166304e-02 1.205939220524414671e-02 1.610273783905191367e-02 1.166579499143084384e-02 3.163428977100241068e-02 -1.217057832564728131e-02 1.027280586010983696e-02 2.406906922784353073e-02 5.235861761772525602e-02 4.245318671396005293e-03 1.231149417426501413e-01 4.936406599838716325e-01 4.365419645602799381e-01 8.599662576160271854e-01
5.157122278490663264e-02 9.057880509543943248e-02 1.501044791953555502e-01 5.568634444149758722e-02 4.088323582920713739e-02 6.436718764143438409e-02 5.120484579569328093e-02 6.992767982786665670e-02 8.371478067844630166e-02 5.302215994454392151e-02 -2.982499933170932455e-02 -9.105749945539988455e-02 -1.887513394907382874e-02 1.464414019358543234e-02 2.967241195042442936e-02 2.517623045343859306e-02 4.166824490618283028e-02 2.390237544697940489e-02 4.344888693954609971e-02 1.610602921687835642e-02 -1.461343484348069377e-01 -7.360332966211050043e-02 -1.317537222525026197e-01 2.194352706389176080e-02 1.509517166115948578e-02 2.720013640222214424e-02 1.031915049170824689e-01 1.899101238152378981e-02 3.279011738371137240e-18 9.548620775382093669e-04 1.175553063476726234e-03 1.641122601229567934e-03 1.335084025583858653e-03 4.026399029672356536e-03 2.266931807221882200e-02 -8.373856905689197605e-01 -2.704078244555146449e-01 -7.837498150811361697e-02 3.332195613061649658e-02 2.453640579899559609e-02 2.570658511300152882e-02 1.563309877438764234e-02 2.307832898740314401e-02 2.167847579255282459e-02 2.965065388413645753e-02 6.827671712508932844e-02 1.017137639261391779e-01 2.448691000264943685e-02 1.769282505065495129e-02 2.472700954728663039e-02 3.383964578358190228e-02 2.753063015081429199e-03 -9.574978605897252124e-04 -4.991686667439924202e-02 -6.559623171196179992e-02 1.289074711311866406e-01 8.626221735889391673e-02 2.029263451618416314e-01 1.376384597778956553e-01 1.300331413710678341e-01 -8.369656786665981979e-03 -2.054693424884537420e-02 -6.276163268126202198e-03 -5.711136228104037732e-03 -2.858238797765020709e-06 -1.415808057833924279e-02 3.986686567965237911e-03 1.024299882931468605e-02 2.059345483651018726e-03 9.643646288231709843e-03 -8.752937905697130705e-04 3.485610484740107179e-02 -2.789231881620552692e-03 -3.656777413730045306e-02 3.184953644878295298e-02 -2.976366714771788366e-02 -1.388393867375061971e-02 6.166592836254641136e-02 -6.563816938270601586e-02 -1.684444014838802064e-03 -6.115672426771865539e-02 -4.915153349917858461e-02 1.470731213440077713e-02 -1.388093737490209145e-02 1.140890496225181738e-02 -6.179292455915579947e-17 -1.046581612949816441e-06 -5.666032248537465447e-03 -1.521505617909308179e-08 -5.968254553163350289e-04 7.167788918201381705e-03 1.772804457194823713e-02 -1.894102389908410011e-02 4.919545971742753099e-03 -4.355547368300541344e-02 -9.769545096599840245e-03 -7.575385558765295770e-03 -3.814226125926294814e-02 2.189968643923090319e-02 -9.058168392519114556e-03 7.884906203601582328e-03 -8.725590419959331281e-03 -3.449427463030675378e-01 9.657476818192005374e-03 -2.137925084779041668e-02 -9.661771104311375714e-04 -1.247524378891884953e-03 -6.009299750869057967e-03 1.541048536193953761e-03 -8.796478965185413191e-06 -5.981097320732244038e-04 5.270469477596634311e-02 -1.425349287751468026e-02 -7.878245422660698841e-03 -1.503644462199511291e-02 5.826611281391254338e-02 3.733121241544441055e-02 1.625367799388957590e-02 4.859234143231660336e-02 -2.032053010668276403e-04 5.623378657684884041e-02 8.473321903836572577e-02 9.379698808209144256e-02 4.663445928200112695e-02 2.807769159209263718e-02 1.260971420144268718e-01 7.724234056569517781e-02 2.635082533537789866e-03 1.030204846181876667e-02 3.017676096777502631e-03 4.822638394483675928e-01 8.459395965916355342e-01 6.640262253616651122e-01 2.561159146972605272e-01 1.501425373914001549e-01 3.049707515840316963e-01 1.136004108406004770e-01 2.465125636829966493e-01 1.961539183770285855e-01 4.037258642112017126e-01 5.646787149005444428e-01 7.407401658449368487e-01 5.642808216415250433e-01 1.472102354572302728e-02 1.303531137671321526e-02 1.274563983957116564e-02 4.553839976722136307e-02 3.296580784804644792e-03 1.561946944541664842e-02 6.616067454933191594e-03 2.787892819803406558e-02 6.682036266285974746e-03 1.487539559200055234e-02 5.516267634486976892e-03 2.595766187821797397e-02 8.390665331838632612e-01 7.524082038794508431e-01 6.475178601724833616e-01 3.268421827395261853e-02 2.658792755362749186e-02 4.560759882625624317e-02 1.731251146396396365e-02 4.066004191016296676e-02 1.448451145951252378e-02 3.201870315529406485e-02 2.528190041957309106e-02 4.444038862840372917e-02 1.335838859603772147e-02 2.218341170704014528e-02 7.074843806473722803e-03 3.300048322293459846e-02 -2.264417065579402491e-03 2.839729114608026128e-01 2.939802649763222919e-01 2.315727907858418366e-01 9.846867506626322958e-03 1.076287904346578533e-01 4.245420553684212117e-02 2.731619800742259019e-02 2.659928219512798508e-02
1.084806948348730760e-01 -6.310196167047105176e-02 6.834888212723037870e-02 4.032470148254146958e-02 1.785457569787403387e-02 1.817273819283546019e-02 1.051610892922726775e-01 2.145180535206081199e-05 3.800021063287156026e-02 5.711491029358661126e-02 8.512321681682374908e-02 2.626915650342246875e-02 2.023265531513563503e-02 1.345865562640063268e-02 2.034640606465636450e-02 2.054752283741041047e-02 -7.018096395629883077e-02 -7.036354284599086184e-02 -7.298473578594923128e-02 8.371994768422234487e-03 7.051321098288028388e-02 1.925326904865028860e-02 2.393979323072335449e-02 1.823926839449545312e-02 9.187606864799729295e-03 2.939214415271542910e-02 9.560816272188704668e-02 1.670622213227427114e-03 8.591990737263561699e-16 9.673905327506410667e-04 5.477208224085075051e-03 7.800768418222310541e-03 5.035205949338242856e-14 8.848623826495133522e-04 1.603315676499021333e-02 6.053040434446817913e-03 -5.327888568499573180e-02 7.358736091971592938e-03 1.366592555498025510e-02 1.207812142767789032e-02 2.280560380799144873e-02 1.718139414750559368e-02 -2.506850958402716701e-03 6.489909283944116092e-02 -2.392487879932147718e-02 2.639180965421784597e-01 9.056476971409168630e-01 2.027014290370076632e-01 6.940942065724091392e-02 3.737110387552471102e-02 3.628577806579640769e-02 2.461065309216269037e-02 3.513697541514668871e-02 5.058893318333596706e-02 7.306512340015428042e-02 9.008131890075288251e-02 1.122510311499881341e-01 6.091335630342303853e-02 6.636210205954035102e-02 5.103676625285048785e-02 -1.213628314812670306e-01 -1.565742899282294368e-04 2.925183671074901154e-02 2.945495337176979797e-02 2.048965535184440329e-02 -5.732966945431938396e-03 2.814094624681102649e-02 3.243383584165978467e-02 6.121102695124507748e-02 -7.364023626910229815e-04 1.036204652677338162e-03 6.735994732816012222e-03 1.149197747403044208e-03 -1.306983428584135992e-02 -4.166235725471728601e-02 -2.299061185812123487e-02 -1.463279598025192356e-02 -1.103099170255531780e-01 -5.813616716304628035e-02 5.413530954518075422e-02 -2.719060468810712575e-16 3.128347048683690484e-02 1.486084144783669618e-02 -2.648181807760171397e-02 1.310013817954530993e-02 -1.386810423128606628e-03 1.786192203606958712e-02 -2.141089978768964674e-02 1.670533538195758186e-02 3.581500605631834921e-02 6.369586074684331191e-17 2.845370284578312495e-02 2.146743186688027799e-02 -2.311835897253169958e-02 -1.780579968359342381e-04 1.792055919823115351e-01 5.066051810804233363e-02 2.304454892710334678e-18 8.651721691602301720e-18 -2.583074199090271833e-02 4.635295090056156322e-03 -6.474550309011644538e-05 -2.350593709372573049e-03 2.750293378551273496e-02 6.510786053567747637e-03 -1.951240379315967444e-01 -1.686353476968219592e-02 -1.603321827053500251e-02 3.924316796938867591e-02 8.758686281480944358e-02 1.098528061023531927e-01 -1.587708286170024310e-02 4.907696473625423883e-02 -8.948115038578653702e-02 6.764965996841737148e-03 1.095380650958640472e-03 1.138807570256373386e-02 -1.626002994155384754e-02 -5.431595689032619155e-03 -3.342556763259280778e-02 6.244854026565264116e-01 4.454541119021153128e-01 1.011780585309056749e-02 2.026927800101059173e-03 3.175119869543870793e-02 5.782436032046191221e-02 6.936303557863515712e-02 1.934920671043552365e-02 2.793256968006770091e-02 4.499229576315557505e-02 4.551639076867848577e-02 2.033299196525093638e-02 1.945401423903556429e-02 5.892186442831680825e-03 2.408117586380597794e-02 2.220652308424817412e-03 4.431839850225791078e-01 8.078109383372849006e-01 4.644677619586618511e-01 7.872458546087427594e-03 3.615770083507256066e-02 8.549505244346309457e-03 3.954984376401123741e-02 1.836592754968017355e-02 3.010792115157472082e-02 2.105140576943686018e-02 5.541308272257311485e-02 7.743859190504345444e-03 8.310729341198868844e-03 1.619499927278323726e-03 1.831391016159912566e-02 6.251752196957015978e-03 1.745137659848721920e-02 8.297620594787050069e-18 7.918730456635599685e-03 1.851722635647628807e-03 1.828936395833246796e-03 4.854597560238187322e-04 1.211151441988156906e-02 7.259064557188917369e-03 2.746106226673080547e-02 1.652052033104748534e-03 5.183016614349480466e-01 6.858493925068633867e-01 4.153369466827193102e-01 3.786925497796375661e-02 7.821641922338179453e-02 2.173301569036816538e-01 2.705683858192512448e-01 4.200920092973015607e-01 1.992504791758164528e-01 -9.604864438331117346e-03 7.290499522057847237e-03 4.961145731126566545e-03 4.883537695726077738e-02 4.038359410544371114e-02 1.227951901482774305e-01 8.873442517124800755e-02 6.469297085067481079e-02 3.792144522732846662e-02
1.074504612841182261e-01 -6.188322750319152216e-02 9.361157217695884047e-02 2.122882293372277754e-02 1.063777768145794493e-02 3.506955442507365961e-02 8.872882016901598168e-02 1.466348926347962010e-02 3.470474911224152165e-02 2.488889100132323645e-02 4.919938372906677154e-02 4.384373128978195922e-02 1.832017150164688343e-02 2.023136410155620610e-02 2.359438961941076607e-02 4.290562935112938320e-02 5.039175327777051933e-02 1.782770501792842535e-02 1.050957909377339856e-03 -4.379804687798424800e-02 -2.428828806029119380e-02 1.264826302736335624e-02 2.605095223376727512e-02 1.710981975736055455e-02 7.260035779926236194e-03 1.758164395034469973e-02 2.028563487476798544e-01 8.841477997360242067e-04 1.510033553304543809e-18 3.640844467781073819e-04 4.696841316640302730e-03 1.080253707448744840e-02 1.606739820172915518e-04 8.868501525531604333e-04 2.692079636145551600e-03 1.010766086788525523e-04 1.427875580476525421e-05 5.467370134117463705e-03 1.330779004206244197e-02 1.751738044147854723e-02 1.342215009816809246e-02 1.606533563476382487e-02 -3.283901795695344451e-04 -1.458989131115809995e-03 -2.394368739086848163e-03 8.348346530349400052e-02 3.587024098975370029e-02 3.065330922448495851e-02 2.805395406266761663e-02 2.643394101072569710e-02 4.632587983761322914e-02 1.719974858114346750e-02 1.485089950574072261e-02 3.488779275569219712e-02 5.806442094093311468e-02 2.113153567321392548e-02 5.447482125760230026e-02 1.193062383645682005e-02 8.187654168477939698e-02 6.117960222533502107e-02 -1.062377389777139219e-01 -2.013408313505380461e-03 8.034849107884685371e-02 -3.256056239112200407e-02 -1.577052826550865740e-03 -2.829279948071699441e-02 6.268513011234060073e-02 9.918769094268815867e-03 9.286115578704572104e-02 1.256216675705081401e-03 -3.898575295348181285e-03 1.182170977968100348e-02 9.304340847928478653e-03 -1.104961333300520133e-02 1.353750079727604992e-02 -4.339570386867504237e-05 -3.259845440752669191e-17 -1.881053350555444426e-01 -3.382805498647472459e-02 4.399256331072687320e-04 -7.634541333577872513e-02 4.298673414209855381e-02 3.472376266746961260e-02 -1.999495604672677089e-14 5.879078954042801786e-02 -3.543335285471225193e-06 -8.353167158858560305e-05 3.221650323683186972e-03 2.227798577046473730e-02 -1.066013637431171401e-17 -5.884944148940718508e-04 4.543700577490719650e-04 9.804892872776525478e-03 4.892600086739608105e-03 -7.935688229143271123e-17 1.985199765617038509e-02 5.996998628609689214e-02 8.289879769453457545e-03 2.136963933724194906e-17 -3.947827183324817239e-03 4.727775542078448893e-04 3.241816387423789893e-03 -3.949546850296965610e-03 -2.309408191453026817e-03 5.776382389583161608e-02 1.224420017510991074e-03 4.391806367135300815e-02 -9.152475298516050614e-03 -1.207947194382870820e-01 -9.907933783882996552e-04 -4.355466074743937148e-17 -4.496225848808146561e-03 5.698078105636514377e-04 3.533533481883954774e-02 1.329256281795976324e-02 1.867958971048218038e-03 4.794791317721393969e-02 1.593907988038975321e-01 2.468973303584157120e-01 3.443191677642226667e-02 5.954360118060529894e-01 3.705125277423114150e-01 2.566525227175989882e-02 8.999938324606137838e-03 1.989893442131131923e-02 3.283587632933588490e-02 5.846092637171772244e-02 1.090360638358760063e-02 3.018514360697292093e-02 2.855917628547400416e-02 2.484593211610829772e-02 1.376594336329400869e-02 3.742919048915761132e-02 1.077701452377091001e-02 3.924659946206350558e-02 1.082510546720573050e-02 3.215206286153241000e-02 1.271136776212612382e-02 4.369224866783822381e-01 6.917360309122547735e-01 5.384364884424159836e-01 1.714343965899713194e-03 3.820363351627355936e-02 2.317937414085250933e-02 2.290339910937743556e-02 3.069687755333628393e-02 5.919930313423404922e-02 1.225501932679931398e-02 1.402643642286773538e-02 1.937849898778295664e-03 1.139418077557734742e-02 2.816410830033681204e-03 1.535145746874172143e-02 2.128272991563806386e-03 1.128991836556264040e-02 2.286425918395935172e-03 7.241175047548359216e-03 9.717074403924446865e-03 2.682657616481238030e-02 9.576724414656352657e-03 2.116262325089412338e-02 4.988385963819484854e-03 5.214363788046478820e-01 7.561837948130376574e-01 4.771174992443086849e-01 1.261606528877148685e-02 2.859257977407760748e-02 4.004410974076100076e-03 1.407047669038163397e-02 1.722819172237200977e-02 3.309713245546207910e-02 9.996259055153782977e-03 1.238325484612690444e-02 4.554113685936202194e-03 5.895763780077607763e-02 6.482249962639372032e-02 1.173861348184395276e-01 3.208355439266415421e-01 5.411394173431527399e-01 9.269522769131748330e-01
1.042709398341544330e-01 -6.197545675798084991e-02 7.392049279838570908e-02 4.329456694452675503e-02 2.696514086556292197e-02 4.176300313871374181e-02 9.574032665778206552e-02 2.082996096066064284e-02 2.450128621173891866e-02 4.707390037639019170e-02 1.271993204647504649e-01 4.096848675258477068e-02 1.456937285040684447e-02 1.901622585276819058e-02 3.412963202457958839e-02 4.839789607343209649e-02 8.202422353910789055e-02 2.578812674168734845e-02 1.267170582078362175e-02 6.075755433604026340e-02 1.155377866176385826e-01 4.732469737007784921e-02 5.474775475824970178e-02 -2.641136634706662753e-02 1.154506133315708755e-02 3.940886757109830080e-02 8.057495495596596335e-03 -2.602751544452942822e-02 3.225342438006831725e-03 1.623523559210225328e-02 7.329160116348747839e-03 8.137306057462601050e-03 3.109402889587934826e-03 6.314867728557424509e-03 9.742606370831749865e-03 9.726992572275095644e-03 2.296204693112817452e-04 1.246912198208444469e-02 2.370098878097844086e-02 3.566205736022707434e-03 6.295954189308296776e-03 4.483425322075281633e-02 -5.079355745878045766e-03 1.243474262008889726e-02 7.178599811801847363e-02 7.617458515693864562e-02 1.182761485726531309e-01 3.002217805955912505e-02 2.457400089110423069e-02 2.951603782945888890e-02 -1.965925473464828072e-02 7.615528977574944486e-03 -7.371469000812585527e-03 5.114122724116787477e-02 1.428187454956291180e-01 2.233413089889662306e-01 1.182551810245775492e-01 1.375927482840143823e-01 1.342991790047488032e-01 4.711837661951605305e-02 -9.633524875669363663e-02 -2.598392815663057211e-03 3.089741559080896177e-03 1.092439315806484519e-02 1.526494504919909817e-02 -3.859050532947816414e-02 -1.477363022132362549e-02 1.847407815745805149e-02 3.497561852534000065e-02 9.065750251118340122e-04 1.773127432195972363e-03 3.009918463551296622e-02 -3.849026573574625045e-03 7.733963292587226965e-03 4.914894236516394524e-03 3.100277833689988466e-02 -1.200032104909048541e-16 -1.257193731552336269e-02 -4.691118922627385279e-03 -2.537733361281044060e-03 -1.187691262352379455e-03 3.253179501521100830e-02 -5.747053291582678056e-18 2.279537925463728798e-02 1.963995801083027667e-02 5.382886275440130347e-02 -6.312236063484575472e-18 1.093078685713484884e-02 4.966820686885986547e-02 4.249906035769619728e-02 9.916255922537253742e-03 4.338302553088579927e-02 -3.847792718779833123e-04 5.682587292658739658e-02 -7.823103824956612658e-04 6.332146780406258148e-04 1.494658071079217089e-02 -6.202932315334896626e-03 1.905294291748558730e-02 2.841689090279536298e-02 4.937693038675958213e-04 1.025769555388688151e-02 2.220677648794697548e-03 7.072050423320462575e-02 -8.532149772499957388e-03 1.525601334561580702e-01 -2.784430575077132680e-02 3.668371391339228516e-02 -1.230667623813739946e-02 -5.019969850739775986e-17 1.481135055873543556e-02 3.561441798235304851e-02 -2.999686258899415575e-04 7.659227018434972088e-04 -2.952049575505928322e-02 6.855156968957464448e-04 1.028640101209575596e-03 4.597362069012536060e-17 1.008082211152035168e-01 -2.415750782193853887e-03 6.222093656441494147e-01 3.982822617030734591e-01 4.298442706437002880e-02 3.187898099489273884e-02 2.435069454296387437e-02 4.624456865595698296e-02 6.100084767900981753e-02 1.004722364572882207e-02 2.053064698757396320e-02 3.596271817587318176e-02 3.591223080543395896e-02 1.315099629785263825e-02 1.908864600730814098e-02 2.174805083441268619e-02 3.283765587873685443e-02 2.167713733123177028e-02 5.210070448977077717e-02 1.097042842230895598e-02 1.692837475093254829e-02 2.906060326374630340e-02 4.448062048301472393e-02 1.276067052013090536e-02 2.967764675262098834e-02 6.800533006686710902e-01 7.865103718533738020e-01 5.340299492672704318e-01 5.652084858642008464e-02 9.729961194198314192e-03 1.891066113419740888e-02 1.850669953568273920e-03 1.961895030084067834e-02 5.021111706735663892e-03 1.152933547159505634e-02 5.442709500268720765e-05 1.056167674068617113e-02 1.211761526870862825e-02 5.661450068504667379e-03 1.684419263302794432e-03 2.004706366943397391e-02 1.037285678181982043e-02 1.660750183904868491e-02 6.222106829031814890e-01 7.023897500355968937e-01 5.844530747044305308e-01 6.266011427330660244e-02 1.160434304092144639e-02 6.274238226814302100e-02 5.320281625606634174e-02 3.141724855985366521e-02 4.776345676211336128e-02 2.152970174756523858e-01 4.982967923947348443e-01 3.445733721427239882e-01 1.215323224800788846e-02 3.839251391963883348e-02 4.844345172263356952e-02 1.235553794938385463e-01 6.951191865452997776e-02 4.016046156889653551e-02 2.958649781466315618e-02
2.168307239595495262e-01 2.071352503884049678e-02 2.934460086899160890e-03 2.622966012061125982e-03 6.412170638545345853e-05 1.621637903791264693e-03 3.845956288967926611e-03 4.506223925279564078e-03 4.405544448628539653e-03 5.263128629621102381e-03 4.792191735939924091e-02 2.018482113353914018e-02 2.483670293644350477e-02 3.126352591285334187e-02 9.416292546232998828e-02 3.916246418834055942e-02 1.453619109873118508e-02 1.601831460189900164e-02 1.039521403757857204e-02 3.966950469972137933e-02 -8.134901755642605370e-02 -6.332823146919347446e-02 -1.354125877407001410e-01 7.570437881540219294e-02 1.766303908007911261e-01 3.787243101796015921e-02 6.394561835868507738e-02 3.810301855735405419e-02 6.033279147469577247e-02 9.528239891586975929e-02 9.029973476221163364e-02 9.146068704298628005e-02 1.013558048610961743e-01 7.041839522575146781e-02 5.039417356415101801e-02 8.670222739630488451e-02 2.286857471199087860e-02 5.855258295579016115e-02 2.713023117249053345e-02 4.046113747653696494e-04 -9.564398922734786146e-02 3.885978788383828708e-03 1.043441243551651923e-02 6.314896489139841876e-03 2.333018767217731257e-02 7.445325960720840242e-02 9.638017807551944871e-02 1.743246511382749997e-01 5.316118696962623263e-02 7.137528709493341783e-02 -3.041232158704589017e-03 3.740867235836385640e-02 4.203580283639265364e-02 7.268839602427858149e-02 1.133309370528711307e-01 1.419088125265337008e-01 9.020134193082907093e-02 9.202076352120822089e-02 6.767772609069426515e-02 5.230349849435764892e-02 -2.222848266194685229e-01 -9.390572766332355020e-04 2.456870733025884237e-03 1.189330240551556422e-01 -7.263121189498940167e-06 -7.726877975688011338e-04 -1.953441835804812718e-03 -2.637046287928214367e-18 5.804571866447883322e-17 3.217024557788376871e-02 5.577075950508654121e-04 -1.434625287066252199e-03 -8.187926289299741708e-05 -2.268725737293642020e-02 1.721411539633507315e-03 2.283125615330132419e-02 8.066371094047950982e-03 1.243792801040704762e-02 2.522779873098439374e-02 -8.888301610967873443e-04 -6.514726626853216017e-04 -5.436191206403400561e-02 -7.198500611398536697e-02 5.933311320625313939e-02 -1.520940250682921934e-04 -1.678214556081862700e-18 -1.388591006228667539e-02 -4.885928251112201665e-03 -4.110262507312471706e-04 -2.691521381827876279e-04 3.292220575074499694e-02 5.346569634449380853e-03 -8.465507653509440421e-15 -2.185259935196486730e-02 6.345776693144940250e-02 5.299860392208639859e-02 3.376555431572968863e-02 2.391441200176177076e-02 1.673973909490135067e-02 -1.438104437063444652e-01 -9.638318081698713110e-05 -8.898563840493775579e-02 -1.367944598160850990e-02 -5.096155384429635476e-02 -1.762882165965804285e-02 -1.601255572664758044e-17 -1.747363935059796343e-03 1.944756596388858699e-02 3.978333090329436161e-02 2.608052822387939285e-02 8.811954732810371727e-03 1.547984234190241323e-02 2.719971207895320250e-02 -1.003382760974449832e-02 2.159826921050978649e-03 1.895565152581231601e-03 2.098725367041288231e-02 -2.528221193450153406e-02 -2.132861511908239194e-02 -4.677549278322003928e-02 4.132024489158956926e-01 2.774677332126888052e-01 2.954155261823765422e-02 2.036569963684872602e-18 8.053086979368042492e-03 4.174101465936834302e-04 1.559280959453106935e-02 4.670355170257326632e-03 1.189016640494514922e-02 7.081014163708460908e-03 5.739386692912591681e-02 1.161637144247725656e-02 3.286057350395048954e-02 1.501749813476013599e-02 2.744969165530015270e-02 6.422825352555177822e-03 1.873595900517957547e-02 1.040771886883636033e-02 3.084531642231438248e-02 2.257875777351418811e-03 5.048780049041553530e-01 7.831494757194648049e-01 4.509171551584063020e-01 7.419506783417600081e-03 3.469871766842778319e-03 3.079231746329264864e-03 5.167275028841259582e-02 4.559024917583882897e-02 6.769820600543363653e-02 6.441901647194653302e-02 7.414882964747122884e-02 7.772702523145230691e-02 1.041972736399168348e-01 1.055306037303599814e-01 1.564558784359183427e-02 5.465183078988882226e-01 6.616066709957484360e-01 5.782172276794710353e-01 4.350816434752945855e-03 -6.546808023256270488e-03 1.619819462928447085e-03 1.000753071086758934e-02 3.464096670464074379e-02 1.525727260301587790e-04 1.680776112993903615e-02 5.861549705195034932e-03 2.829956509787939778e-02 3.786661764925051878e-03 2.998118872135853152e-01 4.532319623985699431e-01 2.347999779563975931e-01 -2.341308136374761414e-02 9.939536516251209000e-03 5.151383536540187630e-02 6.947690188391465738e-02 8.758694443709647004e-02 1.281844601181831855e-01 5.928999806780160808e-02 6.140118808467255607e-02 1.159769597008678162e-02
1.112803663392867987e-01 -7.201287131223883808e-02 2.495989562418506058e-02 9.155430934695136386e-03 1.400590159005519163e-02 2.273642286689545045e-02 6.740167570418365006e-02 3.297521736976685713e-03 2.923552585849586646e-02 3.467308532399679399e-02 4.972012610228244284e-02 2.005933986987911907e-02 1.230564751470281451e-02 2.042228056037481698e-02 2.253970230779473935e-02 2.236461320520688120e-02 5.192944838636523452e-02 3.407501723012418965e-02 2.856269853639890888e-02 -3.608341757066879651e-02 -4.739135089685168195e-02 -4.107372407032208739e-02 2.863893223927050855e-02 9.008770332957166324e-03 9.386773195495849709e-03 3.668887277317996187e-02 9.149617888556896284e-02 3.399623598643401723e-04 4.627643611218602501e-03 3.950302656892251143e-04 3.195173316084471034e-03 5.447542899107467827e-03 4.062786284281142551e-03 1.472749078520706623e-03 -3.474080474128926066e-01 -1.105349112222713398e-01 -6.574124041028073240e-02 3.546896929231299622e-02 1.773136508305499931e-02 1.529508991306431469e-02 2.566883335293594984e-02 1.699267918842714900e-02 2.823574077831077569e-02 3.246834669136269835e-02 2.938231804732548483e-02 2.697059070005612333e-02 5.759582150845574122e-02 1.844409388526815907e-02 5.955491310156233592e-02 1.897826831569435560e-02 -7.337288694429865143e-03 1.900668192367786433e-02 4.433384995116634864e-02 6.877208534393171313e-02 1.090331022116609039e-01 9.021293193029258917e-02 1.157469947789241255e-01 1.023983775005158081e-01 1.151011890369903479e-01 5.167384053327529064e-02 -9.035429113857952610e-02 -1.464178521152966926e-03 8.358508753131811643e-04 2.940696558459297491e-02 2.812403259584181706e-02 7.935903278383747705e-03 -2.991327282740742933e-03 3.317039200215581018e-02 -3.330972403197224115e-17 6.733365730182111486e-02 -1.264328415105780631e-02 1.019752685344644215e-02 7.020353487139168457e-06 2.893469203527354941e-02 1.676321830857135931e-02 -1.485811553624323761e-03 -5.073878200879789866e-18 6.828542692858315111e-02 -5.359933374354879793e-02 4.189889984894059477e-02 7.359751391355733724e-03 2.482455407229536282e-03 4.753735613716971592e-03 2.190958155493372139e-02 2.361786939054398744e-03 -1.767294790229836194e-03 -1.910778806737805726e-16 9.329410045693016451e-18 3.112593909266411774e-02 -3.829846117332181415e-02 6.818770436882416244e-18 -2.632283413270042338e-03 8.552647697124004406e-18 -2.815626890961468124e-03 1.266897094999503565e-02 5.470770668891184645e-04 1.863022225533009918e-02 1.906730660858511938e-03 3.138261536994855047e-17 -1.489656091408815032e-02 1.224231948059677742e-02 2.389568370069016130e-03 2.566405114696807682e-02 -3.390133235171446935e-02 6.221481345236688906e-04 -1.029771326717592028e-02 -4.421535469235210024e-18 3.183064534912408150e-02 5.383987076978519713e-03 4.094013964163372571e-02 4.543950024455521708e-02 -6.969844567159836669e-03 1.816388077959468772e-02 -9.618057578410338024e-03 1.019831698168193954e-02 1.234353915465183143e-03 2.042685161903096819e-02 -5.611192497580940949e-03 -4.646069077756928621e-05 -3.934282626792178633e-02 6.187587438685975272e-01 3.523341759629820547e-01 5.804374219702657101e-02 1.635296483033411505e-02 2.127677598354792490e-02 3.313626194039675371e-02 4.529242390803544377e-02 7.234235101525512274e-03 2.474703271585664693e-02 3.288937818226368714e-02 4.482185710730160477e-02 2.455991137917339431e-02 2.130844935352226716e-02 1.002021682490102442e-02 3.388700326241842614e-02 1.305616673408179808e-02 4.012375118703267324e-02 9.019600544983888760e-03 8.482539470267230416e-03 5.483153744456554790e-01 7.078820654929867118e-01 6.563105114313629862e-01 3.301334019005258691e-02 6.885101933199127590e-03 2.011390653435526921e-02 1.543736439381174219e-02 3.782341458574371013e-02 4.435403698627632396e-03 1.439387064758716750e-02 1.305126877183845294e-03 2.634627498126301121e-02 8.481823122347876068e-03 1.894679546172088613e-02 1.180108194240946269e-02 3.763377889945129784e-01 1.098934862410770430e+00 5.718966730079919625e-01 7.901751517477226261e-04 1.595909283241632795e-02 1.198064107732366851e-02 4.239994197625095856e-02 2.579728332629515480e-02 3.959994694738255827e-02 1.403602581874152135e-02 3.506390896107611233e-02 6.604176317363165405e-02 7.178888558212356830e-02 7.004766573000108243e-02 3.142752393884381545e-01 5.950926992930332116e-01 2.423376362305246501e-01 -3.499426622991308640e-03 1.034381459378443807e-02 4.387396864882275177e-02 6.213963899938061697e-02 4.397673105032664370e-02 1.235794213817369702e-01 5.855423287705212310e-02 5.000426864520723241e-02 3.119897630779269390e-02
6.101240428261884841e-01 6.064456891769441915e-01 9.080493254024576966e-02 4.712973466051145754e-02 3.855780847618930224e-02 3.677946683292736968e-02 1.179182898671093244e-02 3.398439257733625301e-04 5.284713100427791688e-02 2.836611903268987378e-02 6.086549194827011594e-02 1.619133955649258022e-02 8.824048704367952120e-03 1.132657717369225069e-02 2.162150473679416360e-02 3.128976890668176691e-02 8.139901870532294248e-02 1.983764131319760982e-02 1.483356029285298229e-02 -4.475175293980341445e-02 -3.000369744138648917e-02 -1.269729520956770080e-02 3.314066634571070924e-02 1.627644519132354084e-02 4.343928336896400039e-02 3.515803735974964289e-02 1.084058360278722516e-01 7.669966272352994607e-03 2.025850052834255922e-17 7.097711925208803219e-09 3.189268208234347177e-03 7.613568519476720566e-03 -2.515570043740753686e-02 -1.807929780579940748e-02 -7.971616438613626787e-02 4.232648251202273705e-03 6.250192703351602772e-04 9.385327347774559750e-03 9.955339766130093943e-03 5.260397691225185586e-03 3.596457525181677439e-02 2.735592130131045407e-02 2.812302287423746466e-02 4.335792755605009756e-03 5.191599598575384611e-02 3.366265967954391175e-02 6.567666698482445276e-02 1.751113368460778183e-02 5.641106416514424932e-02 1.427211800333960494e-02 1.828300969120087716e-02 2.503336202451488402e-03 8.596817671328011931e-03 2.405176912031659758e-02 6.437814976952918600e-02 5.623125086346864032e-02 6.574791119674597284e-02 4.936705965227232429e-02 1.030481566301477553e-01 1.257864347802120121e-02 -1.887028554038686357e-01 -2.734267911813588470e-02 3.231735861384201769e-01 8.172966507273551895e-02 4.064079143300934338e-01 1.068214174298239005e-01 -7.960858465420432573e-03 1.235907762726385362e-01 1.272039339807190181e-02 1.959154104334625418e-02 -8.631202904915698942e-03 9.654405407655799326e-03 1.815533707742442160e-03 2.493259742134499093e-02 -1.825028728288044695e-03 -7.697924883697814013e-04 -7.580813466144725732e-03 -3.758829725865142179e-04 1.152129438755115087e-01 -2.298438048441885789e-02 -2.444219513179395759e-03 -3.809716195256765414e-02 3.407448942314016821e-02 -3.481343288407751915e-04 -6.392737531035019849e-02 -4.484887962847811799e-03 -5.016132126173245866e-03 8.778317818067198249e-18 3.068929464404507226e-03 -5.537502425638776010e-02 5.034910584728356309e-04 -6.261085923322041221e-04 1.857054088788102214e-03 1.213343645540212404e-03 1.246041066039233966e-02 -1.356696782261118629e-03 1.368962199714140182e-01 -1.063041545893880224e-01 1.551890236910885311e-17 -7.590890546464586219e-02 2.080306410747979895e-02 -3.424547528735197244e-18 3.840526125786406647e-02 -5.540395361640301514e-04 1.006103134038553726e-01 1.011019666700446878e-01 2.970082551197694015e-02 4.054376769873627162e-02 -2.268551672030980071e-02 -1.177280560193364492e-04 1.013371772108504487e-01 -2.561634896562661398e-02 1.698837027543950900e-08 -3.764517602979473854e-02 2.173802962774943823e-02 1.997718077764705155e-03 5.533603582493751977e-02 3.182719153129916212e-02 -2.598946669187234887e-17 2.519208821407103284e-17 6.337908867510839983e-02 5.106402901936875349e-02 -2.583997546164978343e-01 5.727204048492685606e-01 6.418558619989302994e-01 7.520980531688610249e-01 6.551456777965904144e-02 7.267251197528824640e-03 2.262812904187021940e-02 5.695447005076788999e-03 3.262097936356286865e-02 1.600931194095644647e-02 3.293280510081150053e-02 1.048147146676449426e-02 3.584911122260054511e-02 4.497865400192817276e-02 4.144888524806217972e-02 1.845543082498585924e-02 1.579366058642630438e-02 4.920769437342357722e-01 6.951194193861243154e-01 5.827576185313526613e-01 3.254053033867116557e-02 1.283648486373159536e-02 2.687664571063749439e-02 1.424357131106139714e-02 5.414197846298523381e-02 1.836835358353616776e-02 1.386570761571399683e-02 1.046643964647278949e-02 2.826138571600443974e-02 1.279415423246463464e-02 5.892971898096619476e-01 1.212542626558535952e+00 3.662389567450151651e-01 3.636915474903836421e-03 3.110778554899444343e-03 1.636362150347538125e-02 2.625704084315038517e-02 2.453566432084983151e-02 5.395186750087521688e-02 3.703922739396944136e-02 5.313660789641674181e-02 5.681790102446656991e-03 2.704550277119113430e-02 1.743959949553237562e-02 2.340322964823366228e-02 4.597211813357412469e-01 4.723066162666781920e-01 3.741170054263817657e-01 1.563721123923692696e-02 6.845738234468864113e-03 2.748069108518170350e-02 4.842137026803322586e-02 7.074785490261564780e-02 6.118570120411320756e-02 1.196719901853685875e-01 5.496156432416995918e-02 7.453150802423343102e-02 7.060725728273885282e-02
1.252725081065312607e-01 -6.551061460621633581e-02 7.179544740184864993e-02 4.030484047837609718e-02 3.952323180349986281e-02 3.952582828108148344e-02 1.423939799187160515e-01 5.492222807628948690e-03 2.432552184103876566e-02 3.915022462278280879e-02 6.102797843454440402e-02 8.693345772820746525e-03 7.839220967716777413e-03 1.950042399683250827e-02 2.487427738642165875e-02 6.272657423441839464e-02 6.778421324512250157e-02 1.355065302352562520e-02 2.159972063684390681e-02 3.872152295217368839e-02 7.132285927868062259e-02 4.025607497662329198e-02 2.973605189274574831e-02 2.759556030710879834e-02 2.284137992775454179e-02 4.151484821533285574e-02 4.737579720860185528e-02 2.577018682252914250e-02 8.174257276163301306e-03 4.462470556801542113e-04 2.833550061596511530e-03 2.035358494386404174e-06 -1.164010227257734030e-02 -5.871317695415483734e-02 -9.936703147943466707e-02 1.349107363006836480e-02 1.625262127225369927e-03 1.699709996724811309e-02 6.731419488677193100e-03 1.216594904749108107e-02 2.319570736148222553e-02 4.433626662443806987e-02 2.102462082605697016e-02 4.062003191730237384e-03 2.351159929272176111e-02 1.027174931785833530e-02 -4.897223027810136686e-02 3.505797564655221965e-02 -7.732162394010593139e-03 4.091311677767572180e-02 7.093372616496727845e-02 1.198985800083998847e-16 1.154381622083582547e-02 1.215421754040461877e-02 5.195484362259270017e-02 6.092932397097207409e-02 5.754966195844880145e-02 4.103884560499480166e-02 6.003448904027168520e-02 5.132493418183626394e-02 -1.047965462004345205e-01 3.225710365752799264e-02 -2.265248432857907418e-02 5.758705600668903753e-03 2.604727039921115456e-02 4.655486929427493187e-05 -1.598964368638486260e-05 5.618630895499215037e-03 2.025225188432332527e-02 5.186586745890296550e-03 1.604208544004927933e-02 1.970098652724831464e-02 -1.755129228272736671e-03 2.331297304105462223e-02 9.572911474084178085e-03 2.537296594913234990e-02 -4.062012727429445934e-03 -1.241171537041538785e-02 -6.265329995970746885e-03 2.121471049781995411e-04 2.466730472078190392e-02 -2.002959496463948413e-03 3.262207016031792167e-02 -6.075299913420071718e-04 1.072421771780230140e-02 1.811335414341005631e-02 2.922288835786402555e-02 -6.742715956901186333e-18 -1.640908401364196309e-16 -3.713663395882174645e-02 1.250630536056555886e-02 -2.346123044921310380e-02 1.083266530912847456e-02 -6.000940388234677660e-03 1.519629379881645703e-02 -2.505180980420930806e-02 2.298526269160897084e-02 -3.171245028531399779e-02 4.587080106907080040e-02 -4.381359007813041481e-03 8.951641719835862210e-02 -8.469289111254920495e-04 1.864834912110662629e-02 2.125923881509823518e-02 2.288200341942162735e-02 5.600206920656532064e-04 -1.167789215728792102e-02 -1.578967738502382728e-02 4.442622227749203762e-02 7.737036432508589794e-03 -1.546033424915181469e-02 -1.816678542779323613e-02 2.783680920725503313e-02 -2.489459598489519535e-17 2.583371703468771360e-02 3.838184358221709801e-03 3.238717714306047629e-02 -9.984122297674260030e-03 3.319640964570124558e-02 -5.478768212586614811e-02 6.487494464374635728e-01 3.752651947560680656e-01 4.517407171371424363e-02 2.100853575059031045e-02 2.147059965982438950e-02 5.409843915634000855e-02 6.210134905594320159e-02 6.566523504766288988e-03 1.405480105334271958e-02 1.494467573338472029e-02 2.318339699624989009e-02 2.151500047937112900e-02 2.485957079025371760e-02 1.093622664808749888e-02 3.502477604638803282e-02 1.913447581276538773e-02 5.128596465519655312e-02 1.470607920726985096e-02 1.925884730265400163e-02 4.463769937110444869e-02 5.385280598151895304e-02 2.661301593613757399e-02 3.610907794147263233e-02 2.594374409779238652e-02 6.355283747226606744e-03 6.948455509099963345e-01 7.381491709630924047e-01 5.232564574977103256e-01 8.082746652995499714e-03 1.630689011278746491e-04 3.564026761043624647e-02 1.744944189638660687e-02 5.397025952898485412e-01 1.061543559278774973e+00 3.543224975406522037e-01 8.747471989062801647e-04 2.526720664454331928e-03 1.184038886118904703e-03 1.137037086920986324e-02 3.054750499786619555e-02 4.319225198978796088e-02 4.738764480782663852e-02 4.788022988246394551e-02 5.402847618427188484e-03 3.052267095114061940e-02 2.236582513487198401e-02 5.092604575316125315e-01 7.865055070801072468e-01 3.713025820718178216e-01 5.210514740835784503e-03 1.309069729039977030e-02 2.987770718932386549e-02 2.834682176497310344e-02 1.256836439235549038e-02 6.307897850796477524e-02 3.990479376160200459e-02 1.121906921528416090e-01 3.725560340070419030e-02 6.336521259908695081e-02 5.126887728525227061e-02
1.325969454494023980e-01 -6.822285920116286095e-02 2.501268074240079603e-02 1.411214487177315721e-02 1.617014214851671128e-02 3.847647190080986934e-02 6.374089281834338649e-02 2.638470741140624964e-02 3.821660681034591556e-02 3.934411889907656751e-02 6.487389017413200398e-02 1.796368029011150003e-02 1.556248802360016108e-02 3.546447818768500015e-02 4.815526142430304885e-02 1.463378187999314628e-02 4.046741893235080573e-02 -2.026744266209178129e-18 7.642667933655662715e-02 1.365400972145764828e-17 1.207141002605093688e-02 6.719052958518353111e-04 1.515974191299004903e-02 2.034156885170514822e-02 2.370919556529752295e-02 2.335362009458146967e-02 7.114395537642939638e-02 7.631330676008575044e-03 3.804650223029942935e-18 2.224223466347441077e-04 6.428552450070888248e-03 2.806517565067435244e-03 3.681365668354373392e-04 1.650332341001077563e-02 2.956578003725715120e-03 4.679173081439076849e-03 1.629693688224607973e-02 2.317916860470961224e-02 9.115446469682798869e-02 4.482104844836356031e-03 4.695308275515329768e-03 7.102749939768231158e-02 9.004808642346154302e-02 4.784119136212915818e-03 2.773831041721289831e-02 6.207223205013031458e-02 -8.037604105406489464e-17 9.277778150878800145e-02 2.287044533382257341e-02 3.809350105206073950e-02 4.885771155631129375e-02 3.084095020977112300e-02 1.796526319162317087e-02 2.531301213159014782e-02 5.063018170480626523e-02 6.401260546667593421e-02 8.251128536419871651e-02 6.534493566973743894e-02 3.450900921588147263e-02 6.689918999449333270e-02 -8.693664648979854137e-02 8.766544039499980789e-03 -5.516890931490740085e-02 7.963027301504652100e-02 1.100731030937970173e-01 2.095229175770569967e-04 -3.583027609339033029e-04 -1.427164719373257603e-04 4.046622052284729043e-02 9.780319710645032916e-02 2.903756424082942333e-03 -1.770092100487481962e-04 7.843925942731781384e-03 2.079488693350830045e-02 2.453805311701009323e-02 7.739699894867155261e-02 -2.009485738692033674e-02 4.425221209668687289e-02 9.119281107091674332e-02 3.724042914148329581e-02 7.825965979039946341e-02 6.583495436040434634e-02 1.946619976075037979e-03 -1.672405827015103000e-02 5.660735467130631127e-18 1.961257222305247405e-02 -1.039852480686761366e-02 5.439039329703288655e-02 1.392236729869193767e-03 2.326594454705787490e-02 -3.271210715731774719e-03 3.070660877285183790e-02 -3.060628126352804221e-04 3.124665748020963654e-02 -5.366837577687478752e-03 1.975170160992171728e-02 -2.082482691958233503e-02 2.564255496528752560e-04 3.970769744061609929e-03 3.669170248929808868e-03 1.388396753132997707e-01 -5.280378814387115538e-03 1.203359398992563667e-01 1.274874442115775039e-02 -3.561344762268776193e-02 8.869980409707630201e-03 2.649023848418267935e-02 4.790889903299165066e-03 -1.087664886856131059e-02 3.714973276120322673e-04 -7.616798602296105972e-16 -5.857639959105865035e-17 1.235244306808215564e-02 -1.334109646450518105e-03 1.879212899514853125e-02 4.492443960963472468e-03 5.422928739464463743e-02 -1.238335274538393711e-02 1.893412212108487938e-02 -8.391600851797428526e-03 6.636055121434872861e-01 3.683187738429355984e-01 4.959095124344758271e-02 3.783716226867363253e-02 2.592434152524115964e-02 7.149422076091553613e-02 6.689482878731016491e-02 5.273445574720397877e-03 2.333382591269057837e-02 2.165814072201192181e-02 4.419505784070996979e-02 5.660553853458281937e-03 2.630386220256579824e-02 2.693537531268802576e-02 2.393934315323541576e-02 1.067927132217858822e-02 3.705158716303428440e-02 5.923139145740921041e-01 6.157970087201024567e-01 4.923825272638664430e-01 2.316746950590485576e-02 8.277588501116652300e-03 4.201733627215253897e-02 2.391332183130091826e-02 1.908183273390341717e-02 2.482718144533099108e-02 6.665949239147746230e-02 8.637237391123415639e-03 1.284406822309816336e-02 6.511806319261131612e-03 2.047964637722078651e-02 3.304066649942648346e-03 1.620768680574649623e-02 1.219633987909364906e-02 2.445457456318323591e-02 2.760024215044246543e-03 6.517254648513244077e-03 6.377240922256943723e-01 7.751238068383784485e-01 7.742406270563176873e-01 1.386304643691608010e-02 1.426473266586184181e-02 5.953213909794317221e-02 1.509457676120457004e-02 1.083501195045458512e-02 6.262296490828924700e-01 6.756525784338677232e-01 6.906738013528381792e-01 1.635868707019064519e-02 3.094343682863150407e-03 3.279092436822345080e-02 1.376134260193053621e-04 1.949893599773146885e-02 5.566452374684903275e-03 4.882064247574544741e-02 5.264393941297545315e-02 1.335680269000730225e-01 1.132752983971882510e-01 5.231338014054681684e-02 5.131260774745829978e-02
1.267713248546203653e-01 -6.219705576538053088e-02 2.756873968816480128e-02 9.662444346689167893e-03 4.480375014892851691e-03 1.508683051510015076e-02 4.924587490323601574e-02 1.538875585993362653e-02 2.531034721565592754e-02 1.197656890947814724e-02 4.720131517031343760e-02 2.272953616043776187e-02 1.386029383286388472e-02 2.693257561984079701e-02 3.583074567485788203e-02 3.650085792316191957e-02 6.124593939090208078e-02 2.224354212544674483e-02 2.712033680796077650e-02 -1.016856587372412801e-01 -4.344720757170808523e-02 -4.478916580792544067e-02 3.553590140587205987e-02 2.143087662284235556e-02 2.653883069488869109e-02 4.655385929550404783e-02 8.353256025146231500e-02 1.679304899216401206e-02 6.370169584669698251e-03 2.531398190093036172e-04 5.158458936885927659e-03 9.169294687382809669e-04 5.236557372606342022e-04 3.222800720585982433e-03 -1.602834835469011676e-01 -1.274734056457514086e-01 -2.899788627476168265e-02 2.590600662849862693e-02 2.915307112840412329e-02 2.662612341012742662e-02 3.795596357701931656e-02 3.088975350350694690e-02 2.522977072498406095e-02 3.363941165183737415e-03 4.021483529345790120e-02 7.307535527002252673e-02 1.237155762222190009e-01 5.327579103672649974e-02 2.476966387169581008e-02 -4.364067370977469523e-02 -6.793238622411485253e-03 -3.228507690174600014e-02 3.473220951979565096e-02 2.376010184616931045e-02 7.525269097140506458e-02 5.738217891201152776e-02 9.010411983486817100e-02 5.870880979802520910e-02 4.669080302510646152e-02 2.309346988093197509e-02 -7.953546165432659465e-02 2.103083472943831009e-02 -4.575694447790967789e-02 3.340976358711081340e-02 2.106650386133859798e-02 1.701023536345705131e-15 -1.363302388811175226e-15 2.169679589495848021e-02 3.400005313020996317e-02 1.649771736187159374e-02 -9.254484453745512951e-17 1.425708252362942879e-02 3.204452254960771240e-03 1.998238054769493205e-02 8.833138929329075644e-04 2.128363177636142384e-02 -3.480173422679567596e-04 1.791509015497138105e-02 -1.025950751638145669e-01 3.145801998037400099e-02 -1.362234673270215331e-03 -1.049792180871479685e-02 3.368931302969870034e-03 2.434451661716942883e-01 -1.883222311941391705e-03 -1.727696116328053283e-05 -4.501077202258189393e-03 1.586548798075854876e-03 3.661782010494429862e-02 -4.562022337286377426e-03 9.746270950203180586e-18 -1.440393767001784363e-02 -9.025742760258522013e-06 -3.185604044405513507e-02 5.627193106405007153e-03 1.451206618182428708e-02 5.163549112767252397e-02 -7.819984166229174138e-02 2.297218262252833168e-02 -2.473778986907001261e-03 3.402938754547234906e-03 3.100506308169545186e-02 3.556119325586258421e-03 -2.073059434826844162e-03 1.347594945725139023e-05 -3.158743497229104125e-03 -1.619248418623315047e-02 2.677712940633348616e-03 -1.387839694691137167e-03 1.769156311938311146e-02 -5.089355412281736796e-03 -2.789057841634794577e-03 1.809707225227305191e-02 2.417781617199247254e-04 1.906208225240765331e-04 6.921979277327846647e-04 4.093512613117736981e-02 -3.461092542111858517e-02 -7.368801965925882158e-03 2.999526777068759267e-02 6.440951136198991289e-01 4.193756846554028139e-01 3.527997716964218627e-02 1.279880490290683827e-02 2.033579465319687238e-02 4.283496615353691445e-02 4.172261302417904633e-02 1.349550171975689473e-02 2.693815507920133828e-02 3.447758338593402694e-02 3.495985135417543910e-02 1.072421354690803949e-02 2.887441318514254521e-02 1.566084709454277843e-02 2.473203884563714150e-02 1.221680724474722042e-02 3.762658849497459851e-02 1.243142079706933967e-02 9.474366962947145324e-03 5.001157903568063112e-01 7.324264973911990273e-01 6.957920264795224297e-01 3.720850119293124536e-02 6.564042636695134950e-03 2.027091303995578672e-02 1.366082573148291041e-02 5.081760697115432490e-02 3.740784279062762951e-03 1.898153183907355310e-02 3.816843673137920940e-04 1.522710077561729991e-02 9.890586703036826494e-03 1.798625562470810429e-02 7.954842773253581120e-03 3.828858753966735695e-01 1.167246283420000807e+00 5.578356164149015362e-01 2.435520130776817806e-03 1.135621926497961363e-02 2.554175386586784655e-02 4.266856389599923971e-02 2.971780168136407829e-02 5.248334343164703930e-02 6.600810586550205465e-03 2.807255542818308872e-02 4.183645642678696791e-02 5.261493995815495067e-02 6.129594072758206072e-03 1.353948308623377506e-02 2.538142354488583941e-01 1.834876348943116353e-01 5.936057851463716339e-01 1.734450964043593099e-02 1.049930492319734246e-02 7.120063696839414036e-02 4.355457170565980862e-02 1.087941901040368475e-01 1.161228996671221786e-01 3.031681584861939047e-02 4.973631911522378470e-03
2.462279982772281051e-01 1.864908603474695059e-02 6.966266982418270470e-03 6.198087801184075374e-03 4.926007120941087157e-05 2.600593311566876033e-03 1.442024514592737602e-02 4.841257408056917101e-03 1.368380995471420490e-17 8.038707181141104691e-03 1.163310762818327466e-02 8.492209175698128484e-03 2.061305156481454959e-02 2.082113053826946739e-02 4.275151269992507469e-02 1.222998573454161381e-02 5.864563260877918814e-03 1.809865818478087127e-02 2.473152667306163441e-02 2.614658202801465811e-02 3.600533711933436715e-02 1.192915773186142316e-02 1.011743612976496662e-03 -1.897176358296232823e-01 -2.675086129403309018e-17 3.632464239741187845e-02 4.269650801421262520e-02 3.123001292411119525e-02 5.026328305735729834e-02 5.222079277011412723e-02 1.517522574265151858e-01 1.092185278243811702e-01 1.389471765681345172e-01 1.045854563066476589e-01 6.117589987797422690e-02 1.662894678546011917e-02 1.790086796988978046e-03 5.329327810205107541e-02 -2.213346898604393737e-02 1.419808283187067597e-03 1.061795408630901091e-02 1.298137021190943000e-02 2.227804669824283781e-02 1.164258639674572906e-03 2.880899363345197292e-02 3.248280423758432428e-02 6.437372772663418896e-02 3.739537320436169161e-03 2.883687588196545692e-02 2.220773635044008543e-02 2.494562226938886948e-02 9.702513826393064506e-03 5.582627693736692820e-03 3.524292577193377345e-03 9.855844834392696319e-03 6.012379359491380878e-04 -2.488061369138019896e-02 -4.800873555069296722e-02 5.028962793229081463e-01 1.230472248553309694e-01 -1.392807247935587966e-01 -5.929271307438485107e-04 -3.316138299926096183e-02 5.520854185390885321e-02 -3.135038108660805761e-04 3.037007671211470073e-03 -1.581397012114833236e-03 -1.436847001244389044e-17 1.205587361487311019e-16 2.055585941538331726e-02 3.817007093932766652e-17 3.146606924919911202e-02 3.888200891764035522e-02 -5.488304233745042792e-18 1.149566660131244068e-02 -1.268400947226413448e-03 2.032348557446226559e-03 7.951817696805132604e-03 9.172029130372988392e-02 -2.230359288475744928e-04 -6.706890428421306968e-04 3.077947671301758253e-02 -4.513542912493919562e-02 -5.185605610345143252e-02 -7.344950735239419437e-02 -9.875711422936346226e-03 -4.679344584797702445e-04 -8.830206426526791483e-03 3.293085707471679613e-02 -4.460341965842263418e-03 9.869135849961102602e-02 -1.424296226820737955e-02 -4.774501381599293294e-04 -7.086983769707641257e-18 -1.597313535506967815e-04 -2.117582560114794621e-03 6.259630830586320935e-18 1.031501279905247407e-03 3.516496330957118166e-02 4.888892417791743106e-02 8.241159662786358420e-03 3.278933712002392309e-02 -1.437784327110989773e-17 4.693819351079530147e-03 5.330108779810807662e-02 -7.903251434640693254e-03 -1.939025719227086345e-02 -7.168421803099359925e-03 -1.158181026875837807e-02 -1.813744831396616602e-02 1.987045937565105251e-04 -2.843470728311039628e-02 7.700964468152154442e-03 -1.771786967383453051e-02 6.216078614520985195e-03 3.767155204442395461e-02 2.614699233681012577e-02 5.713847785116748373e-02 1.776957018987113460e-02 1.281623576992355906e-01 5.441824099189865871e-01 3.482428117199741768e-01 3.517379040439382043e-02 1.287786065984413645e-03 1.217573570810287845e-02 1.388426189329136466e-02 2.186456899240970739e-02 4.366913981879618455e-03 1.114073427455222233e-02 7.116085191837880075e-03 2.260400789493968721e-02 1.953913403146442448e-02 3.874836964050167726e-02 2.153644210305537959e-02 3.580488582478606047e-02 1.053148740430664976e-02 2.772238917639649122e-02 1.691202088272635018e-02 3.855316920851458579e-02 2.115117376047247547e-02 3.702907786920342503e-02 2.915457223708689365e-03 4.653778366085579710e-01 7.772541064078369866e-01 6.689486307985044178e-01 3.287126234187978061e-04 2.890421358563345991e-02 2.835080493294652587e-02 2.209866044654259370e-02 4.866220403654997195e-02 7.350871160833229168e-02 7.255050033594068193e-02 8.058141012267865111e-02 9.810303918572016446e-02 4.317605710695267274e-02 1.392572309762359420e-02 2.539420261759636060e-02 5.938891097054669332e-01 7.497727530740779800e-01 8.065680358209889045e-01 6.629822743200701127e-03 2.807573153595690693e-02 3.643249973365025179e-02 1.460029878548934137e-02 4.425947735685287210e-02 5.031961526540835311e-02 5.923672942828977356e-02 2.387669347414608095e-02 1.528502928393546401e-02 1.817265603105321439e-02 3.686528832058338229e-02 1.136865075983140277e-03 1.426563415600110192e-02 3.589606465792606919e-04 4.706136987650996750e-02 5.729400139721368967e-01 1.039995419321100567e-01 3.007678785593205228e-01 -4.022756380081436434e-01 -7.740093641298868565e-01
2.446834177001909749e-01 2.316642633336641818e-02 4.056190656129753059e-04 7.642158378155224172e-04 2.394925703731948046e-04 4.040930100674257862e-03 1.261469045093718822e-02 3.225687110773742695e-03 5.940339812969943310e-03 1.120057236627923676e-02 1.934644844482733689e-02 9.786028860627670259e-03 2.301104818914243011e-02 2.179314330793485563e-02 5.257361095239320575e-02 6.611074681840490089e-03 8.136487235145860655e-03 1.772651414929233074e-02 1.887729886161264997e-02 2.737272428907190636e-02 -1.066421473144543858e-01 -7.208894949984079314e-02 -8.059140175665291395e-02 3.834250458926707905e-02 1.363439696571882020e-01 1.504707152277909993e-02 1.756994109935882536e-02 2.283753047029284805e-02 1.243979778921992102e-02 2.838046133405758981e-02 6.716817626091418914e-02 8.573516916379933517e-02 8.924048885185213720e-02 5.664839144274470134e-02 6.230211348649002806e-02 1.897803144750095927e-02 4.879996529757242574e-02 4.197627681173132091e-02 8.996319876242268865e-02 1.902450563431071218e-02 5.604142138960425089e-03 2.558670661104479188e-03 -1.147179356552734467e-02 -2.678668579722169249e-02 5.905200438030549365e-02 -9.695677942062011501e-03 -8.683648689211584046e-03 4.396481790859971088e-03 7.505053475365400066e-03 4.375448136310355896e-02 5.916597661684798448e-02 1.838187674958115247e-02 1.133867085647568239e-02 1.005518939563973015e-02 5.144141287835599097e-02 5.308877770063439444e-02 5.075405524174856492e-02 8.354549185174429959e-02 4.137156682707743283e-02 1.269658079401242944e-02 -1.667952774579807840e-01 -2.145294273399859944e-04 4.167463019030869485e-03 2.306033782665507098e-02 1.838978681141940213e-02 7.253034250685896148e-06 2.512099956978071369e-02 1.010256837801665891e-03 7.417410460050997446e-03 1.250029471595237932e-02 6.813283700787902608e-17 4.766057031864859295e-02 2.639367323376325472e-02 -2.216521144110526406e-02 2.888742542175264105e-02 1.932037322596955736e-02 7.349443998668419437e-03 2.899582811776154137e-02 1.403111979223348356e-02 8.149142115505558087e-03 -9.361638721552097619e-04 -8.533508734163547382e-02 -7.832065974766010130e-02 5.138012627051960490e-02 6.299885921687625734e-02 2.517622888001010487e-02 1.017530609799355279e-01 -2.762429866086220809e-03 1.349497417679823520e-03 -3.033780112367364789e-03 2.181665928445658889e-02 -1.883683971748615488e-02 -1.396447593212523687e-03 -3.042603901884397628e-04 2.044934151547002366e-02 -6.251066669218251128e-04 2.770415182623203049e-03 -2.551127203706343716e-02 3.700080506492802682e-03 2.718236271612565202e-01 -1.758887017692640012e-02 -2.305726105340928867e-17 3.175646876481811876e-01 2.283108849093433857e-03 6.993744784303036234e-03 1.002812192750233804e-03 5.917746633555773933e-02 7.711739981873085232e-04 -3.939323590628077826e-02 9.294060828142530576e-19 -8.360240186155301931e-14 1.241475014664951589e-02 2.118998943525218781e-02 -3.094946016142930334e-15 1.014496135457169414e-02 2.902321765672780628e-03 7.883502888951093857e-02 5.917689489659842897e-17 3.047742133066909734e-02 2.054235510706535137e-02 5.841416639724491544e-01 3.997275899019278600e-01 3.214725313239186893e-02 7.150303921648529628e-04 1.677334777899857146e-02 1.148788550682626648e-03 1.899422968764459888e-02 5.071257095681085472e-03 6.333162271036738879e-03 1.070271000795495206e-03 2.740847727986165522e-02 3.682065901909367667e-02 3.209960181014517161e-02 3.972194140813626456e-02 2.942902836080816403e-02 2.138102316526574107e-02 3.099662783506217795e-02 1.814147799142670769e-02 3.028382367523919927e-02 1.526712612606366884e-02 4.499355622196706261e-01 6.916572538623692035e-01 3.851211162899037532e-01 1.343547506007222725e-02 4.353034425108716354e-02 1.179327420268915481e-02 4.495708872911829967e-02 2.508281020521967858e-02 2.210436191619936236e-02 2.052298236894459227e-02 6.433872855477569541e-02 7.028138482694779032e-02 5.466674036189640012e-02 1.062819338782117623e-01 3.543825929260775026e-02 1.193355579937457542e-02 3.291585392730984189e-02 2.864669710973479291e-02 6.068761173511859341e-02 1.879044629187053006e-02 1.015988186476181505e-02 1.978742276108321971e-02 2.861323169037209915e-02 5.247024194065999687e-03 4.891497532757715927e-02 1.489579402488254434e-02 2.713000246362530163e-02 7.811430733494245660e-02 2.894139484275269972e-02 1.229841944141036626e-02 3.952812367493591800e-02 7.141484197112995223e-03 1.301688856228461547e-02 7.333140624483504115e-03 5.608627403218844482e-02 2.293846470095829010e-02 1.176053384214143654e-01 4.053531024173794711e-02 7.277435563538464658e-02 2.067575736519114069e-02
1.320451523287806406e-01 -5.457709670928008461e-02 8.667217938288129808e-02 2.862350206812574668e-02 1.482536901763409007e-02 3.049494606468486357e-02 5.211817548691879270e-02 4.674652396611094288e-03 2.838835734785051523e-02 2.695084984731298608e-02 5.501832025013751737e-02 9.213150415021987075e-03 2.436074887571523448e-02 4.624271202626027699e-02 3.291127918941578329e-02 3.189554171417793155e-02 6.125901564036474733e-02 3.692829148751452378e-02 2.907970967616062274e-02 4.571483770704211358e-02 6.402578024361081643e-02 4.361870179047568769e-02 3.260603612051984901e-02 2.576550170488635383e-02 1.996110015310645358e-02 6.066178872139136452e-02 8.238922057945041733e-02 3.245947967008525753e-02 3.631944941169761021e-02 1.108244222095319462e-02 1.008188949859578284e-02 1.266326130804820531e-02 3.529245268050548636e-03 1.193101787486769427e-02 2.006774325136632606e-02 8.256251625920224546e-03 1.319447409407554714e-02 1.951096629061252127e-02 1.018140539430257979e-01 1.782137010596583984e-03 2.187418419094269012e-02 6.124959378538062660e-02 6.193503713359670149e-02 2.707939507684081065e-02 7.267270968365657779e-02 9.563806522237135177e-02 2.098639147501467905e-01 4.118301512654019836e-02 2.640558232415523079e-02 2.545744213007881257e-02 4.138165288918069923e-02 3.530719855024665177e-02 2.012728950326441932e-02 8.220748651520372319e-02 9.765927526518179691e-02 1.055149562073719877e-01 7.484389016524818639e-02 6.706259432371058926e-02 1.153606808145175994e-01 7.060446892432856170e-02 -8.498619142254396075e-02 3.554246107945900103e-03 -2.380084766365316365e-02 -9.234939845451069565e-03 7.996550780592698304e-02 -2.216805517008497235e-04 2.394776726337896448e-03 3.724787577919942282e-04 3.511150126462337512e-02 3.078766128790172245e-03 8.085379244356311404e-04 2.301047968977078945e-02 8.058401590131603065e-03 2.130487994081850609e-03 1.299800420925451179e-02 -1.253024914290247829e-03 -3.418857388052789961e-03 5.473138604518150603e-03 -5.387617720662717796e-03 3.620505653833489511e-03 1.481898625602397906e-02 -1.455146555503981616e-03 1.550362097784994085e-02 -2.799676733392841149e-02 4.402404409823629412e-03 1.448773117369180422e-02 2.292283546007919948e-02 -1.538414809865691507e-15 -4.187765924879914385e-03 2.972873028209602198e-02 5.428673152024500155e-04 1.445874633616767114e-03 6.565925927630757164e-02 8.568589647044610319e-03 4.298743129173317884e-02 2.418854694843668857e-02 4.586402225046760928e-02 -1.207595191937245113e-02 1.880117927611676953e-02 1.919902682239560746e-02 3.156468417752131966e-02 -1.148238056956089360e-02 2.143625247663053651e-02 2.999005970810675673e-02 -1.804123187618236410e-02 -8.352046587723584800e-18 -8.288357158561020233e-02 4.125837287748422566e-02 1.016301999480444078e-02 3.729620283384042084e-02 1.247123887721345858e-01 -6.080316157011741822e-03 2.878807394593323882e-02 -2.963532598634179699e-02 1.744439640111403200e-04 1.625993059889759007e-03 1.327485084628372142e-02 -3.069516268678687890e-02 -1.450429929715923993e-02 -2.388731981196716425e-02 6.350523022820308006e-01 4.054943294037695600e-01 4.190823664142144522e-02 5.785648283845443857e-02 2.623441634653674026e-02 7.525941706558365862e-02 5.849154349735929265e-02 1.602345602877128239e-02 2.489519753182367623e-02 3.065491634952487165e-02 3.513478837246800102e-02 1.907701006339413211e-02 2.498357924287570447e-02 2.071534311072189063e-02 3.717833047830557125e-02 9.455189997799821194e-03 5.216758664578232535e-02 1.742736808644352806e-02 2.593688003307980175e-02 2.696078670355600906e-02 5.016798709638171233e-02 1.122577998931818290e-02 4.192554847598956025e-02 1.943310335384344431e-02 1.611469568291654803e-02 6.657086339462123803e-01 8.453048949256253275e-01 7.835987503693518930e-01 4.144264018851891163e-02 5.775320175467328621e-03 1.253352255747528822e-02 1.123750462516055065e-02 1.191593350845151361e-02 6.313642656955449628e-03 1.985502784294033291e-02 6.415058091159845689e-03 1.362824917016508368e-02 7.644914256835961819e-01 8.521677236668515620e-01 7.433331678073376025e-01 2.093445907637271966e-02 2.691983049192294992e-03 3.888282879794796587e-02 2.076992122287059719e-02 2.935316531694270176e-02 4.773180767979399997e-02 2.625070097831650948e-02 -3.276501122727477205e-02 2.937412488382697973e-01 5.219919653789120151e-01 1.750109009152978068e-01 -1.436311234141573600e-04 1.125689546426326959e-02 3.947713036190825253e-02 5.712083359681584954e-02 3.790001646010159736e-02 1.247473524243172399e-01 1.064481568664288336e-01 5.278934882414479390e-02 4.146149606948947713e-02
2.447493354224663242e-01 1.782011068142686644e-02 5.089929476538860437e-03 3.559972906143047837e-05 1.145212234599286231e-03 3.299411032819642327e-03 4.968652604595605836e-03 6.708257886257183548e-03 2.680864387600594232e-03 6.117130443611029991e-03 2.152662886680089052e-02 1.550049303079438327e-02 2.799094829010302654e-02 2.095785939763555178e-02 6.494944398694273147e-02 1.277546362392227575e-02 1.761207739369104186e-02 2.229532224354083617e-02 1.481215189363353259e-02 4.462771534394382317e-02 -1.216429049387167394e-01 -6.143882098906282285e-02 -8.354039302567894953e-02 2.606785798281677571e-02 1.122817708526386293e-01 4.511795124064958695e-02 2.598078937687976070e-02 1.563962583103385012e-02 3.683006043140941987e-02 4.206880632422805438e-02 6.018362688092973589e-02 8.966924892111539924e-02 8.016086259717662699e-02 5.376162384819279522e-02 5.986554540857797618e-02 3.031248522477710677e-02 3.042365233430768812e-02 2.444389177806537833e-02 8.890644861113038144e-02 3.810655062108475599e-02 2.146405510784707166e-03 2.782296473268508138e-02 -1.102590246929631423e-02 3.513002842592751612e-02 4.062130118130132667e-02 4.250970441563079599e-02 6.414238110142669869e-02 2.324431477221593212e-02 3.514987224252941189e-02 5.519685042765951760e-02 2.727027308781665091e-02 4.019921968523797705e-03 1.674580327836086568e-02 1.309866275033781668e-02 5.942083229447688020e-02 1.528647536376321996e-02 -1.255241076377228426e-01 -1.588504988939014317e-01 -2.021724918295747242e-01 6.320133415207533634e-02 -1.702717842988352992e-01 -1.704331383621784631e-03 -1.078961385257234257e-02 3.149123764388127300e-02 3.260229725132338108e-02 3.008064278956525377e-03 2.492906146078797958e-01 1.606644481463298871e-02 1.799056590411497042e-02 8.026721599009023489e-03 5.692408361004008438e-18 6.634331032236022441e-02 4.127491551074371524e-03 -2.936239366740109324e-02 6.618704696770107143e-02 9.803326687268189635e-03 1.347960858723659325e-02 3.127590729933225205e-02 4.581741648272017603e-02 1.609197323001947122e-02 -1.204618304999180607e-03 -8.647248538015045449e-02 -1.899479544545648046e-01 8.410043690283476592e-02 5.624925731032871107e-02 2.543897594528841608e-02 3.170590855519715007e-02 -4.823412807398623492e-02 4.130962626189569505e-16 1.363256629128863345e-03 3.378064579876944445e-02 -2.145809607520647588e-02 -9.671317619056856579e-04 -2.445860359776120901e-04 1.712365015298679058e-02 7.223959529605142142e-03 1.279226980958073599e-02 -1.175039630883735128e-02 2.598218843898220196e-03 -1.786645318028142241e-04 -1.802973362248643310e-02 1.801027096388580634e-02 4.647503132668111617e-03 4.969471232098936903e-02 1.297085091304118164e-02 -5.775114720097058313e-02 1.134203756883573298e-02 -9.048745658502488831e-04 5.449832873812383849e-02 9.904667648410062433e-03 -1.115423574852815458e-02 1.611336318529011172e-02 6.854951477353674868e-06 -2.063934758783624816e-03 3.230870748916652487e-03 6.377289752896729731e-03 -1.152601889257009599e-01 -4.876692619271393719e-02 -3.378570606695628920e-01 1.784356982721819751e-01 5.677370761149025391e-01 3.907198569001310573e-01 2.976386905783551251e-02 9.624318005356963550e-04 1.151035310137336551e-02 1.229506117165803103e-03 2.205254431478695723e-02 4.146115542047822990e-03 2.408786412220540601e-02 1.967858760033882031e-03 3.363213291698152058e-02 4.269516563349813089e-03 3.709155850533145393e-02 1.381234087262208504e-02 3.086521298144581302e-02 8.723972396967398166e-03 3.286116602984220231e-02 9.310752329127112981e-03 3.875296499097591330e-02 1.525368847895543049e-02 4.347347145097235632e-01 8.665782441032264805e-01 4.015002877122180180e-01 1.478591390405700409e-02 3.415104191080006041e-02 1.489035573382339883e-02 4.266542375757489264e-02 1.017299785728389278e-02 1.775730267760175160e-02 2.975794223817660247e-02 5.905313525934979130e-02 4.643001105828160197e-02 4.011811251899227021e-02 6.943582689556938858e-02 3.453805281047232084e-02 9.059402621724981647e-03 2.790616683860661432e-02 3.368965698936297881e-02 4.743972039902815546e-02 1.368244040907439472e-02 8.951381681824834013e-03 6.340789742663284212e-01 7.593772458566574457e-01 5.766215371157553671e-01 2.147993569522397075e-02 3.429837414044827060e-02 4.612354790784977537e-02 1.761624889880846270e-02 1.781140453689244660e-02 1.459264143994235680e-02 2.519889847034699829e-02 -4.427411265031300745e-03 1.731219523284684494e-02 3.672742617699189388e-02 5.328763026212027243e-02 1.671334545952540829e-02 1.372884838585790024e-01 5.107143912065570390e-01 3.938210072907213366e-01 8.501890409464107901e-01
1.717145032349383094e-01 2.411744238783845515e-02 4.067360055284563285e-03 7.703582651295236977e-03 4.842555767818749676e-03 2.201043912234101050e-04 5.555634088144877948e-03 8.754505149733967512e-03 2.944110918756922754e-02 2.307518903785174636e-05 2.607304708419523262e-02 1.248143327042492735e-02 1.494343513359123327e-02 2.219762998129156881e-02 4.612787686851242686e-02 1.144793279008416917e-02 8.047089417593113192e-03 1.300284300039667752e-02 1.148891822547878395e-02 4.871509265128903116e-02 -1.140182330276702516e-01 -6.978596314988251559e-02 -1.229357946374126931e-01 6.531176752103458327e-02 1.116297701831316935e-01 3.052368426548147246e-02 4.146040230631257290e-02 3.056933003168380708e-02 4.988849963873154963e-02 5.500706947356464455e-02 9.978417631504682417e-02 8.639740773404788743e-02 9.892581021569318844e-02 9.717104230491813155e-02 4.704602737521609845e-02 5.013426005608850877e-02 4.765273199227523793e-02 2.710053378199880347e-01 4.948639779840643266e-02 1.016909374865387905e-02 1.302232082531250239e-18 4.239868357191158010e-02 5.101166143700423816e-02 3.004049155809663044e-02 5.594851117337673596e-02 6.408862846033153149e-02 1.572527579358760363e-01 1.698139391918291122e-02 1.402527731983566235e-02 1.605406942508695242e-02 -4.128806859564508602e-03 8.236825773692439678e-03 -1.310623676442319169e-03 6.530542305086528554e-02 1.359351432179472796e-01 1.953240721849205386e-01 9.961858876364909632e-02 2.457788003781187780e-01 9.839119804250737178e-02 4.481743955899353954e-02 -1.625109306308621104e-01 -1.817513341389590874e-17 6.927138211552202191e-03 9.030748673177492469e-02 1.666125860470559286e-02 -8.885440645043634483e-03 1.733913021798638363e-02 -2.110874459099104079e-17 2.799864950133624728e-02 7.773942686686222472e-03 3.574269323780803094e-03 2.067593684388089811e-02 -7.052723883782776237e-04 -2.778415638665363008e-02 -1.914066345721425759e-07 1.092346495543665939e-02 8.155001266292078929e-03 1.576490777995765216e-02 1.716913773103590293e-02 3.982769301673535581e-18 -1.645742924166032064e-03 -6.364695778201141796e-02 -9.116505304212856042e-02 4.224326449706296976e-02 1.506329230289675525e-02 5.773498852596507455e-03 -9.689297369952394601e-03 -5.477714499130456431e-03 1.007689652696708814e-02 -1.611917114493189793e-02 6.237795102511341921e-02 2.983311319912966119e-03 -5.429809933048915542e-04 -2.563587395687990864e-02 1.702343567568903870e-01 5.558616665495910020e-02 5.040892987413504028e-02 2.683175366049755234e-02 1.769542183058728582e-03 3.126208791193470921e-02 -1.861198387109885018e-17 -2.604376673807624540e-02 -5.848860958650707424e-03 2.896833228453602957e-02 6.799081863971235921e-03 -8.033284211770681197e-04 -5.745332922472066312e-02 -2.238111868072051661e-02 4.891909821741007545e-03 8.418630539443175756e-03 -1.338428479258219929e-03 -1.169386862087529004e-02 -1.577940326570282867e-02 2.596051001834055391e-02 1.597980866683576547e-05 2.676258539780222191e-03 1.175951002334085999e-03 1.028212292924901398e-02 4.486848083823211097e-02 -2.675345344135340864e-03 4.574363932438074709e-01 3.013284519504366155e-01 2.663357122010242758e-02 -5.408324522083501808e-04 8.642372502397585934e-03 2.524083134629912569e-03 2.217411029463873043e-02 9.171089299163963024e-03 1.446758462283290365e-02 6.645497050774457765e-03 2.572551475248194969e-02 1.856662108543434553e-02 3.254706829424125480e-02 2.265093151918559644e-02 2.736767357282673604e-02 8.855326842438069126e-03 2.058773255346814379e-02 1.206811477724540130e-02 3.556930039705168828e-02 6.764662734223276207e-03 3.917844582744992588e-01 7.445725157161073859e-01 3.882508930637679767e-01 1.092816694161041595e-02 2.048108432630865727e-02 1.073624789904975910e-02 5.249985296633881415e-02 3.817765935800643273e-02 2.903650304495018070e-02 3.775931943566820242e-02 6.522249621772284456e-02 8.346404379573139909e-02 7.472927829253657450e-02 9.994942011121367431e-02 2.116529958568849623e-02 4.806697489757855157e-01 5.501197400424417960e-01 5.906929139754669711e-01 9.174163091381203755e-03 -2.139000123461652636e-03 4.037401062145495705e-03 1.260108517052625603e-02 3.883069043795401437e-02 1.168059427356234192e-02 2.532902721081069231e-02 4.204238187635465918e-02 5.133557389515301661e-02 2.688256020161445756e-02 2.134968022989458883e-02 1.005876167696636257e-02 1.947306968367378543e-01 5.105347112852917091e-01 2.890043111996399849e-01 1.591486429836451444e-02 3.796084858749781965e-02 3.305089187249689309e-02 9.276606062038891820e-02 3.620665626486421068e-02 2.535149785607343845e-02 2.222009099263054163e-02
1.358569154068369200e-01 -6.013152146609896526e-02 4.092861659911314898e-02 2.270103857685169466e-02 2.349411716066557299e-02 3.615184977290413632e-02 6.918465417846035581e-02 2.941320273575682986e-02 3.007442456965185584e-02 3.867342831373683787e-02 7.062261343845058215e-02 1.029061885664022888e-02 1.428211288012617465e-02 2.486175113161685421e-02 3.051222998551436985e-02 1.887663350780080518e-02 2.538191519134084043e-02 -3.818942026805487167e-03 7.819832755430410065e-02 3.947070363701040210e-03 -2.440030547292799945e-02 1.495099970211265445e-03 4.470729063727504893e-02 7.996431232320005272e-03 5.258977913228170785e-03 1.021322688562267822e-02 7.876627383775972047e-02 1.145471018019094192e-02 7.899289912402131286e-17 2.072721051742594284e-04 1.184571947751203794e-02 1.053261750394969554e-02 1.032664647818826989e-03 1.511237484748909760e-02 2.706804499228230437e-02 -2.810339358442494606e-01 -7.451996623732642933e-02 -7.164793227888303917e-02 3.940215794990720860e-02 2.433405797368079146e-02 2.489416144410985798e-02 2.319449270189071974e-02 2.042316850225824737e-02 2.596905079384403611e-03 2.730962214259574986e-02 2.925384791923682851e-02 5.540063998739760270e-02 1.949441197366317755e-02 2.426474922126532763e-02 3.204133199120957204e-02 1.659327892328834805e-02 7.525410238667023383e-03 5.782932091492249717e-03 1.636294669214418893e-02 2.882552844269366948e-02 1.309132973731857693e-03 -2.099503520146208529e-02 -7.515008190930083654e-02 5.357872223839438286e-01 1.154270757281177229e-01 -9.123051761097193679e-02 7.934336051640403925e-03 -1.416031936030857398e-02 2.337479063454183173e-02 2.944703604243596728e-02 -4.210477248675165846e-02 -2.571278621410200398e-04 1.213671523855969605e-03 4.096078750975760963e-02 8.045328122436870566e-03 9.210597427009931809e-03 -1.230717350481728587e-03 2.236698697921739309e-02 1.751218212165498236e-02 1.371407949058199745e-02 3.697356266441052114e-02 -1.244725641981875991e-03 5.405501653336860363e-02 9.099539622675574535e-02 4.813421844330079136e-02 8.588104103334654252e-03 1.462052435002376798e-02 9.348170361773494585e-02 -2.358618275747202675e-02 1.551437364366462131e-02 -3.778009300646855320e-18 2.535383804181313636e-02 1.970921886414716759e-02 -1.781365529135186072e-17 -4.893998302449992233e-04 7.961660512798445540e-05 -7.209323111455092671e-03 9.211330997922500055e-03 1.156186566077044287e-02 -1.084677501685469142e-01 -4.790463271060636623e-03 -1.742910892376653073e-02 -4.789840403563852544e-02 -5.263186947351214395e-02 -9.235058004820717429e-05 1.165468157032295039e-02 -6.896596192859012726e-18 1.739852816210999027e-03 1.376669305103986052e-01 2.817836244687047962e-04 -6.202791999382271272e-17 5.025536204257131939e-18 -5.332100411235381893e-03 6.232041944891761527e-05 -2.199811466982246011e-15 8.483752494137793837e-03 -1.165058952356284638e-04 2.023699518939251063e-17 2.683441040292979321e-02 7.029304581118874483e-03 5.499502843773205335e-02 9.886511450409410026e-03 4.622414107487540780e-02 1.539951585057022440e-02 1.490996485387909021e-01 6.440830703669512269e-01 3.812865516778286556e-01 4.266046065652488034e-02 6.560753023768534087e-02 2.657899154576514461e-02 8.887770465408049991e-02 4.503419347839553166e-02 1.053480159137446664e-02 2.326379808755434792e-02 1.368002357791871064e-02 3.435287415138635336e-02 3.010385895604991355e-02 3.610810142355855290e-02 2.794868336729006489e-02 3.178180631492485281e-02 2.620154738193002791e-02 4.107580693991431942e-02 5.825274669474034273e-01 5.676574933793624433e-01 5.197430374909979278e-01 8.256356657348008776e-02 8.356998515815496032e-04 3.888462894216600241e-02 3.325378007403965303e-02 2.224151550624607621e-02 2.295513056521430312e-02 4.271546556037281173e-02 7.465771094773922603e-03 1.548602811001719218e-02 5.891949318763566382e-03 1.546550025271629589e-02 1.115490307644200494e-03 7.659635191470541386e-03 2.263284774803913692e-03 2.697573681780575036e-02 8.668344343856407841e-01 7.973825284468006869e-01 7.093819913475084560e-01 1.895150522464164600e-02 2.124761757738915721e-02 3.795913078053306244e-02 4.207442810120123888e-02 4.975015171259675184e-02 1.730706317431137189e-02 3.414681035838140755e-02 4.359326358804928303e-02 3.949241166208183884e-02 2.372606621014303543e-02 2.060739339304403991e-02 1.839485424298716032e-02 4.273702731328109933e-02 2.551854912063184627e-03 1.746138877130246417e-02 3.235855988065387823e-02 4.538542543468991608e-02 5.558919755971184484e-01 8.528708571481317946e-02 1.787515751852012114e-01 -3.902961463804581754e-01 -7.625259805093702603e-01
1.599725968350385052e-01 2.641555565386218343e-02 1.384002443016787163e-03 4.759515605319208031e-04 1.448701594911122473e-03 1.431664240449076909e-03 4.922035321996676670e-03 2.676429791113018979e-03 5.184357473949867748e-03 9.813187490872706323e-04 2.090694401958567358e-02 7.947233288309192220e-03 4.517656804685733773e-02 3.277525787377658789e-02 4.598390226117132079e-02 1.293236572776115985e-02 2.136864953250766641e-02 1.291698283346997579e-02 1.316210721508985666e-02 1.541265355430788790e-02 3.513394189777255955e-02 -4.534959601190092365e-02 3.378430997234662114e-01 3.989318067015664165e-18 8.387013549003972945e-02 2.798605045162824018e-02 4.686700796203778496e-02 4.537838849609891062e-02 2.213344067649922198e-02 5.941605726113063923e-02 5.750977555935835039e-02 8.743536108504974813e-02 5.051034999303608691e-02 -1.258240895376443007e-03 -1.884431939836418322e-03 3.012524963932236804e-04 2.496552784197995575e-02 3.119141220553515656e-02 3.934948273325068746e-02 2.538487157826469637e-02 -1.830382239926419199e-02 1.898218720968742473e-02 1.385800000951301764e-02 1.562227555865587029e-04 1.226206386646767378e-02 2.743378091151973258e-02 6.309501129517747220e-02 2.028078128202596281e-02 4.926384105918005241e-02 2.832365420379131041e-02 3.751181236240271055e-02 1.698267845405710113e-03 2.136923726796108314e-02 6.375697426608228392e-03 3.753630386028065924e-02 5.012659428057339439e-02 5.627384886678844789e-02 4.584992004081921146e-02 1.199824951221737379e-01 3.052203184395864873e-17 -1.612960395214267650e-01 -4.703594581128103282e-03 -3.288121445114116498e-02 5.991537735805466114e-02 -2.711513060338519368e-02 1.979279459126725375e-03 7.333686830116964428e-02 5.032855430385841516e-02 5.848584768104590770e-03 1.256969315426947204e-02 5.904124292321530874e-03 2.350263194995409061e-02 8.602428813119589526e-03 1.106931677692790487e-02 5.298280022414654164e-03 -3.922247921698360663e-04 5.097973145707906245e-03 8.711088748897274914e-02 -2.671677645722975863e-03 5.081440415358074759e-03 6.224480096077660007e-02 3.954200269908945270e-02 7.577868532993807527e-02 1.095980955143074642e-01 1.214215795418470585e-02 2.429753702570488019e-02 2.390823035672896141e-02 -2.015533719745862470e-02 7.251782265229934883e-08 -6.900902217837306951e-02 3.857043191067922872e-02 -8.410137388792746083e-02 2.969641271880430269e-02 6.389702034878669279e-02 9.024232403602287528e-04 -1.068627618536316902e-02 1.319256487791782465e-05 3.158466934064735332e-02 6.535883325772366692e-02 -1.028062460923198120e-02 -2.509991896691173004e-17 2.805130701909538601e-03 5.171816677568553434e-03 -3.874203781644881134e-02 -1.469219798680657281e-02 -8.226845054369795116e-03 2.449972256378760843e-02 3.825976814705735890e-02 -4.739629455951532150e-03 -8.718093771899195713e-04 6.550593461269432760e-02 -4.304167367146990997e-02 1.104573639716914258e-04 -2.645433023292435501e-02 1.174598496158619880e-02 3.003423400350053313e-03 3.757271588081435854e-02 6.515483104117367941e-17 5.460884160124819531e-02 7.142300453242973726e-18 6.110199476642608207e-01 3.081781278374666200e-01 2.216857650365220908e-02 3.839469482832818300e-03 2.220002079397968842e-02 1.677045379029677274e-03 1.555941158624549528e-02 5.982008068946795785e-03 9.804188131015773022e-03 9.540962612972949796e-04 3.755432501535860279e-02 3.470277689668761073e-02 2.872037485009558758e-02 1.725046192346365789e-02 3.004441932687318753e-02 6.367632221829821679e-03 3.155686483901089939e-02 1.815254581480553811e-02 3.482592877088206557e-02 1.660864671263956663e-02 4.843454275335336984e-02 6.364798411812188084e-01 5.805430285590708062e-01 5.253819951171347924e-01 1.913994455640299908e-02 1.688825038387085919e-02 2.484793221489790122e-02 9.329802798329282776e-03 1.068274256936207339e-02 1.650586768718189518e-02 6.318884643774100307e-02 6.375358869450942212e-02 3.157012675820605740e-02 1.663825913900374154e-01 2.920779103949126543e-01 3.724693717306499230e-01 1.309855491780676298e-02 3.564411758845664226e-03 3.378997524919513662e-02 5.250523296170783207e-03 6.868480321116923575e-03 3.783854557930102502e-02 4.050200635583977443e-02 9.090383838409264380e-03 5.826027660891775622e-02 5.054665949249880635e-02 6.741065436817207657e-02 3.928648834374379728e-01 5.022302379267735706e-01 4.328060083821669290e-01 2.601155792974196163e-02 3.483862169485094232e-18 1.934424627424345330e-02 3.672714664751618174e-02 6.736264062115021600e-02 4.350916821311077987e-02 1.116422359304331119e-01 6.254359343964363549e-02 7.825153680038361703e-02 6.705707647030335894e-02
1.214885110799081847e-01 -5.762679499680238571e-02 3.792520971396683149e-02 2.218138299709004543e-02 1.773429850949066397e-02 3.669213028537719745e-02 6.693456143159312588e-02 1.272035407501513095e-02 5.158600711084436463e-02 3.211376152307834908e-02 1.261337885730082764e-01 1.878278190140991732e-02 2.060442837120695161e-02 3.780183803671987447e-02 3.802941400254927773e-02 4.020918756564808259e-03 2.721809297642417691e-02 -9.501610776305681808e-03 9.585807592144697520e-02 7.463822779050046860e-03 1.860233365909243902e-02 -1.635913183132582660e-02 2.259768175959352490e-02 1.886022358614705685e-02 2.514187784776298826e-03 1.649032020107971466e-02 4.139805188085865506e-02 3.677899026122838908e-04 5.343245836685388229e-04 1.127429798856268350e-03 2.359431129211009889e-03 1.044651720494708990e-03 9.204179131158999293e-05 4.366918703216102924e-08 4.445376222007605023e-03 4.546154979408373147e-03 4.945339281061100593e-03 1.834982791004416414e-02 -9.882861448178521802e-03 -1.461191762839737091e-02 -1.560664357230333657e-02 3.024193878446895800e-02 1.137379757726137142e-01 1.126300269218428740e-01 1.123990993974468278e-01 1.007066829352597781e-01 1.558657361530897956e-01 3.160208364118120145e-02 8.992900614527668535e-02 4.972645057920144218e-02 2.262042799984840397e-02 4.217897882770025447e-03 3.554634081223993769e-02 1.092579315424455078e-02 5.897842199163178672e-02 8.763605011294083036e-02 6.427411795426291041e-02 3.088040982762138564e-01 1.482084695206820069e-01 2.919257633116612710e-02 -9.497362823903578166e-02 1.609199716729341895e-02 -3.176295569447314115e-02 3.305038755974919196e-02 1.790225933935941580e-02 -6.748924059656295771e-02 -3.879909059974364324e-18 -1.544324553958120174e-03 5.439098312729243739e-02 4.467143581099089489e-02 1.021474336660532084e-02 -3.012234393776036002e-03 7.554377923590324152e-03 5.170424211046331653e-02 -2.705167013852581483e-03 8.669242742982606609e-02 -1.597089691343340362e-02 6.016868273279649126e-02 9.617091588530926605e-02 1.103954707050017747e-01 1.610128827293542120e-01 1.222220795059702908e-01 1.104122992933345149e-01 1.977599640861025035e-02 1.272127645879300717e-02 -1.783187708133892463e-02 5.790538457864366495e-02 5.660084144057563743e-02 1.942546902361083497e-03 -2.242753748030423489e-16 -3.811636323714066160e-02 -3.891453060499296991e-03 2.271782047538611904e-02 7.651877004581035535e-02 -4.413389166724370621e-04 1.031333348634857257e-03 -1.822385522263168800e-02 -7.554021915822910216e-02 -3.104944393413638232e-02 -1.786238815607062760e-17 -7.732770908261185801e-03 1.726070754527488649e-01 1.693116914638475570e-04 1.652409054476242922e-01 -2.047256522000919648e-02 -2.622787353026664420e-17 4.097103895137234608e-01 3.178331838407449372e-01 -1.257469404374524525e-04 -1.436910743741234625e-03 1.001981480153724818e-01 -2.840583713800020216e-03 1.417943068231400632e-03 -2.149073193827666373e-02 2.629554573143332366e-02 2.748501405877041594e-03 3.089429700727134437e-02 1.066684740867135800e-01 -2.137905695914549785e-03 6.035984857733873825e-03 6.536570479423634561e-01 4.420540963180705218e-01 4.525649928721848375e-02 2.923108406510901980e-02 3.139504552056383846e-02 1.201043920247555352e-01 4.896094541645972659e-02 2.134719944041326214e-02 2.192170049294935377e-02 3.478209052580219507e-02 4.328073938806042964e-02 2.628495867806568062e-02 2.578271954729407719e-02 2.473538150791605023e-02 2.916019305649761306e-02 2.032304331534778505e-02 3.163958551436808070e-02 5.621877611532495322e-01 5.920824689588765066e-01 4.948425185776322843e-01 1.737901788519450805e-02 1.551931813230999699e-02 4.420630916956964918e-02 3.603164603762918500e-02 1.686518055728631130e-02 7.972365569473243221e-03 4.245557160701355498e-02 7.651865052072303816e-03 1.550625594987218268e-02 2.494761816150418803e-03 1.059317326090085387e-02 1.441076559731445730e-02 1.160539116360380721e-02 3.109809121029272812e-03 1.642522913407016769e-02 8.740853483255889983e-03 6.490751760688023321e-03 1.660054050689779848e-03 7.738028530648483061e-01 1.276443054942485666e+00 6.359722615242922039e-01 1.444713908693570359e-01 9.867075508448633347e-02 1.265657041281005200e-03 2.628661718506700856e-02 1.024716958258999749e-01 2.115610369722225118e-02 3.973747198937406799e-01 4.867071179425587202e-01 3.726129105177256595e-01 2.218579334817899137e-02 1.262574355806191616e-02 2.286148613836142679e-02 1.211503569595788060e-01 9.127510297044681697e-02 8.385830492511178735e-02 1.112904721967333671e-01 8.228004474452038641e-02 9.185907241989024752e-02 4.077789518273413488e-02
3.188523314098269945e-01 1.944031164012823346e-02 6.751529230662977471e-03 4.537999222436205386e-05 2.183091562427930902e-04 9.651185311514464890e-03 1.657760485029994429e-02 4.597050675561893847e-03 2.456444981937848208e-03 2.682225683755364281e-03 2.677978656772642824e-02 1.788015986564890636e-02 1.051072728464921315e-02 4.291517606773868898e-03 1.196583101331403750e-03 -7.164975007207471935e-02 -3.557391900724037048e-02 -3.780601780872149681e-02 3.488441994594847911e-02 3.775227310762373917e-02 5.101741888622472049e-02 1.068308578525891187e-02 2.592158433050252654e-16 2.356033768426298719e-02 1.320718009850092300e-01 8.259408063407591011e-03 2.041375947506479313e-02 1.821137524613888198e-02 2.093486749546281250e-02 2.256786127381802537e-02 8.599894554708539807e-02 6.714198597018486425e-02 5.575142124327687326e-02 6.414048439280237157e-02 7.130010962223494453e-02 6.319017992743321266e-02 2.016769159323133706e-02 2.201586104522765611e-02 7.084634328112263124e-02 1.653542584265383850e-02 1.058344917792622544e-04 2.052897163952519324e-02 -1.311337089209511832e-03 -1.021193170242520355e-02 -1.398316764725884495e-02 1.214121431739407508e-01 1.987625464722838076e-01 9.901129994948652813e-02 4.409699330703159559e-02 5.058860438640220419e-02 5.218935433251169903e-02 1.480144595417570708e-02 -1.522510922018083246e-02 -7.955128291675260532e-02 -1.167447517682602132e-01 9.863483609626436110e-02 9.755346733323118358e-02 3.625395798617024323e-01 1.425819446120193956e-01 1.360131248122016523e-01 -1.264035759494964162e-01 -2.144824591541973080e-02 1.697188708921715944e-02 -2.653959164718816500e-04 2.329936648955362524e-02 1.408339875152457912e-02 -4.257410357715401780e-05 2.137463044435849654e-02 1.856731618073755030e-02 2.856000581719687903e-02 -1.065965254967485157e-03 2.718360532743896094e-02 4.020233191916699339e-02 1.061656811645123565e-02 5.377243204812878329e-04 -9.165538138409090972e-17 -1.875990805850426876e-03 -3.736382938624931597e-17 3.505199822876219930e-03 -1.876571270994255650e-02 -1.539885636266250793e-03 6.247389731938291987e-03 1.311910373021338630e-02 -6.225131397807124982e-03 2.587543769983809486e-02 2.243888459989126088e-02 2.722187015431423024e-02 -1.811456026794960178e-02 -1.424699019644458907e-02 -5.571025863983737267e-03 1.473758243824124792e-02 -5.647919081835403414e-03 -1.544326635885869079e-03 9.150262944783113875e-03 -8.637401069702278084e-04 1.365540639703366650e-02 2.100286241637684941e-03 -1.705568225220921788e-02 1.590168686195661537e-16 -3.870630853418195391e-03 -1.168195538528027377e-02 8.142149037655064397e-03 -3.363687874342403787e-03 -3.117240166218161485e-03 8.161459732889306262e-02 -7.763117479809659993e-02 -7.285513507555984626e-03 -3.804462185116955913e-02 3.175749330942199272e-02 -5.661361967398152335e-16 -2.088819950417683507e-16 1.765132340212787962e-02 -3.194744753585288671e-02 -5.537138288779393236e-02 -1.645045344997687550e-02 7.309195610713403735e-02 3.280670872753929657e-02 6.629143463567802756e-17 5.036871761742572201e-02 1.254953141287627402e-02 5.958412305252882568e-01 3.432056321137342825e-01 3.365218352260022916e-02 2.140721552724887070e-03 2.226537460787556683e-02 1.956034719523870060e-05 1.556413107952177473e-02 2.022222465810150603e-03 2.490449082720974996e-02 5.881195639797647673e-03 3.889945544642604836e-02 4.256632728144173206e-02 4.223804249666476829e-02 2.478398727226778855e-02 4.482569720856048601e-02 6.090659687681996282e-01 6.922259036636877472e-01 6.849405556366283676e-01 1.585278281022954941e-02 2.499147983014518223e-02 4.985116944218276336e-02 1.313503141368534827e-02 1.062240100644902541e-02 2.390689487315990347e-02 6.472940104569711861e-02 2.569951914857339023e-02 4.440302027244451971e-02 2.636532396564903249e-02 3.369022236655046937e-02 2.965678053746143364e-02 7.067386014436689845e-02 5.335623826327920427e-02 5.090362120381821537e-02 1.214001367798899500e-01 2.267497000419828324e-02 1.808013579676116117e-02 2.178345408863678959e-02 4.575474844689854034e-03 4.702826817159916351e-02 -8.359561377793915237e-04 3.828989822048729091e-03 1.568622397581075252e-03 5.781673387505991135e-01 8.496937919899379965e-01 5.139226338709280695e-01 4.724896765793224623e-02 2.628436114319603750e-02 5.246613717693918849e-02 4.079007160106681851e-02 7.327824566061721945e-02 6.884376905916475553e-02 4.899973073610162100e-01 2.871452615784389484e-01 1.454485461010345482e+00 2.415489166491410211e-01 7.539969063267195834e-03 1.019929297641755800e-01 6.122341983565347118e-02 1.575445386626843011e-02 8.469332862497751593e-03
1.434228264419884491e-01 -7.112012604449642617e-02 7.559574190701161267e-02 3.753322979276602772e-02 1.347409458819083874e-02 2.634540829980068152e-02 4.995512936607212490e-02 1.399620375349492664e-02 3.032038251642121124e-02 1.986609646962383957e-02 4.477460601850544175e-02 2.531706387812344278e-02 1.186299436064301716e-02 2.519270368145267239e-02 2.023058127805411541e-02 2.562457386984965876e-02 4.462171901236025584e-02 1.249679810716565649e-02 1.459198040226601482e-02 2.178582775203057736e-02 6.706660725209077911e-02 1.881507130712267717e-02 3.756262978907354277e-02 3.988852561071305758e-02 2.563834592291805783e-02 2.917274692714106124e-02 -8.092805145364068142e-02 -4.938163991493815796e-02 -1.064605942925268139e-01 1.086439995564685360e-02 2.939086797973296944e-02 1.086302364633273849e-02 1.132865503290511784e-02 2.541893771075945212e-03 -1.533563056671560232e-01 -1.161288274574914237e-01 -7.553427782771059140e-02 5.987196262441708133e-02 4.244254249354657144e-02 2.184312154425282343e-02 2.774828609018012773e-02 4.404900856337302245e-02 4.752695824876784075e-02 2.232220718420347208e-02 2.598169818136896250e-02 4.040978402600700625e-02 4.782133874639112048e-02 4.183600315658898144e-02 1.383695272189615355e-01 4.044837410583001364e-02 -2.632867835483417799e-03 3.586444418130809580e-02 2.219739999278460760e-02 6.485437824405049234e-02 8.275345059559281735e-02 6.581614387730952420e-02 7.268362029155747694e-02 1.306551958897594623e-01 7.586919455798948886e-02 4.214941273436287328e-02 -9.851386502079002849e-02 1.377712656154359352e-02 -6.635509594454249219e-03 4.370793858662100919e-18 1.423953398398419140e-02 3.923100112928795333e-03 -4.257872688288759431e-04 1.754183838818730579e-02 4.534651567490351198e-02 2.783175838990499182e-02 -2.484653458179615639e-18 1.391161452088677912e-02 2.911879596779375079e-03 6.445049294113147902e-04 2.602022748353500104e-03 1.746047836770035430e-02 -2.743917053786105141e-03 2.070730998814784146e-03 -1.025089219133934818e-02 -1.089839678117603475e-02 -3.228342443698508439e-18 1.276783317200620368e-02 4.551810287746065574e-03 -6.261096039288979785e-16 -1.002752882577436812e-02 -1.079062842399591306e-02 6.309326331843843720e-03 6.812405904582792028e-03 7.155269770797843647e-03 -4.146539913980199761e-17 3.490463961360725908e-03 -3.831730234064349922e-04 4.118352377674193432e-02 5.131405519523287578e-02 1.373353910253175903e-03 8.900618010112923417e-03 6.488606477104373604e-02 -2.903479601730245968e-02 7.961371472700606595e-05 -1.237715816349771111e-02 1.776932002450789388e-02 2.384452433579515619e-03 3.033830197695503925e-02 -8.049625333398532229e-03 2.600380717199624134e-03 -1.476893852880003177e-04 1.118600661290312243e-02 -1.656155399194940425e-03 -3.367514697227995987e-04 7.997251735152655083e-02 1.099352084418524084e-01 2.878781670746947529e-03 1.539871740427704583e-01 -5.061339843188314425e-02 1.256147053750999477e-02 4.243094836367602117e-03 1.991301242771618615e-02 -3.380804430571968111e-02 -7.660676123637927097e-03 -3.424134626768462336e-02 6.116402164870429203e-01 3.818398877930127555e-01 4.030559823701371919e-02 1.615347630527950648e-02 2.048877708538754047e-02 4.330322038334585150e-02 6.340279675620594246e-02 6.530727637036865159e-03 2.500563199532920833e-02 2.068670799951274203e-02 3.397100015504365272e-02 1.464712382251635338e-02 3.780351853130679840e-02 1.200593914886972056e-02 3.050380136164412490e-02 1.758414409625814676e-02 4.653848832049828355e-02 1.552609691450252544e-02 1.259306028415252410e-02 2.448175213792689112e-02 5.051214680362552806e-02 1.914158950564930420e-02 4.929130223467109023e-02 3.278652957088617342e-02 1.598215577864352602e-02 6.570761932551006518e-03 1.018405999788669636e+00 9.967347164811559690e-01 4.180400194434165662e-01 -8.387597696240242651e-02 1.606976110068062541e-02 7.636365210143799924e-03 6.779036691026319068e-03 -6.494758277227509480e-02 3.398116324118689580e-01 1.185832034302715776e+00 5.522753894459061508e-01 3.717269181604952039e-03 9.031877727669451392e-03 1.868882620577418455e-02 5.526580319452449597e-02 3.749463054073302881e-02 4.320236306971095980e-02 2.779070469294591186e-02 4.559047547987321591e-02 5.718521935894101821e-02 7.154559701628099289e-02 5.466908074758631414e-02 3.645224619437221758e-01 6.026106554964675288e-01 3.543122240892377373e-01 -1.428532877098567452e-02 1.095623029078688498e-02 4.257393853842781489e-02 5.788813919244339967e-02 3.765051851244808484e-02 1.166108767023399384e-01 5.635400406246126453e-02 6.469251712534822285e-02 6.683466482569691558e-02
1.434158448479453118e-01 -5.488860170046246101e-02 8.043777349118876041e-02 5.044272008762924753e-02 4.550921501561958765e-02 3.109733789459899131e-02 9.436241257290738094e-02 3.909078889750408625e-03 2.685550679903854140e-02 3.556357719614091500e-02 8.977837265282941070e-02 3.874184098397978615e-02 1.618556003950946676e-02 2.580218648182296126e-02 4.029818889937414328e-02 4.060537683870700520e-02 -5.955963650452005681e-02 -7.244649681398183938e-02 -1.092473769329507793e-01 2.102192967679936825e-02 4.048265795222491525e-02 2.710370239069359935e-02 1.746683683067200635e-02 1.555161618912729617e-02 7.565885569685244783e-03 1.480157710736398728e-02 3.704315695612307030e-02 5.338016885941956197e-03 5.491622232178561295e-17 1.115104949643839675e-04 2.958477657636747297e-03 5.801014867259558609e-03 1.986046498257369871e-04 2.346353570358498790e-04 8.088132886843744972e-04 3.422859533811951991e-03 -3.424139663993283622e-02 7.307186659430016905e-03 9.070710781129361772e-03 1.184130030199238154e-02 -3.431697922733342471e-03 -6.241398184605907382e-03 -3.309903133141282872e-02 1.079711265398505468e-02 1.898202795130967888e-02 3.259825652710824501e-02 3.133084383189756184e-02 1.347577301729104525e-02 8.655761800333838796e-03 2.728316776361962986e-02 4.994987322806190150e-02 1.185932531161420014e-02 -3.337715122940419776e-02 -3.396211732034767894e-02 -6.506128224980407826e-02 2.104271541068137019e-01 7.662325123399509053e-02 2.981693934178776262e-01 3.383507691026861330e-01 3.835703107178277116e-01 -1.286011524474170442e-01 5.470142649124094814e-02 4.651747404483455757e-03 8.994461471369579256e-03 2.524040825689293985e-02 -5.172352132800286816e-17 -1.041197925654935785e-04 1.523580787437127555e-02 1.785260566772986848e-02 -4.903759753629975033e-08 2.574745356390330600e-02 5.325082307209635414e-03 9.920448420762565528e-03 -2.447706046782446854e-02 -3.290628648252624106e-02 -8.013480711015586594e-03 -5.212251026972304808e-03 -5.147174804076887211e-02 -2.271469820486204660e-01 1.232470684565539487e-01 4.244068508559263597e-02 7.980626361379632161e-02 3.730060331959932085e-02 -9.038601636793579730e-04 1.359552431248798980e-02 -1.993067808483036871e-03 2.864485687878587719e-02 2.087397445676150709e-02 -3.826526739599731680e-17 -8.635115767332474969e-04 1.531125241010425432e-17 2.093925701445138732e-02 4.740792537973247296e-03 -3.406947478796686080e-02 -2.251314745173614356e-04 1.371208017272582957e-01 9.977506000334210312e-02 -4.001561672599412978e-02 -7.142218177063523006e-04 -2.284093320046415299e-02 -1.728734955307106511e-02 -1.634323826417566697e-02 -2.288131139073115716e-02 7.793230947394849803e-04 -3.684459983668305835e-03 -2.861122290960577911e-03 -3.407156572104117712e-03 1.799722494192619246e-02 -7.484987641834262191e-03 -2.085018233035836749e-02 8.058381260075270225e-02 -6.282748995826120241e-04 -2.200409891737955398e-02 -2.014130702213895799e-02 -1.771539598666876589e-02 5.810932044714844924e-02 6.704570348690024051e-02 3.268796077492477220e-02 3.051536596860874445e-02 5.598260539281619709e-18 6.191642492037081613e-01 5.059107814343472009e-01 1.259500645747836656e-02 2.824211183406420342e-02 2.371935032261847029e-02 8.561695086800585885e-02 8.193708485922446272e-02 3.021847853774896689e-02 2.232185086506901367e-02 2.351569545363003597e-02 3.159779011919838787e-02 1.497890768626321256e-02 1.895115655397712484e-02 1.371954918531324494e-02 2.910911769311460423e-02 2.682862019193724135e-03 5.089483285969680360e-01 8.221043296212360296e-01 4.436434411915934906e-01 1.272868813778471261e-02 2.297142010210512578e-02 7.088012830098554436e-03 4.574618222034324000e-02 4.021643430330898439e-02 2.218798500451497901e-02 6.915240803743591229e-03 3.381766281346757635e-02 6.499202620898517427e-03 1.661843923986910249e-02 6.924565392809980437e-04 1.219338501939130209e-02 6.954045029844348191e-03 9.998308791066213425e-03 4.659172554228822719e-04 7.670761894800981473e-03 1.334048238413259192e-03 3.999625403035269797e-03 2.819837139299502873e-03 8.473465836439333931e-03 7.017802196683613908e-03 5.595178232412175712e-01 1.162586414082367847e+00 8.303318660494622971e-01 1.860020134737294314e-02 2.056738801247460016e-02 6.179761035910743411e-02 6.898679448219187382e-02 2.633201436689827993e-02 1.575993734257124385e-02 1.060750577941461936e-02 2.250314813701153813e-02 -1.535214680704562358e-02 7.684730763265532616e-01 3.713100769536176182e-01 2.972940979561437547e-01 5.131137818285314012e-02 1.165111063274583969e-01 9.140910877740650309e-02 1.200535122478645772e-02 1.556911930130713655e-02
1.452478146563355443e-01 -4.951658081538552053e-02 1.015406387200844529e-01 4.389118759739869119e-02 3.003644065411295236e-02 5.175783523581155288e-02 7.267050153918587496e-02 3.486826523123014859e-02 3.324912060644481804e-02 1.787228503643911726e-02 5.309265529074211343e-02 2.465942842023158624e-02 1.785680257129626178e-02 2.390315879796261686e-02 4.979732126797616065e-02 4.207696054665038127e-02 -6.994226009964521484e-02 -6.406252666448820909e-02 -5.857690648680503875e-02 3.393966896318082599e-02 8.205531021561716931e-02 7.238007233060786243e-03 1.196254902824628492e-02 1.787981808182957308e-02 1.599227321330758816e-02 2.869331726700145857e-02 5.508366672834389549e-02 3.182145118788769558e-03 8.244920030506078004e-18 1.459990693041695112e-02 4.933117437306002058e-03 2.091235850093596960e-03 1.369317900020235743e-03 1.567557110794477997e-03 4.301846743881595873e-03 8.946323793392241611e-03 -3.296097712736477392e-02 9.599848221239071569e-03 2.869331871143493418e-03 1.210717271189592527e-02 -9.298636603868154146e-03 -4.698525491544877121e-03 -3.673096673881325741e-02 2.363328207342953614e-03 2.733942911338071297e-02 3.760915717451467077e-02 4.132777612734470085e-02 1.625883755431372507e-02 3.540842717369122733e-02 3.871492977790783052e-02 4.355621749763133321e-02 1.249066979191602172e-02 2.605413045783620124e-02 3.523884596659629387e-02 7.841430649342778780e-02 6.826566164215962240e-02 1.009223229428241192e-01 7.024517050277623298e-03 6.768596231830252552e-02 3.712271354426179004e-02 -1.346047230708776743e-01 1.494126128828731671e-02 -1.089315899935997456e-03 2.332681531532759811e-02 2.703380628090258295e-02 -2.835959444756007072e-03 1.256841976491285913e-03 -1.139862338973055087e-03 5.294090552367291397e-02 8.570500598184790974e-04 9.702257313147806886e-03 1.228287492549238677e-02 1.134308540813871134e-02 -1.558739694780707577e-02 -1.710532528249456355e-02 -1.471641989237089387e-02 -2.574750020228822305e-03 -6.900327913120730960e-02 -5.803284677148294601e-02 7.133457419511239628e-02 -3.239643126753601217e-05 4.734308025743315385e-02 2.181840466360995823e-02 4.503680644671677509e-03 3.484520251739646468e-02 -8.831938760688004889e-03 2.118379256391227095e-02 1.746972288791516503e-02 -1.120296503613423421e-17 -1.314671621666805602e-02 -7.553473394896841574e-03 2.949429232197199255e-02 2.766124557064720754e-03 -2.544008593546599056e-02 -1.547999073237998934e-04 6.663718987013622641e-04 2.583330385181772590e-02 1.758713131613759702e-15 -4.603361802557267863e-04 -6.341370119468515976e-02 -4.208101820043796439e-03 -1.880071878684821257e-02 -4.075156895686722625e-02 1.453352194958432153e-02 -6.185345328666221054e-02 -9.436380145974234698e-06 -2.305729835068577593e-02 4.421505557974569380e-03 4.269083383400426574e-18 2.912331554763191929e-02 3.517147230073434244e-02 -1.932194876357883062e-02 1.612674047172361913e-04 6.158460835747145423e-02 1.915715831026110899e-02 7.094761485036894973e-03 4.170216569199444029e-02 1.750227993368752011e-01 3.067866024789391854e-01 2.731029112838603140e-02 6.733237178353961294e-01 4.790172767035891233e-01 1.920952348085964642e-02 1.968640203915236753e-02 2.850812473097310076e-02 1.190235133876341306e-01 7.825281148459461988e-02 2.142634603996831227e-02 2.384597838098299200e-02 1.858672302808201879e-02 2.911741507609108201e-02 1.078855012745098818e-02 2.662014687185819406e-02 1.117454263115268825e-02 2.773158306583994523e-02 1.916529101368623931e-03 4.716234897735180343e-01 7.491553899181695453e-01 4.369420387160150021e-01 1.423821737823811884e-02 4.353622070540143218e-02 2.874655782918114286e-02 4.258918698080688819e-02 3.417764764846230319e-02 4.961214216947351652e-02 3.073104547191107816e-02 3.806713565699017948e-02 3.314214890711779295e-03 7.070952702184598801e-03 4.936052507261471343e-03 1.286551138224940362e-02 1.211273306600036716e-02 1.280123674680820611e-02 1.046456054916735734e-03 1.709771916871155248e-02 1.988097622560271865e-03 5.530925088589423895e-03 2.029693588646048520e-03 1.279024170050984538e-02 4.472968823215657877e-03 5.442271752375502647e-01 1.096718573109630945e+00 6.720165907683961493e-01 1.017740498936747654e-03 1.441901260556339992e-02 5.003902668020904221e-02 4.816801190952375855e-02 1.532391849766844352e-02 1.901585862668729240e-02 1.671982656267058040e-02 2.733819907326641777e-02 1.007953001699672321e-02 1.130761284505175934e-02 4.690411570424335430e-02 5.002890592399174968e-02 2.022853135981498509e-17 1.084789468652502253e-01 4.529167604001262948e-01 5.412213607090572420e-01 9.452196332478381979e-01
2.903798863072820993e-02 5.999832800300190783e-02 8.659699716875934872e-02 3.777277326186836287e-02 4.743428383563088463e-02 6.416879962565265327e-02 6.338023769110166794e-02 1.730707412915339499e-02 4.226942765032638827e-02 7.785093828005199001e-02 7.190259036079925392e-02 -7.130562123041223055e-03 7.573691776220855404e-03 1.613070992874928236e-02 2.783773284773194846e-03 8.307371993102674011e-18 -1.340124606795884722e-02 6.420376585262276350e-03 3.203713264642340380e-02 5.522797328066014755e-02 1.168188629919036797e-01 4.660897398797157060e-02 5.023389110066673463e-02 2.386018413429954674e-02 3.614804324863685886e-02 4.748069210708607407e-02 5.053751144254500782e-02 1.050822508083653316e-02 4.716660408116154537e-17 3.861377256562217365e-04 1.342724247260382135e-02 1.739789309566854014e-02 -1.691609171819956695e-02 -3.684932375676185784e-02 5.101994705936429606e-16 6.224012921661067031e-03 7.967366927114788184e-04 3.605057844093458811e-03 1.563396031876403483e-02 3.254502859477423522e-03 2.091519317460657085e-02 2.843967237206168272e-02 2.200570482714822584e-02 1.044352309879757051e-02 2.307343690268924205e-02 2.141972141030288115e-02 7.061320962313782923e-02 9.671188790757057757e-03 7.923288799089055895e-03 1.211982580929490266e-02 2.167362271783749086e-02 2.086226360963996593e-03 1.131528697451231538e-02 1.208648510124040940e-02 6.599480602878819713e-02 2.227314781025453022e-02 -1.001154533332092866e-01 -1.479418875847395010e-01 -1.626972326748975795e-01 6.548407206383959067e-02 -1.059944175938546014e-02 -1.089350731831338365e-03 1.316656204179787459e-01 -5.562403022006853423e-03 -7.118562775780298661e-03 -3.411383489774376474e-03 -5.635361718678879618e-03 -2.537929630940173337e-03 5.629707415167069240e-02 2.946322207973877186e-03 7.177881365569795113e-02 -6.463476383841375938e-18 4.366829149514941943e-02 -2.553129453643257149e-02 1.495796911887278813e-01 -3.431741815955811035e-03 3.485296155179911596e-04 -1.605291347668959950e-02 -1.274922790873601192e-02 6.705160611340470374e-03 1.402172656686441650e-02 2.790835453141273306e-02 -4.354622250594042570e-16 -1.207378086159863195e-02 1.372886285056980002e-03 -2.626694397481689480e-02 -2.247336954893143382e-18 1.225199517515805636e-02 -1.511985417208825305e-15 2.027574975367641905e-02 -2.666978876607154630e-02 -4.586501167250575141e-03 3.977186819942305634e-03 3.467771561359446475e-03 5.109140651873339892e-03 -3.552471218959186907e-03 -8.251639615266503913e-02 1.444349409022470040e-02 9.320678170401207677e-03 -1.027785244822203986e-02 1.354684198117071976e-02 3.860722346060630372e-04 4.760184795964637612e-04 6.654717500296598864e-02 1.704858772045495097e-18 -1.368345512395308207e-16 -2.802444328107691479e-03 3.993465299379425626e-03 6.196180371775481845e-03 -1.073679515855871817e-03 -8.773040008118295423e-18 2.241995790642891881e-02 3.611241609482931242e-02 -1.794513460366247261e-03 9.618927192800233450e-03 1.630976574005714094e-02 -1.322211642134172582e-01 -5.932009374123185796e-02 -3.292006642660230131e-01 1.726845588807482956e-01 1.564037719157050643e-02 3.718508102731461068e-02 4.029849955787065396e-02 2.205840290228783251e-02 2.128007491102058488e-02 3.242414579562444277e-02 5.817118388592695810e-02 2.849284075263222842e-03 1.062517529257225174e-02 1.239124171582736819e-03 1.589635611059743783e-02 3.112890879998760768e-02 8.467221183429471734e-03 9.552702050429972636e-04 1.160102272792439687e-02 -1.077704007454237251e-01 2.287598500138406099e-02 1.972317340455280107e-02 3.032406462765838140e-02 3.747421696852915041e-02 3.147416737839144052e-02 8.857795629389523043e-03 4.636608789871272257e-02 2.184097120606804526e-02 2.923754077612117364e-02 3.779453977555618377e-02 3.515198213549416278e-02 1.404359900974310252e-02 2.476968801840274612e-02 4.721356155156755294e-04 2.762103013808975271e-02 1.621451570291707750e-02 5.732393967973878901e-01 1.090460997389262277e+00 3.524501395384600499e-01 8.381939469672972098e-04 1.158702181091070954e-02 2.209080149919260064e-02 2.653002646826644467e-02 2.301332951223651266e-02 4.998323753842781603e-02 4.310918032729076782e-02 5.048652342363772055e-02 9.661208029463821262e-03 3.382445453592947127e-02 3.520027723234947503e-02 4.342029461177245031e-02 8.200471716652710241e-03 1.774118280454765190e-02 4.199479532962254597e-02 4.675454798432185366e-02 -2.549023564494649462e-03 1.486719704231093467e-02 2.883438647330585289e-02 5.815260392358393565e-02 3.108680227077912739e-04 1.182204793568684686e-01 4.317846048549307536e-01 3.796507557284269918e-01 8.269386192005415692e-01
1.052881858667616827e-01 -6.116886099921532011e-02 4.383528828846625092e-02 2.393121275855662061e-02 1.688071729860026599e-02 3.383841481423583569e-02 1.105757007649562917e-01 7.457732634798926001e-03 2.447687396831144460e-02 2.178627041015434099e-02 6.666586403765357993e-02 1.460274146693710678e-18 8.730947080448666145e-03 2.457153446231175042e-02 1.226071961119873377e-02 8.845157667520902461e-03 2.827359416601680771e-04 3.941836931598683065e-04 1.793923314674660541e-07 1.787475202843319053e-02 1.232721408044161887e-01 2.360535182771740514e-02 1.523546733845304937e-02 3.299875173588981414e-02 1.232453437231481597e-02 1.244980056061453788e-02 2.642330193655327865e-17 -5.017306701319916573e-02 -8.578856343353104263e-02 4.233965330631445957e-03 1.871199788142464612e-02 6.986522032812694620e-03 3.614169585440167945e-05 1.141786118035637140e-03 1.566142839150914609e-02 1.045267181526172810e-02 -7.935383164365002184e-02 6.934615295744810896e-04 8.115837711861053488e-03 8.311386122162594745e-03 2.189614198983623472e-02 8.805470511667440325e-03 -3.438378062466448254e-02 -4.180940547458501799e-02 -5.737337573021775161e-02 6.386180344087317995e-02 -1.015270653515285004e-01 -2.282957906791719879e-02 -7.797521621915936250e-03 4.093911990095819992e-02 5.396751223949677878e-02 3.152043191698181146e-02 2.829585930397911925e-02 3.668307792349553842e-02 4.968929770618065594e-02 3.781163158327328216e-02 5.803129212490255873e-02 3.655923967228800336e-02 6.256770214438327204e-02 4.095543711676941090e-02 -9.866533070789097748e-02 1.901565608038004174e-02 -2.035816780250992997e-02 1.682955689481319995e-03 1.089703120567399493e-02 -1.317343086792969846e-02 -5.466367522686173707e-16 9.201355926774394678e-02 6.351748407891319448e-02 1.634674140389885677e-02 -2.567791395729989573e-02 1.647775210629326675e-03 1.895307254229206201e-03 1.677212549558872775e-03 5.052965972441813246e-03 -1.539247747842422147e-05 -1.059050101228142153e-03 1.935374708987133881e-03 -3.407084094203199215e-02 -1.473656205845029141e-02 -1.088437769017127826e-15 -4.774787059923201989e-04 6.430448740295480366e-03 -4.919432217284504250e-03 2.170757237820585152e-18 -2.643871413186771584e-03 4.338916265636261205e-03 6.161388214788897887e-03 6.631551030168318650e-03 3.092748804541425867e-03 2.194311932206392118e-02 1.889121494447740940e-02 1.710093205815908443e-02 -9.741271156141017781e-03 9.097314865588923549e-02 3.814528583118427518e-02 1.578952477687806805e-02 2.727821919780522644e-02 1.178155726430275718e-17 2.547887463044345643e-02 1.439545071871263038e-03 1.635528740479824314e-02 -1.176947067105763717e-02 -2.038361954876882071e-02 -1.143543144366649972e-16 5.462206125299393089e-03 3.028763926133689818e-02 -2.151944736533185215e-02 3.143250500477769854e-02 -1.895614262685586110e-02 -3.398626442367863744e-16 -1.455749720024661700e-02 4.325420343107675043e-03 -1.565674585925621600e-16 2.552970651309259195e-02 9.794501620106529222e-03 2.280661827371485359e-02 2.748009830892541661e-04 -2.413352281864520991e-03 -6.291669450143860809e-02 6.682037130843397899e-01 3.783784762694318848e-01 4.232503007274372037e-02 2.674078449951718117e-02 2.290321376299584225e-02 5.457630914258898569e-02 5.116298509664000960e-02 1.082827297495695014e-02 2.749901999766879435e-02 1.590697373961076286e-02 3.327196561532296676e-02 1.970953947541752641e-02 2.870351003220228403e-02 2.014697322845209262e-02 3.753782262037722722e-02 2.502923772259764423e-02 4.044919410295904488e-02 1.209121770047286160e-02 1.889435369180853322e-02 3.840977930059722573e-02 5.338541312438635500e-02 1.070211875273126799e-02 5.329636653100576493e-02 4.142111493566804498e-02 2.151514215864913301e-02 5.588992591013287881e-03 7.472193661097549233e-01 1.096149140925258392e+00 5.301881249128956863e-01 5.076870320533998469e-03 8.755433716380654416e-03 6.465848367981686523e-03 1.036751658678769505e-02 3.487850561194600469e-04 9.573543102001980110e-03 1.854009047971804476e-03 2.295333455961060670e-03 7.324826250442321822e-03 1.284789564252832028e-02 3.588678160723279137e-02 4.805599150053443958e-02 3.609687337079908742e-02 3.802195631579179114e-02 1.055631041949126186e-01 4.556302878496968495e-02 4.392399462880897804e-02 3.586876637960689784e-02 4.475967586482357263e-02 1.608940125537008242e-02 1.677964949097268677e-02 2.587893189359165638e-02 3.262886113296378787e-02 1.564890642281693486e-02 2.434669909663572315e-02 5.221012361298556920e-02 1.532078317032671902e-02 1.193452896521375733e-01 9.715566643970320604e-02 6.987625150294951570e-02 7.078974058985240680e-02
2.379532261013726224e-01 6.858556155691503120e-04 5.402994319264216132e-03 1.043376585438712638e-03 2.107837094811159241e-04 4.070043619537159173e-03 1.082395696123913823e-03 4.771830223951186388e-04 3.597077287000354211e-03 1.750355771213997144e-02 1.497046263628318101e-02 3.680432340479974918e-02 2.303437003321522417e-02 4.232912530072321933e-02 3.277343345680780451e-02 9.116954254417376086e-03 -1.538939534041520877e-03 2.418375923978826509e-18 -1.926262886772294597e-02 5.995128896948506009e-02 8.116293898196416667e-02 1.836599198268924066e-02 1.265730569383437113e-17 3.689902532502918608e-02 6.156484622921907374e-02 4.467747970294359827e-02 4.086334081730211981e-02 5.796466237638892710e-02 6.885298411811859032e-02 5.370387846145838889e-02 7.152768626556992027e-02 5.148025058666851755e-02 1.024323724058477103e-01 9.083128801851972356e-02 6.178879580621570583e-02 1.020923331013155466e-01 1.650606883001011277e-03 1.039905064596856055e-01 -2.458972606018825988e-02 6.380946321515798547e-04 4.203618088076013608e-03 2.050952120253769426e-02 3.406681106613709820e-02 1.647829866846855715e-02 4.243401599681375508e-02 5.828153454541787348e-02 1.190018197055581345e-01 2.418672652687171507e-02 3.492155581003860559e-02 1.315206538128610263e-02 5.527473890416298441e-02 3.408542740079630040e-03 1.400735513518580264e-02 -1.946673613473403827e-02 1.784048465919352422e-05 -2.827574054384204838e-02 2.573937906843951073e-02 -2.922047111432391278e-02 1.354220300442868057e-02 -2.627133777548472382e-02 -1.521580889699092887e-01 -8.026984899917969382e-04 -2.649349214991995535e-02 7.206574391163214632e-02 -1.584047760057998846e-03 2.930889991501164048e-03 -1.836264516565621870e-04 1.521044477250443720e-02 7.226741880690254298e-03 -1.457480348393842134e-03 2.030446409301124230e-02 2.594643631253614954e-02 7.294455652806688083e-03 -3.268940594485100248e-06 1.487680506210343730e-03 1.326452584811593435e-02 -2.716033990692335968e-03 -1.903392463227606725e-03 -8.248992393514655019e-03 2.899026198090146403e-02 -3.063800523144722190e-03 1.144523747428555401e-02 -4.186317911328743535e-03 -1.027134685429579274e-02 2.539409233602468036e-02 5.393095192509114212e-03 -8.459797885267509329e-05 -1.841494480735993144e-02 2.092737563726725786e-03 -1.305928955734913730e-02 1.921065203043575975e-02 1.309059677710545672e-03 -9.429790605906963519e-06 -6.035280555598490464e-03 4.295037799614841167e-02 -8.827484626878571647e-03 3.640715438078476983e-17 6.231538324796732919e-04 4.444484132844114810e-02 1.332153567567391506e-02 -1.255788571274338024e-17 8.471428069663911042e-02 -1.536356277681788680e-02 -8.336268815529049650e-04 3.428740952156664884e-02 -8.079888749971899167e-03 -6.381660744688670908e-03 -7.965156659742883300e-04 -5.053642667500531094e-03 -9.547388194793582164e-15 7.439363159313296982e-03 -2.151371733727475383e-04 -1.195451423667237001e-01 -1.138186420717055393e-03 -1.169100120757732460e-02 1.587629764933188095e-03 1.029840992020285631e-02 1.369229848922166162e-17 -4.020057725322403371e-18 1.960916265054169286e-02 5.728124848758181509e-01 3.808198981435277908e-01 1.715291827431238147e-02 3.144473446053538349e-04 1.796651712755146721e-02 2.526204491126116548e-03 2.722288634247671188e-02 1.089364720500714083e-02 1.404846371794888499e-02 1.169587388197062747e-02 2.336194881062426260e-02 3.031785889821638283e-02 3.860506789776452230e-02 1.805112594536020798e-02 1.985549392925988615e-02 3.012376771408102638e-04 5.322230080244340078e-01 1.001091439151770457e+00 4.475373057869225790e-01 1.169427445941723792e-02 3.603728636063489865e-02 7.959211322935859367e-03 1.110622664431714507e-02 3.992936686283511671e-02 3.485376494589962487e-02 2.759741403948670971e-02 4.576270247567204574e-02 2.392001035217919402e-02 2.904279186972742385e-02 4.745329547072635323e-02 8.098749449375643172e-02 8.178782352440798487e-02 5.354108394876561788e-02 1.245553987527267398e-01 3.392881329581894090e-02 1.520650002650958052e-02 1.255417411870043444e-02 6.975432323910250032e-01 7.733005750608584750e-01 7.392400268907806105e-01 2.524720489984426559e-03 1.782513796014794857e-02 3.415564028216269005e-02 2.139586475116727129e-02 5.034394678198936962e-02 6.070311163578932800e-02 5.877915535615345355e-02 2.965673801092227543e-02 2.267928945257752238e-02 1.583469688617493779e-02 2.045025099922970907e-02 -9.901956557480688301e-03 1.465069044361686119e-02 9.651085977856330711e-01 2.790122005242429148e-01 1.730536315819912463e-01 1.506483243142781592e-01 9.203364811876196816e-02 1.138557402047038619e-01 9.823829479735689263e-02
1.368573105015058744e-01 -4.210749766735911309e-02 7.393480543667879656e-02 4.731868748959470322e-02 3.068956984762075818e-02 2.413969772571339725e-02 4.426885244280442250e-02 1.711601053875218834e-03 2.718999733659335782e-02 2.248342116261121265e-02 6.454028161436854705e-02 9.292586594808169981e-03 3.030143087654433090e-02 2.898071690021486110e-02 4.435876010763293437e-02 4.899133588333171724e-02 6.082332388900552056e-02 2.411606773320493918e-02 -5.200683691237846691e-03 -8.053394917520177720e-02 -5.725517230211398662e-02 3.689774874162098442e-03 1.991927865869575626e-02 1.557465960695034526e-02 2.122650193944108227e-02 5.027915224356821478e-02 5.531204141600563201e-02 3.648742801156688691e-03 6.278478811024322016e-03 2.087536643111114788e-03 1.468546911160230166e-02 2.958619056752151161e-03 6.802250422350027267e-04 2.558472029765867725e-03 6.031936766311716235e-03 -5.130984881985588553e-01 -2.555709466007504149e-01 -7.438315993785397928e-02 5.865205290802302796e-02 7.733210685396199602e-02 6.163819821302017610e-02 3.591961028503175279e-02 3.585207490027993854e-02 1.633252172477621736e-02 5.406594005432586308e-02 1.467306974641691686e-01 1.582099850805184227e-01 1.068652163489117457e-01 5.490726831551852316e-02 6.232531083286386303e-02 -1.559663398751770166e-02 5.189378241932057767e-02 4.631966003424043549e-02 7.452701744289008667e-02 9.946487864166385362e-02 8.031127888077903287e-02 7.754002008575781868e-02 6.331432638700328075e-02 6.239861521366410479e-02 7.017579934290860133e-02 -9.211001605763675937e-02 3.365372490048909593e-02 6.818112244372478981e-02 -2.161986944296354352e-01 2.109265141028123652e-02 -1.157952711611384862e-02 1.547545582007857920e-03 9.391470117880700786e-19 2.631411850498562369e-02 -3.240257204116082325e-18 9.099038494811474177e-04 3.756518166467209298e-03 8.489690194288657468e-03 -1.737473247003219756e-02 2.991479073060771682e-02 -1.664970918783359932e-03 -2.293004695504158568e-16 -3.190809698036925490e-02 -4.352480082999861388e-02 -6.413493567572731147e-02 -7.096184004626009123e-02 -1.296926418613061583e-05 5.365967973840068267e-02 2.669006176943093667e-05 1.047779595955128200e-02 -3.495214758986184641e-16 -1.002725651899810535e-12 9.495699597787522414e-03 1.007118079121568444e-03 1.097309093691615361e-02 1.119462398404880886e-04 5.512913385499737529e-03 -1.161928883030034429e-02 4.320278637949353379e-03 -8.998222253761069614e-03 -7.748322904599652325e-03 -3.647675275654014334e-03 -1.265250702908912381e-02 2.204226948016722518e-02 -3.836544503632275532e-02 9.375854493200899697e-03 -1.171650067311408479e-02 2.140274450427625402e-02 -2.269481729555538480e-02 1.362713346073046380e-02 -6.796784284254961599e-18 -3.392644741380646325e-03 2.643300464713043210e-02 2.283066398743995029e-02 8.969559057915684613e-03 1.827904020661687626e-03 2.715882925594743924e-02 2.987448769727689413e-02 -2.124288672256782404e-02 9.822463986750783541e-03 9.890840714128263170e-03 1.566517634576022194e-02 -1.539940382511062640e-02 -3.236487019011816992e-03 -3.689727311557538386e-02 6.165692226568371126e-01 4.030986675915085304e-01 3.585507126640299219e-02 2.828452095002581546e-02 3.345045848632694635e-02 3.647582601509855721e-02 5.616788224205337965e-02 1.038340380576937942e-02 2.563771942768822143e-02 2.570153803154932165e-02 3.046550738665666433e-02 1.492473037693748360e-02 2.372088930129199677e-02 2.360493770036556707e-02 3.632207071862907782e-02 1.071431368113619047e-02 3.318136892334580534e-02 5.234209090170101504e-03 4.301859726327419975e-01 7.085149109877412199e-01 5.120359611343088257e-01 2.817773171843547184e-03 3.849438322836991333e-02 2.467962538808250728e-02 2.509636180024264057e-02 2.242293437020333180e-02 3.127208687996356484e-02 8.701644234486258234e-03 7.130951519732274535e-03 1.812774852831635242e-03 2.352007132769183551e-02 3.327460003419555837e-03 7.962399735756598221e-03 1.758214093518652502e-02 2.890474518682699390e-02 7.204370147582692674e-01 8.242221801344131737e-01 7.291756779050850845e-01 1.451410835991237373e-02 1.864706636440627285e-02 5.044784734097476631e-02 5.570984811876966009e-02 6.625105115736112937e-02 1.648788912168332871e-02 4.477734662108819041e-02 4.205009369458805046e-02 6.772461956458597498e-02 1.425323656740762855e-02 2.345268519510329275e-01 4.140923650736158534e-01 1.903332482254382341e-01 -1.028627033537021958e-02 1.425261577386525633e-02 2.635943878889304010e-02 5.311348115007104298e-02 1.017349294629466550e-01 1.313045434284138557e-01 1.370593302716831552e-01 6.631163877733238510e-02 2.682085442499147765e-02
2.354808270249253710e-01 2.664453649967451349e-02 4.658286763984431471e-03 2.840402599392209595e-03 2.112277811407333763e-07 7.599019942530128749e-05 1.359338201568184535e-03 3.288751178842971162e-03 4.337029384807862716e-03 2.418190925847583939e-02 3.204642206212117428e-02 1.692467808602710413e-02 3.533415830616063597e-02 1.545208226955481316e-02 4.792786809079892935e-02 1.014364345152039826e-02 7.198004593156131885e-03 1.779643734180534967e-02 2.185450580960976441e-02 2.797792087279636977e-02 6.495818079225224029e-02 1.515938289053036722e-02 2.621227248079893929e-04 2.451381005687708683e-02 1.969849131977531587e-01 3.043839255520499817e-02 3.680158157782531525e-02 -9.546319426439754327e-03 9.578935252885761864e-02 4.958520334143979962e-03 8.733551831371089147e-02 1.510958053427201297e-01 1.379199947569509643e-01 1.107427837703794798e-01 1.158209885978075254e-01 6.006382439286420821e-02 4.198110852527895537e-02 6.868062340320388315e-02 -4.375731260441706461e-02 1.766355671954065792e-03 1.072331690588302147e-02 2.852530106728260254e-02 4.331525368484069605e-02 3.133412050581405039e-02 6.038492339449851032e-02 7.401662218623847322e-02 7.338297985731019601e-02 9.973922951616826654e-03 3.414792555837491833e-02 1.669257287721824870e-02 -3.248066169943996523e-02 -1.060096532801883991e-02 -1.541660809732629117e-02 3.010881286700758941e-02 1.294310232401490457e-01 1.692051206712196221e-01 9.085984470254405976e-02 7.493387087085216780e-02 1.330148120134427014e-01 4.331315980616410460e-02 -1.567918177232573707e-01 -1.191179236661295705e-03 -2.496267698441916386e-02 2.795864650045099681e-02 -3.702658485121785898e-04 1.239238198272499605e-02 -2.683278201641125117e-03 1.132022726734640238e-02 3.310202318913426419e-03 2.304323607743462288e-02 -2.292252108078153550e-02 -5.288169932179723854e-05 1.399243328473981406e-03 1.442109290605704704e-02 5.147372858727721628e-03 3.271588023672249157e-02 4.841634573309834544e-03 3.331030177364818767e-02 5.090310593116381330e-03 3.581911484874460150e-02 -2.402797283403428169e-18 2.743924417589000200e-02 1.483604161298711886e-02 -8.471259383689278347e-03 1.890956513074602136e-02 1.533047414376530714e-02 -5.780221531448961544e-03 1.681903100600736986e-02 1.968510235045247206e-02 1.060460416101605130e-01 1.406599030269927927e-02 -7.067807959685855346e-02 -2.822892729787274471e-03 -8.815248750897401037e-04 3.149359819368231783e-02 -2.047038845903621437e-02 2.117694475332840840e-04 4.737425790900600189e-03 4.035748153240608510e-02 2.189426322451840795e-02 5.763608814107389284e-04 -1.150827695833609929e-02 9.696558239125853754e-03 6.435032655978807116e-03 1.274997605502018946e-02 -2.901766706402283957e-02 3.132853308774172230e-02 -3.524122095722808518e-02 6.111347207661370351e-03 -3.406420746131048199e-02 -2.265905918641666558e-03 -1.118825552872367533e-02 -1.240033095508319590e-02 1.317522197150250603e-02 4.098601825084857370e-17 1.971731795091278064e-02 1.071148991053896132e-02 4.519691636278692221e-02 2.962567362566799478e-02 -1.017803040947228704e-02 5.890266908195080919e-01 3.642103870720814762e-01 2.041318782633211315e-02 1.748126106845241225e-03 1.145172448002776747e-02 1.376016855362943432e-04 1.750522429052041654e-02 2.915107644854291608e-03 1.782750121238110483e-02 2.843615199549802321e-03 3.343480906618405413e-02 4.915554762024954005e-02 4.477939929343676545e-02 2.832666671897803717e-02 2.993056008130496118e-02 2.438204961318765282e-02 3.522194833293210842e-02 9.633842327280669679e-03 3.285788077124091044e-02 2.106576015929324808e-02 7.067362292707202176e-02 1.387810494532243366e-02 1.134645530329802740e-02 3.586695411555278906e-02 4.882064076054233592e-02 1.239119716115813508e-02 2.896282351816380218e-02 6.320670565691713927e-01 7.875716427857670787e-01 5.660219721323156428e-01 2.623742922536800512e-02 3.919451655994470041e-02 8.171065388611874203e-02 2.120795708019442838e-01 5.251245683831465283e-02 1.389630394913341470e-02 2.711045343705844116e-02 6.276275177626051693e-01 7.393287872426741458e-01 7.657272177545738634e-01 6.936338087496850269e-03 1.614978122333432173e-02 4.430379399654844896e-02 6.052049185094932754e-03 3.687534916498673432e-02 5.524411098208385773e-02 6.796870628401806280e-02 1.891631619167537248e-02 2.045210100549190238e-02 1.194944696947639225e-02 4.030840220080310066e-01 6.778125752264430037e-01 4.500263444148213665e-01 -1.365737799071437518e-02 3.395355538585414362e-02 1.139718437267356610e-01 1.030133991106858965e-01 2.863633510023116946e-02 3.276226695318410626e-02 2.128377006103919500e-02
1.237252169639841415e-01 -6.044026629071569401e-02 6.385242563916834757e-02 3.260833759101156026e-02 3.685765613029511056e-02 6.249743134145749335e-02 1.204203731008258310e-01 1.352125588525075403e-02 3.078645094204101457e-02 2.035967901986821676e-02 8.169874467126853057e-02 3.318391510723742860e-02 1.304344643915309430e-02 2.756208343130056929e-02 3.954447163703173757e-02 4.583230876612577609e-02 7.951668467053950307e-02 1.017768057576040955e-02 8.184570413450716541e-04 3.251566416330064352e-02 1.717516461302134312e-01 3.670759148335465960e-02 2.045147737554191616e-02 2.132325213577686979e-02 3.591844562616360731e-02 4.075976985510509909e-02 6.371706604106998351e-02 2.818347207954523606e-02 2.159521907393097129e-02 2.123019958359681839e-03 3.063151733745786854e-03 6.887147650212454013e-05 6.023287776633778770e-04 9.971306811411180526e-04 -1.397603046943420024e-01 -1.961383070535421824e-01 -5.887608382407233387e-02 1.270402418579103312e-02 2.051335862547320338e-02 9.282330058384488311e-03 4.330422389867198685e-02 3.579696311747318505e-02 3.048880301153822706e-02 1.723546479285023234e-02 3.734850766488177071e-02 6.434715112653636993e-02 6.002923400657456110e-02 2.637827489882075757e-02 8.273140671614840139e-03 1.881767858707455651e-02 -3.719299203615430982e-02 -2.060227896074898549e-02 -3.235643115709097711e-02 5.873279259967465610e-02 1.369241842627914318e-01 9.233325851397730655e-02 8.996452647009116343e-02 5.785561522207424145e-01 1.699088717931957482e-01 7.953291131500760114e-02 -9.331066804306750728e-02 2.475365235553508153e-02 -2.108382526413736524e-02 1.130577163556447003e-02 2.070061007785553922e-02 -2.341531363431648485e-03 2.776120134344059118e-02 1.104096413348540232e-03 2.397633999444791914e-02 1.698998563970179412e-02 1.026263868500876790e-04 2.549218718363004468e-02 5.564036521547482787e-03 7.382962136183134072e-03 7.624881497694897689e-17 6.783808876445217295e-02 -8.447973500373677502e-03 -2.366166676040570230e-17 -5.108731899065131407e-03 -1.079092808577123558e-02 2.077923956658291396e-02 1.434948965176013544e-02 2.582424299073735399e-02 -6.823678046895634286e-02 1.796730576777830907e-02 1.023944840796948974e-02 3.049879407016554619e-02 2.564019653238133398e-02 -4.054800444940040438e-03 6.144643712419259850e-04 -5.679922927595604069e-03 2.113867513344957864e-03 7.010305924390954069e-03 4.095302614787248857e-02 -5.391158534277384895e-02 4.633779980511301905e-04 1.232454345118365845e-02 -2.481630295307325762e-02 1.157483138237654433e-02 -5.500859389282357521e-03 1.351297415015730030e-02 2.728611245832657239e-02 7.642480866955452846e-03 -2.829379278327326039e-02 4.218899124444138934e-03 -3.212453764500302136e-03 1.215545095159742406e-02 1.836695945928802989e-02 1.161594486781961730e-05 9.932950203712489004e-03 -1.078398743492604839e-02 -4.530179985029470641e-02 -3.057432146907360093e-02 2.674373209010697666e-02 2.994600119386669427e-04 1.416278872639724178e-02 4.245844573340644412e-03 -5.088375485884246048e-03 2.326875353711049116e-02 -1.003021807400465183e-02 6.160417879117980888e-01 4.090996497848517488e-01 5.136671439383633242e-02 4.910265466238415005e-02 2.108802959354483947e-02 6.204492838889756612e-02 8.099128836674127085e-02 3.861045948864052883e-03 2.198656605364871994e-02 2.044654292358697931e-02 3.720865684508229548e-02 6.386986092197558494e-03 2.722519167164953041e-02 1.038499600749633912e-02 2.929092387022221716e-02 4.080491566144496540e-02 4.880377162961686832e-02 9.784364169898867800e-03 1.775992227654568309e-02 4.099906133152970283e-02 4.245583681743730214e-02 3.094343838894770757e-02 4.060749480348196938e-02 2.673848225121955649e-02 2.443602656332392528e-02 6.865092835388627579e-01 8.300583011910674935e-01 7.052156345565674389e-01 2.835546395337235070e-02 2.405710307550921676e-02 2.156060539245053045e-02 7.037334177249673396e-03 1.892155336552391043e-02 7.837903662042371156e-02 3.588243879074758946e-01 1.180933871066073282e+00 5.531301882167330985e-01 3.178311742734534623e-02 1.927325319807481410e-02 2.339482991318135932e-02 5.202652502306946009e-02 3.034215031627905679e-02 3.877533727033361660e-02 1.277019110304785862e-02 5.123025866920714877e-02 3.319471419771467813e-02 5.311810564899043657e-02 2.727352794286449708e-02 2.265644492700164686e-02 2.185794615002958560e-02 4.100825057185819578e-01 7.792406137145394540e-01 5.399354488214583858e-01 5.966721602151819209e-03 3.389951313398643917e-02 1.047522742776388766e-01 1.262301198744424369e-01 1.295138438826768691e-02 4.239662208621267081e-02 1.964439533561306711e-02
1.366872753287762254e-01 -7.116290537189713661e-02 7.160508312098846917e-02 3.344060293661135080e-02 4.550860715404716345e-02 2.089590273351181818e-02 6.353434288832389421e-02 2.595054677225533749e-02 3.962706974805813021e-02 4.276530651783357756e-02 1.115395113624954998e-01 3.732806584087339230e-02 1.294924137831104938e-02 2.222985224014292424e-02 3.462571630526781880e-02 4.238842243903775953e-02 5.824400274591264720e-02 1.739863885081641484e-02 3.733917770859769038e-03 3.165092135259135048e-02 1.085066127434815980e-01 3.694613439951342088e-02 3.181960211622670426e-02 2.985186080647329529e-02 9.206577479409460557e-03 6.343293106055707120e-02 9.544540856027061704e-02 3.126774406444082083e-02 2.907580519997349320e-03 6.285170270469162337e-03 2.279830288641503111e-02 7.411279576348719401e-03 2.238248769892332856e-04 1.598724806243448718e-02 2.664341737004006019e-02 -7.196171712210177596e-01 -6.786696334788336338e-02 -1.979268691885313902e-01 4.093172228803955426e-02 4.060596809238233018e-02 3.384262684284904815e-02 4.471412578641791036e-02 4.610201612403341115e-02 2.957025283009720104e-02 4.738753741158200500e-02 7.454366330446933475e-02 1.315455957928687580e-01 6.216532960306155736e-02 1.926285282022580439e-02 3.738492526572544106e-02 3.289591228402671125e-02 2.236046848455310944e-16 -2.378903726183393200e-02 -7.216076492011332111e-02 -1.112836972598592600e-01 1.133298707802039840e-01 6.735119265966338364e-02 3.525002897506807620e-01 2.824329770909412840e-01 4.806394217373483047e-01 -8.660781492700654616e-02 1.093647572871048711e-02 -1.937620337625926836e-02 1.087863246036078802e-04 8.945940382724462681e-03 -3.017830918578409509e-03 -1.604553044018604182e-03 -4.194928984796677043e-03 1.417032322810319461e-02 6.543211056238139312e-03 -6.335411724547819305e-03 1.516185693447866419e-02 7.730703720809313506e-03 8.785355159519033300e-03 1.779165656812466481e-02 3.687405690448142953e-02 -8.630374932294610987e-03 -2.068118280207236967e-02 -9.152653951365597590e-03 -6.162616962963019820e-18 -5.156609566554327147e-18 6.410437108477365711e-03 2.266012338054504069e-02 -1.622792240250976042e-02 2.616311708925437755e-17 1.867935377385721019e-03 1.548846761374728792e-02 4.070935206428412390e-02 -1.031961188001969537e-02 -1.182914994238909445e-02 2.042296396122856086e-02 1.474730042243723660e-18 6.619389039131427094e-03 -9.246753658703581180e-03 -3.846850415359551190e-02 -6.320727508851228457e-03 -1.306359136180140594e-02 -4.713804230548677976e-02 4.426267882403238374e-03 -3.954960444190893688e-03 1.018604169829168647e-02 -2.805030773376427592e-03 1.967804842524322959e-03 1.924712881610026252e-02 -1.939665049593852775e-02 -6.766314551651760783e-04 -5.621569755261770530e-03 -5.835282149274518540e-04 -9.860617951690313701e-03 1.812309566788508977e-02 -8.618539603473245278e-06 -1.822853792831980986e-03 -2.009251231970081341e-15 -1.002012282433206204e-02 -1.689302895724391351e-02 6.713799412554444235e-02 6.250558894488990325e-02 6.879689896099076482e-03 3.019932189652824173e-02 -8.842401078729730801e-03 6.271120288289551148e-01 4.061110853821899291e-01 3.746765986976097890e-02 3.785896711339989396e-02 2.057997271096936170e-02 4.942483533544406171e-02 5.654281200957698983e-02 4.113349008439593293e-03 2.614336258267356897e-02 2.771047499195498204e-04 4.215471061099770184e-02 2.986375882322833599e-03 3.112821677299492176e-02 1.508171227594270331e-02 3.439271441463350593e-02 3.966596817517423212e-02 4.354958931821414220e-02 1.962452110635787283e-02 1.990196484237832103e-02 1.616248503573191470e-02 4.649285179169519028e-02 3.349770949565220729e-02 5.515544775149968521e-02 2.791346300702865837e-02 7.095992393310020588e-03 6.587704122540031237e-01 8.294441560408302161e-01 7.067851137483740498e-01 2.905948265189754465e-01 3.352164658982014211e-01 1.703764564859612807e-01 2.272131036370602652e-01 1.569819706816912852e-01 4.218900652300192622e-01 2.637747794161633408e-01 9.012711515624383107e-01 7.743348407558229285e-01 7.387424434537648299e-01 2.480089984741863998e-02 2.360182102638272092e-02 4.034498393243260428e-02 4.545637139011796807e-02 5.111953086181388051e-02 1.711784057937544568e-02 2.749121056410715538e-02 4.499828966606576192e-02 6.383342274956305129e-02 2.273153074302004806e-02 2.229501466178224547e-02 1.610000447757902603e-02 2.965064850162267909e-02 -1.385418175031386326e-02 7.043952474985952783e-01 2.978401536309297493e-01 2.285379638015758996e-01 -1.219778924037205989e-17 1.222005554417509854e-01 3.139448037246426321e-02 3.477893235263048582e-02 2.112148601270405837e-02
1.991097337625072072e-01 1.902053427167569311e-01 2.384259676370246162e-01 1.483467668410194951e-01 1.429325710659250515e-01 3.948178869638092059e-01 1.769356897278445140e-01 1.568592131829706776e-02 1.212629659197222709e-01 1.228379005117850344e-02 2.756149774617341897e-03 3.118015376078281061e-04 1.890397291544901326e-02 3.869633498198139787e-02 2.608002536123630102e-02 5.539849808210735999e-02 6.325398939338772175e-02 1.897453859230721976e-02 1.234964598785476785e-02 4.160419025507833962e-02 1.265833323320846304e-01 2.180236835866821224e-02 1.816962036089928012e-02 2.015597931787493566e-02 2.247141124229406952e-03 5.882685736181257619e-02 2.242184278167071165e-01 5.208174623352074262e-02 4.907350962589936139e-02 3.344898726163923752e-02 8.833313213276598741e-02 3.318865108920732165e-02 -7.727983057183667424e-04 3.966492189075267009e-03 7.084999380488998626e-04 8.954908580178797071e-04 -3.755226855400427971e-02 1.410059514267488097e-04 7.151040783916908818e-03 1.767430926301449129e-03 1.727622839171440855e-02 2.053246325301590416e-02 1.973665888482436029e-02 2.728369578109552870e-03 2.497766218411863470e-02 6.484291112085834030e-02 1.014560002716935333e-01 3.392706143294169086e-02 6.028269067569291123e-02 2.557372373719555766e-02 1.918957022942060012e-02 1.377537118682604855e-17 2.024533733428966059e-02 8.194924337985550269e-03 5.655133238552533664e-02 7.229037169979353472e-02 5.671789646306943378e-02 1.210316747478900196e-01 7.099524814909152770e-02 9.601359196596359266e-03 -1.634278162262789436e-02 4.373573958979277976e-02 1.119296270073216470e-01 2.327791097700318904e-02 -1.261080689124642260e-02 -1.286367621574932008e-01 8.531770420319431669e-02 6.298353959061149532e-02 5.240251431925442918e-03 3.295774311031059861e-02 7.247642563387586036e-03 1.534854204134663419e-02 7.660741836586357038e-03 3.556728467919028497e-03 5.813761873736834895e-02 -1.282483800448883451e-03 -3.599401984903548377e-03 9.164351295387423990e-03 5.058545792338589933e-03 1.118186335868052607e-17 -6.467680395308754654e-04 -2.868857350613596147e-04 6.680386606212490586e-02 2.082452846878002015e-02 7.806531978257890726e-03 1.122365304384695351e-03 1.813081422730400058e-02 6.694254314651480742e-02 -2.551682547408378604e-02 -2.839291940964417359e-03 1.528222793945251401e-01 1.713200037395995468e-02 1.544032074606109355e-02 3.731172636810391879e-02 -1.022280227123295443e-01 -9.175680720859433204e-02 -1.055810540678584278e-03 -2.953811219569030255e-02 1.784273980488196645e-02 -1.588212551481306021e-03 2.532751721971620781e-02 -7.652473178915774141e-18 7.225161771341032102e-02 8.735204416642219308e-04 3.920795713091967255e-02 4.212904002518229329e-02 3.543142806065076034e-01 4.131851113300406830e-02 -4.187895362476422648e-03 -5.061274127973584575e-04 1.132823112742022409e-01 -2.602207666757514956e-02 -2.454984150809668905e-03 -1.131885099978676736e-02 2.399147128956484812e-02 1.378471532726615462e-02 6.069239036240445251e-02 1.195053141480909194e-02 1.890714392474530589e-02 -9.953752897338109806e-03 1.122380546203818330e-01 5.961191880748423211e-02 1.145503240309024839e-01 2.509893193334682970e-01 1.165768594267148905e-01 1.185639612072944965e-01 7.206060484515157405e-03 5.567308714629127797e-01 6.359397155974624294e-01 4.236242024206777756e-01 3.975988329505771718e-02 1.690694021509470471e-02 3.405413018959822097e-02 2.277121017118293594e-02 3.351521465713436926e-02 2.953487512383827854e-02 4.903839108953911285e-02 1.564340162233343601e-02 1.709412641852220069e-02 1.650900220120232395e-02 4.469177161398917020e-02 2.583498446044647789e-02 3.959459052402175550e-02 2.165243772047054788e-02 2.246992924674172734e-02 4.673063229905788374e-01 4.214193720004034693e-01 3.108632546030132282e-01 9.454067266915869641e-03 9.111771363111600963e-03 2.371371725830803134e-01 4.427913141927878327e-01 3.574363680726686532e-01 2.412371099667966201e-02 1.839746701951159094e-02 4.301904624964015059e-03 5.307927708128538057e-03 8.623221166352189326e-03 2.613218129171749129e-02 2.449947952525887915e-02 4.060709325526248858e-02 3.981676393099910921e-02 5.060360663500869383e-02 4.888580494374722592e-03 3.089503496969437171e-02 -5.457963213306660233e-02 1.087892355468778396e-02 4.417571809552695017e-01 4.652680215204405001e-01 4.436140407689259657e-01 2.512656846360003873e-02 5.818813414974578074e-03 2.533015902541347827e-02 3.255792950516760154e-02 6.967609708051805684e-02 7.528223671249496818e-02 1.224299789170273101e-01 3.523945702483778641e-02 6.801835890915391980e-02 4.318645308670594068e-02
1.135420093392272151e-01 -6.966413406874306935e-02 4.503202510106075335e-02 3.718739176944281521e-02 4.019226464456138875e-02 2.937457170255335787e-02 6.464423001288532866e-02 1.134489775417622109e-02 3.284475244051358184e-02 3.757329091125118092e-02 6.915173985634055775e-02 1.265177903639741648e-02 2.203475533907128109e-02 2.851006810210222897e-02 3.058390904307224656e-02 2.183191150182574661e-02 3.249220654810791586e-02 -9.357161161710469141e-04 2.521779860933978656e-01 2.255797986550067257e-03 7.772734767197736483e-02 3.873367414561621966e-03 2.203003899240467411e-02 3.989392553916162182e-02 1.168421740290640116e-02 3.154468694350639341e-02 8.389435772408185277e-02 1.733090892739437908e-02 1.757443514662080720e-17 2.448680390488758686e-04 7.699734525512889893e-03 1.274820674115465478e-02 -1.731999679245172993e-02 -6.307431124097356845e-02 -9.562108542899692976e-02 1.205874042617702604e-02 7.521701718252638350e-03 4.504590960104502003e-17 2.183005328730560843e-02 2.195336667906559909e-03 2.621553003185965805e-02 2.500369828242066453e-02 2.252537767426072632e-02 9.848455911214479622e-03 1.423095933241239064e-02 1.435095667596761673e-02 2.450404067130224420e-02 5.773217713330992124e-03 2.086806964403562642e-02 8.121721482369113551e-03 -1.169806736353740800e-02 -1.582165730526993928e-17 -2.833011177876771768e-03 5.055138294150650358e-02 1.121605229076999416e-01 6.595373992991121481e-02 7.289169874496020940e-02 3.316696137486728202e-01 1.456343948341422001e-01 5.272073685776345470e-02 -8.459487092910254746e-02 1.111397578323835081e-02 -2.614614975302667274e-02 2.006770624333534606e-02 2.210496118837253568e-02 -1.586677740554610916e-02 -3.934261259798529030e-04 -1.007153085684271896e-16 5.496307424291015098e-02 2.121097651458139777e-02 -2.844135534927972400e-06 -2.698518090955888094e-18 2.191249789208411791e-02 8.895637003598154402e-02 -4.972627581582475184e-03 8.628777560463427032e-02 -2.962752896088916027e-16 3.081135789122604884e-02 9.644937583188883945e-02 1.075500346207907687e-01 4.624184694801111543e-03 8.557140628600963528e-02 7.346169464922698567e-02 -1.008227503184387294e-02 -3.391834805762798319e-03 -1.896505569113102543e-02 1.311880244775253355e-02 -2.332370853830221039e-02 -7.229863038653702010e-18 -1.069128781321208310e-01 2.773612300528775776e-02 2.854797691096756102e-02 -1.742674023689238030e-02 1.589850777499519701e-03 1.703408737216261654e-02 2.009552692799769849e-02 2.049653431536285417e-02 1.814374594508416302e-02 1.588259760219803193e-03 -7.748983647667945403e-03 6.216135846640209565e-03 -4.396196638889391696e-03 6.044299018823686689e-04 -1.502663742726015241e-03 3.473548763171240093e-03 1.985402804144260208e-02 -5.785447324137178021e-03 1.887853470007795909e-01 -2.428918806128104715e-04 1.804727494003673799e-02 -5.251886273914487674e-02 -2.410106969923340309e-02 -1.345164337018844337e-02 -8.290264109801850380e-05 3.662347776877992534e-04 1.137560945315247568e-02 1.992449822683606264e-02 2.851186874279903078e-05 4.681957934295317286e-02 -4.169162388912640849e-02 6.419121236889057114e-01 3.737932241078241180e-01 5.489406310635672886e-02 2.879796844415605586e-02 2.357021261279616461e-02 5.535791399852744293e-02 5.445262088216939328e-02 2.457154920237789725e-02 1.911318804241040475e-02 1.861932927480687908e-02 3.264753810569492876e-02 3.024407134826665247e-02 2.324357178151964520e-02 1.962614795719792513e-02 2.951788793702101593e-02 3.110002801944966935e-02 3.345387560666467086e-02 5.822635779201852380e-01 5.332358294075068006e-01 5.044869442599048126e-01 6.844905957855567519e-03 2.677889939944506914e-03 3.753600159694920474e-02 3.372407834989191860e-02 2.982716553355155981e-02 2.897293697808066629e-02 4.622913023198807481e-02 2.122514361804645269e-02 1.373674109260998588e-02 7.383285530859391883e-03 2.047367714327843341e-02 1.246671183679875020e-02 5.866213643576211334e-01 1.038932867979268426e+00 3.667171688490211134e-01 4.457847710675737034e-03 4.946380721544741711e-03 6.102019134002534956e-03 1.466551113385824133e-02 2.081875108338147526e-02 4.913472866120848631e-02 3.550339914744079367e-02 4.890522334342834593e-02 7.623854869553710507e-03 3.897971807501172947e-02 4.410036273371969323e-02 6.021655779201568609e-02 3.673464299921411214e-02 2.173784595595589647e-02 1.711599473615416436e-02 2.222829617219500842e-01 5.870651418507657482e-01 3.767201133006110036e-01 3.387057714895806357e-03 4.233198027442619527e-02 8.012985510715341098e-02 1.203339943268921380e-01 1.130557614528996990e-13 2.285213101773647323e-02 1.541029624429326832e-02
1.272265818892383445e-02 2.736407710847390401e-02 8.831846258316271292e-02 4.865848035851818437e-02 2.691213804000719065e-02 4.446016481309868901e-02 4.911202311793665121e-02 2.219064416130947862e-02 5.005672863812386103e-02 4.538769403210415193e-02 6.596893631559204441e-02 4.987672688700358313e-03 1.670828481973318791e-02 -1.641242730745167550e-02 2.253901140945543252e-02 3.046787278629829732e-03 2.420703313655715033e-02 2.852585788273159784e-03 2.131611632408492163e-03 2.936127843855856900e-02 1.348530163064399767e-01 2.569131122117455923e-02 2.574516742678566172e-02 2.292072614775073416e-02 4.777855978294044681e-02 5.480197981631294846e-02 6.555310672335430688e-02 3.100759257993052770e-02 4.033135754024595450e-04 2.399176359281087606e-02 1.399036254422269220e-02 -1.968661051105112178e-02 8.417865131904386788e-03 1.621873947996652987e-03 3.530188575785384725e-03 1.488703386455147255e-03 -4.245055993466049366e-02 2.083474966140058074e-04 1.088133520521267564e-02 3.211511076967323800e-03 1.870642077573519968e-02 3.234744610105937113e-02 1.456655919519489473e-02 5.584855798435916022e-03 2.326826186766290869e-02 3.745285242502784107e-02 3.608070995183840918e-02 1.373853971385946733e-02 1.332359746850085569e-02 2.288093847027775421e-02 3.668339090425581234e-02 1.305308731252858212e-03 1.036124853917331883e-02 -1.356271420606714774e-02 1.563382791382643262e-13 -3.004925750427645675e-02 4.656109097096204630e-02 -2.190952476828303408e-02 3.265948377070545745e-15 -3.584991369132640299e-02 -1.214494175804372161e-01 -5.182620515311354981e-03 3.972795790418794798e-02 -8.309874448457961818e-03 -1.500381794785240943e-03 1.736120022047922583e-03 -8.764412271156662299e-03 4.953180061880370355e-03 4.442299431111323277e-02 2.815914504238645547e-02 -3.001891278199302158e-03 1.982812351481587354e-02 1.716312339699722217e-02 6.347035501380482274e-03 1.388662556795250934e-02 8.113791639284369250e-02 -9.306909893480916505e-19 8.787611042583708121e-03 -2.539585909354778043e-02 2.189214180329849754e-02 -1.614633881198915570e-03 -5.755127941163797577e-03 1.783487540349510889e-02 -4.784792600292368109e-02 -2.562250623109086755e-03 -4.172983859685652774e-02 2.020016079729565484e-02 -4.029632072363885723e-03 2.416969315053442424e-02 4.782875074561030587e-02 2.363473460003427673e-02 1.896671390036515359e-02 -6.228965439284744800e-04 7.908367648406522399e-04 -1.416175724190764227e-05 -1.102410363428795153e-03 5.846449321999593052e-02 2.181252366553632430e-02 1.372496525787237319e-02 -5.998572968706436818e-05 6.041760419757982899e-03 3.006690242489057363e-02 1.059634966812435719e-03 3.850746774979167154e-03 1.485747837101108158e-02 6.638102641908322395e-03 1.424283491213206021e-02 -1.867677848640669821e-13 -1.319892603206228917e-03 6.496200628487212858e-02 1.966133044903948038e-02 7.748290044894717843e-03 4.293079508107528613e-02 -2.928576493498567974e-03 -7.929563573753239947e-03 2.808302918362946821e-03 2.452111790192912366e-02 1.506347616760972274e-17 3.564288769593311045e-03 6.767684053021934265e-02 4.381211407372399835e-02 5.971557463839643337e-02 7.542083196717871973e-02 4.334495595701656767e-02 2.416239681526100372e-02 1.434767202845075584e-01 6.356741743640989539e-02 1.143606792366938980e-16 1.290463449633665743e-02 1.350012530980387870e-02 2.437468481749736904e-02 7.934196904633206149e-03 1.109834785230908588e-02 7.229178953416653952e-01 6.491640756410695934e-01 6.943741595416371704e-01 1.387478500675994042e-02 5.266786902908296604e-03 1.720887063205117293e-02 3.462114017389159720e-02 2.888179589876860645e-02 2.009367713905282904e-02 4.528983407623692059e-02 3.733896560456620439e-02 3.485074007615445064e-03 1.878661703837443062e-02 2.507300474589414449e-02 1.185688165346261529e-02 1.446019364223448955e-01 -1.692688597486805835e-01 -9.325761221725537009e-02 1.354384631518832094e-01 2.578448604639062974e-02 1.263809033999074849e-02 1.752991267109394943e-02 1.609106874734838460e-02 4.240449414801906666e-03 8.005483758567463054e-03 2.239864069688313544e-02 1.522109148906196710e-02 3.546944675994605223e-02 2.814724067498921428e-02 3.844608223703967209e-02 9.947910879922429103e-03 3.124088367299575342e-02 4.113543597364429510e-02 6.620413886906961753e-02 1.619534405173815930e-02 1.314820266406934585e-02 2.794602774377857868e-02 3.690303217180208872e-02 7.582454338771520358e-04 1.204991929849084931e-02 2.556174722063129789e-01 2.531301539357237451e-01 5.935805479007610908e-01 1.573583461664052330e-01 1.143575636053711653e-01 1.433846461477225886e-01 1.098586645857767274e-01
1.565578249683311496e-01 -7.784187052574645382e-02 1.062115911133039159e-01 3.163076513580025811e-02 3.909405026055052385e-02 6.304935181356056961e-02 7.210483455611250980e-02 1.887618599243684600e-03 3.512566936165362197e-02 3.285074016001862662e-02 4.084293147329030177e-02 3.309653612652688898e-02 1.179253839214343576e-02 2.395115038007345973e-02 2.484002199656325932e-02 4.381440179949414038e-02 6.996981675787154742e-02 1.267592438784703561e-02 2.592466521859314310e-04 1.866868761694031112e-02 2.553556103866668203e-01 3.067929501796898967e-02 2.756398347387240683e-02 3.166769214064276555e-02 1.657516671054164170e-02 2.738210275474628128e-02 1.371042815023678363e-01 -1.975605471615211606e-01 -3.234170319966196954e-03 -3.224876250122106838e-03 4.342126901802322025e-02 6.413522218961319188e-02 6.959170183613395955e-03 4.053412176803580617e-03 2.914124513328401064e-15 4.281925687122622186e-17 2.372306116097349857e-02 3.685899502542883815e-02 4.174537522362876257e-02 4.623523070933575896e-02 3.615028667445461391e-02 4.368463162593459737e-02 3.380403248155012391e-02 1.027612129223289157e-02 1.271447498739254091e-02 6.250381534343389667e-02 6.055505738190809090e-02 2.591278099849103622e-02 6.510323732353379134e-02 1.534157079252547115e-02 4.395880092400328981e-02 2.536933594756381194e-03 2.088075767904473343e-02 3.008634804791272474e-02 6.304743797410200523e-02 5.907337316981491754e-02 6.127154079973282791e-02 5.543258177466971887e-02 5.464519907404743332e-02 5.034716733331359023e-02 -9.563860160031605051e-02 8.908236439662873749e-03 -1.829280911634040621e-02 -5.540278627758728940e-04 -1.125641586450980801e-03 1.717540895666401848e-02 -3.235881514458049425e-03 -2.105950895280431456e-18 3.035711343552286931e-02 6.743428827704803230e-03 -1.853428417229973160e-02 9.782743979426142614e-03 1.532246197654371991e-02 1.173812652684642216e-02 1.732343621344163004e-02 1.291114257722926310e-02 -1.959816532886181698e-02 4.815737532202230692e-04 -4.780232912327204813e-03 -7.049687028684197172e-03 4.590540021187228953e-02 -5.404341471286672888e-03 1.239461119048059466e-02 -9.838643437403401040e-03 2.051252470439127173e-02 -9.025620685423557632e-18 -8.173925306800562673e-03 4.074220147278861254e-02 7.073906838423200255e-03 -6.467660278952468145e-02 1.140554291212751183e-02 -2.658598160661001381e-02 3.848053138094777864e-02 5.446596843453399735e-03 8.989845607013173881e-03 8.021843963982863701e-02 -1.131556366997256771e-02 3.528320742236498719e-02 6.722756311982376609e-02 -4.083684040182272712e-02 1.150544166418146497e-02 -1.342512756955398061e-02 2.727031018915096689e-04 -3.246225692228752536e-18 -2.016524411734784208e-02 -3.696726943044596411e-03 1.240410513953306594e-17 5.113922997946541982e-02 -7.415150608381281677e-03 -5.414696817289817946e-04 6.151001491706169916e-02 -1.284209102092050137e-02 2.531055849299703311e-17 -4.026880031856721470e-02 2.545544591356921349e-02 1.405373970420571196e-02 5.318120675767332162e-02 7.074883630486225594e-02 9.188695126382642395e-03 -9.266258660081957282e-03 6.540087609438530736e-01 3.870006390675068131e-01 4.538408649575630116e-02 1.947421355803529916e-02 2.566105136948423904e-02 5.751922677694566027e-02 5.688210323341516550e-02 9.955797152368507305e-03 2.065503867614452288e-02 2.452483970886187165e-02 3.248071941468606838e-02 7.018629573954664365e-03 2.641909304295983474e-02 7.965606256916144684e-03 3.864191613042961687e-02 2.545607415169245588e-02 5.069588713511016809e-02 6.058066117337906724e-03 1.486138483238223090e-02 2.408141062294066187e-02 4.909460760413964647e-02 2.871176306262609307e-02 3.353742201154281954e-02 3.769919171476586450e-02 1.751684963724573635e-02 2.129106320342573008e-02 4.705170217415312155e-03 5.548193066018859421e-01 3.753187543576277929e-01 5.932147871105184489e-01 3.305411903517331274e-01 4.436051558482687063e-01 3.220430850148426338e-01 8.240611955075554107e-01 4.280187189860540919e-01 7.196445670825684093e-01 7.507937809054014006e-03 2.526125965898372294e-03 8.712502960174960814e-03 3.316757075329362925e-02 4.932958440511062137e-02 5.490595709094719945e-02 5.662443590298748414e-02 2.074834809593921470e-02 5.203590331541662478e-02 3.998425376167073081e-02 4.663753931848042822e-02 4.262589450448044381e-01 5.380355196443777910e-01 4.476930433115463570e-01 2.571723695791944611e-02 5.060663907350806869e-04 1.488120080277567474e-02 5.313063057253936822e-02 6.973579894980287686e-02 4.797669585879447279e-02 1.277362769778735418e-01 3.827249542732870380e-02 9.443407504016533560e-02 6.801698134811395846e-02
2.477623285485254512e-01 1.853418111725547643e-02 9.617218309350467412e-03 2.894473354393042527e-03 6.396300547894815183e-04 3.803010529518833192e-04 3.653803550579729116e-03 7.927135178848804313e-06 6.932182513116001352e-03 7.588694016590029692e-03 1.780945798539680311e-02 2.242786476278747776e-02 2.706595180408437676e-02 2.115578986479848406e-02 6.331278673846635341e-02 8.332558746646135869e-03 4.879888562257727984e-03 1.189734424201922131e-02 1.265015674235260741e-02 2.237640157469523300e-02 2.735299117425250143e-02 -1.225198996433509574e-02 1.028782472682692323e-01 2.461517172227953298e-02 3.071101762753860112e-02 1.109121793939891673e-02 2.281783701073991644e-02 3.189131174299612848e-02 7.467313449437109653e-03 4.208650192783282978e-02 9.331472015543304521e-02 7.941017532300617687e-02 9.535066873949431254e-02 5.842855806473703367e-02 5.827103810222628716e-02 4.216034728280048521e-02 2.225091632887645290e-02 2.695275831980194214e-02 8.102937402546744883e-02 4.151916507147496854e-02 1.929454406449943374e-16 3.553518639401988255e-02 -1.512272434676742265e-02 -1.027026390002290512e-02 -8.690236280848410988e-03 2.398126819275513411e-01 5.124121032265090969e-01 3.717040389033634540e-01 4.826990104534521070e-02 3.019766161521151779e-02 -1.414814079271340709e-03 3.002150444335313292e-02 3.626551049089499212e-02 7.105297644108235500e-02 7.889382063331798478e-02 5.984545173294558013e-02 7.243809634216966109e-02 1.328252142324325979e-01 5.237423284585440708e-02 4.651410620152122977e-02 -1.780229785700772893e-01 -9.488895364921775752e-04 -5.201531206232313381e-02 4.252805184815084544e-02 -4.333504082503292568e-03 1.490588864895335021e-02 2.458966363001987426e-02 2.854837750176886929e-02 4.559357870986568452e-03 1.009126306027609624e-02 -3.849779981043534825e-03 1.314039789662301699e-02 8.241238133168052662e-03 9.553017358851740443e-03 3.100810549295288549e-18 6.964387242189932874e-03 -1.899960135848618656e-02 5.037542748303323187e-02 2.094668856594445600e-03 1.245776984698524978e-02 2.239236866454632352e-02 2.417192618464405682e-02 7.994279750234668802e-02 1.579274583777000862e-01 8.629407773113763336e-02 -3.827972181212763343e-03 4.063555816919447311e-02 -1.815396017014728886e-02 5.113715194023688189e-03 4.486940029470305020e-04 2.433814155835186885e-02 -9.376891242308828375e-05 -1.881937258862358825e-03 -7.401220170132612013e-03 5.656499125614628670e-02 -1.637938458581571395e-04 3.892961345189736508e-03 -9.029959072007067572e-03 1.555923245826523228e-02 -1.897330225897343736e-02 -2.286021252599136017e-17 -1.184855003372422763e-02 -3.974927658268587703e-03 4.282822831187635404e-02 -2.704666558610955771e-03 -1.255679731646456021e-01 -8.789832310837075724e-03 -3.355268731300788815e-03 1.608169827134587476e-02 -1.499675511525048674e-03 -1.001657630627685730e-02 1.948633698476184145e-02 5.741547499863810383e-02 -4.822919566096877525e-02 9.500355501350403941e-04 4.673429877563720254e-03 3.629236074379389027e-02 -1.921863017935174917e-02 -1.480005666616564936e-02 -4.052281266520415398e-02 5.548193089587523108e-01 3.512410069991503958e-01 3.358379404849981542e-02 6.596439705807431885e-15 7.260581579712916886e-03 7.287581777800278618e-03 1.861557334961675553e-02 4.260947802946713631e-03 1.952045280288615056e-02 2.010753641419425058e-03 4.096865953674926064e-02 4.309376423885401958e-02 3.502915048664419218e-02 2.387857948794984420e-02 3.852955748336928804e-02 2.632890614138674173e-02 2.723319806151201436e-02 9.580868726624898468e-03 2.959117746181917294e-02 1.089178674587595874e-02 3.220387934167043276e-02 6.211576161064104040e-01 5.339139141494317897e-01 4.848667226346308201e-01 2.567103231916822645e-02 4.149474450810567566e-02 5.844243745938732665e-02 6.246390066746115161e-02 3.428807979543278123e-02 3.551298790335902233e-02 6.343258012774112553e-02 3.726428419584380120e-02 6.069178074551778712e-02 9.865452280216902015e-02 4.515647859207201631e-02 4.888928088999418969e-02 4.224557707882546642e-02 2.752861464247346665e-02 3.385338938705000078e-02 1.077268494205741584e-02 6.141530934756778137e-03 4.788293682320426611e-03 5.293685525440766648e-01 7.188879549321138818e-01 3.990060601855773270e-01 3.221304336712497995e-02 2.264122499817708065e-02 1.067887681125714860e-01 2.541004017792407588e-01 3.983364175634591109e-01 1.978526874116621204e-01 -2.839800806961891022e-02 2.303848464209633587e-02 2.196782415991188966e-02 5.066951495419994705e-02 1.150596777597998471e-01 1.301764309214346882e-01 9.725056524747639763e-02 5.691605250415483763e-02 3.647871579740172288e-02
1.523158993170542486e-01 -6.360827430687500395e-02 8.535475176133432151e-02 2.915618903004566637e-02 5.922995609933878747e-02 5.137383563751123094e-02 2.179771920054082157e-01 9.929210352493457112e-03 2.354046047360791594e-02 4.050114231187237901e-02 5.077611066643442084e-02 5.071574729920253627e-02 1.755644973972028677e-02 1.758130437472923968e-02 4.528866209961483064e-02 5.384963144239857114e-02 1.309009425430731621e-01 7.462229358174359006e-02 2.324383360598830661e-02 1.570761250705112258e-02 2.223681117379897043e-01 3.883858469334501573e-02 4.549995758522450140e-02 5.874984627303456858e-02 3.666567802721391184e-02 3.161372562471310987e-02 2.851443944642059400e-02 -1.264301258431058950e-01 -4.060442011462627593e-02 -2.110479777776586852e-01 1.287183432377831012e-02 6.499506492728452170e-03 4.793940829353891993e-03 3.921160596432943755e-03 1.315154332700105462e-02 1.035182233645806291e-02 4.205954106658906809e-14 1.711138628330500765e-16 1.787301417183263103e-02 5.371281879082531914e-03 7.563066546516140735e-03 3.610287069780501734e-02 1.823866254295946482e-02 3.436257925665252982e-02 8.799694211078934458e-02 1.289596012350507637e-01 4.684606501733577788e-02 9.114022694046000628e-02 7.041074573198906339e-02 1.084496040703049162e-02 -1.965100978976520515e-03 6.011619281738851217e-05 1.850348394838370603e-02 1.737779321038840363e-02 8.185576815478447232e-02 1.179226197000166898e-01 1.395843186463311680e-01 5.394675829126673733e-02 1.396169302892865394e-01 7.849319501447771241e-02 -5.805394852480091905e-02 -8.771307660546460300e-03 3.917811933062335938e-03 3.780514307484006962e-05 2.327544160125682993e-02 -9.049197203378491275e-03 5.276328204922763271e-04 8.052615941678055445e-02 2.786972676659655451e-02 2.325220406996790501e-02 -3.333838272442064886e-04 5.036761215934403013e-02 2.045463854957650304e-02 1.207328917703233764e-02 7.363173402209671758e-02 5.559717226297059001e-02 -1.260158977586509771e-02 1.086861016414086660e-02 -2.413288930057562107e-02 -2.679793016044521373e-04 -1.160620268897738322e-17 -4.010950194436540758e-03 2.723383668721602105e-02 2.294613395836364936e-02 8.093988349968890550e-03 2.428385119307124862e-02 -1.732096300414912374e-03 1.363439521746765901e-02 4.627213292889414790e-03 -4.125130686022178417e-02 1.726501050870205695e-02 8.557088266845015723e-03 1.852128438933130922e-02 6.251484388109826255e-03 -1.090345641733918213e-05 2.110879271182490059e-03 6.253652357032622733e-02 2.743858177287317873e-02 1.438884803869469120e-04 1.279700210681370964e-01 8.778091027743826352e-03 3.701984676084765086e-02 7.116260243879723828e-03 4.899254786620064334e-02 4.396570094208801516e-02 -4.974209360038226932e-03 -1.707422367481606362e-02 1.533156685720452328e-01 -7.433696928158138074e-03 8.914755859439078833e-03 -1.076342888800448615e-02 1.955028279979275155e-03 4.737498649177244309e-02 -2.952411966142669958e-02 -3.104040922825582888e-02 -5.187733818330312469e-02 2.920906962378078814e-02 -2.989232541219138169e-03 -1.351039335470130250e-03 5.165090111020644026e-02 5.763317389833267912e-01 3.457870896937436100e-01 4.389699688686673346e-02 3.166649902037121239e-02 1.553442982218335329e-02 6.838887021232153973e-02 6.293158871161469436e-02 1.724830747068032202e-02 3.730534365534267371e-02 2.704574625458992104e-02 4.491192802246694177e-02 1.222092554841423644e-02 2.437790105262467935e-02 1.797961092331789787e-02 4.824621347992496395e-02 5.740645660922902294e-02 8.718106644908381331e-02 1.834567505736739781e-02 2.539909155898802812e-02 2.945133753281870687e-02 4.481912712215167793e-02 1.903186706351193969e-02 5.045870782224493728e-02 3.929088274401660447e-02 2.585267610462255866e-02 2.522772080616995927e-02 8.712042188070569237e-03 6.106371354538754348e-01 4.888691680313381749e-01 4.942826143543734840e-01 1.627808195452316517e-02 4.132205493760629511e-03 9.242958143517424410e-03 3.300301087048151082e-03 1.150842993382894357e-02 3.946181360104569422e-03 4.944698911595917651e-03 5.657226159329835358e-03 1.623950721172377418e-02 2.844977504680846862e-03 2.122668056663057010e-02 6.471400843889214549e-01 7.204456640364339526e-01 6.085275124458067175e-01 1.609729299878186887e-02 4.385190164003151085e-03 8.021867370022271737e-02 1.157535773269896887e-02 1.362530073480223286e-02 2.793308005275991768e-01 2.495128713232555306e-01 3.131676656278264415e-01 4.170431492545222107e-03 -1.456865677162080472e-02 4.910941490890699535e-02 2.323666172183754333e-02 1.087353632913678897e-01 9.159092274342822693e-02 6.324617059778792072e-02 1.480599247934126159e-02
1.236004247012967433e-01 -6.102724852230084762e-02 4.010314402104961373e-02 1.635910317246275572e-02 2.191843225431803002e-02 5.842897309067579363e-03 3.600656879255082626e-02 1.210968034802092919e-02 2.882898486505020355e-02 2.750207961699653245e-02 7.052238248286327171e-02 1.632714149354126676e-02 1.666936528336501397e-02 2.002308944715482444e-02 4.931412357767965549e-02 3.642430807183122399e-02 5.725936579013339578e-02 1.483482175074266225e-02 2.547984194964174678e-02 -3.710857340616802863e-02 -3.812992927456122089e-02 -7.730089174866315582e-02 2.184394163545199871e-02 2.458463387923125187e-02 1.457542245945068690e-02 2.289742204645159665e-02 3.516985587824835585e-02 1.317571165217940564e-02 5.820689020548077330e-05 2.776214802416545474e-03 6.410510289210970616e-03 5.149069307703271758e-03 4.430253302715408503e-04 2.104872504911647495e-05 5.423416114966445566e-03 2.736890253056636479e-03 -3.893366677283536387e-02 6.131656259016698366e-04 1.012632280262017875e-02 1.172948381979684040e-02 1.110305665293004514e-02 2.057354734832481449e-02 -3.927227335471789385e-03 -2.923612116952988005e-03 -2.985433627621231906e-03 3.776562227701533297e-02 1.142315392790115153e-01 6.793377540518544255e-02 1.383669352657480867e-02 1.828874373132725753e-04 -2.384732840063239975e-03 6.514322748010525351e-16 1.788797963180216444e-02 2.378239899836102900e-02 8.831925384335835572e-02 1.735264446888281908e-01 8.249247365855351399e-02 7.964389569704526106e-02 9.470091767616746481e-02 1.334798810580894723e-01 -9.386506746730832740e-02 4.719275969103531136e-03 -3.738649420874201601e-02 1.889362836648564303e-02 1.261045438387574033e-02 -6.208655661792106782e-03 -2.104594862867481997e-04 6.550510595270371550e-02 7.603823571581155849e-02 1.835323761407270382e-02 -2.785330901974952811e-02 1.140522563910290228e-02 7.517146124799365878e-03 2.015899328249644296e-02 7.511789205898590505e-02 -2.626835077755575483e-06 -8.082850045981030546e-03 2.351279646028576684e-02 -2.972140814887179114e-02 5.932620484971733860e-02 4.436061815403862310e-03 -9.385847350951698198e-02 6.870943777797046403e-02 3.012264815362243262e-02 1.928435487414743507e-02 2.476056148801131576e-02 1.782641122765375730e-02 1.057229648286745274e-02 3.020071072202373114e-02 1.387838134239843639e-02 -1.215695548905518578e-02 5.226084832355767235e-02 -8.835107404690805853e-05 1.454042412569659217e-03 1.384084783303619285e-03 3.122948766852717828e-02 -1.696779461756054222e-03 1.522535412610630029e-07 6.667170678802534883e-16 -9.908923292479783484e-03 2.838884641201644150e-03 1.514391637368127375e-02 -2.438266168032970048e-03 -1.287053027055809007e-03 4.167695590455830490e-02 1.839470691791533496e-01 7.725298719666542913e-02 1.021959471228967020e-01 7.890442583638779261e-02 -1.181627825695064603e-02 -2.746649368219122866e-02 -4.231670838676727768e-03 2.468162616891412045e-02 1.277768929012851329e-03 -2.594431574557573214e-02 1.164975727527162511e-02 4.363872188659860540e-02 -4.234020798826906851e-02 -6.692935337990659882e-04 2.242761638402687718e-02 6.300333111135117958e-01 3.494162531277127393e-01 4.230509158157034350e-02 1.298452639372627178e-02 2.442062510424007279e-02 6.953506453638189089e-02 5.021300065402876239e-02 2.042546606817572438e-03 3.320393537772348136e-02 2.804201420380121254e-02 3.740330727260096633e-02 2.118244825010745166e-02 3.237975371178858364e-02 1.660349140337194843e-02 3.360680982505401260e-02 2.020853407297817927e-02 4.279959296528201146e-02 1.056502863617077292e-02 9.239264464905166951e-03 5.151822033971805714e-01 6.891419058251789798e-01 6.198119166886760789e-01 3.747374297866499365e-02 3.417971827034610444e-02 2.113849446923724007e-02 3.038533000642011997e-02 5.121115233155726365e-02 2.487095627188960073e-02 2.443888813412136915e-02 2.040122186623203552e-03 1.966442437403224941e-02 8.732658890017789416e-03 1.792472833039912214e-02 3.617760411093545302e-03 7.725596276174846386e-03 8.558539813291305386e-04 2.045588907638031667e-03 8.806869721154962291e-03 1.818183676231626897e-02 8.807788935113790643e-03 3.502005307901812081e-02 1.689505942287520060e-02 5.559853781980051979e-01 7.625388372335080156e-01 4.660946809239820965e-01 7.702927327627905052e-02 6.382721793511046682e-02 5.557657246165519094e-03 4.237822321694165867e-02 4.792654365109664805e-01 3.376534174183151715e-01 5.161067139304832407e-01 2.323807193832955892e-02 8.516576196170850058e-03 5.504584358502796759e-02 2.296359546357742976e-02 1.239727785588174286e-01 4.896554299245412661e-02 2.550731063205282159e-02 1.782737753484438204e-04
1.848953894604798420e-01 2.005493720385595133e-02 9.634276829603981643e-03 1.093661677994805723e-02 4.288214134686122760e-03 9.856249081754723737e-03 8.491373071895317801e-03 2.155937724836506919e-03 7.028172153580448771e-03 2.286978400002338385e-03 2.747765667488995950e-02 8.605209260159249127e-03 3.402457869376469823e-02 1.607497652757855500e-02 4.243093250892219365e-02 8.697963944031167002e-03 1.492802323146102082e-02 3.004024514386937142e-02 3.380756401299492980e-02 4.124360998977590065e-02 -1.128938128304206590e-01 -4.688915990222510682e-02 -8.458236114867638544e-02 7.672612127179473385e-02 1.986688471656470334e-01 6.283884955444229981e-02 8.248204900835312547e-02 7.032143461348734637e-02 5.778463751143495669e-02 6.312208654595467816e-02 1.378061535543306237e-01 -2.140744666177579400e-02 -6.298513048788661420e-04 4.076502058499187342e-03 2.993545657449655020e-02 1.964113084471627579e-02 4.906447242438666256e-02 3.619328654835637232e-02 9.131216688305551776e-02 2.840449069113533051e-02 -5.418113961887275482e-02 2.923141073797493918e-03 1.152375664935416030e-02 2.057756640985475630e-03 2.300440741645519774e-02 4.852817330662572565e-02 4.949627112140161239e-02 2.058907572635461196e-03 2.913224251849291097e-02 4.387177352375298128e-02 4.088654669090381610e-02 1.406218675263390915e-02 2.366475935915111906e-02 3.250191994698142794e-02 4.842735428907196660e-02 1.474816662638712278e-02 6.280354029673858307e-02 1.360294873292814538e-02 1.122236439439548289e-01 5.248892153483136758e-02 -1.651675576143083468e-01 -1.941858629984176941e-04 1.940848947997585823e-03 2.090245550727020951e-01 2.057719894667101035e-02 -3.696418694099780355e-02 3.237430670388340148e-02 3.203311652296070289e-02 8.978336285697438427e-05 -1.883296202974428784e-04 -2.724969606617432764e-03 -2.184557137086322060e-03 1.683487557672239460e-17 -1.847549354763012089e-02 1.423177378238640878e-02 7.002567531810576819e-03 1.675538678538037243e-02 6.068669293808578655e-03 6.244241168520181517e-02 9.880676577177149754e-18 -8.650882979592839500e-03 -5.360553752140258948e-02 -8.153441711348470566e-03 7.038554959480430862e-02 -4.789040704546422555e-03 -4.969151191001343858e-04 -1.915143562817749257e-05 6.047348996651125130e-02 4.000686995542275085e-12 1.185729040678758217e-04 1.064696334672219497e-02 8.815755435820052388e-02 1.801657119712057609e-01 1.657706303123462066e-01 5.748169141495739626e-03 3.622476754984401554e-02 -2.800549444415457827e-03 -4.717428625626939737e-02 1.112120182355673392e-02 -2.185144726308505675e-02 -4.112087710307747331e-17 -1.695574438396079894e-03 5.961598749658229834e-03 1.005099110798605977e-02 1.876647385789843164e-03 -9.184323241492695512e-04 -1.986075282196806888e-02 1.407592332629302792e-02 -1.295114270080006090e-03 2.595733656949576404e-02 2.240929309688142676e-02 -1.280914848243040575e-02 2.766669519388536693e-15 8.130776377290317714e-02 1.420976615598991757e-02 5.808856338419357904e-03 4.667295227057978446e-02 8.408792811254681143e-02 2.051267851804655928e-01 1.857502970939424458e-02 5.385995096316334996e-01 3.560876209938928505e-01 2.195603110453420506e-02 2.697047786279499135e-03 1.020888750746456498e-02 1.177572165887540330e-02 2.063413085769773281e-02 4.752071449622189379e-03 1.107182134015392665e-02 2.652954030859394369e-03 3.767513593859573434e-02 1.100861753249450861e-02 4.078993825708028542e-02 2.489865560807011727e-02 3.754709718166516358e-02 1.894498458471597854e-02 2.389410927556287825e-02 1.652212603961067394e-02 2.922061294878580337e-02 1.238217356669073059e-02 4.384208258198565855e-01 7.652997310209228576e-01 3.627173119454960570e-01 7.829975954991028400e-03 1.761901963882750097e-02 1.841275125383251381e-03 2.206482491521155262e-02 5.250098637279373287e-03 1.034413134850757643e-02 -2.295309671987633485e-02 4.120915610376535074e-03 5.441990968736922785e-01 4.242679356391169665e-01 4.145928797769951046e-01 1.639925823909434144e-02 8.411195586130468596e-03 1.922307802080821734e-02 8.393486179280039081e-03 5.141365822093129506e-02 2.869053043710873344e-02 4.122692987135999527e-03 2.767653980097071684e-02 3.903020142279678417e-02 8.785640957004845972e-03 2.782281457269024025e-02 4.748522691539584784e-02 5.779339093914155623e-02 2.746270219241159563e-02 2.413772489060479406e-02 4.386411698157490630e-02 3.462978654892980013e-02 7.999045707035726607e-03 2.973448762906691356e-02 4.340963550483556421e-02 6.239313952783805822e-02 3.107476050232428622e-02 9.681984898480645862e-02 4.048919041453867829e-01 5.549686193432988857e-01 8.963187391016457228e-01
1.173594928838802565e-01 -6.785673312733006557e-02 2.156516401531817853e-02 2.698691987008220924e-02 1.874000508484787580e-02 2.065868352372156672e-02 4.504719043868408296e-02 1.866278403313401113e-02 2.379227291296692748e-02 1.812692656678519670e-02 7.407615555213634129e-02 1.581894218920831471e-02 6.459543587889024670e-03 4.764579189639241152e-02 5.042459654955427917e-02 2.116133002945591590e-02 3.727095703531157350e-02 -1.127402710061884555e-02 9.019711914071507075e-02 7.936680441142309958e-03 9.616153798060307623e-02 1.601482067374199875e-02 2.695781924127429660e-02 3.405551128822115192e-02 2.230458458352325632e-02 3.776535113334911903e-02 9.700559326507965374e-02 3.575557062817912718e-02 1.074226550986238232e-02 3.176511000094848353e-02 4.168721450969094244e-02 6.722408777738601848e-02 -3.685856023572786351e-04 7.733174282503525558e-03 4.743646631383075180e-04 9.115677108403313217e-04 -3.874675712577094200e-02 1.167194828336607401e-04 2.250182612398936993e-03 4.645728499687661051e-03 2.771472598762007855e-02 1.906137989823550316e-02 2.729147208332154129e-02 2.894729716726883104e-03 2.402234261508742752e-02 3.259975180797045763e-02 5.690928511555466002e-02 1.641095114992315116e-03 2.080324812608960044e-02 3.130095388870424156e-02 5.013610878566314433e-02 2.526422509989658099e-03 1.118342874865009030e-02 2.128819587927680002e-02 3.268381020216178101e-02 2.429261392941961181e-02 -1.805492837864381694e-01 -1.481598167186400916e-01 -1.810420125114522716e-01 6.927778059956847534e-02 -8.971644305027078048e-02 5.055401535041749786e-03 -2.761668202667674765e-02 2.428785040661271891e-02 3.489252271007142747e-02 -3.046839171567007273e-02 -4.533931607594387614e-04 1.443964644414826894e-02 3.052821015891765688e-02 4.331163553203487560e-02 -3.388654729352798267e-03 1.551035228784438183e-02 6.768166762821859438e-03 2.551228065901079561e-02 3.790501640784352602e-03 2.486930696710106664e-02 -4.331417201589389879e-03 8.545822354604938609e-02 8.653331530249329384e-02 1.516054334916481505e-01 2.274021430461185128e-02 -1.232279855507792225e-02 1.120073939901429089e-01 -2.457120975693819712e-03 4.119521195612683290e-03 -5.234708048385086632e-02 -1.176738259805027581e-02 3.164774963543020431e-02 9.916756429261806838e-02 -1.141552971977428553e-03 7.499624774701590946e-02 5.084529508962397359e-02 5.334480117605129579e-02 1.335796537523121705e-02 3.238310990762707786e-02 -2.909420904892106793e-02 1.171995486747251514e-02 -1.136304572223572479e-02 8.113554873903808945e-03 -3.449062387512450865e-04 5.558066130199565066e-03 -7.189831647481703496e-03 3.235148156067385768e-03 1.182005780716264415e-01 5.074866284466612735e-04 1.747021405521631760e-03 2.618657730648932952e-02 -7.508669337819438507e-12 -4.680197339824523212e-03 2.463249288556360744e-02 4.138031656113463279e-04 1.047708183261205836e-02 3.415827882146386073e-03 -9.792983758587526025e-03 4.715727663437737326e-03 1.332112814459118257e-02 -5.925624538961583865e-02 -3.289220965208174835e-02 -2.494165776150845115e-01 2.149919148603330332e-01 6.119947495808317761e-01 3.706371259995001610e-01 6.223692672175117069e-02 2.380930784313371740e-02 2.255991197370778995e-02 1.048506463649344772e-01 6.010993231392125491e-02 1.326847156045154498e-02 1.971444448150633938e-02 2.696327883999447353e-02 4.944542591442006862e-02 2.001298302958772271e-02 2.550942137025826936e-02 1.634854724138246757e-02 3.217590368025492059e-02 1.361883830632231764e-02 3.880413463144674613e-02 5.234968276114477792e-01 5.116311038880125839e-01 4.790301983866079860e-01 2.033749868136932107e-02 4.262142485206749792e-02 5.361490437511434032e-02 5.066289844966472694e-02 3.041335592404554103e-02 3.274459375907552011e-02 1.817718144507607728e-02 5.219233032002195804e-04 2.111792073238053999e-02 3.324327698917602797e-03 4.544140105763535553e-01 9.890833454316890760e-01 5.546581298973257690e-01 6.840511641444690406e-04 1.755724216962343057e-02 7.037904838423302348e-03 4.838683220537320917e-03 1.100393669358424621e-02 2.262706982527047661e-02 2.300948789287587948e-02 3.819905227580860235e-02 2.485080097905743163e-02 4.702230831992945670e-02 1.355953691456761832e-02 3.405745720798405290e-02 4.573850808319579464e-02 5.521311472903837891e-02 1.697532446310619572e-02 1.998943972731920260e-02 2.080231947157257580e-02 3.916324185927713003e-02 -6.002900954605543517e-04 2.131231891632448180e-02 2.800276500755344983e-02 5.870612122459573090e-02 1.055571638489699853e-03 1.298168687022208112e-01 3.519377074456863652e-01 3.087117198972993970e-01 7.959240459869862949e-01
1.206996043850771944e-01 -6.423151917757258245e-02 5.516543826493441188e-02 3.181965877931974135e-02 1.277454650926511913e-02 2.800137029211060419e-02 5.340043022216486857e-02 1.441501726004349414e-02 5.027680016371158939e-02 1.720654015296389783e-02 6.487131992740832742e-02 1.631721987578992755e-02 1.904508717746934351e-02 2.287134221891986077e-02 3.154865390007714071e-02 3.392661779853058113e-02 4.678865135176338286e-02 1.358938043887281469e-02 1.426948117835094738e-02 1.533149181185058767e-02 1.211857986961428213e-01 5.545443374681837539e-02 6.601952983850384316e-02 2.376541718067199521e-02 2.584675393854554692e-02 3.865796888068049469e-02 7.251171055434628421e-02 -1.282499376734474728e-01 -5.941957978585715033e-02 -6.829167451207751594e-02 8.665108171094695194e-03 1.368549357245471641e-03 2.291012836979750087e-03 1.334624875689200806e-03 8.982153239989231278e-03 3.766175286944174229e-03 1.688762507310503431e-05 2.571429328396541807e-03 9.104869855274939222e-03 1.596833101747367756e-02 2.710123703207558202e-02 1.679820716564685637e-02 -3.697013867289471439e-03 -9.783185958941675864e-03 -5.085405773033986956e-03 8.666948027933375764e-02 4.988114267469194452e-02 1.767775795417752746e-02 2.674254823868476086e-02 7.801629433157167282e-02 5.794889071992232604e-02 4.490741395955608756e-02 3.002270638598358507e-02 3.519757965703653596e-02 5.823230411161151032e-02 2.029378182361836480e-02 6.289812535049875419e-02 1.544037118160594170e-02 8.775384130373174718e-02 5.129188789307533719e-02 -9.806890096365131204e-02 2.235891762804002869e-02 -7.136781777297406155e-03 6.916931285512701449e-03 2.033992302146456904e-02 3.892122771411632518e-03 -2.026248066196353014e-15 -1.459498874821816596e-02 6.769957243726637541e-02 -3.965712303170977035e-18 -9.366084998677887130e-03 3.269197854576180728e-02 9.512277169985513051e-03 3.226276358659734694e-02 3.777711459735326200e-03 2.499200014466912770e-02 -2.465765660795378456e-02 -2.351095345804804591e-03 -1.408410894439991387e-02 1.983578056544843624e-03 -1.646302478523089291e-05 8.933002587439177813e-03 3.525338103540708035e-02 7.663685044678665874e-04 4.241949396172062237e-04 -1.221672554580708915e-16 -8.721280193820265050e-03 9.401768599809512406e-03 3.726328986730306943e-03 -5.627837452868687113e-02 7.448291482712358046e-03 1.729193504587575314e-01 3.255964244140329561e-03 9.365037584973036691e-03 3.956626483600896027e-02 4.465375038164588750e-02 2.771388145254281496e-02 1.440484269904411876e-02 3.652024759149352564e-03 -1.436959659064891009e-03 2.731985630189362121e-02 4.627455457045473453e-02 -6.817023353457130064e-03 -1.564399414988887349e-02 4.595503531280568238e-02 2.340331575050091928e-02 -6.811569190434726539e-04 -3.780354714916104447e-02 -7.398357539518390924e-02 5.894153580494868386e-03 1.801572910342243844e-02 -8.826455424823947266e-03 1.430775686308840679e-16 4.435729154769609689e-02 7.546773552431443148e-03 9.066601365025194620e-04 8.668442463416395349e-02 1.431336651632625911e-01 2.338775855546002536e-01 1.139911075115976682e-02 6.193271007458538424e-01 4.068392055992423595e-01 4.718788139997397851e-02 2.453575189868155079e-02 2.115214142989009505e-02 4.601764842111150805e-02 6.089540298345041264e-02 8.939966018052638089e-03 2.902469895070851694e-02 3.050364143115049179e-02 4.177129362193715556e-02 1.680698634755022763e-02 2.814527121066529025e-02 7.044188028317363050e-03 2.596467116897604316e-02 4.234247812851677645e-02 4.435222867101501487e-02 3.933039234805154639e-03 2.086259620366899895e-02 2.847397207200038499e-02 5.234231432476309231e-02 2.550233529277157185e-02 4.807506890848772113e-02 3.763787670944598179e-02 1.879945181251749600e-02 2.220403153384280237e-02 1.526374655471605252e-02 5.894598754957572906e-01 5.198071941800314066e-01 5.417230590861596973e-01 1.779068791141674363e-02 2.151236144749115939e-02 2.337997305090988229e-02 2.100450885512459079e-02 2.566154970048755563e-02 3.360632494588366471e-03 4.949496748531847894e-03 1.643622405070253173e-02 1.917996052372073629e-02 1.251192134103168777e-02 3.650708347174381335e-02 2.611153902852841706e-03 5.535460026648836296e-01 8.216594178109913882e-01 4.835643641925954639e-01 1.990474557367029329e-02 3.445527819982051321e-02 4.436169571679592777e-03 1.895193266740177046e-02 2.129690933576964054e-02 4.102505819617069394e-02 9.330131186323925227e-03 2.307664277739469860e-02 6.821833167300964118e-02 7.571305724924783098e-02 4.729871618401500916e-02 1.199106793319645731e-01 4.069337329872071418e-01 5.458864400316777443e-01 9.063058925708038505e-01
7.873906642432438804e-03 6.694270738541153731e-01 -2.826702260855933069e-02 -2.901440046127707825e-02 -5.887865355027463443e-02 2.566042645438677480e-03 7.111044371016787130e-03 6.954740690396302681e-03 6.381161718724752234e-04 1.634177941087097338e-02 2.930405326187314474e-02 1.501546440181321558e-02 2.908697760581550884e-02 3.465438962852834121e-02 5.971059463744313178e-02 1.217134948759465707e-02 8.131951583556560822e-03 2.955248912445278611e-02 2.813385306018457294e-02 9.122273839694167163e-02 1.755374935870736633e-02 4.641584292289747254e-02 5.501312666783182009e-03 1.858030171572382441e-02 4.653787578509870071e-02 1.542231084116839693e-02 3.087687735995012575e-02 5.423657991056107103e-02 4.741766391012554316e-02 4.329216130739296786e-02 7.117944166206380263e-02 4.675175484956338445e-02 6.787023112151290005e-02 6.143079766034312272e-02 4.638354776170256627e-02 2.161051112979975319e-02 2.883384277388743606e-03 6.544442522901862402e-02 -3.926237221833235591e-02 2.027523629810606704e-03 -2.719986392900436423e-02 7.217161971975826797e-03 2.417417558802248054e-02 1.077318872119498519e-02 3.463968150291493048e-02 6.700906573936883659e-02 2.938197612127925462e-02 1.677227095419404121e-17 6.231580530480010198e-03 1.209964578414304999e-02 -1.345231166441158109e-02 -5.861088181269442227e-02 -1.678700606100077403e-02 6.969449514697700909e-02 1.426256520737138600e-01 2.399017102906755849e-01 1.011229629623491694e-01 3.220647743313462263e-01 2.690007148381863833e-01 1.660589925581222592e-01 -2.298622039210784090e-03 -3.680395920640269075e-02 6.254907623519123661e-02 -1.194917345142185727e-01 1.831207505851125516e-18 4.031001865314554067e-02 -4.447298835719189908e-02 -6.837983329228266779e-03 2.167260794733634217e-03 2.048298879995005470e-02 2.138335655625017012e-02 -1.921229587880087311e-02 7.380288811556405017e-03 4.586594831828019053e-03 1.982028166095169130e-02 6.569765262446243537e-03 1.296147050087826345e-03 -1.080997370412097383e-02 2.250852696202477674e-02 -2.965348056090225158e-04 7.518213920403402951e-03 -7.501737408461260116e-04 2.495720302620928199e-02 4.055372070761241823e-04 7.291939064238649182e-02 4.990788466189488570e-02 2.197250849814019052e-02 5.559799424533506257e-03 7.506305087264668998e-18 -1.697930666061863575e-02 3.890305052741106773e-02 5.897776396408433179e-04 -3.129907671213942415e-03 9.453866705133916540e-03 9.872338450705803181e-02 5.891940941873430619e-04 2.079960914506390389e-18 6.565121707261430145e-18 6.006387283527913484e-02 7.915600448914627977e-04 -3.332660082048468676e-17 -1.092300137618771975e-02 1.454876344283338556e-02 1.832508511255418805e-04 1.094923323279769697e-02 -1.577599984722937582e-02 2.636392919567818668e-02 -2.040718967616862722e-02 2.282550586507473946e-05 -4.117345052074153355e-02 -1.249239249535371611e-02 -4.481577755266519247e-02 -2.208745188928172825e-03 2.139010626925465317e-04 5.198861996933696987e-04 1.214837558834780829e-03 -2.286985703851324853e-02 1.793738973403630199e-02 2.792364183273001940e-02 1.196544736539934716e-04 -6.204453353723187181e-01 -4.490484456269091540e-01 3.672762240819242296e-01 7.996635220249409537e-01 4.507900591244551891e-01 1.079934995512749248e-02 2.562148195422316380e-02 1.399140444109628692e-02 1.423456044178601776e-02 8.041959972562739978e-03 3.058256524047125308e-02 6.592907866593725880e-03 3.260461756276555878e-02 1.428931520478751542e-02 3.523265387457668457e-02 1.960121370395475141e-02 2.288197539275229481e-02 9.373630548732910656e-03 1.200979454472313190e-02 6.605809433287403998e-01 5.625433191850617298e-01 5.648285500142801174e-01 1.773487000181566547e-02 2.165940694783754020e-02 2.967444655327607095e-02 1.758715358126660203e-02 4.716938929202033753e-02 2.732307756374569366e-02 3.070303347715840417e-02 4.337575451244520680e-02 6.691013284853283838e-02 7.539639214924433175e-02 7.665190358687627714e-02 7.694195802592011801e-02 3.609036375068883085e-02 9.560859106872752295e-04 6.939029512241028444e-03 5.883121937583303263e-01 7.677150836103552312e-01 7.880764826464492501e-01 1.153957236222296137e-02 1.692116007607474479e-02 3.431843299140165116e-02 2.804067021862719465e-03 3.193723767461228991e-02 4.940350469173520470e-02 5.345193166513862587e-02 1.667748278599236883e-02 2.597099660663269513e-02 1.069949743250113612e-02 3.843164007267048099e-01 6.579744880040139243e-01 4.367962158764280844e-01 1.764117855031166471e-03 3.913097253145345761e-02 4.831021065757000466e-02 9.557506380796795820e-02 8.064174128530217367e-02 2.747198744137590090e-02 3.781219015802360361e-02
1.346410810445143325e-01 -5.366452981822719681e-02 5.046454265586872850e-02 2.550319201877691844e-02 2.914900542038215830e-02 3.922973255049468783e-02 6.188601026920167308e-02 2.060602655227499955e-02 4.175193839656943934e-02 2.088498824283521738e-02 4.341264868264468196e-02 1.677919663033020092e-02 1.415055168139254214e-02 2.887473569886546756e-02 3.168292919988468376e-02 4.037328258949626769e-02 6.109924203622205813e-02 5.276099187261839955e-03 3.418588080183350593e-03 2.351649576459771684e-02 1.705889577097634280e-01 7.068913686371962324e-02 5.474051444677024980e-02 3.161379385938342668e-02 3.481945752739636774e-02 2.184155013450307312e-02 7.869096599833171057e-02 -1.603326745380461715e-01 -4.873460300375166004e-02 -1.441392673450858086e-01 1.157454118399285203e-03 1.211885179934243592e-02 3.654404030016315753e-05 4.004431709887274642e-04 3.441090278819278810e-03 3.538635406811862019e-03 -1.146473827116670794e-03 -6.629535850746032201e-04 -1.445085912468093044e-02 9.675198911612534897e-03 3.011696122821604094e-02 2.048910126440752974e-02 1.838184299423062038e-02 7.105337402216528077e-03 2.334375671223018131e-02 1.915774797898705389e-02 3.435868792712881703e-02 4.244057324621592492e-03 4.555011105980535935e-02 8.159404113925880497e-02 -5.230648832672879500e-03 4.371854111790973024e-02 4.192871313838548303e-02 1.179914448008283628e-01 1.174217776872979319e-01 1.059306824312627598e-01 1.153841014408455296e-01 2.588156584376231084e-01 2.197867462170002562e-01 1.213673443722540074e-01 -1.022092103881721697e-01 2.612452036279579864e-02 -1.340508967819373752e-02 -1.740817722529745424e-03 1.475175322927554586e-02 8.526726763125089259e-03 -3.441536772021882976e-04 9.640932580943440550e-03 2.181172520638638423e-02 4.136847758407783543e-03 -2.338007388486881485e-02 6.877978347059486922e-03 2.055062982880278763e-02 1.385730006615714174e-02 4.220454276052664508e-05 7.665290745865338193e-03 -3.558865331437508547e-03 -1.502676099222205811e-02 -6.617322401676602704e-03 -8.852801480449829646e-03 -5.642147236042491134e-05 -6.658777145192949082e-04 4.551135518083714027e-02 7.562571375824448522e-03 6.958408358749362354e-18 4.517081501355554168e-02 -9.569814255622578064e-03 6.836431370948171937e-02 6.272306518962556150e-03 -6.165876654654003919e-02 -7.065812582417375920e-03 4.247828556198566391e-02 1.992930969523318274e-02 7.967549420496534973e-02 -7.029730827312902089e-04 -6.464432048768917714e-03 1.606744474190254131e-03 6.528114377976557137e-02 1.288377587608004193e-02 -9.596875326663502391e-04 1.240702457835408598e-02 4.869175968290846701e-03 -1.304336355234533796e-17 -2.727156530648408139e-03 -8.000994999576566472e-03 -3.268005172242837072e-17 -7.489638598686603616e-03 -4.943774860264460317e-04 2.883565590054187541e-02 -1.857538301939426137e-02 7.147972718028719648e-04 2.388555033179865211e-03 1.262854410341115140e-02 -1.794079626055603699e-02 1.960439191766542855e-02 1.756708301773945297e-03 1.291973485206651673e-02 1.377053098282362761e-03 -2.417547961269262205e-02 -4.643332371649437802e-03 6.515887685193071377e-01 4.694384974840418723e-01 3.947629744411125519e-02 1.909883445528733417e-02 2.578509091438776699e-02 5.726366811549048891e-02 7.091132262797150698e-02 1.365505382705313291e-02 2.540762995988414194e-02 5.643226249437965393e-03 3.296167735907367530e-02 1.358829006347436578e-02 2.355904708107185025e-02 2.063771387846469185e-02 3.938453680748942726e-02 3.270809141042768653e-02 4.264651472213622230e-02 5.311278344213252742e-05 1.422429370766896579e-02 3.047925681905729392e-02 6.018010203598087721e-02 3.573253425587707977e-02 5.159224163181048289e-02 3.845682579398516932e-02 2.566501891228397877e-02 1.053308914317795629e-02 1.904158469302174791e-02 5.233522298621468005e-01 5.311956873918636246e-01 4.950267443396130784e-01 2.037510847869013902e-02 2.134115149545307358e-02 1.106772325469062776e-02 1.005560348283441978e-02 3.145092008101993408e-02 9.184894697566789792e-03 5.985034257978144590e-01 9.789960155491130855e-01 6.875009848450104322e-01 7.729547763153110651e-02 4.582184139251069266e-02 4.339203941132878384e-02 3.340423991887164767e-02 1.041214841020979146e-03 3.081268506770760279e-02 2.911451258604494385e-02 5.175004565452254873e-02 1.113749952440304590e-16 3.256855525706904286e-01 6.522832050168136941e-01 3.419732389334361744e-01 -7.232870506656677301e-02 7.422221869972886812e-03 1.533992548884156176e-02 6.258938591073015534e-02 5.983236649800836165e-02 1.601935837829541465e-01 1.751584498250926547e-01 7.173938616747080332e-02 2.127696123400661665e-02
1.286448528055103846e-01 -6.604382411788156659e-02 7.727081483381152693e-02 3.049199475395981268e-02 3.460401244310182950e-02 2.865500415726838709e-02 2.889940812865509148e-02 6.926534829994330025e-03 2.833145795394621455e-02 5.735431035703381086e-02 4.260807581248616388e-02 1.673707079447536716e-02 1.905620814895053056e-02 2.544195922177548350e-02 2.865120713507457798e-02 5.312142037336517464e-02 7.378207688811933007e-02 1.717441887071512879e-02 1.418690201836753616e-02 3.157496879163010362e-02 4.028853314153885840e-02 2.305792206505941425e-02 5.419295766422041472e-02 5.377563023185173707e-02 -4.625973750064113488e-03 3.662622608368266974e-02 9.321982207038835566e-02 9.094180698080909370e-03 2.096504996080706881e-02 1.433687728167594693e-03 1.700634859939887441e-03 5.726132733566445532e-03 8.023024258317922085e-04 7.991930229958393861e-03 2.599629902971990758e-02 -4.284651411322940029e-01 -1.541178616044252270e-01 -7.654209473737529079e-02 4.739853090478843983e-02 4.425359203934537550e-02 2.500011595563557398e-02 2.288162816001178268e-02 2.702474838145210029e-02 1.611347311846333258e-02 3.662787249391642047e-02 9.290524040357608926e-02 3.304532836130433943e-02 2.046098156895121215e-02 4.894623061490904065e-02 5.880456186774291050e-02 -1.158281238991505299e-02 9.600702263631179234e-02 7.128605532021008162e-02 1.285480386969206912e-01 1.003789979248005532e-01 1.020407197550379091e-01 8.393792672902110341e-02 2.257581256156644955e-01 6.991294237833976799e-02 1.394973079000509375e-01 -8.890816835875346713e-02 1.489464645064908326e-02 -2.888715008287135247e-02 6.595867791994841070e-04 -2.171035674798784103e-03 5.197176083424362436e-03 -8.765541674095038318e-06 -2.593880194497779137e-02 2.836491542380940867e-02 9.266239711624676620e-03 -1.632230744719603138e-02 2.070266751635467528e-02 1.831851751734963227e-05 2.629993168706839949e-18 1.638910056874894744e-02 2.680489601303809311e-02 2.232435355688786732e-02 -6.818628674709730766e-04 -3.052284372036764837e-02 -2.065863213701911796e-02 -2.038238876702667002e-03 2.940301465968143957e-02 2.061756026142382708e-02 4.105985222823355790e-02 3.131082299571650435e-02 -3.281570793066880434e-04 1.846162024707036567e-02 5.663095987096899812e-03 -4.704121055835121146e-18 2.272027357667176853e-02 -2.028339882505743384e-04 -2.395989610021129375e-03 1.326236178436358479e-02 3.885683178804566201e-18 5.152905407182153605e-03 -1.371936501021379408e-02 1.179320932831843648e-03 -4.467088985464127103e-03 2.898996406040908295e-06 -2.610946803740332809e-03 7.215000070905120159e-04 -6.397064091838793201e-18 -8.437153113464202631e-03 -6.579909286128035606e-02 1.506999479859253734e-02 -9.622924779749715898e-03 -1.005451187475198301e-02 -1.052628686440171962e-03 5.956653387985819809e-02 1.747423313085293833e-02 9.604866027577013687e-03 2.220966653399016055e-02 2.251789547631892138e-02 -2.869330119222976080e-02 2.029661142942280200e-02 -2.173257268705203887e-02 1.656026013051858231e-02 -1.668677924019406830e-02 -2.768395965377332086e-14 -3.012648779192409962e-02 6.253751419091482244e-01 4.014702998283068425e-01 4.522745612757750333e-02 3.159227016647955105e-02 2.031479368532211963e-02 4.124670815691915127e-02 6.801281761483836397e-02 2.581048012139867606e-02 2.646978298375603222e-02 1.172955521328668954e-02 3.600571889737485404e-02 1.050855262902288673e-02 1.463467447845636336e-02 1.379539389735253210e-02 3.488206767368692146e-02 2.098874977448691656e-02 3.376290425918464233e-02 1.316175997639060583e-02 1.277798000143567404e-02 2.606844544757619397e-02 2.412625068273718809e-02 9.621108054520955202e-03 5.346294430123084140e-01 1.013079974240429948e+00 6.489965260462882135e-01 3.075613089069483930e-04 1.809174333052814693e-02 3.617999071989346123e-03 2.332328057238985605e-02 1.037820116222199461e-02 3.262126462006693234e-02 4.189460592819518187e-03 1.216090015882571873e-02 2.232966764559636379e-03 3.001012061332425027e-02 9.350707881853862480e-01 7.884806256538721136e-01 7.504659564757697465e-01 2.604804031087001176e-02 5.411434103785400102e-03 4.055808992538877111e-02 2.296705520287757943e-02 4.460078667964691851e-02 3.935238904293429825e-03 3.104870624915094524e-02 2.290364436446416449e-02 5.311141596605700732e-02 7.362648376332990518e-17 3.311704959883838839e-01 7.447499563522577137e-01 4.066672649489913027e-01 -4.250650408197810537e-02 1.008327182220494778e-02 1.771176552770353449e-02 6.168621641278813211e-02 2.644062861088574551e-02 1.420477910279013822e-01 1.475910018243359600e-01 7.831870047508787680e-02 3.017780218930274025e-02
1.728354810274471687e-01 2.288999296555528987e-02 5.527043620294252091e-04 3.299288636194361996e-03 2.386300768369925085e-04 4.682354514480391999e-03 2.646430007032109233e-03 4.448952049973801080e-03 9.194332340980429838e-06 2.942106453330429507e-03 1.762761340751030706e-02 1.351163667181303450e-02 1.700652518936160776e-02 4.294251255616456792e-02 7.316613359113242898e-02 1.963367109049719145e-02 1.552401753461783891e-02 1.052557135442510651e-02 2.522022701250925883e-02 3.659437990964797827e-02 -9.967163524616404202e-02 -6.072713446895066508e-02 -1.053264128436306318e-01 8.029233332435026238e-02 8.100943294820928120e-02 3.673684527960414253e-02 3.882651148774116795e-02 5.519781997260498480e-02 7.739768548069317355e-02 9.174310085846219820e-02 1.199590768102270499e-01 8.531078483680974711e-02 9.473369655944549605e-02 6.063763791216388727e-02 5.639049014023640727e-02 9.075259257840478933e-02 1.001018957248270280e-01 6.615280542500046046e-02 5.911485824322468274e-02 2.923375586581431623e-03 -3.097007006208930477e-02 2.339096742304146853e-02 3.136380633654757555e-02 1.943805449253664663e-02 4.103067817755626928e-02 5.421649571586017319e-02 9.288167473695141396e-02 3.641605589628544443e-02 2.802554281704995945e-02 2.408815994165239976e-02 4.506948514208392043e-02 5.035402091092307762e-03 -3.036985030880921116e-02 -1.667013819236916050e-01 -2.042821009184342396e-01 1.921250379721991808e-01 1.140043126800991402e-01 3.008322858427850965e-01 3.340175592938235405e-01 3.200047098505363063e-01 -1.671157666767062178e-01 -5.799607389705404908e-04 9.078831070896506428e-03 3.225604180175320468e-02 3.145487934206338959e-02 2.727418400290785336e-04 1.155203439230339524e-01 -3.323895509847071920e-05 4.153419558065832080e-18 2.272527023300135660e-02 6.420203938059141216e-18 3.434258902120423951e-02 3.561343451330212240e-02 -2.704881353825777657e-02 6.626546523278268458e-03 -9.020868743643793708e-04 5.654327388255071113e-03 8.432286525934532678e-03 2.268646278802955579e-02 7.269602692757528434e-03 -2.943143237482814464e-02 -7.393456642691476066e-02 -7.861462800723700850e-02 4.824972338858211224e-02 4.826327436469531051e-02 2.214491091084727289e-02 -4.392276151646206522e-03 -6.310559313409382597e-18 3.272011087941617469e-17 -2.203452555832585355e-02 2.402882092547281578e-02 -2.121602712193784022e-06 -2.804997849250214902e-04 -2.957425242031408094e-02 1.240809246350457751e-01 4.932034058999474474e-02 4.245232451086889541e-02 2.409474138445604624e-02 3.980461057615455878e-02 -2.624624552321587825e-02 -1.356250132892331071e-03 -1.229106894311153553e-03 -1.658076343217477632e-04 7.768972116143065508e-03 -4.962367003474326375e-03 -8.930619759410996427e-04 1.299799246286838758e-17 -1.027791023824460321e-03 -2.466565034916264054e-02 2.188154141761387771e-17 -7.491369371830055539e-17 1.526641858059473236e-02 -1.244275645606056184e-02 -1.449653956670173514e-02 -2.250284624090183316e-02 5.829114503498789013e-02 8.312734798596010943e-02 7.287690820203064634e-03 5.529952998434121969e-02 2.650764006545629561e-02 4.276214137968227247e-01 3.185519959156322689e-01 2.036075311086351508e-02 1.678208225647209294e-03 1.089885620645646667e-02 1.503815351028548819e-03 1.954638663531995529e-02 1.056901975263017414e-02 1.653928302148573865e-02 3.641273489348214286e-03 3.033137579287235527e-02 1.752886650305083768e-02 2.898793265681667147e-02 2.680359158435761252e-02 3.658169316241146468e-02 3.629795977529029841e-03 2.838283359648612059e-02 1.236861598304734439e-02 2.875151264616103083e-02 1.400110625453397303e-02 4.188250477343659628e-01 7.501820411047799286e-01 3.918302805323609084e-01 2.947552470422865403e-02 1.566521617881744613e-02 6.659832254028091175e-03 6.013694950600999289e-02 5.547576462888079152e-02 5.047001208666765382e-02 4.793049167150940926e-02 9.166451242433602820e-02 6.992287804110298988e-02 7.397471852139393678e-02 1.001931826539221304e-01 1.809607212369929857e-02 4.107027407579985634e-01 5.380754769279573191e-01 5.447625052374037713e-01 9.858581203718698102e-03 -2.739226216355308222e-03 3.180842290202679726e-03 3.171249922765120161e-02 4.361048629733020598e-02 4.902100410402155349e-04 1.989710122570834128e-02 2.834049304277790060e-02 4.246479471456015425e-02 2.446978303002718402e-02 2.471836443553772009e-02 1.472507107835544643e-02 2.498193161718190169e-02 4.479273956103932648e-07 2.945780688902950306e-01 3.317956029275168639e-01 2.479254357965354483e-01 1.645301542945447337e-02 9.099320182758413589e-02 6.004766581682113163e-02 4.501383947615543446e-02 2.062749535888302316e-02
2.348266129297571225e-01 1.280278148028717380e-02 5.847810528742646832e-04 2.842824452156723257e-05 5.792195000851943498e-04 1.612914401187705467e-03 2.168517790845077373e-03 1.596868511575943918e-03 2.100884422174297168e-03 4.576383250101007452e-03 4.637334500750950154e-03 7.815013864824676276e-03 2.017588343152388816e-02 2.995254335461916551e-02 6.467480402371378334e-02 2.100046622480062491e-02 9.373710613990698937e-03 2.488569786428626585e-02 3.104879887615735745e-02 2.830087186729985521e-02 4.881946877033001286e-02 6.573681627558496977e-03 2.590610470554189498e-03 3.741564821738550817e-02 5.873564818384095082e-02 2.346630702344783270e-02 4.010246690335842606e-02 5.001283262722490408e-02 -5.271663264643231591e-04 4.620258832567254031e-02 1.728864291803277886e-01 1.315157609544875261e-01 1.596723102027482744e-01 1.042911415111857365e-01 1.269933602178704990e-01 4.928083607570814956e-02 1.406144514146142471e-02 7.511935350102584563e-02 -3.846439736298445933e-02 -2.759241054759978273e-02 5.621898145448859076e-04 1.400009907905698861e-02 2.259811453721932706e-02 3.029814495854836129e-02 2.926517391100627238e-02 7.885631728003739127e-02 6.013580441337804117e-02 1.158299402936444587e-02 1.187965188669151337e-02 2.541766438050279200e-02 -5.065364739362896579e-02 3.539395635195851080e-03 -2.136234137903233327e-02 5.642075163892851242e-02 1.241127772391328460e-01 1.629747546336243191e-01 1.041460181111183575e-01 5.874403666359048204e-01 3.773362429010969077e-01 1.306013920320211463e-01 -1.634021142911110114e-01 -1.939462776192302793e-03 -2.482639368574415459e-02 5.516448039639209461e-02 -1.704535949537987812e-03 -4.889301350472441474e-07 -1.139844631918382825e-02 2.056168155133447292e-02 3.320233652410088665e-02 2.187431443454457627e-02 -2.926469339496968417e-03 -1.759205140429339828e-02 2.208962192487783777e-02 1.647016232076770059e-02 -1.672204426599053443e-03 1.645879033231782348e-02 2.319662353311064545e-03 2.474699426614128395e-02 1.503249232351541904e-02 8.298297045192265129e-04 -9.065751279524184712e-07 -9.962530454922747305e-17 -5.115508176145587213e-03 -1.879538440537595377e-03 -7.933207659232254785e-04 1.234280128888114343e-02 6.952008979518353553e-03 1.347859227931633326e-02 2.969245595900768097e-03 -4.949135993930284250e-02 6.528137653358072334e-02 -1.720118446229983861e-02 -8.891423174913599909e-18 6.393534666358233545e-04 2.694070145003747349e-02 9.264701943856859176e-03 6.515097433079158548e-04 -2.637933639307306344e-02 3.344817079685647593e-02 2.063081614451272444e-03 6.261800520899455822e-04 -1.845157956497332774e-02 1.065139542113101474e-02 -1.729555902972731518e-18 8.516340295052259382e-03 -2.603674024527287412e-02 2.088994938703576595e-02 -2.981016776633740525e-02 -3.143723316883813203e-03 -3.303357307623593087e-02 1.977469895136465053e-02 -1.075020165392615024e-02 -7.480703960105933130e-03 -5.110366672810272179e-06 3.754939047392305464e-04 2.216909381364551030e-04 2.451174895686959038e-03 1.142996772941257295e-02 5.410410275339697095e-02 -1.752479736186659523e-03 5.627336983481231458e-01 3.351425016064126794e-01 2.530745458818202745e-02 6.065204803567726083e-06 9.038397924214331899e-03 9.525077860832901339e-03 2.042256076819810862e-02 8.526274167314896665e-03 1.304643807241640842e-02 2.353443273266943130e-03 2.966635528565790547e-02 2.041572061064711105e-02 3.568595022472739742e-02 8.512693729306726118e-03 2.546177919444671603e-02 1.590864755771213834e-02 3.656426126523815157e-02 1.322857589260143310e-02 3.559159665089643709e-02 2.308348687324131635e-02 4.611183169299335610e-02 7.264718386079039268e-03 1.955370851914663144e-02 2.360531685975559429e-02 1.424696379186909598e-02 3.323781052265820622e-03 5.190661037011490286e-01 9.013636804517742851e-01 5.678117229331349769e-01 1.396487757954012997e-02 1.609167423561716145e-02 5.712328476742368349e-02 5.326018901666283029e-02 9.903984632643254848e-02 2.356648037139000285e-02 -2.154721065856156359e-02 1.219708725840933268e-02 6.485745195026910714e-01 8.604524310648643537e-01 7.620676105608406026e-01 9.156369753598927325e-03 8.750599492350826306e-03 4.315081875768028014e-02 5.186100252957578406e-03 4.314081530695975525e-02 5.529057605240911971e-02 7.381770546961186064e-02 2.422502510288352187e-02 1.966370741549231665e-02 2.701170515656951127e-02 4.072233173704336018e-01 7.513537796406091207e-01 5.173256844898636020e-01 4.342702391972265777e-03 4.333723945084905910e-02 3.574941825353725178e-02 1.145796377876460853e-01 7.433016050120563878e-02 3.953193552203889544e-02 2.190040587886250487e-02
1.324557219167488420e-01 -5.759469443022544044e-02 5.503189391294321248e-02 2.968604284591455297e-02 2.046566273167426883e-02 5.341914013528948485e-02 4.491974533956158422e-02 1.703960790255293518e-04 2.010980193732554364e-02 2.330364530246597371e-02 4.245335505700718187e-02 3.077124588401421168e-02 1.343047927571059535e-02 3.144023504659452378e-02 4.513034971980466048e-02 4.105277574349822794e-02 8.516316818416723444e-02 1.077332690431428812e-02 5.077651815663703112e-03 2.290843179999536502e-02 9.229774174540483767e-02 1.568307212780118212e-02 6.186697399825302757e-02 5.248503609655964103e-02 -3.258264478201001665e-03 3.253426548599427925e-02 7.274702134557792244e-02 1.072418803242535315e-02 1.366795141074853524e-02 2.855521358243507529e-03 6.029825335749349272e-03 2.350321389622475610e-03 1.018087701085188370e-03 3.233135909040296425e-03 2.245364349058087683e-03 2.664312146223218273e-03 -3.890321793157530683e-02 8.738996254568533314e-04 2.675519200094814316e-03 1.236701442712456224e-02 3.484382556149962732e-03 -2.595672775192663442e-02 -1.075664389518441941e-02 -2.393568063763078235e-02 5.295535611427564654e-02 -7.603180505640117631e-03 -1.163654892011924992e-02 8.851537434939609969e-03 1.802088253972480847e-02 4.530892447850364685e-02 7.745415307440588204e-02 1.284756232013633398e-02 2.556742828346738119e-02 3.780891296818710001e-02 6.829519297559563662e-02 3.122319228384743545e-02 1.040086951669848409e-01 5.844714702394751549e-02 6.199833563084947152e-02 4.298353838734528254e-02 -9.997738663806099435e-02 2.214698256689916211e-02 -3.754244790873555865e-02 6.319543672301182122e-03 -1.446323874957334513e-04 -6.753011179467689577e-03 2.522660324068175253e-02 -2.028587987299421136e-02 2.268436055266327342e-02 3.142688300000530399e-02 -1.238263478093758370e-02 2.017004812624534349e-02 2.642980640343215587e-03 5.332269917509906317e-03 1.164766964588252914e-02 2.166019905586615349e-02 -4.784788222675224502e-03 -5.527576009511979936e-03 -3.163477210890186520e-02 -4.487930425016119988e-17 -1.921936519831092842e-03 9.393301725493689164e-03 -2.335951354225468629e-04 3.988972542030793517e-02 -1.247708332610338881e-16 1.796397152757064214e-02 -5.863819730665111983e-18 9.404699963419645148e-03 9.696104948297274134e-04 2.693277703968442738e-02 3.743912007647419871e-03 3.660208345628881282e-02 7.313950150828863774e-03 2.178241281376296426e-02 -4.330623385313455467e-17 7.965818060262296399e-03 1.669379129104984431e-03 2.244419517146004940e-03 1.621516550600559433e-03 -3.440048665928398947e-02 4.381838751885565840e-03 1.124405101378502324e-17 3.396575650843310923e-01 -1.400336801578968941e-02 8.474876304581860711e-03 2.633974559841734467e-03 2.887307017907848536e-02 -2.150494476230153940e-02 -1.447324694437389023e-02 -3.852583817101392326e-03 -9.716310499010578222e-03 1.048952387173439360e-02 1.545687868940900066e-02 -7.678139613327478923e-04 9.373122999117742757e-03 -1.931982637291526986e-02 9.179266272741923305e-02 -3.573780742187876447e-02 1.538327916568080558e-02 6.670470278470068577e-03 6.621504911737499111e-01 3.932812931755339525e-01 4.179003886713733573e-02 2.249207987174193721e-02 2.396684239198816971e-02 7.174981274274218335e-02 7.938453375362648778e-02 1.175743138578351325e-02 3.086709808444682707e-02 3.320622978213626231e-02 4.304463782713797287e-02 2.406823997420081199e-02 2.411010757429576587e-02 2.020672404453748711e-02 3.896421751225341568e-02 1.618944024099506010e-02 4.934904040741293352e-02 1.947184093949965930e-02 2.135176035870347405e-02 2.506952342496179126e-02 2.036140353027714872e-02 4.584774988560865773e-03 5.265243008389153268e-01 9.130373950765428015e-01 7.465561656294328330e-01 1.027002676636014963e-03 6.594661944539456991e-03 3.975384146330918078e-03 1.458763850421232640e-02 5.455399517732415632e-03 1.329830966792023249e-02 8.181033595609663034e-03 1.257689067157842511e-02 5.737583799369995637e-04 1.660626798431190376e-02 7.626950072419535391e-05 8.551022728577193505e-03 2.402986490142324915e-03 1.892920590782019505e-02 1.083173462951855098e-02 2.678699471343475025e-02 3.082075965546509158e-02 3.752816334475653909e-02 1.815587969799220547e-02 4.946593015373273311e-02 9.554273205401246638e-03 4.335804834072108649e-02 6.238704689285617944e-02 1.835284521412991823e-02 2.215057533668807199e-02 3.290037649121078239e-02 1.051642838086151721e-02 1.724632876457023872e-02 4.041016930522816492e-03 6.780454173955172137e-02 3.092479753033818002e-02 1.239394176695646499e-01 1.616743018928170073e-01 7.905120332481727075e-02 7.642604055367409333e-02
1.104769827749917666e-01 -7.505873845765009822e-02 3.464862044208026626e-02 2.494451988251705776e-02 3.569232713062147483e-02 4.167687351239150045e-02 1.231519396664455118e-01 7.474352745920015201e-17 3.498453640001885495e-02 3.047582654443829753e-02 1.227511842864297681e-01 1.779006508773625997e-02 2.490249171266285810e-02 2.055573510980074001e-02 3.428429664580518593e-02 1.101229662798535944e-02 2.287998937512683226e-02 -3.697383530669405716e-17 8.627631035258295777e-02 1.589046619675303293e-02 6.809582047850128161e-02 1.639397054686873910e-04 1.731530037075587267e-02 1.438328134914836423e-02 2.489015259918317749e-02 3.704013080565624061e-02 8.998309550499312970e-02 1.713830165569971489e-02 -8.505428152608635137e-04 1.283912443840662257e-03 2.843593293664074990e-03 4.231546036946459005e-03 -1.380541353211975526e-02 -6.871962166776067471e-02 -5.993148016947474899e-02 1.609218807533458005e-03 1.176294853464696875e-04 9.030063343941455482e-18 3.244246063142993547e-03 1.698691979881789341e-02 3.678156892873542250e-02 2.761694444914943383e-02 2.270418913926152207e-02 1.978674293853046121e-02 4.083826409001521662e-02 4.925606232393224537e-02 4.450613419290051082e-02 2.238937704097436129e-02 2.431117330947075994e-02 -2.091566478459857409e-02 -1.323349205981038897e-04 -3.916202250122306251e-03 3.255598587134544281e-02 2.421163829086105806e-02 8.604175148814899310e-02 8.736121038944638650e-02 1.372302535292371783e-01 9.115121865762698672e-02 6.158468884637481600e-02 1.873963894955732110e-02 -8.043355039766879322e-02 6.878369200176670008e-03 -7.539857031137682519e-02 2.621712179377602703e-02 2.625037408613007522e-02 -1.756585624643428305e-03 8.254349753711516524e-02 6.394896531537350333e-03 5.294276904714413645e-02 3.486653993702815840e-02 6.684332763042559987e-03 9.282054495124104969e-07 6.418015605287689766e-03 5.061119130905285257e-02 -2.222277495322133936e-02 2.486511433115168312e-02 -2.359075173927100907e-02 4.675348559743742333e-02 9.500710650517585343e-02 5.593765719082703647e-02 2.546099555904273587e-02 5.616924815759700984e-02 9.787656893870955166e-02 -3.124081443426718428e-03 3.556129357026653997e-18 -7.238966514886831238e-02 -8.357213419092562954e-05 -2.817187337532975691e-02 -1.812411316419768300e-17 -4.951186739751819305e-02 1.075710002663796822e-01 9.832027270770491226e-03 -3.551010128493240631e-02 5.792829266194973389e-03 1.232211067044718111e-02 3.264950443202038082e-04 2.737258047527144772e-03 7.752333803517225987e-03 2.492351572054945816e-05 -4.932019240632210525e-05 4.395091303149029592e-03 2.776546281078637324e-02 4.178952863960290476e-02 1.102562555535444369e-02 5.907396877988618095e-03 -5.267302906427253750e-04 2.275070731917371719e-02 3.425099598531788220e-02 4.688394721750787503e-02 -1.386030076146916104e-02 -3.245627408585568391e-02 -2.408529124986274239e-05 2.765721244239752880e-02 -9.311239003157405561e-03 1.345749749389654748e-02 -1.977593873479225120e-02 3.907634485547311493e-02 -4.505390390494071889e-02 9.866775740777581141e-03 3.395201147378749257e-02 6.358545918817777309e-01 3.759272058573885356e-01 6.553557250598422701e-02 6.250084229848489048e-02 2.340226973080283476e-02 1.057842354737424939e-01 5.652532844180063049e-02 1.437166299953822508e-02 2.586017981462972529e-02 1.035570284827989906e-02 3.611012521743289100e-02 2.820655689587395556e-02 2.589731714666341708e-02 2.305265574578246879e-02 3.078296454705817600e-02 1.259164993998861809e-02 3.617032280545124517e-02 6.533581770442147274e-01 5.540237462447255012e-01 4.891837466376474097e-01 6.001064259383790886e-03 6.035090156572850504e-04 4.131396323993498376e-02 3.864881776523165779e-02 4.434280171150847133e-02 4.599153752150238195e-02 6.731404254785963714e-02 2.098438054779274212e-02 1.804930786921315686e-02 7.737958768052636757e-03 7.856476589572419789e-03 9.228429008816778703e-03 6.013651592007628333e-01 1.374008040604438152e+00 3.682097139169263067e-01 1.815309823065613254e-02 1.047964776018892215e-02 9.224996074133573693e-04 2.031439790767417333e-02 1.638245333068465495e-02 3.241279415693740229e-02 3.186525621410691428e-02 5.075462118173446752e-02 1.472853436059653673e-02 3.505905806563599375e-02 4.908209393645612123e-02 8.324853509243719529e-02 1.745962545177705619e-02 1.762917145399416768e-02 3.632203560634051920e-01 2.757116317412837136e-01 6.623741844329896766e-01 2.489589258882374950e-02 1.992620673303246645e-02 7.917574078093528311e-02 5.094692941973270883e-02 1.351496710948042190e-01 9.577338684244138256e-02 4.873078549350667865e-02 7.810465551711926087e-03
1.183745234931153634e-01 -5.568336621393339680e-02 6.722138582371293969e-02 2.168559292444114522e-02 1.084638579587541597e-02 3.308128164374237595e-02 5.385692513145729637e-02 2.742540880511085480e-02 3.263246076882220931e-02 1.630001358163043687e-02 3.444543172868582553e-02 4.200296756905727395e-03 1.369847263510209912e-02 2.668080543288892104e-02 2.314386406637959637e-02 4.766962870587457268e-02 4.790654891076927269e-02 3.467890173857081024e-03 2.136129462345941035e-17 2.324220349901739632e-02 6.183522648873085448e-02 5.037885289601597150e-02 3.787200365344343261e-02 1.245046139803447291e-02 1.863053787408078255e-02 2.765287957040951650e-02 -3.082285386969497304e-02 -1.031960227933730323e-01 -8.687633168631765657e-02 2.226677831054040945e-03 4.099744002934238574e-03 2.848273808576331449e-03 9.475063291792611990e-06 1.287159072375458071e-04 6.213831093184195124e-03 1.380799308133253203e-02 3.938799218233592103e-04 2.777234243846255207e-03 1.762097069470692709e-02 1.282030360604383828e-02 7.055881260462442371e-03 2.532911192678304751e-02 -7.703990662381600928e-04 1.859065241541310587e-02 3.019576827718066167e-02 7.478754311498406193e-02 1.053189787566962965e-01 3.523515723657469345e-02 2.007910881632991168e-02 6.931910040832638120e-02 4.846440146752446776e-02 2.287676607015146774e-02 3.875163530979326698e-02 3.749114000367725907e-02 7.581704701265377933e-02 5.564103059445145438e-02 7.916404003985015592e-02 2.501944231505608476e-02 1.736534266144843519e-01 6.029723585766378485e-02 -1.021490439710553283e-01 2.851069642850306753e-03 -2.154760394805981319e-02 9.920003065134508358e-02 1.170638962013461140e-02 4.641722623720469893e-17 -2.584136780355767208e-04 -7.252039381249820092e-18 6.017481023862413653e-02 7.860256617394464906e-03 -6.492505398467866710e-17 -2.614852354736435514e-03 3.521215948557870987e-03 -2.994259278763728433e-05 -1.281828727696678787e-03 1.080616313195749759e-02 -7.422882981075174015e-17 2.124501383849959190e-03 -2.833721384588396008e-02 -1.882425289255001272e-02 -2.749922231201356476e-04 4.408720802815076548e-04 1.413434372453749657e-02 -2.748944846942918269e-03 -6.280093716059468277e-03 -9.989772317987926850e-03 1.236468012098147773e-02 1.077026910830452926e-02 8.997664785226639622e-03 7.788446959186268170e-02 7.039098140682997079e-03 -1.699090211513666279e-02 5.101659748911208797e-04 2.565475124767665344e-02 -3.805344395545610697e-04 8.486777410034791747e-03 9.541137853224902765e-02 1.153864711621819909e-02 1.155612257042844048e-04 -3.266565336618824640e-04 1.519457681582661937e-02 2.937225377360573597e-02 1.548530959916866699e-02 2.541869247907957083e-02 6.700070158077477134e-02 -1.669539790618690198e-18 -6.294285892101167401e-02 -3.775479342687220602e-03 -4.137989557716377498e-02 -1.250947404745628249e-06 -2.317637210126949056e-15 -7.064994474111402703e-03 -2.935893738358098261e-04 3.630974147503064631e-02 1.761389028260785345e-03 4.187884281261867537e-04 1.930219666061103873e-02 1.127338611907432592e-01 1.968677803772684864e-01 3.240548292230284455e-02 6.188641117229036759e-01 4.169209562648928391e-01 4.764235672620997786e-02 5.457285951426597836e-02 2.760458273316846942e-02 1.042215609718690900e-01 4.958361100731695920e-02 1.176457132839092701e-02 2.472635360661915721e-02 3.702364442134171291e-02 4.345238116003347778e-02 1.950957052606218378e-02 2.060737264514195255e-02 6.607093261574071077e-03 2.271618957213463949e-02 1.072543183413929638e-02 4.169630188314511071e-02 1.497208742058012479e-02 1.927200602206956964e-02 2.253464698987514211e-02 5.855169250827466088e-02 2.111506482828710146e-02 4.202044555111585089e-02 2.238797267957985671e-02 1.986525084859683613e-02 1.303388162483331419e-02 8.162961384814806376e-01 1.061439472970398201e+00 5.053966913948763073e-01 1.543012784527877631e-03 1.596800013782079600e-02 1.651293616657226529e-02 1.709247444469105121e-02 8.267801421620393372e-16 2.026356457700007052e-02 4.373285594951703548e-03 7.403978285828238110e-03 1.059107452178981515e-02 2.225406168422868619e-02 1.780400300941467400e-03 2.053275197985734643e-02 6.291917553195391921e-01 7.620339205672922445e-01 5.816960660187950882e-01 1.093319306718155970e-02 4.601554119179895874e-03 3.225756511185469411e-02 1.310625685038418163e-02 2.356922451152226525e-02 3.368402896925424411e-02 3.775380736403575327e-02 1.253395083962429191e-17 2.440131513192630650e-02 5.290195802994007018e-02 5.937879787085718897e-02 1.123383227495587146e-02 1.037193641508392383e-01 3.280389098217988963e-01 5.130178477690113814e-01 8.664677048136787407e-01
2.082001712199257060e-01 2.699136597156035627e-02 6.024871959155094978e-03 2.095673279930117001e-03 6.100832362179750643e-04 5.944904952802569469e-03 4.369942963629623936e-03 6.337799177958907022e-03 5.023065910783228519e-03 3.796421706178393449e-03 2.697003790615003574e-02 1.224809704310426094e-02 3.097898756754454905e-02 1.626103942939262359e-02 7.953794023359071763e-02 2.913822779033424862e-02 9.472293958668860994e-03 1.965506324565307467e-02 2.597359207353478611e-02 3.203667344534041200e-02 2.332262236925388421e-01 1.303402273402002459e-02 -1.175011713531214780e-02 8.803631748728912854e-02 2.857990575246922571e-01 3.990207908590026609e-02 8.384128079067271200e-02 8.184998297143139534e-02 7.081437101297809178e-02 6.691518866550812128e-02 1.494073562057336158e-01 -2.567141768733792809e-02 -4.941887345446945088e-14 -7.161423742476297552e-04 4.414544689167092945e-02 1.295099502430525039e-02 1.894859896400600402e-02 2.645802134217075524e-02 9.720747273597933613e-02 4.204809462744393128e-02 -1.990270552895001197e-02 2.451498437736768232e-02 2.737639063975058976e-02 1.041700846585627040e-02 2.244676149075310739e-02 2.970575380136465313e-02 5.166831577981188806e-02 1.325043487032002802e-02 2.813119601533125430e-02 1.551711424754440190e-02 -6.878184370654179952e-03 -5.474281203337873561e-03 -1.491441727580837540e-02 4.556700857934275345e-02 1.231089149271640537e-01 2.014369786235672144e-01 1.650695409521492907e-01 1.962548068010952773e-01 2.575836183055689732e-01 2.530161036407594863e-02 -1.634030104971292419e-01 2.979438399054459494e-02 6.316114018749093700e-03 3.747201293424573532e-02 2.221475254397006771e-02 -2.762702394003437728e-02 2.003077198347032531e-02 -2.565268803565047290e-03 -3.328519830605056141e-04 -4.593236600897912373e-04 -1.160233113509797492e-02 1.610673910308056331e-02 4.161907135991698217e-03 -2.952960243052958275e-02 4.666989214416049879e-03 4.802090594880438490e-03 1.933021758758224004e-02 1.357934922329173504e-02 2.754692060063328868e-02 1.042383582032149064e-02 -9.996675209622219544e-18 -6.799395221950957358e-02 -4.593764635406166058e-02 6.220364801335184074e-02 -1.004606261303805911e-17 -2.470942854419060433e-02 -1.211431953279610255e-17 1.643501846227548857e-01 -7.371614970313658532e-02 -1.527752931966519685e-02 4.073638152434029894e-03 4.411646229526008239e-02 -7.425154198481673587e-03 3.502014815904100353e-02 3.408236164039914196e-02 2.524568850531247813e-02 1.407135644896291039e-02 -1.387791653873096942e-01 1.045386452874301313e-02 -3.065251550976637318e-02 -7.294188857063009295e-18 2.281737639157265136e-03 2.821624670136900406e-03 8.786411969063915748e-04 9.635030963223685865e-03 -1.522419288188074475e-02 -1.110053467500773721e-03 1.993775226646284093e-02 -1.252218871070181171e-02 -1.487953730286328650e-02 -9.271133885853578838e-03 -1.846790988687750310e-02 -1.999219128171552534e-02 6.064205119443953629e-02 4.158198014797142997e-04 1.219951569880516626e-03 7.585808379696520193e-04 2.211083916398501326e-02 9.367706804595613002e-02 1.932605857695899459e-02 5.668374565620518668e-01 3.227405767049534302e-01 3.384958143560199556e-02 4.684820971575377573e-03 1.006909282701478996e-02 1.236883618756060010e-02 2.012736390854570970e-02 1.636625692015316463e-02 9.917393929010287143e-03 4.926297916388096103e-03 3.353352151599334890e-02 3.558864686701435857e-03 2.964686255453777633e-02 1.355501497641794367e-02 2.823602251819246067e-02 1.316655174817981354e-02 1.920822456396341013e-02 1.033575020057782810e-02 3.354847363173265457e-02 1.998130895907494764e-03 4.702403499904480588e-01 1.034301292964808106e+00 3.754838039746431488e-01 1.991735035113528091e-02 9.799770499948630420e-03 1.308989296388550414e-05 2.703043840304567708e-02 5.462793477780801543e-03 6.392153823160221654e-03 -3.488777629936264113e-02 2.914071269420026068e-03 8.140648084458964595e-01 4.431190309279026129e-01 3.113149226578186823e-01 1.311130414785439263e-02 9.796601606162710571e-03 2.828938906422718419e-02 1.888428005302560456e-02 5.018297375398771826e-02 5.599647079006668526e-02 2.927835772104122550e-03 1.347005607391916181e-02 3.300698365018659569e-02 4.573159932833723712e-03 3.958810806577951064e-02 2.284294604791937272e-02 3.693303141120670569e-02 1.003710826417848236e-02 1.483536491888088477e-02 2.614907913764947037e-02 2.377487695113936761e-01 5.456158763566768544e-01 4.108807239843587089e-01 1.188513507623622373e-02 5.061150185803368895e-02 8.042834405803923892e-02 1.337933983561255813e-01 4.257394988729278074e-02 7.835752802288076246e-02 5.547451101942998952e-02
1.212425184551360752e-01 -7.003283274357872956e-02 5.266837932503104502e-02 1.823734816327094929e-02 1.381838160088802794e-02 1.755418370876430864e-02 4.579926541013761704e-02 2.429232305985586440e-02 2.352510271935862388e-02 4.363866481287673182e-02 5.841355881382765541e-02 1.921920719653403067e-02 1.934879523187997205e-02 3.856614297882464404e-02 3.258422624492417208e-02 4.430534545607845570e-02 8.108591853180167897e-02 3.246169278727923518e-02 1.735427751477003691e-02 2.874865671883172075e-02 -1.237931496496839506e-01 -7.221182877082478468e-02 -9.559531932398755705e-02 1.031085678340428607e-02 4.085265906698371365e-04 2.151742213911184837e-02 2.069695842422163493e-02 6.552555114640984044e-03 1.479288742429383033e-02 8.209040919764929931e-03 1.037794609745706486e-03 9.215436475945398137e-03 -1.074477664704777576e-02 -4.698734308445110669e-02 -4.221541857734569664e-03 2.060771389635756429e-02 3.108850358668742084e-03 6.462656900200353279e-03 1.204616394578879486e-02 1.429049849714061383e-02 3.000289596049452237e-02 2.455698387909407629e-02 3.254610274071509379e-02 5.037448247393840582e-03 1.553654878991804489e-02 2.739108340161428359e-02 3.764481170225416479e-02 4.634214971905249204e-03 7.900138236502632708e-03 7.561043701793166046e-03 1.328848569181789382e-02 1.824559900295259680e-06 -3.370809100225684307e-02 -1.987203757328063689e-02 -1.271554587258326430e-03 9.808577119645131570e-02 7.605589171154419903e-02 2.368187817391382843e-01 3.940093862966729255e-01 1.279580394785897191e-01 -1.225264770119697394e-01 6.197330511829374049e-03 4.246181057674081050e-02 -2.144560056645507078e-03 2.595713838715829613e-02 -2.348718927164817477e-02 -1.646225328259278464e-05 -2.493128572455391954e-03 1.308956306026552024e-02 3.910451102994394068e-18 1.486221587464618688e-02 1.021126136507157466e-02 7.224234467449350127e-06 -4.069314596180741150e-02 5.244470524911691578e-03 1.786983735061898129e-02 3.774865472371182178e-03 -3.396169403144920496e-02 -2.759181674784883659e-03 -2.652315767396276960e-02 1.759155255679699728e-03 -6.350375718363979183e-02 2.263013871551729521e-05 -2.649328572766531872e-02 3.506171119066774783e-02 -1.972625690994558867e-02 -5.440510782791480485e-06 -1.972285485153969092e-02 -1.120844718142057909e-17 -1.541711291561650120e-03 1.575864606378345494e-18 -4.099530303372136975e-03 4.628644395117484205e-03 -4.703716463746585551e-03 1.447889205241193085e-02 1.508535435726829875e-02 -5.677560872736127189e-03 2.691435239711181121e-02 1.248688692171263763e-02 -3.543224183251826673e-03 1.856309481188741194e-03 2.346342375679060736e-03 -5.730811601019649565e-03 8.444403893815949358e-02 4.513351362460999230e-03 1.090761879125016478e-03 -6.610565838112231413e-04 2.357837292213587829e-02 1.375330529892601975e-02 -9.685947141749189075e-17 6.456591282969274814e-02 3.752281734307928718e-02 -1.384078355088381312e-02 -2.919102328261629492e-02 -6.847353568085114139e-03 2.930045509506995147e-02 1.017889188413059059e-01 3.780223484250052474e-02 4.132936615676641862e-02 3.682393433757710083e-02 6.116466559517795831e-01 3.963264235720123652e-01 3.836883546737748091e-02 2.707824860218454449e-02 2.242207802817349979e-02 8.266488444399759661e-02 7.751406921692510776e-02 7.677527598916325351e-03 2.171974936614441340e-02 2.045390428605166344e-02 3.900043699657784052e-02 1.491747039155947273e-02 3.280038259863726768e-02 1.603453924889039831e-02 3.017040891241393186e-02 1.208762357485224960e-02 5.074387426953227975e-02 9.070055291495012634e-03 1.054920864557631899e-02 8.332571034417254191e-03 5.053911270181195015e-01 7.276295141051607462e-01 5.216453424521445559e-01 6.284298205064110798e-02 3.197441601080091428e-02 4.145597664151075279e-02 5.490195646132556284e-02 2.219590651927974903e-02 1.616951755778611027e-02 5.745442570917719260e-03 1.531239104004297746e-02 2.816058025882725233e-02 6.204610043271177044e-01 1.173864671279508887e+00 3.520873024839339394e-01 5.085995169302382282e-03 7.108719055342744599e-03 5.446417890439843659e-03 1.806245849080034133e-02 1.257210599129994326e-02 4.971252369973361979e-02 2.213342333714224744e-02 3.024204826233321819e-02 2.488132729889354789e-03 3.722861035424497589e-02 4.383691690285427406e-02 4.968204111288147817e-02 1.529891895423063884e-02 1.920243190267489908e-02 1.771766836202602266e-02 2.959767952617448766e-02 1.401725174597747217e-03 8.118191503398627384e-01 1.965111098875754969e+00 2.839361260195137349e-01 9.256035597954702376e-03 9.771036459770437999e-02 1.304664034281951546e-01 2.866379363228283364e-02 1.662387338906296669e-02
3.079907587560615223e-01 2.017262295965659841e-02 2.545328775374311112e-03 9.593263587182795915e-06 2.414812199038957313e-03 1.602499581410481242e-03 2.862534900325135103e-03 4.817158608837189201e-03 1.075708454966023755e-02 4.621779486495288837e-03 3.556099610148046736e-02 1.349339436292856798e-02 5.026580757488099266e-03 3.041032152464435978e-03 1.763909132898701512e-03 -5.352541370451106145e-02 6.152492237731573867e-03 -4.195805430853898760e-02 1.664649877198862149e-02 5.191267436989534245e-02 6.506799890026279676e-02 2.739526021558800692e-02 5.428072039576439280e-03 3.964336208009474477e-02 1.342950164950307002e-01 3.733067343312532205e-02 3.894260112713933408e-02 3.864205714392428964e-02 3.805663998199827064e-02 5.659870753752405809e-02 7.811668320182649228e-02 8.758168009676764454e-02 8.048564464705230959e-02 4.118069790487263043e-02 5.700509000789597686e-02 4.657699331437144263e-02 2.463297230864843271e-02 4.868495603458151966e-02 7.484749071546550891e-02 4.923603379256395046e-02 1.872844202544705211e-02 2.825848926472365741e-02 -1.674201382342565061e-02 2.104882471525952592e-03 7.887924039401875143e-02 1.089192973065197245e-01 1.603877979295198208e-01 7.111125684581501294e-02 3.802110726278497277e-02 3.204575437070901478e-02 -6.348030601938275952e-02 4.372243013887235075e-02 -8.175394868028983131e-02 8.794955063136813966e-02 1.682124945558565732e-01 1.109031865031544911e-01 1.122002864950767814e-01 2.333566436881000516e-01 2.646987078251989978e-01 7.979040562922268265e-02 -1.370124675139944570e-01 -5.866075928213858315e-03 2.096194286097561449e-03 2.091670356671004969e-04 2.207305004193553868e-02 9.714308124656963195e-03 1.789543014855878716e-02 -1.494744833670310843e-03 1.238875897584598533e-02 2.961998694287235750e-02 -5.511442387903731782e-03 1.290224024978460504e-01 2.747185665715025024e-02 -4.890107352388230882e-03 1.956033656798408793e-02 -1.047186717452037564e-17 -2.886610173655152715e-03 4.288964275558770217e-02 9.090776029723585637e-04 -1.447139327092690844e-02 -9.450179794511501252e-03 -5.605204792747357515e-17 -5.327889471995506413e-04 6.126345348654414945e-18 2.471315736776968919e-02 1.545578686285486482e-02 -3.253629196546120979e-17 -4.913894147460833912e-06 -2.218130172448238430e-02 1.469285473914423408e-03 2.226721014929677542e-02 -4.605035276857180788e-03 -5.380643559858773765e-04 -3.723008543871788651e-17 -4.102450102914630484e-03 2.052741896114805098e-17 7.868239730990267411e-03 -6.959220523140641212e-02 2.243058551482507781e-02 3.473451154644537076e-02 -5.393960902222344304e-03 -7.606672103151587185e-03 -1.096237328899743284e-02 2.102172100916282951e-03 2.256013411356277976e-02 -3.680428623793360193e-02 -5.837711714138268948e-02 3.149960763638032409e-03 -3.904049952215190309e-04 1.351942518015209274e-02 -1.615122498152249050e-02 -2.298828206185501554e-03 -7.951662807275008960e-03 1.747139828080813972e-04 8.570392100446029451e-05 -3.327999572243101056e-02 -2.015170120447496230e-02 1.857555463031695191e-16 5.195753632983710069e-02 -1.302951300076201193e-03 5.560032888494679248e-01 3.674495285380518506e-01 3.130097569394060114e-02 7.357441732266776016e-03 1.629849297180883255e-02 1.318798373824835331e-04 2.113012974587683696e-02 5.050016996242405465e-03 2.035938073661692355e-02 1.396569443017816724e-03 3.781460495938911842e-02 2.359724756862906225e-02 3.106801468412826242e-02 2.673095287956290395e-02 4.625074276811612373e-02 6.082153637885757691e-01 7.856575104398282328e-01 6.390715055204401196e-01 2.192965171965073276e-02 6.476106130309023691e-03 4.378127881282233708e-02 1.086129469831823534e-02 1.341080910573096141e-02 3.891038580597191293e-02 8.809698786384505265e-02 1.843403157277608007e-02 4.717986203378182058e-02 2.834493434474360390e-02 3.366781430834833722e-02 3.862509376199048383e-02 4.131230365450906661e-02 3.953705108462190337e-02 4.617612153095242145e-02 6.147967740306119200e-02 3.582952960472639975e-02 1.701795758010208465e-02 2.748084350123280659e-02 1.288739654471021852e-02 3.171181454877088773e-02 5.456728891523729939e-03 5.084700644165012218e-01 8.896062439214623696e-01 6.428149893379682123e-01 2.814808139226871453e-01 1.341129565217602237e-01 6.348863687039829617e-02 5.200563660582491893e-02 1.223099448980837500e-01 8.069335968036367435e-02 2.388341464579922524e-01 3.863949356454708384e-01 7.109655447500254910e-01 4.503588566259733028e-01 9.395165978546348579e-03 4.334936698916128789e-02 3.025971550701372567e-02 1.251274177463311821e-01 2.293583040927608604e-02 3.564410391179383825e-02 4.244833694327763973e-02
1.924358592940230428e-01 4.049928385602408892e-03 5.774442640999244425e-03 6.858282382364045207e-03 5.384520101063690681e-03 6.448144388939250652e-04 2.016286781681015522e-02 3.274813908264823277e-03 8.028091865015012096e-03 9.484746826919998250e-03 9.388780652417052022e-03 4.300887860497690229e-03 2.331111090332200694e-02 2.986739349857014947e-02 5.003803235811316030e-02 2.107005846390647488e-02 1.481800322285465460e-02 1.492102188963334228e-02 3.723254821274417370e-02 1.036000057103184102e-02 4.920824722067655699e-02 2.767506396778918187e-02 7.876727698249384735e-02 4.977970680625634636e-02 4.603744628761402002e-01 2.581125208763748180e-01 3.127826940781215925e-01 3.073322921934344976e-01 3.999041761065232903e-01 2.174567052284192559e-01 6.159569842336043433e-01 -1.743731502530607505e-02 -3.814582690718457335e-18 3.045494608984056911e-03 3.440186557537472378e-02 9.200043414752901924e-03 1.815477136624580809e-02 1.932083708483795306e-02 7.786788863704481589e-02 1.308081114143354838e-02 -3.930010658212022140e-02 1.947430632438097026e-02 3.341745245853260798e-02 5.699834648173949386e-03 2.608428174403461253e-02 4.937711980232588338e-02 5.545892308554525169e-02 3.028481639391820968e-02 2.934477217936744958e-02 6.512567931095968911e-03 -9.596984680098350276e-03 5.826727782201884288e-03 -7.918779861575039533e-03 4.786926631507443936e-02 1.478564206201337383e-01 3.303338605024320751e-01 1.334257499888039933e-01 3.849946230656222657e-01 2.290678384616749719e-01 7.649776741809886926e-02 -1.777040334738989591e-01 4.354983737774968666e-02 -2.760837336114965040e-02 3.651870256002191784e-02 -1.244446023024032288e-01 -9.541420867822539634e-03 5.590554604095160363e-02 3.915062499580203997e-02 6.610764053070468469e-04 2.201457501390237728e-02 2.431232856614102752e-02 1.779687631242310789e-02 2.154984615636712508e-02 -9.639658021699883160e-04 9.584916751794218814e-02 -7.679929455250405351e-03 -4.632519423363972225e-03 8.622416364210538919e-02 2.661105297968820171e-03 3.036246603615636377e-02 1.778506857440372360e-02 4.544554216369467436e-02 7.199365047459048428e-02 2.055238325073946160e-01 8.439677186847417342e-03 -2.151281322713809174e-02 7.812347749529467873e-03 -8.072152308746403850e-05 2.257769254775524898e-17 -9.367058098346986092e-03 -5.739047278196410945e-02 6.171484145338717447e-02 -7.184021823741065793e-02 -2.604901026745104833e-02 6.103142791542950918e-02 -3.314193842445817533e-03 1.298086445563436872e-02 -1.366626404732647626e-01 8.160636417066579544e-05 -2.619016349163778845e-02 5.820485110383779616e-03 -5.424650729393758194e-16 1.101610998315167775e-02 9.012529045239464978e-03 -8.492155920430761106e-06 -2.527346692064492712e-03 -4.215780209441262645e-03 2.639427443654256134e-02 -3.145607932832859138e-02 1.465475180588153310e-03 -2.197902495051776497e-02 -3.938267133412144683e-02 -2.578585915074601068e-02 -6.468635918615577734e-05 1.251965206109583589e-02 1.073017525219591237e-03 5.344342924726003426e-03 3.368939135745476626e-02 4.603170730359808921e-02 5.697109790485284549e-03 5.596195193358030062e-01 3.560527440553709067e-01 3.961633324928223732e-02 4.550895527360824394e-03 9.995026766394099829e-03 7.839570426717000842e-03 2.037990241744491032e-02 1.507219848914435022e-02 1.965617637499315201e-02 3.522664588154326894e-03 2.527656730421848352e-02 1.244148243804824722e-02 3.173254896501405409e-02 1.529724404701414062e-02 5.015416539820885533e-02 4.038836784081665382e-02 3.679736222661712103e-02 2.103899692043482636e-02 3.124697416283765336e-02 1.449731272831428393e-02 4.165393980698042786e-02 5.647103157619186575e-01 5.641915726211643722e-01 4.410288597775340880e-01 3.351298689382028843e-03 8.485876092669340787e-03 6.863522611612556112e-02 3.633852620953631485e-02 4.391682010507748368e-02 4.138259816881932618e-02 2.087753641826282577e-02 4.694884299299742492e-01 3.748967583344184340e-01 3.268885935565808976e-01 1.714250423922689809e-02 2.564798205487113947e-03 2.440681737087503633e-02 7.776189092126572187e-03 3.373484576051022366e-02 2.930588930669143083e-02 4.722931875959039336e-03 2.132090337516226800e-02 3.984783835561408333e-02 5.797791686317618244e-03 2.940364112054700760e-02 2.942557398281620343e-02 7.058259017073510722e-02 3.325765149930437481e-02 2.306326674186348971e-02 1.947385382473207013e-02 4.993407221939972218e-01 7.343563457908410808e-01 4.638528161640693148e-01 1.522320497141652247e-02 3.074447172149152169e-02 7.370138577577266259e-02 1.107714092710996440e-01 2.791463465803360397e-02 3.957341827849759214e-02 4.796199773915058029e-02
1.328133030390161173e-01 -5.598977130914477296e-02 3.007381570969943144e-02 2.709473596914599783e-03 2.057493389483393623e-02 6.422214521125824659e-03 1.670767888196397419e-02 3.324433011101630368e-03 3.267063729107805986e-02 1.042225993596545934e-02 6.700251873705495576e-02 2.018103093814634791e-02 1.170671963841734067e-02 2.574914983262904453e-02 2.374609695557668848e-02 4.223596741264847715e-02 4.128117019595613918e-02 3.842986251553583144e-02 3.522532219853069524e-02 -3.573848556788265102e-02 -3.279523759013340534e-02 -3.006574593599039419e-02 5.634436983017049316e-02 2.709709252817493752e-02 3.687446821425333088e-02 5.140320030167530585e-02 5.191677385608461232e-02 1.778827476109940547e-02 2.690218974143377271e-16 1.565310759253349046e-02 2.985652625244558309e-02 7.151307877989943051e-02 1.437973087828232632e-01 1.959348962653049281e-02 1.568069886144324646e-02 3.321091531740424235e-03 1.039232252444990869e-03 8.161697571705778947e-03 8.501613461743252886e-03 6.034557533097417194e-03 1.870259724576665145e-02 2.286206276430063217e-02 1.445885545028134135e-02 8.572866973199168988e-03 1.661677883585303481e-02 1.348663793258229188e-02 6.105855042585162790e-02 9.294705538559988600e-03 1.674747596780953862e-02 1.839732428676707152e-02 1.570971604502416352e-02 4.473027145766436327e-03 4.434590213072213405e-02 2.516205018914291838e-02 2.446296082334743771e-02 8.019243930075004023e-02 5.044539055000454852e-02 2.273361364291163339e-02 4.539749173112741371e-02 -3.774084867821082129e-02 -8.422178909461125462e-02 2.632325290951404925e-02 -3.428009833692362979e-02 6.771926509180321851e-02 2.001590917616352314e-02 4.218994411805407157e-04 -2.199724814301017446e-05 1.512396638666819657e-02 5.133498965159584426e-02 3.873027500275987023e-02 -1.753160657995061003e-02 1.232880150346085668e-02 2.514795343957341566e-03 -1.642893293710246040e-17 4.032031960124979352e-04 2.753742845232146769e-02 -3.067521425138789241e-17 1.663701160729211606e-02 -3.758348917797493804e-02 2.163787899460887718e-02 3.603712842177610436e-03 -2.396430845854851180e-02 1.292313553573315238e-03 -1.800922278563637374e-02 3.964577830997997356e-03 -4.663925419604807165e-02 1.961706507459484022e-02 -2.370819134136518958e-02 9.232011773119219722e-02 -2.797424742215222707e-02 -1.447910087769007726e-04 4.094292771394961172e-02 1.014739682292113351e-02 -3.251954119303953280e-15 -2.886909565018017099e-02 2.457239827797719905e-02 6.340771810254775855e-03 -5.209852945408344552e-03 9.631767386325177857e-03 -8.574559951994019033e-03 2.553428705945676429e-03 2.827019987352253141e-02 -4.657876544400183817e-03 -1.677827867980127667e-03 2.338006821732483559e-02 8.262739634345833695e-02 1.737841705432836489e-02 2.771136415917591425e-02 6.853898856041087603e-03 5.913333901307819829e-02 1.956688438052442761e-02 2.443481911532706732e-02 5.187504538172982294e-02 3.501597070148041585e-02 2.571585847629716606e-02 1.444808588920990129e-02 2.667865307873861641e-02 5.830012144379040362e-02 -2.195919332025672317e-03 9.537673527737279208e-03 6.299952513190529846e-01 3.472073682012304374e-01 5.618777242934720750e-02 2.287114102829165135e-02 2.126978408094713779e-02 2.777650915743141297e-02 6.286547064572986510e-02 1.286622711613575991e-02 2.106000913203567859e-02 1.249059910056293625e-02 3.161839553970308136e-02 5.360966506534139281e-03 1.835269943141887641e-02 9.885433304856488038e-03 3.342476289544793477e-02 4.657943341458654907e-02 6.895486616878637731e-02 1.698087649945160052e-02 1.091830748883481153e-02 5.407415318385058178e-01 7.136679519766437885e-01 6.734177888911946352e-01 5.736746340496808039e-02 3.377599272197469998e-02 3.002607682727775895e-02 4.534166894715803420e-02 6.759628322060780292e-02 1.514186784299943918e-02 1.090911542208736033e-02 9.659647020059900294e-03 2.373489298128750313e-02 7.552015351231002871e-01 7.839707158003139664e-01 7.208990091892161223e-01 1.725709999237308792e-02 8.391733136939117993e-04 9.840916834586184783e-03 3.893129345124454136e-03 2.857811053450299774e-02 2.464072406175460417e-02 2.988553976080954994e-02 3.091826211552464732e-02 4.013101925480883764e-02 8.524759065622770915e-03 4.196478915204512883e-02 5.514595133254739101e-02 7.889692572960854688e-02 3.980228050273189322e-02 3.204788764951578550e-02 3.588996740594873475e-02 2.401450485629116893e-02 4.255553624515205358e-01 4.020823614254540601e-01 5.851810282187178691e-01 1.223553022619731784e-01 2.201261194388559883e-01 1.944249127069741789e-01 4.438913129331407692e-01 2.757406490116207087e-01 1.685754471253747278e-01
1.181498086214466670e-01 -6.123542385878738559e-02 3.194959582938257381e-02 1.396007836631683874e-02 8.773415859498380140e-03 3.429646924102712491e-02 7.287601757602564512e-02 1.239966820758215023e-02 2.801187506235286104e-02 2.086737846983223454e-02 2.544139301515484577e-02 1.532368714604453487e-02 1.301254832159218396e-02 1.844988466635075208e-02 1.572738603119970893e-02 1.451640915476092031e-02 4.638598986154498366e-02 2.032414598813958584e-02 1.336205386713563634e-02 -4.900125319618019865e-02 -1.077120751984978336e-01 -5.804797451505055300e-02 4.538426072022621466e-02 6.478008877035828887e-03 1.400363468087206104e-02 5.427412789502425450e-02 6.421333550414894120e-02 1.554023768424203991e-02 1.730194439011329549e-02 1.031780517758162929e-02 6.240421997969932723e-03 8.266016596479137946e-03 4.028237423788912867e-04 2.455199934789248101e-03 -1.452122988651995961e-01 -1.146286683607658508e-01 -6.953249899971798709e-02 1.286542071916875127e-02 3.259623603649747237e-02 1.634980372024182754e-02 2.745365350453638290e-02 3.584034091221653673e-02 3.580362310929120079e-02 7.096263110519253695e-03 1.611522869391417492e-02 3.493349409825814017e-02 7.175835910437131093e-02 1.244639370759036655e-02 3.190001002500318650e-02 2.379695860954128897e-02 3.531390873510553696e-02 6.231246012045678871e-03 1.045541042854679359e-02 1.322420467631042415e-02 2.443118417889423785e-02 1.005502261389812704e-02 -1.541112304503070296e-01 -1.421718886506538604e-01 -1.750377947345475582e-01 5.989050394973513547e-02 -6.979127207324604953e-02 -3.462736520256703990e-16 -2.176078970046536887e-02 1.366891444943575951e-01 -2.107178669664954915e-03 1.428180920862005127e-02 -1.109355743508277719e-17 2.776718951554158973e-02 3.057536344802603417e-02 2.159233051583275831e-02 -1.139274041746755027e-02 6.980824256539330634e-03 -8.908332280094221410e-18 7.373946485779758579e-03 2.480151809738393408e-03 4.526245551518313232e-02 -1.429056457974917647e-05 8.447398316397089491e-02 -5.241984692596082795e-02 3.741919097969194363e-02 1.212612340214159340e-02 -7.111658008130337418e-03 6.479818467016694450e-03 2.799512860624400398e-02 4.722990467626533677e-03 -4.114058118588439470e-17 -1.149721202265990111e-17 1.616101223997496762e-03 5.082575048092536490e-02 -1.804466887170100287e-02 -1.706669498623888254e-02 -9.102382196008740322e-18 2.292813789545124561e-02 -6.394273262148998660e-02 1.319424487568595857e-18 2.314850323309905564e-03 1.957123552563958327e-02 -1.212533052272776182e-03 4.104211467731579783e-05 -3.933051774687361044e-03 6.782403534936394876e-03 -5.521450736249398916e-04 -9.757932927776991061e-03 -5.346746378207279391e-04 -5.796574415362742982e-03 -5.205121487872338985e-17 -4.683094040739284834e-03 1.682599859498477152e-02 -3.361402874160990262e-02 4.270122554984829427e-02 1.601705705493892615e-02 4.627091749894510046e-03 1.055463781575889833e-02 -8.489944995238640768e-03 2.048924620686855356e-04 9.216581173763123414e-03 -9.869333037320683621e-02 -2.973680974052707315e-02 -2.345256904869458814e-01 1.238472568167938576e-01 6.266743971644588296e-01 3.353484898052276342e-01 5.756015277160254212e-02 2.437114168866980415e-02 2.570514086504755574e-02 4.978641985051090318e-02 4.733364953349641219e-02 2.035925859598684828e-02 2.679236090564090372e-02 2.517648130159491279e-02 3.102316434229395251e-02 7.848788091054091989e-03 2.221860287676003659e-02 2.543606053326634767e-02 3.259225540697217871e-02 1.691538576930144352e-02 3.779095317563175432e-02 4.556491281626992747e-03 9.241435336648154766e-03 5.433106056459839905e-01 6.653289480619585250e-01 6.043007325994874668e-01 3.728626298439571135e-02 8.015912779343254899e-03 2.436387783157499196e-02 2.086407301258593611e-02 6.916931497646421068e-02 1.073468464151729716e-02 1.846233046706448333e-02 7.577250432016084149e-03 2.801392876965518031e-02 1.695781080519147277e-02 1.638774818580608292e-02 7.356038575703589474e-04 4.015480502839883226e-01 1.180324723409208465e+00 5.387191869248969534e-01 4.135423256795285173e-04 2.475078165157020005e-02 2.668164213254207995e-02 4.953256032548161020e-02 4.136174907746457269e-02 4.524669018315011226e-02 9.011417834265072688e-03 2.687485120839246469e-02 4.355824460441802548e-02 5.853359072268425611e-02 1.109514212156547525e-02 1.868785621452861284e-02 2.153980256354517953e-02 3.430156464461089905e-02 5.538748102647320187e-17 2.068185666155552410e-02 5.518712618566661704e-03 4.104321114492901629e-02 8.511972344378024389e-04 1.057112466860354871e-01 3.280648851383975995e-01 2.941613109219828326e-01 7.807113863527205266e-01
1.299330133543792620e-01 -5.054079974413749898e-02 2.757085499238834739e-02 1.548351187538138038e-02 8.915118495283699124e-03 7.477425083884255662e-03 5.899214573694061547e-02 1.124649662996705007e-04 1.820352270077974649e-02 1.109213519602952197e-02 4.911887779953146016e-02 2.189550600143213080e-02 1.354604471700261888e-02 9.808963600745790323e-03 3.676689051656230334e-02 2.690012454406272291e-02 3.738931818133845592e-02 9.410133971558197171e-04 1.583706128915800260e-02 -8.696155461852740631e-02 -1.860697834937404815e-01 -5.782506049335232878e-02 2.850271325704540823e-02 8.500925889077842051e-03 1.199196333473906070e-02 7.428836484825171622e-03 1.068431545822472600e-01 5.134678087142849519e-03 2.257051057350894716e-02 7.117516301095598041e-15 1.591639734274705523e-02 5.015769767246436306e-03 2.252413498388828022e-03 1.193110409095032445e-17 2.994432723602206938e-16 1.027577794354854856e-17 1.697698010522161877e-02 1.254992211695818774e-02 3.783454388566871718e-02 2.472394323350157047e-02 3.843759671657012711e-02 4.529392857292454527e-02 3.511512767419031761e-02 9.321350374104243192e-03 9.445089998949047008e-03 4.211644540907424211e-02 5.170896805049141065e-02 3.473740511806529957e-02 6.433000152827327012e-02 3.311147190009805508e-02 2.607578145025078731e-02 1.636885828405343704e-03 3.005369244411023461e-02 1.309234624519555780e-02 2.928510148858639445e-02 3.489065753481753601e-02 6.275919388005907207e-02 2.241942336953947346e-01 4.817350645331252429e-02 2.631673216890813294e-02 -7.775474052294510663e-02 5.905661086565242371e-03 -7.497249284717610873e-02 2.358420680869559652e-02 1.446460521258234946e-02 1.333314734351530133e-02 -4.012978404555784216e-03 1.585847345513729731e-02 7.139586031640247910e-02 2.937253575750319382e-02 -9.683185710015983910e-03 1.473521073463165097e-02 7.305740551316439353e-03 -6.274229530204781408e-07 -3.937534185519588782e-18 3.917709162635209241e-02 -1.043502335640240368e-16 1.055837086153040422e-01 -5.369556785471969740e-02 2.733353340948567814e-02 2.542612132282632068e-02 -8.780138952067881308e-02 5.334525457607552923e-02 -2.197741098479089050e-02 4.385575379508496741e-04 9.842745954123980057e-07 2.530753845920672568e-02 -3.322641089230684349e-03 2.743558541442181475e-02 -1.697225948379511851e-02 2.071412930803995364e-04 1.823308087743598371e-02 -1.483579569647438048e-02 2.926227879795205780e-02 6.971084194196250014e-03 4.894540303621529309e-02 -1.470114286417307375e-02 4.080097246198320649e-02 1.110822044879671150e-01 -4.077308375447104283e-02 9.345558898779192034e-03 -2.412932428458896411e-02 -1.765738441157419653e-02 -6.182297913716613117e-16 -3.995762142848836851e-02 3.834576391525435033e-03 -7.782058609533473725e-04 5.171380963746063236e-02 -3.798211497299402301e-03 -2.988635460772926661e-03 7.528780913345703429e-02 1.342900481208428309e-04 2.716467898791710787e-17 -1.704586956805181908e-02 2.083254920593761300e-02 -1.782134134845914791e-02 5.978640506949419592e-02 8.963822812400021323e-02 -3.361259136926151962e-03 -1.476611811258139078e-03 6.210312568794994492e-01 3.530097427609750205e-01 4.150176581031665701e-02 1.315005328737105586e-02 2.267085992553822052e-02 4.586459910086657793e-02 5.669358850774792319e-02 4.612001869697116203e-04 1.865014094125782423e-02 1.707353563022180956e-02 4.313979571562499143e-02 1.550629588861330253e-02 2.016782795704278458e-02 1.476419584036801555e-02 2.617430604456866608e-02 4.356717148932026662e-02 7.891011408366212232e-02 8.885599571185396825e-03 8.063182795826566440e-03 5.833669341505141048e-01 7.199749932777611550e-01 6.296021683996023688e-01 4.501544148069071966e-02 2.717563301364394127e-02 4.688938148422963637e-02 7.650903402928181662e-02 7.336139198860071320e-02 2.571639852771645876e-02 1.644142236530906492e-02 2.934160214166462236e-03 1.738200555786139495e-02 4.073532173814031244e-03 1.257371935392454822e-02 7.503634226006763441e-01 5.802121403885844853e-01 8.093553639269155386e-01 1.275529159486648093e-02 1.548679012148227374e-03 2.095724780171282289e-02 1.550243657003091621e-02 3.619983447150847827e-02 4.576826172957339928e-02 7.272938778355216016e-02 2.242031528415055000e-02 4.909862823879232452e-02 3.614645671570856922e-02 5.162108940641335630e-02 4.457410241502923198e-01 5.065378236990756289e-01 4.393039514676789214e-01 1.889171957388581166e-02 1.362108350672747679e-02 2.131940956620563890e-02 4.587368797754637351e-02 4.351649435690582018e-02 1.739217108912137835e-02 1.104244014309507388e-01 4.945582212066688271e-02 8.867093180654021800e-02 3.012348793675821007e-02
2.533592644259836657e-01 2.553599956211131256e-02 5.321990141988633925e-03 5.295650356751768588e-04 5.986537847082982073e-05 1.960019726520434869e-03 2.185189138630786940e-02 2.166621285757096293e-04 2.129478039019548630e-03 2.172374320132737947e-03 4.313298955822816166e-02 1.060099557878075617e-02 2.629086831265948013e-02 4.724567800324198930e-02 5.208592799180020855e-02 1.381026331057686254e-02 7.552894875067357218e-03 2.861173822917082582e-02 2.027744914936796886e-02 2.242500023068447201e-02 -4.549743434796802571e-02 -6.913654747080663632e-02 -7.584380913605207053e-02 2.410851804265377180e-02 1.254339527633673568e-01 2.797598497074951784e-02 2.054405771385970497e-02 2.651606445106715476e-02 2.357219439608319173e-02 3.001611465973051673e-02 9.600752492114039693e-02 8.125993441547517404e-02 1.312940867226065067e-01 5.303252661334359463e-02 6.685294108526042278e-02 8.397220877128079575e-02 4.886767601706265562e-02 3.213184555890227084e-02 1.262365388343963390e-01 4.770932759937540624e-02 1.203257104567246863e-16 -3.998668245961029682e-03 -2.667519577498181360e-03 1.959496579779769487e-02 1.643695808095174682e-02 4.466245746357232205e-02 7.200864619556542057e-02 2.807574120367283682e-02 2.662802834942383165e-02 1.261117524080505349e-02 3.132142571994134955e-02 4.726691026037670190e-05 5.010697718634535280e-03 7.785078746213775630e-03 2.573515297235982879e-02 3.008891699520493696e-03 -3.057873909156652026e-01 -1.596409989527295958e-01 -2.227462397317627463e-01 6.484304471318790009e-02 -2.221532514042066919e-01 -7.870401773526275110e-03 1.688223030080041953e-03 3.126935925110358588e-02 2.280063588952969261e-02 -2.922067131203631707e-03 3.961613146272969371e-02 2.793437008568960475e-02 -4.286831582874880801e-03 1.640781740218244175e-02 4.865538945154226881e-07 -8.601729304740310269e-17 2.115953285740228978e-02 -1.710753360585652236e-02 2.290247457774205464e-02 1.005671225793822697e-02 3.395288791544900237e-02 1.063425817936248900e-02 3.298235066413917477e-02 -1.567077753691722593e-17 2.246365019828629184e-02 -3.433556480625185592e-02 -5.370365802530310123e-02 3.681118455245739607e-02 1.145502320544877911e-01 1.027503622294175245e-01 1.405883510372460225e-02 -8.347428685239221754e-03 2.231398931413546005e-03 2.584069360523888448e-04 -4.112808029438070820e-03 -6.007824279196400524e-03 -1.938915332914743287e-16 -2.100194250560019211e-17 -9.320358763961965028e-04 8.177088847792514034e-03 3.734610662985133782e-18 -2.403638053949620074e-02 2.063590092066811769e-03 8.024967341944521781e-04 -3.401799942495414841e-02 1.332736677866614740e-02 1.321794520261239565e-02 9.536671560219069088e-02 1.832996026055423028e-02 -7.749149127893620488e-03 5.598493805238456271e-17 -1.741374207343050458e-03 -4.910061634557634155e-03 1.200226466075218247e-02 -1.529172853542147895e-02 7.356285448533513883e-03 1.114199251133910558e-04 -3.044064283725539700e-18 9.970901427182640286e-04 3.078088699208729635e-03 -5.368729523836144463e-02 -2.123713888782430628e-02 -2.168197071550424515e-01 7.096605243814588238e-02 6.179295400284569029e-01 3.983196140671616536e-01 3.465528416655631189e-02 2.281956182567308842e-03 6.711171009536242606e-03 -1.242459957106314614e-02 1.813764263850590683e-02 9.299986094712612422e-03 2.560177015402870931e-02 8.942562464164735547e-03 5.869529939704642291e-02 2.581789124094232765e-02 3.414044523914336327e-02 3.320753271997738815e-02 3.102368585180088553e-02 1.976008121600450473e-02 2.532827059783128096e-02 4.743576673112423654e-03 2.548653872628354328e-02 1.961603618534078775e-02 2.529279808952161757e-01 7.267148483370545753e-01 4.008239403879849672e-01 2.097114990856195627e-02 4.614379896003513448e-02 1.575144889269087384e-02 4.096325104959209346e-02 2.681974350473078500e-02 3.825394892136243369e-02 3.280942786732107819e-02 5.938318712364072660e-02 5.855897784452273463e-02 6.365117933266435657e-02 5.156757411219024118e-02 3.984675741285473305e-02 7.811230855187464850e-03 3.956983845453505844e-02 4.203118946791591093e-02 5.659276539695937136e-02 -1.579369815327218846e-02 7.429295895023782341e-03 6.913973132100560415e-01 7.128539709226769938e-01 6.381706597843778139e-01 1.661281585912306041e-02 1.428298802356193185e-02 3.894303069854621047e-02 9.994533846057600604e-03 1.611977816350116527e-02 1.306697979367117506e-02 2.769662249447531893e-02 1.118198573659561108e-02 2.062576113805430350e-02 1.271831732653795231e-03 3.674354355066065947e-02 6.221446612342765737e-04 1.279126690452821180e-01 3.473126116473805314e-01 3.260159189567694571e-01 8.248683539165115874e-01
1.686923994219082391e-03 3.330505225766986627e-02 1.974874446990885005e-02 1.691466639228521437e-03 2.206814081963065152e-02 -2.751369056511906303e-02 -4.229038068502243702e-02 1.558891574719367158e-02 3.177549480426503786e-02 4.681150920424034106e-02 7.516681943712789915e-02 7.767669827563935397e-03 1.056071919526289753e-02 3.458416026633998053e-02 2.348468710906314455e-02 3.626243260710425992e-02 6.078154442676876601e-02 7.018309203241862929e-03 6.596693712567847437e-03 2.332787500039204542e-02 1.361659180560707283e-01 3.966790136219113666e-02 3.468695819496701860e-02 2.026878824122312420e-02 1.310590629807300775e-02 7.065839928723838259e-02 5.655246969683452168e-02 1.013114649529299818e-02 1.265238780451776734e-01 6.327352748691620482e-03 9.304366388585387504e-03 5.824029216256714631e-02 1.791067028249645476e-01 3.662583743678952952e-02 1.077104520735265754e-02 5.671175872094438419e-04 5.771382704840097994e-03 6.545780501127748847e-03 1.354785965587359912e-02 4.447735560145905757e-03 1.516487577478085706e-02 2.038281730227692323e-02 1.519027989067351855e-02 2.355890072703935995e-03 8.275296663096344887e-03 8.118141368667678576e-03 1.324731386300871932e-03 3.902009034688882355e-02 6.986211219336120604e-02 7.860444341461297890e-02 2.061233843981716968e-02 1.982115594314623995e-03 1.169366394646842618e-02 2.258565418029073041e-02 3.159291999137658902e-02 1.620024266983905240e-02 6.570721963180538305e-02 4.653156781779665963e-02 5.015271744022483519e-02 2.704401590576163000e-02 5.022245414269942837e-02 1.768280838219435380e-02 2.030898699509709193e-01 4.074315236620165703e-02 -6.049358017743133170e-02 -4.677495218369266683e-02 2.026817038708219124e-02 -3.028714948105229801e-04 4.567188517741316289e-02 -2.063614514499030259e-02 -1.228196830934022779e-02 9.268369366734754099e-05 1.603414596854182885e-03 3.878850072677473421e-02 -4.575523708113434819e-03 -4.184870026358752894e-04 -3.408578304354128053e-03 9.585378306481602551e-03 2.461941124477921503e-02 3.016794285250768912e-03 -6.469117459829415875e-06 -1.032828769753386597e-02 5.823925931627114971e-02 -2.160059119604772204e-02 2.798635851861151422e-02 2.087830651916056449e-02 2.898233485480712635e-02 -3.993642222119761254e-14 -2.901585260061408447e-02 -3.818274081306758522e-02 1.055305544622550940e-01 1.902324824876495610e-02 2.574685075045845747e-02 -3.974026834310192026e-18 4.390783743857885613e-01 -4.152538652215606585e-02 -4.958042119711181250e-02 1.385082218656705525e-03 1.640370768917109173e-02 -2.397141926245867751e-04 8.467246605227614006e-03 6.280228161359321416e-03 -1.260099017525149502e-03 1.591434369434523849e-02 -4.571817924351980963e-03 1.374102734088459080e-02 -7.072182128497125908e-03 2.650595882361609507e-02 -2.321297634983276841e-03 -5.806714357313443953e-04 8.711671051328338800e-02 5.658053462390751250e-05 1.363447626787101112e-17 -4.911851320950231387e-02 2.218654571500096664e-02 -2.582340338443111019e-02 6.885196852976475934e-02 1.548813429245061470e-02 2.443939267239224333e-02 -3.331559046349711175e-04 2.273529497995008808e-01 5.234312235166984489e-02 9.938844153642499568e-02 1.832984871739374300e-01 4.441558465553644974e-01 1.004910407097634906e+00 5.384264319880817107e-01 1.247545804154005095e-02 1.274516509705991667e-02 1.299724233972600167e-02 3.205807542946285948e-02 2.021084482727514980e-02 2.100310498450240482e-02 1.499315346571843469e-02 2.877918846385251958e-02 3.797704838875663735e-02 7.411775366780312679e-02 8.862026003260874488e-03 1.078221972961227598e-02 1.739563504770451352e-02 4.953835952326992448e-02 3.304999544650709387e-02 4.394057740789981265e-02 2.382973971924155776e-02 1.588896068710299314e-02 6.146593249426502981e-01 6.482664149024869316e-01 5.659811781899876104e-01 3.529483077027943949e-01 6.522053119019693979e-01 3.129653584731806326e-01 5.943269201926615342e-01 6.036761620108089810e-01 7.524536208782824920e-01 1.685014267946445196e-02 6.688061851428263569e-18 1.501093258712415945e-02 1.156108912178028560e-02 3.235608819137142378e-02 2.756316505132468467e-02 3.942114372914659604e-02 3.488221073227546626e-02 4.329543745452766462e-02 1.688961638823139461e-02 5.279765742704690956e-02 9.352763384006045067e-02 1.180444838743562319e-01 4.383845824892067533e-01 5.413946796824861130e-01 4.266939344984658766e-01 1.984388075102673510e-02 7.198240614056416672e-03 2.727014579622362087e-02 5.147363291572052552e-02 4.698928183103891210e-02 3.977193628109448875e-02 1.324510850699861519e-01 4.271035017154990038e-02 8.824203001235272903e-02 3.061872804996013803e-02
1.383997195335673613e-01 -3.999595297747488704e-02 8.053807022066969412e-02 5.526572006503942258e-02 3.086604374880275517e-02 2.841286776496573449e-02 5.110914507061650042e-02 3.223054084969941002e-02 8.035329065739794940e-02 4.067510498370126987e-02 1.760027947693187100e-01 2.198524456063923743e-02 1.267134882659230397e-02 2.446671359737810983e-02 3.316601615734016822e-02 2.816830735919514939e-02 -2.091844224096548605e-02 -4.476703837786152190e-02 -6.803314426872790477e-02 1.646615906627919260e-04 4.706255401348104911e-02 1.079817208556361520e-02 2.187733494467120066e-02 2.099081361404784088e-02 2.883497430759555946e-02 6.122487528820965558e-02 1.702808326876731060e-01 3.202393104087522535e-02 6.008203768038118675e-02 1.122636648041804540e-02 9.346790239016619650e-03 1.133691576972330368e-02 -1.225468048403934029e-02 -2.889042882307909718e-02 -6.075103094099252449e-02 1.063624678881735716e-02 7.223075794371857814e-04 9.645972929420468592e-03 1.144890204484940235e-02 1.713822838669382814e-02 2.489283591436858983e-02 3.181580753711692189e-02 2.213193082343386034e-02 8.982483381302175038e-03 1.642809790921957377e-02 2.565435262047992618e-02 6.003432948167139460e-02 1.570167955870867271e-02 1.093575761601832397e-02 1.213345642535160022e-02 3.857640467510847770e-02 3.834945950454210755e-03 1.916417509681556802e-02 6.711262638864293889e-03 2.337058116602653418e-02 -1.675845214393486063e-02 -2.623541444059120098e-02 -5.573033808217561086e-02 7.182588054679887035e-01 1.327582158116567823e-01 -1.073980544649426883e-01 3.525341062549672885e-03 8.295146931733779483e-02 2.959663618808558092e-02 2.830545110649983209e-02 3.920372057769157063e-18 -1.265384949077461192e-03 1.158426758445415053e-02 5.605344196834881332e-02 1.847979172379961136e-04 2.782748090002131434e-03 1.631870238515688992e-02 3.414958661901203215e-18 -1.677209963369682977e-02 -1.849172905504406494e-02 -1.367478820213003034e-17 1.902865800765882881e-02 -1.653493368725051163e-02 1.577988305154934962e-03 -2.308441008860713864e-03 -2.806582935596753890e-03 2.687735702259996806e-02 1.987963370288604870e-02 -2.504532903999547814e-03 1.490986551336636635e-03 -3.881146638569854965e-18 -4.396602188582287831e-05 -2.272345489376102409e-02 -4.552180194539609866e-17 3.077357042172428406e-02 -5.068494646043147922e-02 -2.884120748672177117e-02 5.636287978720479902e-04 4.741886941011841210e-03 1.826313044442284123e-02 -9.875796107757046287e-05 -1.666179331795756072e-02 -2.221957498354997992e-03 1.045152464748861395e-04 -1.090310128863840149e-02 2.267810140784191041e-03 -3.784161303339619560e-04 -1.966770583504548128e-02 -2.079929455344427854e-03 -1.386795338381841122e-03 1.934512552134885419e-02 -9.177673390368479636e-04 -3.620568433677016849e-18 -1.822815784732756406e-03 9.010455047214848247e-04 3.421992242185018570e-03 1.024275197667273230e-03 1.046851762424548715e-17 1.610153622296413983e-02 2.005038294637999027e-02 -6.361350974623773691e-02 -6.060746950616412165e-02 3.468819084287263349e-02 2.021196897694258268e-02 2.303799387412146515e-01 6.209156361056721307e-01 4.224052423008884949e-01 1.115191514555067057e-02 4.502141920645726623e-03 3.005394788818587307e-02 8.632623100064136523e-02 7.508531423727343723e-02 1.488242291491961161e-02 1.962852841644488477e-02 3.446764594746497962e-02 3.891814939681857016e-02 2.045798499806151041e-02 4.240565983412841744e-02 9.665158729164673168e-03 2.848761495054547010e-02 1.006680437461083058e-02 2.646524283373409836e-01 7.396727433239086436e-01 3.865292048257173430e-01 1.219367508916088978e-02 2.997899349253980300e-02 1.929615979002381049e-02 5.430421599184306197e-02 5.881170487900969773e-02 3.800624526956828536e-02 4.232764812620825462e-02 6.889709667711597929e-02 5.340421819547211055e-03 1.161687892973645164e-02 2.715059454495728514e-03 1.788623501665592697e-02 4.221883838204753929e-03 6.431139527002968093e-01 1.105104144893913842e+00 3.808544798693674349e-01 3.445440595505540630e-04 8.223981257782847151e-03 1.157391629914842528e-03 1.957921178567338313e-02 1.809517057879058907e-02 3.063025065430733176e-02 3.131558251098921081e-02 3.855791818269799914e-02 4.755748000344876651e-03 3.678784890084502851e-02 5.694686132119838179e-02 6.576628741229177677e-02 2.781823388485070089e-02 2.201593180525188848e-02 2.601450700681174483e-02 3.465193373695047568e-02 4.451359570846739194e-02 2.749030422825081768e-02 1.184624795065916496e-02 3.618402394650939435e-02 2.238135836490178499e-01 3.136480875770871757e-01 4.979794363272810331e-01 -4.126690849916128667e-01 -7.657273264950733571e-01
1.199930261867044740e-01 -5.676002024063069723e-02 8.202341228789206129e-02 2.186287981280770135e-02 2.173883093126856292e-02 3.769857746165734824e-02 6.468982744910206639e-02 1.629263459550839288e-02 3.827652277331471120e-02 4.140348678836138635e-02 8.266328698256197871e-02 2.083429036136290671e-02 1.097483380121094462e-02 4.302474610578168462e-02 3.230834230957205927e-02 4.866509608788972818e-02 5.087391179578342609e-02 1.488203572083573144e-02 -5.978000456126433939e-17 -6.276551910771689669e-02 -2.895390342158830899e-02 2.928429448450223758e-03 2.448605105711926591e-02 2.427978980803474629e-02 1.570963288323999271e-02 5.012564692632535956e-02 1.170329588187448144e-01 1.552268080189657916e-02 8.596305792039019705e-03 1.490116120028543090e-08 1.085830118047040153e-02 3.282114318469414940e-03 6.325554827956324880e-14 1.820884628478481649e-04 9.778975131446833533e-04 4.193604322962591648e-03 3.042108248825658610e-04 2.321917035080632141e-03 2.191865568827502941e-02 9.461286300702561117e-03 1.146568316558982219e-02 3.658290704341679578e-02 -2.958273379175911243e-04 -5.019224965476427186e-03 1.743713094735081098e-02 2.823229739439844441e-02 3.601929080029885044e-02 5.619915165440218232e-04 4.178331262355976933e-02 -6.985398700035020487e-04 -1.537933494903761210e-03 5.331146796126481728e-03 1.403707857912819650e-02 1.047610748891938427e-02 2.347489263785337843e-02 1.003272703950613444e-02 5.402244301209724320e-02 4.173327966918070203e-02 4.356993948031689240e-02 1.623879494196801485e-02 -8.919957526716806862e-02 7.345473316872962177e-02 2.678759185699193177e-02 -4.233212104490244321e-02 1.728376371334549438e-02 -1.331673330647069851e-02 -1.584078792854014302e-05 -1.904983492230646544e-02 3.320009156801685679e-02 9.546268224883360312e-03 -4.440215179595450559e-04 5.122494927935451747e-02 3.604438301649553339e-02 -8.375815386651698502e-03 1.650207184467306770e-02 -2.648283911416697746e-15 -1.258953857345313883e-17 -4.042357885596446765e-02 -3.810529555220940057e-02 4.570526379297167562e-04 -6.749939108303719404e-02 1.250074941876478871e-02 4.841961631480256462e-02 -1.364415038660868502e-02 3.259856528844364693e-02 1.065112617432922848e-02 1.880368904190994617e-02 1.305597908437803201e-03 1.704011407684825419e-03 -1.490116119384852975e-08 -1.726313767030748748e-03 3.836813588526379222e-03 2.513772955072780601e-05 1.232225485645371821e-02 -7.477521838335256786e-05 3.606065349198353583e-06 -3.325408816548275793e-04 -5.148579339292032537e-04 1.946418229256369897e-02 -2.798506682707424142e-17 8.792516073468007289e-04 7.142624619574531632e-03 6.747710578921111216e-03 1.981669361293331155e-02 1.151366086778012748e-03 5.159735779964119695e-02 6.702669578176977117e-02 2.668487962372928079e-03 -2.743326414746868717e-03 -3.606395793980764035e-03 1.505054598929715937e-01 5.107563075765982107e-04 5.366632438960896451e-02 -4.397135758579817817e-03 9.190733566665863080e-03 -1.772037219967326910e-02 2.295445687678836569e-02 9.793297673372922899e-18 -3.617314837869256994e-03 -5.364080324441331460e-04 6.073091115443406407e-01 3.549001868608592791e-01 2.808061212097507503e-02 8.615264067316384319e-03 2.054323337545966438e-02 3.645607414721187267e-02 7.761483657312912010e-02 2.313422626244574735e-03 1.522461488901418261e-02 3.205521190189643960e-02 3.493371319296936089e-02 1.163505880648899732e-02 2.002882196589615693e-02 1.221305403937383849e-02 3.856814054266487907e-02 3.250808812736351999e-02 6.093594479776593548e-02 1.195271723739874115e-02 4.671045492398996912e-01 7.760394841767852547e-01 6.011624897551205438e-01 2.393531265674968961e-03 4.797737254624069581e-02 2.145757960250225041e-02 3.488705817598886788e-02 4.346390147799785580e-02 7.307645543185252446e-02 7.882126847030525296e-03 1.297142421438767478e-02 2.406422453814300449e-03 1.781100220906970885e-02 2.060958247125403361e-03 1.457634868521657037e-02 3.250936782768846112e-03 2.276630991212713617e-02 5.029779835474733514e-03 9.421903770018547758e-03 8.741048617748600280e-04 2.993718407304842705e-02 2.987959640047626545e-04 1.689932847845850364e-02 6.179570641586317192e-01 7.033748002860297222e-01 5.699537425397404045e-01 3.688449207650246892e-02 5.034284669362331516e-02 5.138570346437171948e-02 3.975560377312831184e-02 3.093144384762313570e-01 4.199819680656682319e-01 1.528039815296109716e-01 7.936329876186363694e-03 1.937567563192892431e-02 8.231872237405171364e-03 4.702825634680274369e-02 2.713538598097564244e-02 1.257530420783599945e-01 1.201807094668339987e-01 8.462777258886307330e-02 4.647567644624101685e-02
1.322370687422109725e-01 -7.659333067432133602e-02 9.253814197860975310e-02 2.620668157172293886e-02 4.417364726671311331e-02 6.214614675634148183e-02 8.530635447387853454e-02 6.857897926233206458e-05 1.989097406746597949e-02 4.114339794464418748e-02 6.676531909762348227e-02 3.527813236294034815e-02 1.634666336758947014e-02 2.564229345667971144e-02 2.689538727956735167e-02 3.256316711166085121e-02 5.605097990511996586e-02 1.135679796647686389e-02 1.145216816259101572e-02 1.082665766925002687e-02 5.024814274927513758e-02 9.674282023703097838e-03 7.220435856548070019e-02 7.864508838186515249e-02 4.964599263723364769e-02 8.841889290576283866e-03 3.805181491209808087e-02 8.565599488834469560e-03 1.507903967772934468e-17 1.849135782559727798e-02 1.196411240732879813e-02 8.466085104877080908e-03 2.736130756087121869e-04 1.374165014999164346e-03 1.088744713868274733e-02 3.527848110263794718e-03 -4.030445210722840427e-02 1.227984030671855353e-03 -2.913372354265708619e-02 -5.373814562422689345e-02 -1.002932946801531229e-02 7.555431785054401994e-02 4.292865299669907514e-02 1.102262226380399009e-02 3.010705866955714141e-02 2.588493851700421344e-02 -4.154038508194179968e-02 -1.904912064139059223e-03 -1.952834206297177078e-02 7.159659945720291130e-02 3.847308486195843980e-02 1.195005452271709085e-03 1.167524511972295280e-02 2.311051353134765951e-02 3.765640274538935989e-02 4.028649155354859651e-02 5.794311602748723755e-02 6.584941885151743279e-02 6.581902169892386856e-02 2.005878196390070700e-02 -8.484523451246396564e-02 2.320894790327585402e-02 -2.814085009706552773e-02 -1.773327385377450110e-03 1.628155815401836556e-02 -1.330278258796622169e-02 -1.143400306024336701e-18 -1.945621791379287951e-02 3.634669967155999132e-02 2.160115249301628598e-02 -1.513262487094561154e-02 3.016639657484448955e-02 6.290336924908714392e-03 2.069141163174970533e-03 5.897900974211538021e-03 -2.069691157419055461e-03 -8.433885485257886297e-03 4.384702431211729458e-18 -4.450507939256260342e-02 3.513973656032967790e-17 -1.911714134130404821e-02 3.993872967682743358e-02 1.060383144626728807e-03 2.636737385563481251e-02 -8.252047490611719854e-03 1.290675237683228116e-02 8.942887907578170661e-03 7.987897397280702139e-02 1.384856639101473630e-03 2.184695761359018232e-02 8.630755318601142178e-06 2.013407978490238184e-02 -1.273733248819135418e-03 3.154526591065021091e-02 -3.124881472766849819e-05 1.125678053350942856e-02 -6.541705014250717883e-03 1.674508105813788241e-04 -5.271354567245328804e-02 -3.586593860810734224e-02 -3.182452481002711296e-03 -3.349877009500659177e-03 9.176926681491317335e-04 -5.481183529094777601e-02 3.331200661654069861e-02 6.722591958157106591e-03 5.000228466131494132e-02 -6.066919270931323688e-02 -8.457133057743650498e-03 2.195146911804232714e-02 -1.631349801597441543e-02 -1.043548452684627155e-02 2.749280409818042475e-02 -4.840473689670767715e-02 2.654127175882209366e-02 -2.013468405247088866e-02 6.916295224126006724e-02 3.828394602292246288e-04 2.511319696088355757e-02 -1.219799954674481226e-18 6.452221060365271876e-01 3.631373218705487571e-01 4.368894713454829493e-02 2.782930339012806842e-02 2.500860624416736261e-02 7.416654342310510795e-02 7.731983471474587755e-02 1.867067107890130542e-02 3.174877950965412921e-02 2.055398906069468706e-02 3.518532036993058526e-02 1.717843756703218983e-02 1.596547394644438206e-02 1.596980854708222528e-02 3.136817243091442192e-02 2.726476310441274323e-02 6.521048963710539270e-02 1.086625042738622454e-02 2.509433978189702999e-02 2.511293940871402522e-02 2.295029558894493699e-02 4.437329536295187352e-03 5.623982182428832743e-01 9.704806453991571269e-01 6.504629985958855221e-01 1.356761392924686963e-02 6.372605771319955759e-02 2.246007126126681533e-02 3.105787156127985671e-02 7.400238897688080075e-03 1.097499828686310068e-02 4.197295755125268930e-03 1.049542847869213737e-02 4.764031156243953707e-03 1.276714573843897201e-02 7.878344193038293482e-05 4.598479782253231743e-03 9.397350233704740083e-03 6.414655416954565537e-01 1.060017844396179854e+00 5.178209489107914454e-01 -8.910664692183563329e-02 4.024821360124512803e-02 5.664626508149445128e-03 1.744976772308296994e-02 -5.273529877603658211e-02 5.216688000322494689e-01 8.699373069830038663e-01 4.807242668695659304e-01 1.792758834634790646e-02 1.387710471046610904e-02 1.202711351380358548e-02 2.216180536979447252e-02 3.452274774340787639e-02 5.509499393222324487e-02 2.330538252612750563e-02 1.258481894501613796e-01 4.972463381275196714e-02 9.693363642205805875e-02 5.068694422270490774e-02
2.421455409513489088e-01 1.749319482011887222e-02 1.963268340083065226e-03 9.953526073451611847e-03 1.000776848794975672e-06 1.360298149963413278e-02 5.894922407409486712e-03 8.426854259645594203e-03 2.623242349212996649e-02 6.224661821552793066e-03 5.205717459882047704e-02 7.834106488325645948e-03 1.460305160676350113e-02 3.146240665838230532e-02 7.567704598353329482e-02 1.285318931503738951e-02 1.513137516914967416e-02 2.341309118812380866e-02 1.909112880139053961e-02 3.717684461435089016e-02 3.785023815760617943e-02 1.147495134350020630e-02 4.253472096427653171e-02 -6.291170741491682039e-02 -4.110735096181351855e-03 4.662712770286618624e-02 1.854216875095303016e-02 4.105990300710436747e-02 1.073880229626416605e-01 9.221184052232300754e-02 2.029096883228190540e-01 1.192561146015784224e-01 1.306731215746532726e-01 9.169915094758435137e-02 7.802305360134231815e-02 1.318536537662162722e-02 7.364895439747548914e-04 4.194959586644705918e-02 -3.105497010737480468e-02 2.020471162393840808e-03 5.152590068872694995e-02 2.555055581470794257e-01 3.182700450095929945e-01 5.165772200066759412e-01 3.125967731916355907e-01 3.591573111359235848e-01 4.313580863181193381e-01 4.890058218851442429e-02 5.375972249274020565e-02 2.618437419240323208e-02 2.539323900307976362e-02 1.557292756690097611e-02 3.104764517992091369e-02 6.578091930766004521e-03 2.983925229671738691e-02 3.672857944816999842e-02 5.450937256691688187e-02 1.233840525327320747e-01 1.897466737731462183e-01 2.012237353484708224e-02 -2.343287009939905574e-01 -1.653257074180744003e-03 -2.584662440741457637e-02 1.404161334728156185e-01 2.336613000519812203e-02 3.671920373068147253e-02 -3.333798911603237571e-03 7.603477933655840336e-04 1.093200961536939034e-02 -1.748361093149121209e-04 1.363467816361669000e-17 5.450856568921376433e-02 2.704610409145585814e-02 -1.059683962108228308e-02 -3.677905973507483279e-17 1.610023136927969320e-02 -5.697141239974597886e-04 2.307206925337495704e-02 3.079877841992516470e-03 -8.297305909225149642e-03 -4.100277338229580703e-11 9.468267785571048145e-02 -3.498460061899567874e-02 -4.496924703420106945e-02 -7.898831285119385648e-02 -2.750071716639719901e-02 -1.637711685714069972e-04 -1.023844710796578857e-02 1.250235532162700947e-02 -7.354319584471205209e-06 4.277533763019149282e-02 -1.850720272868744409e-02 -2.085349741414056681e-03 -1.679847756368738846e-03 -1.437921856735878141e-04 6.113315645055296746e-03 4.119956233907325541e-08 -2.219030997559969312e-02 3.870113733753736962e-02 2.446250162761310909e-01 -9.074698136923503577e-02 5.355656318914629038e-02 -5.789305545200326697e-02 2.143608613959548798e-02 -4.224718956692684324e-02 -4.783106085006612329e-02 -1.362290074223882451e-02 6.026451254177805222e-02 -1.170307445013088216e-03 -8.106371985660246367e-04 8.012530980090366850e-02 -2.347545876460217784e-02 4.077857052506299491e-03 -3.025206235566325896e-02 2.226521762086837788e-02 -2.500932700524206506e-02 2.928367445033684680e-02 7.594125860084799569e-03 1.304537807115481729e-01 3.314574690157166045e-03 6.187606061057813189e-01 3.658881484597627520e-01 2.656909728691355527e-02 3.087479722450057530e-04 7.790281898945732551e-03 3.874504421248559441e-03 2.006020902815924822e-02 1.217636487110566011e-02 1.525719600111416670e-02 1.048763905043591758e-02 4.512936104770670642e-02 3.828907119489404626e-02 3.339290159302156502e-02 1.205730604203198407e-02 3.059481713176658707e-02 6.413860269919268971e-04 2.680019395074138139e-02 1.599359293237602453e-02 3.270445187651726399e-02 4.040934749662028480e-02 7.340834134390730736e-02 2.592581897118832052e-02 4.571713009698364361e-01 7.767771270785168714e-01 5.961766509779434786e-01 3.480364094163926336e-04 2.763888177083724182e-02 8.724513291530863399e-03 2.635340439402087537e-02 3.750993038150104292e-02 6.281371053396359039e-02 8.932629846431250353e-02 7.913572948193425827e-02 5.988808760972704620e-02 4.727357914418942042e-02 1.145604949805073894e-02 1.786408177699876781e-02 5.785911055373960110e-01 7.824957875733937751e-01 8.691015071184460261e-01 2.607401783619087698e-01 1.321705040874109338e-01 3.161582443178845986e-02 4.237366139333189480e-02 7.223305075386418639e-02 2.983366535526586500e-03 7.029391449358182026e-03 3.911303134910361079e-01 4.503150240366980972e-01 3.465380195052682466e-01 1.758393329636883870e-02 2.414628951670474977e-02 1.573462216063162278e-02 3.478062948340897298e-02 4.180300729175941837e-02 3.094699379271921916e-02 9.188202275759334292e-02 6.059916749316606593e-02 8.876664101139715435e-02 3.776894403566025821e-02
1.366255291809163330e-01 -7.112197825470312207e-02 4.559544499800898681e-02 2.389451143137871422e-02 2.179460251360597345e-02 3.736748173519262306e-02 1.037514416248885019e-01 1.430070932515998573e-02 4.588073242953271541e-02 4.134340700163428167e-02 5.619419602139213987e-02 3.447788781616224763e-02 1.463901223943437323e-02 3.268653447841207704e-02 5.108795680078296458e-02 1.371861538475389886e-02 4.016931086801720646e-02 -1.946634886364971284e-01 -4.917335935183365231e-02 -1.551969933924089995e-01 -3.282638578272469021e-02 2.837138270180119930e-03 1.727512383850834596e-02 1.876619733804686316e-02 2.068980650518104439e-02 6.164696144481430183e-03 3.230274212942627760e-02 6.708268444626148269e-03 2.305267536973849493e-02 6.127288579806767534e-03 6.296806028874265887e-04 3.364768613119696258e-03 4.159517182240691884e-03 1.548023179849023628e-02 3.783380954454958200e-02 -3.338860736106127325e-01 -1.679778961398830928e-01 -4.598291251869991991e-01 4.776650396305207047e-02 2.142092164507438182e-02 3.245680079911013316e-02 2.003370028637014091e-02 3.937520882721202986e-02 1.443215822921105514e-02 2.871469068173402445e-02 5.023632103748362082e-02 5.925687723854253242e-02 1.718801562927480334e-02 2.692533705752343798e-02 1.415331678091792146e-02 3.307002850058254895e-02 1.023774306005519746e-02 1.348453789578202408e-03 2.427880205646164649e-02 4.218450141604808412e-02 3.042711191946539592e-02 -1.860413163108182721e-01 -1.134762701647547417e-01 -1.861605868495448679e-01 9.666000689420341241e-02 -6.247799188722116120e-02 -3.494701634435208593e-03 1.220361063070197170e-02 1.814987451517272032e-02 1.780910476137085099e-02 -2.651659851288941383e-02 2.542897563141942492e-03 2.203456156213007644e-18 6.598375912979977109e-02 5.767943958731556325e-02 3.273927695713328787e-03 -8.342542839283972715e-04 6.276277546890700787e-18 5.671388612860628342e-02 -1.086923940173844771e-06 2.445899296131011369e-02 -5.171931772134365224e-02 1.203390895623504681e-01 4.423555763615704545e-02 1.139605315573899089e-02 -1.128990041527605515e-02 1.715507006612745911e-02 3.957594204686919886e-02 -1.467762496104630353e-02 1.133508229752948522e-02 -6.968386096695060837e-03 -7.182206394244938135e-04 3.386464976568788476e-02 -5.848236257481168911e-03 2.161072502094638081e-02 4.215045276085623710e-04 -2.118944913917902850e-02 9.505897969909901657e-03 6.777998865594746236e-02 -6.455426967542767479e-03 -5.754171473148515575e-04 7.303013314686491834e-03 -8.879286305910935595e-03 1.530905896855216984e-03 -3.579865228760027335e-17 8.358669515060548894e-03 -5.942085265635587039e-03 1.585818204054907860e-03 -1.959216357514863369e-03 1.840285956101936696e-02 -6.916739167520544904e-16 -1.076350564174973108e-02 -1.127358980383697147e-03 -1.167920761328108958e-02 -1.150379353648850207e-03 -2.295095834630865215e-04 1.716367479310108163e-02 1.265967396578871959e-03 -5.823002592479228339e-17 6.242550407902651465e-04 -1.950964514098283212e-02 6.232785640462176584e-03 9.200754997033738780e-03 -1.726941036912834349e-01 1.861486527644859723e-01 6.720757807707239406e-01 3.711212097228633877e-01 3.588222483113702455e-02 3.558762554089448665e-02 2.366820272280375365e-02 5.914440357674561471e-02 6.619750131281837424e-02 1.354598973195178177e-02 1.591818754539551031e-02 3.045356679229990066e-02 4.189191921281815861e-02 1.586246790101687384e-02 2.577813852048865123e-02 3.514322661441158374e-02 3.546719373559593935e-02 4.644958266994716339e-02 3.525094487252951325e-02 2.781578361112456155e-01 7.848647193876520101e-01 9.698583742635372706e-01 7.903739041774668350e-02 1.937166962506098463e-04 4.129234434030627093e-02 2.559517528791608459e-02 3.475256300894559447e-02 4.488877904722290285e-02 5.417733946449326821e-02 6.667759216865987507e-03 2.295583481285017394e-02 9.240565087206125197e-03 2.207352760840546047e-02 1.418384430736617692e-02 1.245611517763553153e-02 7.994001254168079523e-03 2.498685750867457589e-02 7.062515380767222517e-01 7.608122656652915872e-01 6.811744067640939093e-01 2.743179129228344795e-02 1.895841650178926516e-02 4.552371891232758622e-02 3.960346445258056819e-02 4.813360293511814714e-02 3.301021162927267067e-03 2.758132564888917532e-02 4.765630599525590150e-02 5.927938803279642760e-02 1.354278550271078956e-02 1.669313412121774601e-02 1.182856923928281637e-02 2.044062588851254872e-02 -1.399734411477288358e-03 5.293699145805061401e-03 4.166797490587392100e-02 4.919793615867391889e-02 2.655013828223772645e-05 1.236005291714850074e-01 4.611133811995468124e-01 4.079834154912826971e-01 7.629208492396780672e-01
1.293179054155886298e-01 -4.707350296491551178e-02 7.612817688450687625e-02 2.744791744631456046e-02 2.589263617795349234e-02 3.016197215308846075e-02 9.103690188615974621e-02 8.999764214484977680e-04 2.297287199486895692e-02 3.620989847646587101e-02 4.435099653502442424e-02 1.092544692233385697e-02 1.093458800644093488e-02 2.257488680633634379e-02 2.291899545781635530e-02 4.253548752571736924e-02 5.522557421719859194e-02 1.315400342142935934e-02 2.167199496407651305e-02 4.998065486866228729e-02 1.262660349643175417e-01 1.844148683076148737e-02 2.033107207975373004e-02 3.547670775855865655e-02 -2.153923566834321963e-03 1.867029423975300959e-02 -3.206118602337806878e-02 8.516023945852565277e-04 2.454783931084827259e-02 5.172300402339866619e-03 2.196638594721657400e-02 1.227027525673456913e-03 1.360582623116698259e-04 1.309646379166187063e-02 2.528909329131705935e-02 -3.107346344816160899e-01 -2.752148594201738185e-01 -2.589466434644867587e-01 5.124035004787982905e-02 2.307991669696631692e-02 4.070342717696005114e-02 3.463110231992558297e-02 4.190872221475114290e-02 4.674050458902094529e-02 3.652620709110696029e-02 4.075435008503238560e-02 4.533640815358867121e-02 2.996750527572306436e-02 6.229775630045257528e-02 1.888852448817136473e-02 1.918050529853649461e-02 2.474918496745929716e-02 1.737255740133224549e-02 1.818244471023456460e-02 2.202600007177365690e-02 2.393167684548148585e-02 5.991552720560221201e-02 8.729449165967929913e-02 1.352111029200794567e-01 1.573766197174482698e-02 -1.017203861536811055e-01 7.390156108398462237e-03 -2.472510116636831429e-02 2.242932757688228043e-02 1.452724237929817371e-02 -3.486283602446641795e-03 1.399557901964256163e-03 -1.370742539406426253e-04 9.452509519115716297e-02 -1.574839247054369337e-03 -6.575158456412911743e-18 5.869691388783169518e-04 -6.782820145266009459e-04 1.375752527059084795e-02 1.820186956063506861e-02 -7.722591157453441565e-04 -4.854991844020216038e-03 -1.473451469632731817e-04 -9.975398412155738262e-03 -1.589026047401211061e-02 1.816246171291591160e-02 4.033729888328342096e-03 3.467904885406061788e-02 -8.572788116334828512e-03 2.369993892091132662e-02 -4.899374707577695137e-02 8.692408206487402256e-03 -2.338826395334555469e-02 -1.187528617961476998e-04 4.868768881287802164e-03 -1.443844002776742711e-03 3.759483978880748950e-02 1.388464966475307615e-02 6.876431877507688598e-04 -1.894905871395349654e-03 -5.534978839561752011e-03 9.161521758598263144e-04 -2.328564471799889068e-02 7.435037347994669088e-02 -1.104435724796499910e-03 6.436862596556301545e-03 3.504469009203717723e-03 -2.476390373747320781e-02 -5.008006040666930503e-02 -1.625177428656636794e-02 -1.570785821517375544e-02 -7.408326997111034165e-03 1.069843295476411854e-01 -3.654058747222980498e-03 2.017112288439669790e-02 4.541292827632181089e-02 -3.374378861024827231e-02 3.002810866779538623e-03 -2.082330997944285486e-02 7.799342444171295137e-03 -2.074086055307794788e-02 -1.653050598788191649e-03 1.395724203973278989e-02 4.024851310046308889e-02 3.361861473634846637e-02 6.373332701720710869e-01 3.861270074720576928e-01 3.625657971472121682e-02 2.311159273822980392e-02 2.235405256174820623e-02 6.144242537862906750e-02 7.603302450504893784e-02 9.603623687462229871e-03 2.797526597893906786e-02 3.324845108904481150e-02 3.760906111045222294e-02 1.656055128932597958e-02 2.985980118717309478e-02 1.689495258070738276e-02 3.340496093555919005e-02 2.385843271362122001e-02 5.948931203953685548e-02 4.151919758029661185e-03 2.999279562655621928e-02 2.274759267517528771e-02 3.999416006364827497e-02 6.044335494709054630e-15 4.097403489453038672e-02 1.997249791463299399e-03 5.690670634276153717e-01 9.935248681209252197e-01 6.396939696804041509e-01 -7.558899609740746262e-02 2.394882784254884212e-03 -7.367767069429061177e-02 4.579424599884146171e-03 -7.067386504504133415e-02 1.847221088404468877e-03 -6.515483040328844122e-02 1.035919153055830932e-02 6.470901903543190281e-01 7.505471867611070991e-01 6.692008997345476606e-01 3.196500782715351596e-02 2.182340118955774372e-02 4.555248087315820482e-02 2.725981031118897280e-02 3.493505106130086191e-02 1.442604805656858610e-03 2.446484136576221671e-02 3.110411416074631771e-02 6.467246304700001447e-02 4.268025322915224296e-01 4.198014987687183619e-01 3.341876328719715072e-01 2.706083185869134419e-02 3.199302625052275612e-02 1.453239473836664790e-02 4.902521046560060630e-02 4.367638127765485073e-02 2.900041203606087276e-02 1.103954837551588408e-01 5.441654245243961185e-02 9.395027784795353942e-02 1.899862149893823127e-02
1.061148569784683415e-01 -7.211979882755030491e-02 2.011193454772343056e-02 1.785831844359968279e-02 3.573164096629753705e-02 2.957817393656434271e-02 6.023261800036418256e-02 2.484551050075095580e-02 3.246733711286604179e-02 3.548932900911629107e-02 5.156171241001277633e-02 2.035585483664338266e-02 1.299274617554940434e-02 2.914110626708634605e-02 2.926676560812812786e-02 2.258552901440997324e-02 4.221420194487201377e-02 -7.091717187418572835e-03 3.137435920593897309e-01 4.569493616247999671e-03 7.910642870847779673e-02 5.471834230458366873e-03 1.930091843511846927e-02 1.990616309788698426e-02 7.751831623644331315e-03 2.596524671386659169e-02 5.183689934760155743e-02 4.069859745445983043e-03 2.438853500806303226e-02 7.109264055942232305e-03 1.048696779867544157e-02 1.133379015379919370e-02 2.201241715122235782e-03 4.517728486756479567e-04 -1.851793000033767900e-01 -9.013802830336246907e-02 -6.128693502023897094e-02 7.066820189442658308e-03 2.312794640475107980e-02 1.826841865229701656e-02 2.100757030671570116e-02 2.348499090812573506e-02 3.726439959230946897e-02 4.188768738964800120e-02 3.934242502616532716e-02 2.104463932641173832e-02 -3.998024880334503234e-02 -6.503595538839323023e-03 -4.606815012776654256e-02 4.337983177314815725e-02 4.375239858187401215e-02 1.216431581743717891e-02 6.974508945215399797e-03 1.514186598952792923e-02 2.751090770532387680e-02 5.949264426187993171e-02 7.416376467443362708e-02 7.808779580091310124e-02 4.953008537133856554e-02 5.864717019895589190e-02 -5.521752978665528389e-02 1.608483054610350058e-02 -3.520756543364146618e-02 1.624731659026260044e-02 2.552399038079602306e-02 -4.244356313271719500e-02 -1.212881371397308005e-13 1.812915536311361219e-03 6.137365176673857670e-02 3.848167148326680359e-02 -1.046990076331415761e-16 -1.607822250743282545e-03 1.682098715397759708e-02 4.467364818259734333e-02 -1.254672115704186589e-02 3.774579265476411122e-02 -4.947510598570070001e-02 2.461223947618679936e-02 6.858732744534475068e-02 7.814783311182460501e-02 9.959488834960324166e-02 1.629638044204368991e-03 2.042021743969326189e-02 1.003711842665092556e-04 9.030475272645997406e-03 -1.249335263385241944e-02 2.016919063208612653e-02 9.727878973440849328e-03 5.359971982235445936e-02 -5.535583017094974044e-03 1.173712582595218257e-04 -5.257741686968035733e-02 -3.690057086048464940e-02 -5.137540087988206683e-04 1.228917569241649746e-04 3.939205431517633894e-04 3.577128823202792296e-02 9.256257587888921945e-02 7.704469194581006179e-18 -6.584727910326423905e-04 1.905966237621235099e-02 1.165961311019535623e-01 -9.365550775557134403e-03 3.607631258057174967e-02 2.603769450817711856e-02 6.193732603782646680e-03 5.976853112442587213e-02 -3.636648358375493134e-02 -2.605936659265528643e-02 1.333985311871630115e-02 -9.937504190584457719e-03 -1.832199181050122791e-02 1.256770549191571495e-02 -1.238185612452851348e-17 7.634704011896031005e-03 -2.334642654280578294e-02 -1.568504632161330089e-03 5.444024847692618789e-04 1.425020249713019813e-02 1.001807200927154647e-02 6.268825755810836231e-01 3.494454825258636599e-01 5.291222091135529965e-02 3.199978919867104560e-02 3.693740264830909503e-02 5.744673503429773009e-02 5.921156426787774207e-02 8.534638233956845857e-03 1.010018130932303568e-02 3.693057917066942808e-02 4.832631947397932060e-02 2.916173563347034675e-02 2.140126214558753931e-02 2.078564444165030209e-02 3.601092069972584159e-02 1.314752689970112749e-02 5.905941385279214750e-02 5.567292161003009054e-01 5.771456057696555675e-01 5.346890638706186527e-01 1.758952258987130611e-03 2.212277273295202348e-03 5.166864063953547498e-02 3.537014906176431245e-02 3.320744655149836938e-02 3.752364117618441075e-02 6.317065689250964122e-02 8.808347981600966534e-03 2.041603194239740432e-02 6.454002141701751377e-03 2.030368533942832066e-02 3.582293932328780867e-03 1.076880945320836391e-02 4.741012277759938665e-02 3.940635778355119356e-01 1.173858370415912677e+00 5.456697357711878249e-01 5.750969446271349955e-02 3.223352954957535432e-02 2.826577575005865747e-02 4.759024180413571475e-02 3.157143011664680782e-02 4.321802563605208619e-02 1.216018404528782133e-02 2.885475441745772446e-02 3.567477382828758425e-02 5.876850051913823370e-01 8.413022957149408620e-01 4.471950787796483695e-01 1.436697291350483470e-02 3.040971770120594678e-02 2.042969797704470222e-02 2.750417719829980137e-02 2.394763539747885028e-02 4.276184569054383711e-02 1.444747664262924225e-02 9.514558152333177365e-02 3.608447181762212269e-02 9.123460592754993603e-02 3.568705998489139775e-02
2.639802968507961589e-01 1.780283245355697083e-02 1.179119185465261031e-02 1.124563748267812593e-02 4.634671761242290921e-04 5.079325520923494430e-03 3.017076317769083228e-03 8.149087471291604004e-03 4.004750266092348306e-03 4.327561183712654198e-03 7.349327108248120133e-02 1.125113068935596275e-02 3.232131555132751466e-02 9.686690780046408947e-03 4.129347214480173084e-02 9.819390378274154077e-03 9.165822930305461691e-03 2.424033731908964082e-02 1.382400191512180194e-02 2.250996958936642839e-02 4.129906482392737416e-02 -3.212982665224794337e-02 7.480103771811953683e-02 2.048103536799836602e-02 1.216861440326835742e-01 9.395944129610318829e-03 2.176668817244286486e-02 4.619637549278647615e-02 7.031793984132690689e-02 8.668916892158284870e-02 1.425462209810548497e-01 1.297584607561574344e-01 1.473103420116562978e-01 8.394867866895598663e-02 8.171038382126467881e-02 4.830167894746632135e-02 -2.582070230181768636e-02 -1.261265065578571869e-02 -1.061349569178684782e-01 4.183728097536568852e-02 7.184272082973083492e-03 2.806461670054128527e-02 1.741845153448639411e-02 1.013428911419076957e-03 1.648678415147231471e-02 5.310597765755099842e-02 1.965885360059223819e-01 3.759217638549253171e-02 1.473325120723898048e-02 1.845126778671577275e-02 -1.271640824135226826e-02 1.803095539843658016e-02 1.677374216339306071e-02 3.272569940334925376e-02 3.238398029067617034e-02 3.486957754967099504e-02 9.248308342528115122e-02 1.668418809820230997e-01 6.620371535102960447e-02 4.574798089242330112e-01 -1.754492654300464083e-01 -3.279334428798540580e-03 -3.787881310016116138e-02 4.979259577491840227e-02 -5.438319366854553706e-18 5.581032172524581086e-03 1.958784118027626434e-02 2.445606751793073796e-02 3.645556062021078261e-03 2.126292713750666669e-02 9.304251392641875104e-03 3.372310420260749703e-02 2.588988895796623477e-03 1.905671197894634114e-03 8.641153654598744292e-04 1.107557958889869923e-02 2.065780780993581572e-03 5.022477310613555207e-02 -1.380720616401556690e-17 -2.970055381049713141e-03 -2.077292195699840389e-06 6.462096040791853041e-03 4.806058670663545390e-02 1.118660360507178486e-01 4.888945243127734014e-02 2.914921768106604194e-02 2.821198481921383810e-02 7.697262135912832925e-03 -3.863593826225744307e-03 -1.138533011681668906e-02 2.638624039562727230e-02 -3.531416485715026948e-02 -2.751817149353010146e-03 -4.237245715562180937e-02 3.332515873191116418e-02 3.124464788626229658e-02 -1.438822877355924734e-02 -3.858379096344712739e-02 -5.043044018904357679e-04 -3.187798135221913028e-03 3.341559392454928473e-02 1.665335624790276192e-02 -2.576970008197487436e-02 -4.756741962434633060e-04 4.123693727174381797e-03 -1.879798058711515396e-02 -5.652507834223422303e-03 -2.558503135925166427e-02 -1.073095740333648157e-02 -3.410949171775539551e-02 -1.046405113288237897e-02 9.654447141670670016e-03 2.427034751570059448e-02 -2.805216290104245794e-02 1.260442477479078831e-02 -1.871003421357318383e-02 -8.117408362367291991e-04 -2.150826438181409686e-02 -4.394243415651561541e-02 7.872613350550810478e-04 7.523362541239376755e-01 3.592780336544745823e-01 3.366302714455059186e-02 -1.950386874402007314e-03 1.170362096241407517e-02 3.348008091007836115e-04 2.642541908354225505e-02 5.138556640603521471e-03 2.219044535884467928e-02 2.017082155977386292e-03 6.300976897504002860e-02 1.480059702257480875e-02 4.291716886352759525e-02 2.983940162447207972e-02 4.180578774438380163e-02 2.192775845872133728e-02 3.328561306093717753e-02 1.601571369429209690e-02 3.607151621366799094e-02 1.780155182090235183e-02 5.948329837114500751e-02 5.741186566104437006e-01 5.669556828499071610e-01 6.427457680469373935e-01 1.312735960869244704e-02 1.280818904691367888e-01 7.152891555620427555e-02 9.620321574099238715e-02 5.322972270312650855e-02 6.743739089044073487e-02 1.149737841053014159e-01 7.564392651686914493e-02 7.253425651600657609e-02 5.246860133539925364e-02 2.932491534371265904e-02 5.341681791667611534e-02 3.798480314421907034e-01 8.029084278150095955e-01 6.512357800539778641e-01 -5.158885283553098743e-03 7.414031333356548709e-04 1.461112087983310916e-02 3.104158049429138477e-02 3.063598579082478426e-03 5.148292889223121671e-02 5.357429557430443828e-02 6.612790125922320128e-02 8.747268530120026753e-02 3.440941173415968390e-01 5.079827529803995212e-01 3.564093621388453315e-01 3.009059649311049498e-03 2.030026342499809122e-02 4.896778226213144292e-02 3.952825175018202797e-02 3.978395393233363081e-02 1.174987706205495586e-01 7.913598455045402214e-02 1.107334525349428500e-01 2.849700743823130669e-02
1.107601929519083928e-01 -5.818463005507201746e-02 1.946420504093605147e-02 1.256829205766364470e-02 2.362716184607069683e-02 3.508290942725652423e-02 8.429481424381513044e-02 2.493439080280434456e-02 3.703879646009421156e-02 2.762996520931750369e-02 5.028588193025892994e-02 1.415716035039169506e-02 1.192296345135786771e-02 2.907099196725668883e-02 2.872188594092340772e-02 1.666522127736184966e-02 3.495521571954053608e-02 -4.802283564600834040e-03 7.809617007723651527e-02 3.939661001637154416e-16 6.105194178016468305e-03 6.353512075751368247e-03 2.366334607769299270e-02 2.318382112583839208e-02 1.964220660395905871e-02 2.024440784507715654e-02 7.439972128368854687e-02 1.156767915875929649e-02 3.663386985507440920e-17 1.026870059702149853e-05 7.505941360424132729e-03 1.525873978169830711e-02 8.592324337707463481e-04 3.091043817064681454e-03 1.885223544078998325e-03 4.576626066942762402e-03 9.027568208330578187e-03 3.752758464132521093e-02 1.168248471675976813e-01 -1.580442121882403780e-02 3.140490393998705138e-02 6.254407568584437194e-02 9.129911542814546366e-02 4.384601097389430324e-02 7.888952585532878203e-02 1.210364154252008967e-01 2.168737321561224340e-01 4.927790842457122761e-02 1.885421676455056525e-02 1.037400804929494193e-02 5.608938736058990265e-02 4.311443425973609384e-02 5.526867403711354515e-02 5.933210908732069327e-02 5.127580307557992600e-02 6.496035577003102246e-02 1.073244640929010979e-01 6.485610582305106897e-02 5.720247830690939478e-02 6.677279053785389884e-02 -7.066769702464564817e-02 3.901837901649556073e-02 -3.321466967002541237e-02 3.615359686630179731e-02 2.347217833946749210e-02 -3.500264659915951204e-02 -6.331683906448674879e-18 2.622223276442946140e-04 2.488902977466689315e-02 2.089422517044446648e-02 1.013694060496058130e-03 1.133209001422990896e-16 -1.885519244388433532e-03 5.546068367557292766e-02 3.717330249404623716e-03 3.723082382668034540e-02 -1.706618046833836746e-03 2.254168486733294438e-02 7.063520551439676776e-02 7.753753537839670751e-02 4.311832214121046059e-02 8.332362205562153512e-02 3.022462610584599439e-03 -1.600401296405012247e-02 2.197758514042947101e-02 9.274372903140147215e-03 1.111103122173623912e-02 4.208139526229695560e-02 -1.844680505081729442e-04 3.381843564285752501e-03 -7.220973283975716528e-02 3.671608276202182469e-02 5.672890955263434204e-03 7.911953013257280731e-02 -2.824690860484860809e-04 9.722028146054277620e-02 -8.560956468486970172e-02 -5.856645793688258078e-03 1.126548847027509202e-02 1.107924878460408123e-02 9.171982091463515979e-03 -8.413238025712296841e-04 -1.742414214406088485e-02 1.080958495407148209e-02 2.574052759263421229e-15 -7.706112371152226345e-03 -2.279073272807176087e-01 4.180403621595172453e-02 2.429882660033667527e-02 1.630171140038160216e-01 4.278396406014080972e-02 -3.938165982267194387e-02 2.527529711275073449e-02 -1.289688117762704742e-02 1.336578959636412940e-02 -2.078469530152783282e-02 -2.045987794982391094e-03 1.795757299893067990e-16 -4.727119949149231892e-02 3.704727794529755897e-03 6.166092946658316976e-01 3.748806351271342097e-01 5.120659695349129292e-02 3.476521612673515171e-02 3.051935680304748710e-02 5.780757606927126174e-02 6.530164074019760567e-02 1.511521704388561066e-02 1.295056967581631469e-02 7.456424138087124759e-03 2.833800536030959064e-02 2.243944756944356300e-02 1.853384958458468429e-02 1.816993788221254319e-02 3.625462512449530161e-02 1.035046509629226723e-02 5.744142373768486298e-02 5.920970819724140632e-01 5.388716375096019862e-01 6.060910504789021047e-01 2.360719912990378450e-02 7.447974339587561630e-03 4.191246855941468258e-02 1.685450503929787625e-02 2.277187419343154953e-02 2.269573768397621996e-02 8.789038275126496991e-02 2.327179684158900766e-02 2.484003483322320960e-02 1.807444719912975869e-03 1.643329611272474622e-02 4.900212073636121650e-03 1.297860106724779521e-02 8.798984333484896961e-03 1.877312282123285922e-02 6.263804664458604878e-03 9.038237072444227013e-03 7.173239879364371463e-01 7.461391084135878815e-01 7.305046672263062479e-01 1.529213442337718810e-02 1.917296569570000296e-03 2.682324556523458733e-02 7.281804916553288355e-03 3.433180517709655960e-02 2.418500150319207245e-02 2.040336814792661005e-02 1.509286526265816444e-04 3.484480968378623200e-01 7.012091191984506366e-01 2.823024511086884569e-01 3.271341496446747953e-03 1.908742989449948094e-02 5.205488207993817784e-02 4.772849099610763229e-02 5.502738242045647488e-02 1.394658243866336300e-01 1.210908275401359480e-01 6.916535695666575778e-02 2.655916898735972281e-02
1.332533527342347757e-01 -4.809740708261262682e-02 3.304774467055723863e-02 4.554698442805884401e-02 2.230460043200167136e-02 1.316124867840834552e-02 2.165869927804421266e-02 3.064004518719941211e-17 2.062144069887725945e-02 3.582464033091992128e-02 7.707341637520068178e-02 2.291327947100905288e-02 5.965153650288608500e-03 1.736923922450375746e-02 1.497601527035784599e-02 2.406303052868201398e-02 1.446922048900629865e-02 6.320336669459011847e-03 9.416840824098457116e-03 2.521737748782106048e-02 9.088843495807358286e-02 2.337031666351102252e-02 4.374893612162497614e-02 3.224535286808492135e-02 1.842798194731582948e-02 3.682015855619776706e-02 6.477461553042347664e-02 -1.301720293492078517e-01 -7.592548011862715329e-02 -7.581290583235901492e-02 1.839214919211162955e-03 1.102444099138405117e-02 2.534669399436842354e-03 9.862852306841725906e-06 -9.246782287752908225e-02 -2.039829296270341819e-01 -1.019201469942002114e-01 5.883168950083344367e-02 2.319151817850280753e-02 1.025400017463893622e-02 2.643581300604578091e-02 5.735490897507736219e-02 6.263655475036192244e-02 4.082562700764082175e-02 3.179222232397916487e-02 5.516994461177727321e-02 7.058803633236536423e-02 4.086867542968797284e-02 5.272017011882360743e-02 3.874469224536646050e-02 -1.578264396725351151e-03 2.200963825783910227e-02 8.339536272497087219e-03 4.015576316950063979e-02 3.272683015687271463e-02 1.720574833111825422e-02 7.143705359143946720e-02 2.037690658810279720e-01 4.552989765908492525e-02 1.353992830450814200e-01 -7.829907873212528513e-02 2.814136302301567527e-02 -8.496428198826655454e-02 1.386789895604899174e-02 2.078258056600556722e-02 -1.742136867629761818e-02 1.903312879848615899e-02 -3.583253163125064891e-03 5.098476443383074402e-02 1.677045350834813296e-03 -1.181788280878994304e-02 1.310844705597202092e-02 7.608179026106455176e-03 3.689618013090454479e-02 1.209007069678999200e-18 1.715536038960063445e-02 -9.744853038774059853e-03 -1.022450699979164830e-02 -1.696848939322806102e-02 6.401700059634839353e-03 1.296365051303737491e-02 -4.293217708696784817e-15 4.471590743156251407e-02 6.529735055366701728e-03 4.747257561577842774e-03 -8.770429246894167492e-18 -7.337425842810942414e-03 3.421176647826095624e-02 5.620421785066764646e-03 -6.983650733651919351e-02 1.163268935651100705e-01 1.144662855537626096e-02 8.558284750347292099e-04 2.063491086434640631e-02 3.536706965528032664e-03 -4.266272975812263751e-02 9.669807920601502030e-03 -2.992331518942511689e-02 -3.110773303250150088e-03 -4.979926021270163206e-02 9.753562362107890890e-03 2.584420935428623483e-03 4.013184729142606905e-02 -3.879892375888464862e-02 1.367559188484243366e-03 -1.916351235272197376e-02 2.165624647815163339e-02 -1.029789773793756856e-03 3.934659412798442724e-02 -1.240111566577201466e-02 2.267393326054788563e-02 1.926788175571638589e-02 2.269017819358553137e-02 -2.481427668661794275e-02 3.821718200964070462e-03 -4.041417752913147032e-02 -1.169458832887469366e-02 -2.016243872735455633e-02 -1.248996391371761175e-02 -3.467463432216062735e-02 6.378004334715470236e-01 3.925357364255106507e-01 3.111436406383927614e-02 1.805980721519921325e-02 1.914869587117016408e-02 6.182073314921136820e-02 5.262877348628551555e-02 1.667797730721987115e-02 2.169432977328105902e-02 2.031422398337104709e-02 3.479838664276227456e-02 1.012941093630821687e-02 1.703765364058302048e-02 1.045621782701641224e-02 3.102639814462795909e-02 3.431851604531996336e-02 6.586693640796911775e-02 2.521565813105444438e-03 1.733092554114111852e-02 1.971925762651761949e-02 6.680327127145602173e-02 2.136980083965425714e-02 6.343555690969950034e-02 4.534489680955349628e-02 3.171761946393424764e-02 4.416605104026857104e-02 2.590401940233424960e-02 5.405774991249714390e-01 5.078268577878084500e-01 4.726730683361722307e-01 8.745199673238307492e-03 5.680832549493127458e-03 1.142299329514335701e-02 1.650842414660884730e-02 3.106249694151805851e-01 1.047984409548924800e+00 5.310541430722947043e-01 3.298481368519510723e-03 2.207915673542545920e-02 1.945672578324772126e-02 5.108869512505565674e-02 3.439733552835859409e-02 4.181673063865935791e-02 3.552238023613098986e-03 3.816413516925755423e-02 6.243809713372967113e-02 8.013681325658390808e-02 4.955596316332257917e-02 3.362503064891430027e-01 6.587759545357695368e-01 3.174841397899529905e-01 1.000910415781667131e-17 1.759656306003460485e-02 3.869289900769301571e-02 4.967168278738611636e-02 5.049782112656121119e-02 1.218511712151538612e-01 7.758770113240366639e-02 1.082147168517413466e-01 3.931243360083174626e-02
1.165655422807258534e-01 -4.930002863857887302e-02 8.193734821598844253e-02 3.459894210205524678e-02 3.551655864720303785e-02 3.720604257921093089e-02 4.620909590363405511e-02 9.775681391949076968e-03 1.716342143117160698e-02 3.539277134827770177e-02 8.405051353969970884e-02 1.429228545768048558e-02 1.394406699738696270e-02 2.054663023683811485e-02 2.206034991447354543e-02 4.221013033835999823e-02 4.469675658692979364e-02 2.282927573863822654e-02 3.513359507458295623e-03 -7.613178662420096166e-02 -3.896429918450674407e-02 1.734891353367562505e-02 3.809627641916665808e-02 1.456117862707294840e-02 5.281357942961639046e-02 7.116522304584488312e-02 2.901636206509520255e-01 1.042126062407404685e-01 4.288887982731812154e-02 1.344980850058074412e-02 3.581542718001514206e-03 5.540736010453793312e-03 1.473899778518010994e-03 2.769693012048163841e-17 6.931764302731864823e-17 1.177303651112062800e-17 4.104898181663949042e-02 2.206946494616145865e-02 2.349878565579223075e-02 1.764823984733829981e-02 2.938028363045541366e-02 2.795536722181451647e-02 2.514490136042905888e-02 4.817206202537579682e-03 2.066530665931572489e-02 4.298382442054987651e-02 8.930762645874988637e-02 1.193761303003550850e-02 2.015531771036239289e-02 -6.819827359159913276e-03 -3.957659558750084051e-03 1.332879633116357439e-03 3.214922360417919733e-02 9.639573849014796206e-03 4.320534510155187025e-02 8.741078726010644007e-02 1.121608064892999496e-01 6.765924762096374445e-02 3.765400005015661633e-01 1.464656075199385621e-01 -1.511444397437625431e-01 -9.531903043002963594e-03 7.205516101102560356e-02 7.253080161528030478e-02 1.528414532347145000e-02 1.508171419930169838e-03 -9.032722748809798429e-04 -2.696248309343987539e-03 3.548932405765319736e-02 4.027187175544011941e-04 1.343757884256284105e-02 1.582212365769187362e-02 1.685125922063647474e-02 -9.273640875474780873e-03 1.484022742947209081e-02 2.046402872947378562e-02 -6.589988161137354959e-17 -2.123517543422740600e-02 -6.063549297929222287e-02 -1.220594533511530216e-01 -8.916606870216389757e-02 -2.612001166752868104e-03 3.976692425444798736e-03 1.269320377212429518e-02 2.400732725274503168e-02 -1.432205217425932230e-02 -3.590644913277703557e-05 -2.175056231425831054e-04 -8.133888723109634573e-18 5.326433009182712786e-02 -8.619858428173998634e-04 9.614491684763618387e-03 -2.051754098756261926e-02 4.727407195345060750e-03 8.515964759794684633e-03 6.400125135140635746e-02 1.095317202980260370e-03 6.147468542694367599e-02 -2.713822338056237626e-02 -2.946904937956880757e-03 2.428138427534854999e-03 3.922431393725555465e-02 9.388671461623781210e-04 -2.608091503337728323e-02 1.473287459090483892e-02 1.750678769098990960e-03 1.142650290332383398e-02 1.760191597152303139e-02 -4.852086055325692537e-02 4.423108491489084748e-02 3.313243483286903296e-02 6.598564230439388223e-02 4.451459956559273429e-02 8.823653942317594492e-03 -2.043546575148474237e-02 -1.767061762801704072e-02 -4.744501820006846206e-03 1.613828886224119900e-04 -1.703264795275224053e-03 3.091887743248484047e-02 6.338409032697485124e-01 4.299779392469604278e-01 3.230500056862836006e-02 1.641117950973868325e-02 2.330190989910997656e-02 2.997577552705672083e-02 4.681266575718705941e-02 1.011828893068832808e-02 3.351343903819810344e-02 9.037087773656075906e-03 2.548703207775141269e-02 4.619599881550735462e-03 1.890984664251934427e-02 7.198137156346273886e-03 2.813510345959181064e-02 1.537894170170128402e-02 7.095228037593277748e-02 8.521078615905154641e-03 4.569505726528880762e-01 7.516162923796317896e-01 5.624923874936769153e-01 8.561134591878026166e-04 3.615876308579271670e-02 2.698837366017216299e-02 2.272200190379355597e-02 5.098364716893512283e-02 7.864106354970373802e-02 2.954990753482495541e-02 4.034105375483541083e-14 1.941215706992628761e-03 2.430894516989157150e-02 2.671579098242463091e-03 8.393033184972656380e-03 8.376549990947911306e-01 5.509372620835679690e-01 7.961475197494366496e-01 8.856372880278556153e-03 2.674797133830202154e-03 2.423448719553824779e-02 1.398942364470223514e-02 3.834914629139368064e-02 4.150514182346563136e-02 5.889272407695152201e-02 1.941390574850112086e-02 3.594813073983280094e-02 6.043589876564611774e-02 6.846354467122797716e-02 3.970689289317812926e-02 1.693885407782442371e-02 2.282564318438568274e-01 7.066790320093561950e-02 5.164474034693820187e-02 2.219862133124002671e-02 4.879440044081248713e-03 3.555461342231997640e-02 1.490798966065903265e-02 8.009121987401594811e-02 4.770649036030607387e-02 5.237926706921834652e-02 1.674839216118091320e-03
1.440931228486956595e-01 -5.413116068620859450e-02 5.538860561415198669e-02 3.945750933620969375e-02 2.875939718122627956e-02 1.518471606777280214e-02 5.241752101133190928e-02 2.991744095470246466e-03 2.805181123587029146e-02 2.848896489955505748e-02 6.153013068812208941e-02 1.377098748873457727e-02 1.220491551489587441e-02 1.547125450787546069e-02 3.641971371386112238e-02 2.720971408057516402e-02 5.037975842005905369e-02 1.541522645081837978e-02 1.996851400490831912e-03 2.180401211331538594e-02 7.447452426453210483e-02 4.575682148254613113e-02 4.812258293896801153e-02 3.560527682301980329e-02 1.223644555689303720e-02 9.460865551886085689e-02 8.632203883885777862e-02 4.312675771877960695e-02 2.667373558144396986e-02 2.069520754729577833e-02 1.960866393997809723e-02 1.448454471706189794e-03 2.548118590389809347e-03 1.374889312936352671e-02 2.930058262951942480e-02 -3.653407241886017509e-01 -1.426954992367853170e-01 -8.276343034485317274e-02 3.308090431276680676e-02 2.199609001892472923e-02 4.078278626762697062e-02 2.985044593887966127e-02 3.722742786258026459e-02 9.584122999665071682e-03 2.585484378385241935e-02 3.447249697337403246e-02 8.262564963761942494e-02 1.607951726626192721e-02 3.895556339434860793e-02 3.689195811991197865e-02 3.939960214038588926e-02 2.478445047687360698e-03 8.719847248737047640e-03 1.686712544350553802e-02 -2.127579854210513988e-01 -8.044592260424789087e-02 -9.770890681331005634e-02 4.549272338836107266e-01 1.435080843368355952e-01 1.725795828707348545e-01 -9.339384231987257756e-02 2.656878239084252934e-03 -1.830737399124474432e-02 1.032966701899928896e-02 1.873832679040821070e-02 1.239237230547317172e-16 6.356473444346476265e-02 4.397497755878553537e-03 3.307226412243212399e-02 3.588924193927476441e-02 -2.546841782197630227e-03 1.450806166986483986e-02 2.405099966404297620e-18 4.835846797437152317e-03 -1.717055519122604812e-03 2.713391274928662888e-02 -9.685746944688421525e-03 -7.409393313581573264e-03 -3.510229252285344218e-03 -3.228269643801153638e-03 -2.779084049231512663e-05 6.305061746950948835e-03 4.228811889962374310e-02 -2.257744711599959236e-02 1.057013705924746865e-04 4.921477073100478418e-03 2.662086604587995606e-02 8.096143918464300537e-02 -1.404057713718289031e-02 -1.164471839799082788e-02 2.989364171669600667e-02 9.330725709243437851e-04 2.550160441139573946e-18 1.027201749795048427e-02 -9.074134477292046035e-17 -6.838378664719453950e-03 -8.848377480186636265e-03 -2.035900796647241787e-02 4.161118033406221300e-02 -1.045008125076306305e-02 8.062242983561609608e-03 -5.114113839992692583e-03 1.030354940401226153e-02 -3.833083677109899983e-03 2.041112858294793998e-04 -1.285107797681343984e-02 -3.022467174168499970e-02 -1.861037282992447249e-03 -8.922911255191608160e-02 -9.198405490204427173e-04 -1.176957751562644706e-04 9.845178768512772086e-04 -1.135102741338036561e-03 -1.627023869400910303e-02 1.824389580924475887e-03 7.795450495342519381e-03 1.852189070841734567e-03 -1.581724194497317881e-01 2.378788699287069375e-01 -1.751934194887005691e-01 6.575022615710670593e-01 3.809801017602958972e-01 3.438082517613688660e-02 2.516063223392015977e-02 2.350817822101962731e-02 7.180642929672642616e-02 7.716562615261812752e-02 2.727074665483037244e-02 2.555011387199505532e-02 9.715503742291763112e-03 3.684848885088704007e-02 1.151776784248877486e-02 2.351484576746732183e-02 1.918793879968341831e-02 3.388571828332931357e-02 2.968540902253855723e-02 7.895976333453738616e-02 1.569423057774629870e-02 1.516771638158438164e-02 8.097582544773390331e-03 5.488670275870896620e-02 2.923980077661571253e-02 5.193186850100640212e-02 3.055738917415616249e-02 1.169372937654834278e-02 6.573375793874578488e-01 9.001910510799012721e-01 7.830851362465447441e-01 2.898824996545095178e-01 2.012565723697468667e-01 1.608776928349207602e-01 2.069618002123273237e-01 1.595203568697773910e-01 3.997563273317163990e-01 2.482164185475087548e-01 8.307965058703358219e-01 6.977112249519495935e-01 7.097712430099137881e-01 2.479738874625983178e-02 3.172115793243744652e-02 5.740819237454143631e-02 3.948981908640936817e-02 4.344132912861548013e-02 1.561088653627416313e-02 3.186711601694435653e-02 2.102771353226849313e-02 4.791824150487248440e-02 4.067571638664710387e-03 2.319136755288195642e-02 1.695928620178064825e-02 3.473029018272167506e-02 1.860664391132852563e-02 1.197684750061627813e-02 1.519596195422468583e-03 3.142207674276838447e-01 1.002752121672492391e+00 3.780615809430125651e-01 7.987581602956767002e-02 1.222043208759048816e-01 3.893751214707471542e-01
1.330748700571816323e-01 -6.168662421631173481e-02 7.767011629675468765e-02 4.638330944245108028e-02 2.260966903269783640e-02 3.790272640226591272e-02 8.431438452354883539e-02 8.904285626185394834e-03 2.306620247545927357e-02 3.084990526120614390e-02 6.772281983408932893e-02 3.215842459506855489e-02 3.981003914376621614e-03 1.379366952012198513e-02 2.684995887000344097e-02 3.207289182350776813e-02 3.649071190977170448e-02 1.749397789784210011e-02 1.759837533168132680e-03 1.840040250956494899e-02 2.538956556831093447e-01 3.857404900773119677e-02 3.448765891950942436e-02 3.361216179182527353e-02 2.251603274380710798e-02 1.916053278062008688e-02 9.737971448987745027e-02 -1.761948572559242365e-01 -2.345416090634523665e-03 3.469543206355945849e-04 5.997415322454348330e-02 6.330427879078531772e-02 -7.160777490725366012e-02 6.726816719633327361e-18 2.366983760777130021e-16 1.167722226489217480e-17 6.045848797173953085e-03 2.559643563956438481e-02 2.908857975352477365e-02 2.367296546257376469e-02 3.549827871350632463e-02 4.968525575290114182e-02 4.898911907700030655e-02 1.924849113272820295e-02 4.837518167301052552e-02 5.542444917004159255e-02 7.670153128671099163e-02 2.427973332994261615e-02 4.110690773063231662e-02 3.649169156337950248e-03 3.017243709566982957e-02 2.184335879096472449e-02 2.570312164621571874e-02 2.664841128543296231e-02 3.658732947516075973e-02 3.156555222788905157e-02 7.884937197670192910e-02 1.389704879578360641e-01 3.924599676131968612e-02 5.391129048358201531e-02 -7.618088518916926732e-02 3.584685762082734373e-02 -2.138078824927691635e-02 1.597101455158377384e-02 -6.816334924097877718e-17 2.600785912221493373e-17 -5.884662038771939561e-06 -2.161138713491993355e-18 8.943623606868923714e-02 1.735023449965484611e-03 -2.036636276480565180e-02 9.436057887916073431e-03 1.971994958089645731e-03 5.349295815542769866e-02 1.436082508306293394e-02 7.957703424271348955e-03 -2.288153709286590634e-02 -8.574478603457034304e-03 -3.159181386389504215e-03 -8.174455410240519249e-03 -1.641641387521063903e-14 1.978997233003600836e-02 2.965552596145456729e-02 4.814654154238273004e-03 1.380973189335881271e-02 -1.580612727248106256e-03 -1.735232290676408037e-02 4.374683960611025491e-02 8.624349561591058283e-03 -3.582201080161419998e-02 -5.476371674578728657e-03 -4.784855214417228775e-02 5.128773488565824962e-02 1.921966718077225498e-03 5.769411684191534488e-03 1.713945336634859970e-02 2.158017796813968894e-02 6.371628980560490284e-03 1.566493679684803630e-03 4.095732758158431713e-02 3.338021305990744592e-02 -9.222179846210824122e-03 1.967069576120198966e-03 8.749651768065682886e-03 1.312898615937071155e-02 -9.833487348028155037e-03 -2.660355297121268659e-03 3.082893875162348238e-02 -7.064828744809186975e-04 7.211312457865953762e-02 4.124019114496270400e-02 6.089690715869625764e-03 2.079990136605019968e-02 -3.392668261984589650e-02 3.599407774259308791e-03 -3.755662784753316386e-02 -8.662432330568738473e-03 -2.669490414500961042e-02 -3.854406539260869357e-03 -2.862520982462075231e-02 6.395667627689465284e-01 4.248844781328675602e-01 3.732937331607482506e-02 2.042535947307515254e-02 2.449180727369516808e-02 5.449236419198173753e-02 5.953843042076828462e-02 5.311558755256505851e-03 1.360675512094627199e-02 1.062762651680936032e-02 3.264461505477475600e-02 2.230430405859790480e-02 1.878816877106298869e-02 1.282071343066834221e-02 3.127787049706182848e-02 5.136339927204790212e-02 6.973386688880993389e-02 5.210794032652641371e-03 1.975598460232411493e-02 2.691136987337548400e-02 5.743303248293386148e-02 4.981790659747391270e-02 5.990782768681909842e-02 4.565828437107240928e-02 3.540974429543165825e-02 1.921403391603264746e-02 1.576556511794920817e-02 5.455489363564833827e-01 4.147820301463275849e-01 5.068087181665708041e-01 3.576055532717228314e-01 5.038110835573170743e-01 3.003592687916341597e-01 7.568214065989750106e-01 4.980784416090537348e-01 7.424270030845893098e-01 1.371296232231555313e-02 1.361567106682783426e-03 2.214939445915334662e-02 1.541013618086007134e-02 5.147963128393071713e-02 3.934960816716271542e-02 5.305595176899353949e-02 1.991405028572066152e-02 4.620844922994338938e-02 4.758721330771578661e-02 4.232953839758604037e-02 -1.224052869758152778e-02 3.778868783343188320e-01 6.225795491762743383e-01 3.379410341370132076e-01 5.156555777134159274e-03 1.485560826929423726e-02 6.701009751329697894e-02 4.199913444258415729e-02 6.505711000148199641e-02 1.437749326291238305e-01 1.208399389693346621e-01 1.311035201575448117e-01 6.421446560225604894e-02
1.402557375195347955e-01 -6.914042046272822484e-02 3.521889961924200979e-02 2.084845637144270777e-02 1.744604384165726260e-02 4.043330118993992105e-03 4.050124411235604682e-02 6.585918395738889208e-03 2.085712992218695730e-02 2.168602066314913313e-02 4.020220060570692949e-02 1.318089562143200458e-02 4.193750748944133702e-03 2.772085524222954456e-02 2.181243690414613398e-02 2.149876238295918285e-02 4.584350860189124327e-02 1.967308582132853631e-02 2.842435533115048041e-02 -5.981767918502230857e-02 -3.309723337183573949e-02 -4.900577215920817048e-02 3.250149109608448106e-02 4.834147796445339290e-02 2.836544687730918612e-02 3.697822645683459392e-02 8.217161457240390598e-02 2.115575288269345294e-02 1.126193229611978548e-02 6.582727740667963616e-03 3.891321130814601181e-02 1.246986245047966568e-02 1.420049620170878157e-01 2.934348943412809843e-02 3.006668779736542795e-03 1.842398691387813186e-03 -3.896206311889480883e-02 2.595782092670222228e-03 1.044552356814043050e-02 1.749914893460746343e-02 1.253546035219234300e-02 2.197107511676071531e-02 1.098682050850209764e-02 2.489975609501523211e-03 1.285887580638780947e-02 1.778685735481013927e-02 -3.728290120882570002e-02 -7.219107459666762767e-03 -2.214935649289203473e-02 4.324741115367070760e-02 4.304095424406909759e-02 2.483984181598241150e-02 2.288586996484977265e-02 2.697953594082133297e-02 2.825552987421463269e-02 3.499729610988933787e-02 6.839025832138784233e-02 3.354941968437250011e-02 6.410829783096749779e-02 3.716536635270847877e-02 -7.504594743108244925e-02 4.953752757684113869e-03 -3.473127554264213851e-02 9.668246266541997980e-02 1.155289160868896818e-02 1.030461828409789536e-16 -2.127602842552012673e-03 1.638076956760706887e-02 7.912117553580202967e-02 8.144509376293709857e-02 9.785052930068045395e-04 2.996379673798396648e-02 2.155591137435905469e-02 -3.959557272938746788e-18 1.429156471141744024e-02 1.928071892121035677e-02 -1.596391450642409629e-02 2.482414323178640247e-02 -5.682356671269654658e-02 3.206977748897717889e-02 -1.588025958263768861e-04 -2.540289562059517767e-02 2.771699622473433253e-02 -2.751662260931937917e-17 4.011939157243181973e-03 -2.091210416588583712e-02 3.182222842045673988e-02 -8.556745072392525067e-04 1.100643796141099057e-01 -3.247642113196881619e-02 -1.801554562178011784e-02 3.297036730485804201e-02 -1.802002118395942815e-17 -1.686973361738031820e-16 1.707278652752572401e-02 2.693484381560972557e-02 3.124509096675924524e-02 5.398705562072414824e-03 2.064647199636742564e-02 -1.314016434481005309e-03 2.412976367396744287e-02 -5.307472302994134157e-17 4.015821175776717114e-03 3.750837861776947640e-02 1.823481691236524677e-02 7.581849717998986882e-02 -1.781530003351471103e-02 -4.096675877952716399e-02 -1.721425100119010473e-03 1.779807243169729408e-02 -1.014497743536924707e-02 -1.443226842163232090e-02 4.644390665310971222e-03 -2.352381641673704799e-03 1.549252275826062779e-02 -2.849122942773979136e-02 -4.859620503583569201e-03 -1.774284287178933689e-02 2.271423276147909939e-02 1.416569227395685082e-03 6.349359715532478798e-01 3.364035450017643925e-01 3.901890950700118155e-02 1.491954156371099252e-02 1.834271772810414786e-02 3.294317246349225126e-02 4.265536119540352944e-02 8.383357181960510163e-03 2.738168470554091030e-02 3.343765430399159300e-02 4.019880208211660394e-02 2.277229227859369626e-02 2.327716801451551118e-02 1.439075129351097965e-02 3.668858666862612194e-02 2.263495741108101367e-02 7.222638160304813149e-02 7.846483326719744592e-03 8.444932423998671722e-03 5.177597049532353113e-01 6.768498721607402180e-01 5.945735127649659502e-01 6.744792131640360566e-02 2.834211807792584234e-02 2.032920614891812333e-02 3.626766704651140927e-02 8.869756444340146095e-02 2.348574079576665891e-02 1.122177241459214325e-02 1.688800988708273739e-03 2.032286335427446050e-02 6.540257887287755922e-01 6.308605101192386044e-01 6.458195578233899559e-01 1.947890950926734269e-02 1.460063858414437261e-02 8.873097037704232026e-03 1.246965054593733233e-02 2.357742904827681354e-02 3.551780741814886816e-02 4.266192596960696054e-02 3.452648640852801903e-02 3.821048500254816543e-02 5.482664173747452130e-03 3.270602030562363288e-02 1.101647156007719286e-02 5.740929464240882574e-01 8.213916973251674802e-01 4.177141033179654905e-01 1.152899433388368186e-02 2.129716552858557793e-02 2.868840497311432591e-03 2.273419415392233592e-02 4.708234069481948350e-03 4.819520732002587071e-02 2.924600455281630221e-02 1.160041575332830810e-01 3.848496985153739697e-02 7.735904026574323689e-02 7.614666858642950287e-02
1.462112205861118563e-01 -5.732188819731641460e-02 2.103110367400905928e-02 1.944601558167796673e-02 1.721877793692059080e-02 6.461123681286802187e-03 1.172735724088008608e-01 1.190545556829536618e-02 2.000216044166320400e-02 1.132616856876600235e-02 3.799103456027940612e-02 2.632788853826921460e-02 8.363187639440831239e-03 2.989948640714918185e-02 3.945506095660469836e-02 3.257793633078381879e-02 5.432946794026033421e-02 1.329610617651144459e-02 3.310598829014903266e-02 -1.055443277010567893e-01 -1.658227921471447375e-01 -9.170573391129398233e-02 2.112714015112120494e-02 8.856185112743830690e-03 4.941621714965687556e-03 2.096132066393125923e-02 6.679270801392252210e-02 2.949258400650700291e-03 4.435005057871094620e-18 8.301529879377354121e-03 1.754315762687277264e-02 2.590588290321721096e-03 1.413179301713713143e-13 3.505079466037075604e-03 4.967015923764224808e-03 4.391628243495651119e-04 1.435845527100845006e-04 6.445654074359816361e-03 1.354330051103593870e-02 8.572874664496540550e-03 1.409769428285017473e-02 1.266037125922148641e-02 -1.654647485364463613e-02 -1.488635540891022208e-02 -9.783705062459263635e-03 3.620862864078453619e-02 4.440765390143583402e-02 2.239221239917548248e-02 1.045532289393818863e-02 3.298715917976675904e-02 4.873005945999583921e-02 1.913479657600771627e-02 2.489950334906409699e-02 4.176590024964580566e-02 2.562661743134768064e-02 1.830261336426333707e-02 3.719417834099923714e-02 1.880880490570559227e-02 5.133842642909074855e-02 3.924363984972077668e-02 -8.139741751848593443e-02 1.895095795485841894e-02 -4.631206235935694804e-02 4.218466524400987794e-02 -1.218777293405892128e-03 1.674772800336357323e-02 4.934328225550025510e-02 1.803181750774986852e-02 3.508209033692852308e-02 3.097299263550478202e-02 -1.814860508393373664e-02 2.458027347209351676e-02 2.286155511661200606e-03 3.362822295750534513e-02 -1.915485035891291432e-03 -7.448663891127797103e-04 -2.132145408179390161e-02 2.747127302706004262e-02 -6.579305332080277735e-02 2.817499208375932873e-02 7.329592634413581627e-17 -2.194017390397744183e-02 9.572408573113586061e-02 3.158565682651044704e-02 1.837163769794558643e-02 1.116140696905261985e-02 1.999439474340278938e-02 1.999233594926079494e-02 6.626308230073769956e-02 3.157838210815745728e-02 2.911360287252638332e-03 1.257234629586648049e-02 -1.019980304953775965e-02 -1.546659923914705734e-02 -2.154163989878373380e-15 2.241262651530923772e-02 5.949155884290463145e-03 -1.811932233449373847e-02 3.034829301455507430e-04 -1.512731117805773181e-03 2.959294448638780106e-03 -9.201957061126266567e-04 -4.278008110123057185e-03 -1.082707081247793676e-02 7.958557401514326779e-02 3.343548436658746320e-02 1.140817396519868722e-02 -1.551779856521304408e-02 -4.157379715881895366e-02 2.246969157770760048e-02 -2.140479063322302529e-05 -3.777433539050931242e-03 1.702917390540079415e-03 7.407464225663210056e-02 7.936715004895813358e-03 -2.634770009876113289e-02 3.138994962960375834e-02 6.314921493744084235e-02 8.920143672622535269e-02 -5.637227011930670724e-03 6.380846303848309864e-01 3.947542201745120050e-01 4.065166834767802023e-02 6.875586650412175878e-03 1.589997800231234687e-02 3.654998351598206380e-02 5.005698437737728795e-02 1.214534891058091128e-02 3.279243651420249428e-02 2.721424687800200887e-02 3.468633521400522890e-02 1.987764521697990908e-02 2.086998310641435123e-02 2.022106208221455259e-02 3.067967724446711564e-02 4.160444871782158910e-02 6.659286307713710440e-02 1.752768507244989796e-02 9.121282932886132580e-03 5.172617767349905193e-01 6.667924071764163019e-01 5.908552116169143442e-01 4.317558580606475210e-02 2.046113327910427035e-02 1.574491101025836740e-02 3.774227921527890939e-02 7.031172701200548436e-02 1.168595693274466704e-02 1.964817558206285314e-02 6.515778136528666692e-03 2.028808025522771397e-02 4.245216040698687049e-03 7.934386017654672732e-03 2.007262859669876193e-03 1.094921341191818116e-02 2.855683335587644181e-04 5.797832080327533846e-03 1.809515571003341653e-03 1.915787765201326653e-02 1.472987605459840678e-02 3.307318227895606616e-02 9.068929011500852233e-03 5.451465148231533497e-01 7.849955684306401116e-01 4.702713143814308272e-01 3.796534006960631397e-02 2.412584463493586129e-02 3.022851208778767288e-04 1.458554216688071233e-02 1.374219360422077735e-02 2.612669171444471569e-02 1.517190712112384196e-02 1.774422907560495657e-02 1.926075221835888557e-02 4.081391977460544634e-02 6.105708819975676910e-02 9.828981437417211309e-02 3.879765800482483096e-01 6.009763426965255562e-01 9.038538860742928449e-01
2.417513973948937356e-01 1.163006376161226582e-02 7.863390313994391952e-04 5.426081093656195779e-03 6.062441804084007721e-03 6.061890622836584074e-04 5.179478770892292222e-03 2.902166565196297177e-03 5.085287188657591252e-03 5.666524639015266089e-03 7.439630640996944266e-02 2.721448767126122617e-03 2.143061580614794362e-02 2.448278659674343535e-02 4.738534264569565363e-02 9.787851897483910296e-03 1.140877904263989072e-02 7.921346381515224910e-03 1.292847816772312898e-02 2.130392745416850170e-02 3.471196391340086035e-02 -4.549145225299729145e-02 9.085308778432522880e-02 1.933942107362630891e-02 4.310344177975104774e-02 1.867359253559297549e-02 1.578397096197346056e-02 2.286412092911664987e-02 2.981325589652043459e-02 3.743535501606648497e-02 1.169025591580787893e-01 5.541501565123416184e-02 8.680507328577066750e-02 4.271324453220996797e-02 6.545701685392865676e-02 3.113302674518677107e-02 2.355322233984722888e-02 1.980079316604038686e-02 7.504282660720805465e-02 1.368355087753916899e-02 -3.428323459857681582e-02 3.182763055061196139e-02 -1.110814762451025998e-02 -2.210800231373414963e-03 -4.410169264416398741e-03 1.285699704615500538e-01 1.254061172170184979e-01 1.159514029774207494e-16 3.526202601701262235e-02 1.194475092911211241e-02 3.263194152000731957e-02 2.236434791006011377e-03 1.898515847880362026e-02 2.101782146388930794e-02 2.968898349271305651e-02 1.201811792298379533e-01 9.680851736593605594e-02 6.158374128245869966e-02 5.630161753883911741e-02 1.371304954223868250e-02 -2.350701876504936449e-01 -3.759962304675679157e-03 -2.754995597830341147e-02 1.380624380878410773e-01 -1.260728256995997769e-02 -4.545199600952736993e-04 1.543719333054398984e-02 7.870378219869349212e-02 2.232068619346604110e-02 1.086392471193756842e-02 8.203754706370095396e-04 1.531453028315126787e-02 5.034539055525765611e-02 7.028651587386435011e-18 2.235792640098898873e-02 1.186954289215565173e-02 -8.857299103375872867e-03 5.420457445466651120e-02 4.730116528349962552e-04 -1.931732418401561302e-18 -3.985280393512073039e-06 1.090007413256985705e-02 5.100888211185698740e-02 4.609558594073947385e-02 1.042117720045952212e-01 3.128558764243226776e-17 8.115420599964118242e-02 -9.265156349998461506e-04 4.651354989886887481e-17 -2.686444817352591713e-04 1.756248263453878689e-02 -7.694983247300517595e-03 -3.714451051619321463e-04 3.921157361960896787e-03 -5.116112803685965073e-14 -2.244071882920720129e-02 1.537896703670141033e-18 -9.423478058096968965e-03 1.094704161214402928e-02 -1.476307507210145378e-02 -1.275404572995200344e-17 -1.127597184406281625e-03 -5.252393518392484270e-04 -1.150605894163240356e-02 -9.781520929245189188e-04 9.829450067403611331e-04 -8.621994756607440774e-03 -4.028911375345897196e-03 -6.987143642087678512e-03 6.781943713951418817e-02 5.647318906333323452e-02 -3.545952497997464925e-02 2.125316707883937769e-17 -1.627848335764055243e-02 3.212688161876516534e-03 -1.975564941574195141e-02 3.305481489169410925e-02 1.061444209018037363e-15 2.697964979836305066e-02 1.907696269191435159e-02 6.268971393879581022e-01 3.250445687710736276e-01 2.511457302277983125e-02 2.075239557898326918e-03 1.046833607055357163e-02 2.881530511910553646e-03 2.126771771240562256e-02 1.380081430032138550e-04 1.877154885222767286e-02 7.810690279989181486e-03 4.701916714696621757e-02 5.891791298905797725e-02 4.087286746195448184e-02 2.261167638128008112e-02 3.555226033565597049e-02 2.395151637245179072e-02 2.334039649114968973e-02 1.123913634660515283e-02 3.396205034571696951e-02 1.454738783471158127e-02 6.126090012999615281e-02 5.994934237870651605e-01 5.673658181535049527e-01 5.592969576579551738e-01 1.378206486503869357e-02 3.133579117705193734e-02 5.603040481751578317e-02 5.419668956571169205e-02 3.809405443995308310e-02 7.064188518462631572e-02 7.640746361086830918e-02 7.566323049899598141e-02 6.952949007290867844e-02 4.655222807087799203e-02 3.166333423617322118e-02 9.634915265584983665e-03 3.736637684419352734e-02 1.565938713244283825e-02 3.214543138145671552e-02 9.832231470386661176e-02 4.654003100203893233e-03 4.295165575449807691e-03 4.418950008061395818e-01 4.886352764966065410e-01 2.067064204448111897e-01 -1.221133657870308170e-01 1.667791836193610600e-02 2.085717687040809509e-01 1.935433254058166463e-01 2.265014259629062610e-01 3.526667691046943276e-02 1.417112053970153406e-02 1.821550630051647565e-02 2.414116502838742698e-02 4.760409003452260868e-02 2.717949110477051525e-02 1.196812725092021579e-01 3.293830338169469185e-02 8.271828114873627791e-02 5.295256716204554959e-02
1.360223760947077976e-01 -6.271646456162062455e-02 5.676968909124411461e-02 5.182161932497441476e-02 2.875482174089301693e-02 3.975511662679058478e-02 9.451106336536645169e-02 1.487446520242738032e-02 3.114119333009728735e-02 3.134643819855817415e-02 7.417097139298045594e-02 5.481565530371240724e-03 1.158147844755869245e-02 2.993248930978966577e-02 2.367389902666763510e-02 2.914313293741824934e-02 5.329699365574177422e-02 4.774523817779961765e-02 6.888996499941659988e-18 2.138747112751278323e-02 6.976021876344723482e-02 3.352204791602236833e-02 2.802006620374726878e-02 4.097876691172827601e-02 2.063090550135318266e-02 2.503584057313412020e-02 -7.580269992571209148e-02 -1.524659581669210728e-01 -1.136572839975543148e-01 4.194975869222258499e-03 1.037605379859371860e-02 2.511155308888086708e-03 3.426527408291709428e-03 3.350086082755709786e-03 8.129811792133156620e-03 1.728661164295024893e-02 3.358743092248563360e-04 1.522512226409548572e-02 9.470320685034977481e-03 1.349409749513759416e-02 -1.720896196820497154e-03 -1.757039002730324123e-02 -3.641524799390256223e-02 5.901854527666446776e-03 6.218861323632329102e-02 7.851803580335449395e-02 1.153813831841674908e-01 4.170768382330185264e-02 2.749078398574378301e-02 4.794700598405061270e-02 -3.687523226846035123e-16 2.775278636345201305e-03 -9.740812423093233793e-03 5.707682507336279271e-02 1.189497867525643598e-01 2.056919460789500553e-01 1.496620621086442726e-01 3.568594487165284712e-01 3.323423069866606738e-01 8.917457934015454724e-02 -8.990615469661671078e-02 1.121378893036210043e-02 -1.160406347293212181e-01 1.222162883505103353e-02 -4.901911737190262075e-04 -1.767939606545917497e-03 -1.344587618866275236e-03 2.192413814428131912e-04 9.138805446322621762e-02 -4.227153020428670027e-05 -1.671870395911242632e-02 2.106999209904421647e-02 1.092810361632200709e-02 -4.368000592501461443e-03 2.581679305548837560e-04 1.901264936625066126e-02 -6.987583110614633240e-03 1.372069188241585905e-04 -1.964734155902834577e-03 -2.511084367511750085e-02 -3.050766237017358651e-04 2.414273349657530474e-02 2.056929949939862324e-03 -3.325110682419104777e-03 -5.377274496230204960e-03 -8.742541112160728128e-03 -2.763416533682048690e-02 3.018764703216816545e-03 3.782982569604815843e-03 8.165337013192505711e-02 2.841943026413034073e-03 2.558841117819127611e-02 5.636179669959817500e-03 2.242581994895383180e-02 -4.062519003145266712e-03 6.351390347319844053e-03 -2.501001648525685127e-18 5.827470457255200242e-03 1.563166025340683926e-02 2.281659576401962389e-02 -7.005580721296004662e-03 -1.297397865402699461e-02 -3.189829474538361659e-02 -2.234259485637242471e-02 3.274714029704892115e-02 -2.899723024968139592e-02 -1.466880711144066991e-02 7.171082085546859330e-02 -2.405272767357014008e-02 8.277264565660502268e-03 5.407605858254042643e-18 -1.527864447187843137e-02 -5.481073059644531870e-03 -1.209928362005310842e-02 2.198332939159739234e-04 -3.433314008775004839e-02 -1.108850289800625277e-02 3.105580906392485790e-05 8.104710537297636630e-02 -3.375820606358481690e-02 6.510574361783743047e-01 3.617551567885953223e-01 4.120857295138730114e-02 3.077457386888468246e-02 1.714191557045433370e-02 5.303973219188087146e-02 8.632752577765863411e-02 2.799693941429881941e-02 3.723881317618565195e-02 4.293756872508398131e-02 4.296053621235899900e-02 3.136669997171353391e-02 2.853947696927006117e-02 2.108631722345784970e-02 3.312380302132888338e-02 1.724911164968534819e-02 6.140671745163602674e-02 1.781570442957058084e-03 1.656937570698616483e-02 2.725035913823986577e-02 6.174368103788635542e-02 4.778851477341465592e-02 3.956885757349296040e-02 2.345597306106610158e-02 2.524789013131190779e-02 1.980125037760041215e-02 8.333573394124121902e-01 1.186563886340375573e+00 4.848011677013777110e-01 1.605249875895431921e-03 1.716001076838847422e-02 1.342846139351017327e-02 3.486557966770983952e-03 1.864822961637050075e-03 1.088730053719049948e-02 2.055859093155006437e-03 3.220256972216711117e-03 2.883443563852648023e-03 2.349547111577574357e-02 3.823799328638975963e-03 5.421532764938681304e-01 1.049138724472116868e+00 8.698532073178036894e-01 2.889031708409025567e-01 8.886837585926048166e-02 4.875748441556355950e-02 5.191142330350318607e-02 5.389083632847976174e-02 3.758120798071859503e-02 5.675613457389081656e-02 2.606878366529732927e-02 7.149924875401589969e-03 2.495512055138973326e-02 2.121502754936932869e-02 3.993585607249983821e-02 2.968712058995608832e-02 8.864802505401198074e-02 2.559935293109427343e-02 6.267092745459960923e-02 3.489628968368953255e-02
1.497441562148475735e-01 -5.854140352774237638e-02 6.597499534340058036e-02 5.059894621778077833e-02 2.694010933255564302e-02 3.539140029749535815e-02 8.547221331917256859e-02 2.365539912511642907e-03 1.648117832037775121e-02 2.855853783488477432e-02 5.844064496834103922e-02 5.921352335368705415e-03 1.796195784829323489e-02 3.609086092066937645e-02 4.248491450517966755e-02 3.467595002969987156e-02 4.941004271610772991e-02 1.743064035677017662e-02 8.706478612718763013e-04 4.232548529715716296e-02 1.605700173784002371e-01 4.477219362897785077e-02 3.550266791404065730e-02 2.451002243917733969e-02 1.819481659131262641e-02 3.204131745692963967e-02 4.975584914339943599e-02 2.791086632023092803e-02 3.038609974478253140e-03 5.788748134664507654e-03 1.179219021536159241e-02 1.011073474968418682e-03 1.034019233244819896e-03 4.965640223473690568e-05 1.835893435177363012e-03 1.775093785185160103e-03 3.073190780956284508e-04 5.088021606695775079e-03 2.300226794201222849e-02 1.070109098689711602e-02 -6.893872881247494790e-03 -1.756808018322457421e-02 -3.663573956312259766e-02 4.383115038993373562e-03 2.404606026908240324e-02 4.159898638016529260e-02 9.395052248142704565e-02 4.703955539419007320e-02 3.257255323849838485e-02 1.871590143326443828e-02 2.175935291559941837e-02 4.071442485637440621e-03 5.617062304299687801e-03 -8.980169373738904059e-03 7.203448114128570137e-17 -1.315789401743806819e-01 3.098605038297163466e-02 -4.161680802714202987e-02 2.356678025098549525e-02 -1.320227432124706016e-02 -7.687816979488247549e-02 2.318575212543672048e-02 -2.137861481204689565e-02 2.129163840414716874e-02 1.610793693057355550e-02 5.312963069292324890e-03 -2.142417162844661061e-06 -3.121493558850014230e-05 1.751173264190465997e-02 -1.685171392897704421e-17 -3.833914862609894437e-03 1.911461633745668159e-02 6.517900739535832828e-03 3.377312796927701283e-03 9.680677872301678534e-04 3.448942372005891349e-02 -2.126371613240337580e-03 -3.803582860033863366e-03 -2.818786436232938869e-03 -1.097013543718826495e-17 -1.703128747640064158e-03 1.135751247116265419e-02 5.511840390113861149e-02 -5.040509211168888587e-02 -1.628035973644568080e-03 1.877456885965192793e-02 3.011227961047384383e-02 2.776250725957821706e-03 3.261753600503222023e-02 6.351045906347434211e-03 3.054049296290261423e-02 4.516972691351691521e-02 -1.155185063545315777e-02 5.365098666443805786e-02 -1.018070398309768674e-03 3.875886055413052533e-02 3.859096593345017617e-02 3.438383723040260020e-02 -4.116514049511558332e-02 1.153636421821582075e-02 -4.008812347937779075e-02 -3.861232250500552104e-02 -2.612220145321770756e-02 1.037501687756127444e-02 5.959970595806560307e-18 -2.478404756763606041e-05 1.569557427828334462e-02 1.929255386189994773e-02 -2.025926933982363180e-03 -1.168106207158967282e-17 -2.061351786029159373e-08 -2.624748841136486169e-02 1.165648022672607399e-02 -8.164485191849998533e-03 -1.645480138706355983e-02 -2.361666526197598800e-02 3.523285440256979009e-03 1.017386901050047754e-02 7.222908644683069534e-02 3.284544588961815087e-02 6.181841396851940340e-01 3.880941339336559381e-01 4.790739190358132144e-02 2.652148459602300667e-02 2.394626874139890033e-02 3.463351074106203742e-02 7.013607322513368270e-02 1.498755496806621242e-02 2.568579373090499815e-02 5.225207786417233879e-02 3.355401521623266325e-02 1.093484807746147348e-02 1.920186247467404669e-02 1.031587924408180514e-02 3.776806254587525280e-02 1.444802240084572600e-02 7.195072794742242428e-02 9.077077840878520695e-03 1.544662860263058823e-02 2.442701692475851213e-02 5.398702048487571264e-02 2.434664124350845518e-02 4.469442498584339646e-02 2.267372162630502605e-02 1.935614441133949593e-02 6.790752496533553062e-01 8.333253710905659251e-01 6.748185731560935041e-01 1.498075377733634750e-02 4.950004847141262557e-03 2.655720947260451284e-02 2.347401718882999658e-02 7.782661901954960118e-03 7.752240979864097499e-03 2.073963467504869387e-02 5.029304335698744910e-03 8.270658244426685665e-03 2.346669326749754175e-02 1.812873958364819016e-02 1.140651249019979802e-02 5.895236171526591074e-01 1.041681552348540984e+00 7.460705797673717887e-01 7.569393085722173065e-03 1.989065783366086859e-02 3.403808914132011587e-02 2.970541510211703942e-02 7.446942971222490250e-03 2.138438463168132811e-02 1.126065444628009810e-02 3.553059109103774010e-02 5.029644789200570128e-03 1.426293508502591775e-02 8.015167904790041487e-01 2.533533132662389442e-01 6.227512278894390407e-01 1.711906626459625269e-01 6.897956267880510361e-02 1.833157921564498860e-01 1.479532416712173726e-01
2.612017863724547806e-01 1.323104541394520654e-02 3.273158232005028874e-03 1.546652116144596641e-02 2.903216926351313929e-03 4.563732897010323690e-03 5.651940489448698847e-03 1.248841024076438773e-02 1.462803435145786091e-02 8.505856542979241985e-17 4.238845519956421781e-02 8.491437000280195255e-03 2.642275402415995150e-02 1.722715870567863988e-02 1.948487663104105441e-02 9.329985963034569466e-03 4.974714758302078120e-03 1.531108886498287920e-02 1.627180282882551704e-02 7.306319575799647900e-03 1.050057017648144998e-02 -9.146897264901794264e-03 1.016661514629513680e-01 1.455585073635331239e-01 6.794926112718682742e-02 9.033277119795625476e-02 4.519066742436037010e-02 3.555181833247065409e-02 2.012692281512635725e-02 2.695902000743496604e-02 5.964853828620330506e-02 9.604331058053530712e-02 1.057959122204253088e-01 5.760329851873970120e-02 -8.567315831206262500e-02 -3.489765561915519343e-01 -1.657707998848410511e-02 4.122225313453102558e-02 5.249301211660526573e-02 1.548816284487328047e-02 -6.540600135738529330e-02 2.328641218625281464e-02 2.882382549915534636e-02 2.516139418185773385e-02 4.260230944347001558e-02 4.882518402421345827e-02 1.177483245587154642e-01 6.009198658820916888e-02 1.532944792486891014e-02 2.884825021929251360e-02 -5.141863603061916704e-18 3.349529964830401949e-04 3.095949425344619171e-02 4.429419155640653555e-02 5.084481909018480350e-02 9.319743251821846486e-02 1.461434444205088334e-01 6.792187510519955007e-02 7.430892119672756480e-02 5.782037759279049643e-02 -2.110098346425642313e-01 -7.294864830307444190e-03 8.602393402199150566e-03 1.535791461609589126e-01 6.835110511673844758e-02 1.963024673251127436e-02 4.875535805075752877e-02 3.565318617437119447e-02 1.112981534534340268e-02 1.338201855056771021e-02 8.649072413975094492e-04 3.138820946344317298e-02 6.172768767611257678e-04 9.261405132422193334e-04 2.621093219008993613e-02 2.748171841952446945e-02 1.679986648695805732e-02 1.545620023649008599e-02 2.802755586389034641e-03 -5.374733176461391967e-03 -3.734773890956133782e-03 -4.840726358060583844e-03 3.149023451417905586e-03 1.082399531897292044e-02 1.072449084440121464e-01 -1.094373450869793180e-01 6.487637790236270826e-04 -6.196526361547006340e-02 5.193591956232248139e-18 -3.178912469621853615e-02 4.729440747444952492e-02 5.622778340452502799e-02 -7.950620095260316412e-06 2.292281918876770686e-02 -9.445311318886645546e-02 -8.589915991153991570e-02 3.132209601183398584e-02 -6.186800711813813386e-02 2.491256608397747485e-02 1.317118428855484739e-02 -2.593898965254511950e-17 2.035475726514754738e-02 -1.614739573901038472e-03 -7.121394142065648816e-03 2.757973930732850433e-17 1.888411079452691384e-03 -1.018836606813318044e-02 -3.562219535852061749e-18 8.936013795967435655e-03 2.093897462056588596e-02 2.108083017545536036e-02 3.034658410137444987e-02 3.458006751414250540e-02 -6.806085841526183719e-03 2.061309914416095245e-03 -3.602074579008774508e-02 3.287517263258919742e-02 -8.454828876813814953e-02 -6.755495941799180559e-04 4.819803720750212572e-02 6.747428234055183882e-01 2.967088450607137107e-01 2.909403864091297462e-02 -3.106024939322610343e-03 4.792264518825105346e-03 6.524652315105277300e-04 2.537419960722651913e-02 3.274434285025476423e-03 1.471192221167181777e-02 6.919356892641128319e-03 4.174291818437751017e-02 1.262701791220676405e-02 3.577159994989763392e-02 1.346522832105349204e-02 3.791533543368580395e-02 2.232764874222060644e-02 1.798005139986478401e-02 6.197931506974092321e-03 4.075317802725638566e-02 4.418304508388455620e-02 5.127561063081231829e-02 3.460067145034109726e-01 3.214279867884734521e-01 4.078979777228156633e-01 1.871945917304115511e-02 3.337064649136704114e-02 9.008122956311265250e-02 1.293058209647106049e-01 5.004883785763741821e-02 4.144259951379387907e-02 1.011294236347285297e-01 8.033109666697765827e-02 6.619909897752344896e-02 9.665490738907141222e-02 4.297694610091703504e-01 6.213502814137419339e-01 4.003449414820389740e-01 1.453321919640214498e-03 3.809344471859762299e-02 2.274641538391368081e-17 2.094190589478190333e-03 2.362549702468777144e-02 5.167703743350164636e-02 1.471153827629440459e-03 3.886299875930885467e-02 3.521135441774249364e-02 5.573851026429023359e-02 -2.685036278693130052e-02 9.526476585191113317e-03 2.158201175709460462e-01 7.986866827264069157e-02 1.258277187078329917e-01 1.965388333338436572e-02 4.953394688836309026e-03 4.286194771201366771e-02 1.538887862886547699e-02 1.115352805835446992e-01 9.441896074433617225e-02 6.729584918858282083e-02 1.663226332905466676e-02
1.458405448576990115e-01 -5.734147934142364705e-02 6.406939239210031334e-02 4.202110429390185781e-02 4.748733099533856278e-02 3.523106444919794217e-02 3.696917197297856783e-02 3.717000493655325098e-18 2.926621322656488763e-02 3.187084240035500948e-02 8.988427815185465186e-02 2.392455947137219624e-02 1.670634184361034433e-02 3.038084165493589936e-02 3.309013036679448599e-02 2.920032055972488980e-02 4.422399307696125298e-02 8.517774922602005735e-03 5.236231801939236416e-03 1.240994898600128633e-02 5.569667014381422115e-02 6.305635237318654718e-02 3.290685429412350260e-02 1.979571291828790755e-02 4.316010642129056857e-02 4.699795285546507634e-02 9.291126636992455556e-02 -1.192054917698699146e-01 -6.067605049365465061e-03 -1.197644357148594935e-01 2.213353619182297383e-03 1.787696793672682882e-04 6.789988757341735605e-03 3.814590114349366646e-04 -8.257710647587317154e-02 -1.636777479901735943e-01 -1.231716619069992968e-01 7.571196044395024638e-03 2.049745573766649856e-02 3.003790324612341800e-02 2.740732763534015071e-02 4.428792718701601155e-02 3.054005628654368754e-02 7.551538515093212224e-04 2.687042389655470292e-02 2.566535809334766216e-02 5.034540309896588101e-02 1.252134764100499252e-02 1.421753024378886315e-02 2.554080014520833386e-02 3.496919739413912459e-02 9.856420439202574768e-03 1.193245605285007775e-02 1.350171025077057975e-02 -1.212894232101363740e-01 -6.520186620366942576e-02 -1.470348992904815522e-01 2.205355693084002366e-01 1.607984680785417564e-01 5.958386718298718243e-01 -8.476032542648012647e-02 -3.772913664348255089e-15 -3.107173346283609999e-02 6.199714287625951742e-18 -2.114460423330581391e-03 3.218885076372580801e-18 -1.082044756410288107e-04 -8.426905027227003797e-04 1.607164242342470212e-02 -2.481924469681279579e-18 -2.355587089258315975e-02 2.517231719536068210e-02 7.521733630404788332e-03 2.484853854964781672e-02 8.467145167089360283e-03 1.430377807449262351e-02 -5.875619821481269803e-16 -5.454699862844136771e-03 -5.757206403157679071e-03 -7.323410219608511747e-03 -8.974048728646758042e-04 1.389197999013449571e-02 -5.800878257892542730e-16 9.082057099844993969e-04 1.737986910541943278e-02 1.702358757451128204e-02 6.205609117022595558e-03 3.782963144011431550e-02 3.117544101196357154e-03 1.002892429287700893e-03 5.539770172768986728e-02 -5.105352667932763853e-04 2.733953303449985096e-03 -3.236089679032348545e-18 -4.271576540825219176e-02 3.284842894061820728e-03 5.331318738995603247e-03 6.031570350117075076e-03 1.935141157918994839e-02 -2.006051617123633606e-17 4.757017370544484847e-04 3.106560988632518783e-02 -5.903868318736172488e-03 -1.235858394554382837e-02 -9.771287739056323113e-03 -2.018527671428780798e-02 2.630151905140035214e-17 3.541002620481140985e-04 -1.852861622830244068e-02 -5.031258868614040328e-03 2.550453683739683716e-02 -2.071457267180015966e-18 1.364927137960302191e-02 9.771243846397363597e-02 -2.202235759268296802e-02 3.955443535110124940e-03 3.904319591580828223e-04 -1.274285109610865385e-01 2.416761726413993727e-01 -1.817464623825967573e-01 6.439393200339436074e-01 3.738334057860548398e-01 3.818650503310427186e-02 2.528208780252460056e-02 2.432411578777841921e-02 5.628967449780623855e-02 6.250656135709525829e-02 1.056357016111569326e-02 3.160244630095833396e-02 2.530574251828464002e-02 3.443597216552104351e-02 9.976674381547728787e-03 4.042398120358049046e-02 1.703317190253275312e-02 2.762472027791630300e-02 5.000617091665974373e-02 6.358787694252698852e-02 1.532532952035880157e-02 1.226848794820507675e-02 4.461374361582409742e-02 5.545518601726875246e-02 3.883306308436166449e-02 6.554662491156894311e-02 3.008128364770366187e-02 2.299442209714768620e-02 3.075925152770079760e-02 2.246399407078430652e-02 6.332615062351570900e-01 5.275943510246917656e-01 4.580516170431366274e-01 7.940497802809792088e-03 6.559348901972419932e-03 1.770795739137261821e-02 7.303967222222344444e-03 3.078224810690223245e-01 1.292273062900530345e+00 5.123830374573727919e-01 5.685209412278792798e-03 1.900832345331443865e-02 1.898494905332630267e-02 4.770542188845824927e-02 4.411613479894615358e-02 6.074585403902280079e-02 8.566624633489360041e-03 5.142939447586712676e-02 2.870048064355157358e-02 4.056265746285490914e-02 1.955881459585566418e-02 2.147973480938010146e-02 1.273809652792979100e-02 3.750095440660052015e-02 6.305292465299528408e-18 1.942723089197309974e-02 7.348830976650643811e-04 2.828252187173003196e-01 2.394128361478298772e-01 3.210261005886201890e-01 1.076757584961717251e-01 1.351551220748037074e-01 3.745966354682567778e-01
1.382591499952206815e-01 -5.026222537636189369e-02 5.813616660108902184e-02 3.470178329018156621e-02 2.744553686686581825e-02 3.227858032746461214e-02 4.105304668578824145e-02 1.059819521243276287e-16 4.264228945682941446e-02 3.037963823229584276e-02 5.813092046719207234e-02 1.035644070354109318e-02 4.043121669457740326e-03 1.727489028441837882e-02 2.027294399456344362e-02 4.060946039825404780e-02 5.418848346547292666e-02 1.545838272646103429e-17 3.799245479964069496e-03 2.035903157753377393e-02 2.717677998580716459e-01 3.724872292974754778e-02 6.108726887272815376e-02 2.981729869321132070e-02 3.577096916131200510e-02 3.517901585005935722e-02 -1.062887674091647083e-01 -4.796880704980233112e-02 -8.534427112177471464e-02 3.260495442145482526e-03 1.173325718241940638e-02 1.535047461231422490e-17 9.199975547937214691e-04 8.240738932339966788e-04 1.408050138455804734e-02 -1.137884479575437396e-01 -2.266593011803925417e-01 -7.974680479259356469e-02 4.782172676437713987e-02 1.643789498165712862e-02 4.192655431838467062e-02 3.789277861887562632e-02 4.274582689216582798e-02 4.557489056133440242e-02 5.005523642869293133e-02 4.807258062025945161e-02 3.924100667689136429e-02 4.425934975108100633e-02 1.911682485637840878e-01 1.383332626434987875e-02 1.873457157191396605e-02 1.477546082223556026e-03 2.462689743093514350e-02 1.162395755390640585e-02 2.937938499475658410e-02 2.404337386412620445e-02 6.150028347576997484e-02 2.481053282503117297e-01 9.697788318650193973e-02 2.499533069174173644e-02 -8.942603275147105457e-02 2.719230124784615885e-02 -3.122017627729168146e-02 -2.318424562680479820e-04 1.734672460110286279e-02 1.439661916721111630e-03 1.066743491191332171e-03 1.642124954824584690e-02 4.601865907218927187e-02 2.278221074123258638e-02 -4.000006734734049170e-03 1.909300754195098968e-02 1.492175315584481389e-02 -2.579052358261371605e-03 -2.087045971451995281e-03 2.335679924192738885e-02 -9.976028071568493819e-03 9.188949523751696602e-03 -3.279611476390311559e-02 -2.262345991101546647e-02 -1.699835663211668761e-03 1.388564896613971034e-02 5.668346157394736455e-03 -6.451975036992823123e-05 1.123245811874931687e-04 -7.785406237598596103e-03 -5.160765810184054364e-02 8.611324219182946160e-03 3.255289689705630034e-03 2.015437903039859341e-02 4.962844325013654120e-03 1.042577142753702696e-02 -1.511188891249580328e-02 2.092891739975392615e-02 -4.084299390471052976e-02 -5.343610939901822371e-02 5.813898846763132337e-03 -1.251870232538660963e-02 4.211325196551793382e-02 -6.400305608916348130e-03 5.047062765169609694e-03 -3.112800624699696146e-04 -2.801660301448646515e-02 -5.017450372905428446e-02 2.406390860296777235e-04 -3.739321139925123189e-02 -3.074873927435919571e-02 2.990701208408153566e-01 4.528219031810668910e-16 1.895706146008907522e-02 9.871348740937307964e-02 -9.546228147202002676e-03 5.951054909902084127e-03 -1.657185541902773604e-02 1.605941742827679888e-02 -1.728164811079356544e-02 -1.080239749799469240e-03 5.735613994352307607e-03 2.300622222414767590e-02 1.451707957735066598e-02 6.518990538010039915e-01 3.877571627723727099e-01 2.655524281839573633e-02 1.306411838597753283e-01 2.685492278478447681e-02 5.030447040660972896e-02 5.317312838241940176e-02 7.284830600940918506e-03 3.323137521469344408e-02 2.261726455697055452e-02 3.914924107351552135e-02 5.107200111577883325e-03 2.147533564364456291e-02 1.076089134480593540e-02 3.164905768923816087e-02 5.998830378478039321e-02 6.091694437915451399e-02 8.842309739627900178e-03 1.826514764516390041e-02 5.310740951666685118e-03 4.397471535003474874e-02 3.662544879812532506e-03 5.051870660707393629e-02 2.403691605743488516e-02 1.461111132249875641e-02 4.882091341395361495e-03 9.169275024592697942e-01 1.205252472524472651e+00 4.428024317904437068e-01 6.733976626190749451e-03 2.427227272506896419e-02 1.142492295540252804e-03 9.045992328668527385e-03 9.824844505128920574e-03 4.019507055273024987e-02 7.715382569595010676e-01 7.220397653284791328e-01 7.571264914058674078e-01 4.284279346356704987e-02 2.509723180204865775e-02 4.762711025237321288e-02 3.448926001657568602e-02 2.988423117052365491e-02 6.154581811660083442e-04 2.302476719185503687e-02 2.792693234848155764e-02 4.652758515123404243e-02 4.055773307678294803e-01 4.181003367414179994e-01 3.230010411092822387e-01 2.170801582123742593e-02 1.285846273093931388e-17 2.051891892105390386e-02 9.368833077225009395e-03 3.994659298031688621e-02 2.371272054433337315e-02 1.144674512402429345e-01 5.348313172018488021e-02 9.768769416439143649e-02 8.625732316605017047e-02
1.378840756946030366e-01 -7.909198412205531437e-02 5.116272467891355702e-02 4.561898412981944817e-02 5.989191268487398939e-02 6.314837502235814970e-02 1.214306963768271852e-01 3.238095741567131175e-02 5.533287137945088613e-02 5.319079100442699465e-02 6.904878035097110345e-02 3.773614930885420271e-02 2.199777943020046755e-02 2.851324887006600681e-02 4.987509245021033016e-02 3.424837074341218618e-02 9.905468082224963555e-02 -8.144630631891734696e-02 -2.181013729902326389e-03 -1.268474704238596595e-01 1.146054751636503138e-01 1.604046968718756891e-02 1.831376036954802250e-02 2.102182889913890157e-02 2.478961933770788828e-02 4.223119969377416016e-02 1.979745508234244111e-01 3.489412664041691842e-02 2.890167496348164389e-02 7.849332907620047359e-03 1.080503442264690514e-02 7.325563136430687274e-03 -2.444279116288790993e-02 -3.852667158299618450e-02 -5.280137662648402841e-02 4.307481629898621267e-03 8.790122931468213581e-04 8.603708604538597843e-03 1.648279462435155857e-02 1.143475967014278942e-02 2.483651362963468917e-02 2.616002068720392482e-02 2.737357194523024062e-02 6.249870237305845365e-03 2.293037129977073402e-02 3.763976358935936656e-02 6.611264308775613441e-02 1.827158292485702926e-02 1.556506569476087676e-02 2.145255489826949913e-02 1.645636831393051946e-02 3.796680803020328811e-03 3.304707380868066036e-02 6.212391709908801629e-02 2.861798525430144739e-03 9.033590644486808041e-02 7.591974751242365482e-02 7.616318952895725913e-02 6.550140044888740076e-02 2.766306473072408564e-03 -4.983725117776555136e-02 4.449041533477721133e-03 6.750342561030153732e-02 -2.025971737426133720e-02 -4.416864255294093537e-03 -5.141218189042118941e-02 -2.109616564064984259e-04 2.447659359146015911e-03 3.353214628413671988e-02 2.932169095667203421e-02 3.416898045664008843e-03 1.033866943360534779e-02 5.617264549317641659e-02 6.807451214198806388e-02 -2.116311504796311501e-18 5.077153786091954946e-02 -5.955868784450239761e-02 4.218859596871527562e-01 8.144833716228265519e-02 7.163780662154695111e-02 -2.449533835240643234e-02 1.271780398455994443e-01 6.526539588237474654e-02 -4.297658974510844623e-03 4.035502648634852266e-04 -2.824986304235968873e-02 5.172002047117374435e-03 -2.939782788837781019e-02 -5.044977880473854225e-17 -5.353789158307406537e-02 1.323851103576348907e-02 -2.470907974356702144e-03 8.316899780414713098e-04 5.946085987819274554e-03 2.179662503895857945e-02 -3.976078029299765382e-04 8.082110353065630540e-03 2.789289799754112939e-02 1.508601207605096781e-03 -3.155970994991333378e-02 2.516583827415282745e-03 2.168055941545732335e-03 -1.977499333618081695e-02 5.355564546265247604e-03 4.467154835534487174e-03 -6.652832320882828138e-03 -6.289689315585956023e-04 2.145126180915716760e-02 5.105824792483944045e-03 2.554888764580947463e-02 1.408909402492755092e-01 1.098069052777882020e-03 1.876819270101904788e-02 1.811096526204783105e-02 5.452054317382749177e-02 -5.275856582056937145e-03 -3.341376655082254623e-05 1.065811812006349779e-01 -6.772963990231273615e-03 8.571060069042070462e-02 6.462596368688335868e-01 3.428883139740923625e-01 4.246981715015540892e-02 9.867268883782556732e-02 2.757086505845321792e-02 7.734572944999154598e-02 5.357118879980140441e-02 1.465499397020128547e-02 2.265854118996815658e-02 2.850660331408358394e-02 4.286458827263148891e-02 2.564958150478812912e-02 1.795754613001398006e-02 2.751311698078295204e-02 2.768801532234973009e-02 8.052012944804391725e-02 7.275076475529455855e-02 5.445686188940778161e-01 5.324358914842316493e-01 9.627423013821972964e-01 7.642724112060433027e-03 5.140200636954427721e-03 4.875800981279009710e-02 3.525764807759714653e-02 6.521867584955111374e-02 5.951892246234513628e-02 1.050545779286778392e-01 2.611417712255591911e-02 3.652918325479046630e-02 1.576656916181477103e-03 1.935502777166528410e-02 9.269987875743212688e-03 6.113054793362476813e-01 1.095503294106145553e+00 3.481667940306766296e-01 5.280165176945923917e-03 1.056550323084001416e-02 8.081702131082470902e-03 1.999052674402212668e-02 1.567890244212788095e-02 3.096043188500074636e-02 2.777108255855986405e-02 3.107639424984416643e-02 1.273596989237932124e-02 4.024017436522225183e-02 2.598981089393429672e-02 5.756241361417146346e-02 2.270730518354608859e-02 2.142760439890074517e-02 2.326478187553430999e-02 2.843978142615325583e-02 1.401977034427620283e-01 2.052235578602258359e-01 4.675047952231711212e-01 1.320525649648479072e-01 3.633170788910632942e-01 2.486378921199112746e-01 1.882327107019013679e-01 4.087045195280537002e-01 6.160942413455848810e-01
1.311481767529793785e-01 -7.675043390216367001e-02 6.984296182799111519e-02 2.950266452517745497e-02 4.955063221332174639e-02 4.140522093539918125e-02 1.693862595855058495e-01 7.232792449362164706e-03 2.669667349196328671e-02 4.114603461632700826e-02 5.829471701603841649e-02 4.168904386695188730e-02 1.487103322233557234e-02 4.627856096922641077e-02 3.493659688138119751e-02 6.544500179382799210e-02 1.816926226296819158e-01 4.730019831876058900e-02 2.504805886113029359e-02 3.192654944295506697e-02 7.305842943975932036e-02 9.090395053969069911e-03 7.443916347828903601e-02 2.403453967289808091e-01 4.425722515547005770e-02 2.216128788733794791e-02 7.758475019234353787e-02 3.463505401683432405e-02 4.362620486571527900e-02 7.377641823717949639e-03 5.260131358242693678e-03 6.196488118542976646e-03 2.996941792051472297e-04 2.534136269079150655e-03 2.470241531867601800e-02 -2.766799729845773603e-01 -7.934546045649393253e-02 -8.749396912922489944e-02 5.047980290391820835e-02 2.738182848157648397e-02 3.964013406654327831e-02 5.301040649779788227e-02 5.545471842691298420e-02 4.613717094288114634e-02 5.139421169882432694e-02 9.759572488890988495e-02 1.536709442205183851e-01 9.283591437924567161e-03 3.472780138823403084e-02 2.413555214119712453e-02 -2.664804968327562449e-03 -3.189740758396147034e-03 -8.113817958532760707e-03 1.127507760022438688e-01 1.405535157946178892e-01 2.435862881801512114e-01 1.395915725527599460e-01 9.573770766553191258e-02 2.110658661651487189e-01 8.092158409277397380e-02 -6.330287175833984592e-02 3.641110203679778258e-03 1.087273004225416251e-04 -1.108455269247653717e-02 1.338667213883577012e-01 -3.738723461803206832e-02 -2.013741611005288347e-03 -1.564082375921669807e-02 3.598316365309244674e-02 1.711268112137982397e-02 -1.480414995961953049e-02 1.312565796727618167e-02 2.825619790507914383e-02 -1.631560974591941826e-02 4.897229307114280006e-02 -2.805502531231830743e-03 -2.044204887569604431e-02 1.278497937125328259e-03 -3.576701300254551963e-02 2.425244645831661598e-03 -5.167520493675662949e-03 6.680975222575680217e-02 7.887866486962200052e-03 7.219283350554683842e-02 2.894635508311378322e-02 1.027943678549877483e-02 9.922926169068543856e-02 3.023621987256240509e-03 -1.522866337554336075e-17 5.529670014165253417e-02 1.469506788162236802e-02 -8.680810236574671920e-03 5.360369009223361902e-03 -2.981998996413489159e-02 -8.787505662302183941e-04 -8.860684446747570109e-03 -2.537106534235618674e-02 -2.822550764050149597e-03 3.830232825357500807e-04 2.929216675231541972e-02 2.296326859440130617e-03 -1.304967027079924650e-03 7.202364598911023368e-03 2.611309533217721218e-02 5.298654492254854798e-02 -4.144798733243048008e-17 2.026930313276193396e-12 -3.898293589583026741e-02 8.019644143550901452e-03 5.534965824368496407e-03 -1.190060956417423113e-02 -9.336506427623686086e-03 -1.195207837316908037e-02 6.660731513619921364e-03 1.097912614716320097e-03 -3.219120714237698466e-02 -1.670030578412351127e-02 -2.813057726154461965e-02 5.807454042445645004e-02 2.736759064655792939e-03 6.427694800237496731e-01 3.590265979159324439e-01 4.967106578616843626e-02 2.342610588382668113e-02 2.753739228244313220e-02 5.610116672864123161e-02 8.235319919181305348e-02 3.505143471844112540e-02 3.461722067169663297e-02 4.316105447267263445e-02 3.194079759247960698e-02 1.787290143093686612e-02 1.930565890765229345e-02 1.756413542476551945e-02 4.429173277596253760e-02 5.571206361508704297e-02 1.409738331636296860e-01 7.198719056559917118e-02 3.458977523292409983e-02 6.047221645834504472e-02 3.185091471606599223e-02 7.910523386876795390e-03 5.690150923529732063e-01 9.558917582678420066e-01 6.805537647082366970e-01 4.339094044587491163e-11 3.315841264706977098e-02 7.369398603599927079e-04 1.334967586081559250e-02 5.107996265460723780e-03 1.443033486143875319e-02 2.933252276571946243e-03 7.814797632550466930e-03 3.358274678797412011e-03 3.269157835234468706e-02 7.955425955916496372e-01 7.327897002966761164e-01 6.541143775363210677e-01 2.539881888205988009e-02 1.424630540242325175e-02 3.652986609698263137e-02 2.995507599495010609e-02 4.538513061383216141e-02 1.411349155809983255e-02 2.949715676647776327e-02 4.205915267250033562e-02 6.019811189805834850e-02 4.102102931557094490e-02 6.698884604471333493e-02 2.737897616625823566e-02 1.155587662053830234e-01 2.195582493697826043e-01 1.288964060287129121e-01 4.713319425287162273e-02 3.443449535887244811e-02 4.274080733433897883e-02 9.169668341673463396e-02 1.426396849690269621e-02 6.565319679304869316e-02 5.402655432719704470e-02
1.338338143423347393e-01 -3.958985902993056999e-02 6.679105575158726660e-02 5.991746245210380351e-02 6.658510774278372790e-02 6.717984270878554764e-02 1.353594924534699262e-01 1.479104991429778214e-02 3.185667633817673705e-02 6.296125102174041088e-02 8.724923482688050347e-02 1.624952859535336649e-02 2.243945871146442450e-02 2.357430672578338798e-02 5.877095970225846427e-02 6.684897752749115185e-02 1.342486346135847997e-01 6.637626160342660619e-02 3.053065056882618006e-02 -3.342880971281485952e-02 -1.474836078910926607e-02 1.435739351736864226e-02 3.222963308524826737e-02 1.524611982258811463e-02 1.140611493126867867e-02 2.575887565643288774e-02 8.773523655177359326e-02 3.569455855926542773e-03 -1.916436367861850017e-03 9.049746563303649608e-04 1.187019715873597302e-02 1.209482363252760409e-05 5.327291811141409523e-04 7.232311011297817014e-04 2.716810892924710792e-03 2.611032427699703072e-06 1.414097332234858992e-03 5.990822035808032568e-03 2.678448085724373720e-02 1.232550087943989556e-02 1.822161303054713188e-02 2.544734394983150585e-02 -5.445640538610947365e-03 -6.060078178421596934e-03 -1.517226000856133211e-02 6.936716015870296459e-02 2.317679218716255443e-02 1.019180901238267607e-02 1.135837763047091442e-02 2.492740182924061201e-02 2.797715232375101377e-02 3.192493971372876993e-02 1.977934658903125059e-02 2.275866065570455662e-02 3.106167677831531485e-02 1.026655132820891193e-02 4.571629759003718180e-02 2.219166618339361158e-02 7.263647606528421397e-02 4.240745650524511273e-02 -7.781160190155425593e-02 -1.210208909352448146e-02 5.600164319614234221e-02 -5.589575882320742956e-02 -1.948824238733628395e-03 -5.904444708849296980e-03 1.360441553903658828e-03 1.062555938184666787e-01 4.361160243481212832e-03 2.001814080041764560e-02 -1.015056854460469338e-02 4.263339225136132671e-02 2.723739879190418406e-02 -1.844618342499243155e-02 3.788009710622735476e-02 -2.123583314365175452e-03 -2.713448358987184672e-02 3.780461140554305932e-02 -5.500292882731585425e-02 -5.047011208817254319e-02 -8.635774567373319388e-02 2.813449961101108579e-02 9.703555753896461744e-02 -3.352481358825013663e-02 3.095311380245696284e-02 -1.712382808760064998e-18 3.070804048107341616e-02 4.723005594114521206e-03 5.368883179664631078e-02 5.100291097087703007e-02 9.810870391172321278e-03 1.866382555896568471e-02 6.504211057937542111e-03 -6.047133541828122842e-03 -1.932815344423633910e-06 3.425783471117953977e-02 2.950162270079469823e-02 4.703421587857966463e-03 -1.667062156568839895e-02 -1.094568672741163528e-17 7.055775066430277319e-03 6.714754799317419847e-18 -5.325098193949898140e-04 -5.893088840994102802e-03 5.429725487882820778e-02 1.080524106415310659e-02 -8.085525247327353783e-04 -1.565268363079295013e-03 -5.121390585767018194e-02 3.983861733762456697e-02 -1.203028704369697634e-04 -3.237739425065787584e-03 2.383028453662265724e-15 -5.855719923835306113e-04 5.483955393835133552e-04 -2.031511810898952927e-02 -3.709987809298685019e-04 6.098168783567588930e-02 1.048333968554854140e-01 -6.350309979026800764e-03 6.414371984446382013e-01 3.600301064070713375e-01 4.894404607646309774e-02 2.520087910200585757e-02 2.542417577019026598e-02 3.760676890596560684e-02 6.414861659093155921e-02 3.245917033188290297e-03 3.136761082522118715e-02 1.084472797391843740e-02 4.482031708648554630e-02 1.407376515513562333e-02 2.897631048231866582e-02 1.895322102210516765e-02 3.997731056741304911e-02 5.740694927860861413e-02 1.037470753048453098e-01 9.228845239263327310e-03 3.758040860598304556e-01 9.701475538421473477e-01 6.912598345618525819e-01 3.336521626250696978e-04 4.050102846274557244e-02 3.293222893560607961e-02 2.225552644308889313e-02 4.178404900353650114e-02 9.372148042016570368e-02 2.808191042305817689e-02 1.168287861209415732e-02 2.323790710950658644e-03 1.908986860865750229e-02 2.110110776844566650e-03 7.776173094465113622e-03 1.708383593055348270e-03 1.879869200247650043e-02 4.681493923953791356e-06 6.360485744015704004e-03 4.440801428214373112e-03 2.579636379259536358e-02 1.199859013734613268e-02 3.788904950292882462e-02 9.398050067317791773e-03 5.556564801895379491e-01 7.933225498313597468e-01 5.157115905864675387e-01 3.717282172472824442e-02 3.362248753601396190e-02 1.175866067305455934e-03 1.811343813070588962e-02 2.007430962257656520e-02 4.342631595683572876e-02 3.625978258621691958e-02 1.856792719002388695e-02 2.207333350053927723e-02 4.176022371578771986e-02 2.965447821778756377e-02 9.526500156112417683e-02 3.822208292762333537e-01 5.948201335660110445e-01 8.977485953359959359e-01
1.286023249246989464e-01 -6.234633362584282579e-02 5.384144995992573568e-02 3.369921067885431332e-02 5.515419661917793370e-02 4.946028084814386994e-02 7.277242330120108071e-02 2.283422414533137906e-02 3.612287625260758583e-02 4.243055565169933513e-02 5.554170157661630913e-02 6.338259558339515848e-03 2.050196205798632265e-02 2.667130705570144866e-02 5.479733977382151427e-02 6.895867595368665448e-02 1.902175018504938531e-01 8.866080146361159098e-02 2.163183648016162725e-02 7.360562926829586350e-02 3.593974941705520565e-01 7.868809772422179216e-02 5.315191266109473706e-02 5.872516539889311576e-02 4.839898147996474220e-02 4.300544409634630971e-02 2.069379923188607168e-02 -1.033012525730152634e-02 1.554032369429889895e-18 1.218206292718132348e-02 1.109413597115466350e-05 -9.556192274378361856e-03 -2.417172802273287718e-03 2.123469836202399041e-02 9.381477379509837664e-03 9.262288967511272085e-04 2.501576004981993263e-04 1.827120966676391281e-03 3.701185659263447718e-03 1.573961609030586836e-03 7.821396965144382374e-03 2.856471892966310974e-02 2.076101840542129984e-02 6.999315658716390338e-04 6.193217713639818239e-03 1.289160536370333046e-03 -5.584212841128297078e-02 -1.076091329332833427e-02 -9.743760203696388497e-03 3.967375416283951822e-02 5.358779812720289998e-02 2.861850083009639598e-02 3.013760172346518357e-02 1.965679165155264502e-02 3.786377956796141420e-02 4.145777588767298599e-02 6.716578266682063370e-02 8.766202757109686772e-02 1.279750570904780727e-01 5.284082516342237384e-02 -7.850040225619668577e-02 -2.881988726634138992e-18 1.589463989634434110e-02 -2.451152493799765852e-02 2.359333156974299101e-02 -1.971088635079408860e-02 -5.047167088843219213e-03 2.440262835625496909e-02 4.546188102172387530e-02 2.827549039533696143e-02 -9.183416424197825972e-03 1.745181070624287831e-02 2.897093245432494704e-02 -1.013108501878643569e-02 6.423067860880177227e-02 -7.605455878459359406e-04 -3.490426329751614848e-02 1.935150925430344226e-02 -3.806140192692224200e-02 -4.563698580251698833e-02 -1.313196706007432359e-02 9.273117718807174364e-03 2.702629343294234884e-18 1.132939453675738291e-02 2.611946171853256424e-03 -2.169264526045952633e-03 3.244550556083492746e-03 4.243214021599642574e-03 3.656683789208944272e-03 -1.154840843275337266e-02 -8.245171277556148995e-03 -2.718849394557060900e-02 -2.493342652290966854e-03 9.046842935137399191e-03 3.365038055404357548e-02 -5.244373177927359568e-03 1.855168562774057728e-01 2.178107800115477999e-02 8.994725664463436299e-02 -4.529894642877568909e-04 1.912629589074870545e-02 1.280213943988585228e-02 1.225671590144439618e-02 2.459005961275104180e-02 -1.132447612592308618e-02 2.478735263575691794e-02 -5.061081933791876765e-04 -4.958868695290227779e-02 -1.369311177780318254e-03 5.341213317432825722e-03 -6.845840734715643097e-03 7.854070949604411052e-04 9.988220850353541833e-03 -7.096884838822664458e-03 1.120862504941070687e-14 -1.791945510220083887e-02 -1.969581578837183947e-05 1.670700989416898461e-03 1.194184068690791987e-02 1.129348876272250057e-02 6.543904635793695856e-01 3.541643995001574341e-01 4.877880248040893790e-02 2.704228214635590391e-02 3.170267186482695482e-02 6.733391588188664123e-02 7.284222171961760028e-02 3.224542770284810622e-02 2.757069767073523289e-02 5.641510424901331061e-02 3.975543515748620332e-02 3.633935496193746451e-02 2.881003873780836957e-02 2.067290502404136102e-02 4.246149399786230266e-02 6.093567321551570520e-02 1.326859285229143870e-01 3.974174134885947313e-02 2.264039663550876380e-02 2.450863522213292747e-02 2.601115970256265286e-02 8.436941553507392583e-03 3.909935336573534120e-02 3.337579158501768201e-02 2.412211277341088159e-02 2.603631842522066697e-02 6.073369602910404924e-02 7.105908280555897280e-03 1.071804545076915013e-02 2.519600708405568459e-03 6.657988775015766247e-02 4.905889250316683436e-02 7.612909291119396293e-02 2.782087979354936919e-03 1.724565412149049737e-02 6.031013887439496020e-03 4.602760231730001518e-03 2.674530466334246008e-03 1.917439621370447406e-02 2.399297858274088147e-02 4.606240763164058671e-02 4.168400497008079020e-02 4.599302005853637965e-02 9.679519719226661878e-03 3.101550264447423128e-02 8.816089593208068648e-03 5.425284468731071952e-01 8.431400505520068034e-01 3.660619187086675552e-01 1.282371965741093030e-02 2.660903375245087313e-02 6.212912353374946267e-02 4.005763636139798856e-02 3.185052310250009733e-02 4.033226112900151900e-02 1.880043332246409543e-02 1.095566756758703797e-01 3.455554999054113313e-02 6.388946591761249238e-02 5.587384868436613694e-02
1.314287525029095294e-01 -6.160233234790164075e-02 6.078511999840593288e-02 2.548103374504930713e-02 3.272127563195373345e-02 4.740918251871056172e-02 7.916616455606917402e-02 3.865842188162468135e-02 4.681702111266811128e-02 4.225078701195183334e-02 6.244516568795228928e-02 1.950740677099834647e-02 1.675193985627424983e-02 4.392794004577391803e-02 4.769417210194332180e-02 6.968954495922423387e-02 1.687314900898362890e-01 9.027787448487810396e-02 4.601071381624002171e-02 5.064077674403569262e-02 7.590480122260838403e-02 7.249666402223360617e-02 6.075897543612134022e-02 7.949187029420222428e-02 8.061678913617709163e-02 5.643078152878446690e-02 -4.541424447673474141e-02 -8.547439567580852882e-02 -8.376534334668318815e-02 1.723998323680728251e-02 1.843018019325726176e-02 2.017046172662801187e-02 5.173991723157637251e-03 4.541031173525801427e-03 1.551272570050307199e-02 7.778485263976756166e-03 7.091496160303358562e-04 2.298767953059503884e-03 2.108433621721796450e-02 7.025398859316575768e-03 1.072215725314548326e-02 6.172306618011071805e-03 -4.848186577885833792e-03 5.169541759796673908e-02 -1.923702707868164483e-02 3.172144103635603507e-01 6.841976697751387393e-01 4.305134585982474671e-01 3.654222389592267500e-02 1.301711778720614568e-02 -5.705397914864159328e-03 1.118483873377825767e-02 1.657119653957560973e-02 1.347057484707615653e-02 2.368091346908710385e-02 2.140697734357507181e-02 4.357119457360440068e-02 1.497806101484440780e-02 4.249652242308379901e-02 5.809368138687204253e-02 -7.068973396324598757e-02 -4.163810597004066568e-18 1.388036185743748117e-02 -4.977717966745957151e-02 -8.092913818737346339e-04 -3.315907650531495915e-02 -2.915994912115278515e-05 -1.665706368923021126e-03 6.984846516753252799e-02 1.755466823588596598e-02 -1.557183342836768354e-02 1.413910588639127365e-02 3.990516537551000442e-02 -1.318372752201405623e-02 3.833025110867207402e-02 -2.266726348347920220e-04 -3.501077062573458576e-02 6.777032199007430868e-02 -3.938226242665696097e-02 -3.620061610227372217e-02 -1.384692083814355150e-02 2.136561966995407374e-02 1.366850248913887948e-02 -1.942921421785963592e-03 6.747785290415866770e-03 -5.661389615572482922e-03 6.749954070285724139e-03 3.648032032881364135e-03 4.533997898307535676e-03 5.531033637488581844e-02 1.221038161015787667e-02 2.751514784006892089e-02 2.283081955881132233e-02 5.501058366616945645e-03 -4.817976354613112588e-14 3.238055604167412727e-02 3.231078623918557519e-02 1.696398051504148519e-02 2.107737685905840958e-02 4.659878056186166767e-02 5.903695788663306139e-04 2.132918823704510816e-02 -6.307537060460668803e-03 -8.796867569598193362e-04 -3.882713979418083451e-16 -1.777578338977668648e-01 -2.416775851746402054e-02 -2.159212369346789201e-02 2.484083193970890863e-02 6.970466902038653656e-02 1.507125429633753011e-01 1.329187071535887707e-02 2.179155476916172507e-02 -3.062336943783432653e-17 3.632565472457470641e-03 -2.244707888158804440e-02 -9.807333265407483860e-03 -2.095571984540697819e-02 -1.697446440021177803e-05 -2.502230375810193461e-02 6.620724839373930415e-01 3.617669090957812417e-01 5.310798971013151737e-02 2.616464162546220523e-02 2.735042796820272293e-02 2.903868626117946453e-02 6.300157009838490418e-02 2.057513003676784621e-02 2.511008092595427149e-02 3.237913850910559971e-02 4.542802836955211337e-02 1.793290308047102224e-02 3.678961652775887547e-02 1.676841750088256880e-02 2.648493926292248141e-02 4.717481512636164526e-02 1.252258952091687416e-01 2.701978160793776923e-02 1.955449351396596966e-02 3.719705692735103542e-02 4.632650598184206991e-02 8.603104824348844701e-03 4.017475301612242339e-02 2.959740314042859366e-02 1.682057157883617982e-02 1.211503643267769492e-02 9.324127770985328745e-01 1.328538373235649939e+00 5.892997193259142952e-01 9.855801488398332599e-04 8.597015765655089195e-03 3.228802967333074554e-03 1.288280376046183583e-02 3.859856826932428508e-04 1.108595215718472492e-02 1.982252950428875244e-03 3.612449335964073617e-03 3.036236010775569122e-03 2.333225681821268424e-02 1.594386205766968809e-02 3.736472844610438937e-02 1.898341464891586255e-02 5.289915856705539321e-01 6.957991246596331170e-01 4.289046459983434367e-01 4.508107981051249352e-02 9.277134759288716004e-02 1.714752344736334422e-01 2.253613621644648679e-01 2.668211316368451214e-01 9.138841459515761878e-02 2.952263728561564486e-03 1.396276964609247453e-02 4.258864491005289604e-02 5.417811842548923146e-02 3.986955805774823614e-02 1.214002647240851468e-01 7.632406296002752777e-02 1.112599417685271874e-01 2.614873962775911623e-02
1.537395528896273733e-01 -4.763203353127857392e-02 8.081046162342302164e-02 4.170593456001592259e-02 2.734989949774657356e-02 3.195645381371178817e-02 6.911741815607820583e-02 1.869783758183610201e-02 3.223204553753108298e-02 4.809950885024664308e-02 6.597329437505503047e-02 2.033253526516912157e-02 1.009617840366024664e-02 2.600186445136521904e-02 3.241415727616307751e-02 2.801005922795212280e-02 -1.525648940686116783e-01 -1.778174288826536442e-01 -1.281305110842665962e-01 1.490249741657377613e-02 1.258259247791900237e-01 1.607492292090183239e-02 2.830970071768618840e-02 1.199405648493469777e-02 1.701067670928127148e-02 3.019726795180063469e-02 7.857589327576512683e-02 1.875557607217647854e-05 -4.127795475749400887e-03 7.051795571209525959e-18 1.193057072202327769e-02 1.274063452683516515e-03 9.277426343516229426e-04 1.416931590526361606e-04 9.762417112119264623e-04 2.522620182017033981e-03 3.403640207333253308e-04 9.429428089621138750e-04 6.948060300546812090e-03 1.893440177832109161e-02 -1.466783193555257908e-03 -9.954462402949913746e-03 -1.274964981058433755e-02 -4.676822693609651582e-02 6.506948133908818654e-02 1.073361863532230065e-01 1.003856135954973722e-01 5.864045968632256717e-02 3.980789071483867986e-02 6.381940706079076986e-02 -1.610245283180703618e-02 -7.350986492999179512e-05 -5.165746093684801810e-03 4.049910363260760676e-02 8.105831851161403312e-02 9.988037688851822415e-02 8.154129891756792259e-02 1.152515393111882835e-01 5.681397379030314054e-01 1.016554817205699290e-01 -1.088539637840733876e-01 2.187237278839312959e-02 2.184018908249296675e-02 1.141104612805719926e-01 5.835261291661136135e-02 1.808589990331786745e-02 4.006698957337973682e-02 -5.717449791423086887e-04 5.193033156267554823e-02 2.305520669199026193e-03 5.033724416693355222e-04 4.028064624644991237e-02 2.746167578610097214e-03 -1.582016605674932408e-02 4.936707903391313393e-03 -7.793289165886804412e-03 1.186576095705588196e-02 -6.463182073625309276e-02 -1.058665509411471639e-01 8.872761821144933836e-02 -1.208307418323894422e-04 2.811001380552624260e-02 2.608821618953558602e-02 -2.368125833895461801e-03 2.566996541401667456e-02 -7.644737734697625194e-03 -2.725806103633724785e-17 1.905672805601290503e-03 5.919319356621211003e-02 1.078179560479703390e-02 3.780604812430119182e-03 1.281607946681218579e-01 1.825870138637528464e-02 -3.591505188422487033e-02 -1.131069940144422102e-17 -7.290437703074417134e-04 -3.770552528487586957e-04 1.315198478535388677e-04 -2.634865621771966503e-03 -4.869143094860268334e-03 -5.327675123935075342e-03 -1.872044605060243366e-02 -1.608512847257805370e-02 -2.335863456238147784e-02 2.336947768162421155e-02 -1.725329300450655426e-02 -2.782911735385628499e-02 -7.647552037031882070e-04 -1.969333977435116720e-02 2.953078729542792449e-02 -3.762210510317845046e-03 4.171160009681377194e-18 -3.612991215543250750e-04 -1.177049957092840896e-02 1.170730479750405539e-06 -2.695683210864520049e-02 -5.352857834922739380e-03 3.910496945651295402e-03 1.430355350416565030e-01 -1.156755035758539468e-01 6.085641818889871990e-01 4.632215302490026443e-01 1.260626193698995666e-02 1.171982204497946413e-02 2.721973597203564132e-02 5.076033143266008713e-02 5.412031208050545328e-02 1.247963029361481152e-02 2.523311966009139651e-02 1.885870754798631144e-02 3.164572266871268358e-02 8.419103412074994616e-03 2.509586759703474196e-02 2.249548478694165729e-02 2.395710416197680340e-02 3.187855412334999767e-03 2.063226469136065777e-01 9.151795594409246615e-01 3.859003479328580455e-01 4.972404027778409494e-03 4.898234517568530316e-02 2.429796866271908759e-02 5.391130286430192292e-02 2.463370386197608064e-02 2.666496970008614548e-02 3.534088342845388697e-02 8.039941978135595191e-02 1.705421197854941479e-02 2.165948456455012508e-02 5.572962170011918487e-04 1.024094467610297578e-02 1.690356425131408699e-02 9.403128773354838682e-03 3.346207031734945206e-04 1.397254565856608823e-02 9.722215378842675310e-03 4.514601496933171726e-03 9.217922399032260281e-03 2.348220160137742471e-02 1.568496264123926676e-03 5.541278373896949550e-01 1.116670257383541420e+00 8.822940651400814627e-01 3.693335357690557541e-01 1.117355222279785704e-01 1.108925093843055459e-01 5.932712709410051244e-02 1.442774565031818279e-01 6.765154925961662513e-02 1.666176168750640174e-01 4.444572134840275163e-02 1.934360541109860984e-02 3.027177124555076662e-02 3.531504453800498461e-02 3.220843268458210146e-02 1.819743791016749032e-02 9.484176739849160820e-02 4.422513385470713126e-02 5.710316745045024983e-02 8.083478311314087894e-02
8.146597601377521258e-02 8.863892515141474915e-02 8.824362560381027176e-02 4.754192342942836769e-02 7.055154794499121640e-02 6.049307069069621151e-02 7.780419142347522188e-02 2.522680613686456921e-02 4.452296658317114048e-02 9.675274386484962463e-02 5.673365573296606373e-02 -7.476018418232353857e-02 -2.360134073494785795e-02 -3.312064024321158956e-02 4.212571338508335102e-03 1.349765595913051709e-02 -3.943244997866193063e-02 4.930280526629222644e-04 1.749433536016457673e-03 1.611249556417954434e-02 1.228077041430716138e-01 1.950474756079009775e-02 2.198190332286902374e-02 7.358939264582808025e-03 2.374709038544274998e-03 1.484187434102656258e-03 6.052384933325994027e-04 -1.115121459948411331e-01 -5.706197444819233189e-02 -9.020574078384377414e-02 8.415069905125362529e-03 4.465467188099802968e-03 5.843992649494812072e-07 4.115734881683800854e-03 2.205604849636992875e-04 2.025477322980643886e-03 -3.626480293370789448e-02 7.956419730264450638e-03 7.037009036244178985e-03 1.226369498045407373e-03 1.783107673704425328e-02 2.131160875781432096e-02 -3.139207498124942531e-03 -4.577159551876045646e-03 -2.489255091732078123e-02 2.399070064843288053e-02 3.098248010507760608e-02 1.000532414814363225e-02 1.284754802484631975e-02 7.511232950373542515e-03 2.082367759959328699e-02 9.954875815453732499e-03 1.133417167502053305e-02 1.896290793176077030e-02 2.768941075548002340e-02 1.542673282291286314e-02 3.208786661929934753e-02 1.624030551765572733e-02 4.700849382585904623e-02 -1.689751719528845878e-17 -1.442028746230198527e-02 -2.664002474371649984e-03 4.921431773005720961e-02 3.177217349291818504e-06 -6.094455465774187590e-04 -7.469130002812028428e-03 -7.626177420604095432e-03 -3.102609210546567603e-02 7.136947495718481971e-02 -1.214139441453149329e-04 9.386447112469609166e-03 7.698140137805094263e-02 -7.433443690253381498e-05 -5.229629685149170174e-18 5.683124546350519501e-02 -2.500715350221396904e-02 -5.036366516846263464e-02 1.340701614349395707e-03 -1.280621708341786458e-02 4.558179287336416689e-03 8.453032539748900343e-02 4.145669567163354913e-02 2.489693420174060243e-02 8.748580334046636431e-03 4.373137412806238442e-04 3.153020710681632104e-16 -2.880567239115707640e-02 5.708066266368966302e-02 4.914397501298166382e-02 -3.306795959117576900e-02 -9.789131774978160165e-04 5.808111734398517523e-02 -1.483938349022779699e-02 9.561663454729888920e-03 -1.602270984290304538e-07 3.397687036979065572e-02 4.293778619563615867e-02 3.830838714559294775e-05 4.486131165726798403e-02 -9.709169455411824053e-04 8.630300243188977890e-04 5.797299387970991646e-02 -1.816017004063383636e-02 -2.387202986737778452e-03 9.086116441616967632e-02 1.027731227148185114e-02 -1.940805945916713104e-03 -4.860628434385171753e-02 -8.575655639764302469e-04 1.082415085050712072e-02 4.638863248424858823e-03 -4.661736851238416218e-04 1.613789463973509476e-16 4.028181975400803305e-02 1.679013255810420623e-03 -1.449178774603126878e-02 -6.728736187522336987e-02 5.562353988272809857e-02 9.407801838156223950e-02 4.280409893860181308e-02 9.999831691502310965e-03 4.035795108477591153e-02 5.128081103125128881e-02 1.383024456397718982e-02 9.556548998802566475e-03 4.433838110963101570e-02 4.061300659400982754e-02 9.935087819914322632e-03 1.318079293864008079e-02 9.867460927388640481e-04 1.431462856188678995e-02 5.853496077588868918e-01 7.240734400999266906e-01 6.052013765455390715e-01 6.539194954097742041e-02 1.719705190676187531e-02 1.091987075037639948e-01 3.925762270425870587e-02 3.558567490760507906e-02 4.683362966060782007e-02 6.266661660287049795e-02 5.045090403523039280e-02 4.467111484861352455e-02 3.591431301344535226e-02 1.929511395607648797e-02 2.395858395833956947e-02 1.080909682833609484e-02 6.587409684418178424e-01 5.746109688051871389e-01 6.034556713804419825e-01 2.211812022037581982e-02 1.212634726886225450e-02 1.028999949204200889e-02 5.394167846444856106e-03 2.232434940029627210e-02 1.192352858461378411e-03 1.038279659527743121e-02 4.074237494192581473e-03 2.369832659565755673e-02 1.505188866082363304e-02 3.242410648491569208e-02 4.412934615380370226e-03 5.987471914375317450e-01 7.889466438934663595e-01 4.496799771074208585e-01 2.428451820431226996e-02 2.566823307297761911e-02 2.204803738309637356e-02 1.763150802721586211e-02 2.923835170918557408e-02 2.949730713043503122e-02 5.509431079038265046e-03 1.576594160901763353e-02 5.784276662348715892e-04 3.457261043639128090e-02 2.014932095662800085e-02 8.949126752086483672e-02 4.181787946568946146e-01 5.048483646290246041e-01 5.630747184952932249e-01
2.552102968189858268e-01 1.388383998298652056e-02 4.120639666165223404e-03 3.651729142439388235e-03 7.581320613284068827e-04 1.670769516599576385e-03 3.159600420331580995e-03 2.609453538800503639e-03 2.425499037866456209e-02 5.578265975050008402e-03 6.555094179411358191e-02 2.052879782292762328e-02 2.590331673294485673e-02 3.555484815094683837e-02 9.231996334476826049e-02 3.682871591673588917e-03 4.290411790328010552e-03 1.620678303147756946e-02 2.179480431972448395e-02 5.614421276328446925e-02 1.413580818794737959e-01 2.299913891425290485e-02 2.237237176997036031e-02 -3.538405411923203209e-02 -8.300409605346162432e-04 4.217742884390693336e-02 4.049978514262811413e-02 2.920361294538131522e-02 8.022720985747126832e-02 6.692882377909052660e-02 1.281598971478142557e-01 1.095118386713377773e-01 1.108924238857279115e-01 9.073532949849152929e-02 5.841254479176063996e-02 1.200379056192013745e-02 3.379153069332134301e-04 7.571961724150591544e-02 -2.137106866115058568e-02 3.054024013219691202e-04 1.996954226722098824e-02 2.499723768523828479e-02 4.408496275454661800e-02 6.545119359690051465e-03 4.337819278303213510e-02 4.073353519575912546e-02 6.142333917675629751e-02 5.992178867848649195e-04 6.498048165219265544e-03 2.133084989615641192e-02 1.870183497619289439e-02 -1.008330220032239120e-02 -2.916136924614959073e-05 1.688055983721559183e-02 1.886842512526492807e-02 1.573537313263806761e-02 5.852868922263972096e-02 1.871124337777774205e-02 4.232114472463033295e-02 3.470318912352245849e-02 -2.370533753153070411e-01 -9.233361358232648232e-04 -6.654381706188329526e-02 6.794683992148969898e-02 -1.586796980684125440e-16 4.067260235814273289e-02 1.913383812905164461e-03 -2.323970896984279676e-03 6.340212690401267857e-03 -1.851896481227591513e-17 -3.336793506022290266e-03 3.718865664318475595e-02 9.414231549692532930e-04 3.809606619565602398e-02 5.716819004157666986e-03 1.821100039189640224e-02 7.617325793882954314e-07 2.620141758244222494e-03 2.078765676175965693e-02 -7.675641599633324956e-04 -2.624836244200245625e-02 9.852743345147006726e-02 -4.917243696240889556e-02 -4.932109680574094102e-02 -7.846586080631771931e-02 1.523823737452925002e-03 1.223063688321639197e-02 -1.773462686471859347e-03 7.488376946001921476e-03 4.866809177486242435e-03 2.655734605468166101e-02 -1.190559680452654180e-02 -1.490116126316555371e-08 1.973805191616912966e-02 -1.194853957030170537e-17 1.018323252516930150e-01 1.004095546467079154e-06 9.488240366999126346e-05 5.675271019902915909e-02 3.764280802604483972e-02 -1.295625803266805344e-17 5.622705037588593413e-02 -2.979493889739698309e-02 -1.114771283223478602e-03 5.019077951803897070e-02 -1.339134513923192922e-18 -5.166076162187924897e-03 1.719068038531386400e-02 2.489300527573395233e-02 -6.497123290012618568e-02 -1.063845398312805810e-03 8.408605114191022509e-03 1.905829111280899282e-02 9.126532853027995162e-03 2.741534722812452238e-02 -2.068506062162825572e-03 -3.993341927879454339e-05 1.088063269586748427e-01 -5.465553686775706090e-18 2.594544898836489283e-02 5.877392066758077016e-01 3.586681221693337474e-01 2.313644433083340141e-02 3.008083236464534692e-06 1.120813755686765265e-02 -2.098117781353899328e-02 2.321230164187535958e-02 2.801919337002395111e-03 1.286507750358524549e-02 8.351840406074556925e-03 4.194321871633410054e-02 1.818395273378441840e-02 4.818767452849590655e-02 2.349247066577900972e-02 3.019164937362920492e-02 5.627731214990788593e-03 2.354053807132852616e-02 1.494086665197608479e-02 3.804743224432014653e-02 7.150663083547548204e-02 1.370928999607642085e-01 2.316732195313949208e-02 4.088648100013030917e-01 9.650648010524285336e-01 7.020699786057801317e-01 6.386080111089043054e-03 3.266836758567129295e-02 3.157590011105647354e-02 3.548230366964931493e-02 2.887631901010861285e-02 5.556734622475661273e-02 7.482460630667860102e-02 7.814595896242101936e-02 5.478556304091065920e-02 4.928379893964046632e-02 5.065643507997232703e-03 2.256473258777170338e-02 6.905024606104053220e-01 7.958605314942678310e-01 7.861228489085033688e-01 6.728910763571742566e-03 1.297120064443945536e-02 2.770622339502178816e-02 2.248923482882185867e-02 4.024907565684418542e-02 5.853036302611368258e-02 5.999170825879465707e-02 5.525345849075368604e-02 2.063567861192407496e-02 3.229990925543698571e-02 2.983719155228188244e-02 3.722281146685778308e-02 4.382809726556162466e-02 2.473335117935778482e-01 1.011069421547771685e-01 2.307383348777908849e-01 2.071372929629447524e-01 1.191368252270831873e-01 2.171741534464491086e-01 2.386636933516577963e-01
1.556354648885736047e-01 -6.120237496052433751e-02 6.207955959794800999e-02 4.558496669501437698e-02 3.620588930783359694e-02 3.700339377188092971e-02 1.430641709263689165e-01 9.704823687550110170e-03 3.442973593650325292e-02 6.270977757997001079e-02 7.210519976033707978e-02 2.700835440050996031e-02 1.722708030463184631e-02 4.118410111207870122e-02 4.758798460384431467e-02 8.216172264284037074e-02 1.961561300729188217e-01 6.075638879969676165e-02 2.216205166219656353e-02 3.341897026614566030e-02 5.253181558867812961e-02 6.966757535174784334e-03 6.578996634301030599e-02 2.823648096737086943e-01 4.009889548768773987e-02 3.361567927179649057e-02 1.410632401801311664e-01 3.814584385923765197e-02 6.862878347744170472e-03 4.741356028460998488e-04 5.417328910999329693e-03 2.084078796790751493e-03 2.314500603704606422e-03 1.734709794692955616e-03 2.261890013912150679e-02 -3.515358900677870158e-01 -1.530458316726409085e-01 -8.861985702612934501e-02 4.181603022231777933e-02 2.122884842835310107e-02 2.972498692835040535e-02 3.289673262281023192e-02 1.466977683871013155e-02 9.768424408341941367e-03 3.108666149158794550e-02 2.817289213794680822e-02 7.157187748751107914e-02 1.525607458707280671e-02 7.729045169233205934e-03 1.150709472170860330e-02 3.600128774640461160e-02 1.004092347310957603e-02 2.635662535746693264e-02 1.240868977897994184e-02 1.496926528414086605e-02 7.825876358774087535e-03 3.996936077722270525e-02 5.075853280161006509e-02 9.181862176280737586e-02 -2.193551458820836333e-17 -7.109274116351339823e-02 -1.004892240106867269e-16 4.661540741627416204e-03 -2.134589888381669201e-02 2.290750371328923937e-02 -3.064764871089794684e-03 2.080191986684463631e-02 -1.807934035301971172e-03 1.939955983925638686e-02 4.180350849065645608e-02 -1.748207752043837507e-02 -1.199866673825434639e-03 1.470307715289431022e-02 -9.779215283584802593e-18 8.231762001726775080e-02 1.578257618626558237e-02 -1.769153852063823015e-02 3.281367344318106977e-02 -4.451848146337215872e-02 -2.268702087637781029e-04 -8.204154564049771911e-05 1.688063128230883891e-01 6.323676866371402694e-02 4.067485064227444047e-02 3.226217913324284231e-02 2.291108290287858888e-03 2.461044416552836400e-03 3.000236938788381543e-03 -3.797886597041194578e-03 4.853177720323491318e-02 2.171360845907333565e-18 1.111333173249118463e-02 2.608037493731214882e-03 -1.949588630347561663e-02 -1.032714538332049625e-03 -3.218490323366790876e-03 5.374432248838421390e-03 -1.069248883295284384e-02 1.277091243733190904e-18 -2.249935344542999669e-07 6.517130866347348346e-03 -1.509607958089552844e-16 5.718793781618014896e-03 2.219776578419436347e-02 -2.277707203211140532e-03 -1.785060823767295121e-02 -1.453072268009829478e-02 -5.281286335202786154e-03 -5.959038915601730553e-03 9.800492565534345279e-02 1.111211446960465830e-02 -1.073053207232776269e-17 9.144689039761037747e-08 2.341641741591257361e-02 3.724163343824791507e-03 -2.173718046203903381e-02 1.456406406796606143e-02 -1.020012412276205191e-02 8.097734856808185000e-02 3.679490917020069446e-02 6.472377658481638374e-01 3.842200849734562773e-01 4.654416124224805551e-02 3.120249503955795783e-02 2.606683926305670626e-02 5.551589192517147886e-02 6.629912586618723347e-02 2.222143321128553298e-02 2.592811459745237440e-02 2.898228661682425086e-02 3.629222452579165814e-02 1.556691686670492666e-02 2.500775746741732905e-02 1.155147687443322004e-02 3.432644664849231891e-02 7.600617841847188971e-02 1.372048209828272969e-01 5.212752142569895136e-02 3.394901498388042116e-02 4.002489813503894156e-02 4.195596897872551356e-02 3.331381085832730692e-03 5.570164135040128750e-01 8.708415207053770146e-01 6.094969439351802798e-01 7.138957292704011692e-06 2.583160869639718887e-02 1.739645227767577318e-04 2.053063091595726253e-02 5.831154030613195857e-03 1.937487420539350455e-02 3.724971410752348899e-03 8.621852126661760235e-03 1.161270998483594080e-02 2.857490822053263685e-02 8.152052348126587189e-01 7.860442641534208796e-01 7.784490269287503983e-01 2.404884052650165666e-02 2.667883753133730787e-02 4.057464502041321919e-02 3.359327389726588647e-02 4.153629261196490480e-02 1.078867700713896996e-02 4.085429214746126581e-02 3.033145639452157483e-02 4.392038926988098280e-02 6.815988154816246883e-03 1.969171946544445717e-02 3.576622436987770409e-02 3.239748643377568516e-02 1.431750032616366589e-02 2.266656861665423614e-02 3.139281205963168064e-02 3.608389083717290780e-02 1.385665485110906120e-02 1.016876097453056210e-01 3.237925846997196100e-01 3.766525838210246224e-01 3.355996474591451117e-01
2.614146826262800927e-01 2.422173542257244003e-02 2.326946641973453439e-03 8.235636102276891121e-03 3.606722512329635019e-03 1.152799964470631539e-03 8.814728437082978291e-03 5.716888320121379756e-03 5.481047130028808636e-06 1.886317432963969393e-02 6.232854559325574223e-02 1.603572440021633003e-02 4.136236165988968483e-02 2.128844019275640681e-02 5.066391359645822101e-02 1.176553069444794420e-03 2.223741175492139916e-03 6.183576530177680060e-03 1.326527112542276628e-02 2.079521168671575465e-02 5.913627195493811883e-02 -1.511141941182700954e-01 2.868822745799774754e-02 -9.638403984275328157e-03 4.240782176373970869e-02 7.039550842685975141e-03 2.679910034516038222e-02 9.523502027256229921e-03 2.912759244797602856e-02 4.051394566378496015e-02 5.491578108398242203e-02 4.521079223106341111e-02 9.348974381757566010e-02 5.892231300612661010e-02 6.533882908893150210e-02 5.633618717997442016e-02 2.470507638218514407e-02 4.696662405261567419e-02 6.095128291125335995e-02 3.438752789944355542e-02 1.587413301167529475e-02 3.103747452277578339e-02 3.653231252721880068e-02 2.947607647330763303e-02 9.328402091980690647e-02 1.867396354713782569e-01 4.906665566524569255e-02 1.016554192013481156e-01 8.662508932822285401e-02 7.018704503411092067e-03 -1.297603214540298758e-03 1.414982283441673953e-03 3.046929048385474403e-02 4.410275427384525299e-02 6.013454839362799426e-02 1.056084921369449259e-01 1.287051555542112202e-01 5.900247570017538856e-02 3.685334397162346454e-01 7.266368402223814804e-02 -2.123899778046542952e-01 -4.523694571198359424e-03 -5.251589632839882837e-02 2.683575528693938131e-02 -1.238654188720632182e-01 1.730298931780736502e-02 5.026961359832769760e-02 3.193088874981096781e-02 -5.024816904986626961e-03 2.558768219942408363e-03 1.970563235798640511e-02 -6.059907418369458628e-06 7.812717013528459875e-02 1.713919883586991974e-02 -4.768749079506797113e-17 -8.035471722342006932e-04 7.818707555092851286e-03 2.546199591794106459e-02 2.751695433052520762e-02 -9.830744952108815113e-05 -2.430161919054602947e-03 5.562729480349985073e-02 5.054677045501930477e-02 1.207248206240864319e-01 1.364858507539340113e-02 6.144648364047167882e-02 2.474339662272556639e-02 -1.644192952411525997e-03 -1.081419063022678135e-03 1.478155572831218534e-03 5.309622803520317597e-02 1.082440875540977396e-05 -7.851004418393301719e-18 1.039037728272596689e-02 -5.381336427135869878e-04 4.150437685423118313e-03 1.944744551630607112e-17 3.771540121817530677e-02 5.356444729961475949e-03 9.850901255142574184e-03 -5.916471681929423576e-02 4.032963669091543801e-02 -8.804645004140814260e-03 5.251880039222890650e-02 6.914685308467712066e-02 -1.317897694961482190e-03 -2.124224281389197264e-02 1.413878133652707314e-01 -3.576107833102712408e-02 1.223456418458204881e-01 1.560259245767011901e-02 -1.288134949357890892e-02 8.641937726966758548e-02 7.540839240523017004e-04 2.584989278159610402e-17 -2.208958355500435788e-02 -1.263963617317108348e-03 -2.659995537574779204e-02 -1.087793410979045431e-04 -2.037574272586777369e-13 5.754739879508276035e-01 3.617543458754616803e-01 3.077478461800040033e-02 2.850426536060630435e-03 1.302136732669876076e-02 6.301271805641507281e-03 2.356605015879900331e-02 3.532222502597313386e-03 1.028412477155460705e-02 1.724842961962365319e-03 4.168127467441357092e-02 6.112676463843677924e-03 3.983247809088594910e-02 4.578659715396134894e-02 3.435049382789660127e-02 1.447443992145259692e-02 3.696433593653550287e-02 1.778994440299819929e-02 4.110788321458264116e-02 2.108957232204927354e-02 8.349347273218901688e-02 7.488643184881791370e-01 5.831601726022527243e-01 7.508858620607072876e-01 2.606973856608111803e-02 2.462981905127383311e-02 4.901539317153542213e-02 4.447471137786272039e-02 4.055605723097909648e-02 3.245736844524359233e-02 8.063274199714011437e-02 6.947365444916210608e-02 5.557781754489237586e-02 5.000339379492633324e-02 3.836505355571766251e-02 2.423412067271771275e-02 3.340287720294907170e-02 1.273401801014792910e-02 2.575499831211900195e-02 -7.726238261440715287e-03 9.448226772081055880e-03 6.355578658352543098e-01 7.405666050526872324e-01 6.194078979372928950e-01 6.069003553237553583e-02 4.364477105191186777e-03 6.533509771973818048e-02 1.688864675600259060e-02 1.123436627669943504e-02 2.317004315799437997e-01 1.768472237688193671e-01 6.009798836274905520e-01 2.034820356186287094e-02 2.299164961107415684e-03 4.067316701137069224e-02 1.643539408883178121e-02 8.770209895832166425e-02 3.981103953128666689e-02 3.881382073806013966e-02 6.867616041556259820e-02
1.307763671154373308e-01 -8.195101326454445889e-02 2.349048079041352738e-02 2.107674984874016028e-02 2.773081983615948570e-02 1.739345154598130952e-02 1.204779387294323267e-01 1.305074604216556293e-02 1.838275633631611394e-02 5.066996171498486246e-02 8.712835362742174516e-02 3.222791550657400778e-02 7.992166923103278045e-03 3.793557275896952685e-02 6.527394960374396926e-02 1.991212866292900105e-02 6.331927647741393550e-02 -7.384282045944066952e-02 3.710357330014128069e-02 -4.687679371353585533e-02 6.934015257364511964e-02 6.011977319546927059e-03 9.239383898509376594e-03 3.147838677883901815e-02 1.239284822204943931e-02 2.898935270485560392e-02 4.459069700213917631e-02 2.714582766823282756e-02 1.132928589048534142e-02 4.785580866801363026e-05 4.448872103262172667e-03 1.663151268312541104e-03 3.836569055650033831e-05 4.724164909058395498e-04 -1.891851419143897040e-01 -1.345677781465206557e-01 -5.473891367944101749e-02 2.475868858358233875e-03 6.884659582817454458e-03 1.616454224917808838e-02 3.446748888736425598e-02 2.508679036827701769e-02 1.925099617805286698e-02 2.630748911144947452e-02 1.440443991999141617e-02 4.302729368395239590e-02 4.558639046144547652e-02 1.028662513578678349e-02 1.336677925289658786e-02 1.488402090236853707e-02 9.233692031406254275e-03 1.768009602986860690e-02 -2.927562498658884023e-03 -1.775722267916732960e-02 -2.919127100894980745e-02 2.819074287754281052e-02 7.414511082650110385e-02 1.351349142658095914e-01 2.107035388129325226e-01 1.320337449305738486e-01 -4.959963567592112071e-02 1.867952812013280815e-03 2.373741218622735691e-17 -4.941369351469850763e-03 2.838805229249380391e-02 -1.988775520478998707e-02 -1.317310282559820299e-04 -2.427998986570320018e-03 6.507207664766039013e-02 1.934089671925356274e-02 -1.793054980203948739e-02 -3.351517946302799179e-03 1.969148093289869311e-02 -1.422410310181614155e-03 2.809427476080789132e-02 -4.832242112134754857e-15 -3.237950452225028114e-02 1.059296628781277033e-01 8.164589482040222890e-02 7.351622577914104784e-02 1.307774833633511824e-01 -8.787331491228385014e-03 2.357124341241312055e-03 5.861706291123274523e-04 9.133241449918676202e-02 -2.912401653242586638e-02 -2.893563024942481389e-17 2.048202997975029360e-02 2.509258005152149515e-02 -1.430409312833333911e-02 -3.973696650385399182e-02 -2.900725583490369394e-02 -2.864356402491946216e-02 -7.561583343986369964e-03 8.575722434722029240e-04 9.294814938410521366e-03 5.596698525979224709e-02 -1.077189222051574795e-02 7.997827792748742406e-03 -5.245580184885902109e-02 8.335787555766291162e-04 2.523510985732267328e-02 -1.135180424975426247e-02 1.807544847189313398e-03 9.561180742682167309e-08 -2.173217922887802502e-14 -2.483683019796148212e-03 -3.471861167410746855e-03 -1.008262315082610223e-02 -1.351445891555141241e-17 1.610676597960393475e-02 7.845518414789715705e-03 -2.964797863176263457e-03 -1.322831774060661224e-04 -6.134542116433898103e-04 3.507486894530897209e-02 -4.633463434043094426e-04 5.760563459160474714e-02 1.782849202161637392e-01 3.746454311682963917e-02 6.377692203618969824e-01 3.350923758348945403e-01 2.520621163223871544e-02 1.000212128684730001e-01 2.249118789868244719e-02 6.534209253675335449e-02 6.552669451269740875e-02 9.237931958045883518e-03 1.403825807034250750e-02 3.346869850365665616e-02 4.273686886401480839e-02 9.489942962701516938e-03 2.468657804054290306e-02 2.355076420414586869e-02 3.125042478573249877e-02 2.120565095420751459e-02 6.958907828509452997e-02 6.519480073289880551e-01 5.555606104917880650e-01 7.203630197468831042e-01 2.799128594706217985e-03 3.015023115272539266e-03 5.542242244415141600e-02 5.367997295403673791e-02 3.564216729295666131e-02 3.564637025291859246e-02 6.596474565556098568e-02 2.851422053112047383e-02 1.602768267515165129e-02 5.465651923028254713e-03 1.712748319711449724e-02 1.426233040143062500e-02 5.418859849193801911e-03 8.442686668009919543e-03 3.474699552254916224e-01 1.023825334275831223e+00 5.307267670927030556e-01 -1.436106776880427711e-02 2.009659364417127475e-02 2.013082317583729866e-02 4.834753894064647284e-02 2.001079709631147299e-02 4.292977031367725682e-02 5.957571390195676164e-03 3.137764167956514499e-02 3.364319091193063160e-02 4.680521386034540432e-02 1.330016734214367440e-02 1.846092177706096238e-02 2.053554091245095997e-02 3.061345155272509819e-02 1.823051980923647442e-03 1.170997396633260690e-01 5.903423815585543410e-01 1.932130057984641214e-01 4.573922957589296456e-02 1.013515499618832388e-01 7.562834103692332721e-02 5.768966557765156655e-02 1.090291141755111992e-01
2.518648724331276068e-01 1.018609074798291006e-02 1.355864031115067256e-04 2.212088682063139170e-04 -2.925093407479744181e-02 1.731808427826743262e-03 2.480466529563954414e-02 1.979307577521761344e-03 1.325011016840840078e-02 1.576656731539964953e-03 6.515289875040944367e-02 1.192283369678326030e-02 2.974124786758318428e-02 3.604257413130228238e-02 7.976095012808319396e-02 7.180066239709583485e-03 1.283459333665617173e-02 1.769257442550286122e-02 2.447773108441014464e-02 4.615119044862887182e-02 1.293598812430351019e-01 3.338604829326314444e-02 6.112601024659014906e-03 2.652843026511730873e-02 7.128559234315662296e-02 8.470150033879320084e-02 6.747863674489637531e-02 6.181717760655977556e-02 -1.129561657401764382e-03 6.146891239311753907e-02 1.425923494183257423e-01 8.937209168768840695e-02 1.555771237163535436e-01 1.172218767122231442e-01 1.201159867062427389e-01 5.482419952469901742e-02 1.659636048237209810e-02 6.316569261937285407e-02 -3.710564989460437418e-02 -3.230152878806494959e-02 3.641481005489642653e-02 2.164809348724347771e-01 3.462454649865926437e-01 5.724132044166113653e-01 3.533969547143246959e-01 3.700211425555137157e-01 5.362399652072128431e-01 4.261456828076706782e-02 6.843473603966888918e-02 2.206986978649711634e-02 3.605148398689653894e-02 8.759703801658372257e-03 2.489000519241377912e-02 1.469920032896034588e-02 3.534916091466167726e-02 5.072028497858678542e-02 9.513091105876317344e-02 5.303715300384881370e-02 5.811559005634839986e-02 6.691132536419311383e-02 -2.176496535960663337e-01 -1.122836107919994442e-03 -4.995147278461124773e-02 2.237244337587715112e-02 2.351620313557033215e-02 1.746542755105894365e-02 -3.896838318981370956e-03 6.855395697748508516e-03 3.586826977866263214e-03 8.915643295259609302e-03 -3.720658551699938810e-02 -5.268030764195183075e-06 -1.198218992121501418e-08 6.800652783569638773e-03 -1.970635055959963494e-02 5.271212545473405973e-03 9.200885080169381993e-04 4.529915274913710876e-02 1.616567352939233052e-02 -9.634023797777534377e-04 -9.623905187213913082e-03 3.263144324120310219e-02 -7.060087769254524276e-03 6.312729563306770583e-03 1.537019843600679909e-03 1.840313894299411756e-03 1.002797170379151611e-04 -1.313455233368689000e-17 7.256425917617700949e-03 1.984065157457172489e-04 1.829937397766662754e-01 -2.581236015876757495e-02 -2.433459061348287706e-16 -1.350064176084182221e-01 3.994830274391486524e-02 1.702761202821732531e-02 4.151294146741880527e-18 5.515837310721340581e-04 3.103171831487862992e-02 1.430624570852229993e-01 -7.399121744874735673e-02 1.901632267996367920e-01 -3.311773044596789672e-01 1.381834587206488482e-02 -4.037651364812415433e-02 -5.555677940838331386e-02 -2.095381411890340217e-02 8.433430755375158350e-02 -1.191274342631023532e-02 -5.308016560719880936e-04 6.765780469314411349e-02 -1.823261288556990087e-02 9.217239647362552787e-03 -5.099944392791425779e-02 1.459667984694960202e-02 -2.519094932999980940e-02 2.230427628962635550e-02 1.745235896913826731e-02 2.921808609940788545e-02 -8.703331566971293885e-18 6.165993630859962771e-01 3.777945885428030670e-01 2.446892052297057474e-02 1.029361070780253741e-03 1.823997644329737600e-02 1.155466712484764071e-03 1.883332463657644285e-02 1.923367027737492903e-03 1.056529881256018828e-02 5.962883955503622607e-04 3.806974497581738087e-02 2.544287240129389140e-03 4.843995112677505072e-02 3.651029359229933380e-02 3.857170081932579514e-02 1.704384588236155032e-02 2.500172569831317707e-02 2.224473276016736625e-02 4.855856574424834843e-02 8.259263382664411290e-02 1.258314287330806469e-01 5.735619584421217804e-02 3.388870018685441465e-02 1.851072182971628260e-02 2.541866644756123755e-02 4.083821997948627013e-03 5.654244056620282288e-01 8.849624404930436228e-01 5.710248112784923613e-01 4.397189541076171847e-03 8.662460261122181468e-03 4.843623317475340140e-02 5.386703262946991588e-02 1.099760820524738197e-01 4.287319582451580569e-02 -3.124810345169282827e-02 1.592820115234844236e-02 5.938504038722828282e-01 8.207292160112821255e-01 8.672241751724757419e-01 2.587260469416688413e-01 1.073647793405114736e-01 1.735060944067465943e-02 2.541380317645308695e-02 6.480695660682964099e-02 2.817421195537718037e-03 1.207202761713046558e-02 4.260794536551132938e-01 5.577711004186995947e-01 4.396021335087701432e-01 1.913713382169491758e-02 7.923419634192423525e-03 2.180821107528509509e-02 5.906528085377659582e-02 4.314080225364776838e-02 2.515231952268107488e-02 1.135690035905397993e-01 6.128875020338493684e-02 8.411575541007984547e-02 5.234356668238707849e-02
4.655768172191020356e-03 3.389974637652255762e-02 9.390765384224145085e-02 4.581640800858168688e-02 9.316856852260776978e-02 1.080375328156080172e-01 1.529552667917316866e-01 2.582065731218016746e-02 3.448698659890764295e-02 6.994437846803411296e-02 8.583667399666861098e-02 1.623330606122904557e-02 6.061533289724987950e-03 9.412199031320280136e-04 2.346586189296877983e-02 9.283716407612965099e-03 -5.954101174044591077e-02 1.268236606376061011e-02 7.054809379813216028e-06 1.221695359113180064e-02 4.610037019217151644e-02 1.646088747688448026e-02 1.409291385686228283e-02 3.352127285591743888e-02 9.970287466774848794e-04 1.814789296554284587e-02 9.567743449613842766e-02 1.206281051950897076e-02 1.221942758835463651e-17 2.080280417769542591e-04 1.291041429391447115e-02 2.346007907341660733e-18 1.116190744973685582e-03 1.163702802668186874e-03 4.571160907123263117e-03 1.618851652896310944e-02 2.425950690902732021e-01 3.297114067798664894e-02 -2.283369603768975811e-02 1.994713301172165373e-02 5.025718108952750146e-02 4.579248484491450777e-02 3.519692490789354949e-02 1.030328747930674138e-02 2.915056640005782643e-02 6.294129209461576946e-02 1.128530593632709189e-01 2.806220270469908171e-02 5.298158859767522488e-02 2.083898897785748522e-02 -8.892311686678213659e-05 3.396289789701553014e-02 5.470834516783600054e-02 4.488990985303042969e-02 5.119369042703285355e-02 8.861161243746895622e-02 8.521959826708809527e-02 2.025054901133516994e-01 2.636240020843204079e-01 1.014753176577383870e-01 -6.102959731641017377e-02 1.504653628624598102e-02 5.563251659302870245e-02 -1.775413341267216083e-03 -1.094927665750447055e-03 -1.607305662806402056e-03 -7.800469003649469717e-06 6.809828394657876927e-02 -6.696781190657062620e-17 -2.423669677601651804e-04 -3.166477104666268194e-02 4.447281883494744181e-02 4.415488326456241952e-02 1.210636665092434293e-04 -3.752561596934985561e-02 -1.564981188676511925e-02 -5.126839477413667329e-02 3.339066486306271897e-02 -1.001326352392996159e-03 1.429902183632356105e-03 -6.231125168365355589e-16 -5.157871069113241676e-03 3.989663483611304712e-02 -4.835561853743659128e-02 -4.740960014161048862e-04 -1.697610512795856910e-02 -4.597548082360520226e-04 2.548099155997920334e-02 -7.550281293447524406e-18 -2.732926601292325291e-18 -1.698543198317077271e-02 8.614597746947279299e-04 2.988618105276775322e-02 -1.044751125644167306e-02 -3.529252521549971317e-05 6.792764219585510112e-03 5.939417938959811599e-03 8.911524687500464725e-02 9.544167321026941764e-02 1.226470788566609710e-01 1.855641122573372867e-02 -7.390527992513279062e-04 1.431421220820741787e-04 2.705071147410152092e-03 1.673730072687489032e-18 -3.515588381121102341e-18 8.356594331735696379e-03 -1.318189525129601428e-02 -1.023037722963748136e-04 1.044213163076767037e-01 8.508732777535299441e-02 1.382949032146968101e-03 2.647318079902224661e-02 -2.550032217590985500e-02 3.554597141936498630e-03 -2.169795708510411855e-02 -1.447957464171681710e-02 -1.501571918155973975e-02 -3.334914521369312582e-18 6.935247281755963682e-04 6.033595496236742600e-02 5.702635359007496990e-02 6.060548231271629255e-02 7.358097287858723912e-02 2.159413701120071680e-02 4.805536801371806249e-02 4.581761140504725383e-02 9.734900416839836410e-03 1.341229965324783509e-02 1.390014590668203764e-02 1.663760256463161710e-02 1.360845276867516827e-03 1.123875238016255884e-02 1.960635978334252238e-01 -1.358534071437360047e-01 -2.319099714258445188e-01 8.069574152864780747e-02 6.628802149445936140e-03 2.308091518290365102e-02 1.168047294447250792e-02 4.219985994440898780e-02 1.908778487040323560e-02 4.748505869941010982e-02 3.031902271011187311e-02 3.960688043880357390e-02 3.204826193600792544e-02 6.469409478985253814e-02 1.121607862222964713e-02 1.884326888249669013e-02 3.631008762886046241e-03 1.466645985761005737e-02 1.140250317777009338e-03 9.117435238836405637e-03 3.804557570707428767e-04 2.708597388230347611e-02 5.815146897600970101e-04 5.411204776268919714e-01 9.786560861320422466e-01 6.060606767302111342e-01 1.001878994958484148e-02 3.331906766694543720e-02 1.995587573641104118e-02 3.230571901269880392e-02 4.086771563307048903e-03 2.813673487358259867e-02 5.495577775428723416e-02 5.828852683583114758e-02 6.057365835203116361e-02 3.533332936283773984e-01 4.513570256834019134e-01 1.955307015383747904e-01 4.759463098821989889e-03 1.929092800980182487e-02 3.578349249644838942e-02 4.492953014928320610e-02 2.925040259347519087e-02 1.150410853157545904e-01 3.638440546002487397e-02 9.025581242734347087e-02 1.309035997685980834e-01
2.522758686616754509e-01 7.320871274640082874e-03 3.148125797414884114e-04 6.671556861658293472e-04 2.124384930536730499e-13 7.023788966569832846e-04 3.011931411207249918e-03 2.451327829221516211e-03 9.472111288310203908e-03 9.500986210731671155e-03 3.303005965406186767e-02 1.408746862027817834e-02 1.020735287876532553e-02 7.968877191275255262e-03 3.854876021158148286e-02 1.434968406866204797e-02 7.093282074209311672e-03 1.856496668810487735e-02 3.793823430164639660e-02 6.278492848039844698e-02 1.588799807704339706e-01 3.939499591069948320e-02 1.633866202528117678e-02 -2.512512129232346084e-02 -1.117900140133503029e-16 2.198671170635284333e-02 2.358133113054295196e-02 2.642505630304223946e-02 6.239304779517867300e-02 7.170090602617960884e-02 1.688213564205873485e-01 1.116966250889786655e-01 1.222610848015384527e-01 7.001132765097750621e-02 7.052534343759526980e-02 5.238532622757012362e-03 1.452411745227608162e-03 3.454593043385513484e-02 -1.332386413071093890e-17 7.655487467712919492e-04 2.170308479614377860e-03 2.494569747780909022e-02 4.301830405557073578e-02 4.450835510925483553e-03 3.226973290167287067e-02 3.958504992892304675e-02 6.057596013722325867e-02 2.328985704647163965e-02 4.808796411101319557e-02 2.345111559678154586e-02 4.974472556317411504e-02 9.079230720239656743e-03 3.868908581562522514e-02 -3.741561344350859044e-03 1.044109087038300924e-16 -2.938556048757390987e-02 2.990805067215887569e-02 -1.024834127017437008e-02 2.993842887650901164e-03 -4.617456573234243744e-02 -2.226683174956094746e-01 -2.222125737950035151e-04 -6.238551540389099259e-02 5.198628822542626648e-02 1.813132090540813673e-02 4.259121223727650729e-02 -1.553011230958214854e-02 -1.167272309720663132e-16 7.327338665606738209e-03 4.203254655729061671e-04 -5.820815717746908799e-03 2.772995575774969690e-02 5.694549015849921933e-18 1.196172556241854347e-02 2.536069696653722727e-02 -3.925136865599002274e-04 -7.978858757023081960e-18 1.537296474803937808e-02 2.352389218628865964e-16 -2.128686287048350226e-03 -1.600584402806639467e-02 1.930581311656374699e-01 -5.591206864439988294e-02 -3.662423955265802639e-02 -7.805469626849009801e-02 -1.081214533955836450e-02 1.240280650621119508e-02 1.505384642577439411e-02 3.092052426376070556e-02 -1.295757634836053546e-03 3.647877160826421689e-02 -1.640796613591749553e-02 -1.356663226985873162e-03 5.595143056792603944e-03 -2.219479396587861753e-02 2.076918602839578683e-04 -1.094911413365294052e-02 4.264655502608766367e-18 1.037472221904557276e-02 2.806610099915765079e-03 2.841992539943807763e-03 2.271820101806259692e-02 -1.041351239860563091e-02 -1.273987110234982810e-02 7.912367557717486354e-04 -1.884140071510249578e-02 -6.011764026849505428e-03 -2.296942729479908071e-03 1.575654473333272521e-02 -5.012258757041762732e-04 6.649653673963615980e-04 2.300260913679254282e-02 -8.311254214708624355e-02 -2.430840406739554881e-03 -6.236080161388812652e-03 -2.050267638423650118e-02 7.627070426675357055e-03 -8.605402810297343996e-02 3.313199877829946932e-02 3.242868614483195022e-02 5.924488580849684993e-01 3.364138743232297224e-01 3.163190884542316228e-02 4.452628382410786338e-03 9.331244691481251421e-03 2.895496919436959735e-04 1.927922689722956842e-02 5.406800563344736112e-03 9.533681797326888988e-03 3.630372571810834992e-03 4.595803792562135753e-02 1.253066851483105627e-02 3.576810021643150178e-02 1.309336324654141077e-02 3.772452894480364527e-02 2.115874281299818041e-02 3.334456034569703575e-02 1.819729101058781620e-02 3.845765731246989644e-02 6.968396334563843608e-02 1.071402966135887175e-01 2.222507732806909900e-02 4.620225492411009771e-01 1.083913445916137697e+00 6.214321601494587899e-01 1.692437038237199626e-03 2.923357722437080336e-02 2.907579800632409742e-02 3.189191512785564259e-02 1.595896603860603655e-02 5.346698215701972678e-02 6.926824337911877127e-02 7.651541216338607343e-02 5.025313394244786125e-02 3.890148218496683463e-02 6.295907326054933813e-03 1.887721395290289580e-02 4.130853428253952186e-01 5.977682534694984673e-01 8.299319958050725976e-01 2.994854688999827163e-03 1.625872813378606563e-02 3.835926962287575237e-02 1.459377587548645050e-02 4.287715142805749446e-02 5.404599172519933270e-02 5.673323570381043346e-02 2.550798972624834099e-02 2.554360494120197847e-02 2.026153621302561464e-02 2.939282399259274606e-02 2.183053933489838283e-05 3.709871421907655082e-03 7.094661540764057861e-01 2.986899495341543975e-01 7.411916265496671929e-01 1.947248886132105505e-01 1.900784103186819729e-01 1.563306367016080722e-01 1.247666001198445007e-01
1.347721522785603199e-01 -6.707573653474653563e-02 6.341083133632954838e-02 5.741763492717145284e-02 4.517921902577597820e-02 5.967007905650095062e-02 5.598632346022795747e-02 1.886322936503772446e-02 4.961398618168039870e-02 4.364052755052318100e-02 1.054048361283567714e-01 4.015697597519064105e-02 2.136827270420917427e-02 1.927885326816489164e-02 4.239629228515571363e-02 7.521384317914636686e-02 2.223016395027769265e-01 1.071721182823868468e-01 5.435527118723652851e-02 2.542710615009709935e-02 1.121790366177601306e-01 1.437626599655759951e-02 8.008202588796931032e-02 6.605918489246460912e-02 3.578293765406369287e-02 6.962100323437660221e-02 1.027745568616149213e-01 2.770826305309178825e-02 6.988173183233662702e-04 5.456252687372487573e-06 1.553111080982109288e-02 4.042014860065092953e-03 1.412123637330374611e-03 5.361819624571149447e-03 2.588219524375194591e-02 -6.545230857653620005e-01 -2.928425295081628077e-01 -8.021999249419549394e-02 8.335817562932823499e-02 3.242299348415999399e-02 5.040766351525195754e-02 3.075389495408624441e-02 3.727876163727032038e-02 5.736613654009195540e-02 5.913445798840637202e-02 5.015768987295985554e-02 5.166625584423421153e-02 2.999506505885424570e-02 4.861240922465430436e-02 2.686740002022688922e-02 3.056703205718947283e-02 2.271739078630943219e-15 9.046732311135084748e-04 1.063916203694973903e-02 4.844111950683557821e-02 6.181759381116627705e-02 7.444365308491208977e-02 1.758468921140509400e-01 1.729109975672306065e-01 2.494650843428034351e-02 -3.630971978227569286e-02 2.510621741941318052e-04 1.257781366833330555e-02 -2.814685965794595790e-02 3.689254165364038579e-02 -2.475703756926707355e-02 -7.635422347052587839e-04 8.650058745490998759e-04 5.440314570724887749e-02 -4.196914355111256746e-03 -1.536156433345788573e-02 -9.409526937396333821e-03 2.216448764955196785e-02 -1.550493690316537450e-02 1.037929243529452789e-01 1.609019528874783625e-02 -2.756007014558738838e-02 1.480672534610970488e-02 -2.545052824939761024e-02 -4.727603290957052562e-02 -1.383760868771210725e-03 3.588971553878466531e-02 2.762531443382867588e-02 8.563339250616260707e-04 3.665883182933390994e-02 2.543663476575944008e-04 2.883613857118536017e-02 6.049562035702867394e-03 -9.433618484991402459e-03 2.390404766068420297e-02 -3.189640347203742850e-04 1.927215552097755018e-03 4.031193134420406853e-03 3.141400631013828784e-03 -6.069697950001978976e-04 -1.880022947598859392e-02 -1.400116569001781663e-02 -2.218193184189223843e-02 4.285829724698391602e-02 -2.020833332085065701e-03 1.286444947955336406e-02 -8.724752266260503828e-18 -1.813652177531969306e-02 -1.995971492030712280e-01 2.075245537611516977e-02 -1.921636591964153221e-02 -5.612504178804108602e-03 9.175837196094190074e-02 -1.857332134049643121e-05 2.058854063896166028e-02 6.739188288994130138e-02 -3.128503999759265491e-02 1.957869112818405327e-18 -3.317108322209335464e-02 2.094136929088886642e-02 -2.336425214671066994e-02 2.745862227404565967e-02 3.879501194611578024e-02 3.115336417843170627e-02 1.780015021462398594e-02 6.472248617105824664e-01 3.797434329327463165e-01 6.561957460235467998e-02 3.335229392961253786e-02 2.422761673815369138e-02 8.639692040993444955e-02 8.214380966320697364e-02 4.140495640446301834e-02 4.119212872034932332e-02 3.347723654707863983e-02 4.045386230202870775e-02 4.759441580573267085e-03 2.586979501841040960e-02 1.656545528555923574e-02 4.656515084206912236e-02 7.898458361641516601e-02 1.758551294315685021e-01 1.006209035141161640e-01 4.545186217581709959e-02 9.290726178951419501e-02 3.554429438730848795e-02 8.513166885109745008e-03 6.091984893613874297e-01 9.682027684434126868e-01 6.366250770762226230e-01 2.087007961277199865e-03 2.873276187689665501e-02 7.143078140490520880e-04 1.342009853255527775e-02 6.227575117784256258e-03 2.117401669388722929e-02 3.582491955440907094e-03 1.315130431829437778e-02 1.429025068813050831e-02 3.231355431314845983e-02 5.954936883076394905e-01 6.870589282405934473e-01 7.572656092814448181e-01 3.580227975099834620e-02 2.005086666645832308e-02 5.497228025585708389e-02 2.472110386583757646e-02 3.200606487966792951e-02 2.357426009698462659e-03 2.405735618577200738e-02 3.611328342413448744e-02 5.444954435374526475e-02 3.917313800504056975e-01 5.154415461264530984e-01 4.214190924813573935e-01 1.997678755022301578e-02 7.203085527216386835e-18 6.578564266307228292e-03 1.376815202539641292e-02 5.051792722606120167e-02 2.830371899733091084e-02 1.012938469866002111e-01 4.233571921382959374e-02 1.243367454767888014e-01 7.767801997836906724e-02
2.164015366155148112e-01 2.357133980177470237e-02 1.825845502265808790e-03 2.116350720175387448e-04 1.774305031737099820e-03 1.985644881626550794e-03 4.346487248354452028e-03 3.574614218448799244e-03 1.906918001079946356e-02 3.278931507304275724e-13 5.491151466470308579e-02 2.782784231451518060e-03 3.856503728240401330e-02 4.321274402353800226e-02 4.785640572872976173e-02 7.833477578761970789e-03 7.010549440047410437e-03 2.124150574336321554e-02 3.213631611536910820e-02 6.333326401357090851e-02 -3.297204653680776976e-01 -1.945892620951313878e-01 -2.612939462832295501e-01 8.392744453378318814e-02 1.493427142808529950e-01 7.660351982812854676e-02 4.617899632162246470e-02 4.441840688166359685e-02 6.728819758705496812e-02 7.513137138030624718e-02 1.469125778377876890e-01 9.318078870542112480e-02 1.263683810409593211e-01 9.261494772582608281e-02 7.046226690711498986e-02 1.448950207475592153e-02 -4.459870007814444037e-02 1.198989381719057709e-02 4.905583833194367138e-02 2.824386743558239847e-04 -2.178015820502258726e-01 1.202990577742895674e-02 3.210592693504121797e-02 4.265958285249336207e-02 2.102862459609984233e-02 3.679922448333122764e-02 -6.505114819540337712e-02 -3.310673920280185478e-03 -2.075795998898736366e-02 5.358129049507882879e-02 6.007686053375868585e-02 7.981326874357380294e-03 6.210875529657392848e-03 1.555017691448452172e-02 3.230935498866642497e-02 4.120167878324273308e-02 8.193474464419608916e-02 6.320428373589588589e-02 5.056917373616615086e-02 2.893842444867661748e-02 -2.190333075559149079e-01 -1.270920430500127552e-04 9.558445642796708705e-03 3.461979632409052654e-02 9.447969880995241154e-02 -1.627043182105045865e-02 1.007893505316977134e-02 -1.019100484779419296e-03 5.842285868608381415e-03 -7.091694338206045584e-03 9.861206175774019436e-03 1.685313733661984378e-02 -7.922443736091503222e-03 -1.811908154410508953e-02 -3.942459869147300726e-03 1.057136581930400582e-02 3.225838356665879092e-02 -9.902669793202036400e-03 3.615850647344206331e-02 -6.577475127842504106e-03 -7.393990937194550012e-18 -1.275540856606199036e-01 -2.689817301960678031e-01 5.075629391138095775e-02 5.109632412239783344e-03 7.741500645844738918e-03 -2.107091012681324912e-02 5.829912584958507442e-03 4.952852995340740084e-18 -3.761336338155536231e-03 -9.848370571336367017e-18 1.750524941794340261e-02 -7.670228624618600187e-03 -4.349161278865060209e-02 2.088674216827893226e-01 1.554949570272680937e-01 4.550333361676039273e-02 5.768008236864985427e-02 1.722494041510248125e-02 -1.347820017323072617e-04 4.896490413790806177e-03 4.128430928356527363e-02 2.726736209082514933e-02 -9.331016765551857287e-03 -4.629076584472581468e-02 1.905025866826804207e-02 -2.534367151438844906e-04 -1.719114515268250001e-02 -8.723991116083571525e-04 3.414185045423350634e-02 -1.966342234151745999e-02 -1.243426541671695425e-02 1.168166146177223273e-02 1.176851977739191936e-03 2.832994553259861326e-02 -1.974339569117862861e-02 -8.425543184672489071e-04 1.949209299147651561e-04 1.892129879650703839e-01 -1.408485674276559359e-02 4.096831005163489858e-01 2.721092244684185646e-01 2.068731378128368242e-02 1.426904037985072442e-03 3.409064618052190434e-03 -4.823706182699304817e-02 1.819885436213066615e-02 2.157631031298394998e-03 2.988554180827213830e-02 1.408531455284845087e-02 3.913712200692621296e-02 2.109629418864789399e-02 3.486942746307070823e-02 4.007038316094681252e-02 3.656021648693503351e-02 5.155557797028418807e-03 1.674580728703554261e-02 1.535294282659744811e-02 3.320113054423925320e-02 9.603964542915649494e-03 4.058779144806969885e-01 1.182807564648905130e+00 4.860481164794294151e-01 1.462555094310936813e-03 2.849633930754539440e-02 4.799637536660689673e-03 6.049830703363538731e-02 6.158651754239053561e-02 3.650927617377522361e-02 5.984674947213144969e-02 8.053280118186177672e-02 9.632356578429940308e-02 9.940236911419497723e-02 1.164326030032043491e-01 3.256576088153251974e-02 3.778794492500817115e-01 3.624236037190642912e-01 2.561041415970286561e-01 7.371635011507153383e-03 -1.232901221450884988e-01 1.884380068191080773e-03 4.352661050298962768e-03 1.519426752206964708e-02 -5.777449263476779756e-02 2.435906422682998246e-03 -9.152306859937801853e-02 4.450782508429767526e-01 7.942124036126954056e-01 4.056223885395352502e-01 1.192732126049706430e-02 1.605340033389155915e-02 9.886689162695349101e-03 7.869845466672161297e-03 1.818528226083444485e-02 4.294112476276189944e-02 3.014717935457509115e-02 1.246076586444638673e-01 7.437225837425232022e-02 1.051162594872143835e-01 6.019905172822288125e-02
1.259237321688464473e-01 -8.384859037838648943e-02 5.841024570245359482e-02 5.922161893018160900e-02 4.921689967188320330e-02 2.766052861762458015e-02 1.099030388692185806e-01 2.393087919134331137e-02 6.029295121931711743e-02 3.013444318729279967e-02 9.614032450063636492e-02 4.364489451087353450e-02 2.066506741532671276e-02 4.214286330880778059e-02 5.249010447859338985e-02 1.007611728523033906e-01 2.674141961773044418e-01 1.180520823073823999e-01 1.014930390925431608e-01 7.687278716892136099e-02 2.029369202799295724e-01 4.872066919433083731e-02 3.600499016301748206e-02 2.710962317875202643e-02 1.057131263117711169e-02 1.819160126192702065e-01 3.673440994324765466e-04 7.926508507638147308e-03 1.486786315458394313e-02 1.834036738674439150e-03 8.097001150992761201e-03 1.202924099190063245e-03 9.045736719995254574e-04 2.433099846802107462e-05 1.498624312893745945e-02 2.173015079478057832e-02 -3.191752187509508398e-02 8.175423902181555178e-04 1.021129075380952482e-02 7.923105981628942793e-03 -4.823604190504897013e-03 -7.814163704312164851e-03 -2.124089587815481892e-02 -1.501312673563501288e-01 1.171529745862733979e-01 1.809539959592071090e-01 -2.634429352772287708e-02 -7.579198430289792177e-03 -1.250808122882915083e-02 7.524999155367434467e-02 5.893713891008610373e-02 1.368090360119846721e-02 2.568131684466847323e-03 1.615497706020035257e-02 3.523936240393005931e-02 3.933631015781161033e-02 7.392000333545341306e-02 1.244431009986929487e-01 2.708676744893688815e-01 1.744406422169895304e-02 -3.866878550868176750e-02 5.931900431721849609e-03 1.028971530372177826e-02 -6.652122188981579587e-02 2.019913436291899747e-02 -1.105273395912978983e-02 2.621041160202366646e-02 2.789071127512992090e-02 3.038899026902916467e-02 -4.130827049785235628e-06 -1.750668341258628663e-02 1.124271292170942599e-02 4.940668123777101906e-03 -2.538899510919614272e-02 5.534855982235621641e-02 -2.399731598612184057e-17 -3.553825365234437178e-02 7.584181667161676854e-02 -2.635037878784073509e-17 -2.488576488474925075e-02 -5.333968490331421261e-07 6.460680794191409580e-03 1.002586996283039676e-02 -1.720242621225390525e-02 4.918760953971360039e-02 6.502142633537693325e-18 1.710816762162510260e-02 1.704725706409484717e-02 -1.026413569336748211e-03 9.949753451326010401e-04 1.379731415387852080e-02 7.354357958868627809e-02 4.430244026261112578e-02 4.699427151597149188e-02 -7.429523605518723696e-19 2.235760518048467496e-02 -5.913193666353370884e-04 3.808781764133496034e-02 7.479206323246164606e-03 -1.880249075573766845e-02 -1.364477475247268326e-02 -1.973826864597868352e-02 -5.384330925703870174e-02 6.727051561024259552e-02 -4.639778603313066824e-03 -1.411458230239975575e-01 5.028698740396760763e-02 -1.954807597761024085e-02 -7.698514664116932506e-03 1.385181495578298276e-02 -2.041543552466039760e-02 -1.642075150591581004e-02 1.657966305606574636e-03 -1.502131224867840446e-02 3.073612460412413622e-02 -1.919747786452922350e-02 2.421651043266037640e-02 -1.849268840385195206e-02 3.006773060126648189e-02 5.368367017622041604e-03 6.318863818768626350e-01 3.960767501699399840e-01 5.683303293377865717e-02 3.819073247981017577e-02 1.560305100091776212e-02 1.321213851015378693e-01 8.523636187182585811e-02 3.106117144518014128e-02 3.212651987705240908e-02 4.223819507873764062e-02 4.608866595870989014e-02 1.688950234153495378e-02 2.226761969352289774e-02 1.236416865883471251e-02 4.143132785258537426e-02 9.623447522613841343e-02 1.220216124413935621e-01 5.478039164857775173e-02 1.790019542290123208e-02 5.973631328287024728e-02 5.224794892901286902e-02 3.187687344060328304e-02 3.465126216135798465e-02 2.959283661422076883e-03 5.560072783158005461e-01 9.540703808867601055e-01 6.863091364094827673e-01 7.632941693994435561e-04 1.224793507182787150e-02 2.116193962306761340e-03 9.118845449439624828e-03 1.371012168444701342e-03 8.925333686478376452e-03 1.136931771234007960e-03 1.067262939677312769e-02 3.972705425388958420e-03 4.388653599702603721e-03 -1.581613507890431911e-04 1.719149045104562123e-02 2.369719556782981518e-02 5.450128728367956699e-01 9.241828328052544839e-01 7.228359113224550603e-01 5.333191075485796961e-01 4.538452572674182006e-01 8.061591865349455599e-01 5.028027331872055861e-01 7.154252244548274797e-01 4.090590981831127571e-01 4.749010317022997604e-04 1.677801271665948016e-02 -3.711958250800781521e-04 6.713581695448179662e-03 2.317703982167803461e-02 4.844793699536607823e-02 2.515839521039771320e-02 1.160840901604174996e-01 9.115546253242924613e-02 1.023015727984358109e-01 5.783709854064507383e-02
2.537563187209959104e-01 3.647366142966672331e-02 4.635826759581123703e-03 3.335997806122112256e-04 7.972276177378686754e-03 8.992068997611173337e-04 2.733477255127366497e-03 2.947077641366180854e-03 1.415886702486968482e-16 6.712127126956504361e-03 5.036774802026716680e-02 5.981900651997162054e-03 2.622986075531538036e-02 2.045058723493843253e-02 6.109736416146984567e-02 2.934673728335912210e-02 1.807679623077144276e-02 1.858989430431535333e-02 3.673875973936522460e-02 3.804430159410144424e-02 -2.802819368415784829e-01 -1.596876182074885298e-01 -1.979740624680503269e-01 3.943993453520614964e-03 8.732116006152278176e-02 1.978195733181128263e-02 1.554929859841166009e-02 1.713927941584858158e-02 3.575532220553506030e-02 2.909469372392731873e-02 7.751268769832270200e-02 6.921035951120398100e-02 6.549456310209088306e-02 2.614491064889985783e-02 4.487797804997392670e-02 2.108110832869553725e-02 2.749557138425076305e-02 5.343446023836981307e-02 -2.319173740329506794e-02 -1.544838117626378890e-02 -5.600254810430264213e-02 7.139669593673614312e-02 2.550621007544760441e-02 1.106905227875144143e-02 2.603601676005579818e-02 5.058894246278161644e-02 -5.828553902056551189e-02 -2.187343327277263896e-03 -4.247380346906762250e-03 6.001608438551719188e-02 4.727217335512342439e-02 1.005194242564952031e-02 3.441029601171814983e-03 2.173531336092725599e-02 3.643184325138381502e-02 6.957421147701930986e-02 7.316471779965789957e-02 6.934119448626437088e-02 1.168479258388047531e-01 3.730025544069449916e-02 -2.362729752148421125e-01 -1.311335024857360372e-03 1.353781103337351995e-03 7.140943104557898047e-02 3.821482279400588433e-02 -2.356709698834192832e-02 6.538187984009647194e-04 -3.669717635840269135e-18 1.046876112681054648e-02 -1.876719423141069384e-04 -4.666546577187803669e-04 -5.828830090385021785e-18 -1.580515583081128911e-02 -1.855195982955716236e-02 6.831142032837462033e-03 -1.438312168959718966e-18 2.937218809730084146e-02 1.015613888631870841e-03 3.239529829007620904e-02 -6.979671420999377512e-03 -1.059956841329944372e-06 -1.664772235011853507e-01 -1.309766231352503252e-01 3.620137884331738748e-02 1.696674427951284578e-02 1.342316790760871188e-02 -8.476391208497427443e-03 -1.451864960266713994e-02 4.792169043206066965e-03 -2.234005018087236935e-02 -3.066763887180546278e-04 -3.415105476331099787e-02 6.469006967394599905e-02 -2.397849979142533186e-02 2.866937932211094039e-02 -7.620906413322432911e-03 1.170386732310415775e-02 -1.427231803358428551e-02 -6.080073016117170520e-02 4.637368849869552556e-03 1.705845313320017431e-02 2.471131743310784668e-04 2.550764471894223853e-03 -1.096832948380443429e-01 3.281647121606796197e-02 6.415394031677388838e-04 -2.534328323695653803e-03 -2.883035316986319596e-02 -1.569956547722079019e-03 2.246798625561256812e-02 -1.308152883049482247e-02 -9.272954962510760249e-03 1.223080514137762616e-03 3.472197198987505944e-04 2.242829542601748258e-02 -1.075894598415759983e-02 -1.887599936816020487e-04 3.000204430656178439e-08 3.914966736270093783e-02 -2.161533690123694160e-02 5.737167264092259966e-01 2.735085970043948378e-01 2.458155010549300731e-02 4.018820517706061927e-05 1.113893473499650601e-02 1.172301250879981879e-03 2.561483345720547145e-02 1.606673767164509825e-02 1.017618132359727860e-02 4.789372022975201784e-04 4.136136929000886614e-02 1.431260134684810524e-02 2.850553244478070888e-02 1.939914297231040002e-02 3.385002061768230958e-02 8.721998626117702727e-03 2.272743164726553564e-02 1.312884496857382767e-02 2.049287892696872310e-02 9.501721404729374446e-03 4.128339735385281872e-01 1.008587026793649422e+00 4.708255011978629279e-01 9.675091141019307589e-03 6.744020442779391500e-02 1.324982421585362523e-02 4.077827247821980139e-02 1.059946634693544228e-02 2.152761710539797396e-02 2.654037980053113033e-02 3.873266087600589491e-02 4.484781714841564598e-02 6.187206007088535270e-02 1.905219525805360373e-02 2.252102540030465061e-02 8.236564931902749451e-03 2.332059886323863476e-02 4.466557655700304635e-02 2.927410618776996598e-01 5.812044829960473225e-01 3.757740419550831268e-01 -5.888603195056928141e-02 2.338678103885462670e-02 1.587859618484373236e-02 1.785839577051847454e-02 -5.649729428553824928e-02 4.923776182996786321e-01 8.789614515723633836e-01 4.287151881173024703e-01 7.635738051516775642e-03 1.966205632268848078e-02 -2.419161376124907663e-03 6.670815146514240922e-03 1.430730754052150430e-02 4.452936982100282742e-02 2.053170511848171828e-02 1.119296009999435487e-01 1.374910725980368065e-01 6.113789637647923381e-02 7.060711253706788981e-02
1.446819330749077037e-01 -8.089745423981373196e-02 5.721418627259316353e-02 5.697784979944672118e-02 6.183993816429247253e-02 5.643129844797624417e-02 1.339086614922230245e-01 1.562154742396160666e-02 5.124689718060236598e-02 4.506790441044996459e-02 7.514041126977491492e-02 1.241088197747690185e-02 2.428576486854121599e-02 5.202511453339911385e-02 5.472273703799342137e-02 9.896552207017184077e-02 2.532586066373707090e-01 1.330261765226205539e-01 2.633049669691016684e-02 9.405709791858911373e-02 2.388785594518382793e-01 6.842714204974864600e-02 6.374056124478028051e-02 4.760192506438745697e-02 5.716235665612334066e-02 4.418156448924204921e-02 6.471811952860727357e-02 2.995979618030574984e-02 7.462215708374129661e-03 1.664608146436883512e-02 4.650825008162935530e-03 6.035507607790703902e-04 4.779512013439308452e-04 4.093848463012438044e-03 2.617330462692884011e-02 2.147691364406041636e-02 2.663671596949111764e-02 2.686734371138271849e-02 -9.582576951786415942e-03 3.960382567298736944e-02 3.404662437947584414e-02 2.495110726907999801e-02 2.433725844530696195e-02 2.952187074631035246e-02 2.869208507750801995e-02 6.915010952474842654e-02 8.842441298257835980e-02 1.978582248436958416e-02 3.362246559197242662e-02 5.316120215525612508e-02 -1.683208343514890071e-02 -1.425301291509822009e-02 -1.278504582055884839e-02 9.745469892110111831e-02 1.539578363159314067e-01 1.942482044073773062e-01 1.152353172596507352e-01 1.763638332706829459e-01 3.765764198399212392e-01 5.293584977475394165e-02 -2.790890614668671901e-02 1.246210007649132742e-02 1.486777106582242401e-02 -4.354702307441429138e-02 3.432021167388031591e-02 4.121397226118212701e-05 -6.041959528751469216e-04 1.083755913425752532e-02 3.277195237674965927e-02 1.529368031489914403e-01 -3.320770296327839485e-17 2.133087332566426658e-02 1.278058677483433908e-02 -2.750176232546773861e-02 7.172858106099481990e-02 -1.364787089717820314e-03 -6.419944877099915959e-02 1.151834662125206677e-03 -1.550679567757159809e-02 -7.397057680500842593e-04 3.946071803920037785e-02 -9.584067816843673527e-05 4.431340346827180449e-02 -7.570598843392055410e-02 2.495084069741575553e-02 3.796677619823515815e-02 1.106904664836919246e-02 -2.162531044674123864e-03 -2.820536289900679641e-03 -4.853128957792323533e-02 1.135876345139522590e-02 2.010728071844679679e-02 9.166105450608473190e-03 6.624659348632918297e-03 -9.581388120459870955e-03 4.110570533234344723e-03 -1.304223844848130184e-02 -6.943145402393886834e-03 5.155668937496417359e-02 -2.917199291803575581e-02 3.798646785143021398e-02 2.383022728650472516e-02 6.534432610441763718e-03 -1.937266580784173173e-02 1.473494478566211852e-01 -1.797316148167416028e-04 1.109387839807438221e-02 1.419478609001964542e-02 -2.809591421060686489e-02 2.159182032809012393e-03 -3.870692310174193851e-03 -2.223858698787866545e-02 -2.721449610280562981e-03 1.687848030505185656e-02 2.418452436015800714e-07 -2.700702630808066509e-02 -1.206788980940997312e-02 -2.720552384283239297e-02 4.443253809863943055e-02 2.224490543773375260e-03 6.307984466097925624e-01 3.758449845248342247e-01 5.141888964895208886e-02 2.777273539029366176e-02 2.412856541148768880e-02 1.188485683954921962e-01 6.884214617514389234e-02 1.957628651648782650e-02 2.254961466720755564e-02 2.935223866968027229e-02 3.278168481285224922e-02 7.065865238887497624e-03 2.237846126941826774e-02 1.003038648308455551e-02 3.741766493113790615e-02 8.765823067600736485e-02 1.343831270445615400e-01 3.850024641708305745e-02 3.038487355776318954e-02 4.358742849494640081e-02 7.041316891827266444e-02 7.716047123237285144e-02 5.927405455509501125e-02 1.325282384548776872e-02 1.861863980508875541e-02 7.376923080267695498e-01 7.901671773727273473e-01 5.837387338634443257e-01 3.396299500101096144e-03 -9.472323110617276998e-02 1.679123094477374305e-03 -5.055477168007785349e-02 4.169341270078417983e-03 8.391412699708919497e-04 1.172304788174052120e-02 6.557830745880295631e-03 3.994128647826939993e-01 9.212635130227694313e-01 6.810993372041282967e-01 2.267027629905331992e-02 6.428177458050825244e-02 4.589725239225178610e-02 4.844308327409639625e-02 1.511042452323431429e-02 3.512070672893276291e-02 4.658119086403875175e-02 5.647852014029354495e-02 2.162751742844746736e-02 2.634413033515801972e-02 6.736172475624141974e-03 3.919121412352161715e-01 7.786628033759966927e-01 4.613251353811010924e-01 1.705462058501524061e-02 3.387790251780361389e-02 2.937583767828442355e-02 9.480309150392607354e-02 3.888902313614917006e-02 5.638367059699747608e-02 3.150129010947912145e-02
1.409229397572554776e-01 -8.484977807636000235e-02 5.416389584212411012e-02 2.089549607621101965e-02 3.575678504280477982e-02 3.288558425986917721e-02 6.671972881579026071e-02 2.626602616261516226e-02 3.266563283418769809e-02 3.212396475819063318e-02 1.225131822532853770e-01 1.842850633028181653e-02 1.562440058340130999e-02 2.796821832014007025e-02 7.430054343367974157e-02 9.116667694587424209e-02 2.945201910667977763e-01 7.739920861330271129e-02 3.351043646058352032e-02 -7.428128683644208419e-02 -8.349078717812477568e-02 -4.706546727106566963e-02 2.127557096775226433e-02 1.205406823296019912e-02 1.595845605357452610e-02 2.973771471212330667e-02 1.935804945544776223e-01 1.412329197319756383e-02 5.965444126850715997e-03 4.814506701203974387e-05 2.002802087730768538e-02 4.775202879823213509e-03 8.997038686830775801e-04 5.651880248038695671e-04 2.284261842986811970e-03 6.832578303140196714e-03 1.885208789233845685e-08 4.591242843884983313e-03 2.644094663052057315e-02 1.668488392883829899e-02 2.785086207327367239e-02 2.066898793245590740e-02 -2.692400761740306424e-03 -3.793524366274132059e-03 -6.634892634428637970e-03 1.960786041354452627e-02 1.474922534378256267e-01 3.960996878758103051e-02 3.508182915626816435e-03 -4.961120568624681265e-04 -1.337480149765101791e-04 7.071835727632810780e-15 1.507395088134826609e-02 3.385622657180183126e-02 9.409762304980445746e-02 1.403738969444725815e-01 1.978695563600553486e-01 8.313344870580935408e-02 1.079667029244371912e-01 1.612351193677108163e-01 -4.166638129264855356e-02 1.725048233729480768e-02 -1.644866603454529785e-02 -1.576150558102055142e-03 1.292607286530379415e-02 -3.067561715368026173e-02 -5.058334139457131766e-05 2.837922835348538619e-02 3.206954896445544934e-02 3.382692670999665197e-02 -9.425393421151201379e-03 2.934147122520038703e-02 4.753662962720259927e-02 -1.328295475324798962e-02 1.203681591801464279e-02 -9.730508283256909590e-05 -2.597864757080800904e-02 3.467051022103202068e-02 -1.271012697788756751e-01 3.269717423682622082e-02 1.015409202355674854e-02 -1.206568014938921331e-02 7.778546890465656838e-02 1.967800662565276013e-02 1.743786483908011525e-02 -1.604476775960265306e-03 3.628755579571986412e-02 3.753252796579093727e-17 8.758424746028684038e-02 3.447483508343694714e-02 4.230518837744244587e-03 2.117993135337242760e-02 -1.175025503373147966e-05 -2.104908007446048765e-02 -9.288915170118195027e-18 3.186775529902293597e-02 -2.906623705222470696e-04 2.580394774694631647e-04 -1.047731274185735237e-02 -1.633378959599551079e-03 1.441127577410032494e-02 -3.144431680286305852e-04 -5.854241773208514405e-04 4.053952869125318492e-02 5.943399315403962574e-02 1.184676100692651085e-01 7.284572111963334706e-02 1.083495744189826721e-01 2.575168511953452796e-02 -1.711953609091200826e-02 3.861337643174903044e-04 -5.617157834238175294e-03 1.191449282580453461e-01 -1.988621831781842500e-02 -3.172540415085955967e-02 -4.512396005653915299e-02 -9.235093526864179229e-04 1.472531996499061937e-04 -4.574746016694094218e-03 6.776419332036412446e-02 6.100170783301297472e-01 3.648133577388251725e-01 4.651315469211451947e-02 1.845485957495868884e-02 1.685269510277255869e-02 4.749178094726678317e-02 4.747202961692790352e-02 9.998626883958434269e-03 2.042589775303124688e-02 3.789312760969470312e-02 4.748935298919076953e-02 1.886398059153173920e-02 3.662396918296866033e-02 3.117374322945268977e-02 3.622101037508957538e-02 9.302868728750650351e-02 1.560210248329909344e-01 7.261233141578472827e-02 6.658646133294590652e-02 5.356133792374442315e-01 7.332939367851135648e-01 6.086856616135163378e-01 4.329552796009187438e-02 1.201544993166295261e-02 3.470951590360466660e-02 5.485204487402033785e-02 7.314000702258957598e-02 2.727852551225602024e-02 1.641290404631265409e-02 1.571924746851211885e-03 1.635070619937993922e-02 4.032748374493337444e-03 1.195786639043889925e-02 5.960162552790233912e-04 1.447781770404935921e-02 4.947838269423123110e-03 4.428521849722287691e-03 8.288785145789812417e-03 1.514355067038932259e-02 5.202734591559069616e-03 3.102966397914219340e-02 1.310899321848125543e-03 5.238672127428540826e-01 7.686481273418175331e-01 4.683759093742627111e-01 9.890268267197437069e-02 5.427800184772219733e-02 3.027619737898255576e-03 4.701356452551379295e-02 4.834153229627715653e-01 4.540845019643492408e-01 5.917735317793784144e-01 3.031818604261401692e-03 -3.006364598761424034e-02 3.792817144158393272e-02 1.820748969562961325e-02 7.343280958252861035e-02 3.884162594725928258e-02 5.108286886594250870e-02 4.060118655171066178e-17
1.327813849109350797e-01 -7.485815486869573654e-02 4.544808257581581246e-02 2.711747562921560523e-02 3.680811109688645472e-02 5.703463803673422539e-02 7.223874841216587628e-02 1.509609198958459728e-02 4.304115694869926206e-02 4.096974642979404196e-02 8.343890133239861839e-02 4.501776238323851320e-02 2.368473787924701632e-02 5.636234959702138975e-02 6.295692750967619988e-02 5.178909638164932300e-02 1.098605533528167477e-01 -1.026074715375152879e-01 -6.974474755443033254e-03 -1.166545220989471943e-01 3.762089173884580751e-02 4.470782397174356305e-04 1.872935995141720641e-02 7.251624920106336525e-03 6.991183410281492580e-03 3.358324830322995808e-02 1.000931712151271608e-01 1.781028343067560596e-02 1.033707809330646279e-02 1.151743476164113125e-03 1.047721946987725155e-03 1.017182542058553987e-04 4.325132362893002735e-03 1.623575857731792072e-05 3.619361963472665991e-03 4.816997684925061925e-03 2.849549996535527215e-04 -6.635151711438934830e-04 3.797977882258161183e-03 9.396663245011545318e-03 1.642574340513856465e-02 -3.346597731641656314e-03 1.551583903544492306e-02 6.276746119951932650e-03 3.354673402076179672e-02 7.769956166054817204e-02 6.812382022148984195e-02 2.394685141609382770e-02 3.635726593234733101e-02 2.205922736746489254e-02 4.404628180589879177e-02 1.981382764817393781e-04 5.806146761110756761e-04 5.061595023526503125e-03 2.178360242269397271e-02 1.144181581608118845e-03 -4.505694879845402179e-01 -1.618920401063207726e-01 -1.888716196628518196e-01 6.026839977986276492e-02 -5.017896321024922779e-03 3.411854219809095593e-03 1.013906131719651965e-01 -5.121237601701829345e-02 -1.870662874273699159e-03 -1.030243264484680399e-01 2.990024941240563075e-02 8.030357202144374276e-03 2.502244962026112934e-02 1.437593568065981438e-02 1.189919448926471737e-03 1.670913328062703651e-02 2.322906822400988905e-02 1.113083085379205830e-03 1.459159400406472538e-02 8.716298461076374404e-02 -5.232954102363140958e-02 1.424446424588603211e-01 1.761776901128037920e-01 1.749785797047855584e-01 3.307486155373775411e-02 1.924488348360810577e-02 4.320968141384264050e-02 4.071533721315291267e-17 -1.470623191250848821e-03 -3.096912818273973161e-04 1.132972465710482546e-01 5.554903398505657564e-02 4.092141663662123008e-02 -4.533630241654329944e-17 2.795916733955833029e-03 1.143596777862547140e-03 2.961347501375564788e-03 7.086506308646368935e-02 -3.297695916268055227e-05 -9.117449143954146906e-03 -2.960616397960935328e-02 -2.326009183730600588e-02 3.954653203485605090e-02 -1.556026993419818913e-03 6.720111897110442106e-04 1.234028069347403722e-02 2.406141038994610154e-02 2.303388724484126474e-02 7.569956601269393903e-02 -1.080214748783365347e-02 1.415656777364471491e-17 -5.392520197003899273e-18 -2.483138388416863962e-02 1.226810139359748968e-02 -4.780707109691487088e-18 1.547975468797330514e-02 2.070141096779228897e-03 6.102337547072996492e-03 1.128869909825050161e-03 -8.299282121821429520e-03 1.993630841017714900e-02 4.752330787569448989e-02 -1.891211079518404781e-01 1.808113575507568349e-01 6.566961704926310706e-01 3.923955474033043456e-01 3.912714111790673716e-02 4.195449699609381938e-02 1.953125773760220132e-02 1.413526581402361482e-01 5.590506712454253180e-02 1.695001315378310419e-02 1.079204033643997064e-02 2.067275824823541427e-02 4.001648141966108324e-02 1.325026872952361073e-02 2.138350132493228167e-02 2.983692203053464609e-02 3.546340771237393996e-02 5.974991818888223533e-02 3.643684689184766012e-02 4.188052860909348141e-01 6.400055496811337674e-01 1.090448637732913006e+00 1.870455903318889168e-02 1.255878906212301409e-02 4.089303640273522777e-02 2.390851774239198535e-02 3.409708420648162319e-02 4.583632602994219818e-02 5.882649779190996658e-02 9.808533127389286108e-03 8.665966514129630135e-03 1.713303269744707695e-03 1.683192409103714468e-02 5.021125342316894863e-03 1.348177272265937456e-02 1.116822986057308502e-02 1.566804605063417191e-02 5.298369053186238221e-03 9.518274907221783987e-03 1.218102661474762365e-02 2.481371688539800446e-02 8.734198141526903353e-03 1.690217152477393320e-02 6.528266322905078578e-01 7.519655500409240245e-01 5.958653304992508337e-01 1.094252489718149958e-02 1.944096211561160623e-02 4.705668603349742740e-02 1.472813028963168396e-02 2.266317947687698092e-02 1.471753851563111296e-02 3.549109334593652382e-02 -1.323786784530357535e-03 8.509778756604316485e-03 1.454845089333637112e-02 4.423496372133713800e-02 1.573710870734612317e-03 1.737072020602609401e-01 5.317602940144401380e-01 3.713839169467235690e-01 7.750530866942753061e-01
1.203469942675554905e-01 -7.045857127581847679e-02 6.123874610346363712e-02 6.259540852427032676e-02 3.546273266838623900e-02 4.416185343803169516e-02 1.919631934976798460e-01 2.747956706592989684e-02 3.205318866411930451e-02 4.903065074093722331e-02 9.385741011505774334e-02 4.454350130551307935e-02 1.871938567381320820e-02 2.677825181018693435e-02 7.482928564959480389e-02 8.650522231635948689e-02 2.490319735047624439e-01 9.033631602471439703e-02 5.269601948184243073e-02 7.314171594085847050e-02 1.382299301421717197e-01 2.543618146722619391e-02 6.072616127783483919e-02 1.929321450585927133e-01 3.939503824653408509e-02 1.223606764816428417e-03 1.350317047310676567e-02 3.837401064603603983e-03 6.368951658535440888e-03 2.746368877961819117e-17 2.995560202335524180e-03 4.767958859933109079e-04 2.822735224065477371e-03 1.519875343513861412e-03 1.606329191430062170e-02 5.537489424459414271e-03 5.587620706339657185e-05 8.957050317757168922e-03 1.966067314608442237e-02 8.959009010692183667e-03 -5.483233570160313611e-03 -1.392015193996567340e-02 -3.803528016334593170e-02 3.707849032043136414e-03 1.452861386331459094e-02 4.390954808422816574e-02 8.378154009587951423e-02 3.983138914222641297e-02 2.549779141656816073e-02 8.493131529981738301e-03 1.758674750978069662e-02 8.001457544154902292e-16 4.451508095166866429e-03 -3.052493023028953270e-02 -8.373618321090188499e-02 -6.291670498913595755e-02 4.567782373420493230e-02 -6.404096489083536438e-03 8.671894073419551124e-03 -1.765017557837249632e-02 -2.385394678797016221e-02 -9.591093980460919546e-18 2.380983165034698812e-02 -2.066476989269925429e-02 3.295794461874193559e-02 -8.425767091343882784e-03 -1.443281744351782259e-12 -7.839327858828244733e-03 2.705412115652985807e-03 1.620561984891270152e-02 -3.615327905098875833e-02 1.434929166869776049e-02 3.891545889305015465e-02 -2.511383093342290859e-02 1.826159541868407898e-01 -3.378733903549969303e-03 -1.487618192819137493e-02 3.497648078709712782e-02 -3.032849109663796061e-02 5.055370544652075469e-18 -7.571918942837093085e-04 1.201727000498643283e-01 2.076370129578559293e-02 3.466131315851233435e-02 -6.162681218756385984e-03 5.647030117426927315e-02 3.693835310278304612e-02 2.407174791288040766e-02 -6.850942766217782868e-17 1.023707518023135588e-02 7.545127558390643183e-03 2.553162044527174102e-02 8.458248014479642987e-03 6.416760801953445142e-03 -5.236787559385535874e-19 -6.558336685785047353e-03 3.772127330349921148e-02 -1.404739311813448487e-02 9.916753469683103547e-03 -2.459800842788602629e-03 -8.742810673520839959e-03 -3.425260413456411579e-02 -2.434730236915058979e-02 2.961671786450951049e-18 1.075377020143510921e-02 -1.496866779903098559e-12 2.838849778122579798e-02 5.029291020608309126e-02 -2.819476627798222709e-03 4.054975471842347384e-02 1.490419019742424307e-02 -1.701550579664696805e-01 1.754277618601641706e-02 -1.976913875615358713e-03 -9.556472459241177955e-03 -1.047073398423049514e-02 1.412478210295344157e-02 -1.176829603131284178e-03 4.279061249610069395e-02 6.831729018628446282e-02 6.368549811934796079e-01 4.275633140627383422e-01 5.292825932679476320e-02 6.473873448532704855e-02 2.042976076213709122e-02 1.466952425959443806e-01 7.436378130572492140e-02 2.927447598354973243e-02 2.565482134606618739e-02 2.840019881072031877e-02 4.164013062123288200e-02 2.246441574147577114e-02 2.502320119290583553e-02 2.975764142658725991e-02 4.837147225354221369e-02 9.632778230516485241e-02 1.868563123520062541e-01 7.848251299282334170e-02 3.876917328731614015e-02 5.949448008352846429e-02 5.922380867663877557e-02 2.332431189190363810e-02 5.583889676180023853e-01 9.269235127867896384e-01 6.569533890583918456e-01 8.020628370930194461e-04 5.746667767060946452e-02 7.082795382438860331e-03 1.881277462287249402e-02 3.321580200803234042e-03 1.501858274006364485e-02 3.033562401848238343e-03 1.005499042365311756e-02 9.926324664329307101e-04 9.574227649846991581e-03 8.497792822298398721e-17 2.271776742661960231e-03 -4.153082795254611076e-03 2.406862634760024988e-02 1.269962782623105342e-03 5.514231243994415221e-01 1.163234929955374142e+00 6.871275019036071630e-01 2.742551703507486397e-04 1.476340914622853440e-02 2.153651612378625416e-02 3.819386975473169216e-02 7.639984320021193410e-03 2.257864510082873560e-02 1.837676495816186958e-02 3.430492234355947412e-02 6.633945726706351811e-03 5.962273243633706664e-03 2.644719587285612739e-01 3.019069183269834089e-01 6.990882248912001495e-01 2.203033907350535059e-01 1.287545817992183650e-01 1.837466094511947812e-01 1.322293320099873493e-01
1.255650862186617478e-01 -9.005159380327364127e-02 5.488317167443043654e-02 1.870270253079435799e-02 5.386642992470146163e-02 2.520657634778812259e-02 1.132385021127510633e-01 7.483857366248296283e-02 3.794816244692011953e-02 3.754330479264669090e-02 4.942687680459187383e-02 1.186941928350450717e-02 1.427077886251475018e-02 2.351607753398807579e-02 5.709308990887658475e-02 1.106157925909813655e-01 2.259296140196077607e-01 7.140773452132011256e-02 1.644137872074826617e-02 -1.049361389155332447e-01 -9.513132328225161471e-02 -3.160626499608674284e-02 1.937460276882584098e-02 2.126079747451054946e-03 3.319703461502628884e-03 1.240872731826052865e-02 8.705813094665361407e-02 1.022977451866677670e-02 7.064952100829993004e-04 2.063654558016293978e-04 1.557279080932976942e-02 1.949573196451469693e-03 7.564482108428872628e-03 1.182862954291078750e-17 1.278795462089082376e-16 4.169353614449364770e-17 1.084013348585770482e-02 6.706640776909476199e-06 2.191495970804350352e-02 1.288338052030052298e-02 1.328804209920030066e-02 1.511789998806015861e-02 1.036034963931456096e-02 1.701900645395844087e-03 1.955434954154354477e-02 8.544209927994787790e-02 1.523833085422821206e-02 9.436168216887844129e-02 2.262173720856433881e-02 3.984547898528683491e-02 1.020660989713716577e-01 5.875455327087063077e-03 5.471491926821800932e-03 1.901235951691933670e-02 3.643505198994027861e-02 5.924384282970554527e-02 1.017257576559121440e-01 8.539838042087849468e-02 1.993238046772504435e-01 4.051957436262206697e-02 -3.565466465403510643e-02 1.569334197140176065e-02 2.963110447727051181e-05 -4.073854805339100632e-02 1.852443529791752627e-02 -2.827178068189792337e-02 -9.779027314694432837e-07 4.904064214299785895e-02 8.242438970847604240e-02 8.179402195116251828e-02 5.230788227538086133e-04 2.703136983506211968e-02 3.905645329361716322e-02 -1.064984066321006390e-18 9.906915195110446715e-02 2.278961216745316321e-02 -4.170304809990340450e-02 1.627340848090572945e-01 -5.690369152471694281e-02 1.206606043390551913e-02 1.886069367853681136e-02 -5.194356105001374929e-02 4.472354142736611332e-02 -2.394012097646131335e-02 4.987629917557509507e-02 -8.064081406446290540e-03 3.527590063011883798e-02 -8.391009561066181677e-02 1.173286136626345566e-01 -1.086782621686513059e-02 -1.805761874142883640e-03 3.165448748171441860e-03 -1.858027863453693210e-02 -6.099983930261552480e-03 1.142255734288716296e-02 3.717029476807393040e-02 -7.156744793410044113e-18 4.157454397210804536e-02 -4.074630894344191678e-02 -1.167540347356127786e-02 3.054870591772256699e-02 -5.503530443764245627e-17 -5.005753170217912967e-02 2.463258092949930159e-02 1.324135257566019058e-17 2.106635508179923594e-02 3.964222780747372193e-02 6.619954627544385323e-03 -1.715557386056003287e-03 4.576110198578257557e-03 -1.059478930224511985e-02 1.896335652747878353e-02 1.284685063106899908e-02 2.210371904434434154e-02 9.198257122476290876e-03 -1.420309807918630762e-02 4.072265170880960683e-02 -8.355428083407794027e-02 7.952833942563926339e-02 3.155115118685740405e-02 6.307766586342372639e-01 3.879450770890767730e-01 4.514578218192592807e-02 1.882358516463547571e-02 1.737195203504498381e-02 6.326398433201879667e-02 4.160928798420705788e-02 6.872511380203318032e-03 3.106801629209601789e-02 4.628243439575841639e-02 5.340267325199703208e-02 2.697174147542089409e-02 3.420122234635175618e-02 2.845046460806651550e-02 4.005918338821573554e-02 7.793739632692062158e-02 1.468408401712343325e-01 4.685792704401534753e-02 3.597499118304139931e-02 5.404636625437468922e-01 7.592753832433948302e-01 6.884166728243912292e-01 4.175910832978219078e-02 1.945463670835259271e-02 3.683410219655468970e-02 7.503468014999567504e-02 8.697666731776887428e-02 3.107925979543835551e-02 1.478849026175243879e-02 9.918343873876950875e-04 1.641858629040346407e-02 1.358472332123642498e-03 7.105237547617397967e-03 7.179193025421706231e-01 5.631220444513501677e-01 7.044505496127498878e-01 1.616396318796212844e-02 2.387020306535793824e-03 3.174869083445162193e-02 5.632789803204887175e-02 5.869444354286165733e-02 8.063876983734284687e-02 6.509605442570831624e-02 1.481776781275073035e-02 3.656337243016348493e-02 6.717133542365082421e-01 6.615900826593601458e-01 6.310447339445289794e-01 4.286932371884844689e-02 9.747570330475402792e-03 2.833077112365245745e-02 -5.813633749626014383e-03 9.190253144998914575e-03 1.775914726423676038e-02 4.487317638420039445e-02 2.458367586609806407e-02 1.211749949479296873e-01 7.013251757289871990e-02 6.658679289383340194e-02 3.739030987669313827e-02
1.298175382681986401e-01 -6.231411003933033654e-02 8.135236040626817045e-02 4.383286370910295660e-02 5.570745424821550718e-02 5.729906861917033467e-02 1.104248232451022355e-01 2.095434424608439219e-02 3.596028964535142453e-02 7.101626659770575445e-02 1.171484471767849089e-01 3.351028039332711733e-02 3.028280858677685761e-02 3.102674481837695147e-02 5.038379604108901566e-02 7.613109930410567883e-02 2.277374929855304764e-01 1.000426973081945253e-01 4.338742270905375509e-02 4.820736194321845192e-02 1.458650893999194897e-01 6.345959150721976194e-02 3.505076812097067396e-02 5.205567802071871947e-02 2.855279211803965836e-02 6.140148141575506313e-02 9.057335979371565482e-02 -1.480314451030516620e-01 -8.090962137060857262e-02 -4.623933697653004327e-02 3.491621862150083577e-02 5.733344194661219723e-02 1.821661768425061889e-03 9.733080230674186884e-18 6.205908606992894330e-16 1.933862400042488818e-17 2.225100318882070050e-02 8.904039534007328685e-03 2.123348808303763061e-02 2.720893430396742474e-02 4.436787570512571582e-02 3.182160794444389984e-02 2.879875285464146215e-02 1.272944019526876455e-02 2.885030107084221931e-02 2.698051420867808928e-02 4.928783494201447596e-02 2.424227076754341750e-02 9.725590448970779928e-03 1.752576888474657521e-02 3.664621671345157095e-02 1.269334310323352866e-02 3.804084544052516691e-03 4.021487881359048916e-03 3.252133476538517781e-02 4.009693839161321272e-02 5.495682134562351190e-02 2.099213018328257435e-02 9.914056972175841476e-02 1.066109197736073294e-01 -4.468346011364452558e-02 9.040303582991975409e-03 1.845821148486299912e-02 -8.227183961789347189e-02 1.870181296885953595e-02 -1.182838763902319731e-03 2.312139964935664566e-02 1.259846617062687635e-02 4.185108488146765254e-02 -1.588888705645802963e-03 -2.271942018443175865e-02 2.472487942749884704e-02 4.003526129337147549e-02 -2.048634036649440246e-16 4.490424748081078832e-02 2.584694548140285472e-02 -4.455548886786121071e-02 1.932064916797443868e-02 -4.557681643898060103e-02 3.235768771960838438e-04 -2.572664431652643322e-04 -7.287372221286711688e-04 1.606419894003682444e-02 2.189594086618648627e-02 6.852160478535513667e-03 5.869561909815362724e-04 6.150328880629893054e-04 1.116276623780370347e-01 8.795983976869913359e-03 -5.507913948793264486e-02 -2.248181453032623272e-03 -1.665914147918167235e-02 6.764471995508915259e-02 3.747067648632040199e-02 6.214173877615086425e-03 1.068364519758947632e-02 3.050406616291333195e-02 3.125311354055552212e-03 1.827570920323508280e-05 -2.454935329453644156e-03 1.561577163036470799e-03 6.087414407887290759e-04 7.117344899545552904e-03 -5.674014783227059374e-16 1.057363984194962907e-02 -2.870804159975954162e-04 -1.301704278323900918e-03 -1.045982926151623900e-05 -3.254587525016266331e-02 9.167922782013154803e-02 -3.065593034595080339e-17 -4.469437586258422668e-06 1.023144906033411627e-03 3.421971496967275705e-02 2.814256038424182982e-02 -4.049040967472906233e-02 -6.747023611743463499e-05 -3.236420665278104142e-03 8.586072812002726318e-02 -3.138006808051762436e-02 6.364858045204681014e-01 3.838930319787527967e-01 5.212732426101827909e-02 2.938244953434723111e-02 2.234588898168831869e-02 8.502247298297868627e-02 6.832238315406505280e-02 1.439876982163893794e-02 1.891458928454741589e-02 3.266522674547121807e-02 3.477442727356912233e-02 1.444243327074368199e-02 2.258490456929342777e-02 2.066723697887384859e-02 4.623942410593846059e-02 6.380163414080405881e-02 1.212461736423382647e-01 3.890062210762634115e-02 1.936560386252794697e-02 7.704018568913503784e-02 7.556121585877183244e-02 3.866262239639111503e-02 6.080453193964804187e-02 3.148413677760355384e-02 3.356337299585022971e-02 3.578402489374147427e-02 3.016548214944784687e-02 6.355128193298316130e-01 4.512951336394346313e-01 5.929859274133451041e-01 3.818744630250307526e-01 4.027522590495334343e-01 3.182636079027669762e-01 7.172653853464125984e-01 4.261314199329736541e-01 6.686031742328235605e-01 1.304515034685456085e-02 8.786877804141952908e-03 3.262965973128269248e-02 2.327717171328951354e-02 4.495334826835030440e-02 2.682315768339426504e-02 3.722464140651543518e-02 2.750898184654255313e-03 3.951924448441904159e-02 5.229894244971456524e-02 6.684973163995591716e-02 1.966456983671638348e-02 3.233865325488228387e-02 3.754733725530525618e-02 3.518682120705108468e-02 4.581250091896807815e-17 7.911592547410603235e-03 5.176175305324469574e-03 4.468430387376549906e-02 4.398308464278499752e-02 9.293781143281309998e-02 4.249271341278397229e-01 5.811750080502938731e-01 8.557912400686356058e-01
2.364276499318206104e-01 1.769022143265972305e-02 1.835942804902047270e-03 1.324177193356055639e-02 2.175792155518380903e-03 6.109425180242032379e-04 6.334486254642744674e-03 2.196490778374693420e-03 1.057953576848313622e-03 1.222990615218318968e-03 3.419288193097209938e-02 2.530553808398911669e-03 2.511200988208021206e-02 2.048517417942848085e-02 2.732343026891297100e-02 1.139267315861913245e-02 1.357403454213405337e-02 1.028951416222932680e-02 8.386290474898487959e-03 2.633012968139952087e-02 1.329721380589302948e-01 -1.027926253749002899e-01 -3.664952133951121863e-03 -1.068339731151290856e-01 6.029251086627623185e-02 8.215473132052248523e-03 3.460890505914968746e-02 1.204467124581811745e-02 2.684663216025362509e-02 3.716402828055535151e-02 6.238629328295615395e-02 5.320812181786483547e-02 8.832891080959644203e-02 2.881319636410912058e-02 7.271714478200218268e-02 3.374882236411123199e-02 1.204525802494015030e-02 3.611545802692973234e-02 7.539388704444585643e-02 9.305864888947157701e-03 8.988817097161259717e-06 2.023463966981512221e-02 -1.657552717086443680e-02 -8.470057466628273801e-03 -1.029013136474069001e-02 2.132693518250649775e-02 9.693426072895711121e-02 3.847343865197308532e-02 3.499479293593548129e-03 6.833183483592076158e-04 -2.099603998754558565e-04 -3.994709797331549156e-02 1.333574498670783183e-02 4.448920830797607751e-02 7.399377190590067377e-02 1.685688155483524298e-01 1.791555725931491816e-01 8.390698108061515892e-02 2.407457955959903806e-01 3.215519611379734632e-01 -2.088655926264687268e-01 -8.821433155894712773e-04 -7.699557787380238716e-02 5.208532147652952457e-03 -4.786180688221617491e-02 1.698739144227581765e-02 9.851595447284382245e-02 2.410614294419963916e-02 -2.816857352984199203e-02 -7.999263404162552005e-05 1.006570988224492376e-02 2.890580569419535714e-02 -1.904945876052871843e-17 2.519034321823341097e-02 1.410224165045029579e-02 6.957882129995607290e-02 1.074495773718552906e-02 1.290406289714784993e-01 -4.651872331464849638e-04 -1.360296819526948808e-02 -1.849637381316848639e-14 6.229000063080608607e-01 1.560780271312995349e-01 1.883546052396711801e-01 7.895245843054249368e-02 1.414995660913545214e-02 3.931013241515511625e-02 -1.014929710857040129e-17 9.601697784279002168e-03 9.034424883294193964e-03 7.074059190652595870e-03 2.207324801122300534e-02 -3.829038157408182132e-05 -8.002562016146720636e-03 -6.834120408283524128e-05 -5.817689538730937832e-02 -1.004702592318609117e-02 -2.161210029245762276e-03 3.210495155893682701e-02 -1.956081038971667699e-02 -1.069442729448575608e-16 -4.248972014545479570e-02 3.272805492726690163e-02 -1.251838631897702475e-03 4.233021691258891800e-02 1.041363478249391489e-01 6.825296048529792825e-02 1.084058169924202036e-01 5.245158808092519054e-02 -1.096953245186459093e-05 5.458284485981318435e-03 -5.865560539405688216e-03 6.197492555607295867e-02 -1.284692618656336355e-02 -3.680775894052663866e-02 -6.621565803049228593e-02 -3.622428270206369018e-03 1.088373007627731898e-03 -5.918283823988456998e-18 3.435042636513351011e-02 5.597748778754492172e-01 2.851944406532345155e-01 4.271713846604876380e-02 5.319662552641195134e-03 1.324806414255301316e-02 1.292449031840136740e-04 1.788506884100388508e-02 1.566919438084908795e-03 1.754391416162794440e-02 2.669721977849894273e-03 2.575459400845999530e-02 5.037940906889403204e-17 3.275637466506839401e-02 2.979983656760279431e-02 3.853484376386722560e-02 2.095936248413960215e-02 2.606520527325498909e-02 1.787366350892508060e-02 4.016101845289395816e-02 5.344567152158210405e-02 5.859692026462328535e-02 5.826818448075344881e-01 6.800205230597576866e-01 1.281046362215224388e+00 1.670131158419330131e-02 2.392165623467978550e-02 4.493294925824364155e-02 6.266075825738662286e-02 2.782610012153598905e-02 3.614198271961043679e-02 6.958698652072146940e-02 6.673870590890160215e-02 7.036743154923545684e-02 2.921816513131848467e-02 3.095336054157599159e-02 2.114036467484346413e-02 3.648127286455752488e-02 4.311295851066652279e-02 2.358481083676218851e-02 -3.561519174461945619e-04 5.583913398738228444e-03 2.178079607032529143e-03 5.244853781808376247e-01 8.172453641099104438e-01 4.905511713115125461e-01 5.047620131478855465e-02 5.219556929590429828e-02 2.593998388688359413e-03 6.011421616797943401e-02 5.328083849740548050e-01 4.918223999296499893e-01 5.000447943796092476e-01 3.505598716443275760e-03 -3.842379232439515485e-02 4.120304230827186859e-02 3.232077360565490526e-02 7.454250050086198842e-02 2.809522200012998933e-02 5.326108108168818578e-02 2.231681187461500540e-04
1.488652038955634582e-01 -7.022349472195899001e-02 6.894621612939623445e-02 5.030778048741299524e-02 5.172960022678853298e-02 4.810757338048860660e-02 6.651380412196936098e-02 2.005468440806553593e-02 3.382643131091547101e-02 3.371040888676464509e-02 2.205858808217399381e-01 1.383996666613573207e-02 1.164271579598992294e-02 4.079180251095781279e-02 5.173372606781612842e-02 1.163239186185930640e-01 2.414454326640864679e-01 7.365986860393626545e-02 3.263938655074690581e-02 4.260175070635127681e-02 1.843298105828473921e-01 7.300059127184613250e-02 7.633613707941604154e-02 6.845893211926146193e-02 6.832948458656050439e-02 5.996308821816056372e-02 -2.515212996134730261e-02 -4.371480834445092001e-02 -1.029682942134306500e-01 1.562015548503428546e-02 3.613263312631903590e-02 1.988432319161644035e-02 1.497956587882671130e-02 2.342689242109973966e-02 -1.802301360445201228e-01 -3.090590534020045688e-01 -8.741156704927510734e-02 3.626750452229647592e-02 2.284119626223247990e-02 9.224395263011169951e-03 4.190863769798576499e-02 1.295508925280530049e-02 2.750275438176554357e-02 1.591462680360070450e-02 3.368147816550079443e-02 2.823995826362300440e-02 -5.046358161241951829e-02 -4.098817581176219388e-03 -1.463807832662244839e-02 4.909154451992249479e-02 8.785643263357471866e-02 7.527841258934662300e-03 4.844682240652260601e-03 3.051692784383406437e-02 4.837830084604557990e-02 6.312330296857497636e-02 7.340070130021472683e-02 1.265090263837721773e-01 1.003359032573414439e-01 2.378773373462066132e-02 -4.110220748165259452e-02 5.887618010425490177e-03 9.694482194361288274e-03 -2.468394756809506180e-02 1.709057466946030737e-02 -2.758922240438852966e-02 5.268408070121609332e-02 9.319283307851835424e-03 6.743964587170132863e-02 -6.151194406394310532e-15 -2.091064122602247569e-02 -4.726483826986662383e-03 3.834713641247256000e-02 -4.692222807573593013e-18 2.576773343914873660e-03 7.553936143936668818e-03 -2.535035564135586228e-02 4.312139237817904508e-02 -1.341037680429974996e-02 -3.932858997909592647e-02 -3.691499214700886839e-03 2.314648744983355821e-01 -2.806763349692705228e-03 -1.704899026068960183e-02 9.568538848467760590e-04 -1.666668270438131146e-03 3.061950197650438465e-03 5.961915157333182863e-03 6.762890978044237313e-03 2.706658004274307625e-02 2.240977856550626507e-02 -1.004574852893564850e-03 1.166068205008688984e-02 3.306438148072710081e-02 9.635548880355332760e-03 1.308339557319572639e-02 3.445492699748945592e-02 1.452912832859746921e-02 1.028082935062587036e-02 2.683125427872392124e-02 2.814091327106515647e-02 3.758013252758501260e-02 -2.281292148795213345e-02 2.799959873202695695e-02 1.854760843002621271e-02 -9.421272458068086567e-03 3.564146500655970135e-02 -1.804855824266775322e-02 6.611501882008537223e-02 2.460538610962034051e-02 -2.028489975153299416e-02 -1.224753381210318647e-02 5.068682380097132290e-03 -2.254662338341622760e-02 3.505279113933083140e-02 -1.375994514982678552e-02 3.135930515446185007e-02 -1.503563747273662862e-02 4.318924985932080102e-02 1.884278481666869882e-03 6.415187100146353405e-01 3.789552504692293988e-01 4.249044252528757365e-02 3.204243026543381750e-02 2.090724734538544280e-02 6.334925279378772478e-02 8.017005235721663681e-02 2.021908097924856457e-02 2.413939563258635929e-02 3.462011520326574443e-02 4.522930778483457565e-02 2.113348453252824818e-02 4.512319649959048440e-02 2.661894049389603237e-02 3.022600054913729406e-02 7.114211595611735772e-02 1.261652407357413408e-01 4.749374964718335035e-02 3.029551268856326210e-02 4.751013707601659669e-02 2.625026064061089168e-02 1.281896957171636819e-02 4.622976443905054550e-02 2.467972919667458034e-02 2.321606418503981487e-02 5.562546020162112241e-03 7.317563182305637515e-01 1.155769021566573240e+00 4.408095155805641374e-01 -8.651156899305090242e-02 2.236584390838397551e-02 9.146517338742717745e-03 1.046441519110945587e-02 -5.244246631682301929e-02 3.048539664203178123e-01 3.607222010298692383e-01 5.029872874071331657e-01 5.514100214755402601e-02 2.278788190818737541e-02 2.660344047005920934e-02 4.220670518892841866e-02 3.071604154851214838e-02 4.567150958886394274e-02 1.850194131497748945e-02 2.862577206068377123e-02 4.317527222722346986e-02 5.604157655464102605e-01 8.554158761509323972e-01 4.298305999723244231e-01 4.986395420437971134e-03 2.357854906969000103e-02 -3.489380275110410407e-03 9.178930267472223264e-03 1.188316112203444130e-02 5.020694817594347631e-02 3.706843908732652959e-02 1.135110568869229786e-01 4.965752104638974235e-02 9.634195898542939618e-02 8.948983512594017864e-02
2.168718790886492997e-01 2.177874672817818602e-02 9.592095262656599947e-04 4.943110570955844794e-03 4.017737893932403319e-03 4.231275499234193860e-04 4.025307754967575105e-03 4.757405802430773011e-03 4.380467961203752744e-03 6.930894792659045116e-03 4.056352161900165371e-02 2.311188940851734153e-02 3.384819718100064156e-02 4.176604966716156858e-02 8.551556386872982696e-02 2.163749228128903322e-03 8.231685459915486641e-03 2.027120778070604784e-02 3.407613891417599766e-02 3.607545876525557887e-02 -3.202870407430862820e-01 -1.708487020204871032e-01 -2.210873815890224137e-01 6.473551279826450511e-02 1.019776518287696943e-01 6.220183756749235460e-02 5.043762069438929851e-02 4.205448574807480605e-02 6.580173050487057862e-02 4.559476963751963458e-02 1.170364185435222443e-01 9.090655570195665947e-02 1.056153927360555800e-01 6.011231859873650968e-02 5.948167047050492012e-02 6.757395619616378413e-02 -1.964635203990423799e-05 6.662450906930794670e-02 5.592124174472055259e-02 2.508904691180297627e-03 1.119582557543184969e-17 1.807079023685063263e-02 2.363604461612158272e-02 8.708417460736618543e-03 2.471038585229975992e-02 8.503769206018362004e-02 9.319959081684908719e-02 7.005871458158106913e-02 3.536642654634796629e-02 3.532419975529044742e-01 -1.063069737709882757e-02 3.196932097363064385e-02 2.863756674184560203e-02 3.503539368403794235e-02 6.268277492932870787e-02 1.048857272785655159e-01 8.482187972267705989e-02 1.422700528672962428e-01 1.121287594373800434e-01 6.954393513017569084e-02 -2.274042452399413217e-01 -8.990211541473351268e-18 1.279687835167507733e-02 8.225148185819433633e-02 3.569372623553809104e-02 -7.635756400000100370e-03 1.397240356818583128e-02 2.727855998385453687e-03 1.251815999717828705e-02 -2.267757607630225614e-04 -9.513163847918911778e-04 1.377857417526117019e-02 -1.882299454896091753e-05 -1.559993421873026760e-02 1.879123741539844711e-02 2.418641262941623529e-02 2.041648533387375314e-02 -1.538269549618849421e-17 9.790948174130309489e-02 -1.985207703912830485e-03 -7.326863927515622093e-17 -1.386230206608984050e-01 -9.975237693729889499e-02 6.720022074706653858e-02 1.668369904928511394e-02 5.814477136166710386e-02 5.822890115972702849e-04 -6.925472136157169098e-03 7.363289975021365892e-16 5.894104595679060601e-03 1.473129294330177870e-02 9.716454446440153311e-03 -6.239089758986159258e-03 -2.429205730038310612e-02 2.704463845877088102e-01 4.364460436815847943e-02 4.343512499970891144e-02 4.154442568562766863e-02 1.612366046838336828e-02 -2.648776624784569475e-02 3.304563156043159589e-02 -3.436289038452503691e-02 2.153893951690834686e-03 -7.261162487568972868e-02 6.068292862142781081e-03 -1.679753624683858042e-18 -5.811404285051112971e-03 2.485637348201576169e-02 3.442038716424730138e-01 4.811928543338740011e-02 9.380828426847784285e-02 5.333771370420635309e-03 6.614330192311818069e-02 -1.199315154233261682e-02 2.325979212764126350e-03 -3.012663317217521494e-02 -6.653971521292084863e-03 3.226910708678736668e-05 -3.322099651001388446e-03 -3.777784351616760711e-02 4.348142948554911125e-01 3.121662185263536782e-01 2.568789810777647245e-02 1.394228043064382533e-03 1.073490240197966412e-02 6.391199042412559761e-03 2.079038857559278863e-02 5.593746445489624491e-03 1.288712874467433551e-02 3.732798953086419527e-03 4.769873796425308510e-02 1.607554136141951112e-02 2.606695263983869332e-02 2.313943531291987737e-02 3.431417116930476907e-02 4.998572789776347357e-03 2.426564794550624057e-02 2.762970590513472158e-02 3.951288358436678971e-02 8.204113206789138621e-03 4.140811522239036813e-01 1.254945875098372632e+00 4.718320540808783603e-01 1.217391841205736163e-17 2.321983826675222060e-02 7.916216283802178164e-03 6.453843910390709426e-02 8.922073217164053216e-02 4.990393288586050802e-02 7.473595946387573408e-02 8.046210029061089120e-02 7.537549142187399687e-02 1.116667757327807070e-01 8.171768165124047856e-02 2.422125639757768956e-02 3.576407906214331689e-01 3.902031061103530263e-01 3.102764920546783878e-01 1.219882231517285982e-02 -6.457117483993186739e-03 2.838409273074287646e-03 1.803580842983319374e-02 4.380612980441113413e-02 1.148714975502331849e-02 3.608525015950891923e-02 3.432089202080963741e-02 4.271813557647721749e-02 1.112913674074731893e-02 3.549909624722866197e-01 6.294133697541264283e-01 4.130683719494960826e-01 4.621466956577488516e-03 1.058560777870407033e-02 5.087294650697517934e-02 6.087850239640497235e-02 5.833221918963747005e-02 1.626864708875776500e-01 1.497363377264897266e-01 1.183463163038298155e-01 5.087944686175718129e-02
1.093124024639747505e-01 -7.021205205233502222e-02 6.914098433861846915e-02 5.131724913313107789e-02 4.307467065882960439e-02 7.535002325201992091e-02 1.237132704995941163e-01 6.780258727644407346e-02 3.294596917250603579e-02 5.797101034261331609e-02 1.275117144245431211e-01 4.157988215066041326e-02 2.415382852309946177e-02 3.730417078934848696e-02 3.872230085296804458e-02 1.106525406965697400e-01 2.522570813671919376e-01 7.761125935701412848e-02 3.991367512069606688e-02 2.982544524706244940e-02 6.352550818127918053e-02 9.347177210735276368e-03 5.704754747819999749e-02 6.866372043401648695e-02 4.948245696885759243e-02 3.990059974971231493e-02 1.497537507918404731e-01 5.148404622029418426e-02 2.304783459918534685e-02 3.505407482605233136e-14 6.309367454724133101e-03 1.543807791189776707e-02 9.362748220130395760e-02 3.057906976897632051e-02 5.705169992424846681e-03 4.596405720169545982e-03 7.492995955207009317e-04 3.876812574860846738e-03 9.794202204413847077e-03 1.210192116673240899e-02 2.419917218650195767e-02 1.941606898896596628e-02 2.039609647639419390e-02 7.218603377509752493e-04 2.763139840441770217e-02 3.058549653486272030e-02 7.874569592057426015e-02 2.160321005370565914e-02 1.057139840239581033e-02 3.812501887646881776e-02 2.789273324619711072e-02 1.127596318455381143e-02 8.918838538632237883e-03 1.391971095688146996e-02 3.902656941823451353e-02 1.680106523536269506e-02 5.079546493507141464e-02 1.921744246313172866e-02 6.923399557516007885e-02 9.061066132924583405e-02 -4.306801252853469791e-02 1.378279145057123697e-03 8.681394563766440045e-03 -6.232858197589156296e-02 2.150441539520766762e-02 -2.589631607027087848e-02 -2.712416741584986661e-05 1.661937455817789813e-03 7.494659484825828659e-03 -6.421041663604935878e-17 -1.609813636008191468e-02 1.287620601585693671e-02 9.223136261441210529e-03 -6.154998545041214345e-18 5.437889225491582962e-02 2.471351071030127369e-02 -2.841303855780853357e-02 1.791756277527039101e-02 -2.229183781422172314e-02 1.138934977546614479e-17 -7.221575204142217653e-12 5.142465175596265536e-02 1.164556768461998404e-01 4.069821286441762526e-02 5.686692389653889151e-02 -1.529288152155405811e-01 2.820858157618686413e-01 -1.300859252171442278e-02 6.607279951587180511e-02 4.545804787687379722e-02 -3.328596987679250524e-02 6.040468486945610449e-02 1.699238722423770378e-03 -1.309686339329441853e-17 1.139591287717878080e-03 6.915567797259819516e-02 1.468431007648749245e-02 -4.965222070314642688e-02 1.749934639580226318e-03 -3.459443071332773413e-19 1.384919900953006291e-03 -1.089053340183797174e-02 7.807436721482182221e-03 2.290687111287679698e-02 6.534059421144822628e-17 7.114762195952147039e-02 4.342204025300532754e-05 -6.020611483808336424e-04 -4.938861555109695309e-03 8.398163003958368511e-02 -6.568071454743028084e-18 -6.505960451682786527e-04 1.104936420422377233e-02 9.084291176664191503e-02 2.117730698156702548e-02 -1.841741915915145328e-02 -1.260916520901643380e-03 -1.206804284775447010e-02 9.530979632535604984e-02 -1.337441666424367750e-02 6.375155616548139337e-01 3.712899515516897564e-01 5.970787211710419651e-02 3.114531430368315187e-02 2.189279809822295558e-02 7.988508480019081148e-02 8.714312235433376019e-02 3.213210871985974770e-02 2.867062748229194311e-02 3.036292434055520503e-02 4.077862721462362922e-02 1.259783493729717610e-02 3.428035722856143108e-02 1.183685183903857019e-02 5.284711774278444407e-02 9.710747900085058004e-02 1.896184419581847802e-01 1.828600875841127349e-01 4.903153496245718501e-02 4.933254327227925423e-02 4.400091106215467396e-02 7.771669987103548352e-03 5.888198902385485267e-01 9.825232034558003535e-01 7.260117855506215134e-01 2.807747944900524880e-04 1.879920308101720554e-03 2.512908272774383271e-03 2.662907671755208698e-03 6.719674689606414640e-03 1.848686822666167906e-02 7.814143562581988833e-01 7.570020097949726745e-01 6.592272778111701870e-01 1.389928501914364754e-02 4.403018923581409998e-03 4.431694650953258222e-03 1.088784986618995636e-02 2.585146315918659260e-02 2.094267794965163493e-02 3.815339422046332141e-02 3.506082994562619709e-02 4.977730106739584881e-02 3.441636934855818279e-03 3.680133628300539389e-02 6.625469256985544453e-02 5.500215245512798251e-02 2.629004979545798010e-02 2.149335294495128765e-02 2.554137440082846280e-02 3.392739960193793397e-02 -3.065129290957162213e-03 1.177200009476056866e-02 1.291051194886078807e-02 4.545310457998330556e-02 2.470935229024871782e-02 9.763114645362139243e-02 4.269320047218264080e-01 6.287774329390856121e-01 8.419341201257316554e-01
1.157915966595918500e-01 -6.791784202462936015e-02 6.518203280834190816e-02 5.238658528346818583e-02 3.251810045247575942e-02 5.563349054234114494e-02 7.370456540454810301e-02 2.436683737593926813e-02 6.014733935985356467e-02 4.851063980110890267e-02 1.066513231373373560e-01 5.799519332644900022e-02 1.597470614132455100e-02 4.027873579045484348e-02 4.698344686278835974e-02 5.856570932435445354e-02 2.552421927248834099e-01 1.119641621827150990e-01 8.077380456733107561e-02 7.501156659581871555e-02 5.318378512009613174e-02 3.935788126424116101e-02 3.392310733534333739e-02 1.857057927147519327e-02 1.000713124663287282e-02 1.110224421117614671e-02 3.454304362951193114e-02 -9.022508354657234400e-03 1.290297336130949012e-04 8.223300073729983284e-03 8.397211369788225993e-03 7.832311715236815741e-03 3.583434473861633342e-04 8.020565615151211553e-03 6.795738355356434335e-04 -2.787249981509536401e-01 -1.553834057133382796e-01 -7.877071853175642013e-02 1.207325694409693874e-02 2.308674695367135890e-02 3.348499244355683524e-02 4.304228867996393038e-02 3.109500211768612826e-02 7.606877947301601481e-03 3.216857788874761398e-02 5.777132156389247486e-02 7.821238603183121307e-02 2.396782709127586244e-02 1.499435263762548265e-02 3.449680353605320132e-02 3.886532511608709051e-02 6.840770005369418321e-06 8.845845773351620312e-06 7.160466299362361169e-03 2.152221664656787936e-02 7.980709341228161072e-03 -5.072605957616734829e-01 -1.578123259056156136e-01 -1.777462723676481171e-01 6.635527940460345109e-02 -3.319858727566007972e-02 2.331664428769477174e-02 -1.019413173739973887e-02 -4.994829362464753286e-02 -5.108553931064295159e-03 -7.808125502971002829e-02 -1.103751589908048012e-03 4.831902012938706775e-02 1.396135040869662600e-02 -5.624850163073503734e-04 -8.880518659565882005e-05 3.038220279894986164e-02 8.300647978360659041e-02 8.154229184399730546e-04 2.652090367900000190e-01 1.461720187357946031e-02 -2.388794575540400048e-02 2.154530905540199409e-02 -6.713382757820825220e-02 2.969674506443571380e-03 -1.646341679448429110e-03 1.470430594371345545e-01 2.151018411555725984e-02 3.980894608879811253e-02 8.282486726477463618e-03 3.763283162086893813e-02 2.512592213592254597e-02 -9.367418031779140702e-03 6.809896290390828055e-02 -4.306564773752888722e-03 2.093468611132274474e-05 6.654829451576428596e-19 1.995851749885350634e-01 -2.134431569780216240e-02 -7.484679238051066523e-18 -1.753468888014760724e-02 -9.735323931722036916e-03 -4.407158993854458362e-02 5.018990687267511214e-03 -4.099309612734365266e-17 1.681613410206114420e-02 -1.710345166917061821e-03 3.947069438315375413e-03 3.245861500791769744e-02 1.364400771920988922e-03 -8.854899656577229494e-02 -2.917022823078141772e-02 -4.738901622103494052e-04 -7.229306150276212928e-02 4.251683006957662769e-03 -4.045989667022276658e-03 4.714881288042568203e-02 -2.658400280398258084e-02 -9.623806495957730120e-03 8.436419517799599761e-04 -2.995253858478869394e-02 6.945378699419617156e-03 1.676908027673068569e-02 -1.644753532868649948e-01 2.220859600443306214e-01 6.533101626172239662e-01 3.946454570292095010e-01 5.206563281810162869e-02 3.760545272203202893e-02 1.573798583280885260e-02 2.822795276060415665e-02 4.559995319667477925e-02 1.236968180434508228e-02 3.565687537463539669e-02 5.638388658990851204e-02 5.108402492066765194e-02 3.203888449087120144e-02 3.229012172656180180e-02 1.952913690985065218e-02 3.229353556960982280e-02 7.958986330414256904e-02 1.572439485958630201e-01 8.333019938198758803e-02 4.320514335416004298e-02 6.405632975632420678e-02 2.098609497316421268e-02 5.479308676140922651e-01 6.965731956464070729e-01 6.316674183332888770e-01 2.195374356638013155e-02 6.900935378377102969e-02 1.052042657917522106e-01 2.713431427038147034e-02 3.148755770444829072e-02 1.733436248975286920e-02 2.324135138292909505e-02 1.067374156218927825e-02 1.795333495584890138e-02 6.312332288721780350e-03 3.383879413463786179e-02 6.119345393396983246e-01 7.219544452171713367e-01 6.747897096719142063e-01 3.955796023043255116e-02 3.150323219426433979e-02 4.928144165010862193e-02 4.625142927486528693e-02 5.656339816428438555e-02 8.546118329254113674e-03 3.484966727186523949e-02 7.035355073615880139e-02 4.927625176559351922e-02 1.922423527926339637e-02 2.642859230073555526e-02 1.081043152944730426e-02 2.638578540871090380e-02 7.960193969410268161e-03 1.225967140599243388e-02 1.809279087047870743e-02 4.319930090508845855e-02 1.638999819666676905e-03 1.476867640904178380e-01 5.428349558587476364e-01 4.367043036920389976e-01 7.758130470482800245e-01
1.325449960373749902e-01 -6.549142212652495076e-02 7.007911484678908121e-02 5.470941147886872191e-02 5.450379841884474275e-02 7.043095983168355667e-02 7.345463937624202577e-02 5.765778861490710588e-02 4.272788303395218257e-02 4.211713140063142596e-02 1.037588734246331218e-01 4.735755466990795165e-02 1.772253807917401058e-02 3.904933805632455901e-02 5.815490889748058806e-02 7.691656325695407637e-02 2.725951200942520747e-01 1.440367724742324385e-01 5.467390608204185376e-02 5.984194963747058865e-02 2.025327053309294301e-01 8.786855072842805758e-02 1.077388462910765510e-01 5.249872873149078606e-02 5.537991935322753717e-02 6.329624642948698809e-02 -6.878389855541067188e-02 -6.429295160589511282e-02 -8.778714399311880615e-04 -7.857953096958315842e-02 2.823757196869429095e-02 6.208597869723819773e-02 4.078044650750951849e-02 -5.784898974270643451e-02 -7.358648582980024250e-03 9.816931040219989760e-03 3.048534955458225004e-03 6.601561079894775405e-03 1.240829173717033704e-02 9.870331354999782592e-03 1.739638642824338499e-02 2.890792489706036847e-02 2.592759947752002608e-02 1.138884337502076363e-03 2.190891082669342571e-02 4.839993092574902504e-03 -3.095296599483655237e-02 -6.010798441206006981e-04 -3.255179616675237322e-03 5.959178800881303334e-02 6.788070739958795841e-02 2.140107717981664867e-02 2.372296195550053360e-02 4.137992444194146108e-02 4.281879498572265658e-02 1.208574824461770836e-01 1.176968570212321985e-01 8.862199497303124840e-02 6.419086887072650294e-02 4.175853929748294530e-02 -4.324437308617671422e-02 1.254374460650398843e-02 3.149916544233410075e-03 -2.050650833348046861e-02 2.239632382143615444e-02 -7.637053026716025643e-03 -9.162338383533627723e-05 4.793778220566246098e-03 5.346354308546098083e-02 2.819113866348047642e-02 3.321668779318207899e-03 -2.012054042305293049e-03 8.248486251367689870e-03 9.599422114457473914e-04 1.750696210981352585e-01 1.212317301218169066e-02 -4.591947371085679241e-02 4.568572678614784069e-03 -2.740203137536662698e-02 -1.376295681128593962e-01 -3.846507703503376091e-03 2.077445026289528873e-02 -1.989462114489731065e-04 -1.359759923632320767e-02 -4.070561907866231295e-03 -3.881317462385733448e-03 8.629058490266211523e-03 3.181227768073610802e-03 7.482029092758003819e-03 -1.097375105960168989e-01 -1.890946851534927933e-02 -3.881061383903390734e-03 -2.665317543416580864e-02 3.646414983516171882e-03 1.644462131708872288e-02 -1.488472609808180530e-03 5.804191488706015450e-02 3.292465099851893597e-06 2.294346521963522290e-02 -5.518826311841250894e-18 3.992657844835981878e-02 1.315902382898154720e-02 -2.356241771044463216e-04 2.402419981791713635e-02 3.249829214794407173e-02 -8.544476330229365188e-04 3.940679122004005913e-02 -3.159236905103893212e-02 -6.333997826523704577e-03 2.178595437337815105e-02 -1.335396810006846124e-02 -9.790366925408858917e-03 7.612357054546844419e-16 -4.568315879084609149e-02 3.391424254489950468e-02 -2.562513984087410462e-02 -1.623455904207387994e-03 -2.289008703723381319e-02 2.134171587678248411e-02 6.785421188272262123e-05 6.517388204969318810e-01 4.164342012306319618e-01 5.551938050328123625e-02 5.113481655651185703e-02 2.503228190669892078e-02 9.821203462038512189e-02 4.591947370396999573e-02 3.076914026101152180e-02 3.470303837135471636e-02 3.018805216507901637e-02 5.961608621035487110e-02 2.316609773451933432e-02 3.742916035948453540e-02 2.665344244311018337e-02 3.861169569586584555e-02 7.247424636282764898e-02 1.343736144237238916e-01 4.222287089110803721e-02 3.040496772681041890e-02 5.257441849115845889e-02 2.593310990221589854e-02 1.124875955120968650e-03 3.168585061347267695e-02 2.338682882945117281e-02 2.742078852213265805e-02 9.695195393788850688e-03 7.125602405632586178e-01 1.195680985695372511e+00 4.795424150164226962e-01 4.792938099298988952e-01 3.941252554631036742e-01 4.294369491147122520e-01 4.838354560545195526e-01 6.959710456204248885e-01 3.055588344666094747e-01 2.764743398200222423e-03 6.748240868772068142e-03 1.024902759147577024e-02 1.608734116366632508e-02 2.933872344961086995e-02 5.514453606605481534e-02 4.495656925734663883e-02 4.710977761317931140e-02 1.366441764674689030e-02 3.028345449999122682e-02 9.408416189742106750e-03 5.259114727382845356e-01 9.020519667732420643e-01 4.476084641786954865e-01 6.137810345918157454e-03 1.869864574593631604e-02 -1.082874214765256263e-02 7.569744952330996826e-03 3.408494286949018126e-02 4.925356601683596874e-02 3.699162877554328793e-02 1.338321952217454425e-01 5.447535522372049660e-02 7.034042844313620479e-02 6.462366595426845273e-02
1.546126390179760612e-01 -7.283149425389784382e-02 5.011138610000317323e-02 1.939501274457730573e-02 2.595962752394790837e-02 3.336091479333719995e-02 4.119699311966451560e-02 4.053609731062690104e-03 3.996583165397236959e-02 3.691942943053773407e-02 4.876364957373624520e-02 1.744733938250640076e-02 7.187007977034942131e-03 1.631486492957348600e-02 3.143552161892091551e-02 3.392729832088603459e-02 1.007906278134345218e-01 1.741024577130020207e-02 8.520273125627118166e-04 3.049958462076961599e-02 5.175643036227676325e-02 5.507757037091561871e-03 7.042134242422334878e-02 2.645920920105950813e-01 3.812718702278442701e-02 2.899242855755049600e-02 7.654937545957914147e-02 8.508194601468663243e-03 1.141421785737273516e-17 2.100311105019838742e-04 4.685011015404540700e-03 2.501135423584358811e-03 4.948721903331891318e-08 1.762223532586094628e-03 1.072843877978306437e-02 -3.577867194215606506e-01 -1.360913274053185684e-01 -1.904550167849108910e-01 7.624370268692404369e-03 1.273486477500281912e-02 1.565472972279308844e-02 -4.084535159507377694e-02 1.838596651108280813e-02 9.830069678617435122e-03 2.068649912492425552e-02 2.843389334055408668e-02 4.406429079729372311e-02 2.100261353851010787e-02 6.477461012576319656e-03 1.595857220358459436e-02 3.890045034296669979e-02 3.954892200156544767e-03 4.746226853448083129e-03 9.441991530040159605e-03 4.076392006790426764e-02 1.386854864626553552e-02 -3.850913371706453092e-01 -1.533284185226525542e-01 -1.805952776359861789e-01 6.220587188591816113e-02 -1.002814826519018571e-01 1.651296854564642674e-02 -7.826646306345986376e-03 -8.168707218646077683e-03 2.314807870921965424e-02 4.184651606771269574e-04 -2.482461485675104207e-05 -2.212056993885954878e-02 1.268416705735928712e-02 5.585115751864503687e-04 -2.635020285573962199e-05 1.168448204666110678e-02 8.708540533519234725e-03 2.285522513280096278e-03 1.615482164923204289e-02 2.109600413822120685e-02 -3.032385751874510163e-02 2.755272684822225419e-03 -2.116413182589036920e-02 2.837580999857493378e-03 -3.837295991934443987e-03 7.729502838359611494e-02 2.208742936428595774e-02 1.373113399302517074e-03 1.809686008265928153e-02 3.230690852009204979e-03 6.305268796612603643e-02 5.628348224075050240e-05 -1.257924220093845254e-02 2.484965340868290282e-02 -5.663356965234395482e-03 7.538380388466750645e-04 3.851679549227552540e-03 1.209762434071294393e-03 -1.076340892309895310e-03 -7.699985086595223516e-03 -4.818507324154171906e-03 -8.889930446043045609e-02 6.391649915147145550e-06 -2.590264265809399475e-05 1.023016877263444903e-02 -1.944596664439688163e-02 -4.290918726247940594e-03 -1.251598671337873780e-03 3.916935178688419825e-03 -1.085952278351063997e-02 -7.628589146484515346e-03 -1.039439971816688742e-02 -1.583674992727356762e-02 -5.313658613251098740e-04 -4.004438078461560954e-03 2.084130468387046173e-02 5.415788202946247853e-05 -1.721403077409322130e-17 3.001440211185611717e-03 -7.884778423634154071e-03 8.111956009284430891e-03 1.098629437077531038e-02 -1.979079392789918346e-01 1.685964823455705364e-01 6.443180333095417378e-01 3.597774869519173579e-01 5.191888088252229194e-02 4.506809370251824154e-02 1.874296129260628885e-02 5.935124963017564953e-02 5.246358468999083591e-02 1.689152491510958684e-02 1.856171892524449696e-02 1.402120602558946938e-02 3.452634756831940432e-02 1.209222887310510905e-02 1.847163974770252640e-02 1.705158724067772427e-02 2.529174080298908153e-02 1.526986000051054788e-02 6.744402502062957216e-02 3.014490610318779421e-02 2.930796875325530904e-02 2.011604136263785841e-02 2.396388906714768188e-02 1.841888273598247042e-02 5.528549622928262686e-01 9.370029021342702880e-01 6.310334830319564992e-01 7.822185487358922261e-04 9.450747066126686385e-03 2.171829923732704597e-03 7.582510329587517639e-03 3.322201203114513810e-03 2.159274419369983100e-02 1.598871975110958531e-03 8.412884000240391721e-03 9.790907229150061508e-03 2.777165256999376483e-02 7.420787159485752182e-01 8.127704005880057592e-01 7.446660151777747538e-01 1.999179941348749531e-02 6.166969936957910139e-03 2.801744494575331840e-02 4.423815028133847100e-17 2.663148573923954099e-02 4.760607686343632296e-18 2.561701364771522618e-02 2.510476707124754611e-02 3.788190644993518624e-02 3.179455705692679074e-02 1.761058202113004872e-02 1.861776297191130095e-02 1.511436929783732154e-02 -3.667864178740703942e-03 1.678600352948295821e-02 1.632283353613702553e-02 5.639794642584372575e-02 6.352545094977757670e-03 1.561573906682514890e-01 9.283720244107774855e-01 5.883312665228767058e-01 7.961297103116068419e-01
1.431828493851726170e-01 -7.094945489885906564e-02 7.511446702236587725e-02 6.594771404121579916e-02 9.370293621425070107e-02 7.016352702515150919e-02 5.982203017033418946e-02 2.483929635539804809e-02 4.082066659264609043e-02 6.582073186440784429e-02 6.355323184511081636e-02 3.227607399491643630e-02 1.242417994477904367e-02 2.362428728019356702e-02 6.635786054950991442e-02 1.206593862097815190e-01 2.407620863834756542e-01 1.452428221188891699e-01 3.614206872975470053e-02 4.597476886342570512e-02 1.881149063617554296e-01 1.814883589284699630e-02 3.285407964893760635e-02 4.407358942726948697e-02 5.254864438794417530e-02 3.721827751974341597e-02 6.210783264406738446e-02 8.103650073417888525e-03 1.592033395066280374e-02 7.066592211639159335e-03 2.418099565370702975e-02 5.499229712514713906e-04 3.761166870955924601e-04 6.961599617798935880e-03 2.514567815794269134e-02 -3.937251239125395541e-01 -6.477819434468719273e-02 -7.331037445026751531e-02 5.062482816269148894e-02 2.307309908869354872e-02 6.021933900549741869e-02 7.469863362660014716e-02 4.229611534452311361e-02 6.450092030332943427e-02 5.350805588509827126e-02 7.670709715846683874e-02 1.417211168731281334e-01 1.653707141965214678e-02 1.235739013316507727e-02 2.067174948511893429e-02 -9.854700649624490577e-03 -2.850875462185574259e-18 -1.706860779793168987e-03 8.097491153175689904e-02 1.707859371950145588e-01 1.034424473796184824e-01 1.425145903667287550e-01 3.142493299996489964e-01 4.355979342717522518e-01 6.868113961341643092e-02 -5.206024466875230211e-02 2.512511747655938460e-02 6.034062155242819227e-03 -6.666193669272278688e-02 2.426132595240274026e-02 -2.633029253523581914e-02 -2.264484413238202105e-03 1.308313474544554981e-18 -3.121618106161674547e-17 1.652693577620480136e-02 -1.830405826604460020e-03 1.661289586596652323e-02 2.258735721887201661e-02 -3.413389057686041191e-17 4.182247493613214934e-02 1.764977108816116283e-17 -4.534037901045097352e-02 1.288256078010074088e-03 -1.138445837013158865e-02 2.769672468570623380e-02 -5.169050281980098164e-06 1.272054163906832965e-02 5.584867806442531113e-02 -4.041536781039228865e-02 4.460195926157786012e-02 3.163818681838029279e-02 1.989967411514801179e-02 -2.521842108306033517e-05 -3.978800738471201942e-03 -1.131926823726270813e-02 2.077692955934729371e-02 1.219837391093902797e-03 -1.100389572899490055e-02 1.424178942048817328e-04 -6.678968481693354012e-03 -7.898863859676107102e-03 -1.859658857341600982e-02 -8.022495530470550446e-03 1.118225654394206826e-03 4.023300794998176111e-02 8.986437098552339147e-04 -1.653619420139435317e-02 1.946842137897082659e-02 7.480088320918093503e-02 1.070280799434417639e-02 -2.408884348332094097e-03 -2.307258732269265242e-02 -2.153919319479328728e-02 1.794620390749545116e-02 -7.027651899747797202e-05 7.850233203197700682e-01 -2.191976354676403865e-02 -2.797753482047005483e-02 2.242558488905666697e-02 -4.246049021535121953e-02 -2.230905203604396925e-02 -2.796787020914716371e-02 -2.969933561463276514e-02 5.007689287134158007e-02 9.669629878928900262e-04 6.605080416505130225e-01 4.105018957655952594e-01 5.183116503620686932e-02 4.559420724810418074e-02 2.329943317158295057e-02 7.437403485130909819e-02 7.692056229586879956e-02 6.201424866203291905e-03 1.827492227709438810e-02 1.330125193023785381e-02 3.950078756354123005e-02 1.493784002323628873e-02 2.217146853149674218e-02 2.409162808846857656e-02 4.818561234773652974e-02 5.957478284074416847e-02 1.350040554904230472e-01 6.243893818537959994e-02 2.806907993987161135e-02 3.705520471830177354e-02 6.755094722573719290e-02 5.590553598696173382e-02 6.807110682746664310e-02 2.846577751590145464e-02 1.379649385100014704e-02 6.541115262543631959e-01 8.336217138809448590e-01 9.066380180182667115e-01 2.726491729069850378e-01 1.864920506985081328e-01 1.573309976158387413e-01 2.121035857125490032e-01 1.542229110054803565e-01 3.870491978637753960e-01 1.978419784712650564e-01 6.678311673582095764e-01 6.797151668519255674e-01 6.990110834153138164e-01 1.992058907322519570e-02 2.646259514998844375e-02 5.428823676443879365e-02 3.929436510952499101e-02 5.032006444923321081e-02 1.425367605268222778e-03 2.554500560928226610e-02 5.037863850078724837e-02 5.235774686940943001e-02 4.197846901429231475e-02 2.868197828168931110e-02 3.027723529799182522e-02 4.342160304849104757e-01 6.603894623827317734e-01 4.803090397615905061e-01 1.181584067053826660e-02 4.018664279660814909e-02 3.904715527636275912e-02 9.099253259836821706e-02 1.434062332053474484e-02 3.600336237650029331e-02 2.808822487472995011e-02
9.223009091898614481e-03 1.720154206347531203e-02 9.250131146830879725e-02 5.405445316037409131e-02 7.255371260649956522e-02 6.975788881967634469e-02 3.190027974486669970e-01 1.028990160493772482e-01 6.983980119969880607e-02 6.928882861735721765e-02 1.136651475299171737e-01 4.925887672547269586e-02 2.502912719373246234e-02 3.689274427609865598e-02 7.925518039639163625e-02 1.344355384694025449e-02 1.300839983513380760e-01 2.356296216233129751e-02 1.143410235270740823e-02 4.451617667421291918e-02 1.677033753610850664e-01 3.457118715772976997e-02 3.828473784787229245e-02 2.654174342785088278e-02 4.456019734865709209e-02 2.506265116839239554e-02 8.407886461412571177e-02 3.483022581724318223e-02 9.773804205754587065e-04 1.488508830009178940e-02 1.003705415513446268e-02 -1.581585802132997806e-02 1.884104352200786708e-03 5.190685234116221639e-03 1.132141141651628932e-03 1.198895835896166473e-03 1.158392123457089376e-06 2.943477764465488641e-03 9.966786283944392352e-03 1.496879311654144423e-02 2.500097008710579910e-02 1.868054674496305231e-02 3.432915740494107970e-02 5.430311755989991260e-03 2.870396645102517644e-02 3.062145807968939815e-02 5.808261942473563061e-02 8.774035475283180341e-03 8.083023920766348527e-03 2.508540336722941272e-02 3.743945191584630017e-02 3.904215062669797245e-03 1.832480484262745402e-03 1.131675120384276027e-02 3.035544910363115537e-02 3.082705729329579883e-02 5.149609216254499161e-02 9.596402959949987960e-03 6.638213049228314233e-02 2.754402835520263632e-02 -1.576416809502538541e-01 3.586750055067782633e-02 4.372910747611942484e-02 1.245666342421204483e-15 -6.072743559746993569e-04 -3.190074327858321429e-02 -1.148484766923647815e-02 9.552546289803912072e-03 -9.224976751970759442e-18 2.951552721316037450e-02 -2.465852073762712579e-02 4.067058601176695015e-02 -1.318281571914137282e-03 7.391402621022775954e-02 1.653259897356232355e-01 2.991587508378406035e-01 -5.460388714916697070e-03 2.985645492683065231e-03 4.502737889147689215e-02 1.403040784724302568e-02 -2.014042799869312020e-02 -4.714199761058709623e-03 -1.277880900210287291e-03 -2.612335880864216087e-01 -6.159756372700649768e-02 -2.260557445914066280e-02 9.588934325926574964e-02 -1.347854480791844381e-04 3.069229766780834368e-02 3.324144988993701921e-02 1.741544354944984593e-02 1.673256008174566281e-02 4.776633896610198160e-03 -1.384006211133126038e-02 1.957807044196979013e-02 3.974491409416241082e-18 1.299745335126146172e-02 -3.775032430662651761e-02 6.606471489893783570e-03 -3.916488475416238582e-03 6.097387694249526910e-03 5.817335077091179661e-03 3.436091071379886869e-02 3.115416865705090618e-02 1.580623074821747159e-02 1.370761493604636186e-03 -9.824287160841726780e-04 -2.535275361926706082e-03 -6.938768126560073725e-03 6.154986892012935529e-02 2.186196341238069371e-02 -2.295051080471498868e-05 2.023160292460336856e-07 4.544813484602767267e-02 1.194121642250540495e-02 -2.320668719567210714e-02 -1.204380623320442582e-04 5.943080774219289958e-02 1.118549001673595361e-01 -1.098636229645383149e-02 6.831612347877440150e-02 1.026855373360883172e-01 7.659173800919649722e-02 9.792833955960277004e-02 2.715156709624319889e-02 1.132431041415516426e-01 6.630198054246302930e-02 1.953279088016821735e-02 1.810140464522930204e-02 1.366885960045210145e-02 1.928167184770619019e-02 -2.374468904966918134e-02 5.510045520302309791e-03 3.324395457342366189e-01 5.747785227876589520e-01 1.621329613351795429e-01 2.137779018744513995e-02 9.080051495339756806e-02 2.653192663192642867e-02 6.653030151224267885e-02 2.056974461537093915e-02 5.229178954437699138e-03 5.615417017338439343e-02 3.004740605140387988e-02 2.461857046197699420e-02 3.233366454388359595e-02 3.185172150028765109e-02 2.184651640078277141e-02 1.257806327509082556e-01 -1.642148606530259569e-01 -1.015154635203786848e-01 1.480004855551563248e-01 1.280265307976180024e-02 1.586625356985725890e-02 1.811559695101347400e-02 3.857358368744558076e-04 3.925080011628968683e-03 9.229263986298494185e-04 2.242075238078829191e-02 1.281351612921886873e-02 3.697304191785062005e-02 2.740565305509292185e-02 4.047654200189196305e-02 2.151500705713742419e-03 3.864995635605410207e-02 4.105337050741693361e-02 6.811061610128588129e-02 2.125234118373431000e-02 2.441215494122137702e-02 3.094617206743034352e-02 3.984660575437690838e-02 1.167689378649461492e-02 8.772617921195576332e-03 3.519769775326718475e-02 4.807934398901821116e-02 1.476510396563911545e-02 9.915355207667196635e-02 4.078402858987441593e-01 6.461952085724278483e-01 9.053030231637273495e-01
1.312757308079602891e-01 -6.522402092116982852e-02 7.035487775872796556e-02 5.215728574071959445e-02 3.931846565639069563e-02 4.966026872971651684e-02 1.029416721798478151e-01 4.076444061196739937e-02 3.435659686522567896e-02 3.229322961909811274e-02 9.830496481962970601e-02 3.375549656542649324e-02 1.862511029710057706e-02 5.402088816017672906e-02 4.341151244113958252e-02 5.257755021514755323e-02 1.218604271408193218e-01 6.305534129819406752e-02 1.680469175363556028e-02 6.335291951284872936e-02 6.188266385619296123e-02 2.747486866217621268e-02 4.372433659744153628e-02 -9.479668409519458397e-03 -1.166889767961276441e-04 7.896326613109506259e-03 3.528312363938469060e-02 6.046369391647538408e-03 9.396095710787292013e-03 5.172768778208157101e-03 1.422249734577790692e-02 2.419745525029507946e-02 7.721138793337397298e-03 2.968801599164421282e-02 2.420526582617871583e-02 -2.679305449867884770e-01 -1.180211048650962047e-01 -7.330119793637396297e-02 6.022999205616265395e-02 2.831622296415498702e-02 4.322107680806747815e-02 2.972822402897840749e-02 4.290558708535810450e-02 5.883267581738942792e-02 5.604473293399293843e-02 5.629173010424436824e-02 5.304657060544430008e-02 1.383241117092862994e-02 5.821397758836003439e-02 3.393743813561501260e-02 3.759572168109905077e-02 2.596652888858063149e-14 6.378441008367195277e-03 2.880023886470388199e-03 3.315431754569633793e-02 8.142897194468336208e-02 1.035267906504433871e-01 2.444874799993835834e-01 7.738428558311892524e-02 2.585979765964419724e-02 -4.448196536649064864e-02 1.526417077358239137e-02 2.919508415913672555e-04 -6.928831943776520008e-02 1.408664771189092196e-02 -7.508340281320056986e-02 -1.035160073525219146e-03 2.685384303675773798e-02 8.628648716331828450e-02 1.650808079076812943e-02 -3.283691111151658900e-18 -9.142614031403314273e-04 -1.807188423947090478e-02 1.570612847344550955e-03 2.893255028376957166e-02 9.671688348210452646e-02 -2.469445142759711664e-02 2.905349780177076259e-04 -2.639926054184346584e-03 -1.859915877177847204e-02 2.521663361801175854e-02 2.347765536214117821e-02 -1.092459495012698628e-14 3.271658123897888681e-02 7.698652426298950406e-03 7.823243605170523973e-02 -7.403752875053946393e-03 1.108161502034012242e-16 -4.358588591614568361e-04 -1.979179137582252901e-04 -3.009770295960856323e-02 1.833001481465492202e-02 2.398409775049737194e-02 1.991808951630565441e-02 4.890809293878407848e-02 -2.198047058351874569e-02 -3.322604829799230466e-03 -2.262521192661605396e-02 1.071848070427508515e-01 -1.913234775523931083e-03 1.565776011700113074e-02 -1.816908146727595005e-17 -2.331606490535602372e-02 -3.628093271053017127e-02 7.192151694518672198e-15 -5.601819935847435206e-03 -3.674039044818525751e-02 4.410813345457549606e-02 -6.881936499806002097e-03 -2.940009262219521642e-03 9.604957761828535057e-02 -7.716781453848906014e-02 1.589453531346381177e-03 -1.876391692091759419e-02 1.975119491463115870e-02 -2.147180650734583848e-02 -1.456665177637803121e-03 2.055881528435081138e-02 3.004082885997190824e-02 5.560044033160774571e-03 6.274205262843792763e-01 3.990384197196981675e-01 4.369847681144701373e-02 1.903491990814021317e-02 1.969815189382462250e-02 6.507362739299776822e-02 8.285487314796718961e-02 1.073309301784366315e-02 1.472160516649503870e-02 2.130211313983645591e-02 3.799073843206490081e-02 1.769003063924930869e-02 3.090982945661480477e-02 2.513051529794318659e-02 3.369408225798957435e-02 1.883797542213843171e-02 6.846496824969879047e-02 5.778722430511343267e-02 1.210341164737625819e-02 3.873974387549881243e-02 4.238000585419135491e-02 4.705513008911506659e-03 3.092239344136979429e-02 6.058622497784708072e-01 8.181525830786732234e-01 5.765364126378013099e-01 5.759024506454437503e-02 1.288989028596209188e-02 2.282629522308019313e-02 3.807226411020454143e-03 1.781174655973903273e-02 4.056393046796809634e-03 1.405850804280160682e-02 8.250613525111080010e-03 3.961161293507114295e-02 8.003832698751833785e-01 6.336508949756487485e-01 7.274047216359134094e-01 2.587966952264458831e-02 2.980900407108273004e-02 5.561197516228032212e-02 2.723083534895495705e-02 4.199140206441408923e-02 -2.720318995553523689e-02 1.973154210212843887e-02 2.819639967313093340e-02 7.396189078128627414e-02 4.200131360630938726e-01 5.199422697315878583e-01 3.972936494826722065e-01 1.447919969690978324e-02 2.541848275300343158e-04 5.351909602084041657e-03 4.586743878546162972e-02 4.890685840954869801e-02 2.910729694508813747e-02 1.022700509562526200e-01 5.684099349997353418e-02 1.227096333635884773e-01 1.333608806333787256e-01
1.382860545200396973e-01 -6.193275544450688869e-02 2.418317950471927241e-02 2.066246192511488056e-02 1.784327839417701109e-02 1.210534261240279498e-02 6.531713419862206216e-02 3.700078359807472319e-03 2.467924746754059789e-02 1.507914528879568221e-02 9.469049742928449531e-02 3.623888689204211866e-02 7.866910576875663086e-03 3.460933722062557727e-02 3.065513729430941609e-02 3.466091265569907348e-02 8.514903836345255606e-02 3.255886585496806096e-02 3.824654261374155900e-02 -6.997558458270447534e-02 -1.288621305511246928e-01 -3.381870514543537654e-02 3.376401264636674782e-02 1.247807760016383070e-02 2.122483663906048895e-02 5.849256354767892740e-02 1.066948592969513276e-01 1.985276170789250852e-02 1.059493936439432402e-17 1.960519102245389649e-03 4.979795298705324766e-03 5.436639447124254343e-03 1.552283251702070216e-03 1.883689505008949829e-03 6.639181582182347545e-03 2.816430603533886731e-02 3.756708495672449188e-02 3.007777816608913843e-02 -1.668882929949495192e-02 2.907452240758809717e-02 3.038723930636588050e-02 4.166680816795615555e-02 3.721282028326926283e-02 1.990322772915771588e-02 2.672091129569021187e-02 5.363284844359033493e-02 9.726332040478151419e-02 2.551820334908607921e-02 1.742105715468979243e-02 5.807595640823910721e-02 4.314364371142444948e-02 1.328862098089574062e-02 1.113723332994694462e-02 7.842780797391556147e-03 4.402113285593935860e-02 3.677912803445766710e-02 6.317413933859448993e-02 1.935611719654204560e-02 6.230264444863053974e-02 3.409706374889712177e-02 -4.378852070954442760e-02 1.771358620700895850e-02 -1.440744951276977902e-02 -6.086191482296627459e-02 1.356530023409321419e-02 -2.357517879220547702e-02 -3.326652367744587641e-04 3.610957955364454269e-02 3.659592123475834818e-02 7.239840506665937148e-02 -1.233927961828722684e-02 2.970019156330577434e-02 3.705349443505417256e-02 -4.280247249205802153e-18 3.214061444026485720e-02 2.254254487291782252e-02 -2.953888212475446265e-02 6.075777134494438764e-02 -6.934839771572505474e-02 2.446024163760681519e-02 7.163377876641955975e-18 -2.247216712439875641e-02 1.515701172246556039e-01 1.081188463554965733e-13 5.054035427599749570e-04 -4.528977892574882606e-04 -1.461604411743476365e-03 -4.080745472589660276e-17 5.114081454099499852e-02 -5.593503868037632834e-17 -2.356503006664126525e-03 6.140185843640765797e-04 8.679598648597011343e-03 -2.180576878317727965e-02 -1.836823066560668291e-16 2.571565493408971281e-04 2.326795764283707873e-05 1.924623452210796198e-02 4.727335120513094513e-02 -2.363865002575884708e-02 1.553723555209662703e-02 -1.404585689796751351e-02 6.337198312957921520e-02 -6.909760017462095172e-03 2.078327272706851150e-02 6.628745950470853188e-02 -2.822960295869044334e-03 -2.066734890921367515e-02 -1.498815294554303908e-02 2.309816352677500351e-02 3.312370998326820415e-02 -1.070129324121850758e-02 1.804170648051069907e-04 -1.055848369812291106e-03 9.934122047677088063e-03 -2.665467786002740508e-02 -1.777614005156058366e-03 7.272256338399707420e-02 9.849246012322490640e-02 -8.789896568156757919e-03 6.338550381932371858e-01 4.242444035868301322e-01 5.582437136636399233e-02 1.037811442283024035e-02 1.056561572337330439e-02 1.223288336403563908e-01 6.291920033792625100e-02 2.775333011714685699e-03 3.253225370413628109e-02 2.696294549881554301e-02 4.230203731759270974e-02 1.448580021530291619e-02 2.631101684393699625e-02 1.336026272014990274e-02 3.933039358814349046e-02 6.099997702627660279e-02 1.638010090619395898e-01 7.080584289797002551e-02 6.712245739627729912e-02 5.876132139479863792e-01 6.905682776769579423e-01 6.370011023301437314e-01 2.685057724607334967e-02 5.718089481768248543e-03 2.668068414775163003e-02 3.553755703528176102e-02 6.485896922112803498e-02 1.016000375391468841e-02 1.851455649957197205e-02 6.068184058399345160e-03 2.918472554508332387e-02 7.689023768980617589e-03 1.360288092789774771e-02 2.850293373254066814e-03 2.144159974172578711e-02 1.250673262116096598e-02 5.380055521984923317e-01 9.841266117194362728e-01 5.909289686621607141e-01 1.273220448355466203e-02 4.640929728401223253e-02 3.558408533569156623e-02 5.824480702048497743e-02 1.434641439177225704e-02 4.396767488728006712e-02 5.195528778392695390e-02 5.413292547887173234e-02 1.399650271594991835e-02 2.792071313268520982e-02 1.129956541887723771e-02 2.952915623094551703e-02 -1.510869878369335570e-03 1.190902087045518296e-02 2.539975544678130942e-02 4.315555337445297485e-02 9.139333952415548690e-03 8.488734229780932339e-02 4.510859095388555229e-01 6.294344458484706362e-01 8.740318437393191919e-01
1.161706776597733748e-01 -6.304715044746812824e-02 7.518825564308315168e-02 4.253422611498056810e-02 4.835902261343130726e-02 5.583836985457495822e-02 4.379042693910102646e-02 1.778098520465096199e-02 3.356623306077782809e-02 1.723247243196451492e-02 7.931949997986996459e-02 1.798643163455019081e-02 5.874208427736386795e-03 3.324648277069398861e-02 2.171060413168897710e-02 4.050719094788168828e-02 8.618601512625490013e-02 3.905021658083725200e-02 1.890654972115728139e-02 6.090845438034574177e-02 9.195981807944322850e-02 5.419289157368444021e-02 4.439338851137723807e-02 6.065621267513299220e-02 5.176475990536320848e-02 3.411294904984753934e-02 -6.650835458968412661e-02 -3.272224481026142495e-02 -9.041023370209180787e-02 9.024422448745326314e-03 1.673485065811862113e-02 1.296102265073301521e-02 9.398352736072182814e-03 4.328456825012175387e-03 -3.102490621115189806e-01 -1.143139369668887129e-01 -6.097357588620803265e-02 2.790581234840778732e-02 2.201094709480921549e-02 7.879167834579540314e-03 4.053729932860850038e-02 3.897603301906762269e-02 1.589832409429289828e-02 1.289519243392900881e-02 1.393118596219219442e-02 4.273192157063740448e-02 5.559168317437257478e-02 2.132451035038033724e-02 3.869154625957551413e-02 6.694099468758918196e-03 -4.238302518566811755e-02 -1.283574060159546268e-02 -1.254562826018288009e-02 8.444671749883488066e-02 1.130805517181945191e-01 8.854065383094830710e-02 1.631591157416966142e-01 2.006188710919857010e-01 2.927961577088940004e-01 5.295543534471515434e-02 -6.740483548667189351e-02 4.058092881261622653e-02 3.637497246616848423e-04 -1.730136831658151200e-02 2.419880613352618989e-02 -2.071018899669845384e-02 -1.376996455380532017e-02 2.538580763558026879e-02 5.130090980563447450e-02 2.740170778709603502e-02 -6.371330495602148432e-03 1.445182219153500272e-02 8.402484593126658460e-06 1.274134999449240066e-03 -2.575775633719466426e-03 1.229425512074543841e-02 -9.532817299441590300e-03 3.269033477292036804e-03 -4.403662280668080992e-02 -3.564356225718365301e-02 -5.527440433057302909e-04 2.409958104765858727e-02 9.325578905057665263e-03 -1.003875530268025923e-02 -3.177365891782417580e-03 -9.168123345871415520e-04 1.543545235368454661e-03 6.262100122758330889e-03 1.017577243970835786e-02 1.461450987836741822e-02 2.470257574929047954e-03 -1.497702164064886737e-03 2.585336950558771163e-02 3.487274742793694537e-02 2.733335199510078528e-03 5.420812567657408249e-17 1.810082392450227187e-02 1.533763461162385676e-02 1.989328219014329172e-02 -1.571173604108695721e-02 1.245664874050990238e-02 2.694609894740818135e-02 2.459198840517862888e-03 -1.354737955602108956e-16 1.614369976800458448e-02 -8.185079396337287261e-03 1.358413376756638022e-02 2.226989478013979493e-02 -1.712160024669630298e-02 7.963318086358089562e-03 -5.428756325706865504e-02 -4.302037677899177986e-02 -9.593092293327460524e-03 -7.707082742364028405e-04 1.597620811972780453e-06 -3.603013770067609034e-02 -1.501925804897551175e-02 -6.933537564079060404e-03 3.648451497220314005e-02 -4.072826831678172388e-02 6.332986170212612231e-01 3.768811710380530755e-01 4.057773922910602044e-02 3.242880640948787169e-02 2.148651963170857340e-02 5.214665195997039704e-02 3.642022689721531592e-02 2.039896756327336594e-02 2.485956097570603263e-02 4.387746902220088230e-02 5.314262848850217352e-02 1.249045129682881117e-02 2.997332672036509896e-02 2.484934190615115460e-02 3.644024013128401712e-02 2.373055032888403068e-02 6.483897403111810176e-02 1.393014528204651950e-02 2.881910580520451876e-02 3.263319191453013801e-02 4.386366680385377081e-02 1.907897026232380569e-02 3.799878146855966560e-02 1.235005334347214108e-02 2.498995421521631177e-02 1.608942824515336573e-02 6.773963846710847614e-01 1.341638227811692063e+00 4.707799246211561428e-01 -7.791582015889175761e-02 1.875041228448019390e-02 1.651453382763643646e-02 1.753030215089774677e-02 -4.057010449601845814e-02 2.612366308514836866e-01 8.624666695947628137e-01 4.627891902050280115e-01 7.746965546687870942e-03 1.507105393866620818e-02 2.907490507828849186e-02 3.616844849371120968e-02 3.396521799555276416e-02 5.314223931632016534e-02 2.046488233036810114e-02 4.118330428671788251e-02 5.448329974514961455e-02 5.691092139116842724e-02 2.273751649599006774e-02 3.422429922907691685e-02 1.918520867981486286e-02 4.204597994321855881e-01 6.342374238414250076e-01 4.138189406723890573e-01 4.454203468380634086e-03 2.974931517247896007e-02 2.592491691055997041e-02 9.976662346482630117e-02 2.147358980229964118e-02 4.585567280774236615e-02 3.077770565475803147e-02
1.148510226414557328e-01 -7.630468856998408322e-02 8.234609498783132531e-02 4.631723660622710792e-02 3.569065275438436380e-02 4.469673782852831567e-02 1.583251962642448774e-01 1.042845858610160978e-02 1.764146239077361150e-02 3.846745425567017762e-02 7.118509047287917280e-02 1.292359986031013305e-02 1.137373307325143836e-02 1.408343028369181578e-02 2.658625515522393748e-02 2.108139134549246244e-02 8.456687000549491240e-02 3.458000429702719897e-02 9.329821228845009209e-03 5.064060138307482223e-02 3.878468902291428710e-01 6.268025047343341039e-02 3.272186523065722524e-02 2.547901386259231182e-02 4.899306538161486702e-02 4.344318542932055355e-02 5.803224636709489875e-02 1.131737989312209813e-01 6.066279144612749669e-02 6.176479255792212408e-02 9.075174135717531398e-02 2.832186067159568610e-02 -1.674666557170609131e-03 1.595004440648776362e-02 3.589122246035730129e-03 1.954617383415140856e-03 3.969748292751851570e-06 1.292911673654773691e-02 1.275816447156521809e-02 1.016910686851853554e-02 2.188542478449647874e-02 1.870677306443017593e-02 2.563179995552935728e-02 1.148579437239143075e-02 1.492923370636661633e-02 3.111577420838097102e-02 6.914525539228998097e-02 1.379359134698101425e-02 1.174809971235037977e-02 2.545898907247979778e-02 1.961359180464297691e-02 4.280500480163194378e-03 3.779206227355131047e-03 1.565484203306976088e-02 2.626569850350033222e-02 2.418930106595318721e-02 5.628829756068191859e-02 1.286142833066958140e-02 6.478871637365413161e-02 1.546743868891070281e-02 -5.336467179597939592e-02 2.887905372204090104e-02 -2.164816794546015749e-02 -7.177387058793408860e-03 1.806509305217131670e-02 -1.458289538762082629e-02 -7.068948816016568798e-03 1.009574260885908251e-02 1.510872826914613082e-02 1.750270610231899684e-02 -1.064625439281429407e-02 7.992378607505944491e-03 1.118881465787234172e-02 5.541464790395944293e-03 -6.175452663862478556e-04 2.019832221205802858e-02 -1.083070749604612623e-02 -5.935676984790930200e-03 -4.946880720715768603e-04 4.172669509140154026e-03 -7.838055281952535994e-03 1.688408443145195809e-02 2.674858823031598526e-02 -1.459255900353823195e-02 2.408119052434787646e-02 1.159578589772567239e-03 1.709199072884208873e-02 7.192946886938061946e-02 -4.025619467138469645e-03 -3.145507405264878070e-02 8.549483516074993239e-02 1.768365247713376068e-02 3.853307959525973519e-02 3.030184051089571812e-02 4.725695144486651705e-02 -5.364301386091629986e-02 1.824742854622332308e-02 -1.334185133852877278e-02 -5.172448123964471130e-03 -1.658908067614322385e-02 4.035414936704951640e-03 -3.308172692427912599e-03 3.304436112941415427e-02 2.016724045192296380e-02 1.734044696888261891e-02 1.784355548517767287e-03 1.522870765240147070e-02 -1.541008244986985694e-02 -8.479995301330929547e-03 5.489441122255634548e-02 2.215789360206624203e-03 -3.708737605203055784e-03 2.139596168336950835e-03 2.367644989185681975e-02 1.033793761215807440e-02 -2.476980044802958372e-02 -1.058349458357366393e-02 -6.295002835658347271e-03 5.483080096099719747e-02 -2.706388229762468275e-02 6.649294474184337389e-01 4.478033050257678171e-01 4.848785482394959989e-02 3.009712131372736105e-02 2.783333491264475268e-02 6.369540617583262232e-02 7.002030883824046648e-02 1.849428153535818270e-02 2.515952266204040874e-02 1.288894220246358281e-02 2.985130652867802603e-02 3.081681781015173285e-03 2.163031688953789139e-02 2.253412358944211283e-02 2.922682877358851600e-02 1.791141501698148911e-02 6.321294924306070095e-02 2.659955047296208228e-02 1.835043248809351685e-02 3.209239993525057899e-02 5.166952418862664348e-02 5.086779131889783423e-02 5.896007961568388439e-02 2.230138246952490313e-02 1.358844752576217861e-02 4.606264916588241887e-01 3.829290620558355451e-01 2.426574756866621829e-01 1.134055813599744653e-02 1.340359547527158850e-02 2.224833133698374266e-01 4.192503687480557129e-01 3.106710872169855531e-01 2.255683622833819756e-03 1.720568294343531243e-02 5.399322096669495127e-03 3.799871882087009576e-03 -1.046905676327094387e-02 9.715802413663838372e-03 1.528057409964884115e-02 4.577999177294560934e-02 3.000439418955131929e-02 4.308923321648020638e-02 9.340822184813460036e-03 3.038138000229753666e-02 4.295059492192940254e-02 4.924894643605000344e-02 1.742318132213139040e-02 2.671651435655882784e-02 3.615342355174051836e-02 3.147547842332149709e-02 1.322308256271710613e-17 1.259905514407938000e-02 1.636101805972836956e-02 4.648317070956822783e-02 1.999803005239756890e-02 9.197797368594896883e-02 1.651616638151072736e-01 3.772805060891138429e-01 7.274990326907561089e-01
1.249718730640017317e-01 -5.682167106206187368e-02 1.160983065208825299e-01 5.205500546790321065e-02 2.425426020510970671e-02 5.905602493065247038e-02 8.194840717607634883e-02 1.467852073945545392e-02 5.717052273129751006e-02 6.779610262102803153e-02 1.201073226568101088e-01 4.934154610886592324e-02 2.298881361658329167e-02 4.663385827778352560e-02 8.080127922480212499e-02 9.087346491623093936e-02 1.063248774723092177e-01 7.150073430604149405e-02 5.666890877200828416e-03 -4.835273699807420000e-02 -3.447442369782615507e-02 1.979166973756583611e-02 3.556283160122895859e-02 2.482545631743368966e-02 2.964200321945799355e-02 6.806743944603051155e-02 1.956540509144774898e-01 2.445554637998009415e-02 1.682881252393565730e-02 1.267218448458736109e-02 4.447323567628269463e-03 9.976459867600571269e-03 1.377349752947822298e-03 8.291763533632678860e-18 2.403818902254594326e-16 1.083889529733233048e-17 2.078440276335655895e-02 9.971972250966500140e-03 3.129062002164499645e-02 3.207795219319520968e-02 3.037660588851450252e-02 2.926994608681499077e-02 2.156717026078881772e-02 1.702947998991172912e-02 2.765022823429054075e-02 4.281604412785770114e-02 7.156873398117101492e-02 4.067363581430164486e-03 8.353823861038871951e-03 -1.121742750259114901e-17 -2.045916724529655076e-03 2.474777976425164111e-06 1.418945637488424304e-02 5.553596254134118837e-02 9.099954558936229621e-02 1.727626866108692705e-01 1.535193464955720011e-01 7.591529693097734399e-02 3.207978157941429598e-01 2.553835444383427133e-01 -5.980664205797651956e-02 1.018054652767365342e-02 1.059996186622449849e-01 -2.037054394721958261e-01 3.247290896463077975e-02 -3.076391035414598185e-02 -1.122824064777696576e-03 5.033197659399709853e-03 5.776133151676297872e-02 1.048071905400644634e-02 -2.850168351346804010e-04 2.847490255186088323e-02 4.555899580613142173e-03 7.084218670522642817e-04 4.357167656604740008e-02 2.226172099734429269e-03 -3.016385632409935950e-03 -2.996807742421632101e-02 -5.373638168623021127e-02 -4.907176240302912884e-02 -5.193860406142199876e-02 -4.614568285628296851e-02 3.901635368972449034e-03 3.468610289563055921e-02 2.366783239691698168e-02 -1.134464791385088669e-02 -5.629906138668358106e-04 3.561972551652199236e-03 3.017386433266841181e-02 2.785846374383314436e-02 5.760155900811639133e-03 3.309314556934714324e-02 -4.407479342588743049e-02 2.316691154844989758e-03 7.589069354549469147e-03 4.606884683922371493e-02 -1.819010310867003391e-02 5.138830374254756833e-02 2.739148098502181637e-05 -1.459582035993516394e-03 1.385370329181986925e-03 3.914900582953499686e-02 2.053740818124923789e-03 -5.120414223789450381e-02 1.831208063845067222e-02 -3.800787303595891398e-03 2.134299092776143977e-02 -4.947391646789348844e-04 -4.653577737165085632e-02 1.097383436117261440e-02 -1.936715075858873661e-02 -1.494365852953802656e-02 4.595517714332700288e-02 -1.458901545201016572e-02 -2.172430436252147795e-02 -3.637902851271970078e-02 -4.132122905304861683e-03 -2.430849103484326867e-02 -1.296150790349977731e-02 4.534224649194541767e-02 6.272202128528757692e-01 4.124554070111970949e-01 4.644260778078680707e-02 1.653272032148587686e-02 2.568330465193795209e-02 3.472559989222217397e-02 6.712428090445823892e-02 8.723179132930265062e-03 1.591262413543964235e-02 2.839752306027744019e-02 2.799022324815566426e-02 1.642993520691694506e-02 3.124366405463928006e-02 1.692044746428248564e-02 4.041439024679761155e-02 2.038483904849319739e-02 4.124116110316608519e-02 3.240362343156145313e-04 4.718074622904231963e-01 7.505796365596585362e-01 6.257119661013145206e-01 1.305374606652095787e-04 3.787702176530147180e-02 1.607618354196089019e-02 2.130483878682944837e-02 2.860078329153134188e-02 4.549918753991357562e-02 1.545863480472009098e-04 1.118746967424220917e-02 4.701810471823258258e-03 1.426072267655487981e-02 1.211588008002762286e-03 2.002599589424210874e-02 7.211701163599992803e-01 5.137704682949422175e-01 6.846654238135688741e-01 6.952779433525149884e-03 4.630564273383762017e-03 2.705220873059792364e-02 2.174781313095783958e-02 4.353477988148954370e-02 4.578814062693171982e-02 5.090160417331429654e-02 1.785984835529420139e-02 4.299353733251543019e-02 5.613927233857224364e-02 6.773568044930386445e-02 2.593892424884588696e-02 1.547581039647011672e-02 3.835548080486176903e-01 4.567465971464464425e-01 4.983089227006746169e-01 2.890337292494423414e-03 -3.105583308803170142e-02 3.992958224987949029e-02 2.190776552568760374e-02 9.170834380044272371e-02 1.566189778556986434e-01 4.887725829525683269e-02 8.150413367486061743e-18
1.283456090508511838e-01 -7.554332345263979620e-02 6.924391190211057201e-02 5.710708111151275440e-02 3.550550382135728850e-02 3.233230004764402893e-02 8.506662703899987854e-02 1.040375619688934695e-02 2.816817166140460349e-02 3.619858151299257948e-02 9.148786755221033351e-02 3.781984624885517676e-02 1.544048617958710540e-02 2.119741565232026101e-02 3.784516209690802491e-02 5.751935103734553273e-02 9.894926846782481356e-02 2.916181110399049259e-02 1.351857606963260683e-01 4.822862072331370736e-02 2.378439610835536844e-01 2.919469064918812917e-02 3.681884285506096394e-02 2.531348325565961968e-02 3.631640219558682214e-02 4.922720410996794110e-02 8.456539284495805109e-02 3.946508633384395587e-02 3.421521216823800518e-03 2.311839536813975211e-03 8.148986691512129232e-03 7.225604233140335217e-03 4.446855845469124499e-04 4.935113054536228484e-03 1.689433069465357679e-02 9.840321635858930577e-03 4.436201078172640529e-04 4.458470877066585125e-03 -1.354548997432019654e-02 -5.456738804043523988e-02 -3.434368880841351390e-02 5.089352013684998038e-02 2.952379240179483708e-02 1.660122923449349613e-02 3.547631556370249573e-02 6.692112775288322490e-02 8.241391353364212191e-02 3.038635249754042741e-02 3.352859487246144521e-02 4.009995952156103494e-02 4.976074294626332134e-02 1.234845668766310946e-02 3.553839553990933466e-03 1.139692541481431357e-02 3.569889847940520972e-02 1.150506790504146094e-02 1.015855661516464337e-01 1.351293856255911426e-02 3.950779001027219378e-02 3.378715438041190317e-02 -5.322208115521220168e-02 2.909346826565916214e-02 -1.970249229755954151e-02 -2.201361478401058641e-02 4.628735440501031279e-02 -1.790158263655269172e-02 -1.234019489849060448e-04 -3.540447074277001602e-04 2.685910669428609801e-02 2.035757530584525552e-03 6.055995620014208375e-04 1.262581271064985161e-02 3.514017710901403205e-03 -4.006151312702911640e-18 1.309172891385669384e-02 8.632710273852145511e-02 -4.155987752735617637e-03 1.393489063643694532e-17 -2.163614490510907054e-02 5.673575137378553991e-03 -5.345380082434296565e-18 1.346482401369308683e-02 2.553496628444495986e-02 -1.803481763797881721e-02 1.342065159290688377e-02 9.596842204908874116e-03 3.304910083803788701e-02 1.630902046620975182e-02 3.549130001626577685e-02 -1.240489564543059781e-17 1.762735358637355226e-02 7.034769930899439916e-02 2.146654069227167325e-02 7.406573969952188509e-03 -1.492801879991204452e-03 2.586780362640260164e-02 6.884118089606150459e-02 1.349777430394940268e-02 -6.320044282179469730e-02 -1.986602606096264656e-02 -7.232910722158904238e-03 -7.887762348994420414e-02 5.254793981769579486e-02 -6.287806778527141142e-03 1.342626888938930688e-03 -6.543426413904018454e-03 2.085412639690250799e-02 -6.412129278904465925e-18 -2.302083051155767723e-02 2.847257972892856814e-02 7.398419863701054255e-02 -3.658427532581225161e-02 1.207692192966389297e-18 2.119628120195525892e-02 1.370379273823176501e-02 -4.120180564327345640e-02 -1.042719134269548065e-03 8.364978228363313673e-02 1.062548341026950255e-01 -5.766720262565154867e-03 6.202752173185988038e-01 4.026051518933321249e-01 4.119145336645631261e-02 2.879975468281613035e-02 2.057395713894490455e-02 5.348819784737271488e-02 4.912517509681030292e-02 2.236911069782280062e-02 2.246737563249662922e-02 2.762561972008649794e-02 3.223127133933336580e-02 1.458268479743459779e-02 1.684392245422820442e-02 1.133875634572484477e-02 2.468087960083623078e-02 1.962939065542091896e-02 6.462240696718747623e-02 2.305270308490553824e-02 2.296068506441826043e-02 1.783463382151249596e-02 6.248121567030938406e-02 3.299863692861491893e-02 5.517379633772644121e-02 1.249448919176643011e-02 8.649333029560810765e-03 6.782791818639496428e-01 7.418033788683576768e-01 7.010479555365171089e-01 1.953879441633793457e-02 6.138913877630799813e-03 2.345759961816454645e-02 1.403175457226041138e-02 1.872329555133056198e-02 3.828344896954124360e-02 2.349341417933033943e-02 1.411071132170182985e-02 7.995814428697848447e-03 5.251398504009094012e-03 6.960970419067311621e-01 1.085476913699121138e+00 4.919702835311328126e-01 7.701634369795398900e-03 3.513336313839841851e-02 5.646484043467259531e-03 3.710821484411684634e-02 1.898870921068831000e-02 5.889748721134106801e-02 1.318504884616216073e-02 1.749743566680316692e-02 2.103409750505143630e-02 3.378385641496329389e-02 -6.785079773229384667e-04 1.972107138280156102e-02 1.459284517981931754e-02 5.255692379175729706e-02 7.282992121586560549e-03 7.626507863241949181e-02 4.366054448082330453e-01 6.407955260398906505e-01 9.199871463242463854e-01
9.505906093646800115e-03 3.026215360125763740e-02 1.000516088267417586e-01 4.771415397922838808e-02 4.213849262732943801e-02 5.484275640322516104e-02 1.784117794459378348e-01 3.850751611815469294e-02 5.858127673945028729e-02 7.985707363928201685e-02 1.284298822587763289e-01 5.990939731119904882e-02 2.579180149654720575e-02 5.069700449008651341e-03 4.708091649317946298e-02 1.826854345626653664e-02 9.000174721034381875e-02 1.508666069676323739e-02 1.863559960825956650e-03 5.462385683036569128e-02 3.953549084883964393e-01 6.828640846357446337e-02 4.656302977961126327e-02 4.774249906926411602e-02 4.693002465145604957e-02 4.841497949886141433e-02 9.576830147725169662e-02 5.246936284955641683e-02 4.146754665183819593e-04 1.782078969602486629e-02 1.931743828495187257e-03 -1.218232356862039394e-02 2.846553934741760054e-03 2.337062782780155955e-04 5.738075966432761976e-04 3.945762024938344152e-03 2.259408235048148718e-04 4.876026741328254310e-03 1.604596473045344016e-02 4.220450137319967619e-03 2.433296260747802239e-02 2.268909916882767483e-02 3.055017061074824861e-02 1.838050720679977254e-02 2.603444110280467988e-02 4.125342110478354141e-02 -7.273015095301132993e-02 -5.359299141608150213e-03 -1.881112454840607254e-02 4.449686052420877974e-02 8.720183322852229080e-02 2.026827954486552785e-02 1.144612113786416113e-02 3.775373755458561603e-02 4.180955945542687291e-02 3.957506250833039974e-02 7.243182943430495624e-02 6.147665920216880547e-02 1.355908036225514923e-01 8.659973043765448186e-02 -4.127950694144789934e-02 2.871044596161493115e-02 3.397498184868102261e-02 -1.151188549063801407e-03 -1.501164940771084226e-03 -7.461522000576215237e-03 -1.141805216977161792e-03 8.518798405825632816e-03 -2.797263295212942600e-18 5.892522783173687745e-02 -3.226878874949325876e-02 2.968146700462506032e-02 -2.154451521029381630e-04 1.553183028359843808e-02 1.408645248127439117e-02 1.146280139076124444e-01 -9.177643905908855395e-04 -4.971861094153203164e-06 -3.362728924003766454e-03 1.995350624240640330e-02 -7.247827045323266810e-18 -9.499292212834612185e-05 1.888478522405498752e-02 -4.458300646471215906e-02 -1.817768595537738383e-02 -1.682877329845015521e-02 9.485697502537841774e-02 -1.549052601454460869e-04 3.472043691302410806e-02 3.011597687682119120e-02 3.955229874177214333e-02 1.985988752775937061e-02 -2.491450719510946571e-02 3.900696634051509237e-02 -2.212575257138036159e-17 -8.646884655974048544e-18 6.487913801053475649e-02 9.590549323780185251e-04 9.548982620732344995e-03 -3.726043081704355259e-04 1.664239611214893277e-02 2.465117906295285696e-02 1.724077961194286218e-02 3.750020674787964137e-02 2.996179332482377944e-02 -5.238460822131531594e-03 -2.834687443057838478e-03 -5.055586032625918394e-03 -2.327987806381246177e-03 6.418983676968745451e-03 -1.030482034102561152e-02 -5.646478967722529628e-03 2.200152009594854435e-02 -2.698952926127183224e-02 2.078086082915651250e-02 -1.084559047184679992e-02 -1.758239494021013967e-03 3.992789903237395528e-04 -6.366816520717165500e-15 -3.959782299674795975e-02 5.239797865688784057e-02 8.850973710754282686e-02 7.289588944346499233e-02 5.355363223234183012e-02 3.710817743327211177e-02 4.522034638182913929e-02 7.031061236869289022e-02 1.980520674211628043e-02 1.803060677668229944e-02 1.057837672440055214e-02 1.831129078959028642e-02 2.712592186342454702e-03 7.893079596901823869e-03 8.428534292102509706e-01 6.692691823218864799e-01 8.593790564608819738e-01 2.593854743514211261e-02 5.122711913989119042e-02 2.427492386872393687e-02 3.760438536654171077e-02 4.330363927609049562e-02 2.565709217345515589e-02 4.897083355827432094e-02 2.939820429130998031e-02 1.404539659715210893e-02 3.232106546353135518e-02 3.414702462525258386e-02 1.908962723298041411e-02 1.337684143844713736e-01 -2.642780328290405545e-01 -9.574760529025064248e-02 1.337168160066815603e-01 2.449569373277158887e-02 1.908896713332293091e-02 2.026692002364195722e-02 2.216775552820834504e-03 9.680920399421446415e-03 1.468519001676606671e-02 1.966472643132815765e-02 1.743181498161622614e-02 5.413647108044363637e-02 4.051209187771240977e-02 4.816909809857052721e-02 6.810015435613652780e-03 2.169737951545829685e-02 7.422358252027799665e-04 5.002910909668795147e-01 8.023033659308863585e-01 4.019373647323832288e-01 1.644622289098841145e-02 1.841237606688991141e-02 -1.537667956163020518e-06 8.242593044191275087e-03 3.923955318039759610e-02 6.111504984180883560e-02 1.637075196698758658e-02 1.018542956355987378e-01 8.597909847487654467e-02 9.757465704134840601e-02 4.161303304459511299e-02
1.070152788109533792e-01 -8.615523807661676359e-02 5.445128996710524305e-02 4.661484498516985792e-02 4.608067653385816004e-02 2.593427936249818530e-02 1.281800049640726136e-01 4.847126567912941886e-02 5.940067574962894920e-02 7.840894286677457803e-02 1.649741087046883992e-01 5.074394179384145165e-02 2.225730529741306446e-02 5.496284553091781988e-02 7.211839608829630210e-02 1.525576799955142594e-02 7.132824903994050558e-02 -7.891075356166872556e-02 3.861093435718038203e-02 -2.301901683236361185e-02 2.621935772657918787e-02 1.688487562217854884e-02 2.160282350753459785e-02 7.622360517569700539e-03 1.719851008829732103e-02 1.806817295281629848e-02 5.952055710552708184e-02 1.722524869277002740e-02 1.602821706220137244e-18 1.564672563843072566e-06 1.238861534975321376e-03 6.398119686520143973e-04 4.819956355246606027e-04 4.046025450123910336e-03 3.516485071806084266e-03 2.607243173362073368e-03 -3.274385701285223438e-02 2.353928986093449603e-05 1.824985897915662111e-02 1.463561441659527035e-02 -1.074327831598988550e-02 -5.987951414537211886e-03 -4.606800713007790787e-02 3.567406284640485190e-03 2.220541282802014280e-02 3.174551265873715861e-02 4.849291561118777849e-02 2.384644929218311671e-02 3.097640325079721937e-02 2.858506723140358599e-02 3.102456487145033998e-02 2.663352483678216309e-03 5.677525249145820400e-03 2.246432114913310060e-02 4.554711711916819422e-02 4.051643633483250490e-02 1.148337876718972944e-01 1.709932595897470375e-02 7.267069295423697617e-02 3.113245468105311359e-02 -4.109373503139076428e-02 1.080226895360811290e-02 1.033933726361137154e-03 -1.773148026750593145e-02 2.230868564879946378e-02 -2.897733018942202574e-02 1.579731809588109026e-03 1.030015152425466096e-02 2.338492656509984571e-02 1.673796161102044516e-02 -8.356908469473772055e-03 1.289947767489705611e-02 3.837099573791608043e-02 6.758136188688657064e-02 1.619952230237987553e-02 4.907296223074992386e-02 -9.208923302578088388e-03 1.389931849190049473e-01 7.386429377489160675e-02 5.650436081923183168e-02 9.506365221574149260e-02 2.656786708777850170e-03 3.689280520798945973e-02 -5.701650960538441268e-03 5.183548411491083023e-03 -1.465175019751516973e-17 3.031337971182775440e-02 4.548349670112315779e-02 2.192587969766646942e-03 -1.450007854863285227e-02 4.216499270566145218e-18 4.926843266498564730e-03 5.800505710669181560e-03 2.569315856411463414e-02 -2.203240435152173047e-15 -3.502398640136258207e-02 -3.607640797611592898e-02 -7.456293807935246432e-02 -7.388476822783030776e-05 -2.715121931497649702e-02 -4.445131386396342608e-03 -2.133644337503318167e-02 -3.049737569581797633e-02 5.661836716177453014e-04 -4.328499814837181991e-02 -1.214478461770203695e-17 -1.887259970014501520e-02 -1.038950177352361559e-02 -3.476758274005506584e-03 4.185048532719734538e-02 -1.358311461658639869e-16 -4.956891986915823445e-02 4.657423937978221514e-04 3.492109033767580584e-02 3.269688930206480354e-02 -2.850000648551757193e-02 -2.384167290660866440e-03 9.265689824278756925e-02 1.403264364109998164e-01 -1.161044842789860955e-02 6.349374595099853691e-01 3.849021154482976104e-01 3.235545614763826738e-02 4.882208295047034408e-02 2.462637382947676853e-02 1.270642801849410086e-01 4.071616032666416224e-02 9.228774274002952624e-03 2.456868591905226468e-02 3.737835129423418973e-02 4.617347612062350920e-02 1.522117445599258795e-02 1.909606489249223576e-02 3.183684125483968869e-02 3.396913507177311969e-02 2.625000166231517690e-02 3.498305457670056878e-02 7.750156986788981905e-01 6.863169372934343482e-01 5.909343514478402337e-01 7.599711814813463542e-03 1.627701196675322703e-02 3.908533066727323119e-02 1.884075582688636910e-02 4.155328582823006323e-02 4.912347494189122504e-02 6.409427847408950829e-02 2.711633615294210126e-02 1.010125837752811652e-02 4.374851059229545661e-03 3.005171789926415135e-02 1.920083539690057781e-02 1.665458025258106239e-02 9.150656260824906012e-04 1.004224332474154026e-02 8.883444412203931250e-04 3.579645494585746279e-03 8.502566856138387211e-03 1.417058311653106827e-02 2.980017494190188073e-03 5.286684866174669040e-01 1.116322089866458755e+00 7.113786124676426681e-01 2.824236627649339120e-03 1.897968806204423584e-02 5.117797737737393021e-02 6.186972949128170113e-02 1.542879346167620042e-02 2.451448284477588183e-02 2.620563611723456671e-02 3.607309561748704707e-02 1.463328756312224331e-02 6.314105914322964107e-03 6.076652987085693659e-03 3.842973479610607007e-02 1.561613884802751269e-02 6.301426412955055123e-02 4.031237422677398929e-01 6.008588223880686474e-01 9.070904054049222509e-01
1.507930806225379616e-01 -7.210633368292816703e-02 7.654433841786863202e-02 1.873240141801685646e-02 2.727192639674356697e-02 2.291054131486645190e-02 1.157308395518731603e-01 1.607266580416844554e-06 3.246470588405089841e-02 2.547221837254936769e-02 1.005863549510961907e-01 4.297961090770420489e-02 2.544887722978324621e-02 2.763551707963056436e-02 3.184876546124445346e-02 4.924725424104070548e-02 9.713872593820022106e-02 4.078105699916734217e-02 1.009133787726064664e-02 4.381351222543855561e-02 1.017259192437033749e-01 2.923988750345558191e-02 7.112982776557666420e-02 1.960182198187724045e-01 3.879879601124094912e-02 5.082994834447329691e-02 1.542106255749866872e-01 1.415114714592677347e-02 3.390575236218943984e-03 3.349122171763516988e-04 5.716154209341169599e-03 1.681192235631748591e-02 1.262598691069894141e-01 3.516114151684757855e-02 1.733001226474009671e-02 1.059485960474707483e-04 2.529551874088157930e-06 1.031006666759796205e-15 6.373366019447115026e-03 2.060155538835165001e-02 2.812005017084356559e-02 2.375763126251529203e-02 1.639526350268013036e-02 6.816767025180842706e-04 1.353145404745889277e-02 3.968856777592636581e-02 8.944387482527152433e-02 8.659336350850093886e-03 3.050819025067695589e-02 4.148556737544502326e-02 2.614512701771773071e-02 6.731886153626788667e-03 1.212650244264198464e-02 1.272028052019943435e-02 4.046613232503189911e-02 2.152930181863790121e-02 5.845054401291329554e-02 1.304700879761739110e-02 6.821941850683752029e-02 8.605788475093494039e-02 -5.312865424799143033e-02 6.824753956086960713e-03 -7.128846042040054912e-03 -7.559751557223355969e-03 1.395702971437332365e-02 9.855523317943972676e-05 -3.549957399323444389e-04 3.838886917253984260e-03 3.248822124818307594e-02 -8.743577784068057865e-04 -1.509628179373276834e-02 8.374517757550491159e-02 -4.904825771239872123e-03 2.325183825840875310e-02 3.565998247663756948e-02 -2.156749281022313344e-04 -3.584014027453157752e-03 2.142295588149205343e-03 -2.403994262416348990e-02 4.310223422596111015e-18 -7.646583310927534786e-03 -1.214681871777888167e-03 1.863890301042026021e-02 3.806041336711485368e-02 4.896082556360729920e-02 -1.597516766050337544e-01 7.855096377565190524e-02 -8.464012147262542363e-02 6.610131154842142920e-02 5.996822073147786702e-02 -5.105582458080416830e-02 8.179205788915146214e-02 5.801614132411533897e-02 -3.459791541082553204e-05 2.012482338678201269e-02 3.365546865393924392e-02 1.513840265952625871e-02 -6.179035567259649753e-02 5.568879144622922015e-18 -5.329179640422669617e-03 7.160973074536667584e-04 2.530231101634472946e-03 5.677372829745008787e-03 4.005910903005033968e-02 -5.756075888808555438e-04 4.214373196116899187e-03 -1.605684800694595496e-03 1.034830238609625759e-02 -5.562083336126595959e-03 3.778983683146149714e-02 2.826479936063195503e-02 -4.010622761836128465e-18 1.355608537439467829e-03 4.041244914155614487e-02 2.524705545558368625e-02 -2.755488452294026572e-02 -1.054161339791954283e-02 7.315626692253314467e-02 9.268873160785882914e-02 -1.097301763218102438e-02 6.314832643135662726e-01 3.971269746724808147e-01 5.172188975606913791e-02 4.407657629225155554e-02 1.692823390604354314e-02 2.066092968754059955e-01 7.479862619604740748e-02 3.466330945936860197e-02 2.361166730300836430e-02 2.756821691420778542e-02 3.617523058655333634e-02 1.724823916671133425e-02 3.312636522910209630e-02 2.612267435391270320e-02 2.546787804900221830e-02 2.098590889751126426e-02 8.244374980128602037e-02 6.151564529454334879e-02 3.446031194366600686e-02 4.939168613742641462e-02 2.276322148388346817e-02 3.708364132537568612e-03 5.633312009045459146e-01 1.043521947481975465e+00 7.062502994567176895e-01 4.727559806358990098e-03 1.662647040213794877e-02 3.123155262739775537e-03 5.445806354532535358e-03 7.104487377415253171e-03 1.766235958851111731e-02 8.098728650494763093e-01 7.739205074383259664e-01 6.746478513712144132e-01 1.470440403628006480e-02 5.565224093992519622e-04 8.158287111088981941e-03 5.057828463208886719e-04 2.880027842755419204e-02 2.384931650564077812e-02 3.865575159716965609e-02 2.438513553018590782e-02 3.734898925383022678e-02 6.806583003037904728e-03 2.819827220061311285e-02 5.632886123872887785e-02 5.719497409870642757e-02 2.137767798023965798e-02 2.709773831267719632e-02 1.591655268832042891e-02 2.768430731501557787e-02 1.547753555409621569e-02 6.770884589371067158e-03 9.837635936893254773e-03 5.475314596822329627e-02 2.086836064640892538e-02 9.855961565734508434e-02 4.169688602321313109e-01 6.023889380314390030e-01 8.244876399315441695e-01
5.325822237849896917e-03 1.946611819864649609e-02 1.062445556933990370e-01 3.116910528286298554e-02 3.719086795083686309e-02 6.862111497544447214e-02 1.484774024886700294e-01 2.066162778763449420e-02 3.410002377381435301e-02 5.736004904730798715e-02 8.955217931192557823e-02 8.964677171686068896e-03 7.658192144624415172e-03 1.361974098657390067e-02 2.332000966185426849e-02 3.769490746771635148e-02 -7.292149177982637509e-02 4.934827215702333736e-03 6.019859829255477454e-03 3.668594192205562599e-02 1.395402542744161034e-01 2.319465574124703403e-02 3.393031331208910123e-02 2.505363554000197140e-02 3.554916698126940611e-02 3.888727007503264832e-02 1.039922736322543978e-01 5.042610489186550443e-02 1.043129272373510092e-03 1.974466103621602686e-02 4.156158017016366168e-02 7.283237806745164467e-02 -6.254701379368794723e-03 5.449612785766785351e-03 6.183404190375616297e-03 4.002855321654428096e-05 -4.041801186762165876e-02 -9.090961606079148508e-03 2.523447199272756472e-02 3.534805657662529188e-02 2.787395443603580061e-02 2.695506779218278734e-02 2.935703429251720550e-02 4.682211356748504361e-03 2.576389988957669772e-03 6.698819821510065831e-02 1.306909672852177165e-03 9.833685921586575451e-02 1.490874552376715567e-02 5.854713794562944884e-02 3.934103640322495626e-02 1.914072925057239962e-03 2.231851521306439830e-03 1.149322223015947393e-02 5.369989765353407746e-02 3.080706144979778946e-02 8.221115475507612125e-02 5.100310508996757503e-02 6.464464382424527089e-02 4.075910512751683423e-02 -3.378887655160828923e-02 2.546019433251468789e-02 5.748784948011082668e-02 -2.378580349918541642e-02 -3.623715157260220046e-03 -2.449724313043027610e-03 -4.273557620584375169e-17 -9.897617828850901155e-04 -9.666159534648320313e-04 -1.652272210517109752e-05 6.310874653704286186e-04 5.792319421366150939e-02 2.228133119813433538e-01 3.288302013264231619e-03 -4.488475042448337310e-02 -2.305952314899228570e-02 -5.283185388096053869e-02 3.584040315347942346e-02 -3.254592345267418030e-02 -2.163139816906198623e-02 4.524829562083661585e-04 -4.295496847747706445e-03 2.419215713155997011e-02 -7.002559255345212896e-02 2.274568395315414160e-02 -1.608527827057875184e-02 1.439274177459367796e-02 4.311533226119339604e-03 -3.772518215536049824e-04 -2.778368682278308313e-02 6.674588054995095332e-02 3.964826451639705263e-02 1.682880629390627880e-01 1.270163066687980268e-04 3.718138163180795264e-02 1.220140058884570543e-05 4.132893943112205676e-02 3.823260012309744136e-03 1.426185005507216684e-02 -1.414561131514410937e-02 2.218177556391358329e-02 4.739542950874830324e-02 6.853338141382530301e-03 2.881448898566561431e-02 3.002112138309821765e-02 1.461816022213282412e-02 2.750733003399191690e-02 8.115386134243094546e-03 -8.548587788052383618e-03 1.320583172081412140e-02 -1.315898741079611091e-17 1.661355472968224778e-02 4.435131350793738818e-03 9.617919334716366550e-03 1.125035523896605189e-02 -2.760845738321739823e-02 4.360641875351864427e-02 -1.183547071205962031e-02 2.128355720439955093e-02 2.059496296110470762e-02 5.694281043026355738e-02 6.409570788309337797e-02 7.647836050613647518e-02 6.688348213137952636e-02 2.279142604774430855e-02 6.013967542238184832e-02 5.307855121933115899e-02 6.282133298107466304e-03 1.309815076053098826e-02 1.425213142580767159e-02 2.488477883261194096e-02 8.894207018088365335e-03 1.528168170229260436e-02 1.902454597928433244e-01 -1.504335271795712026e-01 -1.842375955354249584e-01 1.118254052106029822e-01 5.780033170354394179e-03 1.568562561332612071e-02 3.941882585759749758e-02 3.811395111910489270e-02 3.322316830462922893e-02 3.594148495579849167e-02 3.068403828192904637e-02 1.945134128474132126e-02 3.176221460338243124e-02 4.794258047031479952e-02 1.743556728662181213e-02 1.621398608183996157e-02 4.321499955807424007e-03 4.772835868032005435e-01 8.313318308393073952e-01 5.382495448044362085e-01 2.491541724696405211e-03 1.849736194153786886e-02 3.392902349082040775e-03 7.157540881105267302e-03 4.629830372317199350e-03 1.489873101156426183e-02 3.513691145951265693e-02 3.928803193015104750e-02 1.890633167654582822e-02 4.955845378697314174e-02 2.843535504362473994e-03 2.836426713764832533e-02 6.922314253775978532e-01 6.971760012215066915e-01 7.383609107251173365e-01 2.975235036735966385e-02 1.735615103308317227e-03 2.817142041198865757e-02 -4.147154525557221884e-04 1.227628301437183600e-02 8.886920900234572193e-03 4.670561871259171910e-02 1.943607067803614433e-02 8.969419964667567358e-02 4.350539367759428955e-02 6.092845341872068360e-02 3.044252805047286195e-02
2.578267188723660785e-01 1.992451103661222606e-02 8.055886815710880736e-03 7.974391793757217017e-03 8.178961253253752171e-05 3.350151785645668094e-05 1.924189300472299363e-03 1.233212625850288978e-02 1.088444043476607254e-02 3.214951282887185645e-03 4.068209968186878994e-02 1.781749026765946453e-03 1.653763239522372183e-02 3.146377961074661722e-02 3.759981389349389275e-02 7.731510036457476392e-03 3.787577217153868542e-03 1.566196101484376713e-02 2.364201589473252652e-02 3.398108433932981592e-02 9.963261062935639678e-02 1.485135741274698842e-02 3.186990182676809752e-02 -2.450945048038715920e-02 2.548967908888079628e-02 1.915525663771324230e-02 3.821438026305859492e-02 2.694960323756496329e-02 6.604343595532027877e-02 6.899605143682864539e-02 9.777504086974964093e-02 8.224643256550789872e-02 1.361384239092776893e-01 1.138741784164227222e-01 8.075085731582348769e-02 5.893571274749328549e-03 3.498674630015427642e-03 3.430010789766727020e-02 -5.983688807843393076e-02 1.968978178128081864e-03 8.519199537934549205e-03 2.151182311272411760e-02 1.802367379851625012e-02 3.237530520466335576e-03 1.231536753697892728e-02 1.819168791383501266e-02 4.456864492733613597e-02 1.586796316454713515e-02 8.767075501698719936e-03 1.377787680954193142e-02 4.027039180702253474e-02 6.194117061484866529e-03 1.339621833886696280e-03 1.480640550740117481e-02 3.190261071042189284e-02 -2.891904360070566508e-03 -2.698772670867709972e-02 -4.272664272145283715e-02 5.502032037767237149e-01 1.101331955632738058e-01 -2.004927230875084010e-01 5.124959642644005831e-02 -5.459388311397805893e-02 -3.484475762443933457e-17 2.521993943546562561e-02 1.814451322815817044e-02 -4.713276518572738494e-02 -1.019765562155530020e-02 5.415147725350452808e-03 1.452880232826901132e-02 -1.096222627183020271e-02 2.909951295612417388e-02 -4.618934421401004256e-03 -1.945883128585785063e-02 2.776199244822718029e-02 1.423180190456437358e-02 1.903629292317191067e-03 3.360420220999733171e-02 -5.267773913791200593e-04 -1.929158366414925139e-04 -2.281991650725653190e-03 9.586513713586890251e-02 3.382307682366983426e-04 6.936629387731171534e-03 -5.969746174320489374e-02 7.094913890221203991e-05 -1.551251299734208826e-03 6.300665249936746211e-03 3.708746630468112704e-02 -9.780150744289173304e-03 4.090605227382387160e-02 -1.696052009075977793e-02 -9.403827416123140630e-05 1.035507482397795068e-03 -7.326227620235492566e-03 -4.403481038784133779e-03 2.909205930511315997e-04 2.971098438441272579e-03 2.368506903909278066e-02 -2.902146555737581798e-18 -1.133234082452550540e-17 2.521716816263931682e-02 -2.510604672705228635e-02 -3.200437797287749696e-03 6.251977574908125501e-02 -9.746450047070929093e-18 -1.615102971817541921e-02 -4.072434122384090214e-05 -6.442288267707352821e-03 -3.190502347322282010e-17 -5.747809048548979498e-03 -1.076638399084635725e-02 3.031677018350066892e-04 -2.165233964572076120e-02 1.802257313625516000e-02 3.137602897845143818e-02 1.900292769066682513e-02 6.499461772260529957e-02 1.705313194410864805e-02 1.797473724215892243e-01 5.363356461201832515e-01 2.832598302258546585e-01 2.068271539920625568e-02 1.486605599935104417e-04 6.714643335133570375e-03 1.187934284332911030e-03 2.536450380180774872e-02 9.965881149906994155e-03 1.482690895202365931e-02 8.314522617175469990e-03 3.120101610258147959e-02 4.048910280747280593e-17 3.319274852959381433e-02 2.970722686420102648e-02 3.273992745366184620e-02 1.234301752076469652e-02 1.911051603940637847e-02 1.201298608294444105e-02 3.226692089553877957e-02 3.831962666273567436e-02 8.349513332695304713e-02 1.560754658917341310e-02 3.167676055702016713e-01 5.634515784820622875e-01 4.298962590303919140e-01 8.867459618770319116e-03 3.028760116965810434e-02 1.377988063557017417e-02 3.339547582007588999e-02 2.840170657715793828e-02 6.628603583291606260e-02 5.657449480727446550e-02 7.699168409614622066e-02 5.141298954926414727e-02 3.290620538747159041e-02 1.415850622104049619e-02 1.410516745317244233e-02 5.297041997537876856e-01 6.374739829634056276e-01 6.824287179694427108e-01 8.049041607587070768e-03 3.757149643607601841e-02 5.064688739829462105e-02 1.155232167079528471e-02 3.869700650921335111e-02 4.837799410683817081e-02 4.862195629516880696e-02 1.914561702130882989e-02 2.485499461117348860e-02 1.148334405991639079e-02 3.253176602634961362e-02 -1.371213692454901524e-02 6.252143114125946477e-03 1.818497359155534442e-04 3.806514707660247493e-02 2.193774425609701373e-01 3.179423989638923143e-01 5.267529582228394069e-01 -3.777687870924993918e-01 -1.113572712116619678e+00
1.492814709642154569e-01 -6.364510249333306391e-02 9.911978813313056247e-02 6.427428212862144885e-02 4.175094630317187361e-02 1.039667702336797006e-01 1.023283298877071396e-01 1.841860861365385987e-02 4.425147285911691653e-02 2.950249436739408107e-02 6.971404205046534408e-02 3.557498849898774185e-02 1.498535014416973836e-02 3.119935348987837309e-02 3.272567451049706450e-02 5.938839969402150143e-02 1.222086383983466834e-01 4.933279314618323891e-02 9.564993423553441385e-03 3.785311393243774580e-02 2.475446041446124901e-01 4.628201889979389333e-02 2.169284037512438784e-02 3.732896275883022114e-02 3.874081576450012082e-02 4.465566810208979054e-02 1.061308999842156681e-01 -1.677067209925225122e-01 -1.017250412301996254e-01 -1.189492154041500022e-01 3.769431879517726180e-02 7.034798203361138091e-02 3.563173914918552152e-03 -7.665183474107867441e-03 6.052515206939250658e-05 1.985251408116728887e-17 2.091075825086423194e-03 1.917048082297168402e-02 7.357852703087742335e-03 1.929298895891551693e-02 2.988123386131259426e-02 3.452253666013130329e-02 3.034477966400665749e-02 9.927098017656740825e-02 1.792145274438375346e-02 3.519302483231903644e-02 -3.973002040205320801e-02 3.028607910306933479e-02 -4.918760919184418136e-03 5.402808738275023670e-02 9.333975965243462847e-02 4.961594742347506110e-03 1.431557843792807296e-03 1.604523841389821071e-02 4.294142461204095845e-02 2.897588232560167551e-02 5.958740185130691897e-02 7.362249558881255040e-02 8.842186138813347418e-02 1.295091469657996298e-02 -4.362482744974801457e-02 1.490227161926172515e-02 -4.429077043985145401e-03 -6.466329667052307315e-03 2.581060089256550366e-02 -1.503736164639036914e-02 -2.028085932601785078e-03 4.989199107426346182e-03 -1.284394684427225374e-17 -5.887312200236835373e-03 -1.803674380757764109e-02 1.331380722716822042e-02 2.444982837847115356e-02 1.701284287189684316e-02 7.671794518034917576e-02 8.923793558389270242e-02 -1.500524436976294167e-02 1.967045399240630391e-02 -3.501124211850518281e-02 -6.433687947306184450e-03 1.862410719849790994e-02 -7.865335268988003087e-03 3.406774373521083421e-02 2.009389585417397164e-02 -2.345523093355385020e-03 -2.783949379970187289e-16 -4.468709190744694741e-03 5.388540874432613120e-02 8.583304580178667195e-03 -6.734386198406726309e-02 2.707566623661976021e-02 -4.937049048413122421e-02 2.028542832907987217e-02 2.085670796837164540e-02 5.012284427119633595e-03 1.143980999084595607e-02 -8.682681240692336814e-19 2.023737542907066284e-02 1.510771337419263265e-03 5.496281440277333424e-02 5.040300813147328279e-03 8.977651836114608486e-02 6.213437732725544255e-02 -2.621273446148936898e-02 -6.744265953598063733e-04 -1.022932623657530141e-02 -7.243258449840058084e-05 -5.384652397499337453e-03 5.308085568265780152e-02 4.033935435412852599e-02 -5.155900336351907851e-03 1.853980610618751607e-03 3.186150119304251814e-03 -1.005796333399677951e-02 2.166222912113760155e-02 -2.337294043588551001e-02 3.255795675934863159e-02 -1.766432451274304802e-02 1.180623325479517594e-01 -2.610411151103960575e-02 6.323091567591172524e-01 4.341196541666074160e-01 4.954790459430583743e-02 4.105430479076613987e-02 2.306419863156161307e-02 4.516545106274726107e-02 5.352615510894807860e-02 3.363649990327676875e-03 2.217934576517054113e-02 1.525119114600856016e-02 3.234141294414605988e-02 1.347254299336256723e-03 2.652687468415979716e-02 1.694004284252296696e-02 3.498462413001781485e-02 5.581972203989567227e-02 7.695052823520268093e-02 1.153861188482980388e-02 3.533367981410411723e-02 5.241066118660986373e-02 5.789759421450161453e-02 3.127103031652358339e-02 5.364324133985590159e-02 2.978502160024739873e-02 3.226483722174097041e-02 3.870911013296481767e-02 3.509263607349751707e-02 5.857451593405447188e-01 4.280526419327038701e-01 5.381564286904693128e-01 3.988486207319646981e-01 4.311961431586673332e-01 2.998442016809670374e-01 6.934601378193583354e-01 3.714052955464980932e-01 6.509403485653185717e-01 1.521349790860797822e-02 3.303733212878341794e-03 1.372771517227610455e-02 1.039897295396984694e-02 4.830603412572274935e-02 4.243048639752030476e-02 4.534096489731584412e-02 4.057709858610453191e-03 2.211363777735244132e-02 8.415846431979918246e-03 5.448893656970293931e-01 8.548932509544923120e-01 3.884551782566665890e-01 4.400570534533071801e-03 1.638716773872687937e-02 -4.978234028001769009e-05 8.178362930636165137e-03 2.516126843638222602e-02 4.544278628712567630e-02 2.619850575266059398e-02 1.114957703832795827e-01 6.090482670765972467e-02 1.420921759119789518e-01 7.611345125954145641e-02
1.376903119441321433e-01 -6.517086424235943309e-02 7.236827669188097834e-02 5.400082230589822035e-02 5.096564369752093482e-02 2.760609173998154059e-02 5.390454893217963561e-02 7.837001308388689275e-03 3.053792842872202554e-02 3.506842590113307817e-02 1.932413046398666900e-01 3.013702077821481945e-02 1.680860167740866676e-02 2.761246515856349490e-02 3.988429135492844307e-02 5.377230136944843170e-02 1.102835362672943686e-01 4.103833191562214489e-02 1.176445324448116038e-02 5.557109286809271131e-02 1.000368477544484186e-01 2.283169439606699824e-02 7.158814800406654300e-02 7.289215475859354976e-02 3.068357493193629021e-02 1.364155903261884406e-01 2.401538614453116172e-01 1.519072756480703279e-01 6.459287781248342075e-02 6.069835218947371852e-02 3.274353215304107688e-03 2.843153780743215564e-02 -1.197062409897511534e-04 -1.515358304415723514e-02 -7.022700557235014851e-02 7.681773270594394361e-03 2.456431585957003535e-04 5.178920395355267022e-04 1.592984459405466860e-02 1.103924579128336830e-02 3.217765418794085036e-02 1.638585605430407932e-02 2.170941452268616845e-02 2.903548679177553760e-03 2.345509582764911810e-02 3.837268065995479843e-02 4.441314740010755824e-02 8.587217683822367942e-03 6.506463462907273801e-03 2.074834849687027041e-02 3.000073047897339196e-02 5.995721365123041040e-03 2.657531640550726696e-04 5.769917145729749723e-03 1.838255356119979436e-02 6.128001487320714272e-03 -3.436254171753313491e-01 -1.695259098917283880e-01 -1.975315985046457723e-01 6.571538975718865805e-02 -5.064639044552243091e-02 7.070671015509116288e-03 1.909789206792147920e-04 -1.671951236740484500e-02 2.251110381389821724e-02 -3.712423803373556680e-02 -2.043155581247955641e-03 -3.338487898927828342e-04 2.201691294226945303e-03 1.327373175783334251e-03 -2.318919835847692290e-02 4.206830439386945869e-02 4.008168359830356804e-03 -1.342315637293347436e-02 8.170958175189302342e-02 -3.632766970063909138e-04 -7.014205252629034798e-03 3.170993855465283236e-03 -2.473295218295572251e-02 -6.533973628886274865e-03 -3.256214930215321768e-03 2.020272376460047203e-02 3.817889464268500472e-03 3.627885085123082332e-02 2.566343837806384434e-02 -7.444046471834588063e-02 -2.028445916189125509e-02 5.955976982471456599e-04 4.950929664600927857e-03 -1.022160264964276470e-02 -2.820472496574880084e-02 -1.043678993540201640e-03 1.276549176265393687e-02 3.065062454423902495e-03 1.238972515430217074e-02 -2.317168112798899220e-03 1.819292542840179536e-02 9.210000100122553782e-03 7.183617511655392890e-18 -2.034703414923540632e-04 8.183347581998815998e-03 -9.585190291709261290e-03 4.675372393537988451e-04 1.286098022910408395e-02 4.134031539478553313e-02 -1.658868665961310798e-17 1.654717553220651574e-02 -1.070239303867555613e-04 -1.275210196126709897e-02 2.667359194785792406e-02 -1.721962392339715931e-15 6.023516102999258021e-02 6.731690072091844890e-03 -9.345722869459268138e-04 1.264224241045025727e-03 -1.473829212804628717e-02 2.159157217827495368e-02 1.468306803379608450e-02 -1.457513986361033131e-01 2.441488306425913801e-01 6.601438737892861841e-01 4.008806699461071599e-01 4.900293240738100992e-02 2.019784891655204348e-02 1.533686121401177382e-02 9.483351012734705932e-02 8.160728377346850027e-02 1.933515871242415599e-02 3.478178424452960799e-02 3.364182175559379123e-02 4.052922940808108970e-02 2.201380984686783504e-02 1.828820789519317730e-02 1.019009151581409291e-02 3.043065398465306590e-02 3.676594758991443002e-02 7.569319452695087713e-02 5.026176275054448916e-02 2.363708162210460470e-02 5.641639997490225517e-02 3.598411867735461234e-02 6.741236815380137817e-03 5.777300037800561228e-01 8.540760980906805244e-01 6.993260153260714151e-01 1.455361059707454829e-01 4.498017951833196970e-02 1.754205439319905246e-01 1.333202525696634555e-01 1.698535493654806938e-01 1.768578400843988374e-01 3.979126179469363866e-01 6.944195423977194137e-01 1.186043941831470550e+00 3.523493733422560781e-01 3.135203827386619647e-04 1.043035742588262899e-02 8.219268223499695374e-04 7.645286886219324757e-03 9.901559551409994803e-03 4.361434619641839372e-02 3.168632582014657001e-02 3.336586675206348884e-02 1.394536664128500497e-02 3.766747308298491054e-02 4.622564051718174721e-02 5.090736280589926166e-02 1.437252737619671220e-02 1.509531835902584371e-02 1.540641090276911873e-02 2.814397228982376889e-02 -1.334067428142868142e-03 1.109430076897977013e-02 8.649523360853783593e-03 4.369041424573282678e-02 9.618961725662854984e-03 1.866946795940027382e-01 5.650949882743835362e-01 3.838427085718574272e-01 7.595945851397266635e-01
1.343804659525988099e-01 -6.743632292908496861e-02 7.016349513957291506e-02 3.773261636318280682e-02 4.775060365374300675e-02 6.012110618424850589e-02 8.433326568120781686e-02 3.776812401319547263e-02 4.337672264657130294e-02 2.914430588223678592e-02 6.779411666882927123e-02 2.581687688723412893e-02 2.159875612072413575e-02 3.537849566893202374e-02 4.876765202933320414e-02 5.452236253654370313e-02 1.298936409903916900e-01 4.286459242739532050e-02 1.765497094831016897e-02 3.314467504598195974e-02 2.951202926090939482e-01 4.804808137121392708e-02 3.419609553982325134e-02 2.949968908794653108e-02 4.559093876318618538e-02 4.829707017134804437e-02 4.245947555123333328e-02 -1.392370163590191945e-01 -7.886428353089736110e-02 -1.084126797825244043e-01 1.314800106645594262e-02 4.590875994669217988e-03 5.864709735287920896e-03 4.517461941145205565e-03 3.669718893883980055e-03 3.930064954581234844e-03 1.176938996916123426e-06 6.061713506030286941e-03 2.126824571397509245e-02 1.335223401543624532e-02 1.205330855533912233e-02 2.046811025232539438e-02 -5.018429578565579158e-04 -8.950347943508521953e-05 4.843660084795096271e-02 3.281294295479928597e-02 -6.511434561663000109e-02 -5.609668011783254354e-17 -2.659462445720680440e-03 4.743556670648656215e-02 6.064754866188390436e-02 1.274500761323030251e-02 2.900556987022859159e-03 1.468378559909419741e-02 5.349870372147885744e-02 5.310836516640106419e-02 6.740614761533433863e-02 4.073644085117680835e-02 5.405348710497940440e-02 1.856061699161272102e-02 -5.607936206593911593e-02 7.540558687410232191e-03 2.802899097526357405e-03 -3.302205934477878829e-02 7.909564721939425780e-03 -1.968503607171038020e-02 -1.268537191010169177e-17 -1.230217346213152143e-03 3.970289805435467390e-02 -1.849196614907334252e-02 -2.115389977610917030e-02 1.733708898486462785e-02 5.577087924907559065e-03 -2.627053985448412693e-17 2.249648874020354974e-02 2.844809412383319039e-02 -1.220411657920479620e-02 8.218332883546571044e-03 -2.200122891622875720e-02 -1.376060904228565591e-02 1.554347763045508049e-02 -5.907965472876161236e-03 2.426528876346735909e-03 2.151951325093325734e-02 -2.106498917784341632e-03 -1.256694253211161280e-04 -3.280436409124911322e-03 1.237826181061280409e-02 5.968937566255523922e-03 -3.993964295703350542e-02 2.124312314014135256e-02 1.075710739194319157e-02 7.539412472402149193e-03 8.391576923662156726e-04 2.790090283657323256e-02 1.737971901850284603e-03 8.762411828074227749e-02 3.315909776095873202e-02 2.100676020631281999e-04 4.060403672731344865e-02 1.506561769658878482e-02 1.010941383452249910e-02 4.636838846542710828e-03 2.311689556785461244e-02 -4.891679438488858583e-02 6.905208112044024585e-03 3.934873494245061387e-02 -3.746181462671144356e-03 -2.720826031838289391e-06 1.417802818315875429e-02 -1.248965719092963630e-16 -1.701964390734733018e-02 1.443856121876957097e-02 -3.883733384329154329e-03 1.790453249064806115e-02 -2.660590635830354170e-02 2.214864380599003771e-02 -4.536796917497978865e-02 4.526643151593788745e-04 -2.373175328089729974e-02 6.419208696606597542e-01 4.305467019530117567e-01 4.929461988985732457e-02 3.545041630201226091e-02 2.494906750139243290e-02 4.131632762276853804e-02 6.247138335599595332e-02 2.646577979950592852e-02 2.329799350532624908e-02 5.555316820614412260e-02 4.054382113866437931e-02 3.819780005413013404e-03 1.668038456852524026e-02 1.860319608410198472e-02 3.489444308604904704e-02 4.661499079525811828e-02 8.440062019933254611e-02 2.673210364285100865e-02 3.426922424657866117e-02 5.618025738624092547e-02 8.095659733413161641e-02 4.034366317549931136e-02 6.028989954475303531e-02 3.117362712606326983e-02 2.962601166056208143e-02 5.177747353102547800e-02 1.880501059796546581e-02 7.028269638883358761e-01 5.681556155740731251e-01 5.505081393959183611e-01 1.866379818478292388e-02 9.821579380801441148e-03 1.335096306168947859e-02 1.327966737242872175e-02 1.549336095750025559e-02 6.269187874129569847e-05 3.095162461358242668e-03 7.760349537517215244e-05 2.110793015183387925e-02 1.214720492357601773e-02 2.112757666070499279e-02 4.196532000546922547e-01 3.907980221431009427e-01 2.384531032467721856e-01 7.522046089077903154e-03 -6.743402888996657962e-02 1.892242721815470818e-01 4.025797313370652608e-01 2.790039603156215242e-01 1.702096626053955022e-02 1.729438189922838723e-02 -1.057136762460911868e-03 6.222589170002773735e-03 4.267115191381324994e-03 4.527593789434761901e-02 3.410694169338459453e-02 1.118000237480459430e-01 7.326172579586424927e-02 1.018239378090777941e-01 6.495694365331371178e-02
1.414028628710875257e-01 -7.373628910517765478e-02 8.978906699099387101e-02 4.282237966779980226e-02 3.202528055725092221e-02 2.131485450720756875e-02 4.704276067497512731e-02 1.823710855057137509e-02 4.818185864504665600e-02 3.911828814254419279e-02 1.168688743138948938e-01 2.208964410246723117e-02 1.643048430872431215e-02 3.257480236251019540e-02 5.490200689695227954e-02 5.917277746159085861e-02 1.131076645833865996e-01 5.364539554179510727e-02 3.115970268330254395e-02 4.422273549212413224e-02 9.172244823837122363e-02 2.892621954823489799e-02 8.265653616585505459e-02 1.422645441484698958e-01 3.832722678105979913e-02 1.230717643922474899e-03 6.282548053180074532e-02 7.701311352759102058e-03 1.332001323991986036e-02 2.310056376883320127e-03 1.589599626200413488e-03 1.299911546338248201e-02 2.786625075090948690e-03 2.110810617786341471e-03 1.408620814035821921e-02 6.242328446637608087e-03 -9.317176940685888142e-02 2.563141481440181567e-04 5.032982122722832845e-03 3.169962105797386603e-03 5.985128427941531319e-03 -5.197924589715964029e-03 -2.884297323934598839e-02 1.093309360356733116e-02 2.210812254670799226e-02 5.666135028000068158e-02 4.217461586579288046e-02 8.280901178747404265e-03 8.835333142553900010e-03 2.699949712118406356e-02 4.502341191830923350e-02 6.803757077447308128e-05 1.716859739107472974e-03 9.244429413239378668e-03 2.988530401885155369e-02 1.908418685801939463e-02 -2.701711479165403995e-01 -1.551782126072382417e-01 -2.002951098582661238e-01 5.808888532954811779e-02 -5.706117546299680626e-02 1.222471252810308946e-02 -9.268980820978796678e-03 -1.289082035499081701e-02 2.260195703642329385e-02 -1.970753045615031979e-02 -8.220305676921679141e-06 -1.707430864694056694e-02 2.296590833118236988e-02 -1.384137941183397405e-03 -2.002012004382946325e-02 1.606959516467518570e-02 8.246134780146270951e-03 1.742284232561802041e-03 2.331499461576288720e-02 -2.623446451765165405e-03 -8.311537324196651214e-03 1.553509795285053100e-03 -5.213554791011176803e-02 -1.778225124435987276e-16 -1.240902305441625659e-03 8.808599774110364988e-02 2.483411335885182467e-02 3.156486515229027512e-02 -3.584231726202067236e-04 2.191699938484913471e-02 3.224988668804917163e-02 5.052703253459201599e-02 5.717740236156423905e-02 1.499779557254736495e-02 7.148825371283455514e-03 3.381880876613689579e-02 7.278763619720034564e-03 -9.724539636317737143e-03 -5.904740605855573077e-04 3.469211338172208808e-03 4.776686375898485382e-02 9.327901787968755987e-04 3.239243044673375399e-02 -1.120408950602972548e-02 -1.070896085903795363e-02 -2.864356374050894966e-02 -1.834193784615651898e-02 1.590627114340621012e-02 -1.204357189196031538e-03 7.209015633411229584e-02 -3.839791042180894145e-03 5.959147699930895936e-02 -2.929913599695464342e-18 6.260274552056507846e-02 -6.342273695188742778e-04 -1.676892498141165783e-02 1.523989499790435846e-02 -1.081033011843109300e-03 5.517073917045179103e-04 -3.130291256835424779e-02 8.684093327192195580e-03 1.261094704769709536e-02 -1.617710016785992855e-01 2.689478909002762097e-01 6.488585839931823029e-01 4.309924768253489691e-01 4.368148790175808982e-02 1.668636726214861588e-02 2.251575867962267788e-02 5.080708754157180901e-02 7.424652419137016401e-02 3.464869787378793003e-02 2.768939680438730136e-02 3.233058246559054882e-02 3.050392623945822473e-02 1.295994241300159160e-02 1.857139229149042461e-02 1.530071868177881184e-02 3.113755080612436166e-02 2.184270423496433022e-02 8.456420619270049732e-02 3.785945122308564253e-02 3.263446689793898098e-02 3.975194824069681887e-02 4.540501426979743488e-02 3.019321470194605197e-03 5.379417955135211704e-01 9.201324712990377463e-01 6.434322717972189576e-01 1.650617695528209042e-02 4.850600811199701212e-02 1.157435753324163527e-02 1.035464894044930972e-02 2.192595423220327147e-03 1.020193493630309339e-02 3.808737580968779134e-03 8.021634652194306950e-03 1.420483970444509419e-03 1.358618081640159195e-02 3.479692702762666469e-04 6.734720446014788066e-03 4.712081367359514245e-04 9.008293899653652473e-03 1.223583643551002728e-02 5.539440585958892438e-01 1.062156253669150452e+00 7.651169955457013394e-01 -5.019359031895889028e-02 2.451697842124027360e-02 5.123475282146974785e-02 4.737260333190144307e-02 1.744663216541229855e-02 2.367793784609655353e-02 2.564490956148943526e-02 2.607797615888000561e-02 -5.771034975817802837e-03 6.299313556109038120e-03 1.533977715754128446e-02 4.385110728717007272e-02 2.358908635508086965e-03 1.899573076173837194e-01 6.350733363273318721e-01 3.618788689507145828e-01 7.489109761631635998e-01

`projections_test.txt`

2.878302427261433261e-01 2.901531857874368320e-02 2.971710641946135786e-17 2.305946804660580045e-03 1.379152072465770389e-03 1.186741425385955898e-03 1.605764210629536358e-03 4.102606724741067468e-03 1.434065308921900726e-02 5.958583437429383015e-04 1.122737069774341553e-02 3.921401629043507094e-03 1.557201873092487095e-02 2.131545401813801296e-02 7.875335573747321849e-02 3.986459367915190857e-03 9.478029005014347239e-03 1.088739018871304892e-02 2.704333639341854889e-02 2.713705564281946453e-02 -2.996042638146843018e-01 -1.716695065668934206e-01 -2.036884307064390776e-01 1.750068859441673841e-02 8.003511259858094029e-02 1.239426570277758766e-02 1.767974655253408758e-02 3.240062166319680054e-02 2.045905007120899996e-02 4.699551230649015587e-02 1.036733814841171719e-01 8.678070597497665462e-02 7.382420377129010614e-02 4.205692777414141853e-02 4.488796691535547334e-02 3.691869081758011178e-02 1.931844639669136332e-02 2.886794406819147285e-02 8.262944033039161529e-02 1.905316877176270463e-02 7.437174674563023599e-06 2.879097392488967275e-02 -9.543157154946376721e-04 -2.429001858958935740e-03 -1.737592358989528338e-02 7.039301668041042515e-02 4.742115235122071337e-02 5.681244684178556742e-02 1.352804213802811031e-02 4.081645475561583580e-02 5.416812633679046751e-02 2.474812745680352380e-02 1.250275835259146741e-02 3.013168889325877967e-02 5.149648591209673998e-02 3.903070861444361828e-02 4.976872938124938389e-02 1.036843725389637906e-02 6.834009600799498263e-02 3.195975709483656751e-02 -2.576914406343685515e-01 -2.974099520799093534e-03 1.272725673282400216e-02 6.544012102295293509e-02 4.407043964842639883e-02 4.865499397764439134e-03 1.389250939305218125e-16 -3.320933971920239040e-03 1.496258886933404114e-02 -4.269565002855905955e-03 6.728977996731740248e-03 3.075297270137119718e-02 1.319965758646291995e-02 -1.534525984300964298e-02 2.808010226565966355e-02 2.322622640703607793e-05 4.894709614939454778e-02 1.964400458291513940e-02 4.833213845320920299e-02 -3.078421199398684305e-02 -1.651585061005834618e-03 -1.587887700862241624e-01 -1.563254423180366892e-01 6.124769828692699924e-02 1.713661656737844319e-02 3.548875190886731368e-02 3.207905221011796149e-02 -3.479280316062571474e-03 3.847599516136510078e-02 -1.091403394929922784e-02 2.192337421258294672e-02 -1.235933451749927046e-02 -2.960963750212077840e-03 -4.490402330404330566e-02 5.753033602440900830e-04 1.271869399374463486e-02 -2.250653527307839782e-02 -3.708510739788694471e-02 4.063319581030649941e-03 -2.009570674596266454e-03 4.979666038100892880e-03 -4.958716304269207868e-04 3.217360730846384465e-02 -2.042591841603484237e-03 4.561856069375241213e-02 1.306322708057392964e-02 2.430222331898164792e-02 -2.808857461010710876e-02 -6.318373563016761729e-02 -1.844086545473579232e-18 1.233283217122265948e-02 -6.333334330793179666e-03 -1.362153907437474269e-02 -1.959152487062671331e-16 3.451137337275703414e-03 -1.737701525970766769e-02 3.655760599277429329e-02 5.167636643195763074e-02 6.223461286252029295e-02 -3.695315035444392271e-02 5.856774560900807058e-01 3.701300949921354499e-01 3.082801611371563735e-02 1.086804097106880206e-04 1.866392539164140815e-02 2.626099642759575394e-03 1.022385673730101009e-02 6.830461379894128653e-03 1.279572455314385154e-02 3.879577155746976886e-03 1.901966731950187686e-02 -3.344341960690826177e-04 3.726861658544092931e-02 1.104185926453103705e-02 3.395271303817488234e-02 1.372164618072069242e-02 1.136264203815807658e-02 1.017797066957519883e-02 2.956860840166108256e-02 3.424961229227891247e-03 4.082647347556646600e-01 9.673862396636346839e-01 4.006229081950696824e-01 2.110129475592088452e-02 6.668031312233567431e-02 2.083045363564256167e-02 5.839725277923643598e-02 3.754990802960495727e-02 3.006269299314871107e-02 3.244241273799104708e-02 6.014513282473740036e-02 5.382010163907376826e-02 4.946414430057752276e-02 4.461600602660322629e-02 4.516927615002498675e-02 1.591380263192818459e-02 2.404734841209452481e-02 1.904682537999749861e-02 6.337584277309091241e-02 2.409347144210009090e-02 4.497918943581571440e-03 3.012985624017634582e-03 5.352251030543551114e-01 8.417062014073032339e-01 4.607441650134912470e-01 7.786165206083293065e-03 3.537707886260949702e-02 5.057774916913223423e-03 1.725507833714095413e-02 8.709855658464991587e-03 3.316376228699274148e-02 2.881599326760123439e-02 1.676776840602020938e-02 5.017638266738223829e-02 6.131477781749537087e-02 3.210383766626577484e-02 1.353427481149837019e-01 3.067081382807733680e-01 4.709875277950001937e-01 8.701572278626673596e-01
1.376502929949463550e-01 -6.002419593216773430e-02 6.269185726095699573e-02 3.347022779637769813e-02 9.868962657090511736e-02 1.030127972699854400e-01 1.624486841781879187e-01 8.376990176028218868e-03 2.900797355160050425e-02 4.676178238309259411e-02 8.233857741170289013e-02 1.611629778902676571e-02 2.132670847379204010e-02 5.664526511729794933e-02 7.911034749672807287e-02 1.103073538987272978e-01 1.963853069283564845e-01 8.050635544227145723e-02 2.105857994969480979e-02 -3.175630361674035979e-02 -1.532152245840251101e-03 1.772107030249454179e-02 1.166841902002393495e-02 1.094959898339372198e-02 2.355288671904986163e-03 3.210773663440650173e-03 1.770751955700169619e-02 2.955323299195507930e-03 7.963288187889506695e-03 1.317044599287298459e-03 3.719838410161995998e-03 6.335025052836180948e-03 1.182938960046144494e-03 1.319220172834111027e-02 -1.589019784643682376e-01 -1.627841008564107628e-01 -9.548536549858949174e-02 3.241825484805528623e-02 2.458329708235188621e-02 1.706061933002189085e-02 2.306693367432320618e-02 2.368161822745815451e-02 5.164146860579180154e-02 2.075912260324344963e-02 1.627500321401659927e-02 2.605508584220760440e-02 -3.359584455893713240e-02 -6.723194167504850584e-05 -4.015205189781059780e-03 3.471584762551682690e-02 4.802220771838265623e-02 2.838069941526687873e-02 3.287186812700246263e-02 3.320791742358975701e-02 7.603762250109342979e-02 6.376069681702759007e-02 6.662694806243633916e-02 1.733029558099854484e-01 1.431993366435525550e-01 4.137892885023197048e-02 -7.397843854276214826e-02 -6.083478434310732146e-03 5.262519690783457049e-02 -6.606504643872784799e-02 1.607374519889452624e-02 -2.187353240842781696e-02 -1.142908487098289543e-07 1.046721604845278070e-01 1.526325752995745522e-02 -8.733745973691070618e-16 -6.260845972151234638e-03 3.463000431653569938e-02 5.961698004924993223e-03 -3.150131031125019687e-03 2.868250610635056722e-02 1.050625536467136335e-02 -2.279661209122649246e-02 5.413418251207610839e-03 -4.895388766993825913e-02 -4.137079763366600937e-02 -5.445658561193333219e-02 -7.407310406774635743e-04 2.740063505446413503e-03 2.427371386519549656e-04 2.897289747545809743e-02 -2.819587215371132088e-03 -4.436152627240882805e-06 -8.639655045325617816e-03 1.862361468544160292e-02 -1.379728285657497860e-03 -1.625180461525538994e-02 4.178238426455083837e-03 4.466011176971525914e-02 -5.184818569169435953e-02 3.025875171860157968e-03 8.136782824639139255e-18 2.152525846679667967e-02 2.465874959414945250e-02 2.059981968022987081e-03 -2.173332460236867959e-05 8.599696838740367513e-03 7.301954288476636412e-02 -1.124468813769385726e-02 3.186727738474231381e-02 1.448538280420876012e-02 -2.701086050269600707e-06 4.732247138990295504e-02 -4.910153476043392640e-02 -1.452816972138188528e-03 5.553133230425725403e-03 -1.159357324949543411e-02 -1.039904812270760030e-02 1.594735429864158754e-02 -1.550812510565018412e-02 5.226467002191405309e-03 -2.418306930541285274e-02 3.626494057072767591e-02 4.278073499329347413e-15 2.879170516424439316e-02 -2.165929645365166634e-02 6.653404380087978298e-01 3.986693779153284711e-01 5.138733007109323042e-02 2.482320277518521717e-02 3.024706202799365418e-02 3.498381503351306182e-02 6.967262699385132207e-02 2.113990598289105724e-02 3.396312184568310577e-02 2.866003141276903324e-02 5.444199713434380583e-02 2.160933730278778719e-02 4.554410734526519872e-02 1.320812058696475486e-02 4.438770868708583756e-02 5.176453028074925328e-02 1.014199231436470894e-01 2.185281870070240190e-02 4.064909761766351548e-01 9.781279264553657260e-01 6.290548634598043254e-01 5.633985541778257014e-04 4.315626066430858915e-02 2.655449652134309751e-02 2.589676377256447046e-02 3.536220224882875851e-02 1.779090266687021263e-02 7.329496743016024156e-03 1.491802260131306831e-02 3.093762701329839156e-03 1.522859824857704207e-02 4.578262995385440348e-03 1.778192654469750236e-02 3.266436009294879702e-04 3.570066875713656107e-01 1.306965753827163734e+00 5.072409710461545096e-01 4.969689587429221289e-02 2.726795449045802539e-02 2.752010810221213605e-02 5.930744750988903996e-02 4.243436945745648892e-02 4.331717815231175134e-02 8.628773419341245060e-03 3.441920258405292005e-02 3.655157360238538850e-02 4.927642200375626502e-01 7.681697024601950030e-01 3.863346202981886024e-01 3.573223702450898601e-03 2.214220399453524155e-02 2.246775964668462633e-02 1.725031708417382181e-02 3.222640027151306430e-02 7.164956375232420238e-02 3.943401845165669939e-02 1.165379220419025252e-01 8.848240145771195386e-02 6.738315518352375189e-02 5.930007029661275852e-02
1.341905835820521586e-01 -5.713890323440116992e-02 6.375687902642285243e-02 4.515249785958116296e-02 4.563061868820471323e-02 4.777565416923616748e-02 4.370863495145165672e-02 1.211994109380861058e-02 6.266511586747919305e-02 3.210783639658672944e-02 6.020636791890662115e-02 4.742069158773966342e-02 1.780656186513081621e-02 3.099215444416069179e-02 5.444635008431755741e-02 6.921352200753562445e-02 2.206270404846894517e-01 7.922530790647396015e-02 1.864280302157910948e-02 3.422397830586605683e-02 6.467015943556220225e-02 2.016273860054918626e-02 6.301159434696172734e-02 8.665993946549588323e-02 3.962097826305730625e-02 1.623660559127776057e-02 1.437905863296894821e-02 2.171875470929147435e-03 8.396095943990185620e-03 3.855627635166801459e-03 3.762233476759064329e-03 3.724453994954245693e-03 3.477983233339778085e-03 5.000080626347510812e-03 2.726128292068981038e-02 -2.991785205678257342e-01 -3.521217298292682041e-01 -8.527505803778520632e-02 5.188779282972608153e-02 1.828704587649413252e-02 4.150125903800699206e-02 2.142435040796146495e-02 4.371642989546611902e-02 4.187696837658014953e-02 4.318321284402563964e-02 5.389532283160285847e-02 4.816733409661705378e-02 3.339594628996689352e-02 4.973730196336168669e-02 3.386566280661444661e-02 1.424649988175431120e-02 3.165153936620467793e-03 2.023360385309802903e-02 2.024653330033376192e-02 5.865015923312136281e-02 4.111085095148783036e-02 5.802865325929514734e-02 1.526757673293702988e-01 6.065159304348857056e-02 3.499568279267072824e-14 -6.852621881244677637e-02 -1.223664464498293953e-03 1.424152175163588552e-02 -3.151012472281914528e-02 2.721023346856626945e-02 -4.255057523926750185e-02 -1.307142582154765652e-03 -8.812058843237356026e-03 7.145945958657597651e-03 1.692191872142242634e-02 -1.087388730455149215e-02 9.391850142265418394e-03 5.215036652058505778e-02 -2.056350222971629546e-02 2.757401824299694770e-02 -5.576360538447065404e-03 -3.372656796402912632e-02 4.016681661960410515e-02 -2.576701837148953969e-02 8.789766743829922208e-03 -3.257975802297626857e-04 3.484357561486605553e-02 2.297851686155649373e-02 3.662663851702614715e-02 2.508038143539514583e-02 -1.906756352509156966e-03 2.654249395651286578e-02 6.955291884553499622e-03 -8.517784676543002642e-03 4.554903888649606061e-03 -8.562916915843699767e-03 7.576378482918443881e-03 2.554027617603427808e-02 4.084076275471885389e-04 2.767194970330084589e-02 -1.244310890946832317e-02 -1.477814060303996372e-04 -2.137177297886946534e-02 7.459554874212047382e-02 -5.320745802639719434e-03 8.349294268292648674e-03 -5.487119185433587260e-03 -1.784788856768879453e-02 -9.614382755845191086e-02 -3.245502937976515838e-03 -6.813682450583723703e-04 -1.795732335299517549e-02 1.361220183159400354e-01 -1.431932280378560875e-05 -7.537539108966277797e-18 8.518148600818628569e-02 -2.434040143969360553e-02 -8.409376056082445636e-03 -2.454358624887306023e-02 2.835711624903455852e-02 -2.180297029969107195e-02 3.732279028717377933e-02 5.662443032934587339e-02 1.894448236154230164e-02 1.347773202351221222e-02 6.679342986314890096e-01 3.990242589301239429e-01 4.084594224947386137e-02 4.599243513715989273e-02 2.322395957346577375e-02 5.837349193996034524e-02 7.144568659473091166e-02 2.693078348897156096e-02 4.424478750729979315e-02 2.743787911998938484e-02 3.922406892534681389e-02 1.144794830760400076e-02 1.914165795397006209e-02 1.002478766602296970e-02 3.665563034443913154e-02 3.330241702431824269e-02 7.680782645647597773e-02 1.905746630044919190e-02 2.822297945941273228e-02 1.980682600936849863e-02 1.289246598021983464e-02 7.321501598818011460e-04 5.798571877252558515e-01 8.684942773580742736e-01 6.580972786411714015e-01 1.969193936964719078e-03 1.175459949081153130e-02 4.735667884249171108e-03 1.962449225663704527e-02 7.665600655760279256e-03 2.382778180847671887e-02 7.254563392363367821e-03 2.242147483599051319e-02 9.107104493869906323e-03 2.803975986685820071e-02 8.160351567049138888e-01 8.066457799557872343e-01 7.372370844142749347e-01 3.083690350024695034e-02 3.068969854492703195e-02 5.973250845227363881e-02 2.860993729569476576e-02 3.575751857108420972e-02 2.713598531173952270e-03 3.612054407701683484e-02 2.585493758853382729e-02 6.768326707096664063e-02 4.554208292139252379e-01 4.165250039506049329e-01 3.238478206587273034e-01 2.574142379720111354e-02 2.091389444014019050e-02 1.759661079727814145e-02 4.663905493092243559e-02 7.504930394993350462e-02 5.452869699647216323e-02 1.217994480060288764e-01 7.896637800452346867e-02 9.571602299256953827e-02 6.863957833295021571e-02
2.652260527470438056e-01 7.057565304095268950e-03 5.813352886077224543e-03 2.551523288840218222e-04 8.988586933094774856e-03 2.215138261270969879e-03 9.062061254350169126e-03 4.880654080922446590e-03 2.447926731691598377e-03 3.261020808945088757e-03 1.850333796231433161e-13 6.953955832070855304e-03 2.614861138000214302e-02 1.835318254316280900e-02 7.236698984186602790e-02 5.963184526168665775e-03 4.304156797217919238e-03 1.837223154994101562e-02 3.060827172695813891e-02 2.303246619103097026e-02 -4.294884475976597504e-01 -1.897387055589273008e-01 -1.568954228088071945e-01 8.020345748298277067e-02 2.287738031461784005e-01 3.565764802623345997e-02 8.538904690385921981e-02 1.022771781771797395e-01 7.213520984255018809e-02 4.895200291590316027e-02 1.380840811689664216e-01 9.102356968020675065e-02 1.561145682735932128e-01 1.078572298205616453e-01 -5.620631392074435656e-02 -9.990458351811155879e-02 -3.199001620973433713e-02 1.670837450141662503e-02 3.732043476652040226e-02 1.054314792540376328e-02 -2.628182064632431786e-02 1.989884383344604421e-02 1.532278749890833222e-02 2.115165076089421236e-03 1.224457338583394457e-02 2.287055492970476930e-02 4.468944804194898562e-02 1.034300958397559034e-02 2.853849418192838661e-02 8.211281486201472274e-03 1.540819096041467530e-02 1.908227138228036149e-02 -5.185329150561447664e-02 -9.134085368613652223e-02 -1.184452154306222943e-01 7.214038975816663979e-02 1.030125909070941648e-01 8.361853266800796380e-02 3.085129280363527737e-01 3.579264820004210423e-01 -2.312501680130216175e-01 -2.092803744054595998e-03 -2.455877844404126542e-02 8.638405758888460795e-02 2.426804622763378133e-02 -4.144072033445458589e-03 3.315437544685856386e-02 -2.850085681568899124e-18 1.072678795766358884e-02 -1.331334673326597659e-17 1.237118728145514095e-03 1.416871191385454982e-02 3.578381819813052089e-02 -1.413064341896535202e-02 5.361415509010119401e-02 1.075917947176040050e-02 6.480940826242584385e-02 4.740090077218038142e-03 7.999320600063956899e-02 -5.829295963705551087e-03 1.196136324267751960e-02 -3.622171515058109775e-02 1.072517471356667284e-03 -2.568997204292184428e-17 -1.736549706720353454e-03 -5.278236618640778314e-04 3.913772624542592379e-03 -2.515578786622308174e-02 2.914755071445320270e-02 -1.573121632508600641e-03 3.075798779296860797e-02 -4.203972001344601778e-02 -5.774290385477004494e-05 1.545859679950583807e-02 -4.305378496944774203e-02 -6.299711940543403083e-03 5.185388017108985559e-02 -5.817597326089896277e-03 3.048074844942909312e-02 4.416563479542273087e-18 -6.947153822021711186e-18 3.380446780426288322e-02 -6.060562940408470954e-03 6.298362525101589872e-04 -4.870916592195110965e-03 -4.684007832357908740e-03 1.128149394262289822e-02 1.118653722774117301e-02 -1.550511102409736367e-03 1.810720922596784077e-02 -1.123554487762947793e-05 1.155070586594801607e-02 -6.242664000976085731e-03 -7.158902412605478403e-03 -4.630117071597743793e-03 2.407670126339160957e-02 -5.840867019822233719e-03 2.316217285998507791e-02 6.839729697800607522e-02 5.870493885354488572e-02 6.542762403126040072e-01 3.196903168743041590e-01 3.165511297133698915e-02 2.425717113593548607e-03 2.106933121019574293e-02 2.330747005032852807e-03 1.991017511744644017e-02 9.348535634340160091e-03 1.081705294806183278e-02 3.863532341921474458e-03 1.484812791530745052e-02 9.535910435303640229e-03 2.404668507878099382e-02 1.513643870708390390e-02 4.563329458121363064e-02 1.384035620110406553e-02 2.080440338988937682e-02 1.632414850686279839e-02 2.689822938749994799e-02 1.180080761944690172e-02 2.132365192407386645e-01 6.634106467264456741e-01 3.234794068112734622e-01 6.426797311937722766e-03 5.074155249250301980e-02 1.264164151632243653e-02 4.049025601585327233e-02 7.976934618549411105e-03 2.324676324134801117e-02 1.157983022137890235e-02 5.495191376791018489e-02 6.867229578776105903e-02 4.865073879478547941e-02 3.329173143718343331e-02 4.005991380169291460e-01 5.860240136474242911e-01 4.156484370153739061e-01 5.197672522947334277e-03 4.403779578303807646e-02 7.786578825867190769e-04 1.556697582876410448e-03 1.213796669549750337e-02 3.475952445805220031e-02 7.692738161925629335e-03 3.916579251916806753e-02 3.960017871640220816e-02 4.725289739547835349e-02 3.072345052358045267e-02 1.930932683164364450e-02 1.123681558944190917e-02 2.751107448635744038e-02 1.490668372506596178e-15 2.456083429114571393e-01 1.169009356988390369e+00 2.513088362428694267e-01 1.901151679752744253e-02 1.166707259120547124e-01 6.947275982129928751e-02 2.928312926341521821e-02 3.986162785001376961e-03
7.007951315729605765e-03 3.134808979342619539e-02 9.543267996993143321e-02 3.828881265666012057e-02 6.020260851239553346e-02 4.869216645888693046e-02 9.235355315463047998e-02 5.289149333714395634e-02 6.192751396728406882e-02 6.279760631486734634e-02 1.158422752958797458e-01 2.964514020930027541e-02 1.829703100178064687e-02 2.090165186962495564e-03 4.272383166631918416e-02 4.242726501705916531e-02 -1.194589171111486575e-01 7.549196294977419432e-03 1.764892578541538788e-03 9.980474272146318965e-03 5.456740461540202092e-02 1.457837128361364788e-02 1.052811280891322206e-02 2.748396071757114783e-14 1.440257064019991214e-02 9.432893013945020623e-04 9.099786568688354726e-02 2.916019738885809471e-03 9.920129292185551981e-04 9.566059874772154040e-03 1.343508576355529477e-03 2.338931030621282676e-03 1.035486840055802439e-03 1.626365456928838618e-02 -2.558982240795143470e-01 -2.252265366469176633e-01 -1.180188604950635628e-01 1.577273473891607769e-02 1.832963503458882176e-02 1.026640892346202504e-02 3.312046591566557979e-02 1.913166754388958071e-02 2.316652872002411892e-02 2.109099518780701427e-02 2.787776219249935364e-02 2.838364908684034432e-02 -2.802078447455068344e-02 -2.381282441858832591e-04 -2.199888272403938919e-03 3.221209044596785170e-02 4.367943363599322698e-02 1.615849885640796221e-02 3.499307048174845475e-02 3.520351406106913411e-02 4.875092163622975205e-02 3.049754191493234123e-02 4.974519299701017638e-02 9.560295297008492010e-02 8.323486580873902529e-02 5.129088218423638601e-02 -2.861828914651074612e-02 1.120839643260173983e-02 2.552888790582887729e-02 2.568653430923081170e-06 -1.070138240899952973e-03 1.677474376621804642e-14 -1.140720142073475395e-03 8.968848086275839349e-02 1.929650504410742760e-03 -6.872661077580871200e-16 -5.629325312871357595e-02 7.283459910168373075e-02 6.126238185755951415e-02 2.803570619226627893e-03 -2.491505534483111697e-02 -3.417411330083738874e-02 -1.574829839658541675e-01 5.711506237137674996e-02 -9.050748495650677572e-03 5.118114632008652007e-03 -5.649151954759838506e-04 3.140935585716396038e-03 1.346837079885663006e-03 -2.344569544402687767e-03 1.846195437013243082e-02 -1.615589233120108201e-02 -1.968916984202122983e-04 2.655982957235408434e-02 -1.802824550548045036e-17 -4.879080564196185479e-03 -8.659717618622978300e-03 4.598485255558126122e-03 1.283371765995351167e-02 7.616235921542489733e-03 2.691951701426864730e-18 1.352563139977519188e-02 2.412337999422403678e-02 2.036393110492554399e-03 2.824778621145798113e-14 -6.196384768003652954e-04 1.189100899860066896e-02 5.221723173897407227e-02 -2.633743995932200099e-02 8.586536926146173809e-02 1.329220159855144844e-03 1.760754853794056080e-03 3.880489657448174534e-02 -2.891906495187212753e-02 6.158230373787918627e-02 5.712567878420678636e-04 -2.254107495852368526e-02 -7.105671472782941304e-03 3.783756639964751878e-18 -3.047055491281555704e-03 3.599202738216385200e-02 -1.782696318412799880e-02 -5.814773913749623352e-04 6.810306205871059248e-06 5.731347834797640567e-02 -1.303920380450205273e-02 6.301407980017888211e-02 6.042571001797872321e-02 5.558056974687862473e-02 4.658158904354345320e-02 2.097715498550133464e-02 4.947009737442800009e-02 5.174401253417930391e-02 1.991930794145487654e-02 1.727965477010308046e-02 1.285672876554506266e-02 2.931675919685892279e-02 6.180628913696829645e-03 9.882382903024608534e-03 2.160175838760940714e-01 -1.535631299969372754e-01 -2.066767957273125955e-01 9.170958808794488215e-02 7.582979938076992062e-03 1.455433759445118672e-02 2.192214716725751286e-02 3.981457184828095242e-02 2.295830059594014625e-03 4.167181420785105728e-02 2.527150539976672980e-02 2.629132108479004595e-02 1.061249838490719785e-02 1.331472556041306786e-02 8.766799137797235894e-03 1.437228742993633401e-02 4.638573469920487941e-03 1.330715000717902496e-02 3.169094243861476055e-03 2.245656330203601656e-02 2.407546822635778781e-18 3.448392524233415135e-01 1.201745865579866068e+00 5.099890910453199755e-01 4.815974505847351456e-02 2.154696649162878702e-02 4.835460664159766525e-02 5.513450643198888196e-02 4.069575598742146416e-02 4.956617131786621167e-02 2.889744823347922473e-03 3.103592512289888444e-02 3.455014956870757631e-02 5.265151003935902851e-01 7.786221698917933365e-01 3.826599024760048984e-01 4.594760940204023827e-03 2.653832963043864165e-02 -8.398859968278372468e-04 2.036698674397987088e-02 1.787811797977804582e-02 5.006075996338693601e-02 3.863406595750018546e-02 1.374638174762650988e-01 5.963887773506106049e-02 7.438158052774417051e-02 6.672458573992164077e-02
6.764130951468404207e-02 1.004596276577456454e-01 1.742127776646847648e-01 4.590049093330898777e-02 5.437590995572118974e-02 7.287905229138758045e-02 1.313025126295820078e-01 5.338735110498540187e-02 7.383651279747546758e-02 3.811458808468681791e-02 -5.915677782297302373e-02 -7.296304728451526167e-02 -2.434836815106326824e-02 6.546878739243233714e-02 1.545092827417575976e-01 1.761376057948356721e-01 2.502911855683327969e-01 1.929572505340489685e-01 1.892571036222580327e-01 1.269010621073840828e-01 -7.802339394605263267e-01 -9.006611456306645724e-02 -1.605475589101282019e-01 1.832747933828456771e-02 8.856396418447751162e-03 4.137538882638186997e-03 2.234522619458094092e-02 7.176412388304454631e-03 3.244418953446068406e-03 1.215323419711205575e-04 2.848861720185335146e-04 1.940074220883670261e-04 6.707806788746591229e-04 3.350231825471032196e-03 2.432117550251607085e-02 -3.290636661761134341e-01 -7.611904786733438011e-02 -8.580374507477460388e-02 2.911397933633632501e-02 2.543999012413278304e-02 3.888525584237089577e-02 3.918298293377114883e-02 2.832898520468858303e-02 1.315288758919109895e-02 2.219640028224172112e-02 5.426304385323741514e-02 7.788686406509487403e-02 3.133250579886021764e-02 2.505552599265054373e-02 2.880733365052495776e-02 4.248227060990742571e-02 6.508069857457185990e-03 -1.810955688465891791e-02 -1.317058248251486707e-01 -6.859658317603412048e-02 2.375833412342006901e-01 9.444854303036434584e-02 1.535304889436006626e-01 3.030780020939133901e-01 3.373211446763700883e-01 1.131869569449916839e-02 -5.647300996722329908e-02 -1.976064793857723514e-03 -2.650423361692804541e-02 -1.542440291102234072e-03 -8.919546003699424866e-03 -1.561427309334449447e-16 3.437741058299630115e-02 2.442482688052199924e-02 2.728093650903426481e-02 -1.230344825539570093e-02 1.719738562330457715e-02 7.580253475863771295e-02 -1.886424314242760372e-02 3.230890437297694040e-02 -1.004062563713371131e-02 -4.443585619254340063e-02 1.245543457240371743e-01 -6.987456036474259946e-02 7.322692431740569774e-02 2.259764637371770694e-04 -1.540067133868055771e-01 1.228498397361012934e-02 -3.587059753774358550e-03 1.150453167878751418e-02 -1.199742174308461610e-17 -3.383819256400667774e-04 1.016759571228752559e-02 -2.280395383899578067e-16 4.706541314845756087e-03 1.058889797936798417e-02 5.819779157160220352e-03 -2.170371518763650892e-02 3.586044721784666794e-02 -1.159005385790040643e-02 -2.186193321654193722e-02 8.066400660510559704e-04 -1.879801001391799564e-02 3.559248133331381625e-02 -3.994963687670349730e-04 6.187421866973952939e-03 -1.115444375273100984e-17 4.836001208934827700e-04 2.973315452453241785e-03 -1.282392814050977922e-02 -6.587954905988898623e-18 -8.544056931644020136e-03 -5.780838956358853170e-03 -1.222017005617394986e-02 -8.288993628975654254e-03 5.587387644413730620e-02 2.940381519870647470e-02 -4.139494859586625476e-03 -4.815413271730375339e-03 -8.128424737703338065e-03 -3.092665412223065229e-03 6.358335233390814689e-02 3.434002437291518162e-02 2.009938950304707150e-01 9.903680333190988117e-02 6.888960713892886001e-02 7.530386302914301999e-02 5.994236787571494834e-02 4.922388945931507193e-02 2.381664574627622571e-02 8.576642919628972572e-02 7.430862203111272823e-02 3.026946603595706813e-03 2.260442701688597858e-02 3.237367366238957429e-05 4.435460596884787110e-01 8.552212707024484661e-01 5.607289168181850414e-01 1.367511034230499412e-02 3.627028562628643871e-02 6.830134912108103140e-02 1.405746235319421666e-01 2.386412380641737135e-02 3.824180377016853094e-02 6.148375546655952123e-02 5.870368302371723468e-01 8.462408088071655676e-01 6.214131285004902416e-01 2.483506348255194122e-03 8.850933964264798282e-03 1.141350914656929193e-02 6.260696271399223897e-03 7.209327553107958706e-04 2.079493054919206996e-02 4.344852842238027327e-03 1.976838851743503017e-02 3.774125344239725836e-03 2.468313410581671544e-02 1.170377024827244650e-02 3.252109856596394066e-02 7.344175781334247866e-01 8.340558517067592215e-01 7.691903220763312010e-01 1.881653340310955255e-02 1.567831436419013166e-02 4.688242182693073967e-02 3.323859006819898654e-02 3.989070782994771913e-02 1.572046548454271095e-02 2.542193839356451240e-02 1.835584633276252792e-02 4.140096297192434605e-02 1.450511390908548921e-02 2.214519056132420796e-02 1.513932197362154002e-02 1.959185815877432721e-02 5.232665272060343063e-03 2.227742070327043133e-01 8.967199822712272894e-01 2.439183539248834243e-01 2.281767268555982484e-02 8.099989937124936479e-02 1.307314648973953997e-01 4.301567044605241613e-02 3.305551382375103257e-03
2.745260003509032987e-01 2.439255323335328199e-02 2.190113463524515688e-03 1.344231212336196787e-03 2.573854305255133222e-03 3.341900747369471077e-03 1.031399253907830426e-03 3.251269870477745648e-03 5.251336540658352401e-04 2.235874002124820393e-03 1.529592452947670274e-02 2.728092895799784715e-03 2.432802144743330397e-02 1.683187423135172611e-02 3.331627172669272136e-02 2.289804123586530217e-03 2.859909912220119643e-03 6.670481968800860521e-03 1.169089618287530961e-02 8.407078808026930208e-03 1.103264223011122813e-01 -1.108741808736414258e-01 -4.511729186066510203e-03 -1.011721721531594065e-01 4.562969119707849935e-02 8.060477005568005883e-05 1.375403786053229840e-02 5.208881087939186416e-02 1.205659828935569215e-02 2.887400753101489193e-02 7.414533594689953766e-02 6.371854438447677182e-02 7.804387978414806248e-02 5.279346061131617862e-02 6.076888987719883395e-02 2.793287547621403286e-02 1.038610632344954578e-02 7.020035870625182528e-02 8.426966684688305675e-02 2.884542769820502145e-02 6.706818032575592853e-18 5.764808771864916859e-02 -1.238223503476355500e-02 -5.428894031767134143e-03 -5.585874841451760053e-03 1.907575651124179261e-01 2.188907233212836478e-01 2.789217673837914857e-01 2.180414019778581861e-02 -7.282101157290542107e-05 -1.234615685749562708e-03 3.244397138287898746e-02 3.584858162435310830e-02 5.213683009280584146e-02 8.382958864827302903e-02 8.608764535646187821e-02 9.434851495878002203e-02 2.290979590790694354e-01 7.718458581671067131e-02 1.652910025814968709e-01 -2.234384773939984692e-01 -6.254514092842057690e-05 -7.853064951550051132e-02 1.532766240592441445e-03 -1.326560881083019283e-01 3.381611282173010991e-02 3.622949931406635460e-02 2.889160628831297789e-02 -2.596812275997607486e-02 -4.060738345195408325e-04 4.039004707910460318e-02 3.011562757422518721e-02 1.885314829190797101e-03 2.766601083537964523e-02 -1.187992218283883596e-05 2.574780767679363952e-02 1.410358894164218807e-02 2.151608812258985412e-02 7.772055771842017174e-17 -1.205524645655099182e-02 -1.892519069270575185e-02 1.163807687634079713e-01 2.211011977834924647e-01 5.200567919895234675e-02 7.686764410892375776e-02 6.288685502561537025e-17 3.476114139408853498e-02 -3.182032833977520292e-03 9.205256279570882272e-03 -1.997757670389657160e-03 4.572563478346221982e-02 -2.895926638944580026e-03 -1.281347691524529001e-03 -1.533163665931322299e-02 -1.038747937168714455e-14 -1.132390371566354900e-02 9.733644109888061821e-03 -4.392729729559828621e-04 5.860173404360929277e-04 -5.348929831163108661e-02 3.465150752594738873e-03 2.203351495538307097e-03 -2.678175152531703067e-03 2.590589371635186947e-02 5.071843269025291812e-02 -1.038903928437951152e-01 -7.424914595757531321e-03 -2.675728150566883227e-03 3.200041765904349748e-03 2.723233405349128645e-04 1.183405570399752852e-05 2.352281156137848994e-02 2.314766078422188234e-02 -3.589396890836421550e-02 9.108214675458598669e-03 -2.935338685144720655e-02 -1.901070885101981221e-03 -1.502599304192004520e-02 -6.843168600926147205e-02 1.156148832771117328e-03 5.837310456058092401e-01 3.552392354313680944e-01 2.428058875285871165e-02 7.378441234420848741e-04 1.708359023331288556e-02 2.976483218352623996e-03 1.960016173283186253e-02 4.360883022717082849e-03 3.344654631671326733e-02 2.184190329300254516e-03 2.037732014092604599e-02 2.412159599746330815e-03 2.708840122716641916e-02 2.774067536133745931e-02 4.400466476159536205e-02 1.844649706101008160e-02 3.249853474409385851e-02 1.269696490202653662e-02 3.499582719628600846e-02 5.569532959232156855e-02 1.149271777116443599e-01 4.514419544639959869e-01 5.055514610037202816e-01 9.412679662450321727e-01 1.578717007190625493e-02 3.471395351653530637e-02 6.042285671020251880e-02 6.024200525635622916e-02 3.482602537621667244e-02 4.331096956662329467e-02 8.039045942811694212e-02 5.865797037042255746e-02 7.002458988382528893e-02 7.176412064255134615e-02 4.007827401433271958e-02 1.577711778981913393e-02 2.727537551488246406e-02 1.108261918861330221e-02 2.462759628091796063e-02 4.374287019965546718e-03 2.324860909118976922e-03 8.363069906546758987e-04 5.154852960589393351e-01 6.599597976904585384e-01 4.169729103124966696e-01 6.613345653272832692e-02 9.367570568204545978e-02 1.418023670950780135e-01 2.037351826832870250e-01 3.128170741690320389e-01 1.410570239161942352e-01 7.717930845064550037e-04 2.044442691633802101e-02 1.819722740176706391e-02 5.896647446249363850e-02 3.796761873275856375e-02 1.254228012395945113e-01 1.040980606813379644e-01 7.753266424893272279e-02 1.641895722043338821e-02
1.439082908976282993e-01 -5.914017050574189860e-02 6.146002883395412092e-02 5.212451237436896423e-02 1.397542324828576592e-02 1.742205844506685211e-02 5.191562130282003501e-02 9.378434373630013809e-03 4.430409989610135935e-02 4.809693820330762648e-02 5.385742872124416741e-02 1.208463034685943946e-02 1.763924689376262719e-02 9.668444964752543475e-03 4.631661967353044307e-02 2.441329909752072355e-02 -2.478763633616877238e-01 -1.906884597315261776e-01 -1.683796633090650641e-01 1.332418603610303898e-02 4.950977954206874088e-02 2.582080274979383500e-03 2.011419361115273849e-02 3.187769394071942632e-03 2.682386004524811660e-03 1.545423687920929878e-02 5.104406017856518048e-03 2.613800149068675263e-03 9.002328575401955663e-03 2.100319121097758354e-03 4.011211089499425869e-03 1.233502385991433470e-02 6.060232587456134985e-04 2.625865890460713642e-03 6.598150471976644942e-03 -6.987366991827465013e-01 -1.746124723902006681e-01 -9.049501738819452912e-02 3.478763606551497217e-02 1.318769229227922599e-02 2.160389304589703866e-02 1.635080915146342947e-02 3.308545061429619266e-02 2.218590017835996887e-03 2.544924217433235797e-02 3.818905182998161979e-02 3.558458810079623280e-02 4.954544274274258649e-03 8.632962337113593218e-03 1.253195477746606515e-02 1.660852716009876898e-02 1.457879903476923256e-02 1.324986101268821846e-02 3.013984135983173007e-03 2.200830423242704226e-02 -8.424489266429868034e-03 -2.162480372966365735e-02 -6.711023048427437743e-02 3.911540002380420322e-01 1.212887820111221288e-01 -1.209370176730888424e-01 4.771252379039046809e-02 6.018833293120438854e-02 3.266862135312271348e-02 -1.502707281778903640e-04 1.228290981101317324e-07 5.395555131043595042e-03 2.596113545062957509e-02 2.420923013334154439e-02 7.265888227503408334e-03 2.059989463954644152e-03 -1.674432714612962775e-03 4.952540752934994801e-03 -4.786485128080332496e-02 -1.668256987001423899e-02 -2.914044471739381248e-02 3.196090888780529771e-02 -2.069804713446914482e-02 2.145380676745953772e-04 1.524778544283062441e-02 -1.965463208096164936e-06 2.213401523663253989e-02 1.906299193731891481e-02 -3.605317706816540870e-03 2.475464463999915363e-02 -1.319134452250118140e-18 3.222195641048888892e-02 1.520509448088516039e-17 -1.048106986742619730e-06 -1.890180915074253336e-04 2.838534701152120840e-17 1.478672529925986225e-03 -1.334174753129706856e-03 -6.277795141682129820e-02 -1.931511133325575225e-02 -1.498043750555282973e-02 -6.079168041438625806e-16 -9.906980297156312704e-03 -9.979537254676744407e-02 -2.124317569937600719e-03 1.499673317070947427e-02 5.909263056415991339e-03 7.285757514266326049e-03 2.318621458463314411e-02 8.952744554332592308e-03 -1.266758610656068193e-02 -2.226536449164870715e-02 -2.071266750506198847e-03 3.019475148850610420e-03 2.201683530309153994e-02 -4.426970013253025142e-07 1.439214492692210455e-03 1.795153562011551050e-17 -8.046499280734697459e-05 9.855674786914368768e-04 5.107668782055759665e-02 1.857896765788617788e-02 8.670909268884230414e-02 2.005210853153399642e-02 1.370249129744521011e-01 6.437857460162168532e-01 4.268923224447118692e-01 1.544311195254670019e-02 2.044132665439397922e-02 3.033837029489292944e-02 6.306674180636273608e-02 8.652055244195119843e-02 1.717459469392973101e-02 3.758212260021049494e-02 2.420694687880901863e-02 5.050374707921427564e-02 1.021573265914445250e-02 1.549412343444360809e-02 1.306640672430834337e-02 2.413639525636649200e-02 4.863385954529767310e-03 1.922579617466984303e-01 7.519949166752902814e-01 3.458517964933796596e-01 6.084640518724802362e-03 1.616800484977112470e-02 6.111579995927437321e-04 3.986354178919319924e-02 9.092793507132242506e-03 1.687734304794008891e-02 1.054380170472488110e-02 9.610905649449656696e-03 4.954015740855649379e-03 1.880946033251654215e-02 5.109449778542543216e-03 1.266526883348800120e-02 6.474001398206439238e-03 2.081981403937245914e-02 1.178865668184087521e-02 3.422044096300105170e-02 7.614695135510005652e-01 7.996637568372290117e-01 7.744052686775425576e-01 2.323707525442652785e-02 2.029504715462859474e-02 3.805509780559642591e-02 3.524700712216673887e-02 4.490216699634543701e-02 1.373222729860458860e-02 2.345371967433709792e-02 2.663772832203548449e-02 5.249596067239339914e-02 2.345241141795081224e-02 2.194462768709054545e-02 2.384767823222775848e-02 3.096937774527074722e-02 5.917651336885546703e-03 1.499358806503718958e-02 5.294979135787673645e-03 4.455654063895902323e-02 6.350245804944437689e-01 3.583533393692961422e-01 8.454480976411852255e-01 -4.126097483788691567e-01 -7.720794071632386402e-01
2.622909865455171907e-01 2.482826555011813374e-02 6.354089730225620687e-03 2.780365817905751727e-03 4.467328442473390837e-03 4.540272944302494416e-03 2.007330208817400315e-03 9.614278330928418756e-03 8.132967726608897677e-03 2.613947001316678899e-04 7.421685887824648782e-03 4.470265998614443158e-04 1.365941316211637981e-02 3.744187223813103677e-02 1.146754312939675807e-01 1.254995493768681393e-02 1.014679118796122347e-02 1.727545707357211469e-02 1.899919063745626399e-02 2.068278801221478530e-02 -2.081664142281222374e-01 -1.559396868554907001e-01 -1.703127714648277891e-01 3.627356093785011337e-02 2.821644955336187199e-01 2.545869846749249793e-02 6.745028068086800221e-02 5.382691207546257128e-02 1.416317356594961052e-01 7.748261345524136423e-02 1.607920505603364081e-01 1.210005908893332649e-01 1.350289397650832612e-01 9.030010830631431118e-02 7.558326551180871711e-02 3.589144784259073323e-02 1.823597376468571660e-02 9.705392423998419893e-02 -3.936979014039593272e-02 6.380452161729212052e-04 -3.347220049737784781e-02 9.345815918314348547e-03 1.093236892530754546e-02 2.357424837862230962e-02 4.237449311874724733e-02 4.501893608046794032e-02 3.819702239222026036e-02 2.859736578032003214e-03 1.403164740125970207e-02 1.875550096799227448e-02 -3.402133603692598842e-03 -1.055612503222645195e-02 1.327978984998511025e-01 5.142461324649514470e-02 1.384733636325128636e-01 2.085929277741784549e-01 1.131834321052264947e-01 1.906492196631040803e-01 1.064074344389393523e-01 7.320662554638836250e-02 -2.169640671899546180e-01 -3.195723482574027319e-03 9.321077167958663132e-03 2.980576120079566141e-02 3.236490673518353789e-02 1.158077692353602466e-02 5.192798451540927424e-02 2.441911809534953043e-02 1.471469223470210468e-02 5.088331070920134105e-04 1.287316331693616792e-04 1.789456474046340062e-02 1.030383279555834378e-03 -1.864984912151284452e-02 9.589866796820771533e-02 4.848049196261865375e-04 1.623896378806674937e-02 3.520053321396336005e-02 9.556620221558528513e-03 -8.607031111241203111e-03 2.657994853907445082e-02 -2.273202858614073105e-02 5.173064366897885341e-04 1.465717107059067159e-01 2.922026627614250638e-02 2.109898903852378341e-02 -4.385968799262368120e-18 -1.415818194059225918e-02 2.739266930613371356e-02 -8.351294823861984268e-03 4.967500556317302335e-02 -7.722797610519276101e-02 -2.022441368747275158e-03 -2.194174489797164063e-02 -1.690813833076963376e-09 3.564045520095023012e-02 -6.646700428595592181e-02 7.534139494566181004e-03 5.247907946815381919e-02 2.780620341590045452e-02 3.548082665483404965e-04 -1.418317793881267511e-02 7.945178213325531508e-03 -8.115248816870300821e-03 1.288032630474184384e-02 -2.914752230723232154e-02 2.581994786744876494e-02 -4.648074868679520633e-02 -7.224897375528111301e-04 -6.187294827678878339e-02 -1.033675341345583962e-02 -1.710365914350126665e-02 -1.245650177094423557e-02 8.279506015425910914e-02 7.113978473527730016e-03 -2.794750254655033184e-02 -6.058003077649352919e-02 4.302108241317477433e-02 4.578578697104088852e-02 4.454848161193745298e-02 6.005889182094257261e-01 3.631238705061246574e-01 4.480653256064755924e-02 3.633204480490706805e-03 1.502066812320366866e-02 8.723871220190729811e-04 1.596382703071263717e-02 2.626885900125523407e-03 1.511325065901436039e-02 1.088291722597517497e-02 1.548053660187542237e-02 7.558060249065390833e-03 3.923006093432345603e-02 1.871240051972748383e-02 4.246722365911207581e-02 3.963305054276175868e-03 2.034683636865613002e-02 1.193290999266519506e-02 1.901992917033954247e-02 4.933687640467245179e-03 1.928508957814528135e-01 8.259705112971820196e-01 3.477161503804083176e-01 4.334009322440053602e-03 7.812583126466229530e-03 2.132616226613855484e-02 4.818893106811367871e-02 3.358308676129528186e-02 3.036648327461760263e-02 5.781515125610745681e-02 8.473941975193315101e-02 6.374081607849546582e-02 6.437744557355258579e-02 6.436079852579522365e-02 4.466657815103516199e-02 7.958718391554249136e-03 1.379072517038636495e-02 6.880960313737534140e-01 8.246368967770969505e-01 8.018635157627230781e-01 6.833409379420938602e-03 1.138949404221364925e-02 3.204423913479568370e-02 1.901540805035188339e-02 4.692136080506791734e-02 5.864539948121124530e-02 5.864629444313587681e-02 1.341338831306335307e-02 2.466825590406994048e-02 9.014587063375236603e-03 1.659539489244129529e-01 4.623037759486613996e-01 3.222734535325677552e-01 9.983807121837894732e-03 4.831521517191315201e-02 5.114752712481787644e-02 1.292274473360629783e-01 4.787634081150460297e-02 4.605534344294809346e-02 3.803311852664698761e-02
1.277275206255147066e-01 -5.657818991392268154e-02 4.651427917434039494e-02 2.105044822462006499e-02 3.169901630812328036e-02 3.051372571123873306e-02 8.361116208676028227e-02 3.432951835157195541e-02 2.482614878920463494e-02 3.681180742533834982e-02 6.049375848673217010e-02 4.228267804440710015e-02 1.843975722505906273e-02 2.590369409555219604e-02 4.830339504454934929e-02 4.684487448664989562e-02 2.263729843897150651e-01 5.189370476804647248e-02 5.855937525614659384e-02 -8.944447056358198478e-02 -1.222260204778735510e-01 -7.034129610455930326e-02 4.440685005357473784e-03 8.357234088110291476e-03 5.634107567558078837e-03 1.297435955889887291e-02 3.475570058718584210e-02 8.206414313471729893e-03 1.490116119504278996e-08 7.515884129729905223e-04 4.060788233951481523e-03 1.083024706523923652e-04 7.303460220160967048e-03 3.086845793748548613e-03 4.750704073992336146e-05 9.469246024890403831e-18 4.106330104881399622e-03 1.352959860666548379e-02 1.931748082134649921e-02 4.452257523684864660e-02 5.091700324321624810e-02 3.986536425954515656e-02 2.098376808990339015e-02 1.654797603700829470e-02 2.508048124797438230e-02 6.195942137982390213e-02 6.298023850199950535e-02 3.369826857266676468e-02 1.931660690616365458e-02 1.403126353507424054e-02 -2.725776412194134794e-04 2.124363847287496707e-02 4.622951759969569363e-02 7.510033040296268392e-02 1.012955384980886719e-01 8.763731652169882980e-02 9.774103801127924374e-02 7.342358540591466254e-02 5.720982392678596734e-02 1.002521836578445513e-01 -7.174537117578447254e-02 2.497379074378127881e-02 3.357078870321546921e-03 -2.763055163402366315e-02 -5.535952192495987700e-04 -2.879272971269606837e-02 -7.166592290479628128e-18 2.861593505657257161e-02 3.416679889587265206e-02 4.324462576433843464e-02 7.872262269319949685e-03 3.568277987753085828e-03 3.744002996521827303e-02 -6.401745334842865744e-18 3.282473503511190022e-02 -1.188851461537970703e-03 -3.429713685210129814e-02 3.779055691318525256e-02 -1.038647789080792139e-01 9.438448212145072666e-03 5.124909004494116363e-17 -9.546113772727887747e-02 4.170270295716112935e-02 -1.051436768151507994e-01 2.232004376789201161e-02 -5.317062332626565401e-04 2.640255696677971942e-02 1.716968308285500133e-03 4.869901876976332616e-02 -3.038995541056453698e-02 -5.052709985834869313e-03 2.941877032313998126e-03 2.242277032749305857e-02 -1.614527239905542977e-01 1.025786796744043543e-02 1.331031079904731150e-02 1.032966231145134202e-02 9.874943192885548421e-03 -3.261280939885328328e-02 -6.550559132625571402e-04 1.989669490837746302e-02 -1.086272536661647722e-16 3.407070023324589546e-03 1.414338825462479413e-03 4.342494630704298725e-02 -7.555778116834248753e-17 3.259952299087130051e-02 1.878659231703913579e-02 2.166311453953196145e-02 4.891270967381901313e-02 2.877967314498688575e-02 7.538011216719431523e-04 9.821035579849111963e-03 -7.248358110453848036e-02 1.226213468065123942e-02 -4.183991757697969743e-02 -7.715463299142221940e-03 1.260348516554367237e-17 -1.442493644539124533e-03 6.530148435870079121e-04 6.670474912571916493e-01 4.105858461809611670e-01 3.668018807970278850e-02 2.175812241527142615e-02 2.554751384712396559e-02 2.696738606627127263e-02 4.811005402915687762e-02 2.912333095923866962e-03 2.619023355094255226e-02 2.052080983816766124e-02 3.697302986347394266e-02 2.082565158445361803e-02 4.132288961864470644e-02 3.188707232335554614e-02 3.354067087244413936e-02 7.498275175216684296e-02 1.519477829169758043e-01 4.944773334465128733e-02 1.922695994186317606e-02 5.473638320255783674e-01 7.320934278236961301e-01 7.028185520256940988e-01 5.381434157688027436e-02 2.529229749611066475e-02 3.809320103956562176e-02 3.064899975731325346e-02 2.528944094388649658e-02 8.101010650889944875e-03 1.834962519388223454e-02 2.476248256334330836e-03 1.504786469815915656e-02 4.939957472652352181e-03 2.199242834616190953e-02 2.282792425892264943e-01 4.237108408730582143e-01 8.145649830106307387e-01 1.458126333654082875e-02 3.351031666285923111e-03 1.967049922574828566e-02 2.365272182050936686e-02 4.848560407367918934e-02 3.696046598242394565e-02 5.470558263717720943e-02 2.876001738396050111e-02 4.446614025773377998e-02 3.547964496884981056e-02 4.004517289793602108e-02 2.834689708201144909e-03 2.259034556881454325e-01 4.655545514878137725e-01 1.468932531128126218e-01 2.317199627895793340e-03 1.048200636106819995e-02 2.029411716385740028e-02 5.974055990968971108e-02 5.676991642643477998e-02 1.088495301058222814e-01 1.095091939945748438e-01 9.149688407021810965e-02 4.114748829744902664e-02
1.251318967524799042e-01 -7.843228973425818429e-02 5.448540736844181137e-02 2.107841420008303712e-02 2.386269327287203826e-02 1.592145812325723025e-02 6.716251887621325911e-02 1.904366257283797437e-02 3.942230643605558471e-02 3.717038029978041863e-02 5.311478879983852486e-02 1.401949515982703343e-02 9.505290305971416179e-03 3.001468341386438407e-02 5.313058976700498620e-02 5.697764363188500408e-02 1.818211481778359839e-01 4.214200284535445790e-02 2.659629731970972188e-02 -1.675797649680880186e-01 -4.011665828222563818e-02 -6.845237110544422077e-02 4.068867624901430224e-02 6.588045959329397877e-03 1.389637062161557142e-02 2.247208217909861527e-02 1.285501153918953143e-01 3.770625205507712732e-04 3.757625082711398064e-17 1.296453787532756601e-04 9.817346584114704754e-04 4.497666754118595143e-04 1.345960550987944718e-03 4.568383164967083909e-04 -1.651621750358084895e-01 -1.429826378811359655e-01 -7.312868120539586791e-02 4.440858614369042751e-02 3.775296332573016556e-02 6.726130375084043989e-03 2.398256618384833269e-02 4.740838847370282966e-02 6.068024558364463333e-02 2.892476202361695531e-02 3.146681819270975844e-02 2.141400193065120389e-02 4.340130754104919164e-02 2.003259820726445081e-02 3.538876504645623033e-02 2.303768857511643772e-02 2.621833792012276398e-02 2.246149609580880974e-02 5.214360404563633117e-02 5.914828445329457784e-02 1.017046670742932507e-01 4.207173758969327826e-02 9.021311673626328242e-02 7.552441010285920742e-02 6.796339552166896780e-02 6.706874216145514156e-02 -7.272910133078665240e-02 1.172280003212197198e-02 6.677051448964838000e-04 -2.817105061932722795e-02 3.302654745008409137e-02 -2.008025184135534685e-02 -7.139092388615106335e-04 7.647774203219276368e-02 4.701225552835641697e-02 2.110835169528762351e-02 5.635675994441756735e-04 4.110649208714206582e-03 3.095020475557270709e-02 -7.156411230366127758e-03 2.787406207003361933e-02 -3.940523498087757375e-04 -3.439453624340967075e-02 3.014739301912128591e-02 -4.903136107263234705e-02 6.918385774411769940e-02 9.133800069601696880e-03 8.062476783227393376e-03 -9.883643475936761549e-17 4.969739745622820531e-02 3.975481085186950703e-04 5.027483032672367541e-02 9.992529459823538615e-03 5.269752492304434180e-18 3.020170561568950515e-02 -8.126513175433757938e-03 -1.515614422650005469e-03 2.697930881818045960e-03 3.392364939383441663e-02 -1.568855007509129245e-03 -3.239851473858073455e-02 -3.807243342766247068e-02 2.117785725906492708e-02 4.022768391486391398e-03 1.397108677439747298e-03 -8.225648942160105855e-05 2.441924041570530703e-03 1.307182680987633606e-02 6.060520091139477705e-02 -1.525132562747008938e-02 5.578924223390202238e-18 9.627863036329790914e-04 -1.469136025595245117e-03 2.643784383043050815e-02 3.966648020703027883e-02 -3.065745008990553022e-04 1.834112833855920319e-02 2.686690888771968086e-02 8.412206903722182355e-03 -3.997215702233810307e-03 2.547701236459278679e-02 -1.637168004559566917e-02 -1.778238365860745906e-03 -1.504380398183857454e-03 -4.668950765197173581e-18 7.343544535979826066e-06 6.642695924816920394e-01 3.545553246369769762e-01 3.637687051143752986e-02 2.981520356664429244e-02 1.966919019760524426e-02 3.593755168179658954e-02 4.948408172557022117e-02 1.333400890097006380e-02 3.108528263342109216e-02 4.321278999757388606e-02 5.707574321748465351e-02 1.845760336513331545e-02 2.625342998876418338e-02 1.744134117881958324e-02 4.031353023710462086e-02 7.474400890119219099e-02 1.656427502482988867e-01 5.641149372839899395e-02 3.201819223361125621e-02 5.767835335366112703e-01 7.422822821963592776e-01 7.247376958139966074e-01 3.225632713382573652e-02 5.469429986665231766e-03 2.520587250870399282e-02 1.566536501750528249e-02 2.089695962318573880e-02 4.391266364736134964e-03 1.556426680568981369e-02 5.954731304125856876e-03 1.502890230738953073e-02 2.000813066013494741e-03 2.124327472066561001e-02 2.515729651794400973e-02 3.116492361874212413e-01 1.222059739534926193e+00 5.318027648406858932e-01 9.930074088116502481e-03 1.404301613884779976e-02 2.354425428122379482e-02 6.185554529010982988e-02 3.418244632647687853e-02 5.056407096619931069e-02 1.569160030690365526e-02 3.643091961574759091e-02 5.461219699269845990e-02 5.523580032807631768e-02 8.843932851576986887e-02 2.491233795844483290e-01 3.505559164077700629e-01 1.484384399941969801e-01 4.332952076122975160e-04 9.897389011538450243e-03 9.595302141634835802e-03 6.791038869584854754e-02 4.508381558066026917e-02 1.185664496905260018e-01 1.445991417697987391e-01 7.846615357236418065e-02 2.659438053418220621e-02
1.455023763754538235e-01 -6.359915899776262538e-02 5.560341320981761204e-02 3.101503013658822583e-02 2.059579831346053033e-02 5.799091486455518196e-02 7.712565142708774357e-02 1.874022284248938811e-02 4.821353528568002311e-02 4.386177349156403227e-02 5.794786360197123204e-02 4.904269375665462616e-02 1.248638857570177722e-02 3.071332376158936520e-02 4.705398022533455876e-02 5.913450578669286056e-02 2.045347529894752658e-01 6.457307170266275709e-02 3.329448631787389201e-02 -8.131494949096136626e-02 -1.032247005510069371e-01 -2.470316249413525081e-02 2.375894689376443700e-02 1.068757696352700369e-02 3.963567665742712132e-03 8.539821525915838180e-03 2.786026248510228495e-02 2.100005209155879902e-04 4.202515953420308940e-04 7.728597439036629345e-04 2.535934464225408219e-03 4.684660904864638292e-03 9.141461860761129392e-18 3.034390934155193394e-03 1.431991193412573450e-02 4.292719390280219129e-03 9.086930708788420162e-06 9.722427347900238631e-03 1.613192777707856893e-02 3.478021035064530336e-02 2.587063106666002976e-02 3.106953973039358607e-02 8.185983513591810548e-03 2.107867539187124883e-03 2.244490997063654930e-02 8.585537457185030141e-02 1.108719541225541977e-01 5.257064880487061914e-02 5.725677110697689927e-03 1.596049573797062374e-03 -1.984547119802473197e-02 2.010148841485924834e-02 3.024939217214545409e-02 7.464923191809436087e-02 9.912101991323951644e-02 7.938968652786022873e-02 7.120815652470122159e-02 1.573831733409307432e-01 1.634342102127418483e-01 1.208228465640243110e-01 -7.305909575737817852e-02 -2.413794695547175571e-17 -1.347196573444694664e-02 -3.136990270785226798e-02 1.973266058191112948e-02 -5.759701273094301049e-02 -5.769062002606877482e-05 2.178810293869567691e-02 3.303265765597866461e-03 7.839855176695152694e-02 -5.662186709896337503e-03 2.103295181881984638e-02 2.925215254710081605e-02 -2.301720533063251681e-03 2.193024389318379633e-02 -3.073164789863668585e-03 -4.454244149680961634e-02 2.767550418919112118e-02 -5.997774206772418792e-02 1.941729316200339547e-02 1.538922742131536119e-02 -2.969868359209163269e-02 -2.404716007393914364e-16 8.964540955394721677e-03 9.587874373414694701e-03 9.850829383483141258e-03 3.873623873410796115e-02 3.520946943569010744e-03 -1.510276480432462425e-16 1.235099034785379878e-02 -1.702787373502535201e-03 1.126567341199972142e-04 2.408369259332036796e-03 1.079967053788872741e-02 -2.848437149822811271e-04 3.158786862814698393e-02 2.069021254269017701e-02 3.586548001216716688e-17 -1.721596225984001194e-03 1.926587878970801096e-02 8.506508916462381109e-02 -1.413919621033435858e-03 1.831038325157049182e-03 1.607716923392411498e-02 -9.922849391516270004e-03 -2.304150158719547523e-02 4.420077371233083011e-18 -2.325340638921717575e-03 2.619372652189937492e-02 3.522540518049729896e-02 1.416882791844178502e-03 1.023011142768429430e-02 7.625618863778982348e-02 -2.358912489864710763e-02 1.715708202012572459e-02 -2.264823155636932345e-02 -4.210679125148384435e-04 -1.534768733531553624e-02 4.006670057205613013e-03 3.069875847287111686e-03 6.521665169360407743e-01 4.075836857304859229e-01 4.528201799307522701e-02 1.090538192407286555e-02 1.765813839300913249e-02 4.695307590442161783e-02 5.564146681735156891e-02 1.201995622042587031e-02 2.036940117335866302e-02 3.470722865082710024e-02 5.230699912122455802e-02 3.711468251683693043e-02 2.860532974142953114e-02 2.674518245963567603e-02 3.738594415831734963e-02 5.173700750201312010e-02 1.549762225356220990e-01 4.692152222564374697e-02 2.888189051749000258e-02 5.438163743455481924e-01 7.139299498238301789e-01 7.664286619982495496e-01 3.727096733290173602e-02 2.073943856640821787e-02 2.964367319237087503e-02 2.111344847313658540e-02 2.172213811853518362e-02 2.483246054128322960e-04 1.945528974347831425e-02 7.824905605771477956e-04 1.179993198849313245e-02 1.848419854313751443e-02 1.770611262901644611e-02 4.989387372091808395e-04 1.405037539205150100e-02 6.850033685984570960e-04 6.510248685457315637e-03 2.196649524132074024e-02 4.027078222408803088e-02 6.781740133191843656e-01 6.588562388852704510e-01 6.651240264243977185e-01 6.257552202710807421e-02 7.337155453218836831e-02 6.785008605558955075e-02 8.404941406926327852e-02 4.603789862916513043e-02 1.320054936559458998e-02 2.033150058301550489e-01 4.588821691679526027e-01 1.264167589423046967e-01 2.166749159001838190e-02 1.665818891510421201e-02 3.352413959564985030e-02 5.925488851393940354e-02 3.172814261281419607e-02 1.147942215156404766e-01 1.127766697505424021e-01 6.757280172165816556e-02 1.844190583216640238e-03
1.325901858028424307e-01 -6.191970159399845519e-02 4.837483341687408983e-02 3.578159119023904822e-02 4.996111068494527713e-02 3.893621442002635075e-02 1.060453710965461455e-01 3.491940569738555961e-02 4.617881739331174129e-02 4.781262205493929524e-02 6.951333897213712865e-02 5.151938575883969335e-02 2.370920304666693582e-02 5.492345014960588190e-02 7.057026490754993164e-02 1.255688691610292451e-01 2.198910042725185643e-01 6.394335448093159402e-02 4.033728351711245164e-02 3.579381891636178314e-02 -1.323932865293760697e-01 -3.599180136054901946e-01 -2.621330647785333201e-01 2.504803161849950867e-03 7.337009574444493514e-04 1.619155088558475036e-02 2.037604051742755024e-02 1.596783816699033631e-02 3.414126492497261689e-04 1.271665842161562657e-03 1.035964331344389061e-02 1.321921690848984650e-02 -5.937169871314460640e-03 -2.314633736759330943e-02 -6.101790530798055501e-02 1.772090362139901176e-02 1.024367159694824841e-03 7.111213128966572573e-03 1.406871107882982398e-02 3.904355550184254904e-03 9.931424664730025692e-03 2.500486382626614448e-02 2.561869274095598600e-02 2.443524235345291302e-03 1.200005018908547813e-02 1.759840238573031485e-02 3.879777133815564411e-02 7.001056167098704958e-03 1.159549029709159781e-02 2.057959847880084375e-02 2.834222948851067292e-02 1.074266116153857649e-02 -1.638633467258503867e-02 -3.291465930415388152e-02 -4.171101349795199031e-02 1.051737297406552113e-01 9.184977738401604863e-02 2.408900065733354801e-01 4.140859447985930641e-01 1.214345412808421898e-01 -7.740568129336804215e-02 -3.046279856354956392e-03 8.950287834667439157e-02 -3.739297536790218285e-02 4.613990545193322884e-02 -4.646615153860385317e-02 -3.438614136440377653e-03 3.834143484010798814e-03 9.352571834268585483e-04 9.136775034692902475e-02 8.062445702748145815e-05 8.325954489873039696e-03 5.563340276796327130e-03 -1.804475479366226751e-02 5.107917411723619100e-02 -2.198230089033048097e-03 -3.633065852223861525e-02 1.933688513125219124e-02 -2.347665172085811047e-02 -1.951572927432252619e-17 3.368746344632338771e-04 -6.876935912166770581e-02 3.765309749404675875e-17 -3.032920062534320491e-02 8.411499640226532959e-02 -2.341675262575472483e-02 3.725660241297008152e-02 6.214513084178922995e-06 -8.539977688069299520e-18 1.283062946510271293e-02 6.438476161025842234e-03 6.063241072160536257e-04 -2.675184942237251312e-02 -1.885042615175872667e-03 6.872912903742718800e-03 1.266392368905196360e-02 -9.389662662258480702e-03 -1.530114137433738605e-03 1.242342896260549133e-04 -5.502233052385727304e-03 4.028022856703629938e-03 -8.355237648077064561e-18 7.837420135281309048e-04 1.155288635253789130e-02 6.972774811833865917e-03 -4.599854759137866726e-04 -1.263123387784750168e-03 -5.068177461979475494e-17 7.980227175482565438e-02 1.248745345783030571e-03 7.178603436100126911e-02 5.179223752680350845e-02 -2.572624396760391993e-02 -1.897292423060176897e-02 -8.840063674811089386e-03 1.055122114961871971e-03 6.948524646894847778e-02 1.984199627729612767e-02 6.896150340329451622e-02 7.735609117932337742e-02 6.572560821827577726e-01 3.850530721251498312e-01 5.461961166478414981e-02 4.919899181871984634e-02 1.970073145484552840e-02 4.307369539201785780e-02 6.708099212399808164e-02 1.442318292695368188e-02 3.235264529575997139e-02 2.414776783030965583e-14 5.612453038520508891e-02 2.244030581682417841e-02 2.582945495417397069e-02 1.172390575165518575e-02 2.617598420846056761e-02 7.925100386956487886e-02 1.693561659921240525e-01 9.504005835077006248e-02 3.308457004561763531e-02 7.747628536019316933e-03 5.181078797137020908e-01 8.234044343075860306e-01 6.588203748596005438e-01 7.764161066536610922e-02 3.561599112490122787e-02 4.138030123805267607e-02 5.818778661428538473e-02 2.976346349531698923e-02 1.010572669352923988e-02 5.830527090714119655e-03 9.432828450481918134e-03 6.448873307423831983e-03 2.885851757910712911e-01 9.602693657754708401e-01 2.754108350208598344e-01 3.386344580398865478e-04 1.164860740529819957e-02 4.385602977026162828e-03 2.087413634725544462e-02 9.333617598959842443e-03 3.769309753519515388e-02 3.103052169590918632e-02 4.779030229046775380e-02 8.599833203034572965e-03 3.322239265495381677e-02 2.045392091922197328e-02 4.942291787505688483e-02 3.080663068584397341e-02 3.044750559707564211e-02 1.238025146023506370e-02 2.848851771853316839e-02 1.186643844957717972e-05 7.384632930852780275e-01 3.481782874225024527e-01 3.074468415605877691e-01 3.467788103507584568e-02 8.024585104270907931e-02 9.526856564580567233e-02 3.729824467640519697e-02 8.300490467185953763e-04
1.282741691373444626e-01 -5.331672204261462406e-02 6.155002891071302551e-02 4.866391671865270829e-02 4.257045744923374647e-02 3.856030687723253914e-02 5.913113627753200513e-02 1.041123524697833667e-02 3.845044324692493271e-02 3.590667759522987029e-02 9.682718785015932694e-02 5.318819864255346780e-02 2.761461415356153415e-02 2.309899146947255039e-02 2.724078994500024262e-02 6.908685684725378306e-02 1.706430923006909406e-01 5.777601214922126194e-02 1.508404439696655469e-02 3.295463669547658847e-02 5.343466688425022038e-02 4.921354419422242082e-02 2.631665452988572393e-02 1.554779373486574537e-02 3.374422931267896317e-03 7.940374116667923629e-03 1.570473589030929595e-02 4.202117219032138785e-03 8.118575836729265138e-03 5.033815712342590808e-03 2.296161183237069913e-05 7.983719341044812415e-03 4.130716761236193427e-03 4.835773102924466500e-03 1.445128091197955822e-03 -1.175584747994002227e-01 -2.694824551861093043e-01 -7.126722278812053224e-02 3.386225314094631206e-02 1.210191446816553419e-02 2.994597277295845197e-02 4.643818632163856236e-02 2.956014958344109489e-02 2.253891276273460587e-02 3.195770996760453897e-02 5.476492737805047778e-02 8.564514144763275827e-02 3.253452566136687263e-02 1.676169895869021945e-02 2.589471469742819776e-02 3.968540950992999888e-02 1.334393129751835876e-02 9.761265335467202500e-03 1.896852755973873692e-02 3.840368800552887429e-02 2.942830748842654651e-02 -2.341029883400434775e-01 -1.687642236192423151e-01 -1.974088349318759705e-01 6.562374449154319278e-02 -6.133982091171267387e-02 7.540163684746658580e-04 9.563210837904904255e-03 -4.825849485942999839e-02 3.913165192288700633e-02 3.802381620206969046e-07 3.921427782182779903e-02 2.642166498577507711e-02 6.670899850225694983e-02 6.659130996850439951e-03 -2.611201771969986693e-03 2.868078238711747885e-02 2.267421684447720742e-02 -9.400813526225407918e-03 1.085727728059156377e-01 -1.269547172981817448e-02 -3.797534373543505481e-02 2.442341812686953453e-02 -4.184383475462159080e-02 -5.003139500280315635e-04 -7.549534744838245657e-04 6.587509850176181958e-02 1.882166137823797214e-02 6.445613044220929733e-02 2.750262048713119037e-05 1.773945920327058834e-01 -3.390755015247691310e-04 9.610986090575151727e-03 1.267155980905542833e-03 7.270694047852825588e-18 -3.033256687775404137e-05 1.499315429335788645e-02 1.418243018523939261e-02 -1.377623733946003667e-02 -3.199371876561351614e-03 -1.310047233033116536e-02 -6.935618463654150828e-03 -1.691802137586080959e-02 -1.064829811387386221e-02 -1.594182452118220877e-03 5.315909575288798108e-03 -1.079561771144300908e-02 4.190244412678947525e-03 2.158745822382604831e-02 -6.222147857609536476e-04 -8.481749847415545793e-18 -4.326391682451566356e-03 -8.610152883644552751e-03 -5.643033048263173396e-02 -4.202483556790586640e-03 -2.596105943417420764e-17 1.373338030877242419e-02 -3.174263500329748550e-02 -8.089363149242518347e-18 7.204253140022079307e-04 -5.273041872574495310e-03 2.363862389754803339e-02 1.091364521124232705e-02 -1.746613041303154701e-01 2.602202700342675090e-01 6.588940489188161997e-01 4.292670249216687939e-01 4.543118068747091509e-02 4.097402823936814598e-02 1.773830892309203461e-02 4.379427927750879912e-02 5.060234077998439911e-02 8.754113766110840564e-03 2.455866955799556814e-02 5.862897424417867071e-02 4.905113056234342017e-02 2.167839823320240333e-02 3.063685163464475308e-02 2.602211356167844977e-02 5.455406986659277341e-02 6.668647396110354009e-02 1.621212773143857477e-01 7.590013176918698756e-02 4.800885826655395400e-02 3.813258016619483493e-02 1.799064259848854111e-02 5.227151148422598537e-01 7.024379927516362088e-01 6.458422040458662305e-01 1.766040467078006104e-02 1.336491799088033311e-02 2.347695377473810532e-02 1.558251059678618725e-02 1.966941794603001628e-02 3.051422592943314922e-03 1.082452826741234914e-02 -2.745062719537753881e-02 2.082666936137377370e-02 4.626556084086845544e-03 3.091973812648222963e-02 7.354951756047735101e-01 7.537985989244140317e-01 6.908109803177812092e-01 3.260070841323009505e-02 2.217260343516211721e-02 5.214954092207094855e-02 4.296468158803276594e-02 4.316985664397494232e-02 2.893699574299662958e-03 2.982835746756906312e-02 3.597932788231020945e-02 6.898066664971407025e-02 1.852529021547820454e-02 1.916514143825811395e-02 1.099674053419153440e-02 3.291637940819007457e-02 6.357822511601655090e-03 1.610727932630500597e-02 2.290987691795403833e-02 5.960755025359286419e-02 6.526226665204815265e-03 1.758489432096626559e-01 7.399379055759333079e-01 4.690264506660309918e-01 8.466727560388316309e-01
2.704636888328966760e-01 1.078848772710168206e-02 1.160308583645676406e-02 4.266073357342037471e-04 2.512356773688188914e-03 1.007335339292074515e-03 -3.847308891019503296e-02 5.397833854548652645e-03 1.491342753767870531e-02 5.945471185332086399e-03 7.648126246478263029e-03 8.175186538993661420e-03 9.919897060577990977e-03 1.054172324050032215e-02 5.550566191927140680e-02 1.485964636340247370e-02 5.560246181859780298e-03 1.573617211771016072e-02 1.509834126905108299e-02 4.823218129727135234e-02 1.936080113950967851e-01 6.300965376101586024e-02 6.764946553996393011e-03 1.997209797593019892e-02 5.250080480966413249e-02 2.305311490329135793e-03 6.685520075260205364e-02 4.980711105029732993e-02 -3.696308372531273770e-03 9.744094242732079247e-02 1.496119991975556895e-01 1.496490748310744801e-01 2.116790884353596525e-01 1.140030320951846421e-01 1.362411711008527371e-01 6.258796564925797612e-02 8.564486275251062021e-04 4.768465002670266989e-02 -1.943543387143975912e-02 2.109426030647189830e-03 1.273843741666375734e-02 1.427182258332664322e-01 2.395457654933283442e-01 2.825985640066025484e-01 2.303882506304403832e-01 3.198490729052286818e-01 4.821678960606214370e-01 4.143699544624059733e-02 9.213403149658327962e-02 1.151250372546082715e-02 2.505528856300495016e-02 8.751107505758529831e-03 7.772540620413127647e-03 2.074952892240768468e-02 7.259334612971482525e-02 7.295806873316901253e-02 9.593814837424005648e-02 1.046823342541488150e-01 5.401411744497592454e-02 -6.732940093482427286e-04 -2.151748105367045638e-01 -3.225130156235708062e-03 -5.269458073012976002e-02 7.432902117872257353e-02 3.920727647623804130e-03 3.769004944282947772e-02 -1.368250542261467695e-02 1.299925295148399683e-02 2.429701717041047801e-02 2.308586337176886036e-02 -1.003140998230502186e-02 3.480640210373217580e-02 7.445783217682952482e-04 1.063434288068496048e-02 5.771099179165365724e-03 5.252733864165737114e-03 -5.063630444506340977e-03 6.140173758012815869e-02 1.033206426399519914e-03 -1.037636238140759483e-02 -3.073886297257860320e-02 4.758276396524174778e-02 -1.266121653713743293e-02 2.719792522152861963e-02 1.176729734414048295e-01 4.274697251319810321e-02 1.350580525889528418e-03 1.391649453322475272e-03 -9.256612214868416532e-18 -1.821867062798039573e-03 6.218399332119988604e-02 -3.098115924181227970e-02 -1.696576502951963732e-18 -1.122127414703560783e-01 6.403200163959514579e-02 2.100201737319565276e-02 9.125464804075664573e-04 8.308044660368342146e-04 3.804298687942599122e-02 7.336189447740164293e-02 -3.606517616397082043e-02 4.960176093056555685e-02 -6.064611257055135124e-02 2.109910365598778939e-02 -3.546773008966867813e-02 -1.004112222971298984e-01 -2.175253132181778626e-02 2.942773988382395567e-01 -1.044003215064582190e-03 -8.769736298016500347e-04 1.909999267969555992e-01 -3.356731081764533636e-02 1.697115884672957683e-18 -9.853474589479403745e-03 9.369570751260594341e-03 -2.642707478924496284e-02 4.704659718795941442e-02 2.003541060581049901e-02 2.861382254491728894e-02 2.474739131487610835e-03 5.833610422820970642e-01 3.663045129267551991e-01 4.373464252599402330e-02 4.754310436205685920e-04 1.605688302423588856e-02 5.213882255807981980e-04 1.706618529269770049e-02 5.020860609052797165e-03 2.121094727025447241e-02 5.554240032803988043e-03 1.263587185760803137e-02 1.376406582933879449e-02 4.118388172971949868e-02 3.477761198957718403e-02 3.610711306320263381e-02 5.266261943820305276e-03 1.686085028173401126e-02 1.629378194066946270e-02 3.522529527760469986e-02 7.861519276611558804e-02 1.844645032246831073e-01 9.214162300699894892e-02 4.212298918218094351e-02 2.086918872527345967e-02 2.746123008367248527e-02 1.571149139328019591e-03 5.659209208411434266e-01 9.694987771578416069e-01 6.122057881891559417e-01 6.681936215481091602e-03 6.727788232724574158e-03 3.238664842080314632e-02 4.201775902057367645e-02 7.986246210706682280e-02 2.255700508215691472e-02 -5.731766543465664082e-02 3.523203921400667495e-03 6.162223817428521544e-01 8.236389828622791009e-01 8.473751243230578378e-01 2.841057340583872937e-01 1.413948196617843822e-01 3.937824315124741764e-02 5.281571613290589329e-02 7.308309173380181378e-02 2.619079532379928188e-02 1.989909850928274840e-02 3.995452630395385785e-01 4.032066423995622206e-01 3.317753297061962203e-01 2.061017390882476444e-02 1.453127663854786138e-02 1.361569447093679240e-02 3.651854597364821708e-02 7.882309803290431238e-02 7.478658704890829567e-02 1.521013467329966062e-01 7.305557439569823619e-02 1.604203459425372069e-01 7.138600772972231168e-02
1.447917597156300473e-01 -5.614710883659826401e-02 1.082474524526209486e-01 4.529319007384623175e-02 2.130022273744959158e-02 2.412359821061135431e-02 4.726941610491316309e-02 1.481154028909434719e-02 3.619332966189051787e-02 2.123837963414997379e-02 5.486031100115892900e-02 3.091632956684855355e-02 2.668853605246339356e-02 1.913300967759227339e-02 2.304626266371248219e-02 4.383928801350264748e-02 -2.318160587320419186e-01 -2.098239697061725639e-01 -1.572552793449864539e-01 1.778201154270894410e-02 8.708085058702791370e-02 2.770929857155978240e-02 1.513661579888064997e-02 1.431310102328805720e-02 2.094043682133868087e-02 1.475452474335214696e-02 5.521653208321821577e-02 1.921511345937495759e-03 4.748621722642201899e-04 9.921549469311575706e-03 4.419382281111324667e-03 1.910890706831387137e-06 4.495773311214736057e-04 4.907707930659185001e-03 1.441944486130671867e-02 5.898315540132931765e-03 1.080995294779110288e-04 9.021601851887998264e-03 1.380566287396517780e-02 1.196850904869913544e-02 1.593083598883333202e-02 4.041669525638320187e-02 -6.527186952867714833e-05 1.393062848586794703e-02 7.197619536577393173e-02 5.141019049806119789e-02 6.990394719640552479e-02 3.768658389811288534e-02 7.683687647353962778e-03 2.283985926058572022e-02 -2.495654885166218723e-02 2.078745301576522334e-03 -1.297103652440935798e-02 2.606432764164199473e-02 1.297564407718897195e-01 1.608441721345104380e-01 1.755258375487981082e-01 2.518516273537009242e-01 3.433015222424198321e-01 1.488087941125579121e-02 -1.127322261878505455e-01 3.745014488069849101e-02 4.180097037535849408e-02 2.045359956378439356e-02 3.806180546759174721e-02 -2.049151307136020186e-02 3.773115683426562367e-02 1.336914424226339632e-02 3.926566643140803198e-02 1.025518850552468303e-03 9.649758459920661918e-04 1.211584972608650661e-02 8.261231543484098827e-03 -8.943784032533548534e-03 -3.194005523700992055e-02 -1.537947565898327984e-01 2.615874214731609207e-02 -2.731090023398959760e-02 4.936875493930023692e-04 2.455083414568884348e-02 -5.941645965298175865e-18 5.024447274618968479e-02 3.592768014012945321e-02 -6.390170661092990818e-03 2.949789548468076048e-02 -5.948560894883453886e-04 4.902988186735220003e-04 1.198897084144777934e-02 2.065581706820980579e-03 1.387530553198685485e-02 1.664832189474177871e-03 7.188148122798687645e-02 7.179316100940885681e-03 1.881199960628327818e-03 -1.184717064066231630e-04 4.793802229836990210e-18 1.018121165716009795e-02 2.939059534199062532e-03 8.767543881062587007e-06 -8.628304491663158365e-18 1.306876371118595058e-03 1.239641253907143581e-02 6.841394206685887797e-03 5.953743963117350685e-02 -1.267956027863918582e-02 4.946159295296601727e-02 -2.743114811293212094e-02 2.498728098387463167e-02 -1.341948478322225720e-03 4.079268833637601094e-02 -6.463907892958397836e-04 -1.277369172343053270e-02 -9.338533633729665523e-03 2.688962366606327883e-02 1.993652683890487567e-02 -3.267645367126414957e-02 -2.442983346929259933e-02 1.653563847914299098e-02 4.747289398818765882e-02 3.472480047722052710e-02 6.444796248998734267e-01 4.190091873632253439e-01 9.651072127405244463e-03 1.839631738327680149e-02 1.733799439807762535e-02 7.515746696952184391e-02 8.157262520827403274e-02 8.012810358535670321e-03 3.269889454701344067e-02 3.901237293712885718e-02 5.054469321163122114e-02 1.096309798053400446e-02 1.098107831381123743e-02 1.580412977901003019e-02 3.549687633577836010e-02 -7.854305195655526817e-02 1.671525158299413394e-01 6.588041950873002595e-01 3.311837863121143144e-01 1.401384564009625121e-03 4.624291184930958559e-02 1.519970327464733283e-02 4.469487526635506497e-02 1.119065624474560594e-02 2.861526960514470738e-02 2.728304380813755675e-02 1.221465538970009992e-02 4.395170715210091131e-04 6.961051786599362252e-03 2.792638039039722474e-03 1.401978961611601709e-02 8.187062267855869588e-03 1.732209659193867032e-02 6.389795109610637604e-03 1.388309348718311589e-02 1.872644608207464179e-04 4.932174316795095471e-03 9.242940547299597839e-03 2.394629009586031476e-02 1.122455629710969807e-02 1.714909923363569164e-02 6.378678255459533375e-01 6.980346297127875133e-01 6.054599187877540700e-01 3.379868176218398662e-02 2.299758324663933942e-02 5.791696028918869033e-02 5.213543195979749439e-02 2.782005344471874878e-02 4.241260035443345650e-02 1.128529191164345108e-01 3.599607484879978059e-01 2.839355901748420452e-01 1.042320361075208326e-02 3.908020241731465644e-02 7.474471544035257708e-02 1.182445114528401509e-01 3.256008310773753078e-02 5.119518509771613102e-02 7.580072556501329939e-02
1.146241064965831391e-01 -7.534122147828498095e-02 4.940119235615472726e-02 5.183716217930767894e-02 5.548155420761494150e-02 8.347824063766204339e-02 1.053119627884515352e-01 9.107936706283365630e-03 4.447279406664127188e-02 4.613806200261182100e-02 6.818700000839837683e-02 2.923380804766702659e-02 8.716849598429401991e-03 5.617787390373338990e-02 5.198013143450511825e-02 4.830328856975631829e-02 1.333252368352331185e-01 -6.037566987292454501e-02 -3.703245480634491382e-03 -1.653123944969601689e-01 4.569558446597184881e-02 8.462667396406159739e-03 2.732762386101401930e-02 1.082701914652872341e-02 2.127746184588250392e-02 1.934543897854792277e-02 4.008803079312277945e-02 1.123053052692039692e-02 6.121958651179002249e-18 1.581105912772613104e-06 2.798061536802570835e-03 4.785436208443185138e-03 7.670160369657797513e-04 2.336256605371858641e-03 5.253910627186833375e-03 5.816765125044020927e-03 7.959778629011318545e-04 2.134941394979407783e-02 1.022947104734306042e-02 1.004870354565894075e-02 1.871541446439743336e-02 3.745061121913379615e-02 -1.457174767981787638e-03 -1.702483927205747968e-04 6.822375435822122802e-02 6.063313235524750433e-02 6.406550784646597441e-02 2.234186558552122767e-02 1.441471787912665219e-02 1.352442553456152489e-02 -2.613746660686331463e-02 4.461746619651982271e-02 -6.544558375075542472e-03 5.207224611234428924e-02 1.714400645780264076e-01 5.063904753174635065e-01 1.575895260156579825e-01 4.617123393424569788e-01 2.682714970518464970e-01 3.963964735997852623e-02 -5.394446909241115695e-02 1.874064152493104088e-05 1.341338857819931174e-01 -2.062104083512009411e-01 -2.224440992793003027e-03 -1.881820327130835982e-02 4.479716502927207078e-02 -1.017698129325249334e-02 1.966843845033686491e-02 1.561214467761926419e-02 2.509500778655322957e-03 -4.327789574175293394e-15 2.188456163232984850e-02 -1.270719160911104032e-17 3.944396059378754010e-02 3.597968409402788059e-02 -4.793779522977876378e-02 1.521862754733595002e-01 1.701830591638986145e-01 6.147215465252370281e-02 3.284388809755434818e-02 3.476466603939019906e-02 6.150954050687638952e-02 -4.719057581327575079e-03 1.761791654708600580e-02 -8.400147037177516443e-04 3.292399830212452277e-02 1.702622964672513908e-02 2.642410950660395094e-02 -1.134805794066938711e-17 2.793583538700381591e-17 1.779804521289085097e-03 1.119015450137788269e-02 1.063040455314920724e-01 -1.354628391395992581e-17 -4.068334196985756539e-03 -1.603741568949144936e-02 -2.765938038887608315e-03 1.395666327431618057e-04 -1.101945568696756014e-02 4.261833614912579650e-03 1.089673168507714930e-02 5.962558655755276216e-03 5.464920337736801220e-02 -1.320082620096178974e-16 1.559701764906541721e-01 -2.233558788229110179e-02 -2.773829511319610142e-04 -2.218507755778641264e-03 5.239069130818423820e-02 -2.631546085775451644e-03 -1.513679120630521784e-02 -2.585542015570683333e-03 3.272114225647860652e-03 1.334151724826716616e-02 -1.382502316324808744e-02 -2.763042945026389710e-02 1.307565986027659291e-02 4.600008260906877922e-02 1.337105253006936820e-02 6.588865547669757738e-01 3.469295424674634365e-01 1.766981965048447165e-02 8.761727319070748699e-02 1.827644546314605850e-02 6.459967138066811942e-02 6.841714807731050230e-02 2.792266612730315983e-02 2.033166804796404659e-02 2.101822730021464525e-02 4.108703647923812874e-02 3.166407339382742991e-02 1.565612036307539520e-02 2.673770019998628067e-02 2.406921564622029053e-02 6.546490165669506534e-02 8.834655980324766789e-02 3.347603451955391551e-01 4.975936754274087326e-01 9.040517557040336216e-01 2.127272064194798626e-02 2.809894666072148411e-02 5.047425176575353473e-02 2.171490123223786955e-02 2.916563221837321918e-02 2.098550508016586505e-02 2.580377344459062713e-15 2.165496009971560297e-03 1.865161954954993109e-02 3.933278637116594846e-03 1.374669877348505369e-02 1.080243633280273535e-02 1.410016170724596313e-02 7.059045707944374684e-04 1.840435405299796490e-02 2.833710003187743607e-03 5.638202544780743231e-03 1.638335880373078743e-02 1.911544270905656331e-02 8.296560208416722815e-03 1.899059787392067683e-02 6.470168674414857879e-01 6.821837830705567152e-01 5.666367169228362366e-01 2.949152385619160238e-02 3.010732933867311945e-02 6.097599293560774375e-02 5.779846623457612909e-02 2.571935908822091735e-02 3.329723512325671875e-02 1.634021365143887849e-01 3.610683240449722442e-01 3.333845062104158208e-01 4.782412097447771848e-03 3.989471879873593418e-02 7.153327646170619392e-02 1.135152047151315041e-01 3.050138088962826149e-02 4.376615861417075537e-02 7.330554987491044727e-02
1.926313264135808001e-02 2.128086371594145945e-02 1.016564315433947790e-01 4.400983823891490426e-02 4.865624204526747959e-02 6.299255007707982301e-02 1.502143097287455265e-01 3.846572431640488121e-02 4.654691151520860132e-02 1.160407080561042525e-01 1.547737310687549162e-01 6.656144516915570020e-02 2.875041661388598457e-02 2.782833193609276362e-03 4.479391350669317817e-02 5.033635478080088904e-02 -1.239012911406250084e-01 1.885432411315070278e-02 7.833763613461965786e-03 1.494309285773246995e-02 1.635353452722802803e-01 3.599084392320290194e-03 2.100050060514602537e-02 1.496601897380027586e-02 1.008958676773118873e-02 1.211074399740178463e-02 2.102441401946868182e-02 1.766546990118877209e-03 1.033832733455725648e-02 1.072720193320164011e-03 1.859863416728221603e-03 2.444572328421171659e-03 1.183857547498205808e-03 1.162866545884663760e-02 -1.460268359015715334e-01 -1.223783982588589059e-01 -8.956269393319310490e-02 1.139714079293819605e-02 2.286308409296881200e-02 2.314478886408045943e-02 3.452021675474718082e-02 2.781904523122017206e-02 3.389584303591883230e-02 4.002592984241814048e-03 2.277347322699487669e-02 4.602377135568717914e-02 8.683676954125826852e-02 1.745002039174605585e-02 1.263575878418215885e-02 3.162364981230301325e-02 2.494743105914491027e-02 5.521663868909090084e-03 1.302650075167141944e-02 1.482174048941991076e-02 4.097139698266286101e-02 2.224713984799284866e-02 6.460215684823725213e-02 6.776109280795858565e-03 7.373075645742982465e-02 9.186327820032506863e-02 -1.099258403466485921e-02 2.285029457802149194e-02 2.003371270081027306e-02 1.830043310662281871e-03 -1.699941692476525292e-03 3.620853310762338112e-04 2.280175848500220689e-02 1.751025330151128115e-02 8.429194173880610527e-03 1.657420998137196927e-02 -9.128294573662808567e-03 3.479856088851265156e-02 6.910709064061609785e-02 1.283157548824090388e-03 -4.388615108984046664e-02 -5.299740215766420015e-02 -1.765675121438239070e-01 6.653449275988447542e-02 -8.209896196625564302e-03 1.914562889628079745e-02 -1.218217842106643486e-03 5.218454042243097016e-03 4.685038953592083645e-03 -2.416641985733689740e-02 8.530269754501914117e-03 -8.454536382913257000e-03 -3.349450757950599209e-16 3.521124227672168920e-03 -5.592925445750854823e-18 -3.621503431546822891e-03 -1.634477348864925988e-02 4.086428891012826368e-04 3.471406667902002946e-02 -2.693385302781423594e-03 1.424735895994964962e-03 1.857479251935606948e-16 1.863089277616118353e-02 -3.584946045462212786e-17 1.073008006845120252e-02 -2.738748542639363182e-02 9.336364896476932176e-04 -3.758163037148409043e-03 4.540442900028508755e-03 2.241709327444755054e-03 8.912973563472455577e-03 -2.407230446092212164e-04 -3.691666484548588451e-03 -6.007908588001898429e-04 -7.504272547634437129e-03 -1.315883325057341450e-16 1.224102721710248771e-03 -9.991216848232890857e-03 4.355871826032998821e-03 1.349654889581014018e-02 1.292762472952009879e-15 -2.358796410424556997e-02 -3.977548705225690366e-03 7.847714190595485306e-02 9.304826612260067020e-02 -1.123783981424357828e-02 5.161296770199182149e-02 5.919354350097218165e-02 7.143780675141359060e-02 4.672315911145683009e-02 2.441464382512699774e-02 3.737576089285531145e-02 4.677394793587411753e-02 5.020895601481703031e-03 2.398237241582467164e-02 3.743375742719738750e-03 3.137650086650851933e-02 7.859679451692625635e-04 8.658300761147864205e-03 2.338360543115652057e-01 -1.555758266145193247e-01 -2.320193442650874127e-01 7.651899372312034875e-02 1.436833132365025745e-02 2.527932753112485664e-02 1.254195542853782722e-02 6.413914662628970864e-02 1.650604944901078175e-02 3.501491227550038166e-02 1.536261204414023952e-02 2.627138264825049779e-02 9.178376087279891890e-03 2.074219364580312183e-02 3.041399024058816757e-03 1.156959912800552444e-02 2.482741909149862713e-03 1.260975646258156307e-02 9.118696964889535378e-03 1.858641397516523977e-02 6.136392606390745838e-04 3.188673271629059447e-01 1.098038369184487939e+00 4.944527573551464328e-01 1.002411546870098829e-03 1.912960222852891357e-02 1.893385249088782793e-02 4.912333491388010120e-02 3.185105736763227391e-02 3.343450709874626336e-02 7.848601249581534448e-03 3.501302000925026608e-02 3.486027448094796560e-02 5.338540445262612150e-02 2.076490372670923221e-02 2.085537280885820660e-02 3.389495446844299070e-02 3.292452831366585869e-02 7.061385067339584314e-03 1.700040113216280332e-02 3.721107392300120748e-02 6.255840137273296908e-02 1.980892414966235748e-02 1.017480914786207530e-01 4.486522117739600302e-01 6.547768630840126169e-01 9.526858861409431833e-01
1.534831134524031748e-01 -6.027595996262209710e-02 5.998707396518275503e-02 3.222721775398106753e-02 3.709358368299944808e-02 5.615000787757924716e-02 6.738638344249009426e-02 3.487340073718780964e-02 4.644820096716217178e-02 4.320489665041807315e-02 7.293871643083715828e-02 3.174453859950806395e-02 2.198689115844406142e-02 5.295453601480256539e-02 5.433127236423355988e-02 8.245318410143764876e-02 2.170135155596149279e-01 8.232080361815403979e-02 4.411062972421537359e-02 6.379674812812061169e-02 2.132816742066757920e-01 8.698361704021952134e-02 4.335225010465900558e-02 5.858652854981272889e-02 4.948303767279804005e-02 4.575690019169582207e-02 4.246264867074363986e-02 -8.987620905713274155e-02 1.153734416873929006e-17 1.641610781973838407e-18 -2.344344231364871994e-02 9.759447880760424316e-04 -1.502573576344462803e-17 -8.192650503191361080e-03 -7.142062394620961319e-02 7.222371668770338586e-03 1.887622024639091589e-03 1.223345881995600848e-02 1.434512783634150392e-02 3.791329600335253264e-03 4.113301354351168310e-02 3.747938345935442062e-02 2.769443848739968142e-02 1.749374761265297049e-02 3.775557845436251087e-02 6.625304325532833938e-02 8.008723195797366745e-02 3.073286084324313577e-02 5.407055078944074389e-02 4.704473532507102290e-02 3.909036406916610584e-02 1.629267109749436100e-03 3.323319791881620966e-02 1.480359300311265801e-02 5.959186090087361526e-02 1.219958981355730165e-01 7.631360482752502783e-02 1.129465271636028861e-01 4.929641815929288601e-02 7.381153728161687737e-03 -6.359256862266628174e-02 4.577467445974561094e-03 1.143154988472079754e-02 -4.053422582663768203e-02 3.486078813098600349e-02 -5.124907610439059097e-02 -1.665994401394389583e-03 1.939211630779101928e-02 2.079941576680651230e-02 -6.349497154027806956e-03 -7.005805095448050647e-17 1.091245458940474401e-02 2.318946703396036063e-02 -2.019205970430431370e-03 5.021674631180141068e-02 -3.712705227996493818e-03 -4.495333638448625879e-02 3.754063873238442511e-02 -2.974567905706786836e-02 -1.065941291580142086e-02 -1.649979276013346713e-04 -2.516155651277605584e-02 5.643797283320530975e-03 2.346794725964654327e-02 -7.303591857344827847e-04 -4.640338790254547492e-03 1.521881869471836471e-02 4.287757399024529461e-04 4.406959716734363923e-03 8.568588619948037298e-06 1.242916632252194925e-01 6.857166798774166905e-02 -7.644267110093641571e-02 -1.232172025788027571e-02 3.517275826101375069e-03 -6.890846339873749754e-03 4.193749078111690121e-02 -3.202275175651319444e-02 3.127931857222477491e-03 -4.451791540586415297e-02 2.333159686288466608e-02 5.244593719356647685e-03 6.196300052951629611e-02 -1.089129737026546009e-03 2.979191291085550239e-02 5.056276730807340858e-02 2.561601633757572966e-02 1.226192893471739764e-01 -3.161359231389185325e-05 -1.686522743820406721e-04 1.592388311947907087e-01 -1.060589136408179346e-02 8.307548136195172401e-18 -7.995113924836960359e-03 2.844664697791329105e-02 -2.774759206306106826e-02 5.584537337280998104e-02 1.282469628638686354e-01 1.505513367561266483e-02 3.830404067608650810e-03 6.792990905049361983e-01 4.158942786080299658e-01 6.485314890973707669e-02 5.567179148049791054e-02 2.337246540124175015e-02 4.909633066388918027e-02 6.060699020554275590e-02 1.553114723150057774e-02 2.597725508545469311e-02 2.844912648081674225e-02 3.855984144201047431e-02 1.281971309842035167e-02 3.252918590768347218e-02 2.731203477249928221e-02 4.220749722575820040e-02 1.044824373931586975e-01 1.501549191762729385e-01 4.180878876774478392e-02 2.376866931884991790e-02 2.620939954609659631e-02 7.507820801690852364e-02 2.001963287954667062e-02 3.667601370433014318e-02 3.659301529042609247e-02 2.561846880343376584e-02 1.794160321197059080e-02 1.188160823316180593e-02 5.555258997036751634e-01 2.390483297835558074e-01 2.046546013232800054e-01 1.381461404050056581e-02 1.082174641148257082e-03 8.173280673273283781e-02 4.915894186200997495e-01 1.792808000332406226e-01 1.801009454106365908e-03 8.904057718571981375e-03 1.528405413067189175e-02 2.998151075990576619e-02 2.315437890610653679e-02 5.939470514864282624e-02 3.695531334170348847e-02 5.778826533889710321e-02 3.790435268721734086e-03 3.992280891668389953e-02 1.496787893702450009e-02 2.061018863554097899e-02 4.524124590564493897e-01 4.169656156887533993e-01 3.062392035773302545e-01 2.433281565907277622e-02 1.194160268883802693e-03 1.432896150018274535e-02 5.146991041189041133e-02 8.102051386553946188e-02 5.383498118401960214e-02 1.166615912597052385e-01 8.000211864155544472e-02 1.215068696149771282e-01 7.769580197519213027e-02
1.510993040682431232e-01 -5.824757841818450910e-02 9.576824729725777474e-02 4.561250003397987474e-02 5.326296543275618744e-02 3.937843653480769396e-02 1.093701997179557101e-01 3.935754764620057189e-02 5.137450502911364292e-02 4.056131341399150936e-02 1.615279471874845885e-01 2.404128910786024198e-02 1.483834908744330848e-02 3.411509907935818958e-02 4.175012175055212160e-02 1.102518440419032880e-01 2.412184089640928286e-01 9.076910269183141589e-02 3.525004800197378135e-02 6.586256221263615818e-02 1.185604507481454462e-01 2.693551074743781235e-02 7.337625344591014453e-02 8.895341089293837267e-02 -3.993990798861314119e-03 6.399762345949558742e-03 2.079513760239321864e-03 7.353361796836713898e-03 5.101143747083760130e-03 6.158038097995565015e-03 4.073331436360751699e-03 2.211024716456227047e-03 5.226885512126568274e-03 7.708568160209679134e-04 1.134751550116823221e-03 6.202219147511355696e-03 1.258237990857390715e-07 2.590471322142457789e-04 1.579301867099553369e-03 1.208734401434324868e-02 -4.794271171353721266e-03 -4.235829540801499052e-03 -3.242187481950915828e-02 3.298994418392564489e-03 1.695269126503047910e-02 3.470444972244022402e-02 5.402356853428435385e-02 1.904066445458098278e-02 1.564933904717623683e-02 2.269178814811531608e-02 3.421570054937368710e-02 5.697779582147182061e-03 1.810807175507419750e-02 2.063423378370642103e-02 6.497640685495004176e-02 4.507552866192471308e-02 9.026256034155215324e-02 1.226428815010009200e-02 5.563208551985552447e-02 2.849502621231886010e-02 -5.983152201362982453e-02 2.847096837582973380e-03 2.736003949598695903e-02 -2.369135912756554357e-02 3.490702206109822653e-02 -1.722671107942372673e-02 6.979610468437355553e-03 1.627145003956094980e-03 -7.832954978140951752e-17 1.233393934011534424e-02 -1.031348453643411538e-02 1.353197071077623902e-02 1.103038860332278791e-02 -1.493515719026575206e-03 3.129681520013088969e-02 -3.073475719051431670e-03 -2.776608226548550259e-02 1.923993195153234428e-02 -4.435399772284004394e-02 -9.228680645702241283e-18 -1.538660858601505069e-03 5.924701713454389590e-02 5.492484215968401057e-02 4.106268199549781284e-02 -1.127826256307792223e-03 5.325141964280919599e-02 2.517455935564106781e-02 4.943321451180512915e-02 3.578135243334729099e-02 3.128876342018506451e-02 -4.828093534492761455e-04 1.999763667334027509e-02 2.382284387783641100e-02 2.026472601956683800e-03 -9.793637617947548765e-06 2.432500538883301012e-15 -6.576721152327825071e-11 2.579723160707895561e-17 1.790628612129440550e-02 -6.677146439833809254e-03 -1.099342521562314719e-03 -1.147726526949254498e-02 -2.782305541166668963e-02 -8.851871538524380342e-03 -3.870411218648957263e-02 -7.128885538051476373e-04 -1.379820284557744560e-02 -1.525038342532286867e-03 -4.479515288510127133e-04 -2.132216269813191900e-03 -1.216403273718213773e-04 -1.402615068138335190e-02 4.675694482584792644e-03 3.085859619993613218e-02 1.066468514814941475e-02 -1.370942272271313302e-02 -1.115467361294423610e-03 9.553243575595157799e-02 1.123430985272080157e-01 -9.556533799426236014e-03 6.565934728360741657e-01 3.975673214040002512e-01 4.428209994735150118e-02 3.443139837166064171e-02 2.712550515145309427e-02 6.013983719891112523e-02 9.617254074664620400e-02 3.019272630014528669e-02 2.857420866656393277e-02 3.106366580715649861e-02 4.436232399113412828e-02 1.809549630816922189e-02 2.384016651542388704e-02 2.473690373235114259e-02 4.035247749258973060e-02 1.007356546191269381e-01 1.779263204359492201e-01 7.943678450476454600e-02 5.466279124805169254e-02 3.954719099671805127e-02 4.416188343494550916e-02 1.454449356436297164e-02 5.931238500385769274e-01 9.200921168777049797e-01 6.979861305406344085e-01 2.719857083824365855e-03 2.173580914793059315e-02 7.593154285793950534e-04 7.748262271581724675e-03 1.472388029189444086e-03 6.101495186753715105e-03 1.236043024030698823e-02 1.782827124234157359e-02 1.166461588041591262e-03 1.404421299756423991e-02 3.663868735471691174e-03 6.069234296837301007e-03 2.860209275439311652e-02 1.634502733616426465e-02 5.085016796590297559e-03 5.512962698435214248e-01 1.147364630113123285e+00 6.962093728872699172e-01 3.626782855827202778e-03 2.131067422175130996e-02 4.172393052493780091e-02 5.285886659669100651e-02 3.542114115174962302e-02 2.848708749649293479e-02 1.608384998622511969e-02 4.137742008161440083e-02 1.494580639323279947e-02 2.049984011406310105e-02 7.848495156923748772e-03 5.228369404565429840e-02 1.619571428099122676e-02 7.869401495529677881e-02 4.617734875898551472e-01 6.765207799345289708e-01 9.462220906700354872e-01
7.117072152777877483e-03 1.759848469396554232e-02 9.239429707632404454e-02 3.399667343012649501e-02 5.095937645472223970e-02 3.632654671645359906e-02 1.293640764937037113e-01 2.696567491710609443e-02 4.732521152671478187e-02 5.955492004501099224e-02 1.708691361439930723e-01 4.534268986789236761e-02 2.071959242891818070e-02 7.630656720535036134e-04 4.253008586735831237e-02 3.578897937948066071e-02 -6.223890971495214114e-02 6.969165969192614440e-03 2.237194569832358891e-03 2.898559065970428977e-02 5.251106023024310759e-02 2.184034540261673568e-02 2.718998287898697144e-02 2.077822987789895021e-02 1.582051685371468203e-02 2.584985872715885730e-02 2.592071858351409352e-02 9.242876230729747070e-04 2.037944966219971213e-14 1.242243790551929771e-03 1.551651972700442790e-03 9.914050185508560542e-03 2.113969299185022941e-03 5.166676927382283851e-04 1.691580496934858751e-03 1.072096848126330596e-06 1.626258886017889482e-05 3.082326216743389545e-03 1.558992294144711903e-02 1.696216822135630364e-03 3.231828004506077934e-02 2.378192085757728502e-02 -6.363680996038989165e-03 -1.634936061828226594e-03 -2.206427529602368945e-02 3.573445426476107301e-02 2.645856047384910828e-02 1.753803707483556421e-02 2.553854519217019226e-02 2.894052757780137333e-02 4.208259372264656212e-02 7.225874887064917367e-03 8.951134272687483184e-03 3.863394508878251199e-02 -2.161060710961330844e-01 -2.238156195953867056e-01 -1.192339798322881450e-01 2.336403542526901689e-01 1.517143460473394656e-01 1.148787603513125172e-01 -1.959988674909524803e-02 -1.590489791016728733e-03 2.787962371878478438e-02 -1.268020700238366681e-02 1.818893558541033809e-02 9.902411844272174159e-04 -5.417820676431635155e-04 3.931196674015886156e-02 6.206672723537047867e-03 6.757998799564536412e-03 -1.089832130345948225e-02 1.193492422702082528e-02 1.005549279490404596e-01 1.943301279621945099e-02 -2.994937580063944393e-02 -6.721899426031850699e-02 -1.714253881158832316e-01 6.976992407210919067e-02 2.207298453316649742e-02 3.537019875573536604e-03 -1.853883310086964532e-03 2.185902565687011800e-02 8.285191375898591027e-04 1.544523373814308403e-03 2.450494072098212406e-02 -1.415610938216009355e-02 -1.460717068355974792e-03 1.964266333534542275e-02 -1.239933498138642241e-15 8.243489188082035735e-03 -1.392405097899599396e-02 1.615787824066672318e-02 3.662145509000179489e-02 2.905314232190068374e-02 -6.920044442256206678e-05 6.356874722407604428e-02 4.733018522273928393e-03 -1.190314197099749324e-02 2.959169821342594781e-18 -4.857040628478144783e-18 1.146227090344737257e-02 -3.288675367358552992e-04 -2.938047848072643416e-03 -1.455511570700360061e-03 -3.435884832153612408e-14 9.035358499193440335e-03 -1.887620785598649945e-03 -1.454569898382633564e-02 -5.130319727532026142e-02 -4.957116011992335203e-02 2.702248051225794243e-02 2.948871294389993764e-02 5.763936049400985792e-04 -3.994336224672639241e-03 -1.862158731886750379e-02 7.554267279937064775e-03 1.688036693279112684e-03 -1.428490929163892809e-01 6.324825404244783789e-01 -2.029327050674611388e-01 2.980832978639937070e-02 5.156187140313711614e-02 6.063289208848075817e-02 3.826108839043636939e-02 2.604201875829687332e-02 3.017485024118109205e-02 5.848285692814545605e-02 2.224818708581435128e-02 2.686007082356436232e-02 3.283843130719440746e-03 2.551593731243179813e-02 9.403245351936131459e-04 9.781679189080867004e-03 1.962922168037865567e-01 -1.599862196729066321e-01 -2.152848340788910131e-01 8.918549692086889802e-02 1.724154450772150474e-02 2.198199760897319791e-02 1.174413088148374557e-02 5.212423869836553536e-02 1.396950576355473171e-02 3.930864277783177596e-02 1.952001384263596026e-02 2.678077914832523165e-02 2.473785338046780719e-02 1.454980292329477280e-02 1.485231444049886973e-03 2.342201605709826745e-02 2.149091931791928447e-03 1.567996370823375105e-02 1.480332760989869429e-03 1.375458960032544323e-02 4.604066768077011572e-18 1.085763961069366804e-02 2.682285831250468325e-05 6.086427430678017711e-03 3.023542593586390700e-03 1.658713043080962957e-02 1.605878846095294687e-02 3.469566786144754472e-02 9.293286545720437150e-03 5.399219185008204969e-01 7.452776041839060950e-01 4.608644748197107122e-01 6.492686906426960169e-02 4.809347832650851667e-02 1.660461951023792238e-02 2.662095185128154906e-02 1.194520405865866046e-02 2.628558102300029148e-02 -1.641480125162900509e-03 6.678222496845185804e-03 7.775308914371669398e-03 3.245538249624216331e-01 2.773464260030556461e-01 2.835568754104387246e-01 1.447336907671210871e-01 1.136074756095691585e-01 5.039497758577864417e-01
1.211805001637536461e-01 -4.486179580042101711e-02 8.008748129751136480e-02 3.132689229886830712e-02 2.879875599636249131e-02 3.149122640899807563e-02 1.227812532507324000e-01 9.840739833330960917e-03 4.988975465954631483e-02 3.367567527542286721e-02 6.580079700888483152e-02 3.043841457248933416e-02 1.313491299691923325e-02 2.249193332987894922e-02 2.782092659547508867e-02 4.488698122460375861e-02 -2.755792493606841753e-01 -2.124700913509057765e-01 -1.498056055426620070e-01 2.171358263646303369e-02 8.189871470409688936e-02 1.015205509094693065e-02 1.444807927803568558e-02 1.852934389753259359e-02 1.245106691111530495e-02 2.697457034544195956e-02 2.172792694063622326e-02 1.542134448514955770e-02 4.704019786809457253e-03 3.369661497053718421e-03 4.519099796336457009e-04 9.161144047524581030e-03 5.738848651002129110e-04 1.132217466144787082e-03 4.644642228858969144e-03 4.853616997173872821e-03 3.134942290533265894e-04 5.018226283664254175e-03 2.048054079274410928e-03 9.612678107218988396e-03 -1.104349579676725803e-03 -2.433187400592028496e-02 -1.075280041777741322e-02 -1.283559481441635675e-01 1.161527977794672706e-01 1.878492300283118865e-01 -1.878744519017953515e-02 -4.681736283629722717e-03 -1.584448180692404548e-02 6.759903655004928513e-02 9.947327530352328973e-02 1.018726115835206190e-02 2.414198216079702433e-02 2.676611643499748894e-02 6.542498763114462657e-02 4.873304262541317683e-02 7.700869728670321768e-02 1.352641452953036449e-01 9.919865596252787354e-02 3.722563750158292689e-02 -1.261565738511582768e-01 3.330694896453947895e-02 2.451989981645958869e-02 4.312665792891419436e-02 2.517452746555425697e-02 -8.709768097656726132e-03 3.148529458019266869e-02 1.120408038738011508e-01 4.112999399065721436e-03 -6.204115476152729722e-04 3.260866724489869944e-02 1.668519136713048426e-02 1.321933951516851904e-02 -1.285923411270501089e-02 -1.260670679712063910e-02 -8.927298529110720693e-03 2.795081862780704127e-02 -2.457286492467953415e-02 -3.532083681032167216e-02 1.067407174248369284e-01 -3.752840437051919656e-05 6.711037417245138692e-02 2.308764105325196783e-02 2.173177561634454316e-02 3.370280205019569181e-02 -1.571804579856488376e-17 2.708157033700945085e-02 6.903188696195414892e-03 3.682217297621431795e-02 9.376191442295617184e-02 -7.734103242358653530e-04 4.370785240859492715e-02 3.536508234315345611e-02 -2.365523945196196590e-02 -6.737750960108541097e-03 1.013576170662706250e-02 2.943016425068385560e-03 1.323135588281382684e-17 4.935422021546935316e-03 -1.542857250382354863e-02 -5.704556796355477605e-03 -1.952611505440884215e-02 -3.570453881740844743e-02 6.657073929659303169e-02 1.741303660972791562e-16 -1.573313960006920142e-01 4.984585619690470176e-02 -1.827475714332630480e-02 -7.647700237886134733e-03 5.217831305093630079e-03 -1.206804485707730669e-02 -4.834108790163771374e-04 4.854005399570306271e-18 -1.371508039432815448e-02 2.783932915908179928e-02 -3.247719639832126470e-02 -1.081810523023687401e-03 1.043848931906404771e-04 2.411635262028864052e-02 1.513194860401354588e-03 6.374095266728863063e-01 4.447154609073188936e-01 1.267060863608713422e-02 1.119549849661326257e-02 2.257632442444174742e-02 5.176758111258399769e-02 4.137365104465716426e-02 1.766593646486669389e-02 3.224108790608336272e-02 4.253800488604637942e-02 5.068644120078540094e-02 1.408621141708752550e-02 1.729306101628209566e-02 1.531796244265621124e-02 2.351157041656628394e-02 1.758155190770443023e-03 1.729910710150489583e-01 7.058430680700028548e-01 3.122305761405924640e-01 7.610704887939701187e-03 3.247890499969426081e-02 7.269692371681905213e-03 4.139639154588810321e-02 3.228205483289033756e-02 2.014550430112493559e-02 5.749985439512275524e-03 1.903631608557671342e-02 4.082042912376763685e-03 1.298979349706964014e-02 7.100067057345654103e-04 1.106460380862411136e-02 -2.767117773279786919e-02 1.788084629478947987e-02 8.652192688731757562e-04 1.599588244375540688e-02 2.404087557047698723e-04 4.613781104962026008e-03 1.705638379350326314e-03 1.574581886586449639e-02 4.130269051325591995e-03 5.091832402976846916e-01 8.710750291836696357e-01 6.361813810459993057e-01 4.746776693299040173e-01 3.642164126323646500e-01 7.365761632186387242e-01 4.774442059854963616e-01 6.575521770537874344e-01 3.677023511344417739e-01 1.391724473015671162e-02 1.466672346802773245e-02 9.130182395874526133e-18 1.438084825613635855e-02 3.193199148940140120e-02 6.132339287445277842e-02 2.907672244967689917e-02 1.128959844973891624e-01 9.346745622511597185e-02 8.577272514170601603e-02 6.800674633899574106e-02
1.261392766754773284e-02 1.289831028153367622e-02 7.670113007660929461e-02 4.744362418726893321e-02 3.800851146626942068e-02 9.030398803990766221e-02 5.130194151467252356e-02 1.762375738865755090e-02 4.223161082686094359e-02 7.661974590480886971e-02 1.180209558823204358e-01 2.508240447818275293e-02 2.220639251782877827e-02 2.636230922224580813e-03 4.779874974837104024e-02 1.786537410230643746e-02 -1.221847212200247446e-01 8.943302650589599750e-03 2.236468368226311883e-04 2.371816941801796128e-02 4.171496819386612959e-02 1.902534435351588282e-02 1.114167832951923169e-02 7.542652505535226044e-03 2.250825737448741704e-02 1.956974646911191495e-02 2.375817229305644471e-02 1.925955810310908027e-03 1.665166632972678654e-03 6.028969213101489488e-04 2.663243528405336881e-03 2.153378472293795282e-03 3.143288311941508806e-03 1.277050246876921689e-04 2.714328033162485138e-03 3.704251832569241832e-03 5.991062951140199284e-06 1.996468156680523617e-02 1.092884295682335603e-02 2.083836553966046320e-02 2.633320408215326577e-02 1.999010070508316222e-02 -2.540921518974676840e-03 -7.090040356021500186e-03 -4.947471003655847487e-03 1.599499423555940728e-02 9.631834181417260610e-02 3.930185096790227151e-02 1.187732055128634884e-03 -1.337251476350318977e-02 7.219596985432670576e-04 2.385999532883768364e-05 2.877959173461997269e-02 3.220675211977697128e-02 9.107303741025596366e-02 1.196211304305744155e-01 1.339522368787685325e-01 9.975045739446722348e-02 1.560111911862020750e-01 8.508964033792979637e-03 -4.937711292579250594e-02 1.829733987517703428e-02 3.423380791918611415e-02 -7.561252712700017569e-04 2.224092897337904065e-02 -9.202359426795973796e-03 -1.708610695522310715e-17 1.443432463690058447e-02 -4.022077546967541342e-03 -2.377288804119218665e-03 -4.906336053869814716e-03 2.171768048995506573e-02 5.393248551203533303e-02 7.904240746870698844e-03 -3.459235073255720327e-02 -6.396179719748595893e-02 -2.005045009402002532e-01 1.002603649179744927e-01 1.796907401125017942e-02 3.918546768401805040e-03 -9.921216695867754540e-04 2.236616813782761615e-02 -5.776396374183625557e-11 -5.511747446337810462e-03 8.633384526667385941e-03 -2.043957831412952786e-02 -7.689943423286419453e-04 5.973410680436672462e-03 -7.535035584019425363e-05 -9.518891925010072220e-18 4.104941876824543392e-18 6.805590318913717733e-03 2.463992635777574602e-02 2.460854599861951077e-03 -5.507264778905734395e-04 9.365665102684547771e-02 -4.466336692643428700e-05 3.774993426460324444e-03 1.540150595405092313e-03 -1.927679084317878572e-02 5.849130073364574092e-03 1.281251636387974700e-02 -3.640712219138485511e-03 3.666821714779770758e-02 5.438644372546918432e-02 1.453332457265360256e-01 6.593942927306047086e-02 5.714507543558408065e-01 2.661899806621630116e-02 -8.361985558760056184e-03 1.038753629961221541e-03 8.839780876365113264e-07 2.097610552919750362e-02 1.830907188484950585e-02 2.750634147821601812e-03 -2.403479859694065274e-02 -9.480108838543952702e-04 -1.190720525986312377e-03 -1.496517491893027338e-02 6.595328202734346279e-02 5.146127710407247952e-02 8.389486164642100119e-02 6.113648090096865384e-02 4.537481052443821661e-02 3.660342572260524147e-02 3.325926311417345960e-02 4.588951683601367321e-02 9.477638458283354009e-03 1.187027306281582609e-02 2.253573134746056361e-02 2.527168834469857919e-02 4.829151465323782387e-03 8.464340642096456618e-03 2.352215828021603228e-01 -1.605652518312750965e-01 -1.815809139192328114e-01 8.395317309302501030e-02 1.734784954941986229e-02 1.954504895089261701e-02 3.586311346161205615e-02 4.440871903370142287e-02 1.198098848331737194e-02 4.323758529647641485e-02 2.606660393785840274e-02 2.806796796126563229e-02 2.096807664433161453e-02 1.679447579741634444e-02 8.085574883151107686e-03 1.126048285513543863e-02 3.613469593733563846e-03 1.310533336470696801e-02 1.351386977282435309e-03 2.211492194200205194e-02 7.530370180529714214e-03 1.343006442483452023e-02 9.375261995389607350e-06 5.920967831896703504e-03 1.547397625347402602e-03 1.473766501693277164e-02 1.649974039778929688e-02 2.856953314354018086e-02 7.095923742548469401e-03 4.998912157467113593e-01 7.103085947814636869e-01 4.355174744961545219e-01 7.402995881595551830e-02 5.485948638208156675e-02 1.877130330191839657e-03 5.110883663166863877e-02 2.212522466541788912e-01 1.323221854670358355e-01 2.514641674195964294e-01 1.572696973212961771e-02 1.874230380177942940e-02 6.265210595426720652e-02 5.118633003088505357e-02 1.153375756879379821e-01 6.203677308447916317e-02 5.086789106322785192e-02 1.541372314496646824e-04
4.086524185704772973e-03 2.792454883949972363e-02 9.289058810871655969e-02 4.280245887152769185e-02 6.864845640132039017e-02 5.600282576546369157e-02 5.484578804071683217e-02 2.701264604831577279e-02 7.114453881037335892e-02 6.397814410690845499e-02 7.678487086746046963e-02 5.332192093864925736e-02 3.203503049642872358e-02 3.343730947615738969e-03 4.890965403420968749e-02 3.354842085558413373e-02 -1.281132848818339331e-01 7.350098141845569366e-03 1.816280729222715176e-03 3.378595608466875011e-02 1.051173497235683291e-01 1.883534056761566183e-02 2.226138379994919980e-02 1.798961031664581683e-02 1.560797553049503641e-02 1.131095977454144030e-02 3.514074682197560612e-02 3.266863013848052971e-03 6.604711247840902404e-03 9.494230689084826730e-04 4.217944996026349452e-03 3.697756946715786454e-03 2.073915544134137447e-04 1.146238935946025054e-03 2.545922605646288098e-02 2.568046783047631121e-03 7.584377926739075571e-06 9.429667079477238242e-03 1.915898922796117551e-02 7.451343422470451013e-03 1.301966434505782630e-02 -1.122997431345751362e-03 -3.999655703095628123e-03 3.536906889789598590e-03 4.327938732458575660e-02 3.518958466225179105e-02 -5.639487658892361477e-02 -5.348053917206175428e-02 -8.220346312093873217e-03 6.165887597071485760e-02 4.987057582894038887e-02 1.545455435476138922e-02 2.353918757482958285e-02 2.480395589374576742e-02 5.910952726842665095e-02 6.642413227531081266e-02 9.434046754247028821e-02 1.131430526833746286e-01 6.160294646835532545e-02 6.471305665978595301e-02 -3.025501139362632430e-02 2.055133973083501930e-02 4.327908268512778434e-02 -8.676370511839680971e-03 1.617451131248248339e-02 -1.682274665009442694e-07 -6.367014504773543977e-03 3.979023805356303456e-02 -3.792343799521469966e-04 5.618237970371444540e-02 2.340907960287161727e-04 3.282017179650133232e-02 3.895336924381576083e-02 2.440149551213222773e-02 -2.668205223572400753e-02 -5.852891316206465522e-02 -1.671564537139844930e-01 6.992891301468551202e-02 -1.096530372382880031e-03 2.303380439070842889e-03 -4.598217212710535313e-04 1.920109406744412509e-02 2.008167735332189868e-02 -9.067915827103568600e-15 1.050054748002109822e-02 -8.862342986345422488e-03 1.598866954575202740e-02 8.315908107595844367e-02 -1.061617612002169290e-04 -5.557282004866517973e-03 -3.761283129929932512e-03 5.945190091826845977e-03 2.496002388496643826e-02 1.789980611048134704e-02 -3.351351401319969064e-15 6.043490240946527807e-03 -4.000216914287785253e-03 3.252162168012275381e-03 1.083696749844789003e-03 -6.762530520319608033e-05 6.352889910251298183e-04 1.125563152328826720e-02 2.204892570813955508e-15 3.615233209613157722e-02 -3.820122180533327971e-02 1.452927598389918036e-02 2.493083168963157372e-03 -5.519910148876529428e-03 -1.147086656179161640e-03 -1.516849567582948671e-02 -5.694100098735986951e-03 -1.544163424633633250e-02 2.599193915163419515e-03 5.386736640149012399e-03 1.586055333556499539e-02 -1.211379487455499504e-02 -1.239423521011879603e-03 -1.778398363438568261e-02 3.112417014101271501e-02 -2.553588250821213837e-02 5.908630635503986372e-02 6.892314090739894150e-02 6.730070106854267598e-02 4.132535622920664037e-02 1.936429596060472783e-02 6.208336874443509679e-02 7.595485274362248629e-02 1.118223125650943776e-02 2.369913717694732691e-02 1.272969567641802670e-02 2.267305321223003767e-02 4.023509125279608502e-03 7.786441067933755691e-03 2.301228170599038114e-01 -1.343512492927910196e-01 -2.232927027671019748e-01 9.412922380094557173e-02 2.341183472000082227e-02 1.810376946727396477e-02 2.129189592116092003e-02 4.071602919521949415e-02 1.860034031944712840e-02 4.833421631606742153e-02 2.386692415801548034e-02 2.116904259222933751e-02 1.282260439098380916e-02 1.408030215994033989e-02 3.344928031017022334e-03 1.354781158661758078e-02 3.180235809724323189e-03 1.603737486345789900e-02 4.816272761767708722e-03 1.741363677264009049e-02 8.050013884807890544e-04 9.707606125882737094e-03 5.037295077605544831e-04 7.728824228830238202e-03 1.246862347340143944e-02 2.517297215597174773e-02 9.482114335674524455e-03 2.549699978214423415e-02 4.766702142012119725e-01 3.438083774009545457e-01 2.242219866588441302e-01 6.113629181267662104e-03 -1.166217969474766247e-01 1.856974378506967560e-01 3.671194143832542389e-01 2.458961752115644828e-01 2.208965081425891437e-03 1.144647311695289396e-02 1.313753028183726806e-04 9.341562454704748983e-03 8.641367428449042362e-03 6.864414946490672575e-02 4.115849102326591541e-02 1.236454818797955296e-01 1.248505059462215738e-01 1.121280047455842088e-01 1.284689670038982046e-01
1.089656393134474238e-01 -6.514814072623802133e-02 7.642549646107427153e-02 5.151533742310367148e-02 4.806357899805548811e-02 4.535499148124003815e-02 1.007408014421487530e-01 1.164476585172823271e-02 2.373138621598945563e-02 1.612168977535841097e-02 7.567223065104411450e-02 4.871958669729221231e-02 2.471752359308155081e-02 3.248647027155592698e-02 4.670435240738178562e-02 5.776876655757308410e-02 1.369540127401804308e-01 5.217462469463904351e-02 5.135638377050317366e-02 3.250952399156200462e-02 6.276825236767179239e-02 4.000001311903498419e-02 4.730919037491702894e-02 4.675861488408026428e-02 2.982835981002984402e-02 3.395934118240021005e-02 3.053927081237182151e-01 3.324338520441331296e-02 2.302556578243621435e-02 2.096608947439211525e-03 3.099208819373579834e-03 4.765604254607348766e-03 4.584738679961989152e-03 3.447089641002107699e-03 2.901869401797649989e-02 -1.268835374278091399e-01 -2.395790308524249901e-01 -8.092920889798696504e-02 5.938981415856687401e-02 2.868529160759199573e-02 4.977140512746488360e-02 2.563511907621547278e-02 3.524534256599698434e-02 3.594686375247491128e-02 4.057144597556183035e-02 4.088317548032612686e-02 3.946215529658382570e-02 4.458810887597820793e-02 5.530523941022597728e-02 7.559258289988816069e-03 2.709118331532461843e-02 3.990930814490006862e-03 2.786655686888097824e-02 2.841647044689903784e-02 8.120536707774260454e-02 4.459782144531166531e-02 7.695145265915748212e-02 1.600478738961041425e-01 8.703305990960687188e-02 8.933149750629218108e-03 -9.167092306377198907e-02 1.558762272841066358e-03 1.083987554132639095e-03 -1.409394508405057214e-02 2.328913697798233248e-02 -1.034187595972627972e-02 -1.960821496226763851e-03 1.281990594834589436e-02 3.623005500611092833e-02 1.294876036270440657e-02 -4.751023783622718055e-17 -2.864613038757805354e-03 5.769184882299374303e-03 -5.206472849775827146e-04 3.003309397845006085e-02 -1.458636758881794561e-03 -2.775864056801394600e-02 -4.109266263686012628e-18 -1.120554219529210777e-02 -4.276727578148737351e-03 4.740568067849681988e-02 -3.827257841089654121e-04 1.671280064641337409e-02 -3.634994890101459625e-02 1.310983864850307108e-02 4.790404033056763766e-02 1.958659196251706647e-02 3.513225024581457384e-02 -1.086809935427412999e-02 -1.730328882439189503e-02 1.162039370507614473e-02 2.683408631406831090e-04 3.829331850983989233e-02 1.064915714708547685e-02 -4.819173049624290300e-03 -4.651582583087818940e-03 1.042263534159538303e-03 -2.589586022471493734e-02 4.498747686058000156e-02 -6.300822916536768549e-03 8.011790013535462218e-03 -1.549067016397727605e-16 -1.281329544351451043e-02 -9.801071179007443679e-02 2.521581242855134282e-18 1.582471754169979079e-03 -6.644132536740217886e-03 3.993528728786703563e-02 -4.195669990203512029e-03 1.989481522956210169e-02 6.909344587772388302e-02 -2.858518605843202512e-02 1.857032447261782916e-17 -4.923384371004358578e-02 1.913739863414005288e-02 -3.632726671583694222e-02 4.365579965932289697e-02 2.997346645847492885e-02 2.471845711951195165e-02 6.792781742733498308e-03 6.626908146718254455e-01 3.838667405325227877e-01 5.086099235158333259e-02 3.137148245310223543e-02 2.209798048489392680e-02 6.625632752463730957e-02 8.004136995534524845e-02 2.745398745366928184e-02 2.051910298852632070e-02 1.046028415386128785e-01 5.093039874892338337e-02 1.080530015043864450e-02 2.059849089668528907e-02 2.390853028444957923e-02 4.914329764950147589e-02 1.045892408435737803e-01 1.533729865529652459e-01 6.751570172924227509e-02 2.240359334737949959e-02 1.672375626620357242e-02 7.966217758354109113e-02 5.583361896659469470e-02 6.786994199501333869e-02 1.087898762725795707e-02 4.071694197056731877e-03 6.685105955951405887e-01 8.664970477070671473e-01 8.374646097913682752e-01 1.770053682210453450e-01 1.182922442375119315e-01 7.701718526469471249e-02 1.750204230663331417e-01 8.467992690831620262e-02 1.852022048959600931e-01 1.546666539452642430e-01 7.980105380747787303e-01 7.413445521075033851e-01 8.133543633300430731e-01 2.243275048608467728e-02 2.290440426052505959e-02 4.629540761178022046e-02 3.011021325044452648e-02 2.692742766203604435e-02 2.845203265795281759e-03 3.193336427801026639e-02 4.060917049913738419e-02 5.871188950657740824e-02 3.733781069329014102e-01 3.803619389251112759e-01 2.572266794469325601e-01 3.255329408910381112e-02 1.688107554633905966e-02 2.770027589752855485e-02 3.884996201110142711e-02 7.669404808162873233e-02 5.488729241770356343e-02 1.528678386528772581e-01 1.956288429976829057e-01 1.441122835921199719e-01 1.167053281793222902e-01
1.199056601136506972e-01 -7.096232491096045314e-02 5.764752664367497892e-02 2.537863375666030499e-02 2.965121047211805375e-02 3.842652824292660391e-02 9.333636505892274615e-02 8.978701972510683485e-03 3.311470134559287015e-02 3.816469426098085399e-02 6.043765966720644756e-02 2.627818187252616705e-02 1.513642854891304734e-02 3.622757174492726578e-02 3.073364628737656340e-02 6.521985145459023581e-02 1.197308511419037796e-01 3.635377849587852900e-02 4.455289955641171529e-02 5.047411511766997094e-02 7.663706448586916908e-02 5.168587514726639964e-02 3.796149700068762733e-02 4.657443561491839829e-02 3.352186064085968531e-02 2.547218172881255197e-02 5.774553362180880750e-02 2.384972226654174556e-03 1.706647694666461185e-02 1.184206616319187673e-02 1.444928812643640152e-03 2.981001767773822433e-03 2.066002853439040301e-03 8.815097136416070100e-04 -1.415349611440026301e-01 -1.640690535281555396e-01 -7.289243876127839139e-02 2.550149995551622556e-02 9.113031479225479348e-03 1.117688031437753457e-02 3.082205512248796866e-02 3.593154230864632065e-02 1.360130771013680871e-02 1.209862968783730237e-02 2.941045098479860148e-02 2.116002032078662456e-02 7.853905895233241619e-02 1.638406194537252863e-02 3.505619369905502852e-02 2.598276258217902326e-02 3.270194302501999023e-02 1.435577075571040908e-03 5.721511331625568457e-03 -1.308371458701752463e-02 3.762142082446729162e-05 -2.117916481809140883e-01 2.915691754488536694e-02 -4.732712561745390512e-02 1.161660979173596842e-02 -2.642738738163959156e-02 -8.373187250930071157e-02 7.330703196381602865e-05 -4.015942974033932279e-03 -6.235206915288622059e-02 2.005400007256349926e-02 -2.608325695822448585e-03 -4.413142900362863256e-04 3.666111697154891130e-02 2.434111746208905344e-02 6.096551963553016639e-04 -2.812771466961612107e-04 2.816668146253601540e-02 6.382154781130177160e-03 -5.560231120545730196e-03 9.714224110579131391e-03 -2.548446717353275957e-03 -1.917995889181789773e-02 5.260275620201771037e-03 -2.378975623565857470e-02 4.866714966741670506e-03 2.857288445037715896e-02 6.156881108569260469e-04 5.722174203817483734e-02 -1.588421112824531245e-02 1.102106222542443940e-02 1.193692304778326205e-02 4.202777285878209895e-02 -9.929676375714825698e-04 -3.558769588026350090e-03 5.325451092634637734e-04 -1.655450161492636038e-03 1.451515827652489869e-02 1.177277632451113193e-03 2.789906831099105838e-02 -5.086616759225112694e-02 -5.987894422940705230e-02 2.766480789754044434e-02 -7.712903256768534506e-03 5.569725570301133771e-02 -1.340061884474087923e-02 9.511654186834250790e-03 3.230978305231220482e-02 -1.495993621879721939e-18 -1.457782454959515375e-02 -6.415411357308851113e-03 -7.826365784717017685e-18 -6.224786905691427619e-04 6.962374687679601971e-03 -5.071676007282935328e-03 3.661654457757538769e-03 -8.154236000209562005e-03 4.402267326147731724e-03 5.029628877856726944e-03 -5.446326782292430953e-03 -1.150549740187179273e-02 -1.619130654025999205e-02 1.319036100815417119e-02 1.515338456015973001e-02 5.734175108819405259e-02 2.878863471216925804e-02 6.345667326931581087e-01 4.044803979371345837e-01 4.670932721424960604e-02 4.048278419348004531e-02 2.175702137984902862e-02 1.007208515729594717e-01 5.545077725067332902e-02 1.526068814596689797e-02 3.429182187569460027e-02 1.077591464626904971e-02 4.721039197679520460e-02 9.568435672873014386e-03 2.677344005754231371e-02 1.603907749376900713e-02 3.842211955650024630e-02 4.919560485416655388e-02 1.022129328516839519e-01 3.897671324262153708e-02 1.653483758109681317e-02 3.602046159784388912e-02 6.811508769139471009e-02 3.516950994548652398e-02 4.370836290382722894e-02 2.408058650031848213e-02 1.253516649798067963e-02 6.969928594976535496e-01 8.385553311826392520e-01 6.487961655239953274e-01 2.812489065260146900e-02 1.656535234040421023e-02 1.678167637186457617e-02 1.139272915969048727e-02 4.795609206756619447e-02 7.284065647789016162e-02 3.653986678930848941e-01 1.040950444780689654e+00 4.995485497101886385e-01 2.346162589266331117e-02 1.332636593738164600e-02 2.174516062630774474e-02 3.919738314800659540e-02 2.909011359029900837e-02 3.369306100365274809e-02 8.313852232573425843e-03 3.703631539139360523e-02 3.221476361655024184e-02 5.004279978496001258e-02 1.124726131269255212e-02 2.075423593976756173e-02 2.516531936834364228e-02 2.865805957742863577e-02 3.591562626861261216e-03 1.210952332290104842e-02 2.699905233859530851e-01 3.365740465947281712e-01 2.220194446229989127e-01 1.680723711657811925e-01 1.257845355571403922e-01 1.750037286095435796e-01 1.386823669455624630e-01
1.265137961597213034e-01 -7.258874828672221002e-02 5.698133289794202194e-02 2.410120262149355666e-02 3.964033965647847935e-02 4.016751550115046920e-02 1.250814053948251514e-01 4.316505255436941385e-02 2.680996337283395731e-02 5.350239894700389159e-02 8.267660767775582287e-02 4.156944302320168816e-02 2.134706834428773403e-02 5.171379139239958012e-02 7.790718477878109949e-02 3.246129665336565384e-02 1.343945140622496981e-01 -8.295896731732911589e-02 -8.475603316036289109e-03 -1.003803533480491644e-01 2.457076185057112394e-02 3.186749648001602903e-03 1.042216144504874362e-02 8.569641178856800268e-03 2.480415304362804454e-02 1.099688379301076713e-02 1.036070834339311714e-01 1.062879656259006593e-02 1.040103972575573237e-02 2.768716363989600672e-03 1.496733726803239645e-03 1.115797144961223116e-03 5.918097006100817752e-04 7.405126941865857598e-03 1.426602774288440734e-02 9.699236462505709047e-03 9.115795952599380775e-03 1.693566533083353670e-02 7.188776862776642684e-02 -5.194008692442755870e-03 2.127323261500056739e-02 3.712574975830810337e-02 4.194430965421107327e-02 4.524624346499936042e-03 1.429686301727136309e-02 4.308741608052978067e-02 5.618401874932654749e-02 3.021979324717372420e-02 3.628212537354411071e-02 1.679882384154449820e-02 -7.877006513681339417e-03 2.322150304205976637e-03 -2.439426191116802253e-03 5.141562050249589427e-02 1.596743193464225663e-01 1.723020714288924782e-01 1.276976993453262099e-01 2.077830161506895590e-01 3.693988606786550166e-01 9.403083129901135262e-02 -5.736707470550719140e-02 -6.623053461876702278e-18 2.603102139920241975e-01 -5.744532634333113230e-02 -3.956435329715613002e-04 -1.298388971396230440e-02 3.575855727243934523e-02 -3.485756598603884462e-04 3.260108825438177982e-02 1.771684929046062756e-02 -5.252151537405838202e-06 1.302195284594843787e-02 4.316411147627734202e-02 -1.803384077900176547e-08 1.406067619250130074e-01 3.453976750605559226e-02 -4.780508526392970164e-02 1.433999064434152981e-01 1.305151383275008903e-01 5.216736751264906130e-02 1.152032463984905414e-02 1.015151137125203801e-01 2.516242506945381985e-02 -4.157317871623588193e-03 1.068168198135042990e-02 5.519413423795706958e-02 -1.126049153753956552e-14 1.680241870465146864e-02 -3.714846305742463071e-17 6.195949816442137409e-02 -2.035038347049936830e-02 1.644370667770961650e-02 4.420353925780608900e-02 2.508316977718271640e-02 -7.988096694303361417e-03 2.102945100264588460e-02 -2.452788498133291403e-02 -7.277321496613680741e-04 1.803586370613997208e-02 2.334561615171998599e-03 5.267997864468784708e-03 -4.488360333802658003e-02 1.830256569734429670e-02 -2.204924238475284431e-02 7.064030496162353612e-04 -8.321366104739193192e-02 -3.969465864809247147e-04 -7.785618355901478428e-18 -5.672464056318600283e-03 -2.085884478324928973e-02 -3.269725758376000688e-03 -5.553393382751271867e-03 -4.238136168512672552e-03 4.507704894723275901e-02 4.121027052713150882e-03 -3.225657559604464680e-02 -3.849924460795851183e-02 2.079678751532610184e-02 3.816641364118030649e-02 1.266793090885235361e-02 6.629425640695073563e-01 3.519478822070480351e-01 2.519416206266125724e-02 8.808871215052060943e-02 1.873545744766137028e-02 5.457473596083805417e-02 5.201847317672585874e-02 2.054184022605196405e-02 1.508346255758088658e-02 3.156191871280093764e-02 3.297366199633383621e-02 1.640784162395147322e-02 2.321051152903897546e-02 4.385952566223501325e-02 3.063680076502883071e-02 8.397978583484444481e-02 9.546664801383192289e-02 3.052295711359914732e-01 5.029502720918824776e-01 8.375061914663179952e-01 2.491336699702788662e-02 1.380457229337620113e-02 4.252616888250731336e-02 4.328498255085361929e-02 2.121477495509620226e-02 1.416080207084294967e-02 1.796333601409095745e-02 1.383933972785774380e-08 1.743431096733099803e-02 2.424280216292169082e-03 8.276624778877385985e-03 1.084983715244178422e-03 2.138948644574419772e-02 1.237393502894806527e-02 2.100122458922338078e-02 3.318812552308393442e-03 1.154718939864715459e-02 7.047167703222045887e-01 6.810645102036432252e-01 7.808971658575847297e-01 2.993009959328445449e-02 3.689811813126469414e-02 6.186637632453012570e-02 2.775503113238739217e-02 5.021525576275375630e-02 7.361388514822019002e-02 6.113238565243322292e-02 2.106737817552671241e-02 3.119274341496428710e-02 2.476193863533396669e-02 1.680594665337631355e-01 4.148619987471502402e-01 2.835213883760885345e-01 1.781926131451356740e-02 4.742924015094398399e-02 4.163957963869486384e-02 1.428755585905612524e-01 3.303041650115148137e-02 4.342832567241371872e-02 3.720574985059299511e-02
1.512116669332846663e-01 -5.309738846094983616e-02 6.113566013248081937e-02 5.852950763452685612e-02 2.449617815597205511e-02 2.908660618314928489e-02 3.798662263019375596e-02 1.808935661315690460e-02 4.593706888073621070e-02 2.029514918711519678e-02 3.604398402095773668e-02 1.784413337530440374e-02 2.376102152658499458e-02 2.485793892473677977e-02 3.053705934816582437e-02 1.761024903387352000e-02 -1.760781604587680904e-01 -1.775990567039002488e-01 -1.802397571274763910e-01 3.849693784588440609e-03 3.579356058950140113e-02 7.069712552581344095e-03 8.784219533248170675e-03 1.316889933012760187e-02 8.918225934536240171e-03 2.504590821323260386e-02 1.799080392759712099e-01 1.332434776621914570e-02 2.064182119731449577e-02 2.857209821245917502e-03 1.375140726139665931e-03 9.509644382057798198e-04 -5.126561158669815099e-03 -3.163354624113028091e-02 -1.500251473084611986e-01 1.204888759643268914e-02 5.615238195801735763e-03 1.201733312583896929e-02 7.856682760761889497e-03 1.095608433250817859e-02 2.955213962502995298e-02 2.708698431208330509e-02 2.863363216768500738e-02 8.461705310742662425e-03 1.932181206542147103e-02 2.019508176372828909e-02 4.223251950032832353e-02 1.138628187228779565e-02 2.790409647274059990e-02 2.463569441894148185e-02 2.917106519146128729e-02 7.561216104922126487e-03 -3.791714707574511678e-02 -9.425938539409886574e-02 -1.155062635970793700e-01 1.026581257432826882e-01 9.501576651533565065e-02 2.424986292730660353e-01 3.378467628009354518e-01 3.625916276033390173e-01 -1.215646819938293460e-01 6.047542607663218724e-03 9.363373883770865005e-03 5.614886043214934591e-02 3.311512989701093090e-02 5.159742821231031243e-03 3.838095944250004277e-02 4.440662638061345507e-02 3.813708986621317659e-02 -4.620157811302443365e-03 7.661754322467876840e-03 1.706155731359048841e-03 2.292271567085219994e-02 -1.248153122175217795e-02 -1.032634726844302957e-02 -8.876551294256195684e-03 1.796798505311163197e-02 -3.353777683701138213e-02 -2.790853134239395725e-02 -9.929201617469395405e-03 -2.764281738445179513e-03 5.019663871046912973e-02 2.953974903603739932e-02 -1.019827159922653305e-03 1.126469677991843825e-02 -1.808157367575737333e-03 -1.079878892164786428e-04 -9.941848334933100009e-03 -4.841374875632078846e-17 2.294325926432524282e-02 -1.463083052153529851e-01 3.391145034202101144e-03 -3.952050523677018423e-02 -2.615057480210957429e-03 3.943578599621034783e-03 1.270630088607099979e-02 2.390190580977318274e-03 2.152245368219787505e-17 1.042752703536907856e-03 -6.587921859353253543e-03 8.426144898984394083e-04 2.160756474543804040e-02 -4.467641284714282491e-17 1.221898640859263308e-02 1.597687875925286162e-17 -2.487381703198796178e-17 -2.860650054047707671e-04 -1.897268440389954236e-03 1.682871533191041338e-02 1.043702730422547270e-02 1.760219210084673115e-03 2.313360030938375195e-02 -2.177397683150860547e-02 -5.165210393709050544e-03 -4.180745636144390237e-03 -2.968587237062279924e-03 -5.008675240594330025e-04 1.912611248862749330e-02 1.185876809397144183e-01 2.616310101596515392e-02 6.335797459567238388e-01 4.399771554356934344e-01 1.470657586368867564e-02 1.451936017570742282e-02 2.728246443107869290e-02 8.730087864930427544e-02 7.739805986953825212e-02 1.450492099239016931e-02 2.410305611448643540e-02 1.429010018043612787e-02 3.582205107576089814e-02 9.223115111088102738e-03 3.798080275605920236e-02 7.475629746873858894e-03 2.896294746789191549e-02 8.854553645705734791e-03 1.682406930472759299e-01 8.941015819655739882e-01 3.513369440404289779e-01 9.868709190473399021e-03 3.445095044684929469e-02 1.830183549836985474e-02 4.630159348454192941e-02 4.641448920094602693e-02 4.520513617713471655e-02 4.081197406209893103e-02 3.348429764299453576e-02 1.661788577877663931e-02 1.979627297874769071e-02 1.074768749414229708e-02 6.075374004282325067e-03 1.249203955740064292e-02 2.549379607018151805e-01 9.095033030540280450e-01 2.679815424613489205e-01 3.228617978535616039e-05 4.413766402728622543e-03 3.583027418837291152e-03 2.522078359960136171e-02 2.294379777455090447e-02 3.750452446347590785e-02 3.046659913679528389e-02 3.917933563862339985e-02 1.149051444575625164e-02 5.224828052440533971e-02 3.047734992553211009e-02 5.243270028927547038e-02 2.551769607619768301e-02 1.967978271391901054e-02 2.265295003337218158e-02 3.333348228436464750e-02 8.732968128340357246e-04 2.499798976261632666e-01 3.199855268449404710e-01 2.858074813117078561e-01 3.656069403226432785e-02 1.249901296740618395e-01 1.287418851148329768e-01 5.137784505499557591e-02 1.879198250364415073e-03
2.823077364955891566e-01 5.413401224116818830e-03 5.938325056972563326e-03 5.718342516710791731e-04 2.138918401460636505e-03 1.317865482762802919e-03 4.701871271234241492e-03 2.747913696866886944e-03 1.546062990800048213e-02 8.672090921490488435e-03 3.359758888234561005e-03 2.013925244609608787e-04 1.257202637328268084e-02 1.577589052015516968e-02 4.113922215992812242e-02 3.933505090022547043e-03 1.944573929058397493e-03 1.295291887860436537e-02 1.670072698407135472e-02 1.077229887510973601e-02 1.196677174083876499e-01 -9.659502578201922818e-02 -1.102514631988748845e-03 -1.105060856542752490e-01 1.247293721801084054e-01 3.503466290513050641e-02 7.408518297861781121e-02 4.077246558553684896e-02 2.507035481063735194e-02 3.501619387129124156e-02 7.100154518407245097e-02 7.630928238792278329e-02 1.119340255884885282e-01 5.449114800564006794e-02 -8.959495561510558148e-02 -1.456455353370020300e-01 -4.547108128421145645e-02 2.595851245220509931e-02 6.252717906650999413e-02 4.652770488307833921e-03 -4.537967181033546876e-02 2.431306487445862932e-02 2.110669673589253906e-02 8.268322256929593744e-03 1.609396751011165153e-02 3.353957083090939456e-02 4.899013425481544420e-02 9.869609688826866747e-03 5.781762174668605286e-03 4.447153859030437972e-02 -6.852308607988625254e-03 5.989670792426008555e-02 3.635711107444308116e-02 6.060278643991746927e-02 9.339414337846478620e-02 6.270405646567707436e-02 7.597176089697604828e-02 2.033339946706717305e-01 8.966256931055378854e-02 6.665260773382257475e-02 -2.186776620388382053e-01 -8.718782256769404358e-04 -1.085687145114434538e-01 5.089363546578983856e-03 -2.130973561785435427e-02 1.841558090751397800e-02 9.617903011364083155e-02 5.441977596761941871e-02 -8.896866222277870218e-02 1.869577372662460310e-02 2.035671034441761604e-02 6.837946158149456333e-03 7.415970448318929533e-04 1.471821212399442025e-02 7.171702245837331680e-02 4.789236805313991374e-02 3.223673549117605369e-03 5.404719299691650852e-02 -2.734331470941428477e-02 -4.871465035728543558e-03 -1.590205392120915689e-02 1.219917009393580581e-01 2.007012435801740491e-01 2.633230746132284583e-01 7.126507407568946628e-02 -7.635545255785253227e-02 -3.396604024940615553e-04 -5.063362755284928768e-02 1.019092745347642374e-02 -3.178208360137739497e-02 4.397573157815041350e-02 5.001923437165713832e-02 -5.379410836823035613e-03 -4.060361297849936860e-16 -9.063252711249204785e-02 -2.561390089702378012e-02 1.840775950984607778e-02 -2.031528733891654805e-03 1.264834447248901900e-02 1.243552745950187235e-02 2.667485661583715931e-02 -2.838540486182102626e-16 4.088079776857574815e-02 -1.822335555294633849e-17 6.249498309736195925e-03 -1.019645383620512366e-02 -2.474253596379562097e-03 4.024470230212765648e-02 -1.008476890258621327e-02 -2.017422538758532197e-02 -3.711348218697464729e-02 5.034303442134638318e-02 1.124663570317288933e-02 -2.622008081306671790e-02 1.289712138111955876e-02 -2.931727942105246537e-02 -7.349137338766751574e-04 5.146595187640589384e-04 -6.166178614688907003e-03 2.503185917422902189e-03 6.300731512547795843e-01 3.097563610096848996e-01 2.790445431790208181e-02 1.434415079176707231e-18 2.272971057294689151e-02 1.155044713050203467e-02 1.119049540813983612e-02 9.934730268020384655e-03 2.120230904783192843e-02 3.758156560428132630e-03 1.970808253689859679e-02 4.814636858643824498e-15 3.496323960909413636e-02 3.315448205789156555e-02 4.741823515059558070e-02 4.938736568607641000e-03 2.482611405326839299e-02 2.274894660700248206e-02 2.600407057599599461e-02 5.595537139685801548e-02 1.103129906381629527e-01 4.283029159815080478e-01 5.381861840059857238e-01 9.282498596965036430e-01 2.114119963600712485e-02 8.541974758931049350e-02 7.984894454251373341e-02 1.012412761654328947e-01 4.828632252004873282e-02 5.430891509472023787e-02 1.047920380871952412e-01 7.594183968840902133e-02 5.325602177248568020e-02 6.331832322619268538e-02 3.465098544089827981e-01 6.042189531945362235e-01 4.245343935051318351e-01 6.160046203461236325e-03 2.614967528052413934e-02 3.910251110474115971e-02 1.418925082217092210e-03 2.648729767077682984e-02 4.016254880736074701e-02 2.722353478601683210e-02 6.646503711535213521e-02 5.974781292444125008e-02 5.764396199070598303e-02 3.958538525704605182e-02 2.213864992793649622e-01 3.290388283360159982e-01 1.652244026053779069e-01 2.576094709825342294e-04 7.144718835233164304e-03 1.497069176490367810e-02 5.585048574721102749e-02 5.622551385220115466e-02 1.190423278115885358e-01 1.201545296134107210e-01 9.556494984672989079e-02 1.305119777587066855e-02
1.513979056084140962e-01 -5.922053421958918540e-02 5.672791351557274209e-02 5.557757321643302939e-02 4.760106551032572408e-02 3.668196765671510179e-02 1.396685853925682952e-01 1.036855701267785923e-02 3.977851091982368553e-02 6.428717461249147669e-02 6.260551907012895656e-02 7.090048119818284644e-02 1.726209066550216803e-02 4.329687277128864420e-02 4.870460698708713176e-02 7.975707146733410080e-02 2.265422707471711206e-01 7.114938482290428245e-02 4.209728549044700063e-02 1.956398130071720573e-02 7.508070105771889879e-02 5.137989912657599245e-02 3.025433931537175464e-02 2.203438358724115961e-02 7.149877153283742265e-03 3.303748576346981353e-02 4.435917922073430442e-02 2.029123438443487804e-02 3.434375561110600186e-03 3.746836500499053577e-03 3.733260498043340474e-03 2.113120451278205358e-02 -1.739053625915218393e-02 -6.851817474126289942e-02 -5.951400285240478205e-02 9.022332469911608852e-03 1.063721539730889867e-03 2.286531027028543619e-02 1.059781998956572414e-02 1.677479346286695275e-02 2.726946111217219693e-02 4.164678178712533096e-02 1.900073086404087397e-02 8.819511094251226979e-03 1.536647715225980017e-02 1.920029855969877450e-02 6.144055031105054299e-02 1.965897628284424617e-02 8.286267470952934575e-03 2.170916781376438370e-02 2.506110971223879380e-02 9.355435455101585376e-03 1.764735139874752320e-02 1.926587703832677995e-02 4.405183208560545238e-02 1.918979754413521407e-02 -1.728799038612526262e-01 -1.198878496356001744e-01 -1.877976844408566770e-01 5.461547008863036623e-02 -7.781355618248964046e-02 -1.175983277946660817e-17 1.539102177018119785e-02 -1.025009827975140453e-01 -1.893680619874357589e-03 -7.722696573893625405e-02 -2.978324925321731314e-04 3.866520077709716076e-04 2.908291926105609215e-02 1.248296680555252038e-02 4.276580713761224915e-03 3.960091627803443565e-02 2.677007486034922376e-02 4.746922854492442723e-03 7.692843891908579534e-02 1.357599386664619032e-02 -4.076707303919064468e-02 6.773296244895604667e-02 -8.634667773124665713e-02 1.340200123173015132e-03 -9.735956576307405047e-03 7.567657235085488432e-02 2.600982613189145587e-02 1.082848111763058847e-02 8.986156279157235874e-03 -1.226225238627920433e-02 -1.074271833970869033e-17 -1.066496188464836903e-01 -7.803474314215960866e-18 1.433806975311351095e-02 -1.563913491716808655e-03 5.758242747611990592e-02 -9.534608569158502245e-02 -4.673173524093358205e-03 7.532135949880964382e-03 2.127547556961486425e-02 -3.453560906512412494e-03 6.949225206148041192e-03 2.820178646269728853e-18 -2.855151262286316797e-04 1.845117175695867428e-03 -1.431395692875006218e-02 -1.500033533261107416e-02 9.047513792023356705e-03 1.062336647230591592e-04 -4.060479412704348004e-17 -2.594538803661596005e-03 1.322539462519541943e-02 6.658889196407035034e-03 9.081911531377245828e-03 4.295526984320991934e-04 2.070744729066041970e-02 6.574308833597869376e-18 2.449385607157803305e-02 7.083313259917270802e-05 -5.265296633456806145e-03 4.677212539224919093e-03 9.609960987552590775e-03 -1.550573912682784439e-01 3.312236738451718177e-01 6.735476139518773442e-01 3.846703383358368034e-01 5.367908938945268466e-02 2.585155936860574974e-02 1.837566850018545681e-02 3.122523391634450182e-02 4.407183183778931590e-02 1.193304437187920133e-02 2.835760780073916590e-02 3.365246581248498847e-02 4.679506797829063175e-02 1.269962535329324793e-02 3.997161367275148258e-02 3.638460675818137868e-02 4.544162630482943832e-02 9.893173807899890981e-02 1.627101310498405284e-01 6.657020917863377962e-02 6.190289451274767069e-02 3.438709339300569795e-02 2.007467066176905901e-02 4.953670755676228832e-01 6.926378576723958291e-01 5.756903310596134249e-01 -5.576556968630930200e-02 -4.966933264645521234e-02 7.097116246029640989e-04 4.776029518101427424e-03 2.993145028359674425e-03 2.663048020187214313e-04 2.997214671247039504e-03 5.671572454390857998e-03 1.984121023593324873e-01 9.613679369167951183e-01 2.647408165520175327e-01 2.179657214899679939e-06 9.662326317947918589e-03 2.529142298702448041e-03 2.363709696954391024e-02 2.136262468300869841e-02 4.592868370409100565e-02 2.685570255267530695e-02 3.475558415202181090e-02 9.671364542544287277e-03 4.181777251961164843e-02 3.498765213316846340e-02 5.440903545517206119e-02 1.799165531139206020e-02 1.346674044890934681e-02 2.070483494047702794e-02 2.949737877215965337e-02 9.719940361809046095e-03 1.273749687146723611e-02 4.144738466639686508e-02 6.406568661330964976e-02 1.214269845465440652e-02 1.770121318182487913e-01 7.340050375021087703e-01 4.536158408275056120e-01 8.212676598263534133e-01
3.263793600675997109e-01 1.807270376147564889e-02 1.111584441426930261e-02 4.427260639212332073e-03 9.698217879714183740e-03 6.878784487960823424e-03 7.285860766199423258e-03 2.912837958537587563e-03 1.316006235794302953e-02 1.621343031916643723e-03 2.427748658559266120e-03 3.362666010163572940e-03 1.456556026986659258e-02 1.288689277212758458e-02 7.209107276207521453e-03 -8.631072011952146961e-02 -2.576757420078388747e-02 -4.618375846027465009e-02 4.819352134343190402e-02 1.374242337497016686e-01 2.008669407014994357e-01 4.919158956549440215e-02 1.334404895846469201e-02 3.017639845853935937e-02 1.294223296481634422e-01 1.878343714344594614e-02 2.299525158553974991e-02 1.453889156944249139e-02 1.567733087046015855e-02 3.828270509351083123e-02 1.006325283075868299e-01 6.684978850285722141e-02 7.380854743488191039e-02 6.052994568286700766e-02 6.444709430035315545e-02 3.992074185257797614e-02 3.185179109218463228e-02 2.491324208991811326e-02 8.503368529520814734e-02 3.563005789000524975e-02 2.139643255249964372e-05 3.038874013761498280e-02 -5.279608798564023155e-04 -8.504541592434807331e-05 -2.401808146935357699e-02 3.275472232185170501e-02 3.321901400963302164e-02 2.977693103854194695e-02 1.774580065780468827e-02 1.917357847134140356e-02 3.812734413247424042e-02 1.494157774663197494e-04 3.703738593844587042e-03 -9.634442325062131732e-03 1.062448247902707812e-05 -2.603311923055005167e-02 2.668454413914224388e-02 -7.261713841680977888e-02 1.250957849040757949e-02 3.339015020841003267e-03 -1.817358672291908561e-01 -2.233942182850544300e-02 8.940795706811131404e-04 9.993531070885856914e-05 1.187675013912593835e-02 2.556045920257854487e-02 -8.552185594072510277e-03 4.293296022367730502e-02 5.159884730845643781e-16 2.462015820166353966e-02 2.442057529240902920e-03 3.025636308587452070e-02 9.361870808518479858e-02 1.005534123339809624e-04 -2.225171809242083246e-03 4.972857516468435525e-03 -8.169923608550410740e-13 3.931741975352044671e-03 8.424487461906618457e-03 -1.050781744258488283e-02 -5.150161030188544348e-02 3.184489463915166074e-02 -9.206855627457346397e-03 1.776503665585036501e-02 7.294041041916206336e-02 3.346439754317959842e-02 5.745247387954934004e-02 8.530929784482853273e-03 2.746881049666936424e-03 1.813947994926509363e-02 1.882167815737592620e-02 -1.810891142778429594e-02 -4.218013481548863776e-03 -3.378313636209021031e-02 2.091087001111220298e-02 3.520921178745577140e-02 -2.993243601612242261e-02 -8.284068847482638220e-03 1.405051732162596924e-04 2.442286161934330197e-18 -9.568976764137095484e-03 5.228125364530299309e-03 3.351005921121270881e-02 4.716742274881763342e-02 5.008744559073693287e-02 -1.046020126594072314e-02 -7.185171346719425312e-02 -1.668859808930186692e-03 7.931363103522159896e-03 3.361268221485858021e-02 2.091922204526252410e-02 4.359469904667184520e-04 3.186204152907112092e-02 -1.109420108625491092e-02 -1.621386964268728761e-02 -1.307844051481118867e-02 -1.580199677286101348e-02 2.009316725705942788e-02 1.334597475046171320e-03 4.964671362569941099e-02 6.087015845241469103e-01 3.532163902300528013e-01 4.253104339548020169e-02 2.982700887792137461e-04 1.480363099670704494e-02 5.017872276264965375e-03 1.548824016167402520e-02 2.085984522562015723e-03 1.738601518435594837e-02 2.034406950052174839e-03 3.333213120585303002e-02 1.079080994202185825e-02 4.169376016502540638e-02 2.120767556496462244e-02 6.826541870304414505e-02 5.732316463976290244e-01 6.976314125486107187e-01 6.935624439846582723e-01 5.492863303011844001e-02 6.476525902971448645e-02 1.619754332904492777e-01 6.874584655483469142e-02 3.096170245556335160e-02 5.446785773047318341e-02 1.125038779899559527e-01 5.429841548998587397e-02 5.256679746586042506e-02 4.589183867810436163e-02 4.966154083483287351e-02 4.168782407706773940e-02 5.808152181110982221e-02 5.295168316169037576e-02 5.582421292425922277e-02 4.749548104349340244e-02 4.195250595840839536e-02 1.029321853469434243e-02 2.938568671695288015e-02 1.367926815108380482e-02 4.515600318912415395e-02 2.231814403386467305e-02 6.362208713357657940e-03 2.693290381788906139e-03 5.201226715564440939e-01 7.287333790243084586e-01 3.966474986461903107e-01 1.409761831703964553e-02 2.005423954916909363e-02 3.003690413099024438e-03 1.764244005621848355e-02 1.618681945793115770e-03 1.339291238056581311e-02 -4.169196271318705016e-02 3.514225751639741060e-03 2.515968488515121781e-01 3.215347211235610736e-01 7.359349883980177420e-01 1.705964882202768151e-01 1.256569731225970488e-01 1.669823094216243975e-01 1.039327534929361807e-01
2.972605318632937088e-01 2.161601568428689682e-02 3.167824550949044707e-03 4.194929940507479207e-03 3.057774584741308841e-03 2.900825111809603781e-03 4.684827204568216448e-03 5.195350486751770455e-03 2.720954137769909017e-03 7.013863462912048275e-04 4.435250840806047139e-03 8.392693477760945070e-03 1.200312165932183699e-02 1.386735330198337821e-02 4.226494489109270530e-02 7.822653349765064726e-03 8.194427396613057288e-03 1.068144304178741087e-02 1.732450698725401111e-02 2.986732321848214675e-02 -1.557342528552257688e-01 -1.545470691540614672e-01 -1.471051707679747189e-01 4.787700406007970935e-03 5.698428418398095163e-02 3.946736373499124678e-02 3.813685927186313135e-02 6.440028822614177195e-02 3.699628220481400442e-02 3.454967155152289388e-02 1.553680047525676511e-01 9.438598528245452368e-02 9.502855687278834174e-02 4.394911438181250962e-02 6.235263924206138619e-02 3.518475614316248640e-02 2.175889104441354124e-02 3.326297774529022450e-02 -5.311234894579902610e-02 -5.543364751847842942e-02 -8.110291401826894009e-02 5.225398594003968084e-02 2.074122388045312668e-02 1.346893113829833195e-03 1.958922221398892022e-02 4.751415042241128456e-02 -5.254685391396795441e-02 -1.434176203085291840e-02 -1.819653438240679058e-02 4.337254215742821634e-02 8.573702007371664102e-02 1.812410544319909347e-02 2.289275461813223764e-02 3.017238356624513465e-02 5.413329827195530186e-02 5.922336260779327566e-02 6.504727701738725887e-02 1.397604756899821776e-01 1.208931337175472159e-01 5.847296510266201258e-02 -2.327617828428028002e-01 -2.640571195090944245e-03 8.879557296214504042e-03 6.116734851176586912e-02 7.618793618751210084e-02 1.492772827537207228e-03 2.808542840040040983e-02 -4.721810356564080539e-05 1.194552815870887977e-02 3.685534053440565894e-02 -2.058777589332555762e-03 3.659943717732678375e-02 -2.025584405958935954e-07 -1.695049137975254000e-02 6.033217580478273528e-02 5.673642510656742161e-02 3.647694781568204536e-02 2.182866675984703603e-02 2.154867259992493678e-01 -7.380157618839246160e-04 1.150962057611141197e-02 -5.732336105137796761e-02 -3.139563501361877090e-02 4.784599160076549729e-02 2.703129516346136976e-02 2.536602821821101056e-02 -3.860011150447025929e-03 -1.831263219721671401e-02 1.822387803200738177e-02 -2.933408363953083145e-02 -2.120054103602737189e-03 -2.733017546812610207e-02 -9.323857048389465746e-14 -2.080877883917108331e-02 3.886952890736706384e-02 -9.662245421410810275e-02 7.704251628364139869e-03 -3.167997773304716963e-02 -3.851808616314583711e-02 6.244456058867144110e-02 6.248437820236546481e-02 -9.648432337839905496e-18 4.358152601554875043e-03 -7.674775095970191863e-02 6.296420702939159626e-03 3.468570141041558887e-03 -3.237598284491640483e-04 -1.928269671543903635e-02 -1.599026808252340410e-03 2.223470303328093159e-02 -2.020117538126153366e-17 -7.232102992326619982e-03 3.944349881652482888e-04 -1.334074124966398885e-02 4.651315258737487301e-02 -1.572285945521267880e-02 -2.636214030873408002e-03 1.446565060243492620e-03 3.743004384603060980e-02 -1.498700563011940226e-02 5.671022986274334432e-01 3.100990836929115035e-01 3.488553329634336980e-02 5.635761449918777564e-04 1.861865081176488843e-02 3.019416778730386192e-03 1.517482628922576550e-02 1.028631425765320039e-02 1.820663033250675966e-02 2.209323883821772674e-04 2.335279242310985687e-02 5.263744219086204674e-18 3.585636882519636914e-02 2.797655389635640352e-02 4.940601537081904665e-02 1.671771154748559474e-02 1.584412460791704072e-02 1.890577461943420204e-02 3.241910295942834752e-02 5.447938894335243275e-03 1.635887151095810343e-01 6.599304771505144496e-01 3.254802604626839124e-01 7.730876922469214971e-03 6.752090009225231171e-02 2.219306172030417051e-02 5.428535262413817369e-02 2.418255089046542419e-02 2.169754657002165646e-02 3.570430217852049032e-02 5.874697514240209001e-02 4.528656631956621409e-02 7.532014324493607749e-02 4.603649337538032876e-02 4.385766078491240560e-02 1.506209817070114539e-02 1.744402602772628791e-02 1.938952291415680783e-04 6.094450976121417174e-01 8.810043216284960677e-01 4.547288676500962912e-01 1.086502691309980395e-02 5.946121415703955332e-02 2.525622089351399827e-02 2.275881958156204463e-02 -4.223045624574587459e-02 4.376072958004187385e-01 7.682053657390846002e-01 3.738941242110405683e-01 2.101923731622718904e-03 2.069924362312993912e-02 2.105586000043023487e-02 1.682751528562944146e-02 3.272717143925479161e-02 5.348091724671654584e-02 1.426454419019260872e-02 1.136538279652858224e-01 1.873237565950860301e-02 5.409063343408455210e-02 9.231173307796029792e-02
1.398435843605306261e-01 -5.391869730828387913e-02 2.334462158704867646e-02 7.192161848028047824e-03 9.467114032048353986e-03 2.442939751282831307e-02 8.158791726250610998e-02 1.195447597031157674e-02 9.889778663639971212e-03 1.955179869526599973e-02 4.807820882948436042e-02 2.889404641469841151e-03 5.710587549228499961e-03 1.353718766399922800e-02 6.303028373426586978e-03 6.062129048439971087e-04 6.887657879710740172e-03 1.560486753969343591e-03 7.913692589885593228e-04 2.610495451476706197e-02 1.072019405698973471e-01 1.326938623615542788e-02 2.381339191393164884e-02 1.114338083973449335e-02 7.333195831996449851e-03 1.192712962079128802e-02 4.480633414870316411e-02 -1.682706243363017995e-01 -5.721543655915052595e-02 -5.897609086072782419e-02 3.974434232027484679e-03 1.772636823444662677e-03 2.922556202605630822e-03 1.260716515180344073e-04 5.312326239230650074e-03 1.838162916963900274e-04 2.921506687214909690e-04 6.403354072529601022e-04 4.182141369127358350e-03 3.106330230220170907e-02 1.836733174692586554e-02 5.602802270566607790e-03 -1.119287832766094337e-02 -3.724684076445045414e-03 -2.780148676848649691e-03 3.189864125764857450e-01 6.682920619794234796e-01 5.413177676996444587e-01 4.553772328660325025e-02 2.039255856659680233e-02 -4.274084390792350779e-03 4.048205667124259166e-02 5.390099615469227945e-02 8.653837443713234545e-02 1.042861086503829721e-01 6.583959503276336223e-02 1.278554686611339253e-01 2.583142693923221600e-01 7.191024464256132176e-02 1.149463569843432037e-01 -1.139577275368366255e-01 1.750969306227261046e-02 -2.444552005245098289e-02 5.428112782310191148e-02 2.218613962010069010e-02 4.985216435937928764e-04 3.435375422145610369e-02 -6.343715647703048084e-03 3.301854739362152652e-02 -1.441539366464344963e-18 -2.265787072799547640e-18 1.996290720089475765e-02 2.939236080419812321e-03 3.874877168493476211e-04 4.127963238940255720e-03 -3.169395064639961290e-02 -1.272954925665243167e-02 -2.099863338128769158e-02 -1.962519686407944094e-02 1.067775098647760421e-02 7.797510012123959333e-02 2.180618550493692090e-02 7.766847581605658413e-02 1.670621981666118303e-04 2.753357127102546049e-03 1.650616749324163197e-03 -1.736967092992452652e-02 2.994547974973287355e-02 3.183545463769730816e-03 -4.913276257476148490e-02 2.776078870492967171e-02 4.925650055143471900e-02 3.098636697327003311e-06 8.099781234576507866e-03 -1.513824056172681618e-04 4.295365604440041368e-02 -6.702153189993956409e-17 9.464865284726212483e-03 -9.987377292331352241e-04 3.238924408178330688e-02 9.516146495096102637e-03 2.220723872332995563e-02 -9.104657292627341353e-04 -9.804388543294129121e-05 5.340621800526247631e-02 -1.166257530552808341e-01 -1.756453768135267726e-02 -2.911016119088927559e-02 2.787861882970571123e-02 -2.597906643584363753e-02 8.453951733145200891e-18 -3.129796286303428546e-18 2.737136766375870139e-02 -7.447802723321554630e-02 1.934310449244121266e-02 -3.119180959081150825e-02 -6.956510143285144117e-03 -5.808209839021949948e-03 -5.727795706573011840e-03 9.495843574541648537e-04 6.515681062641748289e-01 4.156424305252821028e-01 3.645416554166172607e-02 2.219172058229596303e-02 2.518652106981463662e-02 6.618805064656421711e-02 6.251876489931605663e-02 3.157734279558776364e-02 3.016154741250137716e-02 2.605660727727803286e-02 3.583219379413175171e-02 1.611757415109308852e-02 1.557103489994766769e-02 1.154216167880773788e-02 2.926107094368372852e-02 2.043519089991922486e-03 1.423096145500048725e-02 1.662163368863003865e-03 1.638348403853515795e-02 1.647317179699985673e-02 3.865704432635579807e-02 1.146188755040113659e-02 5.039199127925666349e-02 2.560511268946185498e-02 2.632567276657012123e-02 1.261422878407788759e-02 4.668581549395560935e-03 8.308856412530475355e-01 5.187502400026190452e-01 5.629288487307387845e-01 2.089556882796236859e-02 8.025182544194294290e-03 2.068461029612333330e-02 1.587444529964406742e-02 2.309197130999315895e-02 8.006141011561955859e-03 3.961179424708845785e-03 5.921235803298705719e-03 1.999241985743375680e-02 1.302871027226719207e-02 3.651152032458899327e-02 4.633611493100559670e-03 5.128638202691407910e-01 6.796622983243222338e-01 3.901223854416933645e-01 3.668433707021112600e-02 2.391998592259618819e-02 8.557652406055350769e-02 1.736698060625466167e-01 2.743700173858369951e-01 1.482027720261613057e-01 -2.782980266577377362e-03 9.028685899559039940e-03 2.896940946478827883e-02 6.823989763025228861e-02 4.679745166158930797e-02 1.246387432304318504e-01 1.023977429192963889e-01 7.640939055375896294e-02 4.061666711749707670e-02
3.210749764268155393e-01 8.548166302250593748e-03 4.646748244577758244e-03 3.519328117310658991e-18 1.162586756590536230e-02 2.479611708088550522e-03 3.483384105570877087e-03 6.259255050718479790e-03 1.609354863531048049e-02 3.339683985493177489e-03 5.553278804359583314e-03 3.670368964246597740e-03 3.189692629088321414e-03 7.706584747676712284e-06 8.180702644330472423e-03 -8.999679141878019761e-02 -3.144280695320901542e-02 -4.593986781786696394e-02 1.630274535872652461e-02 3.226456443769968631e-03 7.097625016958571250e-04 9.562943520477411134e-04 1.916134150183691548e-05 2.422182558931045265e-03 1.350036806385857269e-01 3.733273957050892156e-03 2.577709784298198858e-02 4.999689186287808833e-03 5.981945330122608734e-03 7.911537671134125441e-03 5.981423652001058300e-02 5.279552314843542760e-02 7.183406674127805480e-02 4.866471203446455507e-02 4.775703783077788617e-02 2.665001084507432477e-02 3.249062473584624078e-02 4.774450824423178558e-02 8.758884493468145627e-02 2.229901357752921837e-02 -9.032110609052686201e-03 3.790589977600443417e-02 -3.072681775962354876e-04 -2.795131905663714774e-03 -3.253922000096060030e-02 3.278783303056960474e-02 4.354884783236641355e-02 1.976361957745670014e-02 1.552342423778567404e-02 2.367593838371292655e-02 2.067410572887908027e-02 6.406171620164067236e-03 1.523704156767695730e-02 -6.817121529250705358e-02 2.875372153557238290e-05 -3.528149518893181763e-02 2.815640379455587247e-02 -1.176507823000566832e-01 2.722046184280431293e-03 3.745907285777989957e-03 -1.761955204210736148e-01 -1.978747296322767857e-02 1.155319779708610661e-02 -2.189840029881188338e-02 1.429562690534240746e-02 5.243569983803737788e-02 -9.443686048196585359e-03 2.400073877823737101e-02 9.010954393243444938e-02 8.582956789747421544e-02 1.605579930478563268e-18 8.364548205512870038e-02 2.645474029235808655e-02 -1.819043535006458073e-17 -2.822855222679439540e-03 1.055670668270365783e-03 -2.128966887938002245e-04 -1.340252476837483090e-16 -1.301260072638991960e-02 -5.971184724856984372e-03 1.322861106372594025e-16 7.863648069611599300e-05 -1.484402038396895806e-03 -1.038011600299572346e-03 4.192952475662624356e-02 6.384446107419901317e-02 7.885836745364261866e-03 4.293884858626348497e-04 -1.500565866545488791e-01 -1.654305354789651142e-02 1.179436078995182990e-02 1.250457538989500038e-02 -2.489819563119804579e-03 -6.071088769914507891e-04 3.268555791591622878e-02 1.082582027800247856e-02 6.970161898757428615e-03 2.938946997234873765e-02 2.193661908090636936e-03 4.506202307829728672e-03 1.428412564950057870e-03 1.679785423960384976e-16 3.342345672636203718e-02 4.144290573830242824e-02 4.609686545692483900e-02 -9.791880601651686139e-03 -9.349336286844832367e-03 -9.812417456404116878e-18 2.889253091199872267e-02 2.593489452884533905e-02 2.759397318947101260e-02 2.509322011753946802e-03 2.920913560066954254e-02 -1.130326447242017453e-02 -1.994334842706509819e-02 -1.106785026311337371e-02 -3.255121936795619952e-04 1.058161949233146944e-02 5.995969313461671849e-03 6.405684027676138148e-02 5.576791273819642347e-01 3.511037485283487469e-01 4.014088776336485059e-02 4.694130213076302613e-03 3.517138929537351466e-02 5.126190080073074994e-03 1.268130142871723258e-02 4.808606555554654355e-03 1.798635776943366404e-02 8.131565500051198903e-04 1.646735416286311884e-02 5.669769767888693454e-03 2.985410193576750956e-02 4.061271840488848289e-02 8.228295813259599301e-02 6.359585626346507814e-01 7.565954647823237567e-01 7.653519480775727857e-01 1.295996086563972784e-02 6.603631785033352991e-03 2.395231306903702548e-02 7.516849155478109480e-03 1.112492531986065437e-02 5.141243522589674861e-03 4.858210142910710655e-02 8.069317386479915241e-03 3.891802866064439370e-02 3.039282581127253358e-02 3.645856410678299103e-02 3.269037144936887823e-02 5.679860162568337034e-02 4.408755578920717927e-02 6.261334282696705278e-02 4.692438725730448124e-02 4.648164856703877745e-02 1.292810137136754696e-02 2.366656666033727097e-02 4.062330127695242682e-03 3.709222983662697343e-02 7.655495396727759159e-03 7.969499838119493626e-03 3.903506990507557214e-03 5.376546303350338007e-01 7.154985707203888046e-01 3.841435241830146730e-01 1.628341890465448763e-02 2.204246197484503733e-02 1.117740757115361387e-03 1.548561109805116250e-02 4.723928595868036838e-03 1.535151949483107812e-02 -3.076898505512414353e-02 6.280700732365075874e-03 2.562458555598796095e-01 2.937410955192887863e-01 5.441374615816026994e-01 1.647326327228715304e-01 1.399184710243631946e-01 1.272767456835020017e-01 9.341121328461074458e-02
2.101836541329388242e-02 1.141102475104662818e-02 8.551805728969792946e-04 4.851825438216615487e-05 4.485740277181037884e-03 8.823834196265830862e-03 1.848363035343266944e-03 6.081276199697290864e-03 1.778655830381549105e-02 7.262983697534353827e-03 1.124964137008908557e-02 4.909986203346480144e-03 8.957020698974747439e-03 4.140409949424159186e-02 1.355585635060206282e-02 1.734583653446392960e-03 -2.794963245137648596e-03 8.156264738187724364e-03 1.461815182799931778e-02 1.231471595868534503e-02 3.821309793211252921e-03 2.720232754795853369e-03 2.520166039986538068e-04 8.060414108717207232e-03 1.157475143476013463e-01 3.036700798034538268e-02 1.768836095870829003e-02 1.656471449044424893e-02 9.684741286739169050e-03 3.026780519061033140e-02 8.330178020073494838e-02 6.501024904775599966e-02 1.407475704447378184e-01 4.161532919108096827e-03 -3.330744828408780528e-02 9.153666695398897726e-03 1.319873360834716500e-02 2.306904170978042734e-02 6.534852930421183870e-02 1.198062811956784879e-02 -2.577274340233208158e-02 3.562403494661828957e-02 3.370179736935128428e-02 2.515443881504472387e-02 3.804838923827844055e-02 5.224914584870647000e-02 9.232978594243781978e-02 5.189966500943490185e-02 1.769310740629376211e-02 -6.927111665858729161e-18 2.226576538798298241e-02 2.142757858960933626e-02 7.018289925288936915e-02 7.529034726008279610e-02 8.163364803801195779e-02 8.465249842310963724e-02 9.990579348813612148e-02 2.469046116730585771e-01 7.653379521466308066e-02 1.527656466834981797e-01 -1.454820950647133480e-02 2.111488661984083639e-18 6.661189135610784406e-03 5.115802240315565214e-02 -1.005199577751055354e-03 3.884948598410169901e-02 1.134522122831287984e-02 2.644653604860760371e-02 3.305924908579416233e-02 5.359986288365261115e-18 6.478630044933246512e-02 2.272977792387506114e-03 -1.954000512437218968e-03 2.692482807148815506e-04 8.745184784592616167e-03 1.602921510833663077e-02 4.716088684408079740e-02 -7.464943860169272194e-02 2.552838984282802121e-02 -1.387326998921678024e-02 -6.102665558462602724e-03 9.064315699635880135e-04 2.850721575624249812e-02 -4.454072488074846403e-02 -3.511261424374619716e-02 1.114895645235101755e-03 4.232946654762168720e-02 -1.248000350108149914e-02 4.084827443837991262e-02 9.813466142626812994e-02 3.074053478587741514e-02 -4.436703309679609214e-02 -6.695998951271561511e-05 1.908384801097078864e-01 3.006583687165057320e-02 2.772793543503747349e-02 7.786773826408857115e-18 -1.215173556297631345e-02 1.172778124976386850e-03 1.157026400469567291e-02 2.332667491183514769e-02 -2.580938224223657332e-02 -1.262000530986069104e-02 2.423127658099680920e-02 9.189514547958695101e-03 -2.219289336979308502e-16 -3.001032313952079226e-04 2.180882286718782009e-02 -3.277071600465947905e-05 8.437691163629438759e-03 7.619195191860017070e-04 7.833065775389646754e-03 2.976247769669247353e-02 -7.557918589059017811e-02 5.656311089468897373e-03 -2.959056448175392470e-02 -3.443451623942286877e-03 -1.288801945643293541e-02 -3.311721954460041927e-03 1.838127507842176895e-03 1.364595375487149924e-02 3.622925598621516436e-03 2.710469319851787837e-02 3.429326939053641335e-05 1.427508190056494923e-02 -1.011595219328318036e-02 2.119823104589000848e-02 1.121793032891872134e-02 2.015101248450855073e-02 4.301695018509702254e-04 3.808640196193375377e-02 8.807133491063009675e-03 3.717262966084189380e-02 1.625784968901501093e-01 1.100346732853148919e-03 -2.227319789961357266e-01 1.927752735997244615e-01 1.006517760577283795e-02 2.432310904139998661e-02 9.471243368436842516e-03 2.932088490593190086e-02 6.031937976882183888e-03 1.954304696913543651e-02 4.648767577914610961e-03 3.174670839132952282e-02 1.732636531403641475e-02 3.834441206739284957e-02 2.355701761434264921e-02 4.262441427758151963e-02 3.847270098582364350e-02 5.428207788511271031e-02 1.980816711898110427e-02 2.372108492919116837e-02 3.996653152146300569e-01 5.141953623772376192e-01 4.929848937930116914e-01 1.791250784003863006e-02 5.683597655572532326e-03 7.216302962750884009e-02 5.131823190822908110e-02 2.424645157944192459e-03 2.098889156691104846e-02 3.965136187832951631e-02 1.723449726076497004e-02 3.705001133316460082e-02 4.961634119725918252e-02 4.861259983114715050e-02 6.462229494633563599e-04 2.436771035596733415e-01 3.843882036522017764e-01 1.961848940244070028e-01 3.553166733649382528e-03 8.299296883450419868e-03 3.268593469145066738e-02 4.821711680836637121e-02 1.848012400220270732e-02 1.090312391989278207e-01 5.398858726034582584e-02 6.877771789270695213e-02 1.817780475257645134e-02
1.410964449306155999e-01 -4.736162213910204838e-02 6.603876945398844123e-02 4.463181041203968802e-02 3.366063123336533042e-02 5.069244802451895737e-02 4.874358481056270676e-02 3.359630532382379409e-02 3.854043377144614202e-02 3.655396717381134697e-02 7.563695867362439651e-02 1.794438960800533514e-02 2.578809308244336268e-02 4.739689456593538380e-02 2.860436321453294420e-02 4.357872566302272571e-02 8.185950705393618687e-02 -3.141219410272909351e-02 -2.241708133916779644e-02 7.208982617046751432e-03 6.260589576892541808e-02 5.762022555640442835e-03 6.070410266667205720e-03 4.071942393403387393e-03 1.009191651512052375e-02 1.426529388344793293e-02 1.921847164851126291e-02 1.594760642123491468e-15 4.797397689041220680e-18 9.823726554673039776e-03 1.570405873982487705e-03 1.561634690829269336e-03 2.040842325910171857e-04 6.665718773512116502e-03 1.560654823689614293e-02 5.764309254523422750e-03 9.025226595758608234e-06 4.158491552065727012e-08 4.656679682005670194e-03 1.576540073447851506e-02 2.196615937191954970e-02 1.885817161034558653e-02 -2.779158927261534805e-03 3.209992802675724205e-02 -2.974165408250623727e-02 3.095956874575054446e-02 3.740016912749712791e-02 2.476559313415263239e-02 1.560899552813412548e-02 2.531661566378819173e-02 3.616158235210797239e-02 6.816512941638288442e-03 1.976975913563356474e-02 4.236156090006915065e-02 -2.797436711439136858e-01 -6.203912756062644751e-02 -8.949938900417023224e-02 2.488941898615260884e-01 1.508698179833995456e-01 2.717573488926180447e-01 -1.100371394542767473e-01 6.634742979323727044e-03 6.974018216931776593e-03 -2.048043240723932800e-03 2.014104731450789415e-02 9.041707285292358703e-03 -5.191926560259448563e-05 1.603107764649128533e-02 4.547290155997740085e-02 3.509810709952459703e-05 8.032335334785268774e-03 6.289825177828402281e-02 8.540082085230007885e-19 -1.484317459550131904e-02 -8.229398055463979511e-03 -6.875241040878411584e-03 3.108911288412908452e-02 2.587567103267217339e-02 1.309942546119077095e-02 1.639537796027946692e-01 -8.099009902750083784e-04 3.365511299810265006e-02 3.924934481692335386e-02 -2.518198560921767939e-03 1.817503245479107354e-02 1.422972674208849552e-03 4.199400836522943631e-03 1.739670368031374925e-04 -1.387484082217763979e-16 4.400219626914583099e-02 -9.376689875705040067e-04 3.162510082873598583e-02 1.702164911326803018e-02 -2.591791528241695064e-02 -2.988990930051425628e-18 1.208677378186118201e-01 3.298248828357470269e-02 5.786169777664467242e-05 2.587857684716920060e-08 -8.944089506380708751e-03 3.485960312285004033e-03 2.270934695799978545e-02 -4.348041567566640793e-03 -4.120176561311128353e-03 -2.149880321328924627e-16 3.896120403044199022e-03 -1.426553005191743195e-03 -8.995698329827011728e-04 -1.817883934885301464e-02 -4.153614501511251715e-03 4.657539513898371203e-02 9.656953439020660168e-02 4.444132742579928914e-03 1.971067588141067490e-03 -3.482897181713356938e-02 1.468279095777764130e-02 1.612012107453836968e-03 -1.072560789367878048e-01 2.259707275200985854e-01 -1.738024468637717757e-01 6.313341232578894013e-01 3.973068269951242626e-01 1.400073888320894563e-02 9.238123577142298357e-03 2.573405772612080700e-02 3.568563983029075098e-02 8.585227390427062344e-02 1.129740107932609289e-02 2.832663834143889572e-02 2.846665760904017531e-02 3.975507850524732240e-02 1.893487105064270773e-02 2.051973561971483229e-02 1.749157949245370991e-02 1.778183110436611439e-02 -8.462193287313585915e-03 2.525275800105639123e-01 7.913220200637459678e-01 3.855324971376637433e-01 5.226841844681940144e-03 4.127469042493668583e-02 1.218256360611554533e-02 4.982004939474260485e-02 2.364008012511996729e-02 3.117711549138509411e-02 2.748356426127066335e-02 2.048730948177217639e-02 5.529502361326795248e-03 5.741009402761845619e-03 4.055191871102773772e-03 6.571639060218360459e-03 4.199943663322970588e-03 2.114590802891719995e-02 9.397253990597312021e-03 1.231123046625691442e-02 3.073103270584017638e-03 5.974151274649021869e-03 8.966525035473807415e-03 1.444369200479728166e-02 1.434653636125511091e-02 3.323482983684678677e-02 8.083345305285290563e-03 5.187986192886661074e-01 7.476777517173344156e-01 4.862139353801356867e-01 5.162637566666780642e-02 5.073341749639078552e-02 1.768319367190573574e-02 2.010005797444983436e-02 1.506173482800489231e-02 3.459198783943352729e-02 4.189343739413003802e-04 1.664853443159137331e-02 6.852336107810188398e-03 3.182520215282644283e-01 2.714225549341470201e-01 3.287835675451958317e-01 9.539167824497726822e-02 1.295796375065402239e-01 4.977926154009771342e-01
1.451871531071751709e-01 -6.758147148764966083e-02 6.840274200222261347e-02 3.077773364238031717e-02 2.390230939061601181e-02 2.242796019325321924e-02 7.625768675886077097e-02 3.362576896445410024e-03 2.175829145835058917e-02 1.334780697537457303e-02 4.981412575750956279e-02 3.672050587076959039e-13 6.678758163653251458e-03 2.177623018459772311e-02 3.609101179351540539e-02 1.075198071449031713e-02 2.290080924650985880e-02 9.317274225464274376e-03 4.740962221026800625e-18 1.720738630366807492e-02 1.335961623002544507e-01 3.002176036605022982e-02 3.333759153945884540e-02 1.775126967803179051e-02 3.004187991805203073e-02 2.139694384960620296e-02 -1.135377719850783930e-02 -2.554092439979449803e-18 6.457514007675994904e-05 1.239475842327956295e-02 5.996727644163354909e-02 -2.259270788975150997e-18 -3.246057240703169745e-03 5.389534191398615351e-03 2.663183228096151732e-04 6.425940465840254169e-03 1.657771935241042477e-16 1.225494316807385076e-02 1.171713815695649488e-02 4.417089722269231314e-03 1.467847471911960279e-02 1.711129799976954940e-02 2.304978623775958552e-02 2.703708243551919497e-03 1.217912722510337137e-02 1.692259138443348229e-02 4.356682521222051546e-02 8.305733494572321504e-03 9.040988866044543174e-03 1.840908602197758107e-02 -6.144323430422718446e-03 -1.174468223490991536e-17 2.005577987917183183e-02 6.291853880316415626e-02 1.232362899894227787e-01 1.556824544614894001e-01 1.375466612503588315e-01 3.580587512916926229e-01 5.675223760333771850e-01 8.993091764188455850e-02 -1.083243979178953181e-01 4.034431621210095298e-03 -3.624693808366562936e-02 1.554229306052058203e-02 -1.954578521878023897e-05 1.397945784217577949e-18 1.239900376195808215e-02 9.008987056789069308e-04 9.048093044583975508e-02 1.522006638065350435e-02 -4.799668831139133418e-04 4.885610882711183094e-03 9.409063610703028140e-04 1.834250681512817911e-03 -8.233236288985635448e-03 4.572051874920484956e-04 -8.532004953252573382e-03 3.698543150649970246e-03 -2.031763842937364672e-03 8.203956932814223529e-03 1.137559124188784222e-02 1.812467374763486461e-02 2.582525755076358828e-02 -1.437917526552809095e-03 -4.028459613505613358e-03 -2.144403906386701304e-13 1.776747118094868915e-03 3.995038914170423623e-03 6.933713689919793373e-03 -3.540769934722153706e-02 4.613020790081442252e-02 -3.025939351596430502e-02 -2.588989605389115795e-02 8.736882789719698991e-03 2.703173815960392690e-02 -2.891511807080494531e-02 4.147812181701943102e-02 2.386840376785190712e-04 2.204336591232784258e-16 -2.342126500449718488e-03 9.236135775213141427e-03 -7.316780395998419978e-04 3.235118795764132388e-02 2.015747143793219370e-02 9.625502386537230365e-03 -4.555873300754256783e-04 -3.563561055267949751e-03 7.326358156542628342e-02 2.906212170989173893e-02 1.519627564832805831e-02 -4.393293583962004645e-02 -3.192295368781095793e-02 -2.614244984030381427e-02 2.522631500054710665e-02 1.611367469310207717e-03 -2.983089147209543740e-02 -2.765669157036134210e-02 2.144213608518389519e-02 1.527261886937610752e-02 6.105332212469036929e-03 6.371736626121508040e-01 3.693073872475923225e-01 3.477818061718399717e-02 2.832448593311318041e-02 2.454940806735334338e-02 4.767450145232367958e-02 6.086910163172405686e-02 2.699229706654848390e-03 2.285669261047893960e-02 4.235921022489164828e-02 3.190466053162621662e-02 2.767470453858829338e-02 2.296987477783752132e-02 1.680966999737619483e-02 3.181457696395974255e-02 2.320690889231656617e-02 3.589886501531702051e-02 1.795222226748050681e-02 1.357705327869786378e-02 6.244275168681553620e-03 4.274994655824827794e-02 1.196293178942949009e-02 4.358977695887938236e-02 1.581117475323188734e-02 2.485299429079867659e-02 2.179850817967940099e-02 2.754496071052368530e-02 1.204209843672751141e-13 1.094256075057495510e-02 4.970825146142654298e-03 9.231948119139535280e-02 3.358462963205396251e-02 5.932105084092365727e-02 2.649687970201874531e-03 1.320913383828826830e-02 7.744720615623995000e-04 5.939782588113428838e-03 1.415868536425292194e-02 1.811957594336535693e-02 2.581313322507114774e-02 4.345772143411948119e-02 3.922349561527456452e-02 3.746326931333426868e-02 1.665322912173803835e-02 4.604268325891251040e-02 2.930905715562473479e-02 4.638571858874983300e-02 2.500289019115553915e-02 2.155221304108648900e-02 2.362331132483922597e-02 1.225139115881626606e-01 3.942904711252596672e-01 2.797312654287233968e-01 6.035977478670068522e-03 4.443217475620292456e-02 5.719021006591844952e-02 1.157329438190755205e-01 6.067634067002398990e-02 4.139785625624517462e-02 4.774583851729272133e-02
1.341732382595000606e-01 -5.539511573918862869e-02 4.826685979969117662e-02 2.884134351892718787e-02 1.696444281945196780e-02 3.240136759405118361e-02 5.150804356784418475e-02 7.134769850518033613e-03 2.738890620011423527e-02 4.996201890782923580e-03 6.237720787707744935e-02 4.730696829484846988e-02 5.199683178706409128e-03 1.544365899370560812e-02 1.230274001221711225e-02 1.853051768734863616e-02 1.907686947174435169e-02 3.903645051427922109e-03 3.765262681456122557e-03 1.976846885058896122e-02 1.109090299605261093e-01 2.983462123846095451e-02 2.495066813350613893e-02 1.996803120549396626e-02 1.089440483636713169e-02 1.502242912580069659e-02 -1.621078105783488599e-01 -8.306271991590249981e-02 -4.812084848621567268e-02 -5.291250876695238747e-02 1.362166068131931071e-02 3.910365182326060174e-02 2.006710255083283059e-02 -6.439116754484580651e-02 -5.142112534961351157e-03 5.073378008844868330e-03 2.014892139020316424e-03 1.097800769288963946e-02 2.485098333850875585e-02 5.675483856295319637e-03 3.664174425750015701e-02 2.769325040659945114e-02 2.585768261809058902e-02 1.613325404758848836e-02 3.137415463815278194e-02 1.862146150813710418e-02 4.163079349385928607e-02 1.495787047141233436e-02 5.343790278687496863e-03 1.248939382292466385e-02 1.172927148315564862e-02 3.558756684112534575e-03 7.471026098403260769e-03 -1.777458017723334617e-04 9.095782152247124802e-04 -3.332892076454489066e-02 3.815233881130798005e-02 -9.170194459756750294e-02 1.148697958444333013e-02 -1.727378346012847093e-02 -1.024409565613853962e-01 2.941909549467299975e-02 -3.226112178373935491e-02 2.283244465135184087e-02 -1.962386131921308938e-03 -4.249837919956913940e-04 -3.169514484908591916e-04 -4.222221667616859325e-04 5.500112333352892569e-02 -2.813459973434520849e-03 -8.215091025423915463e-03 3.912745530469143029e-02 3.020616090353814114e-03 -6.971810979627261415e-03 -1.221450816298757375e-02 -2.283474777366241871e-03 -1.286443123783647016e-17 -6.936910818421955331e-03 -2.799263644044333241e-03 2.894710678957010722e-04 -1.523527451206288069e-05 3.628917882252081454e-03 5.624502894106499923e-02 -1.201561984688980476e-02 6.260184692739775500e-03 -8.259787608748129150e-04 4.607099395066895781e-03 1.125640500128201500e-02 4.574556425580521800e-03 -4.090434701237141008e-02 -7.418416382828814382e-03 2.461709443743822362e-03 -3.055363648501916579e-02 -1.247924640335404227e-02 5.976637506717730197e-03 -9.747271340599314438e-03 3.716488299989407010e-04 2.745173909160474046e-02 1.066728578404185519e-03 -2.265112455801979174e-02 4.099308692293329726e-03 6.990873732562024809e-03 2.687783268978643861e-06 -4.427884235695037369e-17 -2.934994126318653685e-03 7.097133730912496804e-04 -7.604160733905143688e-03 1.888775113605365222e-02 -8.334625742461546256e-04 4.793477106370726964e-02 2.100661290698595599e-02 5.059289900224472891e-02 2.766930469770649548e-02 -1.618954363482292408e-02 -1.880529350071251729e-02 -1.530321611354970579e-02 6.024446306228719607e-03 2.319538926677265633e-02 7.145124587599600841e-02 6.244271679108469308e-02 6.593783576654669298e-01 3.983808443015869893e-01 5.898385283829407633e-02 3.798928510466940967e-02 2.072894723253621355e-02 6.174978346260039802e-02 7.974066988754957563e-02 5.804271334310456658e-03 2.678828244131141803e-02 3.286505140717902590e-02 4.328321767641114121e-02 2.498204515271227125e-02 2.190580664188881282e-02 2.067730870013208266e-02 2.478110412667374612e-02 2.320832149445049494e-02 4.760236064662286964e-02 9.824316416132558571e-03 9.360347492547611836e-03 8.525018861469500428e-03 4.171251207292148849e-02 2.597327616960679688e-06 3.377361831965178945e-02 4.416165164880285487e-02 2.467431841672761833e-02 1.007056681349306412e-02 6.496569378395187222e-01 1.337055714758253577e+00 4.694048982470194198e-01 4.985802175120416235e-01 3.068994707075137218e-01 3.565157204393034074e-01 2.713099881710445138e-01 6.449976238957013663e-01 2.391373132652890865e-01 3.840196180752857274e-03 9.516319624616234249e-03 8.854333266476204639e-03 2.627791878018069169e-02 1.913636129856478504e-02 4.519855329195184707e-02 4.291249777687288630e-02 4.187020693934489363e-02 8.171351292008924294e-03 4.222204593902750130e-02 4.586767764192158092e-02 3.889815399363805160e-02 2.210309737249255974e-02 1.783444761420830554e-02 3.178422311192368593e-02 3.828975315530120521e-02 5.317758596040532492e-16 1.013854122740965626e-02 6.613174801410165449e-01 2.414276381072817668e-01 6.142467045421091676e-01 1.439284903696438911e-01 4.106451642478755465e-02 1.612121695495409290e-01 1.148985436911399982e-01
1.134662169306662888e-01 -5.333872054220446107e-02 2.231607572007136830e-02 5.381490423109548324e-03 1.141900049450566591e-02 1.233074211708673311e-02 1.302270571520667830e-01 1.414484027877650416e-02 3.166069620805495033e-02 2.280978412225189919e-02 6.419431735106362336e-02 3.134261364923499743e-02 1.318891592524366538e-02 1.398155681697671980e-02 3.027998982559379146e-02 1.026417291519448813e-02 4.996075836174215801e-02 8.931311711869586586e-18 1.273748973210138136e-01 5.222038317003059593e-02 2.355026015596366207e-02 7.388081894122347677e-03 1.369324811998599845e-02 3.551389636456980803e-02 7.736374952731683331e-03 1.582270899502312769e-02 2.913583241655216816e-02 1.312943556789922077e-02 1.730633328915723231e-17 5.113074131272880374e-04 1.545717670758172584e-03 1.186500071470715512e-02 9.939807951346538595e-04 1.025027582339541922e-02 5.625114196628850857e-03 2.816313869518548719e-03 6.971886257929318855e-05 2.947729400799559835e-03 1.263609492396261524e-02 3.780426689239357874e-02 1.539926964685627703e-02 3.792466643168626583e-02 -5.779748289536900535e-02 9.564918920695144634e-04 5.445446021346865735e-03 1.647163225962143627e-03 -3.591901624331046705e-02 -2.215199781922497398e-03 -1.126712826503991835e-02 4.548759493290131795e-02 8.635704394002334527e-02 2.738795721634933872e-02 2.641217586747889728e-02 1.925267025819565533e-02 4.314782703226648780e-02 5.935571529498896021e-02 7.622626296297849158e-02 5.305705853409174722e-02 7.334148264701581543e-02 3.617716565104602683e-02 -9.912488338094894991e-02 1.793913494579911375e-03 -2.302706115367052497e-01 2.328124970710351849e-02 2.787047389547684523e-02 -2.139730967081440033e-02 -1.941547559989860041e-16 1.699190922338816856e-02 3.500463517672603581e-02 -7.508249270677228457e-18 -7.200754956804764940e-03 6.516148286535843916e-03 3.640629750638023305e-16 6.374747873823095878e-02 -2.042396983380113940e-02 9.077003513131703949e-02 -1.026361203147783778e-02 1.469276153531969345e-02 4.448518901225556599e-02 6.228386813348088524e-02 5.439072146605595376e-02 1.195037620939022605e-02 1.255860036917412839e-02 1.832850138766329781e-02 3.778659044412101436e-18 1.119328017058233982e-02 -4.310451927561338801e-04 5.288701026105554343e-02 -1.466855236923103412e-05 1.413523068420755209e-02 1.001178310690039629e-17 9.351015054764459308e-03 2.086791808747072777e-02 8.205150417941917129e-02 -7.224068961613306446e-05 2.587960091287084402e-02 -1.872885982900726609e-02 -1.162245401676981205e-02 -4.071051775431249603e-03 3.152002000961676154e-02 -2.405162684464302655e-05 -1.151323383833176804e-02 -6.314066246692884993e-03 1.137214274666637805e-02 1.385721797170894054e-02 -4.865008870921476436e-02 -8.499637219998076171e-03 -8.452378377581178104e-03 -5.315789069710288774e-04 3.281649435392667430e-02 -5.615445345494137147e-04 -2.701417523598430639e-02 2.759651264764437055e-03 -2.676362221997871313e-03 2.029772050601419756e-02 -2.181923932536459360e-02 -2.040685430962212729e-04 2.473388534477084419e-04 4.153684615727360613e-02 -2.375976238366642472e-04 6.470514498366158884e-01 3.674651708281992946e-01 8.155986516081693838e-02 3.390791084382110271e-02 2.218449425086676391e-02 4.923994042463012560e-02 7.025473581994448602e-02 1.465534257083993784e-02 2.140356393179730643e-02 1.668608433528372820e-02 4.602297257575967832e-02 2.544908127961229699e-02 2.519451593860033353e-02 2.010701855159072876e-02 3.723518394202895293e-02 1.213985129041262809e-02 4.705898481394002464e-02 4.858395413008758656e-01 4.823847026331813947e-01 5.140502250423173436e-01 1.873877928019134290e-02 7.513394203858529512e-03 4.890749442692304827e-02 1.724538081514182669e-02 2.720029976985326864e-02 3.417992568890906857e-02 2.567391370527602468e-02 3.563117705965174100e-03 1.090226198807972037e-02 2.016868810619691383e-03 1.787726805018620760e-02 5.351540047654403109e-03 2.243187585168658627e-02 3.307697569346107220e-03 1.952594157631429642e-02 1.457674875429801082e-03 1.869043901251576240e-02 1.161263646849104367e-02 1.512568951451151444e-02 6.412898237298515491e-01 6.319130417498594054e-01 6.576781173004955239e-01 5.335681381308178539e-02 3.942276240156639133e-03 4.080787203309344663e-02 2.835124518000551458e-02 4.915250787231108531e-01 7.309296392469869508e-01 3.221157966228228031e-01 1.058929651521352613e-02 1.432082876873537990e-02 3.693673078706082198e-03 1.603381895346841016e-02 3.437885249810818111e-02 6.990448792241302223e-02 4.021572369974233535e-02 1.083438215052031361e-01 8.674743269802780155e-02 7.856836347664078457e-02 6.001550422702903176e-02
1.326510793462771787e-01 -5.832217473031039989e-02 4.158854314949767678e-02 1.753516377614385002e-02 1.848966182815429959e-02 2.542993446666468385e-02 7.786465629418286294e-02 7.281899695064084077e-03 4.282656992451792932e-02 2.184521171790016272e-02 6.137956216141243065e-02 4.265577600887304754e-03 7.430553211679204011e-03 1.135788296313905779e-02 1.800818847866205580e-02 1.486106765393549294e-02 4.824856077864239125e-03 6.903554528017697388e-03 7.717909598496644399e-04 2.723785465704435180e-02 1.918365500652619293e-01 2.034876148417690683e-02 2.424765376148579205e-02 1.610346517611550843e-02 1.636846178937328630e-02 1.274898948542245274e-02 -4.838306746800907776e-02 -5.575935846781532312e-02 -1.357620671120899876e-03 -6.535317543960987574e-02 1.249582567750010496e-02 4.265022566062829507e-02 5.093479150132276118e-02 -2.636162926364408673e-02 -7.338808349605402404e-03 2.526510594375383587e-03 7.295469759185694517e-04 1.465529064396312212e-18 1.394886238453346868e-02 2.365259324166004914e-02 2.068818113735499190e-02 2.529395703652483410e-02 1.836479306516255322e-02 1.497169943508004121e-02 1.977172753967705587e-02 2.911648483837826068e-02 4.247590774595456226e-02 7.796991732502576468e-03 1.045900011617346623e-02 1.941763091434927854e-02 4.111142737670983593e-02 3.072221249388773934e-03 4.831676460977739514e-03 1.685587336135858927e-02 4.289242963186640223e-02 3.891483774837497384e-02 -2.154567728366277302e-01 -1.535369696240428683e-01 -2.186576075072948488e-01 5.429387299655010629e-02 -1.059490730546082393e-01 3.440575930074487448e-03 -2.814245721987706020e-02 1.460862305199812222e-02 1.241450010578046097e-03 -2.178758613694973569e-03 1.579046903459966483e-02 4.336998554310415172e-03 9.925511472109456446e-02 2.261542721691509050e-02 -4.906742025456785730e-16 1.584117888076149241e-02 6.660363294234730204e-03 9.056604708074211243e-04 -1.739325351651257318e-02 -1.561698491530383392e-17 -3.437632332563644085e-03 1.145567290029181367e-05 -2.209627213694408121e-03 4.928003786686412004e-04 -2.803395329955769822e-04 8.967575395285570794e-02 -4.298025224521917302e-17 -1.414951612972067106e-04 1.478863696978090919e-02 -4.528820320439780312e-04 1.597495690880591096e-02 5.501758570698594975e-03 1.962568668447424620e-03 -3.204580603817520912e-02 -1.476359116543441048e-02 8.254936486311672400e-03 -2.626323024565730899e-02 -5.508807127700775935e-02 1.033941595195365243e-02 -2.462657467043806339e-04 1.390486460159136729e-03 1.292175533643602492e-02 1.058243373521597060e-03 -5.735684091138333000e-04 1.748691345423760909e-02 -1.729473234787669636e-02 7.938408410920693402e-04 9.698068946248179431e-03 1.391647919637443824e-02 3.830032415451684047e-03 -1.182888045886149259e-03 6.152338164545926615e-03 -1.656028421449254112e-03 1.821778496744958994e-02 3.256808641320025483e-02 2.070224957202201552e-02 2.172591215907343754e-17 1.315189279631282133e-02 2.078701233952065519e-02 -7.502808998063292302e-03 1.202682713050752875e-02 1.312524592563135001e-02 1.502333006999288789e-02 3.063039023101867264e-01 6.516207144468341150e-01 3.996792515064214180e-01 3.461883779785721732e-02 3.118887797060079792e-02 2.210521621249153523e-02 2.635794544702535491e-02 6.875968895224300603e-02 1.149347403098376075e-02 2.323930674915634237e-02 2.480490933712918220e-02 4.614931917449195675e-02 9.939477737451723333e-03 2.552113449876595996e-02 1.015930070756029639e-02 2.974923225434912208e-02 1.736902089337870672e-02 5.899180508536765871e-02 7.902926714843477168e-03 1.916162422643620522e-02 4.007472373806229837e-02 5.515050321875095141e-02 5.288739570105150355e-03 4.091183477525234258e-02 2.120812058572712697e-02 1.743740918996185968e-02 9.064307907306187392e-03 6.834352724000442025e-01 1.343644486652502579e+00 5.136033285460663844e-01 5.218120985579438598e-01 3.555198332509851844e-01 3.814793226057388020e-01 2.387132859327170120e-01 9.995757736892659784e-01 2.511852292637192785e-01 2.965188146640040934e-03 7.281812437938587658e-03 1.577066812126233508e-02 1.735812915055112357e-02 2.090486559980831924e-02 4.225849861529664547e-02 3.021231433743298833e-02 3.776714681897266196e-02 5.361908058996652481e-03 3.363846241088723488e-02 3.777255911798863691e-02 4.531166726046104154e-02 1.680863211540761792e-02 1.897033583805504339e-02 2.216510440787994654e-02 3.909168950375994933e-02 -2.395436131837022121e-03 1.969281478477164990e-02 2.508019213011267873e-02 5.456882134886902741e-02 3.808264850897526491e-03 1.275352659248784604e-01 5.362050442219671309e-01 4.183665114812090691e-01 8.029512509924515706e-01
1.425871375317394607e-01 -6.122143218606763970e-02 4.494068510099827329e-02 4.932783623081527652e-02 2.429083322717743090e-02 2.082587584917032150e-02 5.678573850573130632e-02 2.649232675970454934e-02 3.759389766426107088e-02 3.190424969129664751e-02 5.751646696994396762e-02 2.795449087071313551e-02 1.655263630658032961e-02 3.250612763331433425e-02 2.752230060325350225e-02 2.320495246650405505e-02 6.279169642389295647e-02 2.046310335523616325e-02 1.549854469529119913e-02 4.258176596945776465e-02 7.777187509352646855e-02 4.295035406827089897e-02 5.941993633087580551e-02 1.836338833047677357e-03 1.759211140841823381e-02 6.157786749001141480e-02 5.477228642588154346e-03 5.604313519658583832e-04 2.253447105067236944e-02 1.348194881563088976e-02 8.072967390609983931e-03 8.568839642432050235e-03 1.282696330678468168e-03 4.186299689105302431e-03 1.190895451654738584e-02 4.711052297402582890e-03 7.164818038896027712e-16 8.209038260354076938e-03 9.528588881899814622e-03 1.089903758230123879e-02 1.132389651644767452e-02 -7.817162422935074878e-03 -1.668996701143263608e-02 -9.748583378011611497e-03 4.737200740733790716e-02 -1.381396414948484787e-02 -5.902613436923813232e-03 6.682717806669676713e-03 2.587835954528324225e-02 5.200681766870585837e-02 7.226525645919310015e-02 1.019262121753745573e-02 9.835189284970782472e-03 1.893931703800190089e-02 4.422661592419447479e-02 5.858116488711741926e-02 6.874525513256171672e-02 6.367435366820757559e-02 4.397658889648481395e-02 2.124098201853360374e-02 -1.221128272187118613e-01 5.818109418767821762e-03 -3.410105019321096192e-02 4.575999487439147417e-02 -9.505871119448994935e-17 7.735226518090389805e-18 -1.041246712242808146e-03 1.237067476475122406e-02 5.033768082186507870e-02 5.248279591308799980e-02 2.253909924857098145e-03 2.799340186147745643e-02 9.216699090912003692e-03 1.274314878822239462e-02 1.255693919882961904e-03 -6.923539355877677843e-04 -3.068814096281895546e-04 -2.146764283110321435e-03 -1.089544673887223904e-02 1.026861438830139681e-02 -2.096455237252639516e-04 3.288337484075946893e-02 -8.295523388213652267e-17 4.515619380578544256e-02 -1.512125554375902572e-17 3.218427042610493005e-02 4.266783233952227417e-02 1.137417584368127267e-02 3.252717934371447683e-02 2.599047237296067334e-02 2.009393906326758725e-02 3.334409016899201467e-02 1.348555994762135751e-02 3.258693900065427057e-02 -1.983935855705502968e-06 5.213270084097534375e-03 -9.318805313115768587e-07 -2.420684040709403240e-02 2.019009476806436429e-02 -4.582078695729623280e-04 5.376949616151995969e-04 5.361963867810456280e-03 1.456375668461226125e-01 2.394815732837051684e-03 -3.546538256474927597e-01 4.521422353818802149e-04 7.663001105855118955e-02 -1.252479363855129799e-02 -1.761357615770338739e-02 -2.577117420536675918e-02 2.069835263687003560e-03 7.800064733696647490e-03 7.492461644643788251e-16 -1.374660087773928001e-02 1.563008149198778091e-02 -1.234854114373721765e-02 5.817981869059582478e-02 -2.591934364342643460e-02 -1.111631641032164927e-03 1.403691775510297612e-02 6.463184973918602028e-01 3.801089752215268680e-01 4.255131533740148797e-02 3.348734479323588198e-02 2.037566800404485082e-02 4.712186739936241597e-02 7.307914166454135274e-02 8.742336843270139338e-03 2.554751510836076114e-02 1.601797107453014485e-02 3.669698793375894552e-02 1.749931096724836183e-02 2.374765882653053634e-02 1.498523008240136567e-02 2.432278970278729108e-02 2.109860705724379490e-02 3.559791917876231615e-02 1.052012919154385849e-02 1.838596319771550192e-02 2.615452445217452543e-02 3.937087635899650523e-02 4.374232947988486789e-03 2.078772339518595441e-02 6.299755830684231617e-01 7.540318446389100426e-01 5.893907695096386457e-01 2.707842837486493584e-02 2.476243294016029339e-03 1.776926685127493133e-02 9.403231130478301517e-03 1.618633317159784077e-02 7.410588031646665394e-03 2.190305496808166164e-02 5.219680484991598077e-03 1.795111297541998260e-02 5.340206853103494434e-04 4.874187611303527207e-03 1.068008682860821339e-02 2.596197603412794472e-02 1.603909379828842574e-02 2.199871901229954671e-02 3.211128293183001386e-02 3.124596735438555858e-02 1.458018293517628940e-02 6.418110159905689172e-02 1.625604161127932115e-02 2.722307322037240823e-02 9.384943018277161486e-02 2.191811789091533541e-02 1.660508319129694321e-02 2.718968440269351958e-02 8.995078284217586742e-06 1.166057707227450939e-02 2.511010271849792655e-02 6.368741481565963480e-02 7.456647639003430306e-02 1.029338753693104375e-01 1.026212765454796866e-01 8.019536034466974439e-02 5.201135511226550462e-02
2.635739926610132233e-01 1.552531926781031482e-02 8.537543806346572714e-03 4.017497316201876396e-03 2.918202971457397980e-03 1.239681830424636897e-03 2.213195979231422778e-03 3.406486071173696079e-03 6.604273468950281299e-03 6.718946014974759226e-03 7.790140447974274948e-03 5.817902594055348979e-03 1.693993552274954476e-02 9.778284311619839242e-03 4.849972125424239361e-02 9.593117151612302537e-03 2.487469606942878733e-02 6.352628159257416203e-03 1.710644906966523812e-02 1.487478045063273142e-02 1.059021048963200429e-02 2.643704413240659766e-03 -6.833469817704727889e-02 -1.689241004301500759e-01 -4.564223023223291931e-02 2.450516425923274580e-02 3.687090935323273505e-02 2.638606267160476665e-02 8.396233707354242015e-02 9.175469838071981243e-02 1.379046723931546536e-01 1.090393011380215160e-01 1.403352298921917152e-01 1.036092228911059815e-01 5.872829349770725127e-02 2.442059339426135098e-03 9.866092068950089913e-04 7.735363492791338946e-02 -1.865588959917375417e-02 1.718811571414265832e-04 -2.280400517600666976e-02 6.715920016975561457e-02 1.152986720908807800e-01 8.241911530034236322e-02 6.836275649824019585e-02 7.580300140809397014e-02 1.365276952096112772e-01 4.273984216334812680e-02 -2.353637135674505054e-05 -6.732180671767112399e-06 -1.828150456139415693e-04 4.049060560712793716e-02 4.520165514514092042e-02 5.478017050056395443e-02 8.351357756615407357e-02 5.381581463565691253e-02 1.054997408126532726e-01 1.233001356473282795e-01 1.444705305106915028e-01 3.417695333134219970e-02 -2.263988687505113229e-01 -2.482681945901913257e-03 2.526904031922315007e-03 5.105788638879282804e-02 -1.667692094637252228e-17 7.277383968842748331e-03 -3.987661537534028557e-03 4.123256250168370884e-02 2.291351372025780147e-04 1.269296605763997957e-02 1.929430303772066433e-16 -1.041442622819991389e-03 4.542980576530623871e-02 1.571927586191168222e-02 -4.126335301306893396e-18 6.287645472297796918e-03 6.109165798526740125e-03 1.258616876824822546e-03 7.031916734232479538e-03 -1.426585034330526698e-18 -1.125600837404125941e-02 3.878204009946420450e-02 -4.562314723669016331e-02 -7.948849113685110823e-02 -1.041116983185278733e-01 -1.249357445176375522e-02 2.180564116413123699e-02 1.392688467303843797e-02 2.197372527804790018e-02 -9.149625409534860657e-03 3.798004572679915952e-02 2.499666844859398801e-03 -3.659841846244612520e-04 -2.653360183955424312e-18 -5.500939381666695459e-18 5.452010772733087776e-04 5.739878520626545417e-05 4.340780229226482768e-04 4.709139206748245293e-02 7.000499890646962042e-02 -5.006330978841168075e-17 3.320555305016649949e-04 -9.052503987584717936e-04 -8.005618357342771108e-02 -1.535315891218717735e-03 -5.672814677384094586e-18 -7.614886933558308313e-02 -1.450724767684957077e-03 2.405877887119247521e-02 1.997922511038046842e-02 4.534459740293349944e-02 -5.764472895224839223e-03 3.527401822794477837e-03 -1.979653028965556658e-02 3.395837356328546171e-03 -4.064792178040929627e-02 -7.091856315644423474e-03 -3.402541533233442717e-02 -1.121688093047013479e-03 3.502725203089526035e-03 5.754434001208156291e-01 3.720572642474493930e-01 3.568057143314247448e-02 1.150130955217543377e-03 1.541527035537585202e-02 1.077641217313774977e-02 1.978416412886764217e-02 4.963393942653848180e-03 1.408544204229197211e-02 3.376873798973668546e-04 1.490167926625865502e-02 5.371755071591985137e-04 3.948836230437532263e-02 1.242600237503962111e-02 4.047417169456852398e-02 1.259179383385406609e-02 2.551390715524046518e-02 1.300301975590198618e-02 2.700978246352372242e-02 1.374470878699508326e-02 3.888535940161952364e-02 4.737640400959919340e-03 4.625927657627318390e-01 7.427116547298844296e-01 5.372486391851629683e-01 4.141331171869279924e-03 3.673412796846819983e-02 2.093555428977461608e-02 2.614683908558430511e-02 4.213447221971820117e-02 6.955430618403840581e-02 8.218596804703469372e-02 5.968774304964987593e-02 6.035466530408158808e-02 5.546816381472627344e-02 1.114108865101493505e-02 1.923274249450803233e-02 6.689961648779326753e-01 8.190845277784010170e-01 7.036642544224362972e-01 2.285175218881307577e-03 -7.658353073583872017e-02 1.045273556758292803e-02 -3.038348459927852838e-02 1.349697817587580152e-02 3.652990133675025532e-03 1.131787331581630800e-02 -4.969971135667190787e-02 2.512072136167344816e-01 5.658106248060771337e-01 1.755454114079582351e-01 -1.974971497816100513e-02 8.799618481123150371e-03 4.663818018082412087e-02 5.580778959362056590e-02 2.976155402233727695e-02 1.191447996540246773e-01 1.193922856914746145e-01 7.904972478633959243e-02 5.713372784285323386e-02
1.612452862203583859e-01 -5.910476304306464673e-02 6.080473662253836620e-02 8.491047651099176141e-03 2.306206736008691374e-02 3.396923826169556682e-02 2.500067971558063082e-02 6.203585461477332218e-03 2.692812598158894968e-02 1.746021293069427091e-02 4.823012968202743805e-02 7.905489720601351512e-04 6.735711350192699620e-03 1.684959968046056966e-02 7.944958805114315722e-03 1.983039262779273670e-02 5.546137440631752902e-03 5.158885542601764074e-03 -1.926024508994635814e-02 -2.369466512427341742e-01 -1.125508551561199971e-01 1.118317433234113957e-02 2.607418834817596884e-02 1.623159847884252799e-02 5.203820636033903123e-03 1.569531880248463754e-02 3.209528346870115650e-02 4.471337805268487883e-03 4.607166267385365797e-03 1.007778154808192973e-02 1.163908301159499378e-02 1.297224081718056921e-02 9.193809933723621694e-03 1.655424102546895412e-02 -1.689013763905725662e-01 -1.523238841046699599e-01 -4.761038034208570774e-02 5.955237212265766800e-02 4.276921616703081158e-02 7.712925208582042946e-02 6.470321312109980627e-02 6.301245068384390968e-02 2.505827835588742350e-02 3.915120240760538300e-02 5.080717466346004740e-02 6.173336563353028217e-02 9.752853404980893481e-02 8.569328891979292873e-02 4.648023658061146285e-02 1.757449920715854416e-02 3.248484565145454139e-02 1.075959487273804859e-04 4.142150240130771516e-02 1.168959685005707856e-02 5.169010798710704097e-02 3.857420061739805228e-02 7.375770934035204285e-02 1.070551303117043829e-01 9.286033157536909000e-02 1.019442278293990721e-02 -1.234261943148897145e-01 5.532305593080704721e-03 3.553790588469673412e-02 1.622756837956792347e-02 3.075018589877724987e-02 4.902347086861045828e-18 2.625402014229900015e-03 -1.235967600141451607e-02 6.138578501069156222e-03 -3.871374791081625364e-18 -4.581602071556126199e-03 2.479551589264657532e-02 1.046069388028924955e-02 1.606384689026674648e-15 3.919248384061089531e-03 -1.864801018462265106e-04 -1.842166349913179972e-17 -3.680386818546198952e-02 -1.237189876709648417e-01 -6.597036520862753495e-02 -6.068983576649257528e-02 6.348071727682639337e-18 -1.194319772094150529e-02 7.218132836232265225e-03 -5.268045324361142202e-05 7.933972839342558203e-02 1.705203347499637326e-02 3.695148812355756544e-03 -8.889615264171772907e-15 -1.089256262453313615e-17 -1.834495863330501983e-05 1.297653509279564528e-02 2.941700832730515663e-02 -1.757380570561496019e-02 7.064513927633706523e-03 -2.799844602669258192e-02 8.731812036672981225e-02 -2.619278930456587270e-03 3.281609462978870087e-03 2.795434990023896310e-02 3.801854017337385344e-03 2.337152574802334962e-03 5.694573687548347765e-03 2.020896314870056915e-02 4.016337084758772949e-03 1.417841393498633744e-02 -4.083822990922486291e-04 4.780619937614287440e-02 -9.659527240289990380e-04 1.392127094588042208e-02 1.093435657735968569e-01 -8.841026655998134032e-02 1.588526639004758005e-03 -5.104668869137848203e-02 2.052122998161243911e-02 -1.746229838529314918e-02 -1.186871466585453366e-03 2.787946356859350230e-02 -1.965442302292326124e-03 8.867203053464086174e-03 6.383280501583182831e-01 4.023196649823255200e-01 2.648657449638239930e-02 2.774633798173497395e-02 2.303431558604010845e-02 3.303564738177228177e-02 7.379764134492299010e-02 1.099908258808925295e-02 1.902270247041121243e-02 8.444617841803841660e-03 2.974119502745897758e-02 1.987964661558698498e-02 1.831303599362787971e-02 2.219255354626134893e-02 3.124527843997786103e-02 3.574659821731867443e-02 2.917129807807803885e-02 1.149240756484478689e-02 4.076945545341435495e-01 6.748479674011732543e-01 5.270381592863074838e-01 9.585761010796288711e-04 4.098478867596914293e-02 1.754200830734455041e-02 3.368898912075041913e-02 2.747329713719128333e-02 3.423906773115693164e-02 2.652264445163286647e-04 1.104810490902814919e-02 4.828841048030531324e-03 1.304947836941509724e-02 1.592959947595208375e-03 1.988079493467009157e-02 1.744708820714118173e-04 3.422798865164766169e-01 3.679019903774772837e-01 4.565355889046036619e-01 9.518178715190943329e-03 2.208586096809046773e-02 2.223997721562782914e-02 3.427664190179061793e-02 3.927577956186237285e-02 5.284427185980988817e-02 1.143301847113609507e-02 4.222691674401000961e-02 4.813047966911379594e-02 4.422530803798190718e-02 3.749798898321485274e-01 3.426083129810624239e-01 2.676388084826696323e-01 3.391740145601750134e-02 3.260649093189782292e-03 1.962224421241729258e-02 6.300518876921791522e-02 8.322265837768155761e-02 7.280880606160312907e-02 9.199815645010199494e-02 1.023316566508748193e-01 1.284630474986887816e-01 9.302666624391897621e-02
1.813767834682632890e-02 3.384388689774665837e-02 9.498633492717911686e-02 5.044825612906541495e-02 5.971053063425950380e-02 7.093944379501362918e-02 6.903323511737577411e-02 3.004987154148575437e-02 5.636957298238836483e-02 8.140069235255753544e-02 1.359776787122197184e-01 4.995724263857034830e-02 3.189269250642058628e-02 1.767983076922689724e-04 3.725693050500512860e-02 1.123537149207651763e-02 1.474544438428187186e-02 1.570628451472375021e-03 2.726692316761783648e-04 2.471793091719786534e-02 7.749145651899762288e-02 3.011547784182460122e-02 2.846493904939444153e-02 1.996724616055654128e-02 2.078400180299791802e-02 1.584093830817127554e-02 1.316697776342259235e-02 1.130554848044922588e-02 1.816856170303589692e-04 2.091629823945827249e-02 5.568240597980516141e-03 1.451530139683217821e-03 5.929065125363155477e-03 7.337983850548920536e-03 1.393147329667556063e-02 3.974441827867043579e-03 1.130230211681052324e-03 4.326916856482104113e-17 1.266724760954613185e-02 2.264045425106366461e-03 2.062695963950985853e-02 2.608473718136639433e-02 2.125325564742838041e-02 2.002778327853331833e-02 1.832550992736131298e-02 2.924768156992551063e-02 4.785952973814886630e-02 1.712377103331278946e-02 7.890064914680782956e-03 3.132322702388995461e-02 3.076104621271144590e-02 1.049555749258713686e-02 1.476665158574300314e-02 1.466162147871203671e-02 4.731668713626249051e-02 4.684194144416856109e-03 -2.013438021396250777e-01 -1.452016882607525539e-01 -1.902061873691615890e-01 7.131468283240420536e-02 -1.643566113065487999e-02 1.464334623526930036e-02 1.236530239649156863e-03 -2.680300939546400099e-02 1.381039026927599091e-02 1.759534261810115027e-05 -1.357832476560077051e-04 8.048549954052859612e-03 3.572079112887179778e-03 -3.668150944361923092e-04 -1.402074511963974542e-02 1.881702940197486559e-02 1.254416447347394235e-02 2.307510044141065572e-02 7.488142312369741593e-02 9.608403510521054680e-02 -1.107995806665311098e-02 2.461026991533094357e-02 -3.270311197159885127e-04 7.072580538987586520e-03 -1.478818679721149131e-02 5.592991692681414245e-03 9.129036628559497948e-03 -3.466301792569435186e-02 -6.141630442571771251e-03 -8.365157587826100097e-03 4.403966787920555320e-02 -1.134214236289444663e-02 1.014027578194756640e-02 2.417012172734459008e-02 2.439964634352751599e-02 1.766595785724450751e-02 2.804913023966424182e-02 -1.815578101328788746e-02 2.496086881891381240e-02 6.313046394943289119e-03 1.398456983894920749e-02 1.257949150815030622e-17 1.629449473593363659e-02 -2.400583726553662690e-03 7.114848605548244366e-03 5.257777777661050943e-03 -1.085987003839024648e-04 5.559070324002296504e-02 3.883919384093789648e-16 -3.327528287086777697e-03 -5.139861408078244716e-03 1.431554972996055587e-02 -1.407784460051762776e-03 1.763894612099473763e-02 -2.605934839245855825e-03 1.385370634575051278e-03 5.572068923920711972e-03 -4.066227797424078065e-18 7.242111390570247879e-03 -1.029863052433588333e-02 1.267778488186950547e-02 7.795525444604265060e-03 -1.485830186109400042e-01 2.976000373192206339e-01 4.219376332087946407e-02 6.829906354520143275e-02 7.297428545892054030e-02 5.316244021386458407e-02 2.330070501113417486e-02 6.234255751838100462e-02 6.512157818117858543e-02 1.133356423110909103e-02 1.778492221546258342e-02 6.486215127542423707e-03 1.923751390863258359e-02 2.984001863179186129e-03 1.931168190534169465e-02 6.113332756191643957e-01 5.626549845435319108e-01 5.672254567214368715e-01 4.085879011877176920e-02 6.577026342124977359e-03 1.463086413713772863e-02 2.648008856129887081e-02 4.776592217942226409e-02 1.713249734481883321e-02 5.529821196385616144e-02 3.010780197094052937e-02 2.203895111670914342e-02 2.803635939713921763e-02 1.193008024485849895e-02 6.027500872915694460e-03 1.588186927454260933e-01 -2.097385262835771980e-01 -1.039900682995988784e-01 1.085254928980696892e-01 2.915545687521561846e-02 7.118227792533747371e-03 1.478189959302390194e-02 9.901770759093507079e-04 6.220326553454330089e-03 1.528618803185091864e-02 1.648797627091131562e-02 2.701075609866335561e-02 5.585992902306816804e-02 3.765892698202055744e-02 4.618824740723019573e-02 1.962254427691564884e-02 4.141636870911536566e-02 2.326750751718746277e-02 5.839071787581550516e-02 2.131815008892248842e-02 2.616915206978696895e-02 2.193240195821586844e-02 3.520952865928377029e-02 9.027729913871827515e-19 1.724117533034983524e-02 4.116998762080487095e-02 5.749700433867798582e-02 3.673301149761754557e-03 1.401364813900069506e-01 5.254040013968080958e-01 3.781511891652908353e-01 7.828284789984459868e-01
1.300052248479677264e-01 -6.374352778186549717e-02 6.475468433986376893e-02 3.591645589052130938e-02 1.971630532360873350e-02 2.938077030322305480e-02 4.997299661915812624e-02 8.895472648666720786e-03 3.109724374627797702e-02 2.032276381341864557e-02 5.855609353185316224e-02 1.702736532126479171e-02 1.522723807227966547e-02 2.127416471784132276e-02 1.067475439040086425e-02 1.830100404374401107e-02 1.506699478372813990e-02 3.201771525205283301e-03 2.125207583464123421e-03 2.221304986001135934e-02 7.610263724407177743e-02 6.947342812444102025e-03 2.796746158480470368e-01 7.133696306581993607e-02 4.100278264817201757e-02 2.472637174681900640e-02 2.102486378149914445e-02 6.910319296136942062e-03 3.431428745306329495e-03 5.907943152683412273e-04 3.841946230854412572e-03 4.318520458776325260e-03 2.953581393730639500e-03 8.342266382330144198e-03 1.489231946741980880e-02 6.006591551479899434e-03 1.937672753377730735e-03 5.833866671597094829e-03 1.785718485574538245e-02 1.959093719510211773e-02 1.191340836351789841e-02 -3.097892227900476622e-02 -1.751300605518512840e-02 -9.980540950119375088e-03 5.748381786589034509e-02 -2.674116024996538513e-03 -1.990257630297407113e-02 8.891369012790261567e-03 3.214112479828930974e-02 3.802129072514993935e-02 4.756320121285918801e-02 1.096664829726696176e-02 1.416215362279222255e-02 1.791607283857092267e-02 5.448289390675380844e-02 5.576659736101959725e-02 6.069305731036371554e-02 3.978663987952172493e-02 8.614749325206939867e-02 2.351843748986314972e-02 -1.285392367191603713e-01 4.555802057366739252e-03 -3.295659507751780293e-02 2.619561643943902826e-02 1.596452336481116131e-03 6.782434027978270254e-03 3.821947314749208485e-03 -3.709944960736841907e-02 4.933418088427459630e-02 4.116224578753875651e-02 -1.416215153277501117e-02 2.386097320132650468e-02 8.519029843846283659e-03 -1.514647057233614301e-17 1.744405069618557014e-02 -1.442351122316472991e-05 -1.374838774205678054e-03 6.436495191127025207e-03 -2.898198768506329501e-02 5.078174788211508618e-03 -1.154561359890712499e-02 3.328420110783237434e-02 -4.919406034887173690e-05 1.208812362100737947e-01 3.221532251618336451e-02 1.426180294215018303e-02 -1.081255493789430691e-17 9.046372611687239496e-03 -6.587633494740134567e-04 1.680642263330040312e-02 8.836190239464417756e-03 4.583493473614209252e-02 4.189717526765506135e-03 1.547058695228986820e-02 -7.432156130469899950e-04 3.125008538346886033e-03 1.238741208145231060e-02 6.618893861606120189e-18 1.492626513471863835e-02 -3.098870482322772096e-02 2.565938453248103480e-05 2.488056911898842150e-02 1.632362864180779138e-01 -2.553504177182551460e-02 1.312734347365100002e-02 1.593048127505351666e-03 6.237829896381043987e-02 -2.024906446029848747e-02 -1.501702768985825782e-02 -7.481619821826928675e-03 -3.329008380711137149e-03 1.538135921962317104e-02 5.474611935262086414e-03 -1.987735103729970816e-03 7.186574423654288994e-03 -2.364467023098185822e-02 -8.317119544687598398e-03 -2.631066166204686085e-02 -5.843754448880700021e-04 1.256085358118159967e-02 6.636943023748995696e-01 3.536981983792313322e-01 3.963768661358282180e-02 3.174236166851517743e-02 2.379701849998287072e-02 6.448638066367186339e-02 6.222455477045114869e-02 1.890278181252517259e-02 2.130432074319896923e-02 1.616110767389880229e-02 4.040684730835879951e-02 7.931078072072429014e-03 3.224334849387300306e-02 1.762879060503195691e-02 2.277404337913018981e-02 1.428859208020788821e-02 3.500042947319294967e-02 3.820184308597069679e-03 1.990692268646633914e-02 2.288484963127245669e-02 2.157566388886402939e-02 7.755960681336218241e-04 5.907391701535777973e-01 9.166253538864316264e-01 6.978791759918161475e-01 8.033876771809479516e-04 1.377872704118104398e-02 7.238764865261201192e-03 1.396336602286663529e-02 1.284922389821796128e-03 1.439709480086496032e-02 9.007147334928007273e-03 2.726522974888211104e-02 5.588270679869109143e-03 2.206128247510500545e-02 1.032139570471510975e-02 7.976399319195071752e-03 6.015866145118681750e-03 3.045820523217388290e-02 1.657123631946560635e-02 2.537859814746338052e-02 2.165110536863314519e-02 3.052969735660510456e-02 5.291424497284921008e-03 5.200398347838024876e-02 7.337780108188675376e-03 2.859772930281922337e-02 1.037291700634362934e-01 1.626093115399955893e-02 7.046883259667953252e-03 4.313507614930823381e-02 2.575231311469411109e-16 2.162534132130507214e-02 3.307403157208639588e-02 5.313790559712835981e-02 2.543473738418195601e-02 1.392002567911402866e-01 1.502502936161196900e-01 7.787869727966956201e-02 5.041089967924852061e-02
1.523654708403785474e-01 -5.821802979220915691e-02 5.516579443226421464e-02 2.540822545911093633e-02 2.739610463139848656e-02 2.921545845123335941e-02 4.409844271898376300e-02 1.820530016019957984e-04 2.203716138526575288e-02 2.192451306090274082e-02 4.868717146076421975e-02 1.724053652311875739e-02 8.843119139476548346e-03 2.156961086600616556e-02 8.840478941241706676e-03 1.930219689412997555e-02 1.620859411302326414e-02 2.232607817126798250e-03 2.576604614103456155e-03 2.340952012602010371e-02 2.314533234143733575e-01 1.885208828432860181e-02 2.269977591185672050e-02 1.761425543786167974e-02 2.148735315216763683e-02 2.090728252134171558e-02 -3.284990309807361147e-02 -1.123159085873290663e-01 2.005893190934080958e-05 9.652176707461079461e-04 4.986310925041805182e-03 5.750726077689893856e-03 4.224484547740086309e-05 3.234621923359584004e-18 7.449664494348153806e-16 1.331604099909714780e-17 8.613352868496538042e-03 2.261413391330367034e-02 1.705808026893967239e-02 1.339691425082230672e-02 2.380301918425659774e-02 1.977687816161628059e-02 1.195345154344899102e-02 5.798165341602479032e-03 2.300430413815092792e-02 4.572364031695478476e-02 4.400260914570576498e-02 1.885418971292060833e-02 1.052583949350756505e-02 2.394368827683621137e-02 3.679920932716058274e-02 3.879245248812678709e-02 1.571446041905429775e-02 2.462420043950038634e-02 5.421914133512670264e-02 3.404099230626157074e-02 6.465560730240013654e-02 3.576272077525793902e-02 1.247894516583891167e-01 1.416828852709735853e-01 -1.040283754883567086e-01 1.950709769417397974e-02 -1.102123171357157969e-02 1.742769796756963099e-02 1.318880556527088181e-02 6.187133441976282138e-15 3.385950367433745306e-02 3.073908855275240583e-03 7.596193286772010000e-02 -7.240200579523120268e-04 3.568046559093360215e-05 -8.624426822187542261e-07 2.746236490405861742e-03 3.332808627335959173e-02 -9.874481233594649757e-03 -1.036340220941869985e-04 -6.021281934733995790e-03 5.105059436180861944e-04 -1.761843970265620097e-04 7.976266254346778533e-04 -1.886888044822966496e-03 3.380262409330322265e-02 3.942255616920938704e-02 -8.230229963943644020e-03 -4.191141510946841051e-03 -4.722209008612368412e-03 6.418140264605582367e-03 1.113087749626281119e-02 8.982941077850208414e-03 3.737888881648073619e-02 -5.212345434123498433e-02 -9.657833815944656675e-04 9.170913478775025857e-03 3.422005967251201867e-04 4.422845665218180788e-02 5.102131452893193114e-02 -6.762554195785369646e-16 7.575433035958970517e-03 -1.457514187849514621e-02 -3.350915117168804246e-05 3.538788200253084257e-03 -6.249108442906834305e-18 -9.984510094713763134e-03 -5.485260940527364528e-17 2.818383002763041115e-17 -1.796829456886772614e-05 4.522375315407682096e-03 -4.542525686528840405e-17 -9.794172747012090674e-02 3.863179418026362749e-02 -1.216719911344321708e-04 1.218759319723754057e-02 8.658937508595492992e-04 2.207465858882364657e-02 9.988359163677374211e-03 -2.001780695721839740e-02 -5.493453741668342548e-04 6.980660475156028055e-02 9.622117376530989197e-02 -5.618167348703333001e-03 6.354783549467013515e-01 3.654134359223376038e-01 3.371151027134654948e-02 1.408942999192827270e-02 2.732531457380836873e-02 6.447353890646728702e-02 6.280519812573215344e-02 9.963066143136207337e-03 1.434829423823661375e-02 1.856109455302149686e-02 3.593090279370251411e-02 2.200628330825201914e-02 2.620231425135793185e-02 1.947600226549640184e-02 3.360705116393025021e-02 1.929988235462573934e-02 3.721433175168215729e-02 1.194292185382350713e-02 8.276898430051500902e-03 3.346117916159649895e-03 3.939671632058262124e-02 1.626421881428976451e-02 5.203459925361308275e-02 1.560174138345299562e-02 2.372999137648915471e-02 9.588827635034194788e-03 7.270956503581577079e-01 1.267308522177076124e+00 4.187313007698754697e-01 4.811537964083734457e-04 1.382475124010680914e-02 7.532851771639944780e-03 3.809567200758403466e-02 8.139573251349326855e-01 4.495061271881730658e-01 6.594745813216247310e-01 1.791868626401794257e-02 2.316366681326009758e-03 1.829573514590139907e-02 2.285296725004321017e-02 3.815626044234534525e-02 2.698469543689216432e-02 3.228394835236301358e-02 2.238872462079969164e-03 2.922583255001463473e-02 3.382952077638458344e-02 4.897748666630322850e-02 2.251806444244506059e-02 2.837230176553913810e-02 3.062465814466988970e-02 4.589505117050763500e-02 1.765881680060143316e-02 2.255290361062443558e-02 3.239620524336234847e-02 5.784531303958517778e-02 1.509157570512106514e-02 1.270582976011562226e-01 3.959133258947569733e-01 5.699615820100787378e-01 8.531584762378366937e-01
1.525613082174623880e-01 -5.154880898801422751e-02 8.285448538225383130e-02 3.160376158499226401e-02 3.378146100776307365e-02 2.319392907163370793e-02 4.201492309089020077e-02 1.611545027601095706e-02 3.896768706521581277e-02 2.782200383364767313e-02 5.496264801293990893e-02 2.196038048713468832e-02 1.040233562233004207e-02 3.977802679068438013e-02 1.897073424021939597e-02 3.143688977787348349e-02 -2.492612070931408130e-01 -2.069867391544200352e-01 -1.989868036356004233e-01 1.838657973393953426e-02 3.388573492272067139e-02 5.193702753797198294e-18 1.793377574436120711e-02 2.634020923384159871e-02 1.321764546348973453e-02 6.008463366586718618e-02 6.853062891039564397e-02 1.875900047240218471e-02 2.302425463265100891e-02 2.685140636661155142e-02 -2.831320086020210069e-03 4.694793252310604931e-02 -2.088899793521664574e-03 1.099959368585506740e-02 5.865297141427561721e-03 7.851482160422917911e-03 -3.677829528720109181e-02 6.944265890516607967e-04 1.111266229106237270e-02 1.502427739689938613e-03 1.136544461319726092e-02 2.066171434922314809e-02 1.626901659593302241e-02 1.109334553755502405e-03 1.826036545467859054e-02 2.293530807275757574e-02 6.711766745453057181e-02 1.902099413024014099e-02 1.670263969619516578e-02 2.509941499966696771e-02 3.751286806117500039e-02 1.357266465939581607e-03 8.712252193512390330e-03 1.011074715695811817e-02 4.429539328647895402e-02 1.541566712341672224e-02 -2.955417728560530422e-01 -1.471588452677856651e-01 -2.092707920430286472e-01 5.805119692949634036e-02 -1.166167864193741743e-01 1.551986630303910618e-02 1.999223732017409161e-02 1.209409045375263447e-02 -2.387646874611476391e-04 1.263785824847824500e-03 -9.876334551487701009e-16 1.784474750472846150e-02 4.069432936111756771e-02 -3.259787147967636942e-03 1.074883751507824114e-02 2.056981167797970050e-02 1.416459811203832179e-02 -2.949981318460098700e-02 -2.869988973749670880e-02 -1.551574697768307335e-03 2.841937369117560758e-02 1.330983550594898172e-04 5.222801279004603281e-03 -2.095969289641070807e-02 1.604951356329430145e-02 4.772267501190711675e-18 1.676797835735726119e-02 -2.154212694994969007e-02 1.864340155147540742e-02 8.838832694261229261e-04 9.210643372005378193e-02 -3.840242765314571399e-02 5.844337828920279088e-02 -1.312402869371572661e-02 4.793414859933782524e-02 5.914308294008648048e-02 7.990903175814506598e-02 3.208013500241548716e-03 2.857262950735191959e-02 -2.199446629029079212e-02 1.391273160979760934e-02 -2.470605941678666317e-02 2.780911686691448989e-02 -1.169903238701131259e-03 1.408527794805889376e-02 -4.448830375454595337e-02 1.516082859484706169e-02 2.084963576756489537e-02 6.684383145348894121e-05 1.195919821465475002e-03 1.243385257440895834e-02 -5.198183278439494001e-03 -8.656393431824531068e-03 1.545042578072049720e-02 1.194090675510111818e-02 1.874433608105988136e-02 1.029107504811648845e-02 4.250089161777425954e-03 1.145156794307404698e-02 -2.316447193081113201e-02 7.651152627959532070e-03 1.031568740947299244e-02 2.724948392428692620e-02 1.410435823596022098e-01 6.419374643123580970e-01 4.093028090795995388e-01 1.200410059592703013e-02 1.201723477319382831e-02 2.569940371948281968e-02 7.108642661940715868e-02 7.882437799951799717e-02 2.101180534066167740e-02 2.603637747538105340e-02 2.273891710938745134e-02 4.580171284495654360e-02 2.275871437001197525e-02 1.042265780865329547e-02 1.202549875273498797e-02 2.935119865229137912e-02 8.079536130960355417e-03 1.877784422345734339e-01 7.554822216565395543e-01 3.371058271231130443e-01 3.963006188686431924e-02 8.209504276710571924e-02 2.012480140327661468e-02 4.381695070815395249e-02 2.205623019692680778e-02 3.773609649752313056e-02 1.792567641747800505e-02 2.477787993030928365e-02 2.113529998712059509e-03 6.317128603661598119e-03 8.366975252058800501e-04 3.399885111963541484e-01 6.299051214591403491e-01 1.936502494976621025e-01 1.467694491916201639e-03 1.234565070416649563e-02 2.660727652334527859e-04 5.585582979456816394e-03 2.578942256647214198e-03 1.913974924699757968e-02 2.347512993654473842e-02 4.616490324487316360e-02 2.743127830079449339e-02 3.979225762344711786e-02 5.898479736926327995e-03 2.730087295682860973e-02 2.809329055661555913e-02 4.872665809519888952e-02 2.282588787821815401e-02 2.212345845744471615e-02 2.652860248145617883e-02 3.222725090006126752e-02 8.915197407609853561e-03 1.272446914150442317e-02 2.902366294474302208e-02 5.514906141729002120e-02 1.116166834490276739e-02 1.540415922497988810e-01 6.003918811234852937e-01 4.358694841386731023e-01 8.299276831349404748e-01
2.165252288868023400e-01 2.488529790886650356e-02 6.097850477313883442e-04 4.380412340326336615e-05 2.131010104134947378e-04 2.394536548757845648e-03 1.682778471331649109e-02 6.162771114543878770e-03 1.040659826511992903e-02 1.130321998283424778e-02 4.727765607629899614e-02 4.347440954394920891e-03 3.977609459633626998e-02 2.026038643875617204e-02 4.839962796852197852e-02 1.481812521432348401e-02 4.825473850500725664e-03 2.080666252097482325e-03 2.853794698084174269e-02 2.261784907501428882e-02 -3.406859382786242008e-01 -2.297730055426199791e-01 -3.425735036411606260e-01 2.367569523005592230e-02 1.264734919190944884e-01 4.952012650085373113e-02 4.912862547652661893e-02 2.402280525791680721e-02 2.430758679174011699e-02 3.894218797368733526e-02 9.049415382375847194e-02 8.963721250178922906e-02 1.213670002313233520e-01 5.825909719953632077e-02 6.572073211215141708e-02 4.940239060617389322e-02 4.093875309373924687e-02 6.287571115966038293e-02 1.037700451865664897e-01 3.671890984394535901e-02 1.136060038746271796e-17 3.257046121962673541e-02 -1.139333751544388540e-04 1.301861246117676056e-02 2.922935570634430449e-02 5.170406135989596280e-02 3.988126771749868349e-02 2.414208194250625980e-02 1.965881447217466757e-02 3.522590341645802370e-02 6.420905798033860190e-02 7.764210977333151596e-04 2.335939779750468605e-03 1.215873536061808785e-02 2.558237161907113769e-02 2.801559429665775175e-02 -2.648395715222528479e-01 -1.589990857978773597e-01 -2.233973270261637722e-01 6.267304544596546956e-02 -1.597551721794731594e-01 -3.779741999331588925e-04 1.125028184826564609e-02 7.200242068499466963e-02 1.610466292768941016e-02 -1.448877742339760299e-02 2.301242416450205094e-02 3.370544268633333951e-02 1.952352148688223923e-03 1.445041160911267157e-02 2.586478138179196154e-18 -9.657605241438577072e-05 3.819867600201845050e-03 -9.952261144316616737e-02 3.727266822028375143e-02 2.465918976917303926e-02 1.037143436178508855e-02 6.765888279559734153e-03 2.188391411625778896e-02 -1.420400474161695979e-02 -2.011534499641400236e-17 -1.931351237224710193e-01 -8.125283633813261408e-02 1.792587170566369015e-02 3.582937985285460786e-02 2.929044600708709256e-02 1.383882096239594109e-03 -1.654694104526400339e-02 2.901931223226700968e-03 4.086889976934902566e-04 2.855073211271952349e-02 -2.380568236880234445e-02 -5.065293577496132738e-04 1.521793024517163057e-03 -1.752851489450999939e-05 1.301934026685474047e-02 1.286529594233753581e-03 -2.638454184852661449e-02 6.248398597347617611e-04 -4.081211537924628319e-02 -3.698127776079637719e-02 2.372602488423200887e-02 3.049237063142070780e-03 3.954746785675155446e-02 9.582925552980684630e-03 -4.819028354405582889e-16 4.035768816836958311e-17 5.446061220857672386e-02 -2.160546240346187131e-03 -3.298555130369453075e-18 -9.945917286160971007e-18 -6.342565581723736161e-17 1.551484289942353005e-03 1.030517660994084445e-03 1.673430860483602505e-04 -1.418829048486281580e-02 8.050800456553808965e-03 7.088265960811721694e-03 -2.059622939937504893e-01 1.810694316614736565e-01 5.751923667163817422e-01 4.011647817992927134e-01 2.834682815266700157e-02 4.869359742595206868e-03 8.902231432418939230e-03 4.286988258440331025e-03 2.059209000668073622e-02 1.620977614559685615e-02 1.722789782797670699e-02 7.139880450549977893e-03 5.273691128229503200e-02 1.742837318140228950e-02 3.499553150309561017e-02 2.672041123358973383e-02 3.128441069182234108e-02 2.643909431175004571e-02 2.562374798748832255e-02 1.952593263237419746e-02 2.874319913619664824e-02 9.914637136800725711e-03 3.539581655069789501e-01 8.923048990278515946e-01 4.010353277596133692e-01 1.383243310842389648e-02 5.327323898343771374e-02 4.320801925900621260e-02 4.461520879790489835e-02 2.725651194841855612e-02 2.611171201249730747e-02 3.832895798905399526e-02 8.000796698274934005e-02 4.930490578047806716e-02 6.430140027653151458e-02 5.757220346974521036e-02 3.920204173161871003e-02 1.694781436294732005e-02 3.489661412759770959e-02 3.666845889908997014e-02 5.165269901559162852e-02 -3.423070309207111583e-03 8.641057480655450068e-03 6.236875864761259880e-01 7.194487363504804556e-01 6.109280114336660406e-01 2.125927200909317333e-02 1.833019803189805258e-02 4.611297973778165360e-02 1.285165293532423633e-02 1.971257801664487094e-02 1.145983241421906842e-02 4.357587456721198627e-02 -6.564916291662923481e-03 1.360107947461225257e-02 1.116918974313651884e-02 4.028522440596445098e-02 -1.220370064550177144e-02 1.316289491767839992e-01 6.418726734120336896e-01 4.469832447147888410e-01 8.112394304739114492e-01
8.515964929196727373e-03 2.594159978474013320e-02 8.633231701304666661e-02 5.652860519888092822e-02 6.088925551567844069e-02 1.116151267243528644e-01 1.890170222554359569e-01 1.400601627234082137e-02 4.917057659156579691e-02 3.271508704912926696e-02 7.592403819377241403e-02 1.907122077834266100e-02 1.996098144642777933e-02 1.952257967728416059e-03 3.313203685848892938e-02 3.153963511755510790e-02 -1.679828515590149063e-01 7.596004500847155366e-03 1.127018757621376342e-02 1.176857925403773132e-02 7.581692465037152828e-02 4.350666251449986262e-03 1.471726899608816197e-02 1.997491922580351548e-02 2.376352959884292862e-02 1.982568349808749505e-02 8.013701485202390229e-02 1.475951397476564161e-02 2.036821370560397118e-17 3.983364710006391876e-03 2.396223100993436547e-03 1.345192456974803977e-05 1.474348985389024825e-03 3.971227288939162443e-03 3.810772738426953295e-03 6.912306186123685914e-03 -3.120697132488087569e-02 1.024359006638791156e-03 2.106837605923499515e-02 5.053848284145564390e-02 1.924288325370214145e-02 2.996558253012157613e-02 3.368465771017673666e-02 3.024151026439225037e-02 2.049903714962555620e-02 4.825558105870988990e-02 5.924448686025327732e-02 2.543915747098045299e-02 3.169237468140811975e-02 2.471359490005231238e-02 3.208355448106722796e-02 3.140108135591658310e-18 2.291726713292161974e-02 1.369585764619966894e-02 -3.492130756968171434e-01 -1.778015786420276967e-01 -1.455567991769802227e-01 2.765679369714276192e-01 1.359530822264758965e-01 4.299420172777484450e-01 -3.440037865093392710e-02 -1.784254745237658785e-03 1.671666136339248002e-02 -7.985266576183473958e-04 -1.479468916367130862e-17 -6.163206904331530936e-18 -1.065359837097802848e-03 5.009443132258891752e-05 5.571316191581597577e-03 -1.253553639113134184e-04 -4.052416116064307995e-03 2.467991778071013803e-02 4.633846750102370293e-02 1.637703348929950590e-03 -1.288107191000839764e-02 -2.369888141367351361e-02 -1.888726427709266631e-01 9.802692684653357968e-02 -4.683648947047279248e-04 7.773330264063926277e-03 -2.252089755135597586e-04 3.608147986666159716e-03 -1.020811583972742754e-17 -2.200037919301890341e-02 7.195479928008037020e-03 -1.474402241550568440e-02 -3.074667508775433570e-04 9.308984320954753755e-02 2.336902029115159578e-03 9.547592659750107555e-03 -1.859273304793691156e-02 -1.163358566568915668e-03 1.528938850177763084e-02 -1.947200384249800353e-02 -4.149430062153252122e-03 1.319514440490619778e-02 -8.804104969100441658e-02 -4.922732698298057374e-04 -3.794693604721351143e-02 1.704496985230707223e-02 -1.488811454643694010e-05 -2.894297523632210177e-03 2.888900623398311072e-03 2.930965025780531141e-02 2.993019187567309749e-03 -2.928837554685664692e-17 1.260105154434051347e-03 -2.447700510243462201e-03 8.618268100691467426e-05 1.635210488070090382e-02 -2.519071611848544216e-17 3.595201676481750083e-03 5.010653192703718763e-03 -2.267109727731110233e-02 -2.445337626722238200e-02 9.747967980798633025e-03 4.889418953683930848e-03 -1.431999153218239118e-01 2.457667466121208899e-01 -1.810401445991627334e-01 5.098885740566749120e-02 7.754277085507309264e-02 5.609956948628114920e-02 4.945452707090870759e-02 1.732781279082461906e-02 1.596668048559951314e-01 5.581720419823869850e-02 1.163462295304666834e-02 1.407954764868480854e-02 1.810002833464230063e-02 2.672895148904163090e-02 1.907110189332716552e-04 5.309748142883556968e-03 2.072937505573446137e-01 -1.386008309511370107e-01 -2.656470864196153192e-01 7.656753892920289961e-02 1.708106720673183837e-02 2.455537445029120844e-02 2.313993965194505870e-02 4.743365494145972833e-02 8.336167578028829891e-03 2.957321314804218365e-02 2.912148943733367315e-02 2.953549573983014001e-02 2.846546267608226349e-02 5.621425466258910680e-02 2.341162707295992365e-02 2.319105168073256132e-02 1.773503675584025847e-03 1.497120446152828445e-02 1.113202318915612263e-02 1.873945181751014363e-02 8.652074709354730386e-03 1.744473800437223651e-02 1.801372223778468064e-03 3.828059393022583638e-03 3.622831841555233454e-03 1.215814214305555724e-02 7.280296295715218413e-01 7.199608816895542995e-01 6.623447992559572395e-01 1.372023425172734686e-02 1.482977165433996486e-03 3.479683154356148084e-02 4.672147543232259975e-02 5.478584459203410251e-02 3.352814579199154404e-02 2.301676082355721270e-02 8.761824343450514210e-03 3.196732466800904920e-02 -3.181393905137519214e-04 1.362170976326071599e-02 -1.424830776571715547e-03 2.680179805723227338e-01 1.330355863781914172e+00 2.591089477148284526e-01 7.252288629490370730e-02 1.209347648822239191e-01 4.675762804935490458e-01
1.516391210726361904e-01 -5.961620176519343700e-02 7.006787105410049177e-02 6.767373882252131212e-02 2.478578754759593744e-02 9.130417336336048295e-02 1.107743237180734558e-01 1.031075759533169912e-02 2.547912244570240889e-02 3.090682121528601223e-02 6.507305116912000253e-02 1.747272317845435796e-02 1.366748637432160275e-02 6.829548298460752509e-02 5.649097060409161847e-02 1.044345353700357054e-01 2.703414881752631560e-01 7.605700470670388735e-02 5.827388500593098930e-02 4.217458303572534095e-02 1.406350142636263456e-01 6.973294192664036828e-02 6.566580768445767768e-02 6.858842271574874805e-02 9.393405268598027658e-02 5.743144911884599801e-02 -4.685375041685049119e-02 -6.175380045768621984e-02 -1.135058890549245220e-01 2.871084766496141173e-03 2.736941385278580854e-03 2.242993224658536636e-02 3.938951684193979681e-03 5.945692347076971147e-03 1.056093743332830667e-02 5.580821158858558818e-03 -4.587239178671206202e-02 4.598966891811726368e-05 3.374955341303813999e-02 1.763240986583250458e-02 6.031476759279253211e-03 3.433684298137281571e-02 2.424121048457542754e-02 2.485891965203031184e-02 4.346430994055586194e-02 5.334299134303192164e-02 1.428251162268991958e-01 4.090031423294908375e-02 2.954877308364145180e-02 5.040175242771167963e-02 5.539874370449370672e-02 2.993067274752269150e-02 -2.742401466112124805e-03 -4.576889122640618451e-02 -7.201597488988924212e-02 5.746059082264238366e-02 8.734984912400535484e-02 2.779315945674549759e-01 1.958737681652787110e-01 1.916593605734451533e-01 -6.947727801609426823e-02 -3.945915814015506402e-03 1.871047835351576094e-02 -5.550778085141896967e-02 2.084094136740011205e-02 -2.295497957253356258e-02 -5.716311551336171774e-04 3.925436701146403640e-02 7.446188726091770849e-02 6.941086321857907471e-02 5.516308835919337364e-06 7.732411936214012797e-04 7.255057276411127146e-03 -1.693503461142789254e-02 2.197022768532810866e-02 1.938407210437015360e-02 -4.889312807012315132e-02 2.673977991709625829e-02 -2.922436163600928075e-02 -2.208769087485818738e-01 -5.176713959020612810e-04 2.539211791464289839e-02 1.273381778384049786e-03 -1.500566163726597957e-02 -5.997388477308403922e-16 -1.612279023246958928e-03 5.527868702171026202e-03 6.427297080604989915e-03 7.544871419990774666e-03 -1.311950552139022034e-10 9.913604345014913680e-03 -2.910720131533359423e-17 3.836404024921501346e-18 3.826543787436244293e-02 -4.995608121027917195e-04 2.083662261295441895e-03 8.029387063905017588e-04 2.178573539160483935e-02 4.608313049322841008e-04 1.361036869999492256e-02 8.715582776551213454e-03 2.549958656493566192e-02 3.051660521642880677e-03 2.243551154029371081e-02 1.145956159598918687e-02 -4.578077109137451001e-02 6.827918588648119493e-03 -2.703782858526290918e-03 -3.975884993440702807e-03 5.114522658969778562e-02 -1.412780868923060699e-02 5.269319366674033583e-02 -2.905831108730572746e-16 -1.023425703217446629e-02 -5.848059186912341794e-04 3.740664356861306966e-02 6.245617690809650985e-02 2.095332578196302739e-02 9.255089333085832204e-02 2.853747278712150115e-02 6.271517650677121969e-01 3.735744922717559868e-01 6.483945687260157797e-02 3.356481326027031115e-02 1.984541654926789619e-02 5.202614053170775965e-02 7.500304831870363564e-02 1.171165930465834643e-02 3.012856342405995091e-02 3.998263802876415168e-02 3.505298399286727040e-02 2.401439767143927553e-02 3.010506712273600116e-02 3.389490096778591255e-02 3.954251402093592743e-02 1.006148891171486509e-01 1.486171550956280341e-01 4.199935533174405100e-02 3.444992141522193463e-02 2.392774885843280963e-02 3.694385207892912637e-02 9.592040428148217862e-03 4.140804942701300367e-02 3.095964399965576985e-02 2.010921351816665967e-02 2.203954396054610618e-02 9.069479352637651592e-01 1.412695238664137376e+00 4.746845002789081280e-01 3.112763592462768677e-03 1.147121364986892697e-02 1.715548618562397556e-02 1.587549063498953569e-02 6.487019502800331003e-03 1.454269264179944461e-02 4.557451791009348857e-03 7.330556821646195750e-03 1.662696681481794919e-02 2.419583274676304074e-02 1.084706208970435612e-02 2.004657519489779632e-02 6.833305942629286367e-01 7.452131399366777043e-01 5.933609242346697954e-01 2.189287857304009904e-02 1.686199171369945002e-02 4.604551210079022711e-02 1.228104628791847758e-02 1.480103594352608423e-02 6.606079110290027055e-03 9.529927108170679548e-03 -2.700756466228822206e-02 4.478839611167184098e-01 3.132997047864032325e-01 2.581047678166173975e-01 1.016134434690427364e-02 9.141738389119866137e-02 3.647608875255100025e-01 2.411222627685785658e-02 3.007577994369568103e-02
1.182231863202856115e-01 -7.078747755815961717e-02 3.792242223062557149e-02 3.433669970036762331e-02 4.040918015619436587e-02 7.119123083618880687e-02 8.551113690182109150e-02 1.431448486887799444e-02 3.798588343801327649e-02 5.768858242852371865e-02 7.053759900175676778e-02 3.195442251334076195e-02 2.209791566794122239e-02 3.265909157490265446e-02 6.453655911602830531e-02 3.188077338010492090e-02 1.498261236931430485e-01 -1.163651618974951979e-01 -2.281508053181664167e-03 -2.030823738768974818e-01 2.477159605876146087e-02 2.445794965904434827e-03 2.004344572333468036e-02 1.114414218404869567e-02 3.044270594310170136e-02 3.252381620086156055e-02 1.499196957304504974e-01 4.843741321695481383e-03 8.755801363174059090e-03 3.804649990384430941e-03 7.793464784653624140e-03 9.231713612661093414e-04 4.628104323456653678e-04 8.207984271652122438e-03 2.833593158418367536e-03 4.957620844381669875e-03 2.613975204023422441e-02 1.773740283407269566e-02 6.990480538432639057e-02 4.257095578596378944e-03 3.095148240255203576e-02 2.623283136818884559e-02 3.836505836246559970e-02 2.187899711835948960e-02 4.176698059833475196e-02 5.010994907115780966e-02 7.929791522629313361e-02 2.648233311973966983e-02 3.928486772613520861e-02 2.811193334000091845e-02 3.243175900791280680e-02 1.300311360061432407e-02 -2.280774234957033599e-03 -2.976543077338710680e-02 -7.135040716544029105e-02 4.775304616532807800e-02 8.545880492805318918e-02 7.234414784382979524e-02 2.608832058895830808e-01 2.789267807960451595e-01 -5.303526748973219629e-02 -9.502520094660481254e-03 6.735015578364395017e-02 -5.150853895068499555e-02 -7.409040980250225158e-03 -2.410726783280615557e-02 -1.827003644786726238e-06 -4.041248441757637304e-03 3.683405123317110008e-02 3.386447018973313960e-02 -3.831883036922838259e-18 2.201724466597957883e-02 1.947623752708581985e-02 1.301957161873088572e-03 3.067139080634313786e-02 1.816760905994755371e-01 -8.293499562353465193e-02 1.603241050373237220e-01 4.698077333516406662e-01 1.399727590937464572e-01 1.170741104691884471e-02 1.264565231868182982e-01 -4.939631060469923701e-18 -8.412356066149156639e-03 3.837633353839689043e-18 1.154457951182729161e-02 1.324815706308905914e-03 6.404575380443432164e-02 4.515765106680036389e-02 2.268846037860136264e-02 -4.160703697268249707e-02 5.191828307372450824e-02 2.208910702465653395e-02 4.691396688043296404e-02 -3.076786520552833885e-04 1.181126880676855971e-02 -1.019760389523406835e-01 -7.712122579142354532e-18 2.627832510542009101e-02 1.568068586141029008e-02 2.097674908645625821e-02 1.801474983246273529e-04 -1.014076549622248795e-02 -1.215873902802979203e-03 1.180068869849721378e-03 -1.620402533928629041e-02 6.021576595042727603e-03 2.308334841634087850e-02 -6.590382459210062001e-03 -1.533121687034503730e-02 -5.658522338342133734e-18 1.291586742199109196e-03 -8.567181151352684177e-04 -1.303191923723760227e-02 -7.284364444545677166e-04 4.477195262077388743e-02 5.977114742854978263e-02 7.014971595339827837e-02 5.677280496523066239e-02 4.771969348352001622e-02 6.435880281297787775e-01 3.856657457181412729e-01 1.788793935889582132e-02 6.313062352340392813e-02 2.087930892674364283e-02 5.582941107325012364e-02 4.708072939086867942e-02 1.917814054540449564e-02 2.266821604588544620e-02 3.185283323845117254e-02 3.855424503912570355e-02 3.344616130702113277e-02 3.143998172837037669e-02 2.922122865889255697e-02 3.126377931292492057e-02 1.227307993641530426e-01 3.654215725711055379e-02 3.808770697173901043e-01 5.578688483470016779e-01 2.720520237587743528e-01 3.355726755047648557e-02 5.758556359733247275e-03 4.447704950080570707e-02 3.260043185919486025e-02 2.274320317649932213e-02 3.878650309929246309e-02 8.208143338436178771e-02 7.718426685469902647e-03 8.621802112421578582e-03 3.781685704039250417e-03 1.255929360024900590e-02 3.898118526218247041e-04 6.766092478780505286e-03 1.025939716290334702e-02 1.780084494545768059e-02 1.317972514010322090e-03 1.129557376115950841e-02 7.000406270105535844e-01 8.127809271777179934e-01 7.762688697223548706e-01 2.186226923299769073e-02 1.713567314542122078e-02 2.611581129401106816e-02 2.696799907015337214e-02 4.849391173030013352e-02 6.305924576362666178e-02 5.309942797499137696e-02 1.808300185138981067e-02 1.680460618223326985e-02 8.598374741944955768e-03 2.690692290324509656e-02 -2.234951164520246596e-03 6.097961198036370156e-01 1.392131328856648143e+00 2.624905974616277282e-01 1.702258485664850593e-02 9.579413476461426380e-02 5.435547848588629521e-02 4.010585962890850925e-02 1.039917183249672306e-02
1.416267739216678923e-01 -6.554856626102181782e-02 4.208988787099127854e-02 6.458844335626232058e-02 6.249230947125422597e-02 7.278938503153632722e-02 1.492625175600933440e-01 1.926574295170131734e-02 4.821117928171184119e-02 4.076957616311620985e-02 5.570011546403657449e-02 2.877085626537647864e-02 3.323656916562152397e-02 4.973018853137057410e-02 8.399912180587967847e-02 4.984327371424289443e-02 1.105473345332580876e-01 -5.668827501074270164e-02 -2.280639365751812621e-03 -1.131026065840503442e-01 6.041499332234354785e-02 6.270574068621866115e-02 3.757167240533842295e-02 4.764273271296241841e-02 5.513075444619413429e-02 3.681952731048800109e-02 1.931033243807191713e-01 5.067236972188358252e-02 2.664224951740661734e-02 5.221077507416270469e-02 8.290378688269987861e-02 2.467100101093491760e-02 1.181237926126889048e-01 3.007342242707030169e-02 1.523811530563896118e-03 2.107863480971043613e-04 -3.841542578041909350e-02 3.897100011483015190e-03 2.194921002192661355e-02 1.005571057094301404e-02 1.794454264711436292e-02 2.333038911523191775e-02 2.317848703115209241e-02 1.546101190408170033e-02 1.986913095692337608e-02 1.906635316648057701e-02 8.606097729260014928e-02 2.029332889255516456e-02 2.353241533368083394e-02 2.369096462425200611e-02 2.280340917384825541e-02 1.313134723697525520e-02 1.824018147910331719e-02 1.856623315079634906e-02 3.264999423139446771e-02 2.052792160103759228e-02 5.617817691327647162e-02 1.549722482102459421e-02 6.240867548892257283e-02 8.173079982835042279e-02 -4.450043297298750256e-02 -1.102496855324970474e-03 1.679654148652204582e-01 -4.085687226237365299e-02 -6.495611515632093540e-03 -2.071625676129105337e-02 4.312159596039958492e-02 -1.469450435078674409e-05 7.164571722827757505e-02 3.454845013120466657e-02 -1.358746756656581574e-02 5.641768017294415162e-04 5.951563298369327848e-02 2.190410442654768322e-02 1.907869289744050512e-02 4.471394767585908786e-02 -1.308109353151713228e-01 1.611440752750688821e-01 2.566084595993646800e-01 8.684674832424636726e-02 4.692852358627444542e-03 -1.378513281250359247e-02 -7.020073608904419012e-03 -1.954611218217279511e-02 -1.574882771078048630e-04 -1.762946187681469629e-02 -7.653959329595997862e-07 -3.149914523818727469e-02 2.954287724124435646e-02 2.082912517326781371e-02 -8.281914912143412133e-02 9.128032111613090804e-02 3.096970925560678031e-02 -1.880047071833277776e-05 1.226914152233185736e-02 4.210195505073092287e-02 2.117589496701656818e-02 -2.464080199899847459e-02 3.941225303376638721e-04 -3.222863843565212331e-03 2.231607601665833943e-03 3.761996274311575281e-03 9.834632629050798699e-03 -3.867663692108947978e-04 1.491273916331579442e-02 7.334700941349135694e-02 1.871860228697789716e-02 1.062166469552408739e-02 -1.813973528229351944e-03 6.018667599769123022e-02 -1.195908416419698062e-16 -1.273608582882642521e-17 1.957386202659805474e-04 4.980169092595387176e-02 7.590626827643352781e-03 -2.376975299530343158e-02 -3.075965438587910730e-03 7.266142279723435127e-02 1.188810064742434058e-01 -7.236142547575890975e-03 6.495904649801176323e-01 3.342769409694907323e-01 2.394960155870259899e-02 6.737783680205795822e-02 1.999089621163411754e-02 6.520286435575714523e-02 6.153465125761953258e-02 2.129923751957210620e-02 1.336849120327467978e-02 3.344822881291355243e-02 3.544681012959053495e-02 3.777027485518177907e-02 2.203542951797568192e-02 2.608963363531575236e-02 2.365784939982259680e-02 1.197170532137925708e-01 4.080654950211194676e-02 4.711185873114773770e-01 5.328403184934406012e-01 2.669017884201546620e-01 8.011630991608858013e-03 4.481481286576076938e-03 2.529479149701430868e-02 2.267998651648426775e-02 2.822574494999186348e-02 7.052219812602357474e-02 7.517517739293540224e-02 1.818931166179131587e-02 4.941942031287491210e-03 1.148055790115077787e-03 8.614567834135205410e-03 6.719486800626388590e-01 7.993461934463648166e-01 7.507868706795675084e-01 1.551334365753848782e-02 1.576212129060099425e-03 5.245365401743610698e-03 2.416728171040475985e-03 2.104690756133891991e-02 2.439092364225821721e-02 3.993075741566878833e-02 2.352512885759908134e-02 3.862605773416372124e-02 1.013258629573417745e-02 3.308030401134681719e-02 3.785523744382714856e-02 5.827944421344829345e-02 1.873788863360786178e-02 1.750831311456550743e-02 2.192068473625830363e-02 4.605080810585713419e-02 -8.356624404079355259e-04 1.703929783258265399e-02 3.705269721986274856e-02 4.186930504738776981e-02 2.999998952467991184e-02 1.021868807821569858e-01 4.267183562797881025e-01 5.842345483222272096e-01 8.712116140045516044e-01
1.521868186404019552e-01 -5.549691397782408053e-02 5.774454871505790149e-02 5.646665151264945726e-02 5.743709963387139128e-02 6.763534405727730792e-02 3.000345861865467145e-01 2.798595976761836940e-02 5.374756489114056179e-02 4.295079902931234206e-02 6.652107830572798197e-02 9.015848818858369917e-03 1.567116609308651098e-02 6.720678573246630094e-02 7.895582164046764662e-02 9.157987002914703967e-02 2.983028986676906635e-01 9.031725011294398919e-02 6.389688070868215081e-03 -2.741236287970999497e-02 -9.215908393272097265e-03 3.120488102710728892e-02 4.934199052098977439e-02 1.645973344741280195e-02 4.830274372637112512e-02 6.852498588658692547e-02 1.622700235543982350e-01 1.074055355244571131e-01 1.652455686336065308e-02 8.938418309051454624e-03 8.325401822496816817e-03 7.701327102067198442e-03 2.519860266626120556e-03 6.509506118979972548e-03 -3.667798760833876261e-03 1.143793758016802878e-17 1.066525351318305809e-02 1.131749703124965085e-18 5.268940291514634215e-03 2.572343326348375953e-02 2.768840488492323865e-02 4.204235110386420060e-02 2.289090095286005697e-02 3.497227631210672160e-03 2.078090625355369370e-02 2.477386230372277329e-02 -7.145037661982479860e-02 -8.491666280756615421e-05 3.320092735457697702e-02 4.396255698663859013e-02 6.437192976620681462e-02 3.817647728701198395e-02 1.727545698634223859e-02 1.878130972233846632e-02 3.078650597741619185e-02 4.320292829110695260e-02 7.352457439706758247e-02 4.291792994885090112e-02 7.218010753964725079e-02 7.817548082530223486e-02 -6.149501607840249934e-02 -1.263464920184472623e-02 3.954369356510014505e-02 -5.465023685162669065e-02 -1.663450679672573722e-04 -4.754252801644975705e-03 -6.968354445341518553e-18 5.969972454452074362e-02 5.194227096835446825e-03 1.310712709798820923e-02 -5.383342578167835818e-03 4.794663779341250959e-02 1.277829880468262999e-02 -1.785133079134387443e-02 3.266127379296866057e-02 2.514164660837380633e-02 -6.584459969772926691e-02 3.822907988325819489e-02 -4.650210152460854812e-02 2.656016670574029657e-03 -5.300412496575290094e-02 -1.458651015470188717e-02 2.140624981989704134e-02 3.415712989848335157e-02 3.420915272390808760e-03 -4.409229749178873205e-03 3.171788951765219799e-02 8.631473847084706816e-04 8.199334005782987767e-03 1.544117270472587962e-02 -1.743592551537944541e-02 3.460070527351478964e-02 -2.234669485720525497e-02 2.022048568103541541e-02 1.184456367346527726e-02 1.526290783851732245e-02 -3.084214526682436935e-16 2.548084779014214013e-02 2.053904484148873395e-03 5.937711575802645525e-02 7.297542564750687799e-03 2.148525705360387617e-01 9.132917204501313224e-02 -3.191535197213748315e-02 1.253103803608807739e-03 -6.442049162611640901e-18 -1.942354397471972305e-03 -9.139587000478196707e-03 -7.356305638246109441e-04 1.211659033170354657e-02 -5.792999682220100929e-17 -1.307199395686844609e-02 8.056695736922594997e-03 -2.209665912631704617e-04 1.231583772473271790e-02 -1.567491801325990655e-02 -4.383368120831796291e-03 1.409634449808800771e-03 4.308206742948429702e-02 -1.692096583265009779e-02 6.020803991700456326e-01 3.570420916943616851e-01 5.443173538057388683e-02 1.825156230035512514e-02 2.913428758714940725e-02 4.195090440136430704e-02 6.267921357809214111e-02 1.985720895244345516e-03 1.266139503098300846e-02 1.500776181104328429e-02 4.131403195005522305e-02 1.157393481969007203e-02 2.664524865862987965e-02 1.533202348469932252e-02 3.818572712971363137e-02 5.714100110527819965e-02 1.094135478028348679e-01 1.778244342405594855e-02 3.924814386593117321e-01 8.070549706245621069e-01 6.357975920916257850e-01 9.456628149628836357e-04 2.772329290627332729e-02 2.188635396750096898e-02 2.510492879531869836e-02 3.364159257887139320e-02 6.553244824014506364e-02 4.091889905828286107e-03 1.513480322213438950e-02 4.039459874288013966e-03 2.003472085800379887e-02 4.700046211211784432e-03 1.105471675110140592e-02 8.786342617429167001e-01 5.681258101408320771e-01 6.863660431792092398e-01 7.022193242560950850e-03 -1.269508968248073480e-02 8.492997280136922733e-03 1.797977173991363586e-02 4.350534116014927871e-02 5.226862902948634787e-02 4.817589969503385111e-02 1.104722596904737786e-02 2.215211439036325961e-02 4.725915421071796158e-03 5.317916818895667141e-01 7.662640214146612117e-01 3.323681343481291739e-01 2.393208260578123716e-03 2.612449992578701197e-02 1.075320965118195700e-02 1.584093726976021355e-02 2.168425870336862168e-02 4.545863201562998918e-02 1.555477132017800759e-02 1.096001289472481460e-01 6.609741817859969559e-02 8.811750150770959744e-02 4.858240988054950138e-02
1.890087085365660668e-01 -6.582623860728456511e-02 7.265978710799381179e-02 3.662059974822175212e-02 4.448322088120281598e-02 7.603270067585769787e-02 7.313942880928800405e-02 2.166803054225387506e-02 4.388965660324369716e-02 4.123909524113898717e-02 4.524026316827918504e-02 1.967586269315707917e-02 2.135141862864890772e-02 4.916012298817694359e-02 6.016166600562423278e-02 9.260009331310412040e-02 2.771794392135273344e-01 7.317724692923352481e-02 4.549215240031410923e-02 3.031645629866296099e-02 8.589488070815168297e-02 5.170335878158589110e-03 6.482967214522081245e-02 1.623600571196851805e-01 4.776831150170685047e-02 6.678295242716028590e-02 7.751048413583265517e-02 4.759171551974397674e-02 9.331331260642686254e-02 6.015042446400350323e-02 3.205068349968060587e-02 7.496316497021442149e-02 4.745540743982132326e-02 2.829981150444664698e-16 1.906019715423817092e-03 3.767460479629469424e-04 1.165237324121361218e-03 1.950280188865652700e-03 1.785802270828355451e-02 1.139853402756480817e-02 1.704476256508401927e-02 2.233225353675369340e-02 1.906690582887866220e-02 1.561332850101900156e-02 1.887681394912695682e-02 3.509559193916528957e-02 9.090362610587243708e-02 1.341402920583868383e-02 3.032059701676369234e-02 1.871369913815311078e-02 4.714594246725493931e-02 1.282860285012356674e-02 1.111524168343547628e-02 4.296624948872356312e-03 2.962205917821610426e-02 9.985156196740629236e-03 4.383133163057721660e-02 1.480022711303460338e-02 8.137419140216736735e-02 6.841634470443538896e-02 -7.735314303272723679e-02 3.069851273805997583e-03 3.240424499626571447e-02 -9.072498845118703859e-02 2.510380532179356736e-02 -3.562359532056494360e-02 -1.388255289879480479e-06 -6.997017852918139722e-05 1.816558348641763698e-03 -1.343339070676406385e-02 -1.121973247559791484e-02 3.087188216915251818e-02 5.695464365381697203e-02 -1.595276438045249365e-02 7.014954744142856691e-02 1.829329866028336915e-02 -1.958735561948169238e-02 5.652246214220731091e-02 -5.351235610741478937e-02 5.071379481452140950e-04 -5.917652463566833165e-03 4.989968437146857089e-02 4.484809220314887879e-02 5.588585750274134567e-02 5.480608304325901986e-02 7.715842406928025143e-02 5.644000865503807224e-03 -2.098391445867611727e-02 6.031782231947663969e-02 4.171999386150371247e-02 1.042924479135597937e-01 4.317139576687321206e-02 1.626731971735714266e-01 3.949797006328700059e-18 5.882374985375814908e-02 -1.078452965222228832e-01 2.666855334572525854e-02 -1.838093913339188673e-02 3.216961516405165977e-03 -6.615749531916591264e-03 3.615645967518202504e-03 -1.698212762777301377e-02 3.504087732443739167e-02 -2.090494098428085160e-03 1.428666905448904186e-02 -1.177933440614014418e-17 -4.464479756664128853e-04 -3.121578976073396962e-06 -4.295981860168433808e-03 4.697086234290503287e-02 2.691497410081414299e-03 -1.719300921015823672e-02 1.860358835085438841e-03 3.617691109929703497e-02 2.697620395929790568e-02 -2.351377347221100117e-02 -3.037801165393234394e-03 8.359518335338432526e-02 1.050438262058002309e-01 -1.120163708542193728e-02 5.816944430303578883e-01 3.290795992321698393e-01 5.399746887152506725e-02 1.917718909295791987e-02 1.759613074733627422e-02 6.242693215903281911e-02 5.924219854100026134e-02 2.161563023239217174e-02 3.427879553767645970e-02 3.932223227603466598e-02 4.109416678017984575e-02 2.345542275243168648e-02 3.903335265426163131e-02 3.229204546333683951e-02 3.546661946405041399e-02 7.468520838979189613e-02 1.521570275783354076e-01 8.861185748045857791e-02 3.928645002259081864e-02 5.113976966263718282e-02 3.276535171864181778e-02 1.604219909672137054e-02 5.385804893652975656e-01 8.338212999633191069e-01 5.746367653313990687e-01 3.399605135554320794e-03 3.542471976366916991e-02 4.012894483487934594e-02 1.688198168679168332e-02 1.930501174562699468e-04 4.123852992752294333e-01 8.407195841045440599e-01 5.222481752289241941e-01 2.819395056057907958e-12 1.084836923195155352e-02 4.561242338207999007e-03 5.152042406537312401e-03 6.525003524201110176e-03 1.114133633724394973e-02 1.831377953718204415e-02 3.282822324053862734e-02 3.645259565616588288e-02 4.670979654380543605e-02 5.220220224803757449e-03 3.399644303962929409e-02 4.172725002292884677e-02 4.950306706703824083e-02 1.450193677193325516e-02 2.444478545051135404e-02 2.754990580724044069e-02 3.276741846008249942e-02 8.186850725938768156e-03 1.649118340632096946e-02 3.966457646695676492e-02 5.635175430176958544e-02 1.268483829695148113e-02 9.152970991115456922e-02 5.782615642938788714e-01 7.374552695149106940e-01 9.843996723948618133e-01
2.088238372683448862e-01 -4.787280828629471990e-02 5.146120373932706188e-02 2.534143471548462936e-02 1.397576785544896692e-02 2.985191602200666303e-02 7.906886670088879299e-02 2.520339218306908574e-02 2.465929337744026534e-02 3.974167787278763686e-02 1.143272126355509982e-01 1.064372683749797120e-02 8.966203749893311356e-03 2.326180259309019657e-02 3.004729231076915669e-02 1.562869648990397042e-01 4.778974287312897801e-18 5.668580107561758952e-02 3.571754171594138799e-02 2.860471939375286085e-02 1.007808903681410434e-01 5.052869643810484201e-02 4.362359868998891971e-02 3.400523429683967314e-02 6.758015219863838796e-02 1.004911315327780413e-01 1.700741235706188237e-01 4.469856529835192910e-02 3.901135156399340720e-02 7.273728223520938875e-03 1.374580321074778295e-02 3.439844741841579373e-02 -1.058884464523512751e-02 -4.619563198532761900e-02 -6.416208113506004229e-02 7.328403888237388850e-03 4.899553424692551376e-04 6.328037735357470994e-03 1.797353976301840212e-02 7.464560681323995778e-03 2.256457523397991521e-02 4.403738378875684395e-02 2.587774556648193142e-02 9.851823673285681551e-03 3.054895207470740073e-02 3.528563871510151384e-02 8.441146096502084861e-02 1.679788100078503238e-02 1.109010658028095238e-02 1.870463903687586207e-02 4.295581394746117443e-02 3.833182074242547522e-03 9.207854125485082086e-03 9.860059151562414539e-03 2.950355774505513881e-02 2.292353297798260109e-02 -2.575849361241537294e-01 -1.662868541924352461e-01 -2.109651438043639859e-01 6.373288330573333160e-02 -5.177769445931532794e-02 1.037426465097432589e-02 -3.195530961662580716e-02 -7.441514945525691148e-04 2.934178102264722171e-02 2.874731375372127507e-18 -3.908457664813770821e-05 2.273314992819024105e-02 4.742372965847411331e-02 -1.401457202151107535e-02 -3.126233266334919573e-18 5.972995693146938151e-02 -2.670438272661527362e-02 -1.988305647812263488e-02 2.179896951827966778e-01 8.227458683165603942e-03 1.523748511207189127e-01 1.074939451392296040e-02 1.969398323739152482e-02 4.797250380945386646e-03 -1.754426302571614150e-03 7.193894202174521205e-03 2.581459948283855096e-02 -4.599128864484569590e-02 -3.029426927905273879e-03 -1.724314331124042476e-02 -4.491568424044241163e-03 -1.189623551923925729e-02 -5.061692214149853440e-18 -6.234837399001006101e-06 -4.434200077283121516e-02 -1.837329356138200226e-02 2.206488343705842806e-03 1.008636109401951642e-02 1.436541670980118722e-02 -4.557134452749998880e-05 2.374150953870348850e-17 1.861001849109775175e-02 1.274960490747302258e-16 -2.537985764521702979e-04 4.713045010578512800e-03 3.853054824607678869e-03 6.623360652908858934e-04 -1.450235792852901031e-04 6.100791199148985870e-03 2.563138088325180356e-04 1.766131126927039663e-02 -2.760775245226853923e-18 7.166276244614319538e-04 1.607466222289976460e-02 -1.022202717670775700e-17 1.125695802423980901e-01 1.163439757053441950e-02 -2.563817902431163163e-05 3.283143073139119412e-03 -5.333182054576040415e-03 1.421627045071346442e-02 1.671642583341823377e-02 -1.567373790775368203e-01 2.593242562381251060e-01 5.565617094730846981e-01 3.369913819842373925e-01 7.041880365466787750e-02 3.311842286257866974e-02 2.988358172201579718e-02 5.212865414895855726e-02 6.103173313462664090e-02 1.105729142268683553e-02 1.785833166529505328e-02 2.587164447323296590e-02 2.621256306301696093e-02 9.942695175279658501e-03 2.337500687699595875e-02 3.780002985369409086e-03 1.182225903202849780e-02 1.529919503715370555e+00 7.266270236102917446e-01 7.020875491465494722e-01 1.791902284872310194e-02 3.221810496536596513e-02 4.682922079767124629e-02 1.357654767671980650e-02 3.632120498092638455e-02 2.141709857639330361e-02 3.082385215631360323e-02 4.303980754047682239e-02 7.789246821851868174e-02 1.336703580195888232e-02 1.673610103017145884e-02 4.411868440950064002e-03 1.394556794744407531e-02 7.164196253227786242e-03 5.272253624224298996e-01 1.079609265011209684e+00 4.007302450977941466e-01 3.136525962954407436e-04 5.895093244314612499e-03 -2.024180081422568764e-02 1.594662483622843324e-02 2.032089994466365623e-02 5.292646633244541682e-02 2.634550051134803422e-02 3.546884805963101961e-02 1.111393000306104377e-02 3.767204921821560443e-02 3.530780741068884759e-02 5.209297942762317057e-02 1.309317411347404349e-02 1.822395000543947097e-02 1.699520205734036601e-02 2.497441002080618214e-02 -4.503303117382263581e-04 1.074819987669050139e-02 1.118026928044431843e-02 4.304503510586446535e-02 3.732155019337231056e-03 1.347994840385164095e-01 6.168814559575324097e-01 3.980976591496673822e-01 7.724546457029562996e-01
1.953912603326886099e-01 -7.576256862897140509e-02 3.259688413556611347e-02 3.039289982519163824e-02 5.023784147060260385e-02 4.740558332283557191e-02 5.264884793274116070e-02 1.852950475013342979e-02 4.803664589488911008e-02 4.734697023823334489e-02 1.237806585385261005e-01 3.341820021745459113e-02 4.527702567536008582e-02 6.674679880673906052e-02 6.271454668843594316e-02 4.384550157199385706e-02 1.496185906750212569e-01 -7.487045461466991514e-02 -7.251220845253421136e-03 -1.283506416618535040e-01 4.836301768061289363e-02 7.088044081984431338e-03 2.966917330453968521e-02 2.631988754442703832e-02 4.089167421511664437e-03 1.992234535450687627e-02 1.645589462967643746e-01 7.496439612631629335e-02 2.110093443457029345e-17 9.663441993909871767e-04 2.434528480020018131e-03 3.332005098463555748e-03 1.701174598171683748e-04 1.447211876837422445e-03 4.104015780123301815e-03 4.123070008157299002e-03 -1.131096931551206775e-01 9.243240133388720215e-04 1.054743116327273855e-02 1.342266344905607665e-02 -3.654299890719184783e-03 -8.806270200755736410e-03 -4.650594278089274780e-02 1.240479558118118622e-02 2.446069883878774706e-02 2.586556763269721657e-02 5.078615545656377933e-02 1.589720907142973955e-02 3.776304744816973841e-02 2.748126908476138455e-02 2.361886666458873352e-02 2.373097223252137664e-02 2.189133856782774293e-02 3.148477162072905317e-02 4.569525201109804480e-02 6.216912850415678982e-02 8.814877767055766256e-02 1.693626234996806895e-02 6.528318057047476086e-02 1.466639517518917094e-02 -4.668824990531882796e-02 -1.321066865759826735e-17 6.158409544744103403e-02 -3.944333869969045558e-01 -1.455036772793019077e-03 -4.039095666319754831e-04 5.508522812803018365e-02 -6.710432110693631894e-03 3.206282467621563576e-02 2.152353509772036988e-02 -6.236318881921539513e-03 -1.373464491980651335e-03 2.434923864923957751e-02 -1.916107539281152777e-06 2.873486031866401627e-02 1.199134851932049245e-01 -3.655322596712899663e-02 1.693827367323148814e-01 1.052412558928312453e-01 2.414621817941922099e-01 5.545729978608853611e-02 1.413772474960079892e-02 3.744609338108916691e-02 -4.222245616922264480e-03 9.089823997346864906e-18 4.806713532555344226e-04 2.008199254669833927e-02 1.604120071872846068e-02 3.596997059472494679e-02 -1.010295509090628277e-02 -4.816789523434581582e-02 1.125526910484356616e-02 9.111615408710038028e-03 1.604506281256328284e-02 -7.284240666341839331e-04 -6.516719296682099705e-03 -2.589289628163331483e-18 6.123618314062710189e-05 2.897293946353211511e-03 -3.037918653341611236e-02 -3.614477832578349200e-03 -1.882885654344337856e-02 -2.629370584640659120e-02 1.949764478516825871e-03 -3.123626449598762914e-02 -4.482796441694432990e-18 -3.654997749375465434e-03 -2.051583351999267460e-03 7.453145070803341918e-03 2.453009307752049284e-02 3.005716352296837826e-02 -2.981129303368980354e-02 5.861721368829181031e-03 1.977037675248523863e-01 1.508209809307338618e-02 -1.202984584711582969e-02 -1.111775665437656543e-03 -1.323223867861526848e-02 1.194356946711930684e-01 -1.023161448006794025e-02 5.788796111333295302e-01 3.539454142438592799e-01 2.202839464454023691e-02 5.998608141729677712e-02 3.134599263546605785e-02 6.274809459146392376e-02 5.387799435853068281e-02 3.263809182684883348e-02 2.000497101957203136e-02 2.905480120571965474e-02 3.550867704100282851e-02 2.927610139129247988e-02 2.765253792326588175e-02 3.229104611221070287e-02 4.027962610389326098e-02 8.478987804534092454e-02 4.552868369463512754e-02 4.386006125666023769e-01 6.080602158151000580e-01 9.584753936239556182e-01 2.834671770559190826e-02 2.031051395122431338e-02 3.702633269686143869e-02 4.340296661540220952e-02 2.508440532878507614e-02 2.269827667644757491e-02 8.712504327155297379e-02 1.137773456189752376e-02 2.733470702128982383e-02 1.718101858909602169e-03 2.327038482547410039e-02 1.366741959401478928e-03 9.973526801997902036e-03 6.060244851755375653e-03 1.438603058425170882e-02 1.204543546764642696e-02 6.265903857342961132e-03 2.492210128093585303e-02 2.060103898370575434e-02 1.173367537431359645e-02 5.760050705619200739e-01 1.129404992332896596e+00 7.195501650256618253e-01 4.897293062965085882e-03 2.415746276952992350e-02 3.504700861418461738e-02 3.902033248729163545e-02 1.578521370901959930e-02 1.923630156671447822e-02 2.052542164619894360e-02 3.081863417054001714e-02 1.124655139199401063e-02 1.610792230023025406e-02 1.049468380821687655e-02 4.403209715424795390e-02 4.713073168260667467e-03 6.264954856841602193e-02 4.832364840979027076e-01 7.104610438310505760e-01 9.255853273091274591e-01
7.896645613250399126e-03 2.419790647006742068e-02 7.801155972356721424e-02 4.206216877067798371e-02 4.541759303009129667e-02 4.513224417775330011e-02 1.415280069824077780e-01 3.277517433243964939e-02 4.453433048256415228e-02 4.031247180841165956e-02 7.827655483083764687e-02 3.702085792216634058e-02 2.360384117439052326e-02 1.943571315796294770e-03 1.870029266879206717e-02 3.398532222915238987e-02 -1.620720097204559618e-01 1.691571778973354964e-02 2.256154350586987711e-02 1.874639430511352775e-02 5.722779374023034826e-02 2.163960017491799864e-02 3.431836535819728778e-02 3.935290111534827523e-02 1.295879256866843375e-02 4.098137049878222693e-02 2.166290768343586692e-01 4.111727018449403597e-04 1.217814871155022648e-02 1.718872078326165844e-04 5.157131471225331887e-03 3.962709564428836158e-03 -1.306506974858259035e-03 7.779746938709626016e-04 2.990343541247214923e-03 4.509001674123855156e-03 -3.327533098467799766e-02 8.153143697649127553e-03 1.415836470674795323e-02 1.086762135874450724e-02 1.740334533389897506e-02 3.394163819478637073e-02 2.328123571355174282e-02 -1.827768352232586602e-02 8.096436231304925124e-03 2.250152853919138579e-02 4.419450208998985941e-02 5.146719506697370813e-03 2.789187097383779429e-02 1.479893558109197579e-02 -1.233829013123650259e-03 1.052332158177574604e-02 1.403412214913868535e-02 4.470525746110815685e-02 5.295011815007973771e-02 3.820059966986864841e-02 8.085518216525670876e-02 6.894704983087708572e-02 7.568405580457308879e-02 6.322351043181428931e-02 -7.786962831133245611e-02 3.175491868203915435e-02 5.234654721591903981e-02 -2.293589061943303309e-03 8.159198766689983520e-03 -8.119839327750927695e-05 -1.870029958595172521e-08 2.701258575344770742e-03 1.641051760133721907e-02 2.473974424480739412e-02 -3.684439791927341702e-03 3.663825047837866722e-02 7.909552365206980185e-02 2.137694479435643408e-03 -2.936681852421213490e-02 -4.768147689509960596e-02 -2.037748649883262819e-01 1.382846656773840111e-01 2.056495595428725967e-02 1.393070289126372048e-02 -3.571969054040616200e-04 1.729655666473762544e-03 3.478209081438533162e-02 -2.547842765009057653e-02 3.058768725618919888e-03 -8.541144651126563988e-04 3.631717262363397952e-02 1.929803841703784870e-02 1.822695918436654031e-02 -1.467432265737655690e-03 4.911471511359501337e-03 7.676934903746037916e-03 2.112807225635816855e-02 4.170983666885171720e-02 -3.985449155289401196e-04 9.090187548743806911e-03 1.390985475880147405e-02 6.934414571017386659e-04 5.627091575774912854e-03 -2.834634319093197974e-18 3.141106250983718680e-04 1.048323693046676608e-02 1.293109574500875963e-02 1.036030696445756211e-02 4.230906803704966718e-03 1.030030909058870653e-01 2.757127080292705035e-01 -2.141531289757614073e-02 1.429810675918200685e-02 6.103990576147741605e-03 9.293319906497156391e-02 -2.302339385887837431e-02 1.193216347246880693e-01 -1.061070496930402798e-02 1.638738060676586412e-03 -4.836600016121701123e-02 -2.447636793100239211e-02 -2.934862109999183261e-02 -1.363774805461924131e-02 -2.580662441443709251e-02 2.736876368867454876e-02 5.556261018672066510e-02 7.037550433119797721e-02 4.376049752146520672e-02 2.412915929680705188e-02 3.998795565961256460e-02 5.105524592743211632e-02 2.725655286939403232e-03 2.268616262376622200e-02 4.911064721825684869e-03 2.398919457700201749e-02 2.299578874186648456e-03 1.119507223875925231e-02 2.199961840634436228e-01 -1.494772339052082566e-01 -2.027988350414312857e-01 7.759863690509130607e-02 1.431890782357965117e-02 2.294602158463080605e-02 2.375804435235721879e-02 4.416386509172039870e-02 9.402010581212316698e-03 4.484229102731211725e-02 2.409163648787326570e-02 2.836283211508008190e-02 2.591322224506708433e-02 5.775236250525133536e-02 5.803828235950114940e-03 7.091974905355897911e-03 7.673652531108248083e-03 1.803801504114876941e-02 1.285173722618246882e-02 1.176204065677962166e-02 3.019168132548571970e-03 1.105197260247371867e-02 6.378644795571665124e-03 2.487902268942012492e-03 7.767848874753844821e-03 2.169953645221803576e-02 1.292960016929533933e-02 1.999630027656454714e-02 6.394960435960825862e-01 7.097615755070108001e-01 5.570686534669634371e-01 3.826792424445675522e-02 4.239791992785030272e-02 6.472783527731439113e-02 4.245839900563812969e-02 2.190989397852303178e-01 4.819093065882355531e-01 1.504739188174200604e-01 1.205482804671790219e-02 1.951683677334355160e-02 1.864766696211177591e-02 5.023230558956730263e-02 5.725952858055753658e-02 1.284176733506482704e-01 8.318305267149482540e-02 1.021175241055899269e-01 3.677071618440305723e-02
1.606389491750783993e-01 -6.237241472927947811e-02 4.182568426670432715e-02 3.546511362837915377e-02 3.053395246733692919e-02 3.291195713128189371e-02 9.936165234927757384e-02 2.153381067086073547e-02 3.576978313456929959e-02 5.041734789579643133e-02 5.318047342046108156e-02 4.637776579680169542e-02 2.790093902577676227e-02 3.486760349007589355e-02 7.199232578316040743e-02 5.896371415041189862e-02 1.629353766073477761e-01 -7.609091815808825821e-02 -7.844588265249767892e-03 -1.317355579286779710e-01 1.679437185182855730e-01 6.561030859815013272e-02 7.182496957592734987e-02 1.884470328288030683e-02 1.627114695172727013e-02 4.704381896805788499e-02 1.172529936916835019e-01 1.775901302819714311e-01 4.556543505501618047e-02 3.318569005376031944e-02 4.392642262552503424e-02 7.693734475059678390e-02 -8.616594346729104835e-04 1.063002001133655887e-02 8.022238056738028125e-05 1.745191879277820362e-03 -4.788991024750445524e-02 1.932292486767545926e-03 1.628652760726023024e-02 8.194259317976943999e-03 2.184957437496064594e-02 2.200167898340239792e-02 2.557999342394467329e-02 6.745294953855088434e-03 1.823547125719173628e-02 3.333226701034259359e-02 9.341571116561213362e-02 1.858758135346523857e-02 1.711317139421404676e-02 3.071935469575404545e-02 3.176206517920795308e-02 2.194736657779673694e-02 1.862901051079140710e-02 8.687044801322622509e-03 2.729769868354246787e-02 1.317234058004060696e-02 5.375872871267450837e-02 5.561445550841761891e-03 4.780774999816388843e-02 5.462291038605320698e-02 -3.978534157975729957e-02 -1.046933800332048056e-17 1.932499051402349899e-01 -6.109337969814539193e-02 1.030113764108578876e-02 -2.455290179433729930e-02 5.333746359561301192e-02 -1.193045446391318570e-02 4.840584787901303782e-02 1.226455558722087773e-02 1.146840145800871388e-02 2.002771678011644636e-03 1.137851561300682879e-01 1.334321187124249894e-02 8.976468539785429496e-03 4.247852399307827653e-02 -2.035992589051257573e-01 1.620230232670739345e-01 1.825778778451347395e-01 1.934364291870918906e-01 1.365381502282535833e-05 -9.832924344114161042e-04 8.165474890855492496e-02 -2.573479701787358132e-02 1.892413213448069717e-02 -1.933750853276169634e-01 -9.983566068937859684e-16 3.268079442377782445e-02 3.114348337002034034e-02 1.199064632630506000e-02 7.867158288893176532e-02 2.444091087667544723e-02 3.896498446867277532e-02 2.413989537811603997e-02 4.351464237960615128e-02 -3.798870809414382765e-02 3.148049807313179488e-02 -1.429927925315147876e-02 -2.166419810306908361e-02 -2.383163819659167570e-02 3.763780131580243653e-03 -5.894924744611683247e-03 2.783929173601580598e-02 1.614413353341149007e-02 1.994115531568810967e-02 -9.908712242924262809e-11 -1.373268435590614981e-03 1.232030110801675404e-02 2.383989951422804676e-03 -1.426002543163198445e-04 -2.202157372154576720e-17 2.311418986103073309e-02 6.428623154936512901e-04 1.583124442887215155e-01 2.395743260685090706e-02 -2.752318831597995799e-02 -7.643721132315088596e-03 7.613266921058718661e-02 1.044521036951195914e-01 -1.108821439460499873e-02 5.729319969647773680e-01 3.251055313219733955e-01 1.730910382275673010e-02 8.537812732193217347e-02 2.419676903507685800e-02 9.075013930871346501e-02 4.758180111736289020e-02 1.198630709236812517e-02 1.051641761484701371e-02 3.306607696631897603e-02 3.912504413217201410e-02 1.961950090264610869e-02 1.405618289115354198e-02 3.202760667677810591e-02 2.420095237522575451e-02 7.973836131342200428e-02 4.059853932280813732e-02 5.552462302567180430e-01 5.680908438405789518e-01 8.640263115301523023e-01 3.010069257518639338e-02 1.088742675991120207e-01 7.593347385405518901e-02 7.141783344529950561e-02 5.834657955867036255e-02 6.297327046672494821e-02 5.338652608229586816e-02 2.074356780292197465e-03 1.301140814333012936e-02 1.897297057895094841e-02 4.742157191506527480e-01 8.541647737297095588e-01 5.263983911918168612e-01 2.566621711790480556e-03 1.522687119788778375e-02 1.017282792502079798e-03 3.606846014427451778e-03 4.144295994569053079e-03 2.006628328287715993e-02 1.355652660765913585e-02 4.890417837125292377e-02 2.866797750877772208e-02 3.653370471800498898e-02 2.783380723840025908e-03 2.696272698456867248e-02 3.271512969078533423e-02 5.405774768728383722e-02 1.010473567831417056e-02 1.430937695640309079e-02 1.130802661596085751e-02 2.836811199113413798e-02 1.202384832785572018e-02 1.900277632660802782e-02 2.882154697191365023e-02 3.896224986351029540e-02 1.210642534047125414e-02 8.206823723980676000e-02 4.906889712211943144e-01 6.975623787302407397e-01 9.489784672603247540e-01
2.023049427421986790e-01 -5.542830807501762658e-02 5.947991806644851304e-02 5.707313165982784431e-02 5.964487217019891901e-02 3.571908528832226648e-02 9.336046652473380114e-02 2.120007238915575659e-02 3.160462164339052776e-02 4.549560011742434518e-02 7.083764890317027774e-02 1.579794980433078810e-02 1.117193624793973508e-02 2.586566529223814637e-02 6.611595588879869145e-02 1.078943110263489713e-01 3.612565695114740572e-01 1.182827848404543053e-01 7.749349653813025762e-02 8.043696676780397181e-02 1.535912682891840686e-01 1.053443853082575987e-01 5.059053849446984441e-02 7.060569100125814457e-02 5.474325909689725655e-02 6.226240936860252123e-02 -2.993758705941419740e-02 -5.076778151117686960e-02 -1.138360958797923900e-01 2.904132931341033702e-02 1.598938989204263042e-02 2.213800917243958361e-02 6.819162820770266129e-03 2.668955438192269561e-03 -2.574924311264115140e-01 -1.270097570029834200e-01 -2.012148279864773431e-02 4.834044425117012961e-02 4.816282477696113423e-02 9.659745339934749930e-02 4.873802772911341175e-02 6.796507140073172748e-02 3.210275813042745563e-02 1.551216738738615472e-02 5.046247275302880725e-02 8.270378025049042459e-02 1.167171711180440274e-01 2.380695706901104700e-02 6.018618182170289155e-02 3.231233764359141974e-02 4.358237090072055592e-02 1.422062948252011993e-04 1.410898613910246610e-02 1.532258473011047055e-02 4.179018703085422143e-02 1.244582186268677199e-01 5.763544984346904293e-02 4.134343572892707624e-02 1.722609311178949776e-01 4.305972077541524168e-02 -5.294604244113830516e-02 5.922079196557087678e-03 3.951671989951741591e-03 -1.571182642497169143e-02 2.818101208889132320e-02 -7.917958021021227408e-03 -1.300733279024983121e-03 1.906640282727333760e-03 5.651024910714546462e-02 2.318119426157569038e-02 -1.901236839859583392e-18 5.322173021675837695e-03 7.159005563162654934e-03 -1.094583408135649943e-02 3.071842175449455634e-02 3.473761527307753227e-02 -1.819342162072214858e-02 3.544205433102038533e-02 -4.494280789933294273e-02 -5.299140074032462205e-02 -1.333769523448051237e-03 1.046979691242546154e-02 -6.359311693040380596e-03 -2.011470663098821957e-02 -4.099676246954348517e-18 -7.058161844759215783e-04 1.035296404936055176e-03 4.589246087504134965e-03 4.537343566340792077e-03 -1.066437369799727002e-06 3.541306830227619329e-18 -8.339463134400006833e-03 2.229246379278330623e-02 4.070424612268067066e-02 1.667137413284064482e-03 6.824958313586701692e-05 6.772782091916219882e-02 -2.590345642685102344e-18 5.221755933942117886e-03 4.657947319956413229e-02 1.510128078276222214e-03 3.989001936658029962e-02 2.659535742736190631e-02 -2.032485409625719564e-18 1.919355852997544840e-02 7.118799114239175718e-03 2.785741953422424602e-02 5.345453997593534179e-02 9.952624965722106642e-06 -1.812215382396805893e-03 1.044632501221432752e-01 -2.889317468334352476e-02 -2.743734347497094744e-03 -1.298985333026846968e-02 3.592855460060483813e-02 -2.398797813772989354e-02 -4.265656077075679221e-04 7.894943071209341995e-03 3.863065915225442210e-02 2.598891378294860589e-03 5.627053753765965949e-01 3.654462124047890836e-01 5.586463595925525438e-02 7.481138850995337763e-02 2.428978127634708051e-02 4.870719507103047491e-02 6.042910247938083901e-02 1.830048316848422937e-02 3.939577734750870919e-02 4.057834831564271355e-02 5.132721471399445679e-02 2.905789937872785972e-02 2.709059724405086036e-02 2.333499744422347019e-02 4.210079324033731002e-02 1.039753691754121967e-01 1.303953084093179005e-01 5.224391409115056867e-02 2.983867814115489028e-02 1.575013932982930950e-02 2.911673841750306649e-02 1.897182126812920052e-03 4.789794311681298822e-02 2.582306892541473900e-02 2.168581799109022337e-02 2.518534780124022029e-02 7.411476995594514783e-01 1.241833025168002980e+00 4.339328096928976586e-01 -9.047787104804090597e-02 3.068652454696917456e-02 9.316184552634124150e-03 5.498112285429563109e-03 -6.703858097029792573e-02 3.820264429670189310e-01 4.061709240575763769e-01 5.264986457160621969e-01 1.124371230074287797e-02 1.998561585895134926e-02 2.393086738120365348e-02 2.780486834139404492e-02 4.536703692276823630e-02 6.684912698052715430e-02 1.763320994071180051e-02 4.432606142194384091e-02 3.805478770686238077e-02 4.425864832582199898e-02 4.122006612610670251e-01 4.482084547773226846e-01 3.173653631667640584e-01 4.011436720828020358e-02 1.840629112878038043e-02 1.261160698867999468e-02 3.181476311451714167e-02 5.374519887170112892e-02 4.287007759735734364e-02 1.178084392951748161e-01 7.982787425742653153e-02 9.528942954308507185e-02 9.064420096643448221e-02
1.081130093969672606e-01 -6.310172285511816970e-02 5.926235575358228258e-02 3.489369686540771953e-02 4.083721454409997009e-02 3.127951196726823369e-02 1.019422698955859408e-01 1.767080581939335648e-02 5.471540787189427962e-02 3.922802841159166859e-02 7.999095377073858759e-02 3.987151828479906096e-02 1.581688388480682189e-02 4.728509261349773951e-02 3.652766751679219215e-02 4.859794481396390725e-02 1.490909149425382885e-01 -6.879534713604508467e-02 -6.005049270510536864e-03 -1.576714518470531956e-01 6.360195608471484914e-03 1.417364631455955877e-02 2.487307686152297539e-02 1.599946020880590666e-02 2.247194591434704958e-02 4.299938619230502662e-02 9.048297321335936616e-02 2.069893534510713548e-02 5.903727343590748841e-03 1.867421854302468579e-03 8.381072629786068609e-03 8.416867299990476656e-03 7.869953654552293070e-04 1.012997198110871487e-02 7.874679853883287742e-03 1.578681909258355248e-02 6.761284675101003096e-03 1.075205910050567280e-02 1.854492225564064456e-01 -2.227848266370844113e-02 6.017161947268337592e-03 1.146847877364698225e-02 2.045253193849963166e-02 1.629164361955123819e-02 2.502012131680643806e-02 3.370465378158307712e-02 5.186819339019250324e-02 8.048603024661154723e-03 9.817904889931271598e-03 1.258517071233743352e-02 4.660326852992515562e-02 8.654885557293993253e-03 1.798361537647927624e-02 -4.030091611724643565e-03 1.355624614828093713e-05 -2.373519630379644293e-02 1.045223525293310050e-02 -3.759420725230720739e-02 9.084909828161213605e-03 -5.511579672127493029e-02 -4.873878107347075217e-02 -4.306864520355231558e-18 1.810543701752725110e-01 -4.418842958557881589e-02 -4.302915248843798036e-03 -1.015531241311260326e-03 -1.659575460526753227e-04 4.015418429148708197e-04 -8.261642272110716335e-18 7.310737922219109897e-02 -3.577133475371947779e-04 2.414202520999501542e-02 4.423214172666484895e-02 6.526601861362196777e-03 5.066270220275103167e-03 5.047603797755672445e-02 -4.875865499746015463e-02 1.815609547307698302e-01 2.486436834968998311e-01 2.087181544953876644e-01 7.082763609176202270e-03 4.886653032315994849e-02 7.346085219640228342e-03 -1.248084480042973080e-02 3.250495871602907851e-03 2.953801597429364916e-03 -7.153946115787310732e-04 1.932762096040293448e-02 1.461839525075865533e-03 2.094255002965446366e-02 -5.704403212005897811e-02 2.862118489901274015e-02 4.101701916584123142e-02 2.092757591937285597e-02 -1.421646057188616840e-03 2.336134202962450793e-02 -2.362857412612732327e-02 1.203719218365061723e-03 1.261178290507858346e-02 1.891053996685975597e-02 5.168133423718302734e-03 -2.952361669801429733e-03 1.324419243293021610e-02 -6.240046011410230360e-03 -6.797130116612595634e-03 -5.185762820585050988e-17 -4.370679762743814867e-03 2.242852205704471899e-02 -8.803551651295508582e-04 -3.340449038800722507e-18 -5.811027213101788158e-02 1.698144122361297872e-02 -1.585907943898148126e-02 -5.475667370993841584e-03 -7.462927022079761039e-03 -1.932245529387767793e-02 -8.486631303093281479e-18 -4.531219916573916290e-02 4.339304818182652262e-02 1.925390063844262950e-02 6.246074723513214133e-01 3.668406001838563912e-01 1.845357092754432507e-02 5.911010634036700101e-02 2.348700029412740045e-02 1.095006654005043212e-01 6.717916505625616541e-02 3.802120271036961430e-02 2.050316921287332825e-02 3.689290547708166945e-02 4.066179735891619490e-02 2.547094671380632433e-02 2.272102750160748252e-02 2.511191377092003921e-02 2.695228251906430297e-02 6.882179980350446036e-02 4.529350621788381936e-02 3.809642184516494612e-01 5.980205813951668947e-01 9.762549237444055894e-01 2.156188648155468041e-02 6.732171383222404666e-03 3.867203432068685304e-02 3.669038507538864569e-02 3.304834594546470633e-02 4.729447307451241367e-02 1.046719425549968302e-01 1.359302069274727874e-02 1.896449933226780626e-02 2.992142817778408945e-03 1.161144138549842324e-02 4.970880700252258526e-03 1.507660068432012043e-02 1.133545500598913403e-02 1.733646430290382637e-02 4.563643394520244638e-03 5.886081683289271123e-03 6.556799729347245753e-01 8.144775307322515978e-01 8.594123042157054693e-01 4.139753850610111002e-02 3.031777613699883009e-02 4.456968762258695094e-02 1.826823760899011495e-02 4.385279639957150727e-02 5.665568388660393445e-02 5.930416943263488311e-02 4.853001213778568301e-02 1.788472500278525182e-02 1.393649863914421354e-02 2.034550482895512019e-02 1.914246446042868963e-02 9.548781832994164206e-03 8.002007192710614003e-01 3.113756300506106411e-01 7.270693493544568842e-01 1.543727971214848282e-01 2.967064807138871663e-01 1.555690194961826534e-01 1.371014960675774885e-01
1.622852370779125364e-01 -3.988316856291469759e-02 7.476798736257038447e-02 5.118048825081032704e-02 2.258501513063478419e-02 9.934335471480227123e-03 3.310544514441991770e-02 7.720044243268834427e-17 3.868405516241522207e-02 3.332640676846512079e-02 7.313305878695780526e-02 4.132413593198799168e-02 1.065717717168067173e-02 1.382685502675264945e-02 2.534435685259909046e-02 3.524829457965942409e-02 -2.654976399900084716e-01 -2.242981469714094567e-01 -2.072568614252412733e-01 2.464432439337882821e-02 4.023345699249852164e-02 2.830636242548217191e-02 3.110404726710078518e-02 2.744546209351714922e-02 2.467417019573616907e-02 3.423906626674488168e-02 1.008881061318620681e-01 1.010045865554027797e-02 1.851445686171281427e-04 3.200531148987825064e-03 1.027095986599080649e-02 1.898950144151083722e-02 2.197148437390413281e-03 6.185248572623996250e-15 9.452354645588714049e-03 7.315719139117497077e-03 -3.798619715209386460e-02 9.937942500925774775e-03 1.351483500604527127e-02 1.010000457834372110e-02 8.192997755231972920e-03 3.486333635045829604e-02 2.654623089478600970e-02 4.148747147717195272e-02 7.544016892121090112e-02 7.858886076365630136e-02 9.895507954908350223e-02 3.200371026518863266e-02 2.065302681809723026e-02 1.982077091661568063e-02 -4.749521095235641893e-03 1.702981779835852083e-02 -1.744509945196068626e-02 1.313971402068265515e-01 1.454570292703399326e-01 2.001272596375468782e-01 1.653105971823948528e-01 2.272308496790794996e-01 2.901036989864635429e-01 6.001421648674987175e-02 -1.348394363775138605e-01 5.648197660750107346e-02 3.537547433882792802e-02 2.206308895782176441e-02 2.060840028521035427e-02 -1.358341791846704935e-02 3.402047857722048283e-04 6.753663178009307842e-03 8.622091287157714057e-02 -3.313299875065745664e-02 6.776929582888779890e-02 4.040312286862406815e-02 1.430772216333514590e-02 -1.568073306114309284e-02 -5.271909396754293625e-02 1.079055446764603009e-05 -3.797535839668573301e-18 -6.376890539495586041e-02 -5.532607798510073838e-02 6.130642791985050494e-02 -2.290582299570361372e-17 2.122236279155972558e-02 5.436949631755376433e-02 -2.476012304834120711e-03 1.589491269351430910e-02 -4.727125150036735196e-04 -4.838058050122256875e-17 2.940855335983365727e-03 4.755755833400870242e-02 -8.406799472204376681e-18 2.790683389760190126e-04 2.983531987152705442e-02 1.434741933986523159e-02 9.377095163709926046e-03 -7.326791695179035897e-04 7.327738698358335877e-03 6.372870127977534049e-02 -2.945668145223805101e-02 3.218575350734918100e-03 -4.108557928107075554e-04 5.405937917312836349e-04 1.814443442120982838e-02 1.435891356518607248e-02 5.604429845441849523e-02 4.742875379948007647e-03 1.334465162704467356e-01 -2.559855162532414033e-02 1.470999702664805772e-02 -3.151283602267054753e-03 4.213980771371417244e-02 -1.839305538920140912e-02 -4.418492755293746375e-03 -1.506727331087149039e-03 -1.775228323744085582e-02 -3.936595189149370150e-02 -1.765969439824249115e-02 -2.044559788172422080e-02 2.464990233671146037e-03 5.870519839238726861e-02 -2.306535825636582202e-02 6.311370965205951311e-01 4.105852525854881141e-01 1.486361782276768469e-02 1.548867890226946972e-02 1.901516129060462634e-02 4.540905995808838108e-02 7.146287735197981439e-02 8.562705888337887941e-03 2.501312629007731991e-02 4.163713287127750184e-02 3.555272271515177995e-02 1.496750424922258481e-02 1.675287081261683797e-02 8.367064554759192155e-03 3.211195616409030090e-02 -2.254763797043330548e-02 4.211266419208033462e-01 9.914433657535410838e-01 4.200143187951688728e-01 2.465199591390376723e-03 4.161893510442911392e-02 1.867090581298127411e-02 4.702616884252473761e-02 4.552164565183289963e-02 3.727534073150168964e-02 2.190123420572439791e-02 6.248189082329014266e-02 1.545458608594301592e-02 7.961768940810673434e-03 2.974105230578744791e-03 1.886678263563104294e-02 3.643562450626679130e-03 1.278929971625867267e-02 5.864545053813473188e-05 1.449266156322521343e-02 3.492047483646655136e-03 3.285292241109995237e-03 -8.744237298343534096e-03 1.174848229890240736e-02 1.305679621118011231e-02 1.639319009055131490e-02 6.177354346414348774e-01 7.048623531948148280e-01 5.989746596699889736e-01 3.043403572244914729e-02 3.228530191835903701e-02 8.161031542115405879e-02 7.685761409931568533e-02 2.879059156247026560e-02 2.573319113911943373e-02 1.006860245161231293e-01 3.888784875599801683e-01 3.553431923792259584e-01 2.163131243249850899e-02 2.475248755248521393e-02 4.210116941938884388e-02 9.124221476915148066e-02 2.770221616139971130e-02 4.256382400609817068e-02 4.242463174797971720e-02
1.457704717026865604e-01 -4.317380201709786897e-02 8.572594341519187977e-02 3.301532477323104037e-02 4.302379531042533484e-02 8.022936431048946493e-02 7.677539030819853505e-02 1.402572664210226423e-02 4.995079126311648826e-02 4.112429308422168445e-02 8.813213232086940241e-02 3.795556259917411135e-02 2.171027379713904096e-02 2.929419906679769925e-02 5.949351805259994047e-02 7.010766655884224929e-02 1.206150080392549145e-01 4.984318407168707166e-02 -1.079223628709225988e-02 -5.199480267131301875e-02 -2.293448503533357039e-02 7.843123520384139014e-03 3.172234265958370075e-02 1.588991210374597965e-02 2.828983107199203526e-02 4.220926586111715950e-02 1.339031849869288249e-01 1.963282172663531264e-02 1.442642329956956605e-02 2.154406736531554699e-03 1.902611766625611142e-02 1.563751103125904721e-02 5.886718360900062858e-04 2.620019641634485467e-03 1.453476200757540353e-03 4.339769343824446068e-04 -3.971772158723678448e-02 6.793147788604368470e-04 1.250225121631880904e-02 4.392430367131659394e-03 1.640181982944802913e-02 3.412872607337119613e-02 5.908886866383061820e-02 3.248453647618296941e-02 7.195748701530622904e-02 1.364893349889289842e-01 3.664152249735981581e-02 5.774980309232310982e-02 5.616466120755304881e-02 2.225279070455138050e-02 -5.586250855780154709e-04 1.670595830676091422e-04 2.156656017841004566e-02 3.132466572041366082e-02 5.774515085012148219e-02 1.810394572788660561e-01 1.480094828377984950e-01 1.874517515376942500e-01 2.313317008698790589e-01 2.587408142688239487e-01 -7.448548547707797840e-02 -5.612340827730374939e-18 7.980947985961818480e-02 -2.405459998269210697e-01 8.876374037677514780e-04 1.207396096667428821e-05 -3.102396605546523401e-03 3.008531516551077659e-02 2.292519755981972662e-02 4.668679545936605182e-03 -6.053901948247727476e-18 2.847828825369445871e-02 1.832310004511811252e-02 6.664716186409081096e-04 1.568630747010965262e-02 1.485793506605878091e-02 -1.127850070915283921e-02 7.227876714532761350e-04 8.048050157031586254e-04 1.518932280851769756e-03 -6.520492026940764851e-02 -3.125502648078252142e-03 5.798973772910357566e-02 -3.443922376040811097e-05 7.799797104883710031e-02 2.032702560639474462e-03 9.690430315150962495e-03 1.394351905683411351e-02 5.150772076628306467e-02 2.510062979472367766e-03 -1.617921343223514830e-02 9.243111327377716985e-03 -1.590968583291803989e-02 1.935318920863839803e-02 -1.207150232482349738e-04 -7.236109265793836710e-03 3.474803835892072146e-02 1.297862621303690132e-04 1.108112938688070488e-02 -3.935669783372520544e-18 3.160705949750893294e-18 3.691459957188961832e-02 2.688579081652695685e-02 4.558609945094434235e-02 5.792138312577726350e-02 -1.042848516468765363e-02 2.163129095511980016e-04 4.562160597047633381e-02 -6.717248614383229677e-02 2.449368625120640700e-02 2.090277368685780450e-03 -1.137855655202140055e-02 1.118785366965468764e-01 -2.638904889941511185e-03 -3.117679090857798999e-02 -3.755629929172885073e-02 -7.062267488064133706e-04 -8.505152016092712430e-03 -2.088248343611710106e-02 5.532894107450329890e-02 6.552286739805980309e-01 4.184713369890698531e-01 4.954203230481485365e-02 1.732967175478236990e-02 2.334410869586678317e-02 5.257479579999344010e-02 7.354050892700809972e-02 8.453992173287901840e-03 2.048227583833643617e-02 1.673108277211465475e-02 3.476160988167024263e-02 1.648282662447074498e-02 2.078203763766395609e-02 1.259446189581531553e-02 3.640995385989205524e-02 1.447320227741972533e-02 5.424109051012097893e-02 4.762577371701877220e-03 4.234705181351625658e-01 8.561417465962946061e-01 6.178484731187905421e-01 5.732758755140987192e-03 3.631028815779723667e-02 1.962680071463480599e-02 3.433295882264296439e-02 5.098299404989128231e-02 5.898210892992164500e-02 8.764008105768130580e-03 1.912764068392874134e-02 1.785521843607773031e-03 3.441703719292856650e-02 3.291032500854871624e-03 5.018173891835613944e-03 1.569777946275485551e-03 1.217409344125060418e-02 4.027222669885147793e-03 5.306112583511529007e-03 1.120390347230135458e-03 2.184140337053582653e-02 1.509909353690593904e-02 2.238214499760489071e-02 6.288891332684922730e-01 7.433574875232888957e-01 6.225331178109920760e-01 2.557011437449099173e-02 1.803903270011256860e-02 8.065889517842012024e-02 2.250862319692618907e-03 1.459781128940702485e-02 2.261309196055189263e-01 1.560090809906373999e-01 2.942632019137865207e-01 5.714809679596608537e-03 1.385327929721380177e-03 4.715700886856121626e-02 1.452645426155370172e-02 8.268056001038051606e-02 1.186139973345778573e-02 3.979673757763263486e-02 5.690446707851731702e-04
1.282741061460520027e-01 -6.211337705703480483e-02 6.868444053041178332e-02 3.529550258981799249e-02 1.747914979638555757e-02 2.793825158537478759e-02 7.187082536575947755e-02 2.660980126997483566e-03 3.833969854811181827e-02 2.579132169158171103e-02 6.933038076639405634e-02 1.511169266538402445e-01 2.320918312113306320e-02 2.222630521488088567e-02 2.874633650934917475e-02 8.329942233896112103e-02 1.381493349844785112e-01 4.449607483056036705e-02 4.620635275769071487e-02 3.527410599915564021e-02 1.538544481205570691e-01 4.255255603991598551e-02 2.797193941091970573e-02 3.934599766521457848e-02 2.825114668153705030e-02 5.283278488605443884e-02 1.161184361901605872e-01 -1.545220136796893184e-01 -4.987142810020753908e-02 -8.897937952497290970e-02 2.224370608252385084e-02 9.838072495076866444e-03 7.477703521717094697e-04 3.295566839678315442e-17 5.381446119510317240e-03 2.929331418804918884e-04 -3.200484488027820207e-02 3.029211438027628048e-17 3.972724323243167325e-17 9.281008896880483389e-04 1.061820093628403246e-02 1.810864679269284100e-02 -1.795084743349335821e-03 -1.064456455552434400e-02 -1.796404873727612539e-02 4.525746920780872318e-02 5.572552925874330043e-02 8.369479755230472759e-03 1.245940234272274391e-02 1.400902821439961761e-02 4.441920041356230142e-02 1.335014478330773767e-02 2.153153080187822713e-02 1.343494929988072774e-02 2.328543280873721782e-02 -2.136557848496675348e-17 -2.945805700124494095e-02 -3.210546750307803621e-02 8.464411260221095246e-01 1.196919735097281001e-01 -5.447344752347044183e-02 6.001246591233481281e-02 -7.139834088527067905e-03 -1.834591275894034701e-02 2.196600037036217437e-02 -3.969743057257248131e-02 -4.118904770440892503e-09 9.103544730215616665e-02 7.107919415266825136e-02 -7.559104776327210500e-03 -1.192115338373081479e-02 1.695774414700560936e-02 3.213291788467915328e-02 2.611929631422050743e-02 1.662785238945201777e-02 3.006107553750629949e-02 -1.651362181578290220e-02 3.433195384129871658e-02 -1.415843448027494234e-02 3.710837851947144483e-04 -2.772836770307908808e-04 1.217799720924506710e-04 2.195546167925523096e-02 5.459058099794614205e-02 1.577886864887280763e-02 -2.644673126607501121e-03 -3.744615720723443349e-03 1.856987821741616987e-02 2.924038669822571872e-03 -4.483851599524945469e-02 -5.271259807225437896e-04 1.065205695617880144e-01 2.380127536871934693e-02 5.500201798643476574e-03 3.560100765509281728e-02 6.589126007708838439e-02 -1.975517425366580485e-04 4.230119145916369332e-03 3.680789014185714518e-03 -5.722809462161169503e-04 2.113130068417433394e-02 2.476186251372060679e-02 -2.139026748875068329e-03 -1.531013102951597933e-02 4.687965789339692407e-02 1.696390392702426184e-02 -2.111974198106788755e-03 1.967167893676903187e-02 -1.414169873078768841e-16 4.002425133650741942e-03 -1.046385929099704487e-02 -3.601295273118184509e-02 8.699446642159840495e-03 -3.047615097234005422e-06 1.428934199328486289e-02 4.526048045982607182e-02 7.806483706961893886e-03 2.673220588594636682e-02 3.188097165253550407e-03 2.231527722203450192e-01 6.418513395257512411e-01 3.196866849495236251e-01 5.368611175278161646e-02 7.142787087913195221e-02 2.205239419258239048e-02 7.396199215846464836e-02 6.262118001569953740e-02 5.177516180622545629e-03 2.977670619247766229e-02 2.732380434006657921e-02 2.646772450172845195e-02 1.642996671695268890e-02 3.157440349250049960e-02 8.134916768889985383e-03 2.476263640674376820e-02 5.061872276579273272e-02 7.164514622444018155e-02 9.491399414734943626e-03 2.489289306997109763e-02 3.742494466113382989e-02 6.014489626193084137e-02 4.261743649270149231e-02 5.294062194294725521e-02 3.943101300135780363e-02 4.879019630810652736e-02 2.998041754584399379e-02 2.791783168149031616e-02 5.598127154911178849e-01 5.174777292576586696e-01 5.071244888981393251e-01 2.474068074525670805e-02 1.154862747928959293e-02 2.112954778217841084e-02 1.293585356564184302e-02 2.105809613445278061e-02 4.850233837678347495e-03 3.487067349920172209e-03 1.402798725107612388e-02 2.451063988928625648e-02 1.406049647177333960e-02 2.902213125867530649e-02 4.786006197280644713e-03 5.663438527079630580e-01 7.720538565156914723e-01 4.712944057866383374e-01 1.021030545089762938e-02 2.731673286201147938e-02 1.931623943074058450e-02 1.453177501694442701e-02 1.225812866463987620e-02 3.534424291864794038e-02 1.952044511405933878e-03 1.721947490530528410e-02 9.653232871326525993e-03 3.811663780566400261e-02 6.567259533325292553e-01 2.609693998325618991e-01 9.389221680295622985e-01 -3.946916648481404444e-01 -7.527787913300763023e-01
1.264953572731350451e-02 1.772736739302324324e-02 8.241707343816713494e-02 4.122086377049071632e-02 4.178136747377829419e-02 3.904667401413847161e-02 5.431382922523114720e-02 4.436669552575724790e-02 2.156194365921317371e-02 7.630043548716967938e-02 6.571331409554460878e-02 1.080549744597559744e-02 2.476413078415484401e-02 7.596431175149598444e-04 3.693246547787558398e-02 4.192377990091233214e-02 -6.935444815422978082e-02 9.254916024776719055e-03 2.714427639333177270e-03 2.763765958032111253e-02 1.406452921946185763e-01 4.885820959486086856e-02 4.683601426587549049e-02 4.272370419378922202e-02 3.987007450882860959e-02 5.379378134604514278e-02 1.635451007112145583e-01 5.350856622341317037e-02 2.495373112607669752e-02 2.094590109665539490e-02 2.388619505770709500e-02 1.290386636929332503e-02 1.662578729488463358e-01 1.686670903438031513e-02 2.803694181197923657e-02 6.461724118991919903e-03 9.729602850950683859e-13 5.514336888606996469e-04 2.155756800731723627e-02 1.411282847999768310e-02 1.388114014821858686e-02 9.716577792040458073e-03 8.801070078296472851e-03 8.160276449651200983e-03 1.721938981742517777e-02 2.388155910889601385e-02 4.068800319965292650e-02 6.820763624468539899e-03 1.052025446187807717e-02 5.828474347344230790e-03 -2.448951229272464721e-04 9.444477993642539964e-18 2.124457220193821591e-02 3.113779093305765397e-02 8.157346994493716585e-02 1.289521746726174367e-01 1.312027730035019712e-01 6.621227859024264584e-02 1.082400947115610412e-01 1.376451995929245109e-01 -1.999416339149380842e-02 -4.372420074677329876e-04 1.713343015334914052e-02 -5.083751594057435257e-03 2.383906791775327119e-02 -4.376773848031537822e-03 -2.234663449329347249e-05 -5.450732016797063145e-07 -2.162938836731043422e-17 1.599640842955763267e-02 -1.543096220535853866e-02 9.888417227426435363e-03 7.918526404242105798e-02 5.758108049067619605e-04 -1.925257379431838736e-02 -1.249709568697305109e-02 -4.767895940356361301e-02 1.422189291200989976e-02 -9.693138910741855426e-04 2.552263079067277826e-03 5.116429166148526997e-02 5.382191355505339505e-04 1.581692791474805654e-02 -5.567046499563885892e-02 1.598373501628663068e-02 -3.345840441001057125e-02 -7.066750196549187125e-18 8.942851154644529399e-02 -2.499832696103880509e-17 1.485581751522590567e-02 9.160192655319690166e-03 5.122832670851292652e-02 2.313866479563656507e-03 2.648802081724938723e-02 9.615965569022191101e-03 3.519470464565765001e-03 1.135291974355175787e-02 -2.413734111942163890e-02 1.001050042251519848e-03 -2.163980105136364702e-03 1.634770522705962514e-03 7.159342126696850717e-02 -5.515164884110884063e-18 1.971356520534946200e-02 1.959758272249670016e-02 5.284858785648932633e-02 1.930205916978895922e-17 -2.626594993670330786e-18 -1.508751978081878308e-03 1.060262870850775390e-16 4.094535124427780420e-03 -9.370609660629624152e-03 8.188071026692060606e-02 -1.117690965293357191e-17 4.320834835460896604e-04 -3.382404525005694440e-02 -5.520733227664678666e-03 -2.122790272754291122e-02 -3.548372441122581329e-03 4.650205106368865415e-02 5.450947475235204398e-02 6.240424146237395775e-02 4.993791292667494608e-02 3.872693079401773320e-02 2.297687536418099169e-02 9.660827594609271152e-02 4.801799083885332359e-02 1.852730387642488336e-03 2.008024755238109471e-02 1.562693513632465317e-03 2.924465642132545798e-02 1.381179358059199565e-02 3.074302788582136681e-02 2.033418303613660771e-01 -1.408175868608360082e-01 -1.753388053733360985e-01 1.065937773666724014e-01 2.366054426330375590e-02 1.797040422472541421e-02 2.227831618285828694e-02 3.342597014962512408e-02 2.250510137305932953e-02 4.475419986319970067e-02 1.290936164950585649e-02 2.923626856959343767e-02 4.046154120178020036e-02 5.600696926539081716e-02 1.148508539631943812e-02 8.806642779606426233e-03 7.752274581467983108e-03 2.447805228271737049e-02 8.003700566903024782e-01 7.875103991454137642e-01 7.626825153345676567e-01 1.837847758747859078e-02 3.034454207620728133e-04 4.886151413351378335e-03 7.603018837569592257e-03 2.570172352402945448e-02 2.873766651006849637e-02 4.032242514653899823e-02 3.803803123209218567e-02 3.954977278222007731e-02 5.940504866882649962e-03 4.167904207803278804e-02 7.141210189078348036e-02 1.034717624634832467e-01 4.772502259329469698e-02 1.401166600276577794e-02 2.551522285303304272e-01 1.556055258783204942e-01 2.956258415740002365e-01 4.164388596018842557e-03 -1.434946789233694335e-02 4.391839938140685434e-02 1.779636552435013425e-02 7.162476463539144134e-02 2.904268813467527494e-02 4.935553889840996189e-02 4.074393529454833863e-06
2.405326609724212639e-01 2.083713275971488949e-02 7.696265487819018564e-03 1.840188473667349897e-03 -1.937051340098043875e-02 2.699160621802096285e-03 1.875269608990378463e-03 1.202951315318549113e-03 3.290497732841412420e-02 1.572603601127024395e-02 3.994135122148119443e-02 6.044366676915021233e-03 2.201654484232955233e-02 1.473289531145789352e-02 1.016820359409795066e-01 1.479383875845131403e-02 1.101364091826615745e-02 2.571618486508248091e-02 2.092335114939419816e-02 2.510894739773368925e-02 -2.121974378994259947e-01 -9.940045524788920783e-02 -1.606236097095763704e-01 1.772433051536428245e-02 7.534725679589561464e-02 2.532838595783142266e-15 1.549709377270947104e-02 2.259528375014541873e-02 4.087469896020450716e-02 3.711911125240165837e-02 8.855975981789988849e-02 4.694814133210466200e-02 7.864550792552693625e-02 3.022438132690554932e-02 4.721098249632411925e-02 2.907383372009988629e-02 2.173017487785679908e-02 3.052544724308406399e-02 5.964041272094527252e-02 3.185800595274141528e-02 -3.731044507136199034e-02 -3.853041928986919266e-03 -1.337344833715903986e-02 -1.195071466423528184e-01 1.015884348251193253e-01 1.704860880717098404e-01 -5.151191518983359169e-02 -1.119472070701400639e-02 -1.995287600535368064e-02 4.814032894174782168e-02 5.293429629752373355e-02 1.371663574294086814e-02 3.633855427575729408e-02 1.830311771274129029e-02 2.902362427663679120e-02 5.484869329333406235e-02 8.437715440222227914e-02 6.636280660442619994e-02 1.883085153788949251e-01 2.695417024693115907e-02 -1.562617375133891207e-01 -1.044984508630839292e-04 1.609079621818640046e-02 1.320018877781712341e-01 1.092549404141389541e-01 -7.548700835260570112e-03 2.884455560441863756e-02 -3.870693297305531973e-07 6.135761757360237018e-03 -3.468900525839593184e-07 5.991822741421239373e-17 1.747496247856784174e-02 -8.852737385053162378e-03 -2.645137982040297528e-02 4.275906609595885727e-02 -4.371983860963218221e-04 1.192016721984057864e-02 4.106041115804163512e-03 1.036462160107703284e-02 1.175504915931553651e-03 8.065336086708874662e-04 -1.062614688965451970e-01 -1.239835442159722412e-01 1.549904719246436580e-01 -3.638426617235670565e-18 2.321854757649658221e-02 -1.123207411271270710e-02 -3.352161102618280396e-04 1.213177460454818550e-03 -2.659820248590853697e-02 3.177391050619115392e-02 -9.301894981562905396e-03 -1.882221750303305708e-03 2.238560652915157385e-03 2.804867442216224180e-02 1.413874916588128798e-15 3.057550537598152218e-03 -1.709470616023162776e-02 1.342766739953005131e-03 1.027224797988918470e-01 9.966366388943499743e-03 -1.982732201111981479e-02 -4.471653966313138967e-02 1.017279267585326497e-01 -2.064153138215176278e-02 -2.718278062131211301e-02 -6.610482097119818601e-02 -1.912715734786917671e-02 -3.250677693732536254e-03 7.357409740227327001e-03 -2.521560436403521420e-15 -1.530574694891237257e-02 6.049934918351723007e-02 -1.030847582262148752e-02 2.986957883417418538e-02 -1.873114379938571605e-02 -1.880831835817535968e-03 1.126548390887872311e-04 4.672730286721152315e-02 2.494334642051461437e-03 5.516056267619217079e-01 3.842600210813523209e-01 2.525917554716262947e-02 1.387299101908898136e-04 6.637729625018547533e-03 2.373294895920637776e-03 1.818959084696974635e-02 5.306581216463866248e-03 1.541776405660519149e-02 1.288522477999820906e-02 4.510528869852901968e-02 8.641985383228226745e-04 3.276876331285588639e-02 9.721390300290025377e-03 2.844761317376070381e-02 9.450619654995386434e-03 1.055664622768642119e-02 1.194050679492870970e-02 3.781706220542093572e-02 1.123406028597066439e-02 4.957811194717642866e-01 9.064035515332902193e-01 4.520710677402733357e-01 1.440106503877564177e-02 8.765070810675022750e-02 2.243921620233897685e-02 5.157418059990953596e-02 2.283140400382093968e-02 4.077908094571758985e-02 4.840267093261573644e-02 6.172806368110361763e-02 5.411972054478266653e-02 6.641708238251496643e-02 5.581266907924452292e-02 4.251679738126994124e-02 1.777896534109174934e-02 2.962828745569565450e-02 1.606002304762767230e-02 3.795441252856890380e-02 3.661747373004359263e-02 4.966395676974492690e-01 7.733265028179082679e-01 5.268381950630238730e-01 5.743171063964185308e-01 4.345039594334364486e-01 6.071409927595295963e-01 5.063641402052777574e-01 6.557914591462946641e-01 3.253873190660115378e-01 1.947766412321932233e-02 2.715360906194711973e-02 -4.017863671851772068e-05 9.469275501957072350e-03 1.135013892514267694e-02 4.712809638998471246e-02 2.781027037651241998e-02 9.328520368943823970e-02 8.075867328738065010e-02 7.968431185637422920e-02 3.533430478727272311e-02
1.253365126393956586e-01 -5.014709184917087031e-02 6.106304805931288271e-02 6.141220860741072873e-02 3.147415858042883902e-02 3.629776288936221312e-02 6.271909168270778379e-02 3.084125927702173128e-02 4.457102950144768011e-02 4.255045584276519188e-02 7.150306230320700729e-02 1.035468278815072050e-02 1.611370888578015498e-02 2.750011762586307765e-02 3.631112679252813252e-02 9.112983230688233294e-02 1.597230822643709580e-01 7.296322956618983557e-02 3.768533680154968757e-02 2.859634549655658275e-02 3.879138955522623555e-01 3.987636937592344000e-02 3.538605717387019456e-02 4.803071708899530473e-02 2.197437447848363951e-02 4.066223116635472867e-02 5.693557163737202748e-02 2.626909074586188109e-02 1.192181246459644330e-02 1.195930690240295385e-02 4.911823979749998202e-03 2.735858702992463587e-03 3.715778062830976843e-04 1.398526528034435383e-03 2.657168711585718157e-02 9.917596651095119537e-03 3.282789095214400099e-02 3.837071259899879627e-02 -1.900698476987475486e-03 3.362108498394309614e-02 2.880871898891358349e-02 4.494477725971861987e-02 8.288478024133752453e-02 3.810488546770723906e-02 3.917168303648926875e-02 5.881579172527375693e-02 1.950155249286624481e-01 2.688364171895969018e-02 3.465421194244028280e-02 2.152695799063746723e-02 1.582571464277164608e-02 1.025336313915336910e-02 1.326305563522233721e-01 3.489022679899356216e-02 2.675165500829369825e-02 1.376442597615641916e-01 6.110372876330000952e-02 9.525795164355750633e-02 5.622835694880300855e-02 -3.430544292850155191e-02 -7.157080490446221521e-02 2.996615245852434634e-02 -3.873578789677630532e-17 -5.858378422517747375e-03 2.460837017881795791e-02 -1.090762730814705261e-02 -5.036789515784739536e-15 5.395911370628355894e-03 3.266374330802001430e-02 1.460562470802067962e-02 -9.045982858028668236e-03 2.670569627352594247e-02 9.270385994073319230e-03 2.509219831934396839e-03 1.880017506046637379e-02 4.127375909657287484e-02 -3.149960989486554064e-02 -7.775081726220593736e-03 -9.805377112690732283e-03 1.047225597748152777e-03 -2.928950619713522038e-05 -6.650948670239447552e-03 1.380207337845745265e-02 -4.823839182885875793e-02 2.373883228633709430e-02 4.120552416529175283e-03 1.761209588388661443e-02 -3.887652992234156071e-17 -4.038648702655320798e-03 -1.696116244381906668e-02 4.028188283656536015e-03 2.789234446259289055e-02 8.345008730281912313e-02 1.360540259869962932e-03 -9.610816791749790837e-04 4.091026019782023453e-03 -8.001725322790935804e-03 1.348945610042003029e-02 1.098826445705346211e-01 -4.552307892248814991e-03 5.309356184907436903e-03 9.876266451716136582e-03 4.094137357059084311e-04 -3.123808234210331825e-02 3.256970240637877771e-03 2.261200783821965682e-02 -1.318068743485780684e-03 -3.384638324008663901e-04 -8.754403750348727087e-02 -7.173804647837163080e-05 -9.668550027447285249e-16 4.010699811217780658e-02 3.631029074444048349e-02 2.560585118216500422e-02 7.046742623385304771e-02 -2.212386208843921970e-03 -1.832611176610421432e-03 6.081789746094670018e-02 -1.690125760886647746e-02 5.675312809070640757e-02 6.034443322752965910e-01 3.444664972429427041e-01 5.679259512680976535e-02 3.043229091006356690e-02 1.863512349019859696e-02 5.119034253276058544e-02 7.770173004559911833e-02 1.925219036878858189e-02 1.781475883073999761e-02 1.111552203388429041e-02 3.675869080307125891e-02 3.323511734305269194e-02 2.095886408724646419e-02 8.430704128710324319e-03 3.824769952358618441e-02 6.849381021019375348e-02 1.026993759587645494e-01 4.700230606839405495e-02 2.256115284053623188e-02 2.993251682313569778e-02 6.859658573155660855e-02 5.581057037701152301e-02 5.515498244043835069e-02 1.919703963461428728e-02 1.425099572184855209e-02 6.184915892236965140e-01 8.001650068061155752e-01 5.634988240601720433e-01 1.631828826449695452e-03 -1.183114830002247458e-01 1.953091625604294395e-03 -5.853026052367889254e-02 3.427939363725570135e-03 -5.651475978294428354e-02 5.938716102386263664e-03 -5.028451628593279266e-02 4.425873997752683975e-01 1.003719125306837157e+00 6.279221230017058009e-01 1.983493393119519985e-02 3.086411774912515421e-02 3.435202107492138501e-02 3.944931460058816175e-02 1.714112852868634324e-02 4.785504131494738161e-02 4.289018847612840329e-02 4.424467696424658897e-02 1.898351382969614559e-02 2.313977696643547063e-02 1.883715393308877176e-02 2.832610627387451702e-02 3.679525892334321591e-01 5.312018646984109083e-01 9.512658596478826833e-01 1.227210991552583341e-01 3.102232540067290656e-01 2.286841468587419157e-01 6.114013706784682256e-01 3.496714290361668809e-01 4.753641637098391937e-01
1.245028364867485499e-01 -6.525228776577192413e-02 5.278905803372110905e-02 4.168873539813372497e-02 4.325893428914370786e-02 4.426887558036082110e-02 9.497617994351928439e-02 2.635749977892807780e-02 5.381235602645781380e-02 3.293284836969097296e-02 7.380469832400921026e-02 2.400887323766132467e-02 1.656322296372458869e-02 3.982087850650398164e-02 3.301568709214097036e-02 1.655953517149029441e-02 4.282293819838596527e-02 -6.788502708292958043e-02 5.771870814033583375e-02 -2.806698097535966635e-02 3.237963621166518868e-02 7.034138262902326320e-03 1.676100629531212735e-02 1.108695539653998564e-02 2.294593657022712951e-02 1.258795981050234332e-02 1.016894892650316573e-01 2.439133942047070139e-02 1.160660902729545273e-02 3.188993166150642824e-04 4.362400962794722205e-03 1.627354549592060790e-03 1.094333015037811125e-03 2.446038512830593802e-05 7.016915527036710802e-04 1.619822541083770599e-03 -3.925206440261237023e-02 5.161850811416226689e-03 1.205416137867368169e-02 2.345638765191607883e-02 6.738060191930949142e-03 -8.127538803103183038e-03 -3.977198000930059346e-02 -3.449688302381093835e-02 5.802465096375301890e-02 6.876243049456770562e-02 8.110829809038080929e-02 3.299423516321790817e-02 3.906711619245841621e-02 4.948001848111327483e-02 -2.309572498924859080e-02 -5.718352160480618491e-04 1.732429448330452884e-02 6.738120297775072876e-02 1.385662745342415014e-01 4.486617237875555331e-01 1.424182769948857952e-01 9.940318756920590859e-02 3.265771821015621135e-01 1.900511798744314862e-01 -4.491159375326303493e-02 5.508958711998777858e-04 9.719799669711761195e-03 2.036430943161165671e-02 -6.395615598934783354e-15 -3.972720895099220162e-02 2.836220499476860626e-02 -2.894867748702084346e-06 5.102228322771382163e-02 1.392244395722135042e-01 2.165324510691082460e-03 -1.063176488176408777e-17 1.206840313824544958e-01 3.857760068446308005e-02 1.148222733670424935e-03 4.850558854873204817e-02 -3.722837981769649279e-02 1.304233205629750092e-01 8.455918828199029957e-02 6.928999634327046020e-02 7.724728157047440480e-02 2.810129215672555955e-02 1.199104882240186187e-01 -6.685937542371659960e-18 9.702142536543079468e-03 -7.085344193313728237e-03 1.986995571267489644e-02 6.751063360803477575e-02 9.432590462300236098e-04 -1.250249131190375753e-17 -3.178214068753586977e-03 2.630650641775249959e-02 5.700024626181109284e-03 1.467028577730722025e-02 -9.383733490791874811e-04 -1.501752470341088420e-02 -2.427568338420076424e-03 -2.895044664718952887e-04 7.935632594771256307e-03 -7.748441127234324648e-03 -8.353510935509662547e-03 -3.199259742290553460e-02 -2.848238574611587290e-02 -1.882075420964897730e-02 1.383088626382938113e-01 -8.521378761729911377e-03 -2.580604029380156525e-02 3.958875287824133171e-02 -3.005966999269916659e-02 3.386577936582056692e-02 -8.792012450192959738e-02 -3.743630364241835179e-02 -1.569821879409407511e-02 5.392991617372518750e-04 8.112210770733012859e-14 -3.234925358929355288e-02 -1.264835990112109449e-03 -2.689353465126817153e-02 6.017084573760565769e-02 -2.561266965009810839e-02 6.162227901705995547e-01 3.297744711151132657e-01 3.199956068077517563e-02 6.165591227444980338e-02 2.369480063535668937e-02 1.005266652617606560e-01 5.816657066379004454e-02 1.568826599623615245e-02 1.909109051021540965e-02 2.290527781748144184e-02 3.844533536309829858e-02 1.829433924638101602e-02 1.054200432129827510e-02 1.993060541453687065e-02 2.113593605042453361e-02 1.597928770723726008e-02 4.503646090553755504e-02 7.382276630173166820e-01 6.898580273124346185e-01 6.013105051924637623e-01 1.881851437588799580e-02 6.986828916083051046e-03 4.462265267655881468e-02 4.157426460973214638e-02 2.825766834317988002e-02 5.207988208566088212e-02 6.049209724784926356e-02 2.808032129190270870e-02 2.489001418033084251e-02 5.817751622981257205e-03 2.016185460382448683e-02 1.116037711377159498e-03 1.036452756256226772e-02 2.431466839506342664e-03 1.153544955903756421e-02 3.845194844174635272e-03 4.538982240004908741e-03 8.625352471738293586e-03 1.620571411934903366e-02 1.258055579227510684e-04 5.381794224213874989e-01 1.060595264433820795e+00 8.189559603789359210e-01 1.975408639595278570e-01 8.164090500705915854e-02 4.639943377363144744e-02 5.371552263646128811e-02 7.088662381757124265e-02 4.501312905749121213e-02 6.633173690835028613e-02 2.386309913085790380e-01 4.336250888653807034e-01 3.058545647097659637e-01 1.374798431823637777e-02 2.428971203167322457e-02 5.443091070709017015e-02 1.015515424980148046e-01 2.291740031780070339e-02 3.393787393465773350e-02 4.004490225611301996e-02
7.703468460424682529e-02 6.442959961888145015e-02 1.040207067732111523e-01 3.948229360533082877e-02 4.640267026793550714e-02 8.588267442549311903e-02 1.379854167360984873e-01 1.668494233177644087e-02 4.464853698003402177e-02 5.452163900400153890e-02 1.351922307585095573e-01 7.028388087330630918e-02 -3.276818874448098051e-03 -7.212972233343059938e-04 -9.007656506779463182e-03 1.026103196432595482e-01 1.402967923788464299e-01 1.217942328734216012e-01 1.104778082416218304e-02 5.113748789280957163e-02 1.709422441144499982e-01 5.713108637051401983e-02 8.904064963386452980e-02 8.560425519690055551e-02 4.461312084060686106e-02 3.300687164225137721e-02 5.706789775897053163e-02 8.624015053400803377e-03 3.226684188914747536e-02 1.533193946658424796e-02 2.143717112490199986e-02 3.206331739762497551e-02 5.239719396390435553e-03 1.770577686352367563e-02 5.792895687439089322e-03 6.448016518725552954e-03 9.728373418785442317e-03 1.854211643406963178e-02 6.134088327059947915e-02 5.254833300014275996e-03 2.309316012265449081e-01 2.206014909453193340e-01 3.409229701158835169e-01 5.452104188842153976e-01 3.521469250835239095e-01 3.272254146279071074e-01 4.488961017583649737e-01 3.060565179863899976e-02 3.917325144194085812e-02 2.346632516082484965e-02 3.353218768609565748e-02 3.138247549049894946e-05 1.454550150683205263e-02 1.405204527158779651e-02 3.324427470644977789e-02 9.307298989023837787e-02 8.389651066460980522e-02 5.501097752047816314e-02 1.114057464559954774e-01 4.167099552832314452e-02 -5.839370596571780451e-04 -8.480431683683706083e-04 -2.298190340169928986e-03 8.948089683667831765e-03 -5.729192214605485973e-05 1.160473640351638788e-02 1.093549608429698894e-02 1.378404394915696239e-02 6.033422417030333967e-03 -3.232936477132068720e-02 -5.223062053911525817e-18 7.917218353752818183e-02 -9.068553308820319825e-04 -1.651986723625610901e-02 -2.301035341856805336e-03 3.647304411259857937e-02 -5.340027917945180599e-03 1.562769132731763391e-02 5.839167343873766930e-02 -1.027326733501528894e-02 1.177190364833856709e-02 -3.079427726505630861e-02 1.222873656276993182e-01 -1.185719016556973371e-01 2.659774361157609193e-02 2.044400343916921714e-02 4.784469692330552137e-02 2.857364166413548998e-02 -8.965976928100430343e-03 1.837818256666713321e-02 2.237678289234887666e-02 5.129200414634904961e-03 5.023375212658915540e-02 2.483691178514023726e-02 5.811183249197216655e-02 2.842205985313168287e-02 3.081488045692706845e-02 -1.690928703590833734e-02 3.458830145948921242e-02 9.848351477596990755e-02 -5.891440480041153921e-02 3.417073171949457089e-02 -4.287325163124208055e-02 -2.151069858832615693e-17 -3.671125445053470548e-02 -3.333613995381611711e-02 2.019043067415971990e-02 2.904998968025695461e-01 2.732406092851594540e-02 -5.000378388504646623e-03 4.110182451593628883e-02 -2.401150730852782272e-02 6.759679814947312875e-17 -7.566382865706590410e-02 2.213119101284085136e-02 -3.259064550359096202e-02 -2.356867824249864445e-03 2.160546708365366705e-02 2.525850403826330678e-02 9.093311640210700181e-03 7.041469618308104916e-02 8.343815612295003847e-02 9.982415492225621412e-02 8.464971141443844438e-02 3.284131010743369244e-02 1.127571234728635685e-01 8.033649828953134953e-02 6.320412149495927001e-02 4.240129785862090106e-02 2.393464229450985997e-02 3.066269601231233835e-02 2.107411159355692277e-03 5.125879712143848188e-01 1.071657542529719898e+00 3.926907842731736098e-01 5.803180769682146634e-02 7.851741601832402939e-02 3.272667623250717639e-02 4.519090713829657102e-02 1.028991713758833637e-01 1.205128704594548572e-01 6.029931985574454173e-02 4.549784061550254943e-02 2.293923768651731643e-02 1.185398548936632095e-02 6.285498069997116355e-01 7.198292764252258724e-01 7.079588762058112161e-01 4.060253011508108134e-02 4.519855180556405470e-03 9.359830441267256648e-03 1.592641907827125128e-02 8.775703828713702845e-03 3.764363597527121269e-03 2.016089487481981246e-02 3.965083767880584961e-03 1.583886267222894828e-02 7.072415833029205290e-01 7.824159090155956564e-01 8.874352959990835510e-01 6.740879608713296056e-02 4.603193575605278587e-02 4.517388927944143012e-02 7.183694628934512993e-02 7.618771548246322523e-02 1.593840568975704933e-02 7.365379662957114576e-03 3.184124010058838050e-01 3.405250921506247641e-01 2.311334531424143468e-01 1.509390203609702399e-02 -4.745699270749526022e-04 1.563126363049691450e-02 5.046241109903016470e-02 4.299530275851319583e-02 4.266946193209067428e-02 1.077872785769726227e-01 4.602234493921041752e-02 9.920884370942444075e-02 6.404406510144337872e-02
2.028783461111388631e-02 3.637587548215640704e-02 1.128413544443884198e-01 5.535935164531016750e-02 1.389406309077172164e-01 3.316226659945352429e-02 6.661067428950039060e-02 3.521548777696039551e-02 4.973707014709066038e-02 5.491730180928090094e-02 8.246756386759465651e-02 2.353623154165244979e-02 2.942957638749629790e-02 1.445649412897696481e-02 1.156449702674362950e-01 2.536227347531007370e-02 -7.247271894893057842e-02 3.221334475115549944e-03 1.302363086772938966e-02 2.062418920743020606e-02 1.150882039799032974e-01 1.363064790171192829e-02 1.652784027788435986e-02 1.998088764427210715e-02 2.609439321750104193e-02 3.504468829470707564e-02 1.673636693793227603e-01 8.663888820353736128e-03 1.665059705014761832e-02 5.002854546095296525e-05 9.070837361430075102e-03 8.922478781891523061e-03 2.893788742727509272e-03 7.210793747666133062e-03 3.396144431785355113e-03 1.141763497825449983e-03 -4.170570144327711420e-02 7.316040319398453683e-18 2.370455370381768095e-02 2.569133019962595299e-02 2.500568749773575405e-02 2.263991600674476051e-02 -1.184993155543183872e-02 -2.445523812836443347e-03 -2.694261437049279978e-02 5.332907991139520382e-02 4.811086411399921603e-02 1.563823155791453298e-02 2.790844208343937899e-02 2.853209191761654426e-02 4.833780471526642752e-02 6.675429818423826191e-03 1.771240319768287858e-02 1.848625481594884032e-02 3.184560923529714677e-02 -2.856871146882112569e-04 -2.333937700768229026e-02 -3.569164203358202170e-02 6.390288948297456750e-01 1.266575642428232340e-01 -2.808049692122169261e-02 -1.888068856130782388e-06 2.889967489329905298e-02 -1.866230215796463310e-02 1.415649287111117247e-02 -1.362459479370241137e-02 -2.114465121943798209e-03 1.872699100565628763e-03 7.369165293079486465e-03 4.523126899741456569e-02 -9.730304392804121211e-03 1.492225927847179792e-02 5.331139314691238362e-02 2.623406028682604796e-04 -3.316355784872230400e-02 -1.832356973271198619e-02 -6.397192879192553638e-02 5.255538891874594887e-02 2.507084755740220106e-02 -2.798456228544350361e-18 2.175028819547146497e-02 3.977495098030941166e-03 3.589463522036841214e-02 -5.760016798743197719e-02 1.249739903852966141e-02 -9.787108754395151475e-03 -2.723722179539720705e-05 7.941151992454429862e-03 2.350091444328419529e-02 -6.093523583602067329e-05 -1.727030316722209455e-02 1.159591749647403275e-02 4.879189996412344743e-02 1.946748472923692927e-03 2.094536201963336300e-02 5.807044987064314406e-02 -1.939043175312016056e-18 -5.217143083407046501e-03 8.507015625270689108e-04 -1.028323396214956588e-02 1.054695499783820690e-02 -2.389762229764927706e-03 -3.391882900869549783e-04 -4.486843972109107528e-03 5.933630354423435199e-02 1.612171970976290780e-01 1.548257939961562510e-02 -7.353389084896301900e-03 1.217528094563197390e-02 5.722799509800492387e-04 -3.613662787523773222e-18 -2.642658674204626315e-02 5.060618485185491799e-17 -3.419674852635794617e-18 2.530447669367657074e-18 2.555174870043672164e-02 1.061275502203875946e-02 3.862451463931171336e-02 7.325600366579421152e-03 2.013981511552854486e-01 4.732212136191755320e-02 4.623264849148876060e-02 6.175642836720236784e-02 4.474555452672424743e-02 2.803863418320923229e-02 1.056026869105084753e-01 5.872802361201275023e-02 1.910073528984937777e-02 1.074764268252399561e-02 1.184711336474093997e-02 1.587409350762722382e-02 1.000485715667203193e-03 6.503703792493395995e-03 1.961258549011681840e-01 -1.443793394367161309e-01 -1.823025990393996043e-01 1.045327856766707642e-01 5.507473387523572836e-03 1.480595572663642202e-02 2.028780055171794383e-02 5.762788835394174608e-02 1.046706683363417267e-02 3.371288894012914455e-02 1.539531555049316637e-02 2.165339306246339204e-02 4.139001497481380187e-02 7.492941556554208460e-02 1.686322749195228593e-02 1.156743814765023512e-02 1.653567123195049315e-03 1.898889360918972619e-02 5.847717473435855295e-03 8.408738919553555499e-03 1.898956878406828053e-04 7.606928803129407377e-03 1.225327727407625900e-03 1.868642118709780506e-03 1.014646150035795197e-02 2.513580460578612036e-02 1.320925900637618007e-02 3.731615371561929073e-02 5.575182714132529728e-03 5.487073979786302180e-01 7.739931962251234143e-01 4.331702412577994044e-01 1.688489229356955959e-02 3.259469916883881574e-02 1.102698138300356283e-02 1.512300218289125593e-02 2.142466098524384993e-02 1.797345739187808725e-02 5.042887899532645657e-03 1.581866704324006237e-02 9.984981445350100959e-03 3.078165695929663964e-02 2.100075204481659796e-01 2.613478691146249866e-01 5.985215395679627237e-01 -4.032604983426419376e-01 -7.476752633474172960e-01
1.269642803605502135e-01 -6.027374635618883147e-02 8.206849866666429683e-02 3.629340749690294909e-02 4.824349100906625415e-02 3.608136614715547197e-02 2.171651761524490132e-01 2.715062515573927679e-02 2.709460617353929429e-02 1.482786460912945831e-02 3.806869759308718904e-02 2.015635230755113971e-02 1.326091626979555155e-02 3.121416208737234216e-02 4.058461944090260148e-02 1.114068237596168853e-01 1.474059821912445667e-01 4.894374963327748074e-02 3.904120562014639501e-02 3.719099143457228057e-02 8.584724509589614083e-02 3.709403574704110002e-02 3.089287495883399209e-02 3.393884956158942285e-04 -4.591087034251128615e-03 3.562485473045611667e-02 1.850792447730650880e-01 3.567856728162813185e-02 6.868125153240753344e-02 1.343922529005885796e-02 1.095346263676521756e-02 1.536548780555775174e-03 7.962092487440479080e-04 4.598526290534091926e-03 1.715045607736165757e-03 7.320610598030646669e-03 -2.082874343805442285e-02 5.569953838582256470e-03 9.577510379987889261e-03 1.797868906314808654e-02 -6.075394391694008628e-04 -7.380116642411659392e-03 -3.690283273509028378e-02 3.438864104678793221e-03 9.245516846397714567e-03 3.350628819329092012e-02 5.523105039213139361e-02 1.010229058113464549e-02 7.388262993437473385e-03 4.164061532533460536e-02 3.254707671139027425e-02 9.029150938959566025e-03 2.068428920322796377e-02 1.347106780734363615e-02 2.179932979779056548e-02 1.409502448268020944e-02 -2.848922421793925741e-01 -1.623644433242639185e-01 -2.014019915844947606e-01 8.185747388879190123e-02 -7.424046286824884100e-02 5.429223213972931769e-03 -5.297316704726702172e-03 -1.713681917076404979e-02 1.790096864935657173e-02 -2.088943108143832134e-02 -2.286621422824652861e-03 2.197333433540099156e-04 2.007492311143989697e-02 2.102469396616085933e-03 -2.226796414188460076e-03 7.392076458690295671e-02 -1.129499831953724442e-02 -8.942049193932530282e-03 1.002176112988814594e-02 2.727214431900154903e-02 -1.360423767122515178e-02 -4.100823079372754865e-03 -1.869124679622291172e-03 -7.389482463791735220e-03 -9.834348210331102835e-07 1.821458458886907050e-01 -2.232676741022534411e-03 4.034106473716084565e-03 1.805705342767440977e-02 7.585724540791610737e-02 9.206700145398014068e-02 -1.910255536544475325e-03 5.802739278717462118e-02 2.108436235653101537e-02 -1.127724447108590094e-04 5.318227669862619911e-02 1.292927600291315648e-01 2.594606342402885207e-02 1.733225274270421348e-02 -2.465048749732768346e-17 4.707354482080633451e-02 1.963189476352252427e-13 4.691294499973356594e-04 -1.078397191281298545e-01 -1.300639441241385579e-03 -3.112312953288259507e-02 -2.877362484001526680e-02 2.566648602476913690e-02 1.942502988310979006e-02 -1.368994898001606461e-17 2.204950545932772912e-02 -5.518693375152807987e-03 -2.293374703809638107e-03 2.225530148287963722e-02 -7.081014507832850454e-18 -6.468168760271226017e-03 8.533532115164612145e-03 -5.391515847978406613e-04 1.352752326947086366e-02 -2.571233120982754222e-02 1.054654309741599044e-02 5.866645582937604630e-03 -1.539937690573668083e-01 2.326757491324841498e-01 6.112622523476270997e-01 3.456603380917528390e-01 4.998991548005164709e-02 3.020376353230979846e-02 1.866078023715498843e-02 3.044288506248720230e-02 6.996781958087722186e-02 1.479538818080688468e-02 2.051431165127115541e-02 1.700973812113129244e-02 3.780207741351207518e-02 2.990781903409777537e-02 2.203120026391602176e-02 1.751580873078736542e-02 2.885756233651879873e-02 5.241743713782064107e-02 1.099702573226868929e-01 5.576607009175792856e-02 3.492729835010240785e-02 3.490081459366975447e-02 5.700629115267726033e-02 1.416780570730057781e-02 3.970847742632343386e-02 5.858382834540485362e-01 8.125333290138585518e-01 4.956829153471186689e-01 2.050285436927902494e-03 8.442142872569598959e-04 1.388236056173056468e-02 6.866448946090021252e-03 1.639627658580663713e-02 1.372041378198734715e-02 1.258191070115087518e-02 2.720424545566475609e-03 1.459752112163308427e-02 6.629662295776159349e-03 4.556054843503879077e-03 6.110825029118464868e-03 1.561780343757016888e-02 8.576503755302045812e-03 5.360055871224655988e-01 1.173669052585827988e+00 8.028060064399027418e-01 -2.696769920352325850e-02 3.434720093704018290e-02 2.724889417770475636e-02 6.009665045717939302e-02 2.487719619191808285e-02 2.087546783323097205e-02 1.913389963368496149e-02 2.240884963819743045e-02 -5.274482857072958220e-03 9.464490902949646359e-03 1.340804552268969399e-02 4.145260632993284788e-02 5.236672020380059994e-03 1.552006117907232807e-01 5.752862259501263509e-01 4.107583530430699015e-01 7.711728656748975119e-01
1.527138720615754328e-02 2.409473342968946122e-02 8.318940170527691635e-02 3.775647801097031103e-02 3.706825235485551595e-02 4.626832590317753180e-02 1.000773157877303532e-01 4.348535558869092571e-02 4.805594515832735392e-02 3.634897533257564622e-02 6.325115733574379617e-02 2.563645550072198367e-02 2.175682902316398992e-02 9.638394562600778245e-03 3.049150421772789735e-02 1.146317840127908903e-02 -7.080678467264521492e-02 8.321024014435447411e-03 9.493020439057406334e-03 1.560947925404073365e-02 8.882254398090384806e-02 2.345871366739215452e-02 2.252021026369598403e-02 1.195457327302966444e-02 9.085959348074044636e-03 2.268497369683187409e-02 6.361355126696820184e-02 1.603996224669751447e-02 7.965581767773172801e-18 3.262421546018252037e-03 1.308133870783418197e-03 3.876660389774797162e-03 1.528265608732645676e-03 3.441237554510720963e-03 3.357618137321745702e-03 1.063725276080962085e-03 -3.540645316861816549e-02 3.908267419785688515e-04 1.542311270422161949e-02 1.987428644967583966e-02 2.332828891107336169e-02 1.152437728942646081e-02 -8.362583130532586298e-03 -1.332625592582491539e-02 -5.734548144752727589e-02 6.250546591794835238e-02 -5.807538117165311070e-02 -1.067415859045364561e-02 -7.530779869602441737e-03 6.629902688323725812e-02 5.239033888601268840e-02 2.091359519429232192e-02 2.862119167404946277e-02 1.865445983489577228e-02 4.389867479545261908e-02 7.457422265033068765e-02 6.417271264449593560e-02 7.160317447970750149e-02 7.184610790006203118e-02 1.344889751691897006e-01 -1.704847285927192299e-02 -4.634564882206220274e-03 3.854010147621209081e-02 -4.584580644041011052e-03 -1.062996798005596405e-02 -3.275196625126134548e-02 -6.552387943343770732e-04 3.150259264408998561e-02 1.365980056451494666e-03 2.332606982123619065e-03 -3.048306932491870805e-03 2.559037929852385831e-02 3.599739115353733920e-02 3.256886918847286955e-03 -1.532995384329909234e-02 -1.802158681543246377e-02 -4.639184076956238967e-02 3.687573168636791576e-02 -8.347178349824446069e-05 6.093016620305894901e-03 2.375480301262992977e-02 1.108507190762853109e-03 2.435402490378606871e-02 -1.692063686826853625e-02 1.096533926545328365e-02 -7.258054343762049979e-18 2.721213777633480976e-02 6.873953424918284941e-03 4.443197482972095769e-02 -4.995440827266225628e-05 -1.624113084478273628e-02 4.071317343347211023e-18 2.340918610453494517e-02 9.747396860454067558e-03 3.009022821998262945e-02 3.325374972546234903e-02 2.382635044253442885e-02 4.937318872525934101e-04 2.061352361910927244e-03 -5.233541386011658809e-03 1.186910377160961191e-02 -2.602226711828003355e-18 -4.911644148347638766e-02 -1.808588700069682642e-02 -8.290860127002936144e-16 3.356817079058359039e-03 6.286789421731896199e-02 -4.778718725005884696e-03 -2.245300394434775030e-04 -2.017925246165377419e-03 -2.683874452863638238e-02 -7.645520977908709138e-03 1.565973445589917759e-02 -5.818305805014355701e-03 2.321309789382807257e-02 -3.018785946723709368e-02 -3.433150429472769352e-04 1.008477932303392880e-04 -1.755217677245726185e-03 6.858098261201109325e-03 4.790793841927596525e-02 5.446634359044339596e-02 6.300297429540331617e-02 5.006506900542392197e-02 1.555113948430422895e-02 4.224516471401786738e-02 6.829233155000348499e-02 1.480043879244589619e-02 1.632583338549021060e-02 1.204078346530228043e-02 2.193373376289411980e-02 6.957563527799500228e-03 1.053735179881527990e-02 1.808412051076614679e-01 -1.500293006046468092e-01 -2.101494659986669622e-01 9.997649434233694954e-02 8.006266918281197340e-03 1.960917869789670018e-02 2.725941490258183136e-02 5.171622427972108471e-02 1.624640152421743741e-02 3.601349674627506159e-02 3.526642828225829557e-02 2.781742298556219573e-02 5.902453111771199107e-02 9.408448494617303870e-02 1.899342139850715774e-02 2.165207537664103760e-02 8.531756857373668087e-04 1.874885472509153772e-02 1.806210450602842993e-02 4.124896113508246497e-03 3.410121480886271566e-04 1.133712103131596927e-02 4.780794162689039643e-04 2.050484107129141049e-03 8.954258477147124251e-02 2.476928447105640127e-02 1.489756572136151543e-02 3.914299070454596574e-02 1.368224876081224235e-02 3.378645897391608505e-02 4.978583541955474562e-02 3.820630679687211928e-02 2.381717848225189602e-02 2.041925150120327617e-02 3.008987798674413877e-02 2.240277957373659062e-02 8.725804883759742669e-03 2.134324040422819202e-02 8.691180546766461706e-03 1.446372729228797421e-02 3.607764138762238615e-02 5.269263063675860836e-02 1.608460212879643525e-02 1.133552576362649167e-01 8.509106145757056039e-02 8.389722840090263478e-02 5.294032189030667412e-02
1.312882819254727607e-01 -6.451345399612193332e-02 6.061781127998069202e-02 3.470146714166272295e-02 1.314499587489134820e-02 2.833847012333433682e-02 5.283866942877132183e-02 1.669528671887618443e-02 1.737948534193405753e-02 1.955494759346096401e-02 9.565513811071653383e-02 3.535203963883550016e-02 1.499235110344148206e-02 1.297814247278769248e-02 3.321535529685588212e-02 3.547607246209955045e-02 -1.456744388827688752e-01 -1.102981658927696951e-01 -1.264773937267316961e-01 4.320475495553422612e-02 4.025303265985218093e-02 4.491369346203874860e-03 1.064890389127644450e-02 1.896998680086748462e-02 4.591147315446367921e-02 3.888325442691236561e-02 8.970982870463574399e-02 1.906911772332816127e-02 5.775451787836009127e-03 3.037143303174463143e-04 4.035457621383378018e-03 1.104799271418152335e-03 -1.803225619406713083e-03 1.251670637978029299e-02 2.138761837977649904e-02 -1.321837428889918731e-01 -2.824854325543542655e-01 -8.275229735702054057e-02 3.500423684948825165e-02 7.187645467814882637e-03 2.784968506185061740e-02 5.808912320036636878e-02 6.763597514075379846e-02 4.040586314059491785e-02 2.130833074897403590e-02 2.815129613126057459e-02 -4.040607844301401935e-02 -1.054202812853166680e-16 -1.157666055807441731e-02 4.478123244681680298e-02 7.325538765631474381e-02 2.823532639731006141e-02 1.628234387739179237e-02 2.033756166094535800e-02 2.924895416347449151e-02 4.334211212911646266e-02 8.429617198112057674e-02 1.054114005037625479e-01 7.423351289253496410e-02 4.856386043162085053e-02 -1.197765398074244064e-01 4.008228228879277127e-02 3.081354097239534517e-02 8.719515713045508654e-02 3.907881985604385461e-02 -2.666122410620971883e-02 2.508870083071580367e-02 2.865143348884997831e-02 4.200092503022915397e-02 2.341883186300908464e-04 2.382948636829737132e-03 1.191500843894146774e-02 1.533591602029932624e-02 -2.593617670849171844e-02 -3.523342180141816188e-02 -1.645080478129409829e-02 5.809377787589465551e-04 -1.648743091892773127e-01 -5.275369125558831279e-02 2.106835146680227991e-02 1.595097128951710602e-02 3.691805886155813399e-02 2.195749050726909096e-02 -5.477128827941356477e-03 7.436607644032785895e-03 -2.022800522837610391e-03 -7.793244552521869214e-18 -3.150190320357385779e-03 -2.104218287671637527e-03 1.003324321189540819e-02 -8.704676261869348508e-02 2.190242311815700921e-02 -5.095887493206217636e-17 -7.163716411918720495e-02 -1.748935660076405413e-02 -1.909291515824056151e-02 -6.590508154186883877e-03 -9.789627948401489685e-03 5.156714182765016405e-03 -1.956332518432342057e-02 1.435562732990024355e-02 -1.447018981309892149e-01 -1.342884353542272459e-02 -1.528789732130225601e-03 -5.824680158096133542e-02 4.231287437832505910e-02 -7.886539874371897938e-04 -1.088844539028407422e-02 -1.393165800321714094e-03 2.666835970447771834e-03 -1.746457621164933249e-02 -4.103929207979259549e-04 1.588381791348795992e-03 -5.702544843072252997e-03 2.050414492643961792e-02 -2.475671070058779386e-02 -2.374786907158160047e-03 6.582642688701503039e-04 3.313218047041383008e-02 1.452515569074639768e-03 5.838354612746257111e-01 4.137492084063107978e-01 1.055049931481213087e-02 1.192637187423960937e-02 2.167036497417606372e-02 7.462327015419560616e-02 5.976385275913529554e-02 1.258985980993662483e-02 2.380968918913214591e-02 2.727747269708101843e-02 3.914391874339238214e-02 3.038743595054065397e-02 2.008561087910350207e-02 1.982221806666708760e-02 3.302701419507104302e-02 9.951565002756602996e-03 4.866468688068871784e-01 9.146563396197050499e-01 4.170335467823631515e-01 2.972376898610460012e-03 1.948884841830251374e-02 6.764290305103151137e-03 4.176598232613508432e-02 3.489425503279096341e-02 2.748514913831960615e-02 2.296631452304553553e-02 5.873659908616774006e-02 2.186239427517919481e-03 2.865361899807462864e-02 6.737003309955674063e-03 2.833117051133127173e-02 8.300790497688287015e-03 1.413271501573188337e-02 1.535508286439163546e-02 2.337270417645278056e-02 8.878997143057968389e-01 8.323027926610592209e-01 6.502011772804039413e-01 2.635581072352856467e-03 -9.722378361662174784e-02 8.811751753805946094e-03 8.783305587874717588e-03 1.342046360577207713e-02 -6.279477658084055280e-02 6.198512964158793374e-03 -8.653914636588583287e-02 3.965199237805535071e-01 6.768465857029860588e-01 2.651834763292176267e-01 1.200880820638282331e-02 1.941648403403739218e-02 2.086998924648576087e-03 9.856994388801240334e-03 2.372060790453156195e-02 4.526797355659491900e-02 3.535475998202763925e-02 1.177069377852710541e-01 1.104146493108592447e-01 6.075731769315726588e-02 4.046009406406548409e-02
2.391876287506803089e-01 2.735090251232039429e-02 2.661210211813874391e-03 5.192775057378513567e-03 -6.493089424660948832e-04 2.726725385657075874e-03 5.018631137856150067e-03 4.172160542507272890e-03 1.098587201712429573e-02 1.773034615436085148e-02 5.911532610482835015e-02 2.294129122961148168e-02 2.519619780495758674e-02 2.429646845148199230e-02 1.004758971131444051e-01 3.825299491913961708e-03 7.501078041916491544e-03 1.727841840718438368e-02 1.556190363274997240e-02 1.277189549747851510e-02 -1.898341724267748054e-01 -2.017922564939029495e-01 -1.161656981586439985e-01 2.820902419928761340e-02 7.346801044755271737e-02 3.420899074083783714e-02 4.021379833908788476e-02 7.759494051745122190e-02 7.441283781400616359e-02 6.633668303140574651e-02 1.157259515068700811e-01 1.088985260830756113e-01 1.114152981230190631e-01 7.601145093758789051e-02 7.014398154419516918e-02 3.883894001193582968e-02 2.540250196023264434e-02 8.263536731813193870e-02 -1.565897868571299728e-02 1.086343219363721836e-02 2.002501139521907850e-02 1.308444519871181100e-02 8.821505241382266924e-03 5.605333942651167731e-03 2.478507738782260647e-02 2.556777330669905007e-02 5.016372783482968278e-02 1.643554889302394420e-02 1.650358806819850893e-02 2.124810321546815803e-02 4.519807730705632104e-02 1.398262288767011547e-03 1.359136355604679408e-02 6.865599882775097820e-03 1.914387384347507673e-02 -3.339176974848140540e-05 -1.934221561478883730e-02 -3.687219240726829356e-02 7.427713755813194485e-01 1.142577841067599492e-01 -1.587482741428443700e-01 -2.322846324397034455e-04 1.590738615648747004e-03 4.209560258188213555e-02 3.598236248600830689e-02 -1.124452467362916513e-01 2.840597901040813056e-02 1.687008774545954715e-02 1.920108523510455920e-03 5.335636535975212501e-03 6.923425806083014330e-18 1.155812207618416888e-02 2.826020246856114637e-02 1.272583276963649155e-03 2.977157680196520443e-04 2.288759847974831604e-02 6.385994405277714038e-03 7.281650151175317678e-03 3.250348688859212853e-02 1.744151589331082337e-17 -5.763824044257695456e-03 -1.465163344985302984e-01 -5.168874514303556378e-02 3.134540807906070758e-02 2.583617837523746741e-02 6.962729962198497278e-04 -2.282668198371797405e-17 -1.252162786352693791e-02 8.251952602787019850e-03 -3.853441247703660519e-03 4.959521615344021639e-02 -2.513294223395844476e-02 -2.163423094035662325e-03 -9.171111990163769573e-03 -3.257321711317642192e-16 3.201279962006317592e-02 -5.045843658738376897e-02 1.051551969168386391e-02 3.262539749798302330e-02 3.862872097920379277e-02 3.363615978593427567e-03 9.519054013583448659e-02 -2.352384866700096486e-02 -1.383190652013987706e-02 3.903062400030329199e-02 -7.494458207007054576e-03 -4.949069907745491138e-03 -1.290989096170967222e-03 -2.598261337401300622e-02 -4.064236225387039586e-04 -5.109961468806450786e-18 -6.223466743200665487e-03 -3.248513313692138853e-03 -1.357597344145563753e-02 5.582530807195008361e-03 5.437025777806343829e-02 2.095860409291478571e-02 5.301570205270173669e-02 3.753090820672754738e-03 1.982808022682635563e-01 5.667043866249721340e-01 4.000293348336940724e-01 1.452017423691705222e-02 7.689532386525671339e-17 1.502968829634679197e-02 2.253766555055043523e-03 1.466207701051856077e-02 3.819578256518131511e-03 1.767158646999168747e-02 1.339993396187320711e-02 3.941155077159431280e-02 4.253052423104102323e-03 4.381904243548103567e-02 1.374636606949772299e-02 3.753532767519596886e-02 2.077076616910261339e-02 2.311816500388882284e-02 1.274027553481335735e-02 3.152559368054899669e-02 1.651910865457258473e-02 4.937921244817921052e-01 8.417753559080881631e-01 4.101113971406867842e-01 3.286739373945541092e-03 5.441764972296331097e-03 6.758801355400041039e-03 5.277353898592675868e-02 4.894195589303744487e-02 2.210970707453499898e-02 4.692586274795693124e-02 7.544514457013033748e-02 8.298839941804590781e-02 7.596565149816750240e-02 7.430007358863842393e-02 4.072583023494479038e-02 1.609875308438048561e-02 1.099855267133719415e-02 5.563308063796204017e-01 7.043348564501893394e-01 6.493228691837942268e-01 8.907582419623737494e-03 2.604238961839622918e-02 3.760683578874345068e-02 1.573118146595611083e-02 3.813183475404178374e-02 4.879638129808341190e-02 4.928093598441439516e-02 1.575447807426260532e-02 2.080185851190610030e-02 1.890675543003400572e-02 2.839351660377410769e-02 -3.934257220739664426e-03 1.432018699923794103e-02 6.230364853472110213e-03 3.823779604576194729e-02 2.373671771687126675e-01 3.010981418412408406e-01 5.550104339647436991e-01 -3.776974857885575521e-01 -1.237723139330315414e+00
5.221091182507695688e-02 7.280222351382414914e-02 1.303975991567831449e-01 6.078728176582702536e-02 3.542548395242146181e-02 6.290059841742030866e-02 1.496809048749527804e-01 5.891661129417869597e-02 1.049399464199442661e-01 8.063350143104291989e-02 -1.522738403141352115e-02 -9.250499463089507934e-02 -6.363658928161965156e-02 4.399819827798640698e-02 5.155173520026384154e-02 1.475738792852873682e-01 8.450459448307534593e-02 2.558406812887880147e-02 6.130369019247879775e-02 7.933504108117458986e-02 1.263170005756872605e-01 7.601051348078934067e-02 5.862445863963487019e-02 1.689013851731734628e-01 -5.950371644640673613e-03 5.091287365163581213e-02 8.357381848289008319e-02 2.387905717906683642e-02 1.174037522499831419e-02 3.245151539828867310e-03 2.106883455399945117e-03 1.010540665251429638e-04 6.335409694630198809e-04 2.698249429149835029e-04 1.816751224267598372e-02 -4.979354235532865713e-01 -3.443882776431745341e-01 -7.680614641855061653e-02 4.976374646504146820e-02 3.156386597854698300e-02 2.697762426655575529e-02 2.102916330489579613e-02 1.624942257560624964e-02 5.003138217081351882e-03 1.269025585817216278e-02 3.059927984031005077e-02 5.172633793220862003e-02 1.644581196260328643e-02 1.494380781467766900e-02 1.743237186319323076e-02 2.005441368163060969e-02 4.112977814560304490e-04 1.667180528839583287e-02 1.055811578432706670e-02 2.417262382545248139e-02 -1.777248144232982871e-02 -2.808940576554527085e-02 -5.609847208265978519e-02 6.936391138127062650e-01 1.121303669180582496e-01 -5.620460918614816702e-05 -2.722402447673303791e-02 2.512332858570024210e-02 3.621217562365136589e-03 -1.668163626063233658e-03 -2.639885485888490702e-02 3.786190588112154504e-02 1.843540366970812874e-02 4.153617602511922718e-02 9.892908821151817286e-03 1.037637803665625649e-02 -1.233261147741932006e-03 3.108562414837631044e-02 1.619124492045419944e-02 -1.640157192236095865e-02 -2.373120816114059062e-02 -1.845844521232290272e-02 5.395265344318831043e-03 -1.284453456517833762e-02 -6.491087545840795725e-02 5.533912023353515508e-02 -4.328237060298392030e-02 7.382673475166126598e-02 4.870399619432310562e-02 7.372763235893801947e-02 8.675630819362976920e-03 -1.819282543040191130e-03 4.170870781980881223e-03 -4.774354713540070707e-18 1.192895235444072777e-01 5.076851730853839295e-18 -2.803531162553217807e-18 -6.771618850252017962e-05 1.636300272328799428e-05 -9.915448217433255312e-04 -7.026209024993480649e-03 2.776873232146452684e-03 -1.022145963610060146e-02 5.578633646697012506e-04 -7.276574382660375455e-04 9.370932304533141979e-03 -2.529962351502574246e-17 5.730464256071298498e-04 2.245816485388830303e-02 7.235199931201684656e-03 -1.755280891187740652e-02 -9.173862523614625258e-03 -1.174755243322465371e-02 -2.578244814939385007e-03 1.848483612912690116e-02 -3.114223635759999677e-08 -3.674230366301321099e-04 7.513661443929735458e-03 -2.822325158400305119e-04 2.911025977856131902e-02 4.888889635667853506e-02 6.537891764366160312e-03 7.707252340405053304e-02 -4.437122208230392939e-03 1.963069777914084602e-01 7.884357210136797023e-02 7.525710982154670825e-02 8.897145000564371520e-02 7.728750459372711601e-02 3.041213462845548249e-02 8.531755729861861814e-02 7.504890768549529745e-02 6.540831430397677122e-04 1.682424264975164452e-02 1.640846823671927813e-03 4.344206819409007769e-01 7.230356516094681441e-01 4.114832078787620806e-01 1.931274559683099010e-02 4.011718045465485077e-02 5.259631099406106292e-02 9.454684593628337463e-02 4.050261693402108093e-02 1.937318323816359170e-02 3.189169156849054609e-02 3.741223358434737911e-02 8.813017441316017830e-03 5.536586365413975752e-01 8.218107981209699098e-01 5.493602831718090718e-01 6.695700060259273875e-03 4.072903402034595138e-02 1.327692864074415859e-03 1.418494780149113198e-02 1.581999031750959753e-02 1.480004304412260545e-02 7.941381297019809316e-04 1.580247609019286639e-02 1.230428057506656112e-02 2.102092472273225585e-02 8.185076701830067414e-01 8.067371896155585143e-01 7.782855538726712874e-01 1.252111848407867073e-02 2.144828676155007691e-02 4.597942601253120903e-02 4.072999247211615131e-02 3.583751941907331595e-02 1.398622472099288368e-02 3.168033607479030234e-02 3.449950015473988302e-02 5.681068463947772096e-02 2.807545027370493204e-02 2.734959404862054647e-02 1.614897415803724645e-02 2.154817730896350231e-02 7.195449085126793230e-03 1.923577401633774775e-02 1.802258095125067530e-02 4.527950631755143368e-02 1.999067109112729890e-01 2.662491015352343449e-01 4.695595347737325231e-01 -3.855022976142288282e-01 -1.165146084802210824e+00

In [ ]:
if run_experiment:
    projections_train = np.array(projections_train).reshape(
        len(projections_train), -1
    )
    projections_test = np.array(projections_test).reshape(
        len(projections_test), -1
    )
else:
    projections_train = np.loadtxt("projections_train.txt")
    projections_test = np.loadtxt("projections_test.txt")

### Support Vector Machine (SVM)

We can now run a classical SVM on this precomputed kernel, and use the kernel between test and training sets for prediction.

In [10]:
# Range of 'C' and 'gamma' values as SVC hyperparameters
C_range = [0.001, 0.005, 0.007]
C_range.extend([x * 0.01 for x in range(1, 11)])
C_range.extend([x * 0.25 for x in range(1, 60)])
C_range.extend(
    [
        20,
        50,
        100,
        200,
        500,
        700,
        1000,
        1100,
        1200,
        1300,
        1400,
        1500,
        1700,
        2000,
    ]
)

gamma_range = ["auto", "scale", 0.001, 0.005, 0.007]
gamma_range.extend([x * 0.01 for x in range(1, 11)])
gamma_range.extend([x * 0.25 for x in range(1, 60)])
gamma_range.extend([20, 50, 100])

param_grid = dict(C=C_range, gamma=gamma_range)

# Support vector classifier
svc = SVC(kernel="rbf")

# Define the cross validation
cv = StratifiedKFold(n_splits=10)

# Grid search for hyperparameter tuning (q: quantum)
grid_search_q = GridSearchCV(
    svc, param_grid, cv=cv, verbose=1, n_jobs=-1, scoring="f1_weighted"
)
grid_search_q.fit(projections_train, train_labels)

# Best model with best parameters
best_svc_q = grid_search_q.best_estimator_
print(
    f"The best parameters are {grid_search_q.best_params_} with a score of {grid_search_q.best_score_:.4f}"
)

# Test accuracy
accuracy_q = best_svc_q.score(projections_test, test_labels)
print(f"Test accuracy with best model: {accuracy_q:.4f}")

Fitting 10 folds for each of 6622 candidates, totalling 66220 fits
The best parameters are {'C': 8.5, 'gamma': 0.01} with a score of 0.6980
Test accuracy with best model: 0.8108


### Classical benchmarking
We can run a classical SVM with radial basis function as the kernel without doing a quantum projection. This result is our classical benchmark.

In [11]:
# Support vector classifier
svc = SVC(kernel="rbf")

# Grid search for hyperparameter tuning (c: classical)
grid_search_c = GridSearchCV(
    svc, param_grid, cv=cv, verbose=1, n_jobs=-1, scoring="f1_weighted"
)
grid_search_c.fit(train_data, train_labels)

# Best model with best parameters
best_svc_c = grid_search_c.best_estimator_
print(
    f"The best parameters are {grid_search_c.best_params_} with a score of {grid_search_c.best_score_:.4f}"
)

# Test accuracy
accuracy_c = best_svc_c.score(test_data, test_labels)
print(f"Test accuracy with best model: {accuracy_c:.4f}")

Fitting 10 folds for each of 6622 candidates, totalling 66220 fits
The best parameters are {'C': 10.75, 'gamma': 0.04} with a score of 0.7830
Test accuracy with best model: 0.7432


## Appendix: Verifying the data set's potential quantum advantage in learning tasks

Not all datasets offer potential advantage from the use of PQKs. There are some theoretical bounds that one can use as a preliminary test to see if a particular dataset can benefit from PQKs. To quantify this, authors of [[2]](#references) define quantities referred to as classical and quantum model complexities and geometric separation of the classical and quantum models. To expect a potential quantum advantage from PQKs, the geometric separation between the classical and quantum-projected kernels should be approximately on the order of $\sqrt{N}$, where $N$ is the number of training samples. If this condition is satisfied, we move on to checking the model complexities. If the classical model complexity is on the order of $N$ while the quantum-projected model complexity is substantially smaller than $N$, we can expect potential advantage from the PQK.

Geometric separation is defined as follows (F19 in [[2]](#references)):
$$g_{cq} = g(K^c \Vert K^q) = \sqrt{\Vert \sqrt{K^q} \sqrt{K^c} (K^c + \lambda I)^{-2} \sqrt{K^c} \sqrt{K^q}\Vert_{\infty}}$$

In [12]:
# Gamma values used in best models above
gamma_c = grid_search_c.best_params_["gamma"]
gamma_q = grid_search_q.best_params_["gamma"]

# Regularization parameter used in the best classical model above
C_c = grid_search_c.best_params_["C"]
l_c = 1 / C_c

# Classical and quantum kernels used above
K_c = rbf_kernel(train_data, train_data, gamma=gamma_c)
K_q = rbf_kernel(projections_train, projections_train, gamma=gamma_q)

# Intermediate matrices in the equation
K_c_sqrt = sqrtm(K_c)
K_q_sqrt = sqrtm(K_q)
K_c_inv = inv(K_c + l_c * np.eye(K_c.shape[0]))
K_multiplication = (
    K_q_sqrt @ K_c_sqrt @ K_c_inv @ K_c_inv @ K_c_sqrt @ K_q_sqrt
)

# Geometric separation
norm = np.linalg.norm(K_multiplication, ord=np.inf)
g_cq = np.sqrt(norm)
print(
    f"Geometric separation between classical and quantum kernels is {g_cq:.4f}"
)

print(np.sqrt(len(train_data)))

Geometric separation between classical and quantum kernels is 1.5440
13.114877048604


Model complexity is defined as follows (M1 in [[2]](#references)):
$$ s_{K, \lambda}(N) = \sqrt{\frac{\lambda^2 \sum_{i=1}^N \sum_{j=1}^N (K+\lambda I)^{-2}_{ij} y_i y_j}{N}} + \sqrt{\frac{\sum_{i=1}^N \sum_{j=1}^N ((K+\lambda I)^{-1}K(K+\lambda I)^{-1})_{ij} y_i y_j}{N}}$$

In [13]:
# Model complexity of the classical kernel

# Number of training data
N = len(train_data)

# Predicted labels
pred_labels = best_svc_c.predict(train_data)
pred_matrix = np.outer(pred_labels, pred_labels)

# Intermediate terms
K_c_inv = inv(K_c + l_c * np.eye(K_c.shape[0]))

# First term
first_sum = np.sum((K_c_inv @ K_c_inv) * pred_matrix)
first_term = l_c * np.sqrt(first_sum / N)

# Second term
second_sum = np.sum((K_c_inv @ K_c @ K_c_inv) * pred_matrix)
second_term = np.sqrt(second_sum / N)

# Model complexity
s_c = first_term + second_term
print(f"Classical model complexity is {s_c:.4f}")

Classical model complexity is 1.3578


In [14]:
# Model complexity of the projected quantum kernel

# Number of training data
N = len(projections_train)

# Predicted labels
pred_labels = best_svc_q.predict(projections_train)
pred_matrix = np.outer(pred_labels, pred_labels)

# Regularization parameter used in the best classical model above
C_q = grid_search_q.best_params_["C"]
l_q = 1 / C_q

# Intermediate terms
K_q_inv = inv(K_q + l_q * np.eye(K_q.shape[0]))

# First term
first_sum = np.sum((K_q_inv @ K_q_inv) * pred_matrix)
first_term = l_q * np.sqrt(first_sum / N)

# Second term
second_sum = np.sum((K_q_inv @ K_q @ K_q_inv) * pred_matrix)
second_term = np.sqrt(second_sum / N)

# Model complexity
s_q = first_term + second_term
print(f"Quantum model complexity is {s_q:.4f}")

Quantum model complexity is 1.5806


## References

1. Utro, Filippo, et al. "[Enhanced Prediction of CAR T-Cell Cytotoxicity with Quantum-Kernel Methods](https://arxiv.org/abs/2507.22710)." arXiv preprint arXiv:2507.22710 (2025).
2. Huang, Hsin-Yuan, et al. "[Power of data in quantum machine learning](https://www.nature.com/articles/s41467-021-22539-9)." Nature communications 12.1 (2021): 2631.
3. Daniels, Kyle G., et al. "[Decoding CAR T cell phenotype using combinatorial signaling motif libraries and machine learning](https://www.science.org/doi/full/10.1126/science.abq0225)." Science 378.6625 (2022): 1194-1200.